In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style 
import mpl_toolkits
import re
from copy import deepcopy
from collections import defaultdict


plt.rcParams['font.family'] = 'Times New Roman'
plot_dir = "./plots/"
#style.use('tableau-colorblind10')

class DataBase():
    def __init__(self):
        self.database = {}

    def get_data(self, model: str, dataset: str, initialization: str, optimizer: str)->dict:
        key = f"{model}.{dataset}.{initialization}.{optimizer}"
        if key not in self.database:
            self.database[key] = {"model": model, "dataset": dataset, "initialization": initialization, "optimizer": optimizer}
        return self.database[key]
    
database = DataBase()    

In [2]:
import mpl_toolkits.axes_grid1

def average_common_keys_with_arrays(dict_list):

    value_sums = defaultdict(lambda: None)
    value_counts = defaultdict(int)

    for d in dict_list:
        for key, value in d.items():
            if value_sums[key] is None:
                # Initialize with a copy of the list
                if isinstance(value, str):
                    value_sums[key] = value
                else:
                    value_sums[key] = value[:]
            else:
                if not isinstance(value, str):
                    # Element-wise addition
                    value_sums[key] = [x + y for x, y in zip(value_sums[key], value)]
            if not isinstance(value, str):
                value_counts[key] += 1

    result = {}
    for key, value in value_sums.items():
        count = value_counts[key]
        if count > 1:
            # Average each element
            result[key] = [x / count for x in value_sums[key]]
        else:
            result[key] = value_sums[key]

    return result

def merge_dicts(dict_list):
    result = {}
    first_dict = dict_list[0]
    for key, value in first_dict.items():
        experiment = [value,]
        for d in dict_list[1:]:
            if key in d:
                experiment.append(d[key])
        if len(experiment) > 1:
            experiment = average_common_keys_with_arrays(experiment)
        else:
            experiment = experiment[0]
        result[key] = experiment

    return result

def string_to_vect(input_string):
    lines = input_string.strip().split('\n')
    regnumber = re.compile(r'^\d+(?:.\d*)?$')
    matrix = defaultdict(list)
    for line in lines:
        split = re.split(';|,|:|%|/| ', line)
        for i, text in enumerate(split):
            if regnumber.match(text):
                matrix[i].append(text)

    line_i = 1
    print('data[""] = [', end="")
    for i in matrix:
        if line_i == 1:
            print('[', end="")
        else:
            print('        [', end="")
        for j in matrix[i][:-1]:
            print(j, end=", ")
        if line_i < len(matrix):
            print(matrix[i][-1], end="],\n")
        else:
            print(matrix[i][-1], end="]]\n")
        line_i += 1

def preprocess(data):
    # for pruningcriteria in ["l1", "random", "lobs", "nrelief"]:
    #     if pruningcriteria in data:
    #         base_acc_list = data[pruningcriteria][-1]
    #         for key in data:
    #             if pruningcriteria in key and "_" in key and isinstance(data[key], list):
    #                 acc_list = data[key][-1]
    #                 for i, acc in enumerate(acc_list):
    #                     base_acc = base_acc_list[i]
    #                     if base_acc > acc:
    #                         acc_list[i] = base_acc
    
    for key in data:
        if isinstance(data[key], list):
            data_list = data[key][0]
            for i, num in enumerate(data_list):
                data_list[i] = num / 100
            data[key][0] = data_list
            data_list = data[key][-1]
            for i, num in enumerate(data_list):
                data_list[i] = num / 100
            data[key][-1] = data_list
        if isinstance(data[key], float):
            data[key] = data[key] / 100

def plot_loss(model, dataset, show=False):
    fig = plt.figure(figsize=(4, 2))
    
    ax = fig.add_axes([0.2, 0.2, 0.6, 0.6])

    initialization = "Sinusoidal"
    points = ["o", "s", "X", "^"]
    alphas = [0.1+0.3*(i+1) for i in range(3)]
    for i, optimizer in enumerate(["SGD", "Adam", "AdamW"]):
        try:
            data = database.get_data(model, dataset, initialization, optimizer)["train_loss"]
            if len(data) > 200:
                data = data[:200]
            label = optimizer + " with " + initialization
            alpha = (i+1) / 3
            epochs = np.arange(len(data)) + 1
            ax.plot(epochs, data,
                label=label,
                color='C4', linestyle='-', linewidth=1, alpha=alphas[i],
                marker=points[i], markersize=2, markevery=10)
        except:
            print(f"{optimizer} fail")
        

    initialization = "Default initialization"
    for i, optimizer in enumerate(["SGD", "Adam", "AdamW"]):
        try:
            data = database.get_data(model, dataset, initialization, optimizer)["train_loss"]
            if len(data) > 200:
                data = data[:200]
            label = optimizer + " with " + initialization
            alpha = (i+1) / 3
            epochs = np.arange(len(data)) + 1
            ax.plot(epochs, data,
                label=label,
                color='C1', linestyle='-', linewidth=1, alpha=alphas[i],
                marker=points[i], markersize=2, markevery=10)
        except:
            pass

    plt.xlabel('Epoch')
    plt.ylabel('Train Loss')
    plt.title(f'{model} on {dataset} Train Loss')

    #plt.gca().xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.1)) 
    #plt.gca().xaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax=1))
    #plt.gca().yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax=1))
    #plt.xlim(0, 1)
    #plt.ylim(0, 1)
    #plt.legend(loc="center left", bbox_to_anchor=(1.04, 0.5))
    plt.savefig(f'{plot_dir}loss_{model.lower().replace("/","")}_{dataset.lower()}.pdf', 
                format='pdf') #, bbox_inches='tight')

    # Show the plot
    if show:
        plt.show()
    else:
        plt.close()

def plot_acc(model, dataset, show=False):
    fig = plt.figure(figsize=(4, 2))
    ax = fig.add_axes([0.2, 0.2, 0.6, 0.6])

    initialization = "Sinusoidal"
    points = ["o", "s", "X", "^"]
    alphas = [0.1+0.3*(i+1) for i in range(3)]
    for i, optimizer in enumerate(["SGD", "Adam", "AdamW"]):
        try:
            data = database.get_data(model, dataset, initialization, optimizer)["val_metric"]
            if len(data) > 200:
                data = data[:200]
            label = optimizer + " with " + initialization
            alpha = (i+1) / 3
            epochs = np.arange(len(data)) + 1
            acc = np.array(data)
            if acc[-1] > 1:
                acc = acc / 100
            ax.plot(epochs, acc,
                label=label,
                color='C4', linestyle='-', linewidth=1, alpha=alphas[i],
                marker=points[i], markersize=2, markevery=10)
        except:
            print(f"{optimizer} fail")

    initialization = "Default initialization"
    for i, optimizer in enumerate(["SGD", "Adam", "AdamW"]):
        try:
            data = database.get_data(model, dataset, initialization, optimizer)["val_metric"]
            if len(data) > 200:
                data = data[:200]
            label = optimizer + " with " + initialization
            alpha = (i+1) / 3
            epochs = np.arange(len(data)) + 1
            acc = np.array(data)
            if acc[-1] > 1:
                acc = acc / 100
            ax.plot(epochs, acc,
                label=label,
                color='C1', linestyle='-', linewidth=1, alpha=alphas[i],
                marker=points[i], markersize=2, markevery=10)
        except:
            pass

    plt.xlabel('Epoch')
    plt.ylabel('Validation Accuracy')
    plt.title(f'{model} on {dataset} Val. Acc.')


    #plt.gca().xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.1)) 
    #plt.gca().xaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax=1))
    plt.gca().yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax=1, decimals=0))
    #plt.gca().yaxis.update(tickformat=".0%")
    #plt.xlim(0, 1)
    #plt.ylim(0, 1)
    #plt.legend(loc="center left", bbox_to_anchor=(1.04, 0.5))
    plt.savefig(f'{plot_dir}acc_{model.lower().replace("/","")}_{dataset.lower()}.pdf', 
                format='pdf') #, bbox_inches='tight')

    # Show the plot
    if show:
        plt.show()
    else:
        plt.close()

def plot_legend():
    legendFig = plt.figure("Legend plot", figsize=(4,2))
    fig = plt.figure("Line plot")
    ax = fig.add_subplot(111)
    model = "ResNet-50"
    dataset = "CIFAR-100"

    lines = []
    labels = []

    points = ["o", "s", "X", "^"]
    alphas = [0.1+0.3*(i+1) for i in range(3)]
    for i, optimizer in enumerate(["SGD", "Adam", "AdamW"]):    
        for j, initialization in enumerate(["Sinusoidal", "Default initialization"]): #, "Sinusoidal 2"]):
            data = database.get_data(model, dataset, initialization, optimizer)["val_metric"]
            if len(data) > 200:
                data = data[:200]
            label = optimizer + " with " + initialization
            labels += [label]
            alpha = (i+1) / 3
            epochs = np.arange(len(data)) + 1
            acc = np.array(data)
            if acc[-1] > 1:
                acc = acc / 100

            if j == 0:
                line, = ax.plot(epochs, acc,
                    label=label,
                    color='C4', linestyle='-', linewidth=2, alpha=alphas[i],
                    marker=points[i], markersize=2, markevery=10)
            elif j == 1:
                line, = ax.plot(epochs, acc,
                    label=label,
                    color='C1', linestyle='-', linewidth=2, alpha=alphas[i],
                    marker=points[i], markersize=2, markevery=10)
            else:
                line, = ax.plot(epochs, acc,
                    label=label,
                    color='C3', linestyle='-', linewidth=2)

            lines += [line]
            
    
    legendFig.legend(lines, labels, ncol=3, loc='center')
    legendFig.savefig(f'{plot_dir}legend.pdf', 
                      format='pdf', bbox_inches='tight')
    
    plt.close()

def plot_acc_4inits(model, dataset, optimizer, show=False):
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_axes([0.2, 0.2, 0.6, 0.6])

    colors = ["C1", "C2", "C3", "C4"]
    initializations = ["Default initialization", "Orthogonal", "LSUV", "Sinusoidal"]
    linestyles = [('dashed', (0, (5, 5))), ('dashed', (0.5, (5, 5))), ('dashed', (1, (5, 5))), ('dashed', (1.5, (5, 5)))]
    min_acc = 1
    for i, initialization in enumerate(initializations):
        try:
            data = database.get_data(model, dataset, initialization, optimizer)["val_metric"]
            if len(data) > 200:
                data = data[:200]
            epochs = np.arange(len(data)) + 1
            acc = np.array(data)
            if acc[-1] > 1:
                acc = acc / 100
            if acc[4] < min_acc:
                min_acc = acc[4]
            ax.plot(epochs, acc,
                label=initialization,
                color=colors[i], linestyle=(i*0.25, [1,1]), linewidth=1.5)
        except:
            print(f"{initialization} fail")

    plt.xlabel('Epoch')
    plt.ylabel(f'Validation Accuracy')
    plt.title(f'{model} on {dataset}')


    #plt.gca().xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.1)) 
    #plt.gca().xaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax=1))
    plt.gca().yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(xmax=1, decimals=0))
    #plt.gca().yaxis.update(tickformat=".0%")
    #plt.xlim(0, 1)
    #plt.ylim(0, 1)
    plt.ylim(min_acc,None)
    #plt.legend(loc="center left", bbox_to_anchor=(1.04, 0.5))
    plt.savefig(f'{plot_dir}acc_4inits_{model.lower().replace("/","")}_{dataset.lower()}.pdf', 
                format='pdf') #, bbox_inches='tight')

    # Show the plot
    if show:
        plt.show()
    else:
        plt.close()

def plot_4inits_legend():
    legendFig = plt.figure("Legend plot", figsize=(4,2))
    fig = plt.figure("Line plot")
    ax = fig.add_subplot(111)
    model = "ResNet-50"
    dataset = "CIFAR-100"
    optimizer = "AdamW"

    lines = []
    labels = []

    colors = ["C1", "C2", "C3", "C4"]
    initializations = ["Default initialization", "Orthogonal", "LSUV", "Sinusoidal"]
    for i, initialization in enumerate(initializations):
        data = database.get_data(model, dataset, initialization, optimizer)["val_metric"]
        if len(data) > 200:
            data = data[:200]
        epochs = np.arange(len(data)) + 1
        acc = np.array(data)
        if acc[-1] > 1:
            acc = acc / 100
        ax.plot(epochs, acc,
            label=initialization,
            color=colors[i], linestyle='-', linewidth=2)

        line, = ax.plot(epochs, acc,
            label=initialization,
            color=colors[i], linestyle='-', linewidth=2)

        lines.append(line)
        labels.append(initialization)
            
    
    legendFig.legend(lines, labels, ncol=4, loc='center')
    legendFig.savefig(f'{plot_dir}legend_4inits.pdf', 
                      format='pdf', bbox_inches='tight')
    
    plt.close()

def plot_4inits_legend_horizontal():
    legendFig2 = plt.figure("Legend plot 2", figsize=(4,2))
    fig = plt.figure("Line plot 2")
    ax = fig.add_subplot(111)
    model = "ResNet-50"
    dataset = "CIFAR-100"
    optimizer = "AdamW"

    lines = []
    labels = []

    colors = ["C1", "C2", "C3", "C4"]
    initializations = ["Default initialization", "Orthogonal", "LSUV", "Sinusoidal"]
    for i, initialization in enumerate(initializations):
        data = database.get_data(model, dataset, initialization, optimizer)["val_metric"]
        if len(data) > 200:
            data = data[:200]
        epochs = np.arange(len(data)) + 1
        acc = np.array(data)
        if acc[-1] > 1:
            acc = acc / 100
        ax.plot(epochs, acc,
            label=initialization,
            color=colors[i], linestyle='-', linewidth=2)

        line, = ax.plot(epochs, acc,
            label=initialization,
            color=colors[i], linestyle='-', linewidth=2)

        lines.append(line)
        labels.append(initialization)
            
    legendFig2.legend(lines, labels, ncol=1, loc='center')
    legendFig2.savefig(f'{plot_dir}legend_4inits_horizontal.pdf', 
                      format='pdf', bbox_inches='tight')
    
    plt.close()



In [17]:
text =    '''
2.07%. Val Acc: 3.46%. -
5.12%. Val Acc: 5.88%. -
7.37%. Val Acc: 7.33%. -
9.22%. Val Acc: 8.72%. -
10.87%. Val Acc: 10.08%.
12.44%. Val Acc: 11.21%.
13.89%. Val Acc: 11.98%.
15.32%. Val Acc: 13.76%.
16.75%. Val Acc: 14.10%.
18.21%. Val Acc: 13.80%.
19.63%. Val Acc: 17.05%.
20.99%. Val Acc: 17.28%.
22.40%. Val Acc: 17.22%.
23.69%. Val Acc: 19.09%.
25.00%. Val Acc: 20.22%.
26.26%. Val Acc: 18.87%.
27.46%. Val Acc: 22.10%.
28.69%. Val Acc: 21.73%.
29.86%. Val Acc: 22.89%.
31.01%. Val Acc: 23.50%.
32.16%. Val Acc: 24.02%.
33.28%. Val Acc: 23.97%.
34.36%. Val Acc: 25.94%.
35.48%. Val Acc: 25.06%.
36.49%. Val Acc: 26.67%.
37.58%. Val Acc: 26.65%.
38.65%. Val Acc: 25.56%.
39.64%. Val Acc: 27.76%.
40.64%. Val Acc: 27.42%.
41.62%. Val Acc: 26.92%.
42.56%. Val Acc: 27.67%.
43.53%. Val Acc: 28.35%.
44.47%. Val Acc: 28.23%.
45.39%. Val Acc: 26.96%.
46.32%. Val Acc: 27.47%.
47.24%. Val Acc: 29.24%.
48.11%. Val Acc: 27.86%.
48.88%. Val Acc: 28.77%.
49.77%. Val Acc: 28.89%.
50.59%. Val Acc: 27.77%.
    '''
string_to_vect(text)

data[""] = [[2.07, 5.12, 7.37, 9.22, 10.87, 12.44, 13.89, 15.32, 16.75, 18.21, 19.63, 20.99, 22.40, 23.69, 25.00, 26.26, 27.46, 28.69, 29.86, 31.01, 32.16, 33.28, 34.36, 35.48, 36.49, 37.58, 38.65, 39.64, 40.64, 41.62, 42.56, 43.53, 44.47, 45.39, 46.32, 47.24, 48.11, 48.88, 49.77, 50.59],
        [3.46, 5.88, 7.33, 8.72, 10.08, 11.21, 11.98, 13.76, 14.10, 13.80, 17.05, 17.28, 17.22, 19.09, 20.22, 18.87, 22.10, 21.73, 22.89, 23.50, 24.02, 23.97, 25.94, 25.06, 26.67, 26.65, 25.56, 27.76, 27.42, 26.92, 27.67, 28.35, 28.23, 26.96, 27.47, 29.24, 27.86, 28.77, 28.89, 27.77]]


**Experiments 1**

In [3]:
'''
    data[model, dataset, initialization, optimizer]
    data[train_metric, val_metric, train_loss, val_loss]
'''
# ResNet-50 CIFAR-100
database1 = DataBase()
database = database1
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal 1", optimizer="SGD")
data.update({'train_metric': [2.14, 3.192, 4.246, 5.48, 6.626, 8.206, 9.38, 10.92, 11.866, 13.016, 14.082, 14.856, 15.876, 16.796, 17.544, 18.4, 19.05, 19.968, 20.654, 21.256, 22.154, 23.03, 23.732, 24.904, 25.204, 25.806, 26.508, 27.606, 28.104, 28.652, 29.336, 29.96, 30.502, 31.554, 31.984, 32.764, 33.284, 33.85, 34.24, 35.078, 35.758, 36.184, 36.446, 37.112, 37.592, 38.132, 39.044, 39.42, 39.88, 40.392, 40.842, 41.464, 41.89, 42.73, 43.158, 43.376, 44.232, 44.298, 44.892, 45.476, 45.802, 46.038, 46.988, 47.25, 47.648, 48.178, 48.386, 49.208, 49.436, 49.838, 50.016, 50.804, 51.258, 51.636, 51.908, 52.362, 52.76, 53.336, 53.25, 53.96, 54.32, 54.814, 55.184, 55.624, 55.958, 56.254, 56.9, 57.546, 57.854, 57.944, 58.466, 58.828, 59.308, 59.56, 59.942, 60.406, 60.714, 61.146, 61.582, 62.074], 'train_loss': [4.534679867727372, 4.386006892489656, 4.299680637886457, 4.2258796066884186, 4.152515753455784, 4.065445942951895, 3.9808492151367694, 3.8967499059179556, 3.8297051773656663, 3.7529261932348654, 3.6895555977321344, 3.6318826504680506, 3.5792412254816433, 3.5338226693975345, 3.4812494931013687, 3.4324694871902466, 3.385623440718102, 3.3420727713333678, 3.297494882208002, 3.2618801490120264, 3.214351981192294, 3.172759969521057, 3.132121423321307, 3.084818551912332, 3.0510983909182534, 3.009363681459061, 2.9743685502835246, 2.9387422757380453, 2.904583956579418, 2.861804867034678, 2.8324655809670762, 2.802068174037787, 2.768462208530787, 2.7319063106766137, 2.7059493211224255, 2.675027488137755, 2.6462092948386737, 2.614516401229917, 2.5905486659320722, 2.5643315745131745, 2.5318944281934166, 2.5073868923479945, 2.4822545478411038, 2.4538851734012592, 2.4313734669209746, 2.406133297611685, 2.3756971452242275, 2.3511149512837304, 2.325691602266658, 2.308521153829287, 2.283850220768043, 2.2592530718544865, 2.2354743748979495, 2.212959734527656, 2.186089496935725, 2.168815431692411, 2.143479797846216, 2.12687621336154, 2.104817124278954, 2.084025524461361, 2.0647301067171804, 2.047170567695442, 2.0215919354687566, 2.000215114382527, 1.9849214713896632, 1.9654522086958142, 1.952104903547965, 1.9239154782746455, 1.9125012095322085, 1.8895971017420445, 1.8791964784302675, 1.852277957417471, 1.8324008524570319, 1.8181716840895241, 1.8035022921269508, 1.7829872447511423, 1.7676605175218314, 1.7482688285200798, 1.7352008365304268, 1.7114135738833787, 1.699078002701635, 1.6812493549588392, 1.6687738970112618, 1.643094562203683, 1.628989121188288, 1.6183757730152295, 1.597137992949132, 1.5765176756912485, 1.5681516226295316, 1.555987441783671, 1.5346526407524752, 1.513062529399267, 1.502759141690286, 1.4883444749790689, 1.4762439551713216, 1.4567803504217007, 1.4384941783402583, 1.422996341648614, 1.4112290379488865, 1.3947922596541207], 'val_metric': [3.06, 4.07, 4.91, 6.18, 7.82, 8.82, 10.2, 11.65, 13.0, 13.55, 15.11, 15.77, 16.31, 17.71, 17.98, 19.38, 20.12, 20.4, 21.26, 22.16, 22.79, 24.06, 23.39, 24.96, 25.6, 26.32, 26.84, 27.02, 28.72, 28.56, 28.67, 29.29, 30.33, 30.8, 31.47, 31.43, 32.33, 32.35, 33.06, 33.33, 33.71, 34.45, 34.4, 34.95, 35.72, 35.84, 36.23, 36.6, 37.36, 38.08, 38.04, 38.51, 38.79, 39.21, 39.13, 39.12, 39.35, 39.83, 40.88, 40.83, 41.05, 41.62, 41.14, 41.58, 41.93, 41.85, 42.77, 42.92, 43.31, 43.92, 43.14, 44.64, 44.87, 43.35, 43.66, 44.52, 44.55, 45.59, 44.2, 44.99, 45.73, 44.77, 45.65, 45.76, 47.38, 47.23, 47.05, 47.43, 44.17, 48.21, 47.75, 48.14, 45.94, 47.27, 48.03, 47.76, 47.65, 46.68, 48.2, 47.65], 'val_loss': [4.421704559569147, 4.338748376080944, 4.2531545298874, 4.174378642610684, 4.089658421315964, 4.014144168537893, 3.9368014548234878, 3.8559439288582773, 3.7654137079882775, 3.709077329392646, 3.647386997368685, 3.598432177950622, 3.543312453919915, 3.492984759579798, 3.4556616582688253, 3.394900839799529, 3.3503291880249217, 3.3286160086370575, 3.2782994455592647, 3.2450737512795027, 3.195277270238111, 3.1613679296651465, 3.136164347836926, 3.0788076546541445, 3.0407821008354237, 3.0026697505051922, 2.980483430206396, 2.9587605455119137, 2.904832555989551, 2.8957335645226157, 2.874700716346692, 2.839110489863499, 2.798763923584276, 2.7762315911092577, 2.7316930537011213, 2.723825978625352, 2.682251974275917, 2.674694153913267, 2.647072139059662, 2.64023134055411, 2.6021981163389363, 2.5792816490124744, 2.5806471678861387, 2.561023596745388, 2.536859726450246, 2.5099106639813464, 2.491366603571898, 2.4980745983731216, 2.4431491414452813, 2.4156754085212757, 2.3936194113105724, 2.390895410707802, 2.3581663339760652, 2.3390181345544803, 2.343531271454635, 2.347457672380338, 2.3464865266897115, 2.3090992991332038, 2.2625547229864034, 2.2720470018447583, 2.2841446984345746, 2.2334997699518873, 2.255423894353733, 2.243272400206062, 2.240957865289822, 2.227568428987151, 2.18184965810958, 2.1844886488215938, 2.1620260006303242, 2.1365676875327044, 2.1619424979398203, 2.106281913010178, 2.0963809270008356, 2.160519476908787, 2.1534860027823477, 2.1099611399280036, 2.092349619622443, 2.0612109899520874, 2.134966933043899, 2.0883937853916437, 2.0693376519877438, 2.1039489029319425, 2.0508731079709, 2.0626816886245827, 2.006467169257486, 2.004932437732721, 2.01481336848751, 1.9786665629429423, 2.1494144429067137, 1.9526640996811495, 1.9636193111443976, 1.964064734756567, 2.07888632367371, 2.01113617647985, 1.961965363496428, 1.9677015429089784, 1.9781810683049974, 2.070879977979478, 1.9720434368036355, 1.983571814123992]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal 1", optimizer="Adam")
data.update({'train_metric': [12.638, 25.098, 31.276, 35.614, 38.562, 40.922, 42.612, 44.598, 46.314, 47.744, 49.348, 50.386, 51.46, 52.582, 53.83, 54.364, 55.346, 55.408, 56.628, 56.95, 57.458, 57.91, 58.586, 58.892, 59.242, 59.752, 59.796, 60.186, 60.364, 60.618, 61.31, 61.168, 61.404, 61.804, 62.306, 62.042, 62.268, 62.514, 62.558, 62.842, 62.98, 63.156, 63.51, 63.53, 63.722, 63.602, 63.388, 63.926, 63.986, 64.016, 64.086, 64.258, 64.318, 64.336, 64.19, 64.73, 64.722, 64.99, 64.636, 64.838, 65.192, 65.01, 65.0, 65.122, 65.234, 65.258, 65.474, 65.232, 65.684, 65.616, 65.54, 65.568, 66.016, 65.78, 65.982, 66.052, 65.938, 65.974, 65.55, 66.254, 66.374, 65.968, 66.078, 66.354, 66.256, 66.226, 66.336, 66.498, 66.458, 66.302, 66.76, 66.678, 66.482, 66.616, 66.406, 66.618, 66.376, 66.896, 66.778, 66.808], 'train_loss': [3.7884592692870314, 2.942139051149568, 2.6207311433904312, 2.428070173696484, 2.2902342027715403, 2.188286230234844, 2.1027347823543012, 2.022798404974096, 1.9541649197983315, 1.8877154943888144, 1.8257041989689897, 1.7786647278024716, 1.7310942223919628, 1.6872508307856977, 1.6464497143655177, 1.6205300156722593, 1.5833454697638216, 1.5729802050401487, 1.533913619606696, 1.5214791625661923, 1.495704149498659, 1.4800451263747252, 1.4641754073864968, 1.4455739236853618, 1.430720463128346, 1.409776666204033, 1.4116520820676213, 1.3947805602989538, 1.3840903203048365, 1.3790023788771666, 1.364781484960595, 1.359884664987969, 1.346450178214656, 1.3371127281347504, 1.326098334682567, 1.3264498761700243, 1.319407252323292, 1.3095645348891578, 1.3032165303102234, 1.2980914842289732, 1.2899844294313885, 1.2829190787604399, 1.2789249256291353, 1.2775163318190124, 1.266848438764777, 1.2669175442527323, 1.2687866790672702, 1.2537729456601545, 1.2552289702093509, 1.2507146730294922, 1.248416432639217, 1.23936530726645, 1.2414949066041376, 1.2373022112395147, 1.2397072226799968, 1.2304212829798384, 1.2262161925930501, 1.223380942173931, 1.2249598713482128, 1.2138100511887495, 1.2144457825157038, 1.2117603293922552, 1.2084521709195792, 1.2090295102742628, 1.2056386196399893, 1.2048465893853961, 1.2024659868853782, 1.1985977093124633, 1.1906009832459032, 1.1953661196372087, 1.1935756737771241, 1.191851109647385, 1.1765954213221665, 1.1888316020636303, 1.1783502604955298, 1.1803775431250063, 1.181746693752001, 1.1811597086584475, 1.18553399658569, 1.1725256908732606, 1.1636470252900477, 1.174309453772157, 1.171306187996779, 1.177974439033157, 1.1638522959883562, 1.163896918296814, 1.1604836654022832, 1.1567387139553305, 1.1624340179478725, 1.1639897381253255, 1.1562676332948152, 1.1526638627662074, 1.1562321667018753, 1.15796147587964, 1.158108133717876, 1.1528316453442244, 1.157198204408826, 1.149188813117459, 1.149055551308805, 1.1505555932021811], 'val_metric': [20.25, 23.73, 30.54, 31.18, 33.41, 38.15, 41.26, 39.29, 40.92, 46.08, 46.33, 48.9, 49.15, 50.63, 52.26, 51.5, 51.52, 53.4, 53.92, 52.32, 47.8, 53.84, 53.39, 56.1, 55.24, 53.66, 54.47, 53.91, 52.68, 55.99, 56.71, 55.7, 55.28, 54.38, 55.22, 56.41, 57.65, 55.3, 55.41, 57.48, 55.84, 56.42, 55.88, 57.11, 57.35, 57.59, 58.84, 57.0, 56.4, 55.4, 56.85, 57.11, 58.74, 60.05, 59.01, 58.28, 57.23, 55.91, 60.15, 59.69, 58.58, 56.63, 58.5, 58.09, 58.61, 57.83, 56.88, 59.89, 58.71, 57.87, 58.92, 59.69, 60.15, 59.54, 57.99, 58.57, 60.06, 59.81, 59.93, 58.07, 57.87, 58.17, 58.15, 57.49, 58.99, 59.76, 55.74, 59.95, 59.76, 57.18, 60.97, 57.04, 59.54, 59.1, 58.53, 59.14, 59.53, 58.42, 58.06, 58.96], 'val_loss': [3.252288168402994, 3.06893735934215, 2.6242147387972303, 2.6365335488774972, 2.5676276167486884, 2.327071691014964, 2.163827447374915, 2.314379906958076, 2.2039506207605837, 1.9488059008956715, 1.9293017440540776, 1.8261687057033467, 1.8077387999577128, 1.7502451231525202, 1.7078839593632207, 1.7587112704659724, 1.7712513109680954, 1.676363954878157, 1.6658697485164473, 1.6937424718954002, 1.93155712914315, 1.6464224229952333, 1.6380852589941328, 1.5485465146933393, 1.5716631032858686, 1.6861074320070304, 1.5914966779150022, 1.638603438996965, 1.6914712198221, 1.5659496024915367, 1.5415418949096826, 1.5648692990564237, 1.5653709233946103, 1.625762550694168, 1.5746383610045074, 1.5267131723416079, 1.5092792389499154, 1.587120312793999, 1.5779567930349119, 1.5066272164605985, 1.5632346333212155, 1.534285574202325, 1.5658743130932948, 1.5280097301598567, 1.5156809330745866, 1.494584035721554, 1.4689434447865577, 1.5032471543664385, 1.515363041003039, 1.5841981969821226, 1.54687849047837, 1.5271279166458518, 1.4634143884774227, 1.388627023453925, 1.4431103217373988, 1.4734203359883302, 1.4985284099153653, 1.589185171446223, 1.404014744576375, 1.434304441616034, 1.4433470121614493, 1.5252087571818358, 1.4638078011524904, 1.4735992201574288, 1.4579945555917777, 1.5150289812665076, 1.5098677562300566, 1.4164492381606133, 1.4652076443289495, 1.4848685275976825, 1.4773143154041024, 1.4386051076992301, 1.431164876290947, 1.4380779270153896, 1.4837617335046174, 1.4564668950001904, 1.3944532586510774, 1.4139798526551313, 1.3913840423723696, 1.4749062357434801, 1.4874496247358382, 1.4385273456573486, 1.485741235647991, 1.5308408418278785, 1.443390042918503, 1.41791832447052, 1.6315333440804938, 1.4015024015857915, 1.4095152631686751, 1.5511028990624056, 1.3706681048794158, 1.524667403500551, 1.4268040398883213, 1.4454983780338506, 1.4688695213597291, 1.4316580329731012, 1.4472051612131156, 1.4459383734472238, 1.4944794747480161, 1.4570573401299252]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal 1", optimizer="AdamW")
data.update({'train_metric': [12.652, 28.542, 40.132, 47.886, 53.426, 57.964, 61.996, 65.326, 68.658, 71.768, 74.716, 76.798, 79.86, 82.25, 84.252, 86.406, 85.93, 89.732, 91.19, 92.11, 92.792, 93.474, 94.19, 94.458, 95.12, 95.294, 95.604, 96.18, 96.096, 96.65, 96.586, 96.79, 97.17, 97.114, 97.17, 97.572, 97.202, 97.856, 97.724, 97.688, 97.976, 98.108, 98.056, 97.984, 98.09, 98.198, 98.106, 98.532, 98.364, 98.392, 98.458, 98.44, 98.676, 98.472, 98.554, 98.768, 98.674, 98.606, 98.908, 98.566, 98.886, 98.742, 98.86, 99.08, 98.988, 98.766, 98.97, 98.846, 99.092, 98.82, 99.12, 99.006, 98.908, 99.184, 98.968, 99.04, 99.174, 99.212, 99.132, 99.168, 98.522, 99.21, 99.302, 99.068, 99.086, 99.164, 99.226, 99.378, 99.184, 99.154, 99.284, 99.15, 99.292, 99.318, 99.258, 99.252, 99.244, 99.306, 99.318, 99.386], 'train_loss': [3.785556883153403, 2.786282896690661, 2.2375708571480364, 1.9001897629874442, 1.6593565221332833, 1.4739531462302293, 1.3189659889244363, 1.1805010041617372, 1.0585061548768406, 0.9383629911467243, 0.8308623995622406, 0.7561155642237505, 0.6467400705418014, 0.5658538687564528, 0.4866575904362037, 0.4180351397608552, 0.4377195744029701, 0.3142303469521768, 0.2665311729778414, 0.24076886849520762, 0.22176465019107322, 0.1960896947766509, 0.17737677169110996, 0.1696896140351701, 0.1474764996429767, 0.1399832441783546, 0.13211102510357034, 0.11678617038166203, 0.12084383850731432, 0.1039641472377607, 0.10523277982269101, 0.0974593218380247, 0.09156145800864968, 0.0891200481530498, 0.08860154048232433, 0.07601605894227448, 0.08491423959865252, 0.06739641376145784, 0.07118689832861161, 0.06935668796601008, 0.06635947364276451, 0.06069026535074405, 0.061783702417319675, 0.06243579770209711, 0.06157331152663083, 0.05505157890015036, 0.05851879813994193, 0.048001785836446924, 0.05256904828433028, 0.05247921210623172, 0.048049938173957474, 0.04957809824647758, 0.04157804175475946, 0.048189855114285314, 0.0445129343442788, 0.04010570472177199, 0.04280845292881393, 0.04422160380584238, 0.036627592539469546, 0.04657336842010269, 0.0373823079555218, 0.041866044046731235, 0.03662649068943894, 0.030858172150805135, 0.03297923681146228, 0.03880041081424388, 0.033605890534862834, 0.03627620315585045, 0.028957385862784464, 0.03657421253836247, 0.027616309912522775, 0.03248593663385012, 0.036949002147426524, 0.02572716829773989, 0.03444484564984251, 0.03142515878068626, 0.0268698868908301, 0.024620034329346358, 0.029502208663589533, 0.028237479797731477, 0.04443300277483888, 0.025722938808328186, 0.022084089544931632, 0.030481794533808657, 0.02959517217287377, 0.02610382172969066, 0.027731714772170377, 0.020157430854449144, 0.025442655952351824, 0.02606931353374051, 0.021913697227330692, 0.026870637690699646, 0.02297715812116442, 0.023762021142182677, 0.02357709287684244, 0.02396737436427201, 0.025051465626489525, 0.024201070770326242, 0.022656882701166538, 0.020240975571546197], 'val_metric': [20.49, 34.71, 42.26, 48.32, 53.34, 57.1, 57.4, 60.02, 62.76, 63.56, 65.33, 65.6, 66.32, 67.29, 66.51, 53.82, 68.43, 68.47, 68.81, 67.74, 69.4, 69.33, 69.35, 70.02, 68.53, 69.59, 69.64, 69.54, 70.32, 69.85, 70.04, 69.4, 71.31, 70.87, 71.04, 70.52, 69.72, 70.6, 71.17, 71.14, 69.93, 69.98, 69.88, 70.39, 71.22, 70.19, 71.25, 69.76, 70.6, 71.47, 69.42, 70.85, 71.85, 72.2, 70.33, 70.64, 70.65, 71.38, 70.17, 69.99, 70.59, 70.37, 71.57, 70.25, 71.4, 70.79, 71.6, 71.35, 72.38, 70.96, 70.9, 70.87, 71.47, 72.29, 71.77, 71.12, 71.8, 71.34, 71.67, 71.85, 71.69, 71.58, 70.2, 71.19, 70.56, 71.78, 71.62, 72.14, 72.02, 71.18, 72.53, 70.6, 71.74, 71.74, 72.56, 70.34, 71.98, 71.1, 71.13, 72.3], 'val_loss': [3.1596191096457704, 2.4896527726179474, 2.132386842351051, 1.8849983374783947, 1.6757763943095116, 1.5127231216734383, 1.4843513790968876, 1.435429403736333, 1.3247139552596268, 1.286020912182559, 1.2580064709778804, 1.26228628796377, 1.2439269981566508, 1.2222732256172568, 1.2639779254889032, 5.078588473569056, 1.27953657792632, 1.2866446994672156, 1.4055062931054716, 1.4753573810219005, 1.3747498393058777, 1.368430973617894, 1.4039556628959193, 1.3944224369753697, 1.5320926532623873, 1.5140277754728961, 1.5116924408135142, 1.5744834174016478, 1.4521086265327066, 1.6185542758862683, 1.5128047815553702, 1.623515890662078, 1.4803501523224412, 1.4935386590896897, 1.5436264238539774, 1.573352003553111, 1.6098629498178032, 1.5776322962372167, 1.5665449238127205, 1.6066116800733432, 1.5734458265790514, 1.6831055215209911, 1.7244196837874735, 1.6162738953806033, 1.594234577979252, 1.646690241850106, 1.617572743421907, 1.6492222327335624, 1.6822798544434225, 1.623017429546186, 1.6988910022814563, 1.646860220629698, 1.5680690210336332, 1.556765196429696, 1.65361881332033, 1.6840760009303974, 1.739448103935096, 1.7437517365832238, 1.7283683126899088, 1.7064006992965748, 1.7313801946154066, 1.6870647657449078, 1.6862310368543978, 1.7229194371563614, 1.6255396946220642, 1.7676592135125664, 1.6767924623504566, 1.7366527159502552, 1.6670222893642013, 1.7071507873049208, 1.851059606500492, 1.8278240903167968, 1.7075614560941221, 1.7100917267951237, 1.6812478934124018, 1.755370009097324, 1.7653852021618255, 1.6528084798223655, 1.6870789136856226, 1.6792536981546196, 1.7514118548411473, 1.771897934045002, 1.9019999910312093, 1.8072868771613784, 1.791246080474489, 1.8268023156056739, 1.7817606075554138, 1.7304812035742838, 1.7334884222905347, 1.7713923332797494, 1.7409073383944809, 1.7465149566626093, 1.7487009277768955, 1.7279282956366326, 1.69030220721178, 1.8969969229333719, 1.7497218923204263, 1.7764135477649179, 1.813794609847342, 1.721171015387128]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Default initialization", optimizer="SGD")
data.update({'train_metric': [1.85, 3.024, 3.838, 4.388, 5.298, 5.876, 6.702, 7.412, 8.188, 8.898, 9.698, 10.504, 10.898, 11.442, 12.118, 12.658, 13.046, 13.748, 14.116, 14.812, 15.304, 15.57, 16.24, 16.69, 17.084, 17.308, 17.85, 18.196, 18.872, 19.212, 19.63, 20.114, 20.474, 21.022, 21.508, 21.978, 22.154, 22.616, 23.202, 23.366, 23.902, 24.306, 24.914, 25.368, 25.626, 26.026, 26.36, 26.868, 27.276, 27.802, 27.942, 28.412, 28.524, 28.834, 29.38, 29.644, 30.13, 30.272, 30.966, 30.978, 31.334, 31.75, 31.872, 32.45, 32.576, 33.046, 33.406, 33.706, 34.016, 34.32, 34.6, 35.014, 35.264, 35.55, 35.768, 36.138, 36.2, 36.422, 37.012, 37.26, 37.424, 37.938, 38.052, 38.27, 38.506, 38.7, 39.07, 39.486, 39.68, 39.952, 40.104, 40.578, 40.892, 41.002, 41.44, 41.664, 41.846, 41.908, 42.598, 42.578], 'train_loss': [4.524250310400258, 4.436608221219934, 4.394717168929937, 4.351327861971257, 4.314161389685043, 4.284593755936684, 4.231219347175735, 4.160355478296499, 4.081600403541799, 4.033755227123075, 3.9896610688675396, 3.944743996081145, 3.899265499675975, 3.853838883397524, 3.812680852382689, 3.775247651902611, 3.7408469512944333, 3.704523805462186, 3.669631700991365, 3.6348235546170597, 3.6040103731252957, 3.5740886996774113, 3.5443095560268976, 3.519808950326632, 3.4913395493841537, 3.46901502084854, 3.4431512136288616, 3.4212217769964273, 3.3926395205280664, 3.368965121791186, 3.344467059730569, 3.3243211120595713, 3.303600327438101, 3.2838207553414738, 3.2575240699226593, 3.234966696375776, 3.2144880758222105, 3.1924318566041836, 3.1718585966798045, 3.1505703142536876, 3.1292136087441995, 3.1058075037758672, 3.0845649278987093, 3.0641972747300286, 3.0382673651970866, 3.0217619832519373, 2.998995161727261, 2.9761817879079246, 2.9597170209640735, 2.9349134108599495, 2.9177976715595215, 2.8945002936951036, 2.876090400664093, 2.8552718217415576, 2.836025743228395, 2.819876427540694, 2.7948584080961965, 2.7843774813215445, 2.7671359661594988, 2.750944917159312, 2.728706949202301, 2.711047139923896, 2.6974092487179107, 2.679699864838739, 2.663639606264851, 2.647784738894314, 2.6316328304807852, 2.617259163990655, 2.5964041143427115, 2.585991445099911, 2.56956731007837, 2.5518578354964783, 2.5391697113776144, 2.5273295412283114, 2.513672412813777, 2.499280552425043, 2.4885264414045816, 2.4721539281213376, 2.4522277100006944, 2.444062452029694, 2.428506023896015, 2.4099359000125506, 2.3957655888689144, 2.3852739485023577, 2.3770297784024796, 2.3621376390042514, 2.3488819227194235, 2.327712423203851, 2.3188357249550196, 2.3122485398941333, 2.2938155664507387, 2.2801601908091085, 2.268373331145557, 2.256191647571066, 2.2396917959003497, 2.2262385112550254, 2.2166034882635715, 2.2074456188989724, 2.192270027097229, 2.1772570946942205], 'val_metric': [3.25, 3.41, 4.58, 4.88, 5.92, 6.97, 6.83, 7.82, 8.52, 9.41, 9.89, 11.01, 11.52, 12.19, 12.81, 13.27, 13.76, 14.15, 15.17, 15.23, 15.72, 16.5, 16.4, 17.07, 17.48, 17.81, 18.04, 18.74, 19.48, 19.81, 20.35, 20.37, 20.95, 21.02, 21.81, 22.44, 22.47, 23.27, 23.31, 23.58, 24.4, 24.8, 25.23, 24.89, 25.13, 25.54, 25.97, 26.72, 26.69, 27.36, 27.15, 27.58, 28.04, 28.26, 28.67, 28.89, 28.88, 29.81, 29.83, 30.34, 30.09, 30.19, 30.89, 31.21, 31.11, 31.3, 31.83, 31.79, 32.41, 31.97, 32.2, 32.43, 32.95, 33.03, 33.94, 33.74, 33.74, 34.33, 34.66, 35.05, 32.63, 32.66, 35.3, 32.49, 35.72, 35.78, 34.31, 35.48, 35.86, 36.6, 36.36, 36.12, 36.78, 37.44, 37.22, 34.56, 37.33, 37.45, 36.99, 37.04], 'val_loss': [4.461550779403395, 4.408418096554507, 4.366058030705543, 4.329603829960914, 4.29833591060274, 4.2431134008298255, 4.189876299754829, 4.109794150492188, 4.034274196928473, 4.008786004060393, 3.9518265131932155, 3.9100524969161694, 3.8584170630023737, 3.8229177347414054, 3.7778403667887304, 3.7441368558604244, 3.7145128736070765, 3.676572679714033, 3.6438253411821497, 3.6097127434554372, 3.5815165938845106, 3.5512234891296193, 3.525465717740879, 3.4959390846786986, 3.4881410067248497, 3.4513951653887514, 3.4296096464630903, 3.3990204941694904, 3.378592319549269, 3.35630309961404, 3.338028968519466, 3.3222772892873, 3.2890023684046072, 3.287420541617521, 3.2492896720861935, 3.2231890107416046, 3.2136532759210867, 3.1919408300120358, 3.172769625475452, 3.1509882416694786, 3.132803907819614, 3.1158870648426613, 3.0930238893836925, 3.0948533768866473, 3.0675736673318656, 3.0468416699937952, 3.0221575946564885, 2.9995437245460073, 2.9781517997668807, 2.959015209963367, 2.952236145165316, 2.9417890879758604, 2.9176126908344826, 2.897412292516915, 2.8772071319021237, 2.873633923803925, 2.875014352191026, 2.847605416729192, 2.8254571401389543, 2.794751420901839, 2.798289845703514, 2.793731179206994, 2.7633439507454063, 2.7450978938181687, 2.763905879038914, 2.7555401446713006, 2.7151396957932006, 2.7106615175866775, 2.69546768923474, 2.696574886893011, 2.699114831389895, 2.6954583301665678, 2.6785358395546104, 2.6572758012516484, 2.62688590614659, 2.6275961232033507, 2.6260402627811312, 2.5869128749628736, 2.581557146303213, 2.5771078197819413, 2.672120896114665, 2.696882954069004, 2.5395095485031227, 2.7236073381581885, 2.52071574387277, 2.5427563843453767, 2.6002342366868523, 2.546567061144835, 2.51746790576133, 2.5122539313735475, 2.4900534001125654, 2.5166595316236946, 2.473334161339292, 2.4590263776718433, 2.454349643105914, 2.6068650696687636, 2.4492256094695657, 2.431462119339378, 2.4781913666208837, 2.4720336510117646]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Default initialization", optimizer="Adam")
data.update({'train_metric': [6.958, 12.092, 16.722, 20.85, 25.172, 29.934, 33.656, 36.684, 38.782, 40.294, 41.284, 42.766, 43.602, 44.556, 45.16, 45.756, 46.526, 46.944, 47.628, 48.45, 48.536, 48.986, 49.53, 49.88, 50.4, 50.554, 50.776, 51.186, 51.556, 51.66, 52.056, 52.324, 52.39, 52.488, 52.942, 52.77, 53.284, 53.582, 53.602, 53.746, 53.91, 54.114, 54.412, 54.19, 54.322, 54.8, 54.756, 54.962, 55.062, 55.142, 55.184, 55.29, 55.502, 55.702, 55.84, 55.808, 55.992, 56.042, 55.986, 56.316, 56.196, 56.47, 56.666, 56.5, 56.474, 56.528, 56.726, 56.834, 56.864, 56.88, 57.094, 57.338, 57.06, 57.208, 57.394, 57.458, 57.228, 57.396, 57.522, 57.488, 57.706, 57.514, 57.526, 57.76, 57.558, 58.04, 58.054, 58.084, 58.142, 58.196, 58.35, 58.262, 58.252, 58.606, 58.446, 58.368, 58.49, 58.598, 58.4, 58.412], 'train_loss': [4.086174638070109, 3.694949817779424, 3.45098330755063, 3.2189914850932557, 2.9830929644577338, 2.728053690222523, 2.5357977081747616, 2.390016735209833, 2.301051013152618, 2.231092012599301, 2.1715243364234107, 2.118388666368811, 2.075725157242602, 2.041140952531029, 2.0056843219510734, 1.981309314060699, 1.947816099657122, 1.9309755495137266, 1.905003922979545, 1.8807156067675033, 1.864183785512929, 1.845276248577001, 1.826351699621781, 1.8147573562534265, 1.7951184351121068, 1.786373828378175, 1.770448654661398, 1.7610081981515031, 1.7428140079273897, 1.7441283066559325, 1.7301986368415911, 1.712061859457694, 1.7108423383644475, 1.6996492132201524, 1.6900272819087327, 1.6877030353716878, 1.6754687905616468, 1.6695006329690099, 1.6608033855552868, 1.6511039448821025, 1.648435533504047, 1.6446764659698663, 1.636910368109603, 1.6322363956505075, 1.62555036306991, 1.6154453649240381, 1.6134734726927775, 1.6094064543314297, 1.6065085123262137, 1.597368747986796, 1.5959039387648062, 1.5915174853161473, 1.5857795563042927, 1.5783464441366513, 1.57254422793303, 1.5740685034590913, 1.568820387201236, 1.5608386258639948, 1.5646597328393355, 1.5594534779448643, 1.5567045648537023, 1.5501566093291164, 1.545319440953262, 1.547192642908267, 1.5473093951449675, 1.5430139644676462, 1.5392124486701262, 1.5336201010305253, 1.5341912672647735, 1.5298529628597561, 1.5251328603690848, 1.5226195957654578, 1.5236170536569318, 1.521963419816683, 1.515260342213199, 1.512743856977014, 1.5134781727095699, 1.5121857209126357, 1.5085859839873546, 1.5054536341401317, 1.5062411040296335, 1.5084419709337338, 1.5025487687734083, 1.4979922152541179, 1.4985189210728307, 1.4919297695159912, 1.4925490994282695, 1.4934181754698839, 1.4862786972004434, 1.486874005907332, 1.4807813804777687, 1.4838916430692843, 1.4793815567060504, 1.472380348788503, 1.4806519320706273, 1.4761893064774516, 1.4741016986120083, 1.4731015672769083, 1.4712383117517243, 1.470550324834521], 'val_metric': [9.27, 13.27, 17.86, 23.38, 26.77, 31.87, 34.43, 36.34, 37.65, 36.94, 40.0, 42.03, 41.91, 42.18, 44.64, 45.01, 43.89, 42.72, 44.6, 46.92, 46.63, 46.96, 48.75, 47.98, 46.95, 47.47, 48.71, 47.96, 46.69, 45.33, 48.5, 48.54, 46.85, 50.42, 48.24, 50.72, 45.88, 48.41, 49.91, 50.05, 50.93, 51.45, 49.54, 50.59, 49.51, 49.83, 50.94, 51.4, 50.74, 51.52, 47.24, 51.34, 52.42, 51.99, 51.12, 51.85, 50.02, 51.05, 51.54, 51.21, 51.86, 50.11, 52.77, 51.53, 51.64, 51.67, 52.27, 50.41, 50.28, 45.22, 53.89, 53.07, 53.06, 52.21, 51.49, 50.36, 52.72, 49.74, 53.43, 50.52, 53.11, 52.92, 51.26, 50.3, 53.39, 53.67, 54.11, 52.81, 53.02, 51.07, 53.81, 53.12, 52.09, 53.99, 51.29, 53.72, 53.4, 53.36, 52.89, 52.26], 'val_loss': [3.856482715363715, 3.6669752385206285, 3.354743082811878, 3.074368997743935, 2.9156528035546563, 2.650025079204778, 2.4893217443660567, 2.4029835948518885, 2.334473300891317, 2.363673182050134, 2.219264597649787, 2.1489752485493945, 2.155672540330583, 2.1432988021024473, 2.0308706752813546, 2.011430259722813, 2.0614643248782794, 2.103089700838563, 2.0125309039073387, 1.9432238249262428, 1.9611456394195557, 1.949082354831088, 1.8807457100813556, 1.9051965353595224, 1.9202316824797612, 1.9053489949293196, 1.8327709740134561, 1.891130543058845, 1.9691333019050064, 1.9894644439600075, 1.8786431459864235, 1.8464858957157013, 1.9661903001700238, 1.7894435703374778, 1.8934324091407144, 1.7712671528955934, 1.9922605191066767, 1.8663904477076925, 1.7767233810607035, 1.8105985609589108, 1.7658832498416779, 1.7471332375411015, 1.8419060760242925, 1.7709786557847527, 1.7915014446161355, 1.8217867012995823, 1.765147272188952, 1.7306878073200298, 1.7508594519013811, 1.7452066567293398, 1.9481076260281216, 1.7569234447114785, 1.7097295780850064, 1.7065057822853138, 1.7486257006408303, 1.717833938871979, 1.7992967192534428, 1.7549159754613401, 1.7193451724993956, 1.7530253981329074, 1.7272964100928823, 1.811442550580213, 1.707083764349579, 1.7241607342556025, 1.7411045147355195, 1.7229038370642693, 1.7094440164080091, 1.8279806208458675, 1.8046803390903838, 2.045560955241987, 1.6443207643593951, 1.6968599497133, 1.6772836849188348, 1.6862331833809046, 1.7247294233103467, 1.8040223076085375, 1.7086821756545145, 1.8124049423606532, 1.6714533802809988, 1.8044561026202646, 1.6524303688365183, 1.685625869757051, 1.7840576171875, 1.7720807479445342, 1.660540494949195, 1.6534756300555673, 1.6196871297374653, 1.6773440716372934, 1.6657821532267674, 1.7827055309988131, 1.653914569289821, 1.6734092903744644, 1.704404555308591, 1.6430153983413793, 1.7668137535168107, 1.6490855300502412, 1.6591983661530123, 1.658240130752515, 1.6670301795765092, 1.696215511127642]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Default initialization", optimizer="AdamW")
data.update({'train_metric': [7.842, 17.056, 25.444, 33.0, 39.638, 45.282, 50.562, 54.916, 58.15, 62.52, 66.104, 68.778, 71.692, 74.584, 76.99, 79.866, 82.12, 84.114, 86.304, 88.06, 89.274, 90.93, 92.244, 92.494, 93.492, 94.07, 94.26, 94.86, 94.964, 95.108, 95.4, 96.318, 96.08, 96.332, 96.53, 96.57, 96.402, 97.118, 97.154, 97.114, 97.152, 97.43, 97.022, 98.056, 97.186, 97.862, 97.582, 98.038, 97.804, 97.342, 98.37, 98.206, 97.906, 98.444, 97.956, 98.414, 98.026, 98.63, 98.1, 98.52, 98.352, 98.462, 98.526, 98.536, 98.612, 98.508, 98.542, 98.632, 98.582, 98.752, 98.512, 98.788, 98.728, 98.68, 99.136, 98.734, 98.66, 99.05, 98.722, 98.952, 98.882, 98.904, 98.946, 98.804, 98.904, 99.166, 98.922, 98.994, 98.924, 99.128, 98.99, 99.01, 99.106, 99.004, 99.364, 98.9, 99.15, 99.278, 98.918, 99.244], 'train_loss': [4.044475985609966, 3.426085855040099, 2.9719615168583666, 2.5902303917633605, 2.2844202064187327, 2.015165862524906, 1.791137375795018, 1.60713630213457, 1.470310378867342, 1.2986605492851617, 1.162235268241609, 1.0557815304497624, 0.9413333980323714, 0.8373377818966765, 0.7444092025765983, 0.6506323407182608, 0.5658249035477638, 0.49842596922993965, 0.4288539564060738, 0.3654090499653078, 0.32636467858081886, 0.28253889027649487, 0.23712341515037716, 0.22793311753388865, 0.20045385833191293, 0.18460261880817924, 0.17390509881079197, 0.15613633249660053, 0.14985902485725902, 0.15063331185666193, 0.14350351222250088, 0.11541729217723888, 0.1191413909139688, 0.11257059691245179, 0.10643453408947305, 0.10577884759477642, 0.10658639939823437, 0.08968182082962045, 0.08702652534658133, 0.0901060562105635, 0.08702536872731012, 0.0791002128998537, 0.09339050084824109, 0.06229584723594301, 0.08750076500622703, 0.06780872741461638, 0.07433381192195598, 0.06205876880799375, 0.06776809983569985, 0.08117553817294061, 0.0516545950033752, 0.05490936531254764, 0.06396400004886138, 0.05098220643729134, 0.062654763065955, 0.05022833437439712, 0.0629764440275319, 0.04465051634919584, 0.059852216909032155, 0.04702273670948989, 0.051110623974104166, 0.04919250659427851, 0.046376029221380316, 0.04496593676957652, 0.044037013946463116, 0.04596201624007314, 0.04586651287612665, 0.042517933470513816, 0.04498090887504161, 0.03876400033346689, 0.04667239225260518, 0.04019742107401123, 0.04050370317559553, 0.04155337401380515, 0.029092080530460716, 0.04034327365697095, 0.04013315327343462, 0.03069781495289177, 0.04137058795103446, 0.032812487996999734, 0.03576142189543709, 0.03465506688403168, 0.03246414205385432, 0.037698239968691254, 0.033152924611127774, 0.027687244680951306, 0.0344581038944478, 0.030853621400249623, 0.03388212890158195, 0.02865770685857476, 0.0319690260978992, 0.032809611695432735, 0.030300678618837628, 0.03186596091725039, 0.021260607346836977, 0.03459493550272477, 0.028440950316605648, 0.023651413794662304, 0.03255952018104098, 0.02339540930513097], 'val_metric': [13.44, 20.88, 27.7, 35.04, 41.86, 41.87, 42.66, 52.5, 53.29, 57.42, 57.84, 60.0, 60.77, 61.71, 62.23, 62.11, 61.95, 64.35, 63.93, 64.13, 64.56, 64.52, 64.83, 64.57, 63.21, 64.72, 64.93, 64.6, 64.3, 63.74, 65.67, 65.68, 65.36, 65.04, 65.46, 64.46, 66.19, 66.3, 65.67, 65.43, 65.43, 65.5, 66.2, 65.12, 66.81, 66.3, 66.06, 66.01, 65.35, 65.65, 65.5, 65.59, 66.17, 65.82, 66.18, 65.83, 66.46, 66.77, 66.67, 65.95, 66.25, 66.97, 66.51, 66.3, 65.88, 66.01, 66.37, 65.89, 67.2, 66.74, 67.0, 66.29, 66.65, 67.1, 66.96, 65.74, 67.51, 67.48, 67.3, 67.04, 66.26, 67.06, 66.63, 66.38, 66.67, 67.53, 66.7, 67.3, 67.5, 67.57, 68.33, 66.92, 67.51, 67.14, 67.55, 67.99, 66.96, 67.45, 67.36, 67.64], 'val_loss': [3.629621976500104, 3.2983606347612513, 2.9094836043704086, 2.5176669404764844, 2.3667392085312278, 2.246840336520201, 2.714585828173692, 1.8464407199507307, 2.1069708553848754, 1.543465340972706, 1.674411752421385, 1.4269663399192178, 3.4120706023683973, 2.2828454310726967, 1.711041644120672, 2.8944761153239353, 2.5713485748904525, 2.7676665035023054, 1.5344351761659998, 1.5816935445093045, 1.6272044033761237, 1.6308952983777234, 1.693472610917061, 1.7101698992358652, 1.7962550517100437, 1.8119668611295663, 1.7680265971809437, 1.825585286328747, 2.2234135129649166, 1.945175627234635, 1.8082946390862678, 1.800106984414872, 1.8919261173837503, 1.8851486584942811, 2.045350804450406, 2.024097051969759, 1.9262148578455494, 1.9001546995654988, 1.8803945426728315, 1.9540696903398842, 1.945865278790711, 2.0417639389159574, 1.940867576629493, 2.0408855813324074, 1.9206451033331027, 2.0145898161420397, 2.0612271162355023, 2.070667194333046, 2.118390523703994, 2.0827157391104727, 2.072125102304349, 2.05533478745989, 2.0294279547253993, 2.1187328616524956, 2.051721196265737, 2.111307325256858, 2.041116480614729, 2.0584045591627715, 2.010542738209864, 2.106669479874289, 2.061238909223277, 2.0259279529000542, 2.1096003397255187, 2.0351977724178583, 2.0745483784918575, 2.1808919496596997, 2.0997982765458953, 2.1866378427311113, 2.102871632120412, 2.105599118266136, 2.0422414104649973, 2.1642429202225557, 2.1160607542961265, 2.1237364210140934, 2.144611278536973, 2.1798223621526343, 2.0302685772537425, 2.1162389000509956, 2.0858789443210433, 2.1577298975294563, 2.2538786416600463, 2.12529941425202, 2.1852418210855715, 2.2193130474940985, 2.175701801184636, 2.1980533106311873, 2.110032906577845, 2.174358116213683, 2.2134300861389016, 2.1417757280313285, 2.012559291663443, 2.1539985286961696, 2.0881272114006575, 2.1370601232644098, 2.111134154401767, 2.120379275197436, 2.1872430394409568, 2.20698237039481, 2.21096085059415, 2.2079217881913396]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal 2", optimizer="SGD")
data.update({'train_metric': [2.298, 4.456, 6.108, 7.142, 8.486, 9.804, 10.906, 12.16, 13.518, 14.434, 15.426, 16.358, 17.206, 17.972, 18.916, 19.576, 20.52, 21.658, 22.212, 23.186, 23.778, 24.412, 25.312, 26.05, 26.844, 27.6, 28.14, 28.816, 29.334, 30.074, 30.684, 31.258, 31.954, 32.35, 32.882, 33.306, 34.008, 34.562, 35.068, 35.464, 36.172, 36.698, 37.262, 37.418, 38.108, 38.622, 38.964, 39.41, 39.984, 40.346, 41.212, 41.566, 41.732, 42.47, 42.928, 43.546, 43.55, 44.106, 44.566, 44.928, 45.54, 45.986, 46.434, 47.076, 47.53, 47.706, 48.348, 48.374, 49.112, 49.562, 49.836, 50.518, 51.11, 51.262, 51.842, 52.178, 52.698, 53.176, 53.634, 53.908, 54.442, 54.696, 55.452, 55.92, 55.994, 56.714, 57.256, 57.45, 58.282, 58.438, 59.084, 59.324, 59.7, 60.308, 60.816, 61.014, 61.696, 62.052, 62.468, 62.622], 'train_loss': [4.518145558779197, 4.366277572139145, 4.234608295018716, 4.135199695596914, 4.039503209121392, 3.9456307619733884, 3.864962026286308, 3.7888606262328985, 3.7122905153752592, 3.655932342609786, 3.5935725385270767, 3.5398882176260202, 3.487532168398123, 3.439081699342069, 3.3965108239132427, 3.3474581872715667, 3.3031814561780455, 3.25684927613534, 3.220704439960782, 3.17859621456517, 3.1363284965915144, 3.098075368215361, 3.061216670229002, 3.022751055410146, 2.9846674161189046, 2.945414832181028, 2.9132841285842153, 2.8790552716730806, 2.8471750871604664, 2.811320258833258, 2.784054254326979, 2.7561605537638947, 2.7206199047205697, 2.693931000007083, 2.6659345004869546, 2.6379556643688464, 2.6066038253362223, 2.5869310600373447, 2.560984199309288, 2.5373039186153266, 2.507711351984907, 2.4847545815855647, 2.459705021832605, 2.4425155079883076, 2.4126568922911154, 2.387871476238036, 2.372032467514048, 2.345797864372468, 2.324514372574399, 2.299726173396001, 2.274891344025312, 2.257511039981452, 2.2344897537280226, 2.2167358680454363, 2.1897104169096786, 2.1699091543626907, 2.1557367069032187, 2.133155900194212, 2.1108008568244214, 2.0925921197132684, 2.0713175413248788, 2.0591457187367217, 2.0281432085025037, 2.0105900993127652, 1.9894490679511634, 1.9753738865827966, 1.9507253188306413, 1.9353462139054027, 1.9153364158957207, 1.8955817875045036, 1.8756160861086053, 1.8534175047789083, 1.8330703585044197, 1.819169529106306, 1.7962353273730753, 1.7784203969304213, 1.7607110545153508, 1.7416062223941773, 1.7234806069327742, 1.709937611504284, 1.6876016570174175, 1.67346371073857, 1.6467232664527796, 1.6322885230374153, 1.616776239993932, 1.5965157528515057, 1.5742311628578265, 1.5635090346836373, 1.5391478290033462, 1.5294255745380432, 1.5052914796277994, 1.4915742550969429, 1.473092436942908, 1.4543615578080686, 1.4374128169263416, 1.4275510096946336, 1.4024991587452267, 1.3873092083217542, 1.3702873309402515, 1.3590397542090062], 'val_metric': [3.37, 5.25, 6.66, 7.95, 9.43, 10.55, 11.3, 12.83, 14.51, 14.96, 15.43, 16.85, 17.79, 18.57, 19.17, 20.29, 21.42, 21.81, 22.71, 23.32, 24.04, 25.07, 24.35, 25.68, 25.97, 26.68, 27.28, 27.36, 28.0, 29.5, 28.89, 29.73, 29.97, 30.58, 29.92, 31.58, 32.09, 32.16, 32.91, 32.56, 33.09, 33.21, 33.97, 33.99, 34.92, 34.92, 35.13, 34.45, 36.06, 36.77, 36.65, 36.54, 36.51, 38.01, 37.66, 38.06, 37.78, 36.83, 36.26, 39.51, 39.69, 39.73, 39.69, 38.99, 39.97, 40.77, 40.68, 40.68, 40.87, 41.35, 40.28, 41.48, 41.6, 39.63, 42.42, 42.81, 42.9, 42.19, 41.94, 40.26, 43.16, 43.45, 41.93, 44.56, 43.73, 43.22, 44.91, 45.03, 43.48, 42.65, 44.81, 44.6, 46.26, 43.67, 43.19, 46.7, 43.99, 45.65, 45.09, 45.59], 'val_loss': [4.454615289238608, 4.283888769757216, 4.17425958669869, 4.074751615524292, 3.9765870525578784, 3.900530364103378, 3.8141778745469015, 3.7451243385387833, 3.674727567441904, 3.6185566088196577, 3.5714725172443753, 3.517197640838137, 3.4676832044200534, 3.4214725114737345, 3.3856223540700925, 3.34060020203803, 3.286842381118969, 3.2573651599276596, 3.2125230731478163, 3.179671550252635, 3.1538265981491964, 3.104155048443254, 3.093534548571155, 3.0446139232368226, 3.0201109427555353, 2.9883557823812885, 2.957774806174503, 2.940939971595813, 2.9218733766276364, 2.869428127434603, 2.851928706381731, 2.827017621629557, 2.8313868379896614, 2.785734688400463, 2.812941473760423, 2.742401094193671, 2.7210072256197595, 2.7053408835344253, 2.6773590631545727, 2.6883659423536557, 2.6431842898107636, 2.62828948087753, 2.6264700692170746, 2.604188721650725, 2.572939827184009, 2.5605459031026077, 2.567296962069858, 2.582177063462081, 2.509603561109798, 2.5003493515549193, 2.5080461266693796, 2.4971966978850637, 2.4994671458651307, 2.4325908133938055, 2.433358027676868, 2.416065323125025, 2.4399594903751542, 2.4709743572648164, 2.5390539723596754, 2.3725225705250055, 2.344713700804741, 2.354723972120103, 2.3736603070216575, 2.3908648156816033, 2.327531777369748, 2.314055474700442, 2.285558235873083, 2.3045664448646983, 2.312255705238148, 2.2639354672401573, 2.3245327875113033, 2.2740895649430097, 2.237662439893006, 2.3708295427310238, 2.2484855621483675, 2.216243907904169, 2.2254196261144745, 2.2420760127389507, 2.256545077463624, 2.3430636384684567, 2.1950353809223055, 2.198359984501152, 2.2831573828010803, 2.147558673931535, 2.1954874278633456, 2.220992543894774, 2.167034912261234, 2.1442381325800706, 2.2063176890087735, 2.2633177429247815, 2.173641211667638, 2.194461981202387, 2.1111830442574373, 2.230693937866551, 2.263973460835256, 2.1001538364750565, 2.2282118379690083, 2.113625196894263, 2.1402832338005116, 2.1242501910325067]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal 2", optimizer="Adam")
data.update({'train_metric': [9.254, 18.248, 24.834, 31.314, 36.258, 39.378, 42.276, 44.234, 45.956, 47.392, 48.534, 49.602, 50.634, 51.286, 52.578, 52.992, 53.624, 54.218, 54.9, 55.4, 55.688, 56.086, 56.604, 57.018, 57.336, 57.556, 57.984, 57.75, 58.426, 58.742, 58.844, 59.308, 59.472, 59.686, 60.126, 59.92, 60.342, 60.254, 60.772, 60.716, 61.044, 61.332, 61.324, 61.556, 61.478, 61.68, 61.852, 62.208, 61.922, 62.216, 62.088, 62.594, 62.764, 62.596, 62.392, 63.072, 62.566, 62.852, 63.336, 63.138, 63.064, 63.168, 63.396, 63.366, 63.644, 63.682, 63.818, 63.742, 63.736, 64.26, 63.892, 63.87, 64.082, 64.166, 64.42, 64.22, 64.464, 64.236, 64.706, 64.466, 64.522, 64.88, 64.83, 64.948, 64.84, 64.93, 64.942, 64.948, 65.288, 64.87, 65.132, 65.234, 65.414, 65.198, 65.324, 65.174, 65.48, 65.434, 65.536, 65.656], 'train_loss': [4.000096704344005, 3.374604093449195, 2.9838338520215903, 2.6351412382272197, 2.40741030365, 2.250818499976107, 2.137194215031841, 2.047177097376655, 1.976451357154895, 1.9105182511117451, 1.867659866809845, 1.818477976505104, 1.7791532308549223, 1.744584312524332, 1.7103872498892763, 1.6871525158967509, 1.6597876815539796, 1.640545463089443, 1.6174479457728392, 1.5985542115043192, 1.5761024883336119, 1.5621217993824073, 1.5419800535339834, 1.5299967928310794, 1.512795680929023, 1.5016315986433297, 1.4876520787663472, 1.4862694943042667, 1.4659300305501883, 1.4604842987511775, 1.4517285012070784, 1.4429840434085377, 1.433911185740205, 1.4270170129778441, 1.4144753244374415, 1.4116353173085185, 1.4000549039724843, 1.399400964722304, 1.3847229811541564, 1.384358508812497, 1.372867082695827, 1.3644231757544496, 1.3604364243462264, 1.3534813998147959, 1.3548059945216264, 1.3482200338712433, 1.342277863751287, 1.3337988440337998, 1.3393471478806127, 1.3264322705433498, 1.327622739783943, 1.3103566676606913, 1.310613455446175, 1.3125144772212525, 1.313717547051437, 1.3009135380120533, 1.2986039562755838, 1.300360755237472, 1.2903271200102004, 1.2869927764243787, 1.2837146208109453, 1.2866582349895517, 1.2837736605073484, 1.279405878861542, 1.2744628066754402, 1.2692642882656868, 1.2644974244067737, 1.2643394496892115, 1.2647353216357853, 1.2529836785610375, 1.255474924964978, 1.256360628492082, 1.251269750430456, 1.2471341176715958, 1.2438989572055505, 1.2378103439612766, 1.2418869586704333, 1.2436375867985094, 1.2303627048764387, 1.2317494004583724, 1.2327564189501126, 1.2274883827742409, 1.2260832220239712, 1.2186729426274214, 1.2201602819478115, 1.2225826509163509, 1.2213639115433559, 1.2206266617683499, 1.2123767639822363, 1.2147103374266564, 1.218944366173366, 1.2146571382232334, 1.2079095890759812, 1.2103531565660102, 1.2034232928929731, 1.2121156844336662, 1.2027958730602508, 1.1997337595885977, 1.2020531872959088, 1.198346670173928], 'val_metric': [12.88, 19.73, 24.37, 31.9, 35.99, 37.61, 39.47, 42.71, 45.69, 43.66, 44.07, 46.57, 48.52, 49.09, 47.65, 50.27, 50.61, 51.87, 51.1, 51.72, 52.62, 53.63, 54.29, 50.23, 51.63, 51.46, 53.16, 54.92, 53.72, 54.01, 54.02, 53.83, 55.91, 55.24, 54.94, 56.99, 55.71, 55.74, 55.3, 55.7, 56.56, 55.9, 55.98, 56.59, 55.76, 57.41, 57.19, 55.94, 57.25, 54.88, 57.47, 56.32, 58.84, 55.31, 57.41, 54.52, 56.28, 55.73, 56.71, 56.2, 57.04, 56.54, 55.74, 57.99, 55.12, 54.73, 58.06, 55.55, 58.27, 58.21, 57.97, 57.59, 55.69, 58.34, 59.2, 58.77, 58.69, 59.05, 57.11, 57.32, 57.77, 57.05, 58.63, 59.42, 57.0, 55.96, 56.43, 57.28, 56.04, 58.55, 58.42, 58.34, 59.43, 57.86, 56.47, 57.99, 58.91, 57.88, 58.72, 59.19], 'val_loss': [3.697788387347179, 3.301493447297698, 2.9690006007054808, 2.617601681666769, 2.4474432893619418, 2.3240840488178716, 2.263760884096668, 2.1101464731678083, 1.9828107493698217, 2.0373677584775693, 2.03869322075206, 1.9395392130894267, 1.870525855926951, 1.8418218246690787, 1.8904364678510435, 1.7893518452431745, 1.7757107512966084, 1.7395516102481041, 1.754183325038594, 1.7030119402393413, 1.7127282870043614, 1.6614889878376273, 1.627252407894013, 1.7798622960497619, 1.7575617360461289, 1.73337923189637, 1.6869336154050887, 1.6047615617703481, 1.670875292674751, 1.650768668028959, 1.6545184341965207, 1.6547677061360353, 1.5751517027806325, 1.6055186015025826, 1.607077750430745, 1.5484430231865804, 1.580940074981398, 1.5668155585124994, 1.602784327640655, 1.570033667953151, 1.571608538840227, 1.5898697178834562, 1.5761603959806405, 1.5436183905145924, 1.561610814492414, 1.5055714401469869, 1.520829326787572, 1.560285916373988, 1.5196035432208115, 1.6258155821235316, 1.5067491037830425, 1.575436707514866, 1.4791503795392953, 1.5914493670129473, 1.5197923137883471, 1.6553949542865631, 1.5615797847699209, 1.596888120007363, 1.5467774010008308, 1.5680375175111612, 1.526883902063795, 1.5347102880477905, 1.5824533674367673, 1.49543392164692, 1.590294665591732, 1.6202432357581558, 1.4920621381443777, 1.594219478831929, 1.499469029295976, 1.530849866047027, 1.494498910417982, 1.4969034825161005, 1.5644655227661133, 1.47652631428591, 1.4427753208549159, 1.4421373522205718, 1.4801955086410425, 1.4636582280420194, 1.5423334958447013, 1.5043976967501793, 1.4859385737188302, 1.5260454036627606, 1.4695842573597173, 1.4430248631033928, 1.5388722131206731, 1.6090077290869063, 1.57392172448954, 1.5513569184928944, 1.563319773431037, 1.4855012050859489, 1.4718231570189166, 1.4784382991730027, 1.420911398662883, 1.5264819814900683, 1.5959472580320517, 1.4917983203936533, 1.4682002986312672, 1.5078172509078007, 1.466230248569683, 1.4577847309173293]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal 2", optimizer="AdamW")
data.update({'train_metric': [8.07, 17.49, 24.484, 33.262, 41.704, 47.31, 51.648, 56.802, 60.616, 64.59, 67.702, 70.796, 73.652, 76.33, 78.864, 81.352, 83.96, 85.826, 87.582, 89.372, 90.338, 91.57, 92.496, 93.332, 94.086, 94.236, 94.794, 95.348, 95.37, 95.854, 96.356, 96.176, 96.474, 96.65, 96.758, 96.984, 97.262, 97.08, 97.284, 97.526, 97.458, 97.666, 97.878, 97.696, 97.89, 97.912, 98.238, 97.822, 98.124, 98.106, 98.258, 98.212, 98.254, 98.512, 98.22, 98.484, 98.66, 98.196, 98.7, 98.614, 98.706, 98.618, 98.542, 98.83, 98.702, 98.63, 98.782, 98.868, 98.622, 98.96, 98.734, 98.956, 99.05, 98.62, 99.078, 98.876, 98.884, 99.042, 98.95, 99.038, 98.982, 99.062, 99.076, 98.868, 99.132, 99.104, 99.12, 99.074, 99.07, 99.208, 99.028, 99.256, 99.062, 99.374, 99.188, 99.068, 99.11, 99.362, 99.282, 99.028], 'train_loss': [4.121554567076056, 3.442722308056434, 3.0588508130949172, 2.5935024620626894, 2.1876782697180044, 1.949730718837065, 1.7449387275349453, 1.5239366244172197, 1.365520294669949, 1.2213821599398123, 1.098566095466199, 0.9820694697025182, 0.870114212565105, 0.779463123444401, 0.6796023557367532, 0.5925189178160695, 0.5011223826528816, 0.4375452775403362, 0.3780491737758412, 0.322949259158443, 0.2946532059489461, 0.25297795097960535, 0.22571958243237128, 0.2026653449453623, 0.17950055437624607, 0.17489914704219003, 0.15667311699770373, 0.14287244623093426, 0.140713042539099, 0.1261922843477991, 0.1143965300558435, 0.11656225883685377, 0.10822597466399679, 0.10272593317610566, 0.10012281121200194, 0.093949636950603, 0.0868917258773142, 0.09137060857304108, 0.08441437480797338, 0.0774001467971803, 0.07779226331230815, 0.07287426362626369, 0.06759762591616872, 0.07123560188999138, 0.0663584543795794, 0.0668477362207592, 0.05641166357587918, 0.06635773856588223, 0.06060780509603698, 0.05941258432627048, 0.053916818874380774, 0.058306640181827414, 0.054452232768326815, 0.04845449316215313, 0.055247460127822445, 0.04800452078264469, 0.04332393608883719, 0.05733486658374272, 0.041135917860142826, 0.04395693088682187, 0.04280274464895287, 0.04352190765514112, 0.04745994649617278, 0.03919998085265622, 0.04141298975528139, 0.043142404195869254, 0.03767797236716079, 0.03742843979979744, 0.04278180702878853, 0.03369882868265238, 0.038881858281256475, 0.03339386112620885, 0.031052899184425973, 0.04337827261725483, 0.029434389481574053, 0.035228454320536104, 0.03665247441181923, 0.03169650285627724, 0.03397428298034393, 0.030936782341286696, 0.03306323042630082, 0.028277921766522662, 0.03002010674409085, 0.03603903534921908, 0.028738188622233547, 0.029547582027431497, 0.028298842856009036, 0.029761824344829393, 0.029183218054517464, 0.025826644647093562, 0.030614733658990017, 0.02336537778877583, 0.031222675940679276, 0.020781474682288675, 0.026416745886843486, 0.02893135367791154, 0.028818952582959356, 0.020593706306645913, 0.023018960161563344, 0.030164963244880334], 'val_metric': [13.81, 14.97, 26.71, 37.96, 42.7, 47.94, 52.99, 54.06, 55.33, 58.08, 60.64, 59.82, 61.55, 62.98, 63.78, 63.11, 63.79, 63.89, 63.81, 62.59, 64.79, 64.47, 65.64, 64.76, 65.08, 64.94, 65.94, 64.8, 64.91, 65.85, 65.79, 65.01, 65.18, 65.6, 65.92, 65.52, 65.85, 65.89, 65.72, 66.35, 66.26, 67.18, 66.4, 65.19, 66.31, 66.11, 65.12, 65.67, 66.05, 66.8, 67.1, 67.11, 66.76, 67.09, 65.68, 66.98, 66.34, 66.44, 66.74, 66.81, 67.34, 66.66, 66.5, 67.54, 66.53, 66.94, 67.04, 66.77, 67.19, 67.55, 67.31, 67.53, 67.12, 66.68, 67.46, 66.33, 66.7, 65.79, 67.78, 67.69, 67.72, 67.2, 66.84, 67.73, 66.06, 68.69, 68.0, 68.31, 66.66, 67.72, 68.74, 68.19, 68.17, 66.84, 66.25, 66.66, 68.69, 68.19, 66.11, 67.18], 'val_loss': [4.165807717924665, 3.7288827668329714, 2.948735859743349, 2.356634399693483, 2.314204860644735, 1.9397860932502018, 1.7033009081129815, 1.6833987501776142, 1.6265113892828582, 1.506497946134798, 1.421564889181951, 1.4834644931137182, 1.4206984232945048, 1.3860218741331891, 1.369459127924245, 1.427451705097393, 1.4857204484332138, 1.5146840028702073, 1.5734738863197861, 1.8476472584305295, 1.6152047583252003, 1.6894693549271602, 1.6020547318610416, 1.692935798957849, 1.6571297489913406, 1.7698716441537166, 1.750616729259491, 1.8431439319993281, 1.8391085718847384, 1.7966950683836724, 1.801675445714574, 1.9497002226531885, 1.9317222279348192, 1.8936786085936674, 1.8723191831521928, 1.9107630480626585, 1.8953476472265403, 1.9295332317898988, 1.9366647604924099, 1.922840558799209, 1.9101657563713705, 1.8399473888099573, 1.9653935861435665, 2.04218693219932, 1.9458532781357978, 1.973035969551961, 2.0948365573670453, 2.1009704087190566, 2.0186407619221196, 1.9754991926205385, 1.9154135273520354, 1.9476685675845784, 2.042623938648564, 1.938225782980585, 2.1186525043408584, 1.9543661137295376, 2.0979664591467304, 2.019089019981919, 2.0504146681469715, 2.0657301097159175, 2.013486395975587, 2.04772968049262, 2.030540487568849, 1.9862867400145074, 2.127026646380212, 2.064991386832705, 2.060913991776242, 2.0475304711396527, 2.0255854794174244, 2.028392879826248, 2.091288934088057, 2.0538496014418874, 2.0614138712548904, 2.0995273711575066, 2.007727192845314, 2.193382661813384, 2.0922557176298398, 2.195368224268506, 2.025530096831595, 2.0510045859464414, 2.0460436814909526, 2.0865443869001545, 2.1440303253520065, 2.0768643723931284, 2.213977316002937, 1.9751027575723685, 2.0988671703703083, 2.051317283302356, 2.1997871702643716, 2.1625205711194666, 1.9802024159461828, 2.0912651499365547, 2.0377590663873466, 2.1820152937226993, 2.250312342765225, 2.1570108202612324, 2.0424824884742687, 2.1068999736931673, 2.3069122008457303, 2.1770604522364914]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal", optimizer="SGD")
data.update({'train_metric': [3.154, 6.84, 9.858, 12.418, 15.02, 17.184, 19.126, 21.064, 22.66, 24.506, 25.962, 27.47, 28.914, 29.95, 31.44, 32.654, 33.448, 34.684, 35.494, 36.676, 37.75, 38.692, 39.772, 40.366, 41.416, 42.354, 43.122, 43.938, 44.636, 45.338, 46.366, 47.108, 47.66, 48.534, 49.304, 49.802, 50.588, 51.208, 52.332, 52.692, 53.584, 54.284, 54.77, 55.202, 55.786, 56.888, 57.256, 58.156, 58.818, 59.088, 59.748, 60.628, 61.426, 61.778, 62.544, 62.96, 63.646, 64.236, 64.592, 65.47, 65.88, 66.688, 67.156, 67.982, 68.71, 69.058, 69.262, 70.128, 70.592, 71.044, 71.626, 72.312, 72.554, 73.508, 74.086, 74.584, 75.074, 75.46, 76.302, 76.786, 77.3, 77.874, 77.842, 78.654, 79.05, 79.798, 80.306, 80.718, 81.096, 81.684, 81.992, 82.618, 83.21, 83.51, 84.098, 84.79, 84.95, 85.272, 85.75, 85.794], 'train_loss': [4.498269948203241, 4.14817888444037, 3.9278742358507706, 3.769166477500935, 3.62757148035347, 3.5043405913331016, 3.39456301089138, 3.293765455255728, 3.1940952938840823, 3.102882693490714, 3.021379822355402, 2.943459975445057, 2.8718799127032386, 2.806153077908489, 2.746308752642873, 2.6902133799574868, 2.63680976217665, 2.5795042941637356, 2.539705407588988, 2.486651664652178, 2.4400172314375563, 2.398308402894403, 2.3540970129735026, 2.3108252823505255, 2.2716632133249735, 2.2327318606169326, 2.194250399346852, 2.160448795541778, 2.1265124029210765, 2.092238830178595, 2.0530816060502817, 2.0233851277919683, 1.9887294968985536, 1.957925078509104, 1.929851060785601, 1.9018970776701827, 1.8681973010072928, 1.8409207057769952, 1.8064481026071417, 1.7802829644869052, 1.7536712700448682, 1.7240622125928053, 1.6959414006499074, 1.6722701637031476, 1.646457218300656, 1.6124644870953182, 1.5901736578977932, 1.5627603357100426, 1.5377558329526115, 1.5158464131147966, 1.4914981847833795, 1.4630942141156063, 1.4389237638782053, 1.4217457742337376, 1.392248594471256, 1.3653112327503731, 1.3438757042140912, 1.3190992532483756, 1.3018210950257527, 1.27589933113064, 1.2559482531474375, 1.2274359496658112, 1.2139863813167338, 1.1817463363527947, 1.156793926454261, 1.1400580563965965, 1.128497635845638, 1.0975965815584372, 1.0812936949607965, 1.0601437390612825, 1.0344535151253575, 1.0197341892572924, 0.9999152332772989, 0.9748648511021948, 0.9567270060177044, 0.9365654949032133, 0.9231724416660836, 0.9021949937276523, 0.8809123200452541, 0.8608776938427439, 0.841156115845951, 0.8224642758479204, 0.8092574259966535, 0.7893026476854559, 0.7752689131919075, 0.7525583870727998, 0.7337009704783749, 0.7176196339261501, 0.703318284791144, 0.6875787514173771, 0.6748257362857804, 0.6537793476868163, 0.6377963994622535, 0.626119310593666, 0.6094486690543192, 0.5881805063208656, 0.5760789406497765, 0.5683441744436084, 0.5557412640441715, 0.5451900837061655], 'val_metric': [5.01, 7.76, 11.41, 13.71, 16.41, 18.33, 20.62, 22.08, 23.77, 23.42, 25.83, 25.26, 28.04, 29.31, 29.84, 31.24, 32.06, 32.23, 33.44, 34.47, 35.49, 34.54, 36.34, 36.96, 37.57, 38.05, 39.3, 38.64, 39.64, 40.35, 38.25, 41.52, 41.7, 42.49, 43.8, 44.06, 42.35, 41.32, 44.44, 45.78, 44.13, 44.9, 45.66, 46.58, 46.05, 47.14, 47.8, 47.3, 47.34, 48.03, 44.91, 45.79, 48.97, 46.61, 47.82, 47.72, 49.13, 49.07, 49.47, 49.94, 47.38, 48.73, 49.26, 39.47, 49.44, 50.67, 49.5, 49.81, 51.03, 50.56, 51.18, 50.4, 51.45, 44.86, 49.77, 50.56, 45.96, 50.97, 47.49, 50.42, 51.3, 30.87, 51.02, 49.91, 51.16, 51.23, 50.3, 51.38, 51.98, 49.63, 50.94, 52.09, 52.8, 49.11, 51.22, 47.23, 50.06, 41.01, 49.7, 50.68], 'val_loss': [4.283868492029275, 4.0367487585468655, 3.8326736726578634, 3.6964415258662715, 3.5585112526158618, 3.4393925985712914, 3.3468633518097506, 3.2564330799564436, 3.1643582468579528, 3.143708614786719, 3.0298640727996826, 3.055756439828569, 2.898303663654692, 2.8386731041464834, 2.808760202614365, 2.7428433393976492, 2.6962978050207638, 2.6729638728366534, 2.61891049032758, 2.5860382599435794, 2.532003707946486, 2.5444367114146043, 2.5022826422551634, 2.439753354734676, 2.4180018939789694, 2.421351676533936, 2.35863929948989, 2.4076390106966543, 2.3408994249477506, 2.301126320650623, 2.3909770804605666, 2.243835533500477, 2.252312042910582, 2.2179555619598195, 2.1676538332252746, 2.1635707966081656, 2.201246386880328, 2.2656148360793, 2.117393686513233, 2.076812683397038, 2.158826816613507, 2.120589955597167, 2.074542138986527, 2.020380533424912, 2.063675228957158, 1.997198143582435, 1.992374423203195, 2.0198086181264014, 2.0093371260697674, 1.976036613913858, 2.159723270470929, 2.111146904101038, 1.9467993959499772, 2.031532545757901, 2.0074354212754852, 1.9624943611728158, 1.9432028691480114, 1.9450460801458662, 1.9194187783891228, 1.9050469368126741, 2.006134465241888, 1.9713159298441212, 1.9478434483716442, 2.8463302752014936, 1.9301824934163672, 1.8921632318739678, 1.950727508326245, 1.9447054096088288, 1.909475709981979, 1.9156789847999622, 1.8825441006642238, 1.9221804817770696, 1.8694559449602843, 2.5884590968964205, 1.9671915314000123, 1.963635056641451, 2.3611259748981257, 1.9162672302525514, 2.2126321762230745, 1.9730261328873362, 1.9154882195648875, 4.66035383066554, 1.9125669040497701, 2.0348181306936177, 1.9455585760675418, 1.9486488012751197, 2.0154150367542436, 1.9198643037468006, 1.8911578837473682, 2.144263673739828, 1.9635905824649107, 1.9466624426993595, 1.8744556987361543, 2.1366493922130316, 2.028004920406706, 2.2557048600190766, 2.0912386642140186, 3.310771079579736, 2.126379142900941, 2.067124561139732]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal", optimizer="Adam")
data.update({'train_metric': [12.912, 26.388, 33.74, 37.766, 41.446, 43.992, 46.194, 47.484, 49.834, 50.968, 52.55, 53.904, 54.52, 55.402, 56.724, 57.346, 57.618, 58.514, 59.022, 59.366, 60.106, 60.29, 60.744, 61.044, 61.322, 61.698, 62.262, 62.188, 62.948, 62.914, 63.176, 63.316, 63.64, 63.824, 64.01, 64.044, 64.394, 64.39, 64.348, 64.732, 64.89, 65.282, 65.302, 65.314, 65.726, 65.63, 65.816, 65.892, 66.056, 66.016, 66.118, 66.108, 66.632, 66.386, 66.534, 66.678, 66.916, 66.898, 67.07, 67.016, 67.04, 67.36, 67.114, 67.084, 67.36, 67.438, 67.88, 67.678, 67.602, 67.698, 67.852, 67.54, 67.858, 68.104, 67.784, 67.9, 67.974, 68.262, 68.224, 68.102, 68.272, 68.602, 68.244, 68.414, 68.39, 68.418, 68.608, 68.39, 68.414, 68.906, 68.542, 68.774, 68.8, 68.876, 68.738, 68.954, 68.766, 68.748, 68.614, 69.062], 'train_loss': [3.7765114026911117, 2.884180851299744, 2.5243042539757536, 2.3230205451131174, 2.1743936651503035, 2.0556052936922256, 1.9635283064354412, 1.8887431069713114, 1.8108148561109363, 1.7594721989558482, 1.704087481321886, 1.6509367354080806, 1.6174389947100978, 1.584561019297451, 1.540287816158646, 1.5187843984655103, 1.4967328495991505, 1.4648934084436167, 1.4487534838411815, 1.4292737341597868, 1.4141607876781308, 1.400281181192154, 1.383351459984889, 1.367081568445391, 1.3525602732168134, 1.3377027509310055, 1.327048691642254, 1.3216663638649084, 1.3046010653381153, 1.3049488693399502, 1.2949446714900035, 1.2847980282953024, 1.2746990802495375, 1.2712157219267257, 1.2616877981159083, 1.2526918050578184, 1.2466455816155504, 1.2449579145902259, 1.2407968764567314, 1.2247525179934928, 1.2216657878797683, 1.2170761684932367, 1.2178287183689644, 1.2087196619309428, 1.2001376870800466, 1.2021662964083044, 1.191482763080036, 1.181351231187201, 1.184064427147741, 1.1829108449504198, 1.181571717533614, 1.1769293141182122, 1.1695019588293627, 1.1711612198968677, 1.167727913393084, 1.1635937346979175, 1.160645864122664, 1.159545110772028, 1.149170836676722, 1.1497321312537279, 1.1512392664047153, 1.1404335068924654, 1.1432663482016006, 1.1387840090200418, 1.1352669182793258, 1.1371693406873347, 1.126992677255055, 1.1273248357236232, 1.1270933950038822, 1.1218125158563599, 1.1213482270002975, 1.1216075525564306, 1.1196066328631642, 1.1108838483653105, 1.1201228779142776, 1.1101898550225036, 1.1091742494222148, 1.1020212575907598, 1.1059222648210842, 1.1109764832822258, 1.1038641300805085, 1.1013014283021698, 1.0983998580356997, 1.0989871052524927, 1.0946508620858497, 1.098864595908338, 1.0934052789760063, 1.0991039210573181, 1.0908130260989488, 1.0859073057503956, 1.0903946254259484, 1.0836811921633114, 1.0839196186693734, 1.085425683223378, 1.0860338363501116, 1.0870306552828426, 1.0846916065191674, 1.0824953129376902, 1.0857281091115665, 1.0758141754075998], 'val_metric': [20.43, 29.48, 31.65, 34.94, 39.19, 39.92, 43.75, 45.24, 46.63, 47.82, 52.07, 48.08, 52.43, 54.41, 52.6, 54.34, 55.1, 55.12, 55.91, 55.7, 49.27, 54.53, 54.93, 55.23, 56.35, 57.37, 57.28, 56.4, 58.38, 58.33, 56.83, 59.1, 58.87, 57.52, 58.06, 58.53, 60.29, 59.81, 59.62, 57.99, 58.62, 60.07, 59.33, 59.18, 59.55, 61.06, 57.67, 59.51, 59.34, 58.76, 59.71, 60.65, 60.92, 61.64, 61.5, 60.09, 58.48, 60.47, 60.75, 61.45, 60.73, 57.73, 61.34, 60.03, 61.21, 60.31, 61.73, 59.64, 60.92, 60.41, 60.37, 59.34, 62.12, 62.83, 61.97, 59.3, 61.23, 60.71, 59.58, 62.57, 61.13, 60.42, 60.03, 61.61, 59.76, 60.14, 62.66, 61.1, 61.91, 61.52, 61.77, 61.43, 60.99, 61.76, 58.92, 60.14, 61.58, 61.77, 60.19, 61.96], 'val_loss': [3.1810472831604586, 2.7077645784730366, 2.637407527607717, 2.4590001296085915, 2.2524262423727923, 2.225782936545694, 2.0937823710168244, 1.9723566846483072, 1.9286400785871372, 1.8781702480498392, 1.7208286675677937, 1.8797851967963444, 1.6884945251379804, 1.615155849487159, 1.6711927872554513, 1.6498985609431176, 1.6138750747510582, 1.58851158618927, 1.5573086594320407, 1.5697496237268873, 1.8758707023729944, 1.6414324250190881, 1.597010620840036, 1.5894563699224193, 1.5546409370033605, 1.4912410822643596, 1.5166301252735648, 1.5290271537319111, 1.4678854919542932, 1.4754129743120472, 1.5233053117041375, 1.4301528216926915, 1.4456998071852762, 1.4885197118589073, 1.4587260674519145, 1.4867299567362307, 1.4033859694839284, 1.4165934722894316, 1.419035083549038, 1.4851718153923181, 1.4969675893996173, 1.422345735844533, 1.422374786085384, 1.423412933850744, 1.4296207644377545, 1.3927352451215125, 1.4884971042347561, 1.450887346723277, 1.4496583824704408, 1.4514171025555604, 1.4184165000915527, 1.3802541129907984, 1.4008678953359082, 1.3541168236428764, 1.3621598846593481, 1.3904494115501453, 1.4909028536195208, 1.3668743956620526, 1.3759393692016602, 1.333388396509134, 1.376678809238847, 1.5230530826908768, 1.3680448987681395, 1.400366662793858, 1.3678222032868939, 1.420984571147117, 1.3495696149054606, 1.4404120581924535, 1.3713313424663178, 1.410863915826105, 1.412999741211059, 1.455208167908298, 1.3389313433580339, 1.3059809287642217, 1.3313834279965444, 1.4482009334928672, 1.3703640179269632, 1.366036802340465, 1.4297811054879692, 1.329803967172173, 1.3854151637690841, 1.3981177916952, 1.4219085824717381, 1.3539914242021598, 1.4376144272506617, 1.4114736872873488, 1.3327714926118304, 1.3877485147707023, 1.3551547257763565, 1.367159358255423, 1.3446208557505517, 1.3403874666068205, 1.367993255709387, 1.3347788749227099, 1.4733820823347492, 1.403905875363927, 1.3575223881727572, 1.3511522507211964, 1.4099963299787728, 1.339908645411206]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal", optimizer="AdamW")
data.update({'train_metric': [14.084, 30.53, 41.57, 49.462, 55.27, 59.848, 63.688, 66.99, 70.602, 73.226, 75.356, 78.464, 81.176, 83.596, 85.42, 87.506, 89.136, 90.474, 91.592, 92.34, 93.44, 93.89, 94.88, 95.18, 94.996, 95.876, 95.904, 96.198, 96.352, 96.624, 96.836, 96.878, 97.216, 97.47, 97.444, 97.658, 97.402, 97.672, 97.664, 98.07, 98.018, 97.98, 98.136, 98.118, 98.224, 98.356, 98.394, 98.344, 98.292, 98.496, 98.402, 98.544, 98.566, 98.748, 98.584, 98.618, 98.628, 98.622, 98.898, 98.764, 98.744, 98.792, 98.886, 99.012, 98.608, 99.026, 98.98, 99.078, 98.87, 98.982, 99.128, 99.098, 98.976, 98.984, 99.136, 99.214, 99.048, 99.228, 98.978, 99.22, 99.09, 99.37, 98.962, 99.09, 99.36, 99.034, 99.328, 99.272, 99.148, 99.202, 99.268, 99.368, 99.288, 99.204, 99.266, 99.36, 99.38, 99.244, 99.288, 99.414], 'train_loss': [3.7326319022556707, 2.6832166317173893, 2.1661501356098047, 1.8320239302142503, 1.5839015769836542, 1.4128328996241246, 1.2577357717487208, 1.120758622351205, 0.9922157921220945, 0.8902910598327437, 0.8171384161543054, 0.6998209347352957, 0.60349684749799, 0.5177565938828851, 0.45251168813699344, 0.3900569271477287, 0.33117380695384174, 0.2927929998501716, 0.2561261572248643, 0.2290235997303901, 0.19807453908007164, 0.18706580455822255, 0.15966146294494418, 0.14647834503170473, 0.15240188421505263, 0.125276541546978, 0.1228803015823769, 0.11555423810506416, 0.1125622334951997, 0.10273633459868753, 0.0980941132401757, 0.09733289314131431, 0.08726690625033491, 0.08022353783561884, 0.08072175395131931, 0.07368860574310546, 0.08015181002703727, 0.07358838422902766, 0.07490706709725663, 0.061129216961931826, 0.06415444174233605, 0.06466343418469943, 0.0589539160230852, 0.05779759723054784, 0.05760641402655455, 0.05342946050431379, 0.0519629702386756, 0.05103994436699616, 0.054388024264832247, 0.046662615372470635, 0.0494731324111514, 0.046820138877494706, 0.04551898080083277, 0.04053133032793808, 0.04790863171315102, 0.04346214055062553, 0.04427267161651712, 0.04628645114825748, 0.035451538635708414, 0.03849153015229618, 0.03925283462978671, 0.03754757688497789, 0.035580707998151946, 0.032397313634223365, 0.043603536389295316, 0.03188046616474714, 0.03276070606801897, 0.030681633849522872, 0.03690021450880229, 0.033312095821027635, 0.028208237866733505, 0.030026211608004758, 0.03331193253171661, 0.03286750444329863, 0.0272091734338441, 0.02652768414337164, 0.030219539503484744, 0.025279076607829992, 0.030762537909463844, 0.02602508753631026, 0.027523753665791958, 0.021976404097627925, 0.032344684277622736, 0.030001413455912413, 0.02223663637410853, 0.029803793455246013, 0.02291097560340105, 0.02387310069978021, 0.027356025653400295, 0.02607149978624145, 0.02336242204056243, 0.020240657620125717, 0.023420351642121286, 0.02629413940876638, 0.025591253324279615, 0.02118149410098177, 0.020742446713126025, 0.024033272539882067, 0.024180089761486664, 0.019703887564459423], 'val_metric': [21.81, 34.75, 42.91, 50.65, 52.52, 56.21, 58.15, 61.09, 62.52, 64.67, 63.79, 67.57, 67.39, 67.42, 67.69, 67.74, 68.57, 67.7, 68.23, 69.79, 68.41, 68.84, 69.46, 67.27, 68.68, 68.8, 68.63, 69.91, 69.0, 70.4, 69.8, 70.49, 69.49, 70.12, 70.68, 71.42, 69.27, 71.42, 70.55, 71.22, 71.12, 70.61, 70.73, 71.04, 71.3, 70.75, 70.58, 69.86, 70.27, 70.38, 70.62, 70.64, 70.68, 70.64, 70.14, 70.87, 69.96, 72.26, 70.89, 71.48, 70.38, 70.51, 71.31, 71.71, 70.95, 72.32, 71.09, 70.88, 71.02, 71.43, 71.58, 70.21, 70.75, 71.38, 70.61, 71.79, 71.04, 71.8, 70.8, 72.29, 71.7, 72.18, 70.84, 72.47, 70.78, 71.15, 71.62, 70.63, 71.24, 71.36, 72.01, 71.96, 71.81, 71.48, 73.17, 72.66, 71.43, 71.25, 71.19, 71.73], 'val_loss': [3.1529801484126194, 2.510836920161156, 2.1192511054360943, 1.792904027708017, 1.7511551653503612, 1.5630598975594636, 1.506992068639986, 1.3929554569493434, 1.3294524683314524, 1.2734709078339255, 1.3258465148840741, 1.155854924469237, 1.183547631570488, 1.2361185155856382, 1.2722581256726744, 1.290637434288195, 1.2849235303083044, 1.391219350183086, 1.3825075816197, 1.3496152849713707, 1.5139243276255905, 1.4393001931488134, 1.393289022384935, 1.5652857107721316, 1.525631097471638, 1.4669363244323974, 1.538565879035148, 1.5363509017191115, 1.5702487484664673, 1.5106134889232126, 1.5281345130531652, 1.4987806871438483, 1.6713375851606866, 1.561895511332591, 1.4652399246100407, 1.5272421574896309, 1.6518216421649714, 1.5732275923346257, 1.6175196246736367, 1.5879285468417368, 1.5820406982853155, 1.6585467169239263, 1.6473523207530854, 1.6025190395154771, 1.563992862109166, 1.6584213535497143, 1.6530067867534175, 1.6714229849493427, 1.673332602355131, 1.6355699604483926, 1.6572672037561988, 1.7113840329419276, 1.5915119928918826, 1.7238228279314223, 1.7566135430791576, 1.7048773439067184, 1.7374028062364857, 1.5991540505627917, 1.7110537282980172, 1.6546449319572205, 1.7901944810417807, 1.7660472761293886, 1.7222463207639707, 1.6746039872716187, 1.717672500261076, 1.6461430123657177, 1.6094051231244566, 1.8361576427320006, 1.7722835893843585, 1.703230650181983, 1.675587536802717, 1.7441119449153828, 1.6803186201745537, 1.72996953110786, 1.751182730410509, 1.728985653941039, 1.8291655220803182, 1.7240367744378984, 1.7913898089129454, 1.6854196430011918, 1.752041399099265, 1.7342739769607594, 1.794385613149898, 1.6939620736298289, 1.7862034888024543, 1.7439326826174548, 1.7232679644967341, 1.8392769481725753, 1.7887741661375496, 1.7300010365285692, 1.759183781162189, 1.73208234036804, 1.7567432173498116, 1.88074367517119, 1.694464757943609, 1.749224633927558, 1.7499423011852677, 1.8439172282340421, 1.8669497849075658, 1.767049730583361]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal 4", optimizer="SGD")
data.update({'train_metric': [2.07, 3.944, 5.764, 7.4, 9.166, 10.744, 12.18, 13.662, 14.96, 16.128, 17.164, 18.322, 19.394, 20.218, 21.336, 22.282, 23.452, 24.066, 25.024, 26.052, 26.924, 27.468, 28.624, 29.568, 30.192, 30.9, 31.952, 32.22, 33.218, 34.052, 34.466, 35.222, 35.77, 36.344, 37.132, 37.722, 38.14, 38.796, 39.412, 39.98, 40.83, 41.32, 41.714, 42.482, 42.944, 43.588, 44.204, 44.73, 45.404, 46.272, 46.596, 47.07, 47.58, 48.284, 48.764, 49.352, 49.992, 50.526, 50.796, 51.456, 52.072, 52.708, 53.496, 53.864, 54.072, 54.996, 55.644, 56.11, 56.554, 57.184, 57.508, 58.29, 58.754, 59.374, 59.502, 60.21, 60.626, 61.566, 61.912, 62.426, 62.888, 63.334, 64.054, 64.724, 65.182, 65.726, 66.11, 66.608, 67.508, 67.788, 68.282, 69.17, 69.308, 69.95, 70.404, 71.182, 71.558, 72.07, 72.778, 73.342], 'train_loss': [4.623888571853833, 4.370726868319695, 4.198550576749056, 4.064259421794921, 3.962987079949635, 3.8589281029713427, 3.7760679429144504, 3.6918120762271345, 3.619663478773268, 3.5505424262312673, 3.48937731935545, 3.4272989662711884, 3.368558217497433, 3.309837445883495, 3.256200003502009, 3.210563745645001, 3.1535641317782193, 3.103971755413143, 3.058889225315865, 3.009403050098273, 2.965906167579124, 2.9219430453332182, 2.874818995480647, 2.8330217020590895, 2.7956907569295, 2.757101965072515, 2.720079599743914, 2.690749004673775, 2.653093370330303, 2.6180781540663345, 2.5859516636489905, 2.5557850898074372, 2.5173881895401897, 2.4907764344264174, 2.45769221825368, 2.4334360760496097, 2.3991534126079297, 2.3738251549508567, 2.345867598148258, 2.3183644872797116, 2.2827689050103697, 2.259450155298423, 2.236081884797577, 2.2112440616273514, 2.1811626388898593, 2.1498490930213343, 2.120357757334209, 2.0977664923728883, 2.0730274191597844, 2.0435191537718027, 2.0198998388731875, 1.9933050812967599, 1.9724234036167565, 1.9479599291711207, 1.9210731288051361, 1.8989044305918468, 1.8739884341769206, 1.8441214998969642, 1.8265364465811063, 1.8018159806880805, 1.776332621226835, 1.751564088837265, 1.7284469852971909, 1.7073499368279792, 1.6850306252994196, 1.6592962354650278, 1.6359552204456476, 1.6156915993336827, 1.5929329759629487, 1.5665794748174564, 1.5522619879154294, 1.5218980534149862, 1.5023318986453669, 1.4805464310109462, 1.4688529494168507, 1.4425921916504345, 1.425552086147201, 1.393833171681065, 1.3743715367811111, 1.3544257650594882, 1.3410388071213841, 1.3210591962727745, 1.2946903041713989, 1.2708833221431888, 1.25351883459579, 1.2270579818264602, 1.2131407290620877, 1.197676842627318, 1.1660751031945125, 1.1535646270608049, 1.13291176330403, 1.1115164547167775, 1.0948887514641217, 1.0715091241442638, 1.0536387335613866, 1.0330605054145579, 1.0100913895365526, 0.9964621610501233, 0.9742143813453977, 0.9596043691763183], 'val_metric': [3.16, 5.18, 6.8, 9.0, 10.78, 12.02, 13.86, 14.71, 16.14, 16.7, 18.18, 18.89, 20.69, 20.52, 21.94, 23.11, 23.79, 24.75, 25.17, 26.59, 27.25, 27.94, 28.7, 28.98, 30.17, 30.38, 31.05, 31.83, 32.23, 32.17, 33.38, 33.51, 33.56, 34.52, 33.89, 35.43, 35.59, 35.72, 37.09, 37.71, 38.41, 38.31, 36.8, 39.5, 39.58, 38.16, 40.3, 41.34, 40.59, 41.94, 41.86, 42.73, 43.41, 43.46, 43.06, 42.89, 43.63, 43.58, 43.85, 44.28, 43.64, 44.9, 45.62, 45.28, 46.61, 46.04, 46.44, 45.62, 46.47, 46.5, 46.44, 47.59, 46.89, 47.15, 46.52, 46.48, 48.08, 48.48, 48.9, 48.19, 48.42, 48.21, 44.87, 49.33, 49.14, 49.02, 49.87, 48.06, 48.75, 49.99, 49.25, 46.12, 49.97, 48.13, 50.32, 47.6, 50.54, 49.64, 49.52, 48.32], 'val_loss': [4.527576568020377, 4.272809629987, 4.121767200482119, 3.9993620526259113, 3.890431018391992, 3.801954117550212, 3.7028255128556755, 3.6325851595325833, 3.56485999617607, 3.518350630049493, 3.436048679291063, 3.3997226414407136, 3.3334745100349377, 3.2918867047425286, 3.2350811457178397, 3.1840131738383297, 3.132393597037929, 3.089959539425601, 3.049429225314195, 3.0058009502994025, 2.957452482478634, 2.9179603658663997, 2.8819561839862993, 2.859086691194279, 2.8056696159824446, 2.792238773054378, 2.754793967411017, 2.7168979386615146, 2.6998356998346416, 2.6879289590629045, 2.635312355248032, 2.6158692320440986, 2.625284630781526, 2.569218307543712, 2.589217363649113, 2.519233799284431, 2.5190128703026256, 2.506292447922336, 2.452369167546558, 2.420754070494585, 2.404812263835008, 2.3839100766333803, 2.453279882479625, 2.3224171756938765, 2.3074715942334216, 2.377703984072254, 2.2868731181333017, 2.261380251046199, 2.270353640720343, 2.2105929995798004, 2.21621270696069, 2.1939817302545923, 2.165134445117537, 2.163122508176573, 2.158071218022875, 2.1736110236234727, 2.1206353462425764, 2.139875988292087, 2.137454891660411, 2.1248453035476103, 2.134410769316801, 2.0878272701980203, 2.0655569559449605, 2.087118677272918, 2.029275593484283, 2.0451698485453416, 2.043416821273269, 2.0508955929689345, 2.029078344630588, 2.025902543098304, 2.0402972857663584, 1.994553757321303, 2.0113327290601792, 2.0052973776106624, 2.0290950096336897, 2.0248877683262916, 1.960279613543468, 1.959444222936205, 1.930231069303622, 1.9722904878057492, 1.9781734146130312, 1.9793024169411628, 2.1469520861935463, 1.9552903873905254, 1.9645039055757463, 1.9528093398756283, 1.8974306332837245, 1.9803302158975298, 1.9954983770467674, 1.8938842844811214, 1.9479839619557568, 2.134680869473014, 1.8962413171294388, 1.9988278682064857, 1.950635697431625, 2.0412366056138542, 1.9060467367718934, 1.9498164425989626, 1.974191385469619, 2.0487299140091912]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal 4", optimizer="Adam")
data.update({'train_metric': [11.018, 22.448, 28.968, 34.11, 38.114, 40.726, 42.648, 44.566, 46.276, 47.51, 48.856, 49.8, 50.946, 51.62, 52.37, 53.07, 53.866, 54.46, 55.082, 55.51, 55.814, 56.416, 56.978, 57.486, 57.424, 58.048, 58.574, 58.834, 59.062, 59.664, 59.652, 60.072, 60.08, 60.378, 60.834, 60.788, 61.094, 61.258, 61.586, 61.468, 61.714, 61.85, 62.396, 62.204, 62.434, 62.518, 62.772, 62.876, 62.904, 63.176, 63.194, 63.184, 63.57, 63.352, 63.498, 63.734, 63.642, 63.964, 63.952, 63.896, 64.132, 64.15, 64.412, 64.27, 64.484, 64.578, 64.636, 64.794, 64.79, 64.93, 65.114, 65.018, 65.094, 65.228, 65.24, 65.344, 65.082, 65.256, 65.576, 65.644, 65.412, 65.444, 65.722, 65.874, 65.696, 65.41, 65.64, 65.96, 66.052, 66.3, 66.082, 65.916, 66.184, 65.984, 66.19, 66.1, 66.194, 66.514, 66.258, 66.288], 'train_loss': [3.8391204212632632, 3.0792758605059456, 2.749712351947794, 2.5044089798122418, 2.325384160594257, 2.2011418005694514, 2.1030658105450213, 2.0201453975094554, 1.9625659476765587, 1.904555074241765, 1.8475088577746126, 1.8112248043574946, 1.7687790159069363, 1.736519799665417, 1.7040498941145894, 1.6755204266294494, 1.6449336743415774, 1.6169220428637532, 1.597972073061082, 1.5784366275648327, 1.5662861759095545, 1.5442639965078104, 1.5230566479665848, 1.505494561799042, 1.497552497231442, 1.4761626661738472, 1.4660687676780975, 1.4510467659939281, 1.4382877214180538, 1.4273533189235745, 1.4182625662182908, 1.4053791145534467, 1.393751088082028, 1.3837899757010856, 1.3807497869062302, 1.3710712417769615, 1.3647288184641573, 1.3567560745017302, 1.344334559367441, 1.3477477522000023, 1.3352049810959554, 1.335875057960715, 1.3186261988509342, 1.323633871496181, 1.3104158213071506, 1.3115024066642118, 1.3025554366733716, 1.2957942393582191, 1.2892174386154964, 1.288672558182036, 1.278191166308225, 1.2791126170731566, 1.2693461702607782, 1.270582305272217, 1.2643478545538909, 1.264846840630407, 1.2629483154667613, 1.255746900959088, 1.2561508595486126, 1.2507971349122273, 1.2463272202502735, 1.2450793324529057, 1.2387193582399423, 1.2367951817372267, 1.2332356334342371, 1.2302333286503697, 1.2239216526451013, 1.219757118722057, 1.2227068583831153, 1.2169023920660433, 1.2161297905628028, 1.2189792766595435, 1.2081469683848378, 1.2112659118364535, 1.207459050676097, 1.2042815763017405, 1.2093704365708333, 1.2050540813857027, 1.1962685610937036, 1.1955176668856151, 1.1929516922634886, 1.1979952498012796, 1.1866832705562378, 1.187229664886699, 1.181088500315576, 1.1881362345365003, 1.1839624950495522, 1.1736305656335544, 1.180265579763276, 1.1763234404499268, 1.1741955291737072, 1.178211532742776, 1.1691157114322839, 1.1750510110879493, 1.1665579527235397, 1.1689067570602192, 1.1683390437032255, 1.1634411436060201, 1.1646864603242606, 1.1620829439986393], 'val_metric': [15.31, 21.81, 30.44, 34.43, 37.11, 40.53, 42.06, 41.48, 43.96, 46.51, 46.29, 47.9, 48.57, 48.97, 49.72, 51.69, 52.93, 50.91, 52.28, 51.91, 51.82, 52.32, 53.0, 54.48, 52.61, 49.22, 55.85, 54.11, 55.14, 54.64, 55.31, 54.02, 55.22, 54.34, 54.11, 56.44, 55.28, 56.71, 56.48, 56.72, 54.88, 56.01, 57.35, 57.05, 55.29, 57.04, 56.02, 57.35, 56.34, 56.81, 55.64, 58.6, 58.57, 57.43, 56.21, 57.31, 59.06, 57.34, 58.96, 59.84, 54.86, 58.28, 56.86, 56.68, 57.51, 58.11, 58.39, 58.27, 58.71, 57.77, 59.27, 56.84, 58.23, 57.96, 58.77, 58.13, 58.1, 58.11, 58.88, 58.75, 58.06, 56.01, 57.07, 58.45, 59.03, 59.78, 58.91, 58.73, 58.22, 58.24, 58.84, 58.66, 57.67, 57.65, 58.92, 56.06, 57.05, 57.88, 58.47, 58.36], 'val_loss': [3.4858025213715376, 3.1184704060767108, 2.709249233743947, 2.463880814564456, 2.3451694667718974, 2.1947907718123902, 2.1646007307016166, 2.168670850954238, 2.054202394121012, 1.9332272239551422, 1.9740890659344423, 1.8969311159887132, 1.9041523318381826, 1.8327867567159568, 1.8270623213166644, 1.7420360016974674, 1.6936448896007172, 1.7753666889895299, 1.7100980000890744, 1.714942877459678, 1.7401530370590792, 1.69096265628839, 1.689160053137761, 1.6181269208337092, 1.6994844758586518, 1.838561506028388, 1.5683910770780722, 1.6514668309005203, 1.601095316516366, 1.6243033728022485, 1.5890234458218715, 1.6262559788242268, 1.5928048351008421, 1.646538505128994, 1.6363602610910015, 1.54253955935217, 1.5905701324438593, 1.530643245976442, 1.5320066771689493, 1.5388932061043514, 1.6299408036432448, 1.5759348485879838, 1.490022319517318, 1.5143374504557081, 1.5814847338731122, 1.5263100657493445, 1.551500340935531, 1.520183171056638, 1.5450747802758673, 1.5682547377173308, 1.5946027883298837, 1.4519090785342417, 1.4591623833225031, 1.4934605230951006, 1.5509547765847225, 1.5230082524050572, 1.470068700753959, 1.4980784168668613, 1.4461601900446945, 1.443938134202532, 1.6322945136173515, 1.4734411456023053, 1.5245990381119356, 1.5588586353192664, 1.5278065777887964, 1.4632555785452483, 1.4715525910353204, 1.4718568287077982, 1.4591437365598738, 1.5067761472076366, 1.43697029959624, 1.573552082678315, 1.4819179135523024, 1.4798212727164006, 1.4543196653864185, 1.482426062511031, 1.473682661724698, 1.4556327519143464, 1.4530084831699444, 1.4676470122519572, 1.4741648169839459, 1.6007555282799302, 1.5373307462710484, 1.4813032351481688, 1.4631910210202455, 1.431930027569935, 1.4536983253090245, 1.454860166759248, 1.4723654882923054, 1.5056301279432456, 1.4649778019850421, 1.471907788780844, 1.5239934226509873, 1.5357873128477935, 1.4787024217806044, 1.5584984626739649, 1.5337937090806901, 1.5200268664177816, 1.4609290015925267, 1.4699686256943234]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal 4", optimizer="AdamW")
data.update({'train_metric': [12.214, 28.184, 39.65, 47.376, 53.28, 58.478, 62.422, 66.404, 69.77, 72.662, 75.62, 78.4, 81.204, 83.356, 85.304, 87.448, 89.03, 90.238, 91.664, 92.464, 93.296, 93.526, 94.414, 94.854, 95.172, 95.53, 95.692, 96.134, 96.096, 96.362, 96.72, 96.864, 96.986, 97.0, 97.178, 97.538, 97.204, 97.368, 97.852, 97.984, 97.664, 97.926, 97.826, 98.072, 97.972, 98.23, 98.174, 98.37, 98.306, 98.35, 98.388, 98.386, 98.476, 98.362, 98.432, 98.726, 98.536, 98.59, 98.652, 98.714, 98.608, 98.606, 99.052, 98.644, 98.778, 98.798, 98.842, 98.994, 98.856, 98.858, 99.062, 98.75, 99.06, 98.994, 98.888, 99.08, 99.078, 98.982, 99.2, 99.016, 99.08, 99.17, 99.176, 99.096, 99.206, 99.044, 99.106, 99.212, 99.26, 99.124, 99.296, 99.094, 99.286, 99.4, 99.254, 99.174, 99.336, 99.222, 99.272, 99.33], 'train_loss': [3.7871495427378, 2.8017169407871374, 2.268755693112493, 1.9296389049886133, 1.669506963561563, 1.4708431437039924, 1.3012500787939867, 1.1468991839977176, 1.025225467084314, 0.9088028149150521, 0.8000602495411168, 0.703571102160322, 0.6080549751096369, 0.5243952267081536, 0.46174940722220387, 0.3921166830850989, 0.33779640738730843, 0.30116441315206727, 0.25530130730565553, 0.23176797769506416, 0.2070862402105728, 0.19508893311957418, 0.17219550890700364, 0.15786743175257426, 0.14708107827312272, 0.13696291263732116, 0.13155489694327116, 0.1188332513332024, 0.12079348225894448, 0.10981653142265518, 0.1010955007975478, 0.09601190998671515, 0.09564403864814688, 0.09450117959891019, 0.08864980267213128, 0.07755662533401005, 0.08787011503047831, 0.0816321644617621, 0.06795608374805849, 0.06602872690647993, 0.0729306526240938, 0.06483554710363708, 0.06797554931970184, 0.05974840113918161, 0.06301573879511842, 0.056519726252593, 0.058808654571449875, 0.05261002929882406, 0.05419872370084076, 0.0546827316310976, 0.050646375760833834, 0.04967523356327963, 0.05015191320098265, 0.05303179409698514, 0.048315300769053514, 0.041396982413615384, 0.04589436339694397, 0.046107744765759726, 0.04345898351068144, 0.03952417718096222, 0.04445438618775305, 0.043138755269853585, 0.030283719254702288, 0.043031072606543395, 0.03931848696091205, 0.03889064724310575, 0.03677776418066562, 0.032906303213506724, 0.03594538822877007, 0.0364132161956945, 0.029666568076087977, 0.04034977486228172, 0.03141388510722462, 0.032180011836196176, 0.03440537972315012, 0.029673993639523506, 0.03114169087638567, 0.032960322632393596, 0.027162681711921613, 0.032692828587234934, 0.029216615779190193, 0.027822855179292765, 0.02605771896806891, 0.029717241169969953, 0.0267907207189893, 0.03113146349588228, 0.02653817425474085, 0.02556786257492276, 0.02426207614530947, 0.026790138277130813, 0.023932245580794806, 0.030403210506070335, 0.024507987139093905, 0.019568816839639618, 0.023996076256166572, 0.026703497284309954, 0.022581658892832514, 0.025732867914224293, 0.023256032695279805, 0.022412546302570484], 'val_metric': [19.23, 33.63, 41.63, 46.45, 51.96, 55.64, 58.2, 60.29, 62.78, 63.96, 64.89, 64.9, 65.69, 66.45, 68.51, 66.66, 68.24, 67.52, 67.54, 68.49, 67.49, 69.25, 67.92, 69.51, 69.14, 67.92, 68.29, 68.97, 68.99, 69.58, 68.75, 70.05, 67.83, 68.87, 69.19, 68.02, 68.81, 70.11, 67.87, 69.99, 70.26, 69.42, 70.13, 69.65, 69.13, 69.18, 69.17, 69.03, 68.97, 70.15, 69.4, 69.79, 69.54, 70.73, 70.56, 69.09, 70.55, 70.13, 70.3, 70.66, 70.47, 70.03, 69.49, 69.3, 69.83, 69.57, 71.39, 69.45, 70.75, 69.75, 69.76, 69.95, 70.52, 69.17, 70.97, 70.44, 69.98, 70.05, 69.32, 70.71, 70.64, 71.0, 68.93, 71.36, 71.01, 70.11, 69.49, 69.79, 70.66, 70.49, 70.69, 71.27, 70.85, 69.96, 70.78, 71.55, 69.55, 70.78, 70.87, 70.36], 'val_loss': [3.2678181517655682, 2.516503414530663, 2.187691120585059, 1.9833806411475892, 1.7602384120795378, 1.6161013382255651, 1.4911382483069304, 1.4106493960520265, 1.3274939698018846, 1.2952551970815962, 1.2781514901264457, 1.2922888420949317, 1.3291555445665006, 1.3243915932193684, 1.216324752683093, 1.3711099981502364, 1.2959900714789228, 1.3672982875708561, 1.4463348479787255, 1.3937311920390767, 1.516549610028601, 1.3999084127936394, 1.4817562668946138, 1.4245435337352146, 1.4594650374855964, 1.554687416477568, 1.5681302015948448, 1.5940235412804185, 1.5663986908402412, 1.5532951461281745, 1.614287184681862, 1.5813127213222966, 1.6909738501925378, 1.6134293041411478, 1.5861935638318396, 1.700218442898647, 1.7091104164245023, 1.6121106808352623, 1.8288850814673552, 1.6411619330667386, 1.7069315075115035, 1.706811274692511, 1.6121530327827307, 1.7297773106842285, 1.6905659038549776, 1.7588303905383798, 1.7707537887202707, 1.7431997406254909, 1.8047786279089133, 1.6775857206362828, 1.7575527850989323, 1.7152883550923341, 1.70447286716692, 1.6971391545739143, 1.6797655399437923, 1.8105895633150817, 1.738883365111746, 1.765241387543405, 1.7491046429439714, 1.6759335651139544, 1.671333860060212, 1.7520099627743861, 1.8138306634441304, 1.84387269103603, 1.8252744883488699, 1.8790693970242882, 1.6891414534514118, 1.817481557654727, 1.7741296997495517, 1.8421490602432542, 1.8056430087727346, 1.8172203165710352, 1.7329351222439178, 1.8328126984037412, 1.7205628103511348, 1.761297595349087, 1.7665770209518967, 1.7447842097586128, 1.8586490549099672, 1.8063728296832673, 1.7426633276757162, 1.8235050145607845, 1.971518878724165, 1.7544838042016242, 1.7750563120386402, 1.8146281352468356, 1.841653365238457, 1.760440849574508, 1.8443287599618268, 1.8531210084629666, 1.8230399462827451, 1.7791639262703574, 1.7994030471060687, 1.8403184501228818, 1.8122687260056758, 1.7455224554250195, 1.8932546020313432, 1.786825162969577, 1.7712779177981577, 1.79976621896598]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Orthogonal", optimizer="SGD")
data.update({'train_metric': [2.84, 5.434, 7.482, 9.37, 11.098, 12.586, 13.74, 15.182, 16.344, 17.34, 18.422, 19.45, 20.456, 21.418, 22.274, 23.018, 23.832, 24.866, 25.674, 26.622, 27.466, 27.938, 28.982, 29.608, 30.294, 30.862, 31.618, 32.336, 32.862, 33.786, 34.216, 35.182, 35.536, 36.012, 36.726, 37.274, 37.722, 38.374, 38.808, 39.824, 40.29, 40.86, 41.378, 41.736, 42.304, 42.906, 43.748, 44.392, 44.87, 45.604, 45.954, 46.788, 47.14, 48.014, 48.592, 49.084, 49.574, 50.088, 50.394, 51.436, 52.2, 52.404, 53.054, 53.532, 54.254, 54.65, 55.252, 56.058, 56.466, 56.84, 57.646, 57.988, 58.95, 59.262, 59.91, 60.502, 61.152, 61.482, 61.894, 62.732, 63.462, 63.908, 64.362, 64.882, 65.332, 65.868, 66.604, 67.03, 67.806, 68.084, 68.864, 69.434, 69.966, 70.32, 70.948, 71.848, 72.228, 72.838, 73.196, 73.78], 'train_loss': [4.495636571093899, 4.293181678523188, 4.134871849623482, 4.017091416641879, 3.904788532525377, 3.8041046735880624, 3.71007605464867, 3.6360280279003447, 3.566324341632521, 3.5058520440860175, 3.448122442530854, 3.393694976406634, 3.3391416176505713, 3.291758010759378, 3.2441293909726547, 3.1983297206556704, 3.1461995355308514, 3.1061322496980046, 3.0599880694123485, 3.011668957102939, 2.969466639296783, 2.9298915366077667, 2.8881438150430276, 2.851051030256559, 2.8127667544138095, 2.7760472675723493, 2.734948689370509, 2.704704239240388, 2.6736471088950897, 2.640109912513772, 2.6049429044089356, 2.5699593818096247, 2.543723155318014, 2.517373965524347, 2.485490696814359, 2.4563004830304314, 2.432057343480532, 2.402383009948389, 2.3796114958155794, 2.346897970075193, 2.3151542474241817, 2.2914120221077026, 2.2655724360205025, 2.2368285729147286, 2.2098867768216928, 2.182973119761328, 2.1584161305061693, 2.132003485699139, 2.1056380194166433, 2.077180307082203, 2.0534859421612968, 2.0265453505089215, 2.000851789551318, 1.9675343567148194, 1.9467750354800992, 1.9225428871181616, 1.89410704831638, 1.8756157383894372, 1.8531687733767284, 1.8232839508434695, 1.7969008903674153, 1.7743489891671769, 1.7456799783670078, 1.7211065397543066, 1.698837665188343, 1.6781076218770898, 1.6579890742021448, 1.6242730085502195, 1.6107396513909635, 1.5839728545349883, 1.5573200406625753, 1.536711715218966, 1.511364600618782, 1.4895932855051193, 1.4669728212801696, 1.444517230819863, 1.4176381420906243, 1.399257297863436, 1.3869405895700235, 1.3573692521018446, 1.3377964553778128, 1.3157819692436081, 1.2951157574001175, 1.2732052200133233, 1.2548855331242847, 1.2347997453663966, 1.2037666257842423, 1.1867998878059485, 1.1666262475273492, 1.1498361784021567, 1.1287188574938518, 1.0998454055060511, 1.0811393285346458, 1.066116398100353, 1.041573859617838, 1.0171021153707334, 1.003025627578311, 0.9800413347723539, 0.9595201418680304, 0.9410959178834315], 'val_metric': [3.9, 6.95, 8.63, 10.39, 11.71, 13.32, 15.14, 16.33, 17.3, 18.22, 19.48, 20.23, 21.57, 21.91, 22.99, 23.86, 24.16, 24.73, 25.6, 26.95, 26.5, 27.94, 28.06, 28.8, 29.7, 30.39, 31.01, 31.03, 31.41, 31.7, 32.72, 33.24, 33.53, 34.14, 33.1, 34.54, 34.69, 33.99, 35.4, 35.19, 36.89, 36.39, 36.33, 35.6, 38.37, 37.78, 38.31, 36.5, 40.07, 38.8, 39.56, 39.74, 39.36, 41.4, 41.79, 42.34, 42.16, 42.77, 41.15, 42.3, 42.9, 43.82, 40.36, 44.34, 43.72, 44.12, 41.37, 44.25, 42.61, 44.37, 43.87, 45.33, 43.96, 46.1, 45.61, 43.46, 45.96, 43.72, 44.38, 43.87, 46.31, 46.23, 46.68, 44.92, 45.8, 41.11, 45.09, 42.64, 45.77, 45.35, 45.27, 46.66, 45.14, 48.35, 44.04, 46.01, 42.03, 45.92, 42.76, 44.79], 'val_loss': [4.382848766958638, 4.203800728366633, 4.074669339854246, 3.9597474906095274, 3.8389687340730316, 3.7433645011513095, 3.6548529567232557, 3.5833849345043207, 3.526895805528969, 3.4708385907920305, 3.4086982338291825, 3.3674113143021893, 3.3116160046522785, 3.2701734327206946, 3.2271290083599697, 3.177550224741553, 3.1506512969922107, 3.121621813743737, 3.0654994241750924, 3.0252394721766187, 3.0137304637082822, 2.946031254567918, 2.927593164383226, 2.906746155137469, 2.8569363196184683, 2.8154651207529056, 2.7920772118173587, 2.7620002312265384, 2.7403260522587285, 2.7217758749700653, 2.6813972041865064, 2.6465909207702443, 2.6506642642294524, 2.6215882529119017, 2.6646532086050434, 2.592654023200843, 2.562939654489991, 2.6092735065776074, 2.5420008222008965, 2.5606061326470346, 2.4887258998907296, 2.4912276860255345, 2.4927286258928336, 2.5536337475867787, 2.4274948364610127, 2.448186258601535, 2.403916601922102, 2.532642347797467, 2.3331824093107967, 2.376161349806816, 2.3619712826552663, 2.3515180174712165, 2.353100254277515, 2.2867493545933133, 2.2918302990069055, 2.246799477346384, 2.2481258857022426, 2.207682596650093, 2.3157791066321596, 2.233394460313639, 2.212227057499491, 2.1548700180782636, 2.339360227250749, 2.1453816511069133, 2.1524731642121724, 2.1713344496526537, 2.2764576825366656, 2.137649913502347, 2.2323693636876003, 2.1560511148659285, 2.162991794810933, 2.095161923177683, 2.1712305507842142, 2.0966249833441086, 2.0935374467995516, 2.2466899855121687, 2.0903445337988007, 2.1879918522136226, 2.1879582450648023, 2.1981002115140296, 2.0869392331238763, 2.0753373963058372, 2.0828393530693785, 2.1593094988233723, 2.157110599955176, 2.5686152983622947, 2.123987040701945, 2.388056352639654, 2.1266716992019847, 2.209268976169027, 2.220490776049863, 2.1262186378430408, 2.2251588538953455, 2.051798128778008, 2.3157488472142798, 2.1684119002834246, 2.3996858604394706, 2.1962580794741395, 2.3932108279246433, 2.283881144159159]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Orthogonal", optimizer="Adam")
data.update({'train_metric': [8.566, 15.876, 22.432, 28.894, 34.712, 38.42, 41.418, 43.648, 45.586, 46.682, 48.31, 49.394, 50.31, 50.99, 51.856, 52.33, 53.102, 54.168, 54.372, 54.588, 55.394, 56.122, 56.206, 56.26, 56.774, 57.076, 57.31, 57.558, 57.998, 58.47, 58.092, 58.668, 58.962, 59.066, 59.396, 59.172, 59.56, 59.71, 59.738, 59.874, 60.26, 60.524, 60.45, 60.502, 60.628, 60.64, 60.85, 61.228, 61.296, 61.212, 61.418, 61.43, 61.24, 61.628, 61.662, 61.822, 61.972, 61.714, 62.102, 62.21, 62.58, 62.288, 62.31, 62.35, 62.482, 62.48, 62.834, 62.646, 62.878, 62.858, 62.896, 62.966, 62.846, 63.346, 62.796, 63.288, 63.22, 63.112, 63.394, 63.238, 63.386, 63.692, 63.446, 63.558, 63.78, 63.578, 63.57, 63.682, 63.966, 63.716, 64.002, 63.904, 63.836, 63.798, 63.85, 64.078, 63.916, 63.752, 64.038, 63.956], 'train_loss': [4.024560484739825, 3.4858820721926285, 3.1331663159153345, 2.7834132239031977, 2.492856503904933, 2.311174091475699, 2.180694047294919, 2.080626865939411, 1.996637548022258, 1.9456872607740905, 1.8817585548171607, 1.8368392806223897, 1.799983163471417, 1.773660936623888, 1.7345427860079519, 1.7098917418428699, 1.683372734605199, 1.6499814163998265, 1.6370807087329953, 1.6241935959557439, 1.6008079231090253, 1.5781847609735815, 1.5676925128987989, 1.5568026593883935, 1.5414206934402057, 1.5279317055364399, 1.5164553789836366, 1.510299669964539, 1.4933121570235932, 1.4851938714761563, 1.4787663616945066, 1.4704331857773958, 1.4588849984319008, 1.4556501621328046, 1.4408571132461128, 1.4464929874443337, 1.4308200718649209, 1.4323620203205996, 1.4245133980003464, 1.413407596175933, 1.4129422314636542, 1.4003096745752008, 1.4022237735483654, 1.396536021357607, 1.3933900718188956, 1.3855021533453862, 1.389057673380503, 1.3755596125369791, 1.3691898533297926, 1.36942520638561, 1.364762615593498, 1.36179787858063, 1.3545092308460294, 1.3563818042083164, 1.3515097283951156, 1.3530132764440668, 1.3470146168223427, 1.3502568934122314, 1.3406400154618656, 1.3352671971406473, 1.325470684053343, 1.3286220204952124, 1.3296009361591485, 1.3194594961755417, 1.3211851708419489, 1.3215781437314076, 1.3142174591341287, 1.3157399351639516, 1.3147854629684896, 1.3115007408592096, 1.309394991153951, 1.3054734596510982, 1.3037251382685073, 1.299747716511607, 1.302134419798546, 1.2953013152722508, 1.2932889116237232, 1.2883467709316927, 1.289770007438367, 1.2884728586124947, 1.2881589277321115, 1.280963838557758, 1.2821695058394575, 1.2831362493507696, 1.2775762363163101, 1.2776085292286885, 1.2786662019122288, 1.2741967932037686, 1.2716921602978426, 1.277419610935099, 1.2675580695614486, 1.2627636161454194, 1.266862748681432, 1.269944720256054, 1.2675284587056435, 1.2656288938906493, 1.2625734672674438, 1.2654495413803384, 1.260119910282857, 1.2589425624483992], 'val_metric': [13.08, 16.62, 19.68, 30.8, 33.52, 36.06, 38.37, 44.07, 43.31, 47.05, 47.08, 44.7, 47.28, 43.84, 47.06, 49.69, 50.05, 50.35, 50.33, 51.45, 51.17, 52.09, 53.18, 53.22, 52.62, 53.4, 53.7, 54.16, 51.84, 54.23, 53.8, 53.63, 54.59, 52.2, 54.66, 53.27, 55.39, 53.06, 54.92, 54.88, 55.11, 55.53, 55.74, 55.75, 56.92, 53.0, 55.95, 55.06, 54.53, 54.9, 54.86, 54.9, 57.73, 56.62, 56.22, 56.07, 53.76, 56.3, 56.8, 55.48, 57.57, 55.86, 55.41, 55.86, 57.13, 54.95, 55.81, 56.42, 55.84, 55.72, 56.03, 55.47, 56.64, 54.98, 52.57, 55.94, 54.08, 57.42, 58.88, 56.14, 56.12, 50.96, 55.6, 57.95, 55.74, 58.74, 57.13, 57.19, 55.58, 57.74, 57.08, 57.23, 55.61, 55.33, 55.07, 56.28, 55.37, 55.62, 58.48, 54.93], 'val_loss': [3.6896571080396128, 3.4228269826075075, 3.272701064492487, 2.6578019424608557, 2.5270154460979875, 2.4200055887744685, 2.2768179178237915, 2.0729864980005157, 2.133633299997658, 1.9412127885089558, 1.9208643466803679, 2.0217903358921125, 1.8867663289331327, 2.119012403640018, 1.9389231053127605, 1.8169380669381208, 1.7883508767291998, 1.7787355556609525, 1.7616086226360055, 1.7513271729657605, 1.7423042749903004, 1.6969630665080562, 1.6794709149439624, 1.6898296342533865, 1.6891924218767007, 1.6663032876458137, 1.668865947966363, 1.6336282218337819, 1.7260021996346249, 1.6491597177116735, 1.6711194272253924, 1.627991658866785, 1.6371578357781573, 1.7256902995382903, 1.6187712683039865, 1.6540473160470368, 1.6061718372782325, 1.6816410502051091, 1.6116669815816698, 1.6183003452932758, 1.5903563924655792, 1.5789579737717938, 1.5591617613841013, 1.5682151841509873, 1.5436555982395341, 1.6604668775181861, 1.567865875875874, 1.5967128018664707, 1.6056915999977452, 1.5900582196606192, 1.6089765144761201, 1.6121815313958818, 1.5113514665585415, 1.5379115779688404, 1.533234898451787, 1.5809709513263337, 1.6612780003031349, 1.5444883776318497, 1.5312431306596015, 1.6019590523592226, 1.524237509745701, 1.5546555025562359, 1.5785093451761136, 1.566024777236258, 1.5298049324637006, 1.6149407746685538, 1.5846161257689166, 1.5615193247795105, 1.5648361611518131, 1.5630696996761735, 1.5812847576323588, 1.6180113227504074, 1.5648276934957808, 1.5940882309227233, 1.7230066364737833, 1.5786222029643453, 1.648483906581903, 1.500937732541637, 1.4709691792536692, 1.5636494152105538, 1.5693511226374632, 1.7547948519895031, 1.5744091689966286, 1.470063400496343, 1.5896647614278612, 1.4684610408582506, 1.5568534151004378, 1.552526588652544, 1.5900861351353348, 1.5108657695685224, 1.5245225087852234, 1.5136028543399398, 1.5543605020851086, 1.6000867087370272, 1.6314268453865295, 1.5800128924618861, 1.608293900064602, 1.577017012675097, 1.4592630024168902, 1.610970181264695]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Orthogonal", optimizer="AdamW")
data.update({'train_metric': [9.114, 20.132, 31.19, 41.09, 48.084, 53.696, 57.758, 62.124, 65.792, 69.048, 72.062, 75.436, 77.96, 80.862, 83.318, 85.776, 87.69, 89.398, 90.776, 92.072, 93.142, 93.716, 94.238, 94.78, 94.914, 95.758, 96.116, 96.23, 96.376, 96.76, 96.942, 96.842, 97.078, 97.388, 97.282, 97.628, 97.566, 97.644, 97.726, 97.792, 97.922, 98.168, 97.948, 98.21, 98.192, 98.274, 98.412, 98.328, 98.426, 98.734, 98.476, 98.398, 98.596, 98.572, 98.704, 98.556, 98.79, 98.94, 98.692, 98.79, 98.896, 98.736, 98.886, 98.934, 98.8, 99.056, 98.734, 99.122, 98.808, 99.006, 99.12, 99.17, 98.964, 99.056, 99.15, 99.028, 99.104, 99.216, 99.16, 99.08, 99.17, 99.274, 99.01, 99.204, 99.096, 99.382, 99.094, 99.344, 99.25, 99.146, 99.252, 99.338, 99.282, 99.31, 99.218, 99.486, 99.272, 99.406, 99.296, 99.49], 'train_loss': [3.992348138023825, 3.2593495165905377, 2.6711725097178194, 2.2044003073821594, 1.8890018482952167, 1.661017119579608, 1.491380682534269, 1.3150247293512534, 1.1738493646044865, 1.0404702414332143, 0.9198274429115798, 0.8084744823253368, 0.7090359503579566, 0.6089076573205421, 0.5299182457806509, 0.4489086389617847, 0.38258994582211575, 0.328358340436769, 0.28155233288931725, 0.2460099367324806, 0.2126057580675539, 0.1925068909154676, 0.1793720422507933, 0.15952658227852087, 0.15396755142732882, 0.13235235677036405, 0.12181466546676614, 0.11416561005975279, 0.1113550993042243, 0.10167313494321788, 0.0989732230772905, 0.09646565451636872, 0.09101409745900452, 0.0812372598157543, 0.08547974254488183, 0.07308508327607151, 0.07589720014680434, 0.07340200069596244, 0.07255851009698666, 0.06938944246007792, 0.06434047042333838, 0.05827483685825335, 0.06513840610386513, 0.057709412426268564, 0.05733026770810547, 0.05492314102861773, 0.05071994649958523, 0.05370772185990268, 0.050083955231806275, 0.04173171250899906, 0.04886790159571668, 0.05167168659581434, 0.04514709060616872, 0.04559321497070372, 0.042524517562938256, 0.04550203792971161, 0.039233605134124626, 0.0343294657601277, 0.04187107128862296, 0.039189060217292404, 0.03746512871371973, 0.042512486199610164, 0.036879127247013685, 0.03456173032460272, 0.03853959236946076, 0.03401422729816042, 0.04214690387961478, 0.029428712416501582, 0.03679574437683884, 0.032618798303466694, 0.028411096675902643, 0.02690303949777649, 0.033518666450334404, 0.031129509266858917, 0.027264167131963866, 0.031863414256311264, 0.0301382888913946, 0.024966707004218117, 0.02771965563504199, 0.0305647893681946, 0.027053706835115165, 0.023377326535447465, 0.031336679952763985, 0.02583836669798198, 0.028205943194574073, 0.02152904903880455, 0.02891123662856074, 0.02062793141838325, 0.02307843853695118, 0.026900524195916937, 0.024848697364547343, 0.021739018490099436, 0.022925640217741936, 0.022420762192003566, 0.025616818042869485, 0.018329144522557846, 0.02420304097493341, 0.01997187215406635, 0.022811177052388353, 0.016885213663398416], 'val_metric': [14.42, 25.08, 35.93, 43.37, 48.75, 53.31, 52.71, 58.3, 60.29, 61.38, 62.51, 63.89, 65.41, 65.25, 65.68, 64.51, 65.21, 63.97, 65.66, 65.9, 66.63, 65.5, 66.59, 66.39, 66.35, 66.55, 66.86, 68.03, 65.7, 67.87, 67.66, 66.7, 67.59, 66.84, 68.08, 67.46, 68.66, 67.37, 68.42, 68.24, 68.28, 66.57, 68.58, 68.38, 67.52, 67.27, 68.01, 68.08, 68.89, 68.57, 68.11, 68.48, 68.22, 68.68, 68.33, 68.49, 67.64, 68.33, 67.89, 69.06, 68.93, 68.32, 68.66, 67.2, 68.97, 68.46, 68.46, 69.47, 68.47, 68.32, 69.25, 69.29, 68.67, 68.74, 69.29, 69.13, 69.2, 68.92, 69.3, 68.53, 69.86, 69.32, 69.04, 69.71, 68.96, 69.31, 68.99, 68.21, 69.39, 68.71, 68.63, 68.86, 69.43, 69.56, 69.32, 69.03, 68.18, 69.17, 68.51, 70.24], 'val_loss': [3.5844754443806446, 2.972042030589596, 2.4766851952121516, 2.1048774855911354, 1.880498569482451, 1.677908749337409, 1.727988872558448, 1.4751196461878004, 1.4174131333448325, 1.3810227160241193, 1.3423759652550813, 1.3269851891098507, 1.2638115393128364, 1.3088103225276728, 1.3127104472962154, 1.4800543887600017, 1.4628520980002775, 1.5323785847159708, 1.4671438473045446, 1.5211643439948939, 1.496445716566341, 1.6380416720536104, 1.6003062892112003, 1.676549735342621, 1.6301267158453632, 1.687885200901396, 1.6590031689139688, 1.6827733926712327, 1.8258693176469984, 1.7237550403661788, 1.7530960831672522, 1.7618918118962816, 1.8035515763197736, 1.780212931572252, 1.7289327918344242, 1.7884231217347892, 1.6872741898913293, 1.7536680208649604, 1.7281124819615843, 1.7472989631306595, 1.7735036626742904, 1.8816455849416696, 1.760377152710204, 1.7962781612280827, 1.8610863499580674, 1.8206072802756244, 1.8310234759263933, 1.831039538808689, 1.8182241210512295, 1.7667200850073699, 1.8612149437521672, 1.8725201187619738, 1.8607100882347982, 1.7662802513237972, 1.8478449081918995, 1.8390643494144367, 1.9539779048816415, 1.8334631532620473, 1.8822772878750114, 1.8504956691128434, 1.9654919478544004, 1.8986850418862264, 1.8675511088340906, 2.0204123547122737, 1.8970890975302193, 1.8861711051813357, 1.8848930232843775, 1.8219183273376174, 1.9665225410157707, 1.8693603888438766, 1.8810066679480728, 1.8468713369339136, 1.9439244854981732, 1.8975449732154797, 1.9204215289680822, 1.892872810743417, 1.9718605784853553, 1.9431486627098862, 1.8677292094109164, 2.0311174609099223, 1.8589086198503044, 1.9084228178498093, 1.893923097734998, 1.8171311878854302, 1.9357431386686434, 1.878850378428295, 1.9035010345422538, 1.933634641443848, 1.9422457669950595, 1.9744232422227312, 1.8897125610880032, 1.8865341677027903, 1.967605946549944, 1.9142506813547413, 1.8746128928889134, 1.969501324899637, 1.9939692134310485, 1.9551001566990165, 1.923239084186068, 1.878108588373585]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="LSUV", optimizer="SGD")
data.update({'train_metric': [2.514, 4.418, 5.916, 7.878, 9.44, 10.73, 11.924, 12.876, 13.686, 14.574, 15.362, 16.004, 16.428, 17.412, 17.898, 18.488, 19.416, 19.948, 20.6, 21.286, 21.974, 22.45, 23.014, 23.526, 24.018, 24.846, 25.516, 25.876, 26.24, 26.978, 27.538, 27.89, 28.612, 29.104, 29.28, 30.2, 30.708, 30.958, 31.684, 32.016, 32.642, 32.972, 33.304, 33.738, 34.196, 34.846, 35.136, 35.474, 35.802, 36.224, 36.536, 37.046, 37.412, 37.998, 38.53, 38.96, 39.266, 39.51, 40.058, 40.336, 40.822, 41.032, 41.416, 41.866, 42.258, 42.684, 43.294, 43.626, 44.136, 44.234, 44.616, 45.39, 45.306, 46.152, 46.436, 46.798, 47.278, 47.618, 47.902, 48.564, 48.646, 49.294, 49.628, 49.972, 50.344, 50.696, 51.112, 51.696, 52.22, 52.432, 52.846, 53.272, 53.82, 54.12, 54.308, 54.774, 55.438, 55.568, 56.19, 56.636], 'train_loss': [4.515361134048618, 4.3774990303742, 4.251566345124599, 4.12778649549655, 4.026536406458491, 3.9312028009873217, 3.846178927689867, 3.77479659047578, 3.721803459974811, 3.667546535696825, 3.6195076048526618, 3.577688276310406, 3.5381334416396784, 3.4957862891199643, 3.4618793732065067, 3.4290267893725344, 3.3883812857405915, 3.358013004598105, 3.3239615887327267, 3.29229490135027, 3.258575525430157, 3.226808558034775, 3.192949743222093, 3.1627794988929767, 3.1316676478251777, 3.104595143166954, 3.0714634934349743, 3.041887236678082, 3.0155850426315345, 2.9860612827798594, 2.9580220121251957, 2.928036176030288, 2.899140399130409, 2.873042820969506, 2.84487939826058, 2.810119426463876, 2.7927143274975554, 2.763738229146699, 2.7382649433277453, 2.714216381082754, 2.6920823121009887, 2.664857678096313, 2.6456428281486493, 2.6218519293133866, 2.599661372811593, 2.577455462702095, 2.556342163506676, 2.5352433813197535, 2.5160026602123096, 2.494344374560334, 2.4755120957294086, 2.455975657381365, 2.437002855036265, 2.4172774426772468, 2.3924265041985473, 2.3753703105480164, 2.359169476934711, 2.3341108725199002, 2.3154495367613595, 2.300620941097474, 2.2866083482647186, 2.262588125817916, 2.2466614337833337, 2.227102571588648, 2.215426017561227, 2.1928609877901004, 2.1724675753537346, 2.1531357589889977, 2.1358241717833693, 2.1155761899545675, 2.0992924156396287, 2.0763550704092624, 2.0620739984390375, 2.0421124576302745, 2.0294624818560414, 2.00958645587687, 1.9918603865082, 1.9769359232519594, 1.9599750907829656, 1.9345199802647466, 1.920471833794928, 1.9011626627744007, 1.8847210088654247, 1.8678614023091542, 1.8524708261575236, 1.8345866060013052, 1.8187231174515337, 1.7921843912900257, 1.7833346739754348, 1.7667059096533928, 1.7446447648965488, 1.7294791877422186, 1.713704973383023, 1.6945127424071817, 1.6837198462937495, 1.662628272000481, 1.6420564162151894, 1.6315715404422693, 1.6128088231281856, 1.5899239063567823], 'val_metric': [4.06, 4.74, 6.98, 8.74, 9.87, 11.9, 12.42, 13.36, 14.37, 15.47, 15.75, 16.21, 17.35, 18.1, 19.06, 19.72, 20.3, 20.96, 21.49, 22.1, 22.55, 23.12, 23.52, 24.06, 24.3, 24.92, 25.41, 26.25, 26.33, 26.52, 27.66, 27.36, 28.05, 28.61, 28.7, 29.22, 28.79, 29.62, 30.52, 30.7, 31.39, 30.88, 31.25, 31.92, 31.19, 32.43, 33.3, 31.02, 32.13, 34.13, 34.73, 35.07, 33.48, 34.92, 35.36, 34.88, 34.77, 36.63, 35.62, 36.74, 36.16, 37.06, 37.03, 37.57, 38.06, 37.29, 37.47, 37.22, 38.98, 39.14, 37.42, 39.63, 38.04, 39.98, 39.26, 40.44, 40.7, 39.0, 41.35, 40.5, 40.29, 41.75, 42.31, 41.89, 41.16, 41.96, 41.71, 43.02, 42.54, 41.49, 42.95, 43.22, 43.53, 41.27, 43.6, 44.17, 45.13, 43.73, 43.61, 41.79], 'val_loss': [4.4242434319417185, 4.311705258241885, 4.179144604190899, 4.074316364944361, 3.9713271347580443, 3.8756035482807523, 3.7984061560053735, 3.7455114027497114, 3.68055754406437, 3.6344144891022117, 3.5904267396137213, 3.5613118918838014, 3.5105413449038365, 3.4761189882922325, 3.4351992910834634, 3.4018956658187185, 3.3732911538166603, 3.3411918293898273, 3.303473061057413, 3.2737563795344844, 3.2443127161378316, 3.2208359347786875, 3.184414201481327, 3.158402480896871, 3.1391796230510542, 3.102321851025721, 3.0799280913772096, 3.0630517415939624, 3.020711523712061, 3.0048228297263955, 2.9684552584484125, 2.95423951118615, 2.9277448411200457, 2.915883867603958, 2.87494229055514, 2.871780093308467, 2.871270258715198, 2.8278685800588814, 2.8186730791808694, 2.7952975027120797, 2.7561369823042754, 2.7579792517765314, 2.7541144790163465, 2.7209571243091752, 2.747192549857364, 2.696518908640382, 2.6548841698154524, 2.8121486925015784, 2.7028074036737917, 2.6166959203732243, 2.6024679272038163, 2.578532979746533, 2.6474650240248176, 2.5782660542020372, 2.547911062362088, 2.585765556165367, 2.571050435874113, 2.4976158028195616, 2.5396268261466055, 2.4981550646435684, 2.510739108559432, 2.465240294766274, 2.4829411020704137, 2.4651446615814403, 2.438873163454092, 2.499646886898454, 2.446013098309754, 2.458696006209987, 2.3879534279464916, 2.3757560473338812, 2.458949448956046, 2.3576243014851954, 2.4020883566255025, 2.343635054910259, 2.3735211624461376, 2.307355940721597, 2.325563509752796, 2.389449159051203, 2.2767305495632684, 2.323919388139324, 2.309456519260528, 2.262880436174429, 2.2296987384747546, 2.233212341928178, 2.2770111226731804, 2.2581994487981127, 2.245116681050343, 2.208013757778581, 2.2233509417552098, 2.2763855130809127, 2.1909007152933984, 2.214069718767883, 2.184813422761905, 2.3060197731491865, 2.1955418320977764, 2.1713920740564916, 2.1064029499224035, 2.158950920317583, 2.1814234484532835, 2.2767069294194506]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="LSUV", optimizer="Adam")
data.update({'train_metric': [8.42, 16.094, 21.968, 28.17, 33.134, 37.042, 40.09, 42.566, 43.974, 45.52, 46.99, 47.836, 48.86, 49.536, 50.278, 50.838, 51.526, 51.936, 52.638, 53.014, 53.718, 53.912, 54.12, 54.47, 54.858, 55.18, 55.764, 55.908, 56.134, 56.516, 56.734, 56.826, 57.084, 57.384, 57.452, 57.78, 57.838, 58.062, 58.032, 58.43, 58.52, 58.758, 58.902, 58.734, 59.032, 59.158, 59.38, 59.234, 59.736, 59.806, 59.842, 59.962, 59.932, 60.124, 60.1, 60.262, 60.23, 60.134, 60.512, 60.442, 60.622, 61.056, 60.864, 60.864, 60.888, 61.006, 61.16, 61.26, 61.37, 61.094, 61.272, 61.65, 61.44, 61.88, 61.614, 61.632, 61.89, 61.998, 61.842, 61.806, 62.078, 61.97, 62.176, 62.108, 62.086, 62.068, 62.13, 62.502, 62.546, 62.266, 62.252, 62.382, 62.714, 62.402, 62.424, 62.536, 62.882, 62.716, 62.49, 62.546], 'train_loss': [4.011009982174925, 3.4750942138149914, 3.133990966450528, 2.804005144502196, 2.5570022768681615, 2.373639222910947, 2.235567927665418, 2.131460006737038, 2.0592027006246854, 1.9994439337869434, 1.9401442088434457, 1.9006468614044092, 1.863655570675345, 1.8279939793869662, 1.8021489139408102, 1.7761691307167873, 1.7497597138595093, 1.7289976305364039, 1.705920413022151, 1.6869762559680987, 1.6681533497007912, 1.6536898032173781, 1.6399218978174508, 1.626473951675093, 1.6149130472746651, 1.6041970201160596, 1.588259968763727, 1.5744001303639863, 1.566292635467656, 1.5550186696564754, 1.543577198939555, 1.5406742295645692, 1.5283571269048755, 1.5191732348535982, 1.5087875428102206, 1.50770006460302, 1.5010037871120532, 1.4913291332057066, 1.4854206560975145, 1.4791240469574014, 1.4723784173541057, 1.4679381033343732, 1.4595436414184473, 1.459035049695188, 1.4494949439754876, 1.4495311021195043, 1.437966914280601, 1.440024863423594, 1.4339139617007712, 1.425255036948587, 1.4253025352954865, 1.42149701943178, 1.4188537706651956, 1.4114111974416181, 1.40945726937955, 1.4055855420544325, 1.401008422417409, 1.4061950945183443, 1.3958735869973518, 1.3934322564345796, 1.3878479035918976, 1.382874663528579, 1.3840148855200813, 1.380295204232111, 1.3752637253240552, 1.3770456839248042, 1.3717091151363099, 1.367463448011052, 1.3682962351138024, 1.3685614593955866, 1.3582422608304816, 1.3566219428616106, 1.3571614475963671, 1.3446251745419124, 1.3523643095322582, 1.347583982645703, 1.3448338818824506, 1.3455337564963514, 1.3499619845691544, 1.339017045467406, 1.3426470102556527, 1.3375046556563024, 1.3322346216577399, 1.3327379861603612, 1.3358818468687785, 1.3302695548442929, 1.323944772829485, 1.3219556981492835, 1.320510638796765, 1.3273996595683915, 1.321741276842249, 1.3188624910229003, 1.314071974943361, 1.3151840856465538, 1.3150541956162514, 1.3146110475825532, 1.3114381325061975, 1.3038323615365626, 1.315801245629635, 1.306064189547468], 'val_metric': [11.95, 16.38, 23.39, 26.52, 31.21, 37.41, 40.86, 43.01, 41.39, 43.42, 44.67, 47.07, 48.03, 45.75, 45.71, 48.76, 49.49, 48.44, 48.36, 50.76, 50.3, 47.78, 49.24, 49.93, 52.34, 52.26, 53.37, 52.56, 52.17, 52.2, 51.07, 52.47, 53.79, 52.92, 54.63, 53.76, 52.77, 53.61, 53.59, 52.36, 53.56, 53.54, 52.62, 52.86, 52.65, 54.64, 53.86, 55.34, 54.71, 54.59, 54.27, 55.89, 53.95, 55.33, 53.84, 56.28, 53.17, 53.42, 54.81, 56.56, 56.14, 54.04, 56.11, 55.02, 54.12, 55.96, 55.66, 56.49, 54.62, 54.38, 54.97, 55.37, 54.68, 56.34, 53.34, 55.83, 54.85, 55.13, 56.09, 55.26, 52.83, 55.23, 56.05, 51.27, 57.54, 54.95, 57.14, 55.97, 55.91, 56.78, 56.55, 55.67, 55.74, 55.58, 56.39, 56.18, 55.21, 53.29, 55.8, 55.98], 'val_loss': [3.7339648866349724, 3.448982905430399, 3.0418531074645414, 2.8879617520957996, 2.657435234944532, 2.3468749515569893, 2.194063696891639, 2.125453028709266, 2.170278093617433, 2.091868891837491, 2.0369177137970165, 1.9401206074246935, 1.916914307387771, 1.9940144157713386, 1.981264254090133, 1.8463004273214159, 1.819821989460356, 1.8770880904167322, 1.8899811012729717, 1.781202231243158, 1.797407251254768, 1.911666956676799, 1.8921468508471349, 1.8426141571846737, 1.7086842644746136, 1.7060873349001453, 1.6803674910478532, 1.6906012349827275, 1.7139537030724203, 1.7195928818101336, 1.7409241791743382, 1.717859654669549, 1.6555868372036393, 1.6967101476754352, 1.6212726520125273, 1.650323954357463, 1.6707631425493081, 1.6397111514571365, 1.6630585588467348, 1.7602835324159853, 1.654626463249231, 1.6633115217184564, 1.720575615858576, 1.6825318913550893, 1.7300560049190643, 1.6323532116640904, 1.6563888423761743, 1.5913324143476546, 1.6201436451286266, 1.6250283376426453, 1.6422612188727992, 1.5594424518050662, 1.6751230696963657, 1.6050014389548333, 1.6621515348458746, 1.567306843532878, 1.709066563351139, 1.6601977834276334, 1.6171195248889316, 1.5613146519205372, 1.5554818484433897, 1.653369646163503, 1.5594937368563027, 1.6191999661694667, 1.6537103918707294, 1.5631055862280974, 1.5888767652450853, 1.5589743336294866, 1.6344636139596345, 1.615708987424328, 1.6192401670346594, 1.6047468086716477, 1.6227606762746336, 1.5633658861658375, 1.6917009900329978, 1.5532355407241043, 1.6175713801080254, 1.6315566726550934, 1.5568435009877393, 1.5821088127269867, 1.7048164272004631, 1.6075109387658963, 1.5707959179665631, 1.8180906112027015, 1.5027316337937762, 1.5849217016985462, 1.5203985332683394, 1.5712407014931842, 1.5683246524470627, 1.546419913222076, 1.5631485334627189, 1.5839714867294215, 1.5914948366250201, 1.5990137469237018, 1.5583873311425471, 1.5952599724386907, 1.614234846109038, 1.6855416905348468, 1.5969594489237307, 1.5904729799100548]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="LSUV", optimizer="AdamW")
data.update({'train_metric': [8.652, 19.32, 29.572, 39.19, 46.424, 52.166, 56.128, 60.836, 64.806, 67.91, 71.07, 74.164, 77.308, 79.848, 82.6, 84.716, 86.862, 88.834, 90.432, 91.54, 92.658, 93.37, 94.186, 94.65, 95.046, 95.478, 95.972, 95.954, 96.194, 96.386, 96.86, 96.91, 97.098, 97.23, 97.258, 97.548, 97.382, 97.68, 97.808, 97.48, 98.068, 97.918, 98.132, 98.032, 98.034, 98.25, 98.244, 98.396, 98.202, 98.422, 98.45, 98.574, 98.514, 98.56, 98.64, 98.884, 98.52, 98.602, 98.648, 98.62, 98.9, 98.862, 98.656, 98.98, 98.858, 98.94, 98.794, 98.802, 99.1, 98.892, 99.098, 98.952, 99.182, 98.872, 99.028, 99.238, 99.042, 99.172, 99.17, 99.024, 99.154, 99.184, 99.208, 99.276, 99.138, 99.218, 99.268, 99.16, 99.172, 99.272, 99.36, 99.032, 99.32, 99.33, 99.172, 99.208, 99.334, 99.332, 99.356, 99.246], 'train_loss': [3.9966345132159455, 3.301480365226336, 2.757809374948292, 2.2897755837501466, 1.9559137307469496, 1.7173678791126632, 1.5474634276479102, 1.3649511365481959, 1.2174110257869486, 1.0892570334322311, 0.9735094882795573, 0.8559432535448952, 0.7428998128151345, 0.6485973154873494, 0.5528792023201428, 0.4763152727004512, 0.4064941303351956, 0.34343115947283137, 0.29567751018306637, 0.2587810425692812, 0.22524895203654724, 0.20100555716134855, 0.17906065534828874, 0.16180324250036646, 0.15085695803051105, 0.13974143786932272, 0.12787269398241358, 0.12593208813844511, 0.1206192638024764, 0.11289689680585242, 0.09647193881552048, 0.09727268361384073, 0.08860399365625189, 0.08705636846792439, 0.08366931961072832, 0.07844258042450165, 0.08312952075310795, 0.07179786092446894, 0.07154723894282165, 0.07653879443221652, 0.0606932438567014, 0.06547753165876183, 0.05889752473625, 0.062195679624838865, 0.061018213303997884, 0.05510373209850848, 0.05639097677267459, 0.050256524891521104, 0.0565760457004561, 0.0513121236116885, 0.049302387792352693, 0.04735992917977988, 0.046716272626119805, 0.047610510109395, 0.04425360335994636, 0.03707336374368552, 0.04736946960952838, 0.04432028037411949, 0.04179150415377517, 0.04357594960798507, 0.0354450875166756, 0.03752680959226445, 0.04395276865453871, 0.03314654080523535, 0.03701989639558188, 0.03505529787050812, 0.03722958919583865, 0.03756597926712159, 0.029740762687265894, 0.034904205632578254, 0.029937564783245493, 0.03375462111053795, 0.02766372407307563, 0.035553514214424306, 0.03194835765340575, 0.025305740701098978, 0.03117124758550035, 0.026607743129865122, 0.029172753979501975, 0.031201780756653818, 0.028867708088870604, 0.027819842654660722, 0.025169270768914374, 0.025147467475447594, 0.028059367245510388, 0.02587914756775978, 0.0249777245610688, 0.02643047199528569, 0.02655371441431415, 0.022534663681230565, 0.020550349594016894, 0.03105097733240913, 0.02139831089124809, 0.02248074752822369, 0.025964668013013736, 0.024257161645748704, 0.0224245486942077, 0.021872137376559686, 0.021872192971295824, 0.024375302648312315], 'val_metric': [11.77, 22.16, 32.56, 42.73, 48.56, 50.79, 56.78, 56.81, 58.25, 61.64, 61.48, 63.5, 62.7, 64.48, 64.33, 65.6, 65.3, 65.03, 65.32, 65.41, 65.59, 64.34, 66.3, 66.59, 66.11, 66.04, 65.63, 66.57, 66.84, 67.11, 67.39, 66.17, 66.57, 67.05, 67.02, 66.19, 66.79, 67.28, 67.52, 67.87, 67.27, 67.22, 67.39, 67.24, 66.34, 67.2, 67.8, 67.25, 67.8, 67.96, 67.44, 68.49, 68.83, 68.28, 67.47, 67.33, 68.41, 68.32, 68.28, 68.13, 68.76, 67.64, 67.42, 67.56, 68.42, 67.64, 67.65, 67.54, 68.63, 68.09, 67.87, 68.43, 68.86, 68.1, 67.71, 69.19, 67.76, 68.17, 69.26, 68.97, 67.45, 67.74, 68.82, 67.9, 69.24, 67.75, 68.29, 67.91, 69.19, 69.25, 68.41, 68.73, 68.3, 68.33, 68.53, 68.96, 68.65, 69.31, 68.35, 68.36], 'val_loss': [4.750533968020397, 3.232449730490423, 2.6609899101743273, 2.1952515271059267, 2.1689944608955627, 1.8137082772649777, 1.554971390469059, 1.5494911408728096, 1.5298372305882204, 1.3726185833572582, 1.4003826087447488, 1.3432699016704681, 1.3955681434102878, 1.3135525386804228, 1.378435566926458, 1.4028412764239464, 1.4117463762593117, 1.4781424847378093, 1.5494051814838579, 1.5166682058079228, 1.5964616263747975, 1.7005207390542243, 1.6210599940293913, 1.6153203939935963, 1.677350050324847, 1.7192835709091965, 1.7741759802885115, 1.6995907399305112, 1.701434806274001, 1.6892922866116664, 1.751247440933422, 1.8102743469985427, 1.8484415928269649, 1.846223841427238, 1.829415044207482, 1.8076444051827594, 1.8224545603345155, 1.7908251555102646, 1.8641035909865313, 1.802001433767331, 1.8802882417751725, 1.8019061411262318, 1.8699681132462374, 1.8299957544180998, 1.862491519208167, 1.8549501960444603, 1.8695007034927418, 1.8949819427386971, 1.847766584271838, 1.8589713543084017, 1.8563407264697325, 1.8355472391578043, 1.8119845545975266, 1.8466382193717228, 1.9496647878816933, 1.8959079859363046, 1.9668313872282672, 1.9086218654729759, 1.8632506538348592, 1.8762700880409047, 1.9337730282431196, 2.011575240238457, 1.925222446584398, 1.943405778924371, 1.8315498289788605, 1.924479379016123, 1.9718416576172895, 1.9672134313613745, 1.94262121513391, 1.9534440872016225, 1.9481777229886146, 1.9432301806036834, 1.8823140161052632, 1.9492868447000054, 2.0157404826704863, 1.8914175606836938, 1.9803644016290167, 1.9638221878914317, 1.9343765210953487, 1.946271580495652, 1.997661346083234, 1.9574545902811038, 1.9339627745045218, 1.957985289157576, 1.9307625802459232, 2.1235695094059985, 1.9920213306026093, 2.0667704913266904, 1.9543514547833971, 1.886619859060664, 2.0255421730363445, 1.9351729154586792, 2.020369807626032, 2.0352949944271406, 2.0269101174773683, 1.93476715475131, 1.9718478451109236, 1.9999026861160425, 1.9935760961216726, 2.004621039150627]})


# MobileNetV3 Tiny-Imagenet
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal 1", optimizer="SGD")
data.update({'train_metric': [0.631, 0.736, 0.838, 0.852, 0.882, 0.946, 1.027, 1.153, 1.221, 1.274, 1.303, 1.289, 1.456, 1.526, 1.653, 1.766, 1.872, 1.932, 2.003, 2.028, 2.081, 2.16, 2.23, 2.362, 2.318, 2.413, 2.473, 2.588, 2.598, 2.678, 2.84, 2.83, 2.94, 3.087, 3.177, 3.185, 3.46, 3.528, 3.655, 3.758, 3.884, 4.136, 4.079, 4.344, 4.38, 4.65, 4.726, 4.936, 5.087, 5.257, 5.521, 5.501, 5.876, 6.114, 6.197, 6.348, 6.498, 6.706, 6.961, 7.129, 7.513, 7.478, 7.947, 8.015, 8.339, 8.506, 8.623, 8.834, 9.057, 9.339, 9.371, 9.634, 9.741, 9.951, 10.246, 10.359, 10.426, 10.486, 10.708, 11.043, 11.053, 11.333, 11.414, 11.646, 11.728, 12.012, 12.033, 12.214, 12.464, 12.569, 12.715, 12.918, 13.051, 13.162, 13.364, 13.572, 13.78, 13.744, 13.991, 14.175, 14.241, 14.493, 14.713, 14.72, 15.024, 15.261, 15.453, 15.485, 15.72, 15.898, 16.233, 16.198, 16.446, 16.456, 16.785, 17.014, 17.286, 17.391, 17.757, 17.918, 17.992, 18.187, 18.405, 18.581, 18.835, 19.031, 19.119, 19.484, 19.522, 19.674, 19.986, 20.073, 20.426, 20.631, 20.745, 21.053, 21.11, 21.285, 21.751, 21.807, 21.955, 22.165, 22.259, 22.526, 22.841, 22.952, 23.149, 23.337, 23.609, 23.58, 23.8, 24.195, 24.21, 24.249, 24.905, 24.91, 24.946, 25.056, 25.292, 25.638, 25.767, 25.852, 26.175, 26.309, 26.357, 26.545, 26.873, 26.892, 27.059, 27.395, 27.482, 27.777, 27.984, 28.026, 28.204, 28.434, 28.541, 28.76, 28.964, 29.15, 29.275, 29.63, 29.882, 29.737, 30.284, 30.104, 30.219, 30.574, 30.663, 30.853, 31.012, 31.261, 31.381, 31.401, 31.8, 31.741, 32.033, 32.149, 32.409, 32.571, 32.952, 32.915, 33.068, 33.226, 33.469, 33.66, 33.553, 34.103, 34.151, 34.356, 34.359, 34.779, 34.769, 34.923, 35.091, 35.233, 35.417, 35.549, 35.851, 35.899, 35.874, 36.17, 36.364, 36.382, 36.642, 36.949, 36.872, 37.319, 37.192, 37.57, 37.665, 37.869, 37.786, 38.065, 38.449, 38.378, 38.494, 38.731, 39.028, 38.896, 39.134, 39.27, 39.466, 39.771, 39.759, 39.681, 39.883, 40.133, 40.335, 40.545, 40.608, 40.732, 41.036, 41.017, 41.303, 41.431, 41.495, 41.714, 42.006, 42.178, 42.125, 42.106, 42.404, 42.64, 42.838, 43.024, 43.165, 43.229, 43.303, 43.516, 43.676, 43.873, 43.755, 44.057, 44.034, 43.985, 44.338, 44.556, 44.689, 44.848, 45.009, 45.106, 45.082, 45.573, 45.46, 45.662, 45.907, 46.037, 45.939, 46.293, 46.281, 46.627, 46.641, 46.744, 46.952, 47.046, 47.261, 47.22, 47.273, 47.439], 'train_loss': [5.308517751675421, 5.27941758679947, 5.263842275794012, 5.254387417742631, 5.24702159792509, 5.24030117003184, 5.234365646608808, 5.2275125722586155, 5.221151207626743, 5.214442143589735, 5.207548621977589, 5.199330211219617, 5.188407007121002, 5.17664202649244, 5.161747843656339, 5.14383597993271, 5.123288286700893, 5.099472871745006, 5.07345543896168, 5.045433916652042, 5.020425918234974, 4.997833956073509, 4.979357364615491, 4.963236180010775, 4.950127363967652, 4.936576165378056, 4.924006058478767, 4.912233614723269, 4.900890454678526, 4.889054491210274, 4.875695962289626, 4.862693252124164, 4.852075736948259, 4.839455996578653, 4.827967016802182, 4.814345951653869, 4.799268471759936, 4.781189809910243, 4.76575131944106, 4.747813924641771, 4.730974606497503, 4.713576537710082, 4.693969758290629, 4.67628687036701, 4.656521076890649, 4.638686693256205, 4.619121464261319, 4.603755301111262, 4.584270534649615, 4.568431858214063, 4.549503180283579, 4.535285709610522, 4.514659916370706, 4.499660150865981, 4.484191990020713, 4.469650616038708, 4.453982184807307, 4.438093722057282, 4.419480833882181, 4.406391429046904, 4.38945510138782, 4.37541567005565, 4.358885756228417, 4.340777654946804, 4.324550850453929, 4.30670316602203, 4.290402850354244, 4.274725674210034, 4.259197757858843, 4.242455712054222, 4.225628486292834, 4.211743367763185, 4.195922275422402, 4.183271207797245, 4.167069501931745, 4.152866711314489, 4.140763081271757, 4.126076729573733, 4.111735758076702, 4.100021919072323, 4.0845643372697396, 4.071997468317463, 4.05950559733849, 4.041249699418696, 4.0325403576543986, 4.0204862004583335, 4.007246813252425, 3.9966930803090275, 3.9829083901525535, 3.9676804327461404, 3.9574992717723396, 3.9452338453599145, 3.9353663434756543, 3.92308635934377, 3.909740936123097, 3.899067889553419, 3.8855531281259528, 3.8752655108922274, 3.8609185920490794, 3.849980662895637, 3.839635900557232, 3.8289659342091587, 3.8165686661359675, 3.80335392314352, 3.7928651973214276, 3.7812071876196396, 3.767868474135396, 3.7544233887453378, 3.744619715312926, 3.728720238326226, 3.7129904555725273, 3.704679484407031, 3.6929342958763938, 3.6785020774858395, 3.6659229979328978, 3.653661379506019, 3.6398139741278883, 3.6268378617438612, 3.6111909415732573, 3.6011177189290637, 3.5871532117412857, 3.5747381287817, 3.5603847627218763, 3.548440062839559, 3.5385159912280217, 3.5211667177086827, 3.5109796684213883, 3.496259370066764, 3.4840843875821554, 3.471384050444922, 3.4578188362597504, 3.4466760410228297, 3.4332276276877045, 3.4219015797818235, 3.408010750341629, 3.396798954052721, 3.384726916836075, 3.3717566639356558, 3.358999151338466, 3.348180280460887, 3.3347844421596613, 3.3194361196751023, 3.313550555850936, 3.2990375271380445, 3.2847680877739243, 3.2727535704504733, 3.2600756214737356, 3.24637516995538, 3.2362922232500346, 3.2283884255609983, 3.2149810548478492, 3.204400657921057, 3.1927836736043296, 3.183068456668085, 3.165892516201456, 3.1567424690578507, 3.1472699210495807, 3.133715631713184, 3.1264628549454994, 3.1134840357555307, 3.1027240902662125, 3.0917247284396825, 3.0784285573416312, 3.070010478269268, 3.057467190943234, 3.0477301590883497, 3.0359899491281443, 3.0239158582778902, 3.0150654535604757, 3.0053522251236537, 2.9929803212483725, 2.981002953444546, 2.9700387863493547, 2.9598863753308415, 2.948276210731218, 2.943516821260263, 2.929249384276621, 2.9185404487702606, 2.9075575334020556, 2.89600974462464, 2.8870449397171907, 2.8710236845303254, 2.864496780982478, 2.86264551105365, 2.842557114206364, 2.835537187883813, 2.8310911675256105, 2.8152769211386537, 2.8098348277696643, 2.79731394782405, 2.7857494177302717, 2.77610760229334, 2.7655849642884784, 2.7597623046818867, 2.7442458999973036, 2.7446945901871946, 2.728050428098848, 2.7205135482134, 2.7103017744770894, 2.7032138627077322, 2.6878975238543785, 2.6818687082938637, 2.673249499887819, 2.667545892913145, 2.6511181921662996, 2.6427814879634024, 2.641307188468489, 2.6286946476382926, 2.619845787836662, 2.607463766120598, 2.5994429708823743, 2.5916494883105914, 2.588046056752928, 2.5773901870749505, 2.566386573221618, 2.555797048737739, 2.545083871577233, 2.5389001903363093, 2.5284126942842646, 2.524792007201006, 2.516420968129554, 2.502976675637624, 2.492505836547832, 2.486951534975361, 2.481384435908122, 2.4655725641732635, 2.460631878270755, 2.4514407869035133, 2.449924992431987, 2.437351699143896, 2.427997234381702, 2.4155243908985256, 2.4135368039649188, 2.4058932125453985, 2.3935596169528486, 2.3853247601941696, 2.381513585437206, 2.369894912131536, 2.3589281845916483, 2.3681037731835723, 2.35265259649688, 2.3406731380687185, 2.3362999162228535, 2.325058579826233, 2.3185781903245553, 2.316522996774943, 2.306819164897872, 2.3009722664504193, 2.28790818973756, 2.2758104187970885, 2.2705437943329816, 2.265274377717319, 2.2570158826641293, 2.2507055679041836, 2.2398798236920143, 2.232602493700429, 2.2268765462337208, 2.218897671403598, 2.209244536270488, 2.2005874906223855, 2.199502398093694, 2.195308734725396, 2.18388829296854, 2.1778019522522323, 2.1677245530118108, 2.16058727692734, 2.1479606238680624, 2.1458288123038667, 2.14167990535021, 2.132316323403586, 2.125439146658738, 2.1116893273931394, 2.110154849325169, 2.102081324607229, 2.1009243817491097, 2.0984209011703903, 2.0922415742337206, 2.0777433750649257, 2.0708998933634692, 2.06090908331209, 2.059407175998236, 2.0519746923324624, 2.045945753291564, 2.033916527311236, 2.0363517584742756, 2.0218637412889446, 2.0173720085536986, 2.0126316643493416, 2.0066014500054807, 1.9984684344369177, 1.9915546310001355, 1.9872004845473374, 1.9767988824112173, 1.9748332977905079, 1.9680495091302228, 1.9592663140458628, 1.9530331324249677, 1.949947727390077, 1.9460340441608002, 1.9398187070188808], 'val_metric': [0.55, 0.77, 0.95, 0.87, 1.01, 1.01, 1.17, 1.17, 1.32, 1.14, 1.19, 1.27, 1.39, 1.58, 1.66, 1.81, 2.09, 2.13, 2.13, 2.08, 2.27, 2.47, 2.33, 2.54, 2.46, 2.63, 2.72, 2.94, 2.95, 3.05, 3.14, 3.31, 3.23, 3.28, 3.28, 3.37, 3.52, 3.5, 4.07, 4.0, 4.09, 4.44, 4.41, 4.68, 4.54, 4.84, 4.88, 5.17, 5.25, 5.46, 5.71, 5.84, 6.25, 6.26, 6.35, 6.75, 6.84, 7.02, 7.22, 7.25, 7.46, 7.88, 7.96, 8.33, 8.46, 8.6, 8.91, 8.94, 9.15, 9.19, 9.69, 9.54, 9.84, 9.9, 10.18, 10.45, 10.38, 10.81, 10.98, 11.36, 11.38, 11.44, 11.93, 11.85, 11.8, 11.79, 12.13, 12.21, 12.3, 12.55, 12.68, 12.53, 12.28, 13.02, 13.18, 13.06, 13.21, 13.37, 13.0, 13.42, 13.32, 13.4, 13.5, 13.69, 13.61, 13.65, 13.68, 13.94, 14.26, 14.28, 14.41, 14.24, 14.38, 14.52, 14.53, 14.55, 14.75, 14.71, 15.07, 15.2, 15.4, 15.25, 15.28, 15.15, 15.35, 15.48, 15.78, 15.92, 15.8, 15.63, 16.08, 15.79, 16.05, 15.97, 15.89, 16.18, 16.22, 16.59, 16.58, 16.74, 16.33, 16.61, 16.74, 16.91, 16.85, 16.92, 16.63, 16.7, 16.83, 17.05, 16.81, 17.13, 17.08, 17.04, 17.17, 17.36, 17.38, 16.71, 17.36, 17.25, 16.9, 17.06, 16.88, 17.48, 16.66, 16.87, 17.36, 17.11, 17.42, 17.18, 16.78, 17.32, 16.99, 16.95, 17.65, 17.34, 17.04, 16.84, 17.09, 16.89, 17.15, 16.87, 16.67, 17.02, 17.3, 16.3, 16.93, 17.02, 17.23, 17.2, 17.21, 16.9, 17.37, 17.21, 16.95, 17.37, 17.66, 16.78, 17.07, 17.69, 17.39, 17.16, 17.09, 17.18, 17.02, 17.01, 16.93, 17.32, 17.0, 17.09, 17.37, 16.78, 17.29, 16.83, 16.86, 17.37, 17.39, 17.01, 16.88, 17.31, 17.01, 17.26, 16.51, 16.65, 17.03, 17.43, 16.59, 17.63, 16.71, 16.83, 16.81, 17.51, 16.84, 17.16, 16.97, 16.55, 16.64, 16.62, 16.54, 16.42, 16.73, 16.58, 16.74, 16.7, 16.75, 16.79, 16.99, 17.08, 16.81, 17.08, 16.65, 16.39, 16.82, 17.42, 16.45, 16.12, 16.44, 16.48, 16.76, 16.93, 16.53, 16.29, 16.52, 16.32, 16.2, 16.83, 16.77, 16.21, 17.12, 16.26, 16.49, 16.65, 15.39, 15.79, 16.0, 16.28, 15.99, 16.89, 16.56, 16.57, 16.17, 16.19, 16.4, 16.42, 15.55, 16.31, 16.65, 16.88, 16.64, 15.93, 16.38, 15.83, 16.47, 17.01, 16.43, 16.9, 16.46, 16.06, 16.4, 16.32], 'val_loss': [5.284060948973249, 5.265720832120081, 5.254109807834504, 5.24605655670166, 5.241465088668143, 5.231841518620777, 5.228367225379701, 5.220900666182208, 5.213471364063822, 5.206508642548968, 5.199013224073276, 5.191600395615693, 5.181637636415518, 5.168535354031119, 5.152738270486236, 5.131313536577164, 5.106356646604599, 5.080028611383621, 5.054598304116801, 5.025429484191214, 4.999610346593674, 4.977994773038633, 4.958132841025188, 4.945619276374768, 4.92917467226648, 4.919764506589075, 4.904640223569931, 4.891254379491138, 4.876092648050587, 4.863095582670467, 4.854904076096359, 4.841477102534786, 4.829578627446654, 4.820621493515695, 4.804379476863108, 4.789820583003341, 4.774665691290691, 4.763733112128677, 4.741800687874958, 4.72708651973943, 4.709826152036144, 4.68720152271781, 4.672186849982875, 4.652529595004525, 4.631043677117415, 4.615047954449988, 4.600119504199665, 4.5807633749239, 4.560607825115228, 4.556337581318655, 4.533163717597913, 4.513555537363526, 4.499819313644603, 4.483945122190342, 4.479910212717239, 4.452643693632381, 4.444709474114096, 4.4312653981955945, 4.408368034727254, 4.417285648880491, 4.377833685297875, 4.3632752743496255, 4.34909509397616, 4.330458850617621, 4.320501441408874, 4.29798276864799, 4.28971144955629, 4.2731779684686355, 4.265861201438175, 4.2487839559081255, 4.231349838767082, 4.226045979056389, 4.208555901885792, 4.189204421013025, 4.183312268773461, 4.159145564789984, 4.162373403075394, 4.134311132370287, 4.126510641377443, 4.122789522644821, 4.106294335833021, 4.099818957079748, 4.082720393587829, 4.07054956855288, 4.060247251182605, 4.057349132124785, 4.051010388477593, 4.040155047823669, 4.031272633060528, 4.030205873926734, 4.011285526737286, 4.006157937323212, 4.006965871069841, 3.9851171499604634, 3.979403919475094, 3.9706035085544467, 3.970184153052652, 3.958392147805281, 3.9603980346849768, 3.9514027355582853, 3.9404081523798076, 3.939072262709308, 3.9255078901910476, 3.92857027053833, 3.93940740785781, 3.9133706928058793, 3.9152182029311065, 3.8948559032124317, 3.891510075064981, 3.8835065197792784, 3.8755349566222757, 3.8742736418535757, 3.866550059834863, 3.867265792409326, 3.861668542691856, 3.8654167940662165, 3.8481376975964587, 3.8436009215701157, 3.8471756118118385, 3.8267689799047577, 3.8198849942274156, 3.817828814694836, 3.811782835395473, 3.818688684208378, 3.8174659552847503, 3.81088610363614, 3.801663037318333, 3.800055680001617, 3.797009486301689, 3.789724381865969, 3.78689104432513, 3.8093444283600824, 3.7901444450305526, 3.7875989106050723, 3.8025126092752832, 3.7984240358802164, 3.802506835597336, 3.781847642485503, 3.7766720291915212, 3.7717675934931276, 3.7924296461093197, 3.7880241384931432, 3.7833873010744714, 3.7607011293909354, 3.775602758310403, 3.779354702894855, 3.7900450244830672, 3.7894133169939566, 3.7620383781991946, 3.7736135895844476, 3.7953082801430087, 3.7852717979698425, 3.7773161769672563, 3.813860348075818, 3.8162443015226133, 3.800411358001126, 3.785344296959555, 3.8268550368630962, 3.7842491386802335, 3.8250730280663556, 3.8237189517658985, 3.8361773809809594, 3.828982748043765, 3.851934085226363, 3.8316883676370996, 3.8628718716323753, 3.858248088010557, 3.8426734900018973, 3.846711807190233, 3.883518799095397, 3.8739695252886244, 3.8850881537054756, 3.874004625211096, 3.880443853937137, 3.896890756430899, 3.882723837141778, 3.9165207580396326, 3.9190043841197992, 3.9037759106629975, 3.944441434684073, 3.930733940403932, 3.950320128422634, 3.9591658282431825, 3.9053224272029414, 3.9523440333688336, 4.02532879106558, 3.9576653234518258, 4.017918082559184, 3.9962985211876547, 3.9968529567597018, 3.9806844170685785, 3.9987544071902135, 4.0056363245484174, 4.0002151507480885, 4.025412298311853, 3.9953067834210243, 4.037439148896818, 4.014607125786459, 4.052643233803427, 4.030140068880312, 4.051973971591633, 4.079441848074555, 4.076678758973529, 4.0738479255870645, 4.102327456140214, 4.101727599550964, 4.1372507134820244, 4.0907789795262035, 4.147429423727048, 4.157630517983892, 4.110607605830879, 4.172785549406793, 4.144953492340768, 4.159627294844123, 4.181155646682545, 4.133312234453335, 4.170518074825311, 4.177621401039658, 4.262410300552466, 4.18446032560555, 4.207397470049038, 4.245145612461552, 4.246166081944848, 4.297600167572119, 4.274525041033508, 4.223782748933051, 4.310832886179542, 4.2511371396909094, 4.287093511812246, 4.3008732932388405, 4.305187240527694, 4.3246313827053, 4.40287990630812, 4.285760864330705, 4.351387525060375, 4.356317539883268, 4.357223433294114, 4.372018041124769, 4.377908440911845, 4.420791322258627, 4.3858345602728, 4.370007612143352, 4.405517503714106, 4.420067579123624, 4.432475544085168, 4.440114113935239, 4.502578209919535, 4.394306445577342, 4.46965780075948, 4.432576542447327, 4.445176824642594, 4.542408028985284, 4.48716950112847, 4.470113531039779, 4.487770224832426, 4.547371314589385, 4.514542643431645, 4.530906066773044, 4.60301277440065, 4.626159178982874, 4.604091752866271, 4.582748198964794, 4.55040253499511, 4.649818206289012, 4.543143809980648, 4.625881966511915, 4.654242416855636, 4.658584377568239, 4.601723534286402, 4.7164740311871665, 4.600960845400573, 4.659727708549257, 4.690768059651563, 4.720667117720197, 4.680859476138073, 4.662140208444778, 4.698159709857528, 4.696303880898056, 4.665212540110206, 4.7822133067307195, 4.7352171414976665, 4.790127606908227, 4.7915195872069924, 4.751599740070902, 4.822968288591713, 4.763484746787198, 4.809852376864974, 4.7961967325514285, 4.819475852759781, 4.894821473747302, 4.776001758636183, 4.83029383458909, 4.857027046240059, 4.806372569624785, 4.953485341588403, 4.868705716102746, 4.8506805577855205, 4.887474485263703, 4.877608168656659, 4.821981068629368]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal 1", optimizer="Adam")
data.update({'train_metric': [6.764, 12.681, 15.592, 17.509, 19.63, 21.045, 21.869, 23.018, 24.194, 25.098, 26.224, 26.755, 27.581, 28.135, 28.736, 29.095, 29.823, 30.278, 30.56, 31.104, 31.546, 31.731, 32.337, 32.779, 32.834, 33.19, 33.532, 33.943, 34.228, 34.497, 34.682, 34.803, 35.188, 35.056, 35.489, 35.563, 35.846, 35.779, 36.199, 36.175, 36.147, 36.5, 36.559, 36.697, 36.924, 36.878, 37.177, 37.123, 37.268, 37.441, 37.434, 37.675, 37.614, 37.713, 37.841, 38.209, 37.965, 38.087, 38.393, 38.283, 38.386, 38.505, 38.458, 38.608, 38.661, 38.804, 38.852, 38.897, 38.922, 38.706, 38.773, 38.941, 39.166, 39.188, 39.258, 39.089, 39.302, 38.967, 39.297, 39.547, 39.514, 39.395, 39.544, 39.553, 39.355, 39.473, 39.604, 39.749, 39.709, 39.857, 39.897, 39.827, 39.667, 39.875, 39.916, 39.944, 39.658, 40.109, 40.012, 39.864, 40.116, 40.259, 39.928, 40.244, 39.967, 40.159, 40.102, 40.172, 40.107, 40.14, 40.382, 40.344, 40.158, 40.263, 40.392, 40.235, 40.495, 40.468, 40.382, 40.275, 40.561, 40.55, 40.368, 40.652, 40.491, 40.639, 40.625, 40.6, 40.662, 40.56, 40.759, 40.958, 40.685, 40.795, 40.768, 40.733, 40.72, 40.66, 40.736, 40.787, 40.87, 40.826, 40.778, 40.993, 40.871, 40.966, 40.947, 40.946, 41.013, 40.885, 41.113, 41.026, 41.019, 41.04, 41.057, 41.002, 41.031, 41.06, 41.081, 40.86, 41.01, 41.174, 41.279, 41.253, 41.236, 41.297, 41.21, 41.374, 41.289, 41.337, 41.148, 41.136, 41.266, 41.197, 41.248, 41.178, 41.489, 41.405, 41.419, 41.221, 41.461, 41.382, 41.449, 41.268, 41.425, 41.439, 41.548, 41.309, 41.4, 41.47, 41.397, 41.36, 41.553, 41.473, 41.674, 41.539, 41.463, 41.675, 41.347, 41.521, 41.663, 41.461, 41.463, 41.622, 41.631, 41.584, 41.694, 41.553, 41.627, 41.497, 41.623, 41.625, 41.744, 41.708, 41.793, 41.578, 41.597, 41.647, 41.682, 41.824, 41.79, 41.73, 41.656, 41.675, 41.938, 41.682, 41.821, 41.754, 41.614, 41.682, 41.85, 41.835, 41.713, 41.872, 41.73, 41.863, 42.149, 41.941, 41.853, 41.885, 41.852, 41.763, 42.103, 41.851, 41.771, 41.984, 41.939, 41.806, 41.917, 41.862, 41.867, 41.749, 41.956, 42.024, 41.877, 42.034, 41.85, 41.97, 41.931, 41.88, 41.88, 41.976, 42.177, 41.806, 42.145, 42.042, 41.981, 42.119, 41.782, 42.081, 42.006, 41.9, 42.001, 42.035, 42.068, 42.276, 42.029, 42.16, 42.047, 41.965, 42.04, 42.032, 42.176, 42.184, 42.084, 42.006, 42.222, 41.974, 41.855, 42.104, 42.199, 42.131, 42.175, 42.252, 42.167, 42.381, 42.205, 42.106, 42.005, 42.237], 'train_loss': [4.487607039477836, 3.9302119021986206, 3.7160939384711833, 3.5823035835075743, 3.474910422616179, 3.3952432394180247, 3.334361382043293, 3.2724438163613327, 3.21618201117903, 3.1660085654914645, 3.1167914402156933, 3.0771932083295845, 3.040268194805104, 3.0092812344117266, 2.978255172913759, 2.949873996253816, 2.9239539708072226, 2.8975067703066464, 2.8784563288197482, 2.8550892033335953, 2.8437898641050587, 2.8214689252930274, 2.8011283229271418, 2.786693944583241, 2.7702113187854596, 2.7568587419396398, 2.7468244249974774, 2.725969816352493, 2.718246659939669, 2.7031308857203293, 2.6903595712348123, 2.680134442549673, 2.667491735896466, 2.662080448862077, 2.6561841311854426, 2.6474434609452806, 2.635719900854261, 2.6290770917082167, 2.617823018451112, 2.6156225540053746, 2.6138274857270285, 2.60002355596917, 2.5990778723361014, 2.5923765925970583, 2.579824196247435, 2.5835699816964333, 2.5691213545247065, 2.5702270296850953, 2.5605309834788414, 2.5576303377413856, 2.5508219386550453, 2.547499087477676, 2.54822448469932, 2.5397683876260917, 2.5361424350463957, 2.529580632311674, 2.525893623067718, 2.5231124697323417, 2.5210128471932194, 2.518525150748147, 2.515984123361774, 2.5070438698477573, 2.508651282416653, 2.503944642255494, 2.5005335093917407, 2.49599050140808, 2.4959823565840034, 2.4941535805252526, 2.489621171795704, 2.4941993188339397, 2.488109907658529, 2.4877931794827366, 2.4830209268489405, 2.478110601866924, 2.4788146076946784, 2.4760324196867356, 2.4725454855026228, 2.4776163179753916, 2.471635162639679, 2.465147973556055, 2.464286412021249, 2.4655883909415834, 2.462523855464396, 2.4664059265713925, 2.461939249828849, 2.458444345859252, 2.458180804856679, 2.4543444016615856, 2.454833122605478, 2.451639103340325, 2.4476092010450454, 2.4458390332458118, 2.4500599932533307, 2.449543340421226, 2.445731074552237, 2.4408573829738742, 2.4468417875444897, 2.4387976571069037, 2.440718993191832, 2.440761512925971, 2.4388379567110303, 2.4312408121442153, 2.439166534808837, 2.429278821878073, 2.4343731163101783, 2.42844056068745, 2.430153471418321, 2.4264094310163764, 2.423388231738744, 2.4270484957722465, 2.4262111879508614, 2.4250445719796425, 2.4240526911240696, 2.4212215643850414, 2.4198445492803633, 2.4249376005189816, 2.416010024832825, 2.4167518448692364, 2.415946584440391, 2.4199519486894077, 2.4171428659827146, 2.413273357460305, 2.4153724317739806, 2.412411747112994, 2.4084980484772704, 2.412869303484262, 2.413707655664445, 2.4069564640712677, 2.4041536146451934, 2.4074896185045738, 2.4041875417767926, 2.40493807919271, 2.404817222099768, 2.4012711945056, 2.4005170702857996, 2.3981938942532772, 2.405689472657934, 2.4020913576972998, 2.401273571216976, 2.3939561002619056, 2.39731592874228, 2.397577346934772, 2.3992425654839034, 2.3973062184859146, 2.393147880346136, 2.394217153809731, 2.3910283973715813, 2.3954242283307963, 2.394599437332275, 2.39206553191919, 2.3892038413064265, 2.3891339986200752, 2.3871683299503337, 2.3879774720411002, 2.3864996127417206, 2.3881082955797894, 2.3861653587410867, 2.384404919731716, 2.383381429804645, 2.3861640279138996, 2.384784110905799, 2.3830820309070924, 2.380074125333848, 2.3815855430016057, 2.3843489937956184, 2.3781966688689407, 2.3809805561622137, 2.3758056237006597, 2.377645391954189, 2.373800484400412, 2.3813454283404947, 2.377786266063927, 2.3760690907217796, 2.3759250349519503, 2.3744868049237184, 2.37412180987521, 2.372901733685821, 2.3720547702933303, 2.3728635484258565, 2.3744053595811985, 2.372410208463516, 2.3731909396933655, 2.3720544353937845, 2.3729750529665714, 2.368270520971741, 2.369666455116931, 2.3698131000850724, 2.368489905190788, 2.3741731177669876, 2.369393057573017, 2.3658763189309693, 2.370051392895704, 2.3688291492022846, 2.36812554386588, 2.3675689149650334, 2.3659868514926785, 2.3682404531550882, 2.363342711884321, 2.364629763711818, 2.3656301072066364, 2.364141941985791, 2.361652684074446, 2.3641330569658385, 2.3650754995248446, 2.3604552892256607, 2.362304398896064, 2.360754185888299, 2.3615521181872925, 2.3630853121813034, 2.365871457358011, 2.3610447637560426, 2.361001160918179, 2.361727999329033, 2.358759639282983, 2.356693052360818, 2.362713389723101, 2.3608824392197914, 2.3582914355315236, 2.356759926102822, 2.351941594388038, 2.3567373499379123, 2.358302951850574, 2.3580652698826805, 2.3565326753672466, 2.351675012366404, 2.3557559179329215, 2.359409750003656, 2.3563757976963973, 2.356495861281055, 2.3558890641841534, 2.353336449624328, 2.3510155647287596, 2.3556834797941564, 2.3498262119689817, 2.35192323234397, 2.350780753317508, 2.3473167329435536, 2.350873854247256, 2.3541209465101294, 2.3515449924035785, 2.3506037018959596, 2.3531890122919457, 2.348745698663415, 2.350321336122026, 2.347541467775844, 2.345777602281162, 2.347899893881492, 2.349743626168044, 2.3521260021591677, 2.3483579661399223, 2.3494776748954984, 2.3494836904265832, 2.3477181174094603, 2.343672592748226, 2.3514300715976697, 2.3430693719300106, 2.345398401115769, 2.343560144829582, 2.3481700018439167, 2.3487613706045707, 2.348444238886647, 2.3466772850095197, 2.3434898207299004, 2.34376741187815, 2.3449259306174857, 2.341640894716547, 2.345358280897598, 2.343543489926607, 2.3482179306900357, 2.341862117412833, 2.3416740496190633, 2.3435898505947335, 2.3420191477752996, 2.344022633933289, 2.3424787858473666, 2.3389699764154086, 2.3443993627453033, 2.342106025110661, 2.3388834894275483, 2.3422709304555744, 2.3373021971927725, 2.339779595648411, 2.3400075449366944, 2.3404972759943625, 2.3366536808868137, 2.334955653012447, 2.3382091910428753, 2.3396916213282695, 2.340695749622694, 2.341877843200283, 2.336808177498923, 2.337610763917729, 2.337229032357839, 2.3384081588978196, 2.334366639683968, 2.3345001821554554, 2.3363835052733077, 2.3404564737739735, 2.3371712742748736, 2.33268698033658], 'val_metric': [4.84, 13.17, 13.85, 16.26, 18.21, 19.52, 19.27, 21.95, 22.81, 23.39, 23.47, 25.12, 25.31, 26.36, 25.71, 27.09, 28.91, 28.13, 29.31, 27.84, 29.56, 29.97, 26.36, 26.82, 30.47, 29.92, 30.5, 27.82, 30.19, 27.13, 31.21, 29.83, 33.28, 31.89, 27.38, 32.14, 28.61, 30.49, 29.34, 30.11, 31.03, 29.33, 30.64, 31.98, 33.04, 33.4, 30.17, 29.84, 28.32, 33.94, 31.66, 31.53, 33.66, 31.28, 34.73, 34.21, 31.62, 31.65, 31.68, 32.66, 31.97, 35.06, 35.76, 33.69, 35.42, 34.27, 32.28, 33.46, 35.45, 32.51, 34.1, 34.7, 33.32, 32.9, 34.33, 33.21, 34.47, 35.16, 33.35, 35.44, 31.75, 33.81, 36.72, 34.64, 35.87, 30.21, 33.96, 31.98, 35.88, 35.36, 34.1, 30.68, 36.78, 36.2, 29.96, 34.13, 35.25, 35.17, 32.91, 34.89, 33.41, 35.63, 34.65, 31.18, 34.48, 33.48, 33.49, 34.84, 36.49, 34.16, 33.82, 36.55, 34.24, 37.06, 30.85, 37.19, 36.13, 37.09, 32.89, 35.97, 35.69, 34.5, 35.68, 34.02, 33.69, 34.87, 34.18, 37.48, 32.74, 32.31, 35.2, 34.12, 35.33, 36.66, 35.76, 37.17, 35.92, 32.31, 34.35, 33.01, 35.51, 36.35, 34.35, 38.06, 35.73, 36.61, 34.46, 34.23, 33.03, 34.82, 34.93, 35.71, 35.68, 37.68, 36.14, 37.71, 35.7, 33.67, 36.69, 38.56, 36.85, 33.89, 35.18, 34.05, 35.9, 35.47, 37.46, 35.37, 37.39, 31.94, 36.36, 33.75, 36.68, 37.44, 31.17, 34.99, 33.31, 35.68, 34.97, 36.57, 34.81, 38.2, 35.62, 36.96, 38.89, 35.15, 34.9, 35.97, 35.26, 38.04, 30.93, 36.2, 36.42, 34.41, 36.97, 35.77, 35.26, 30.63, 36.05, 37.37, 36.49, 33.31, 33.88, 34.99, 34.1, 31.75, 38.09, 31.96, 30.79, 37.74, 37.37, 31.93, 35.76, 34.99, 36.02, 33.86, 33.79, 35.76, 36.57, 37.69, 34.88, 36.63, 36.94, 36.72, 37.32, 37.66, 37.48, 33.78, 35.28, 37.01, 35.81, 38.25, 35.93, 37.18, 35.29, 37.69, 37.61, 34.43, 37.1, 32.77, 36.03, 33.43, 35.12, 34.03, 37.05, 35.41, 35.25, 37.83, 34.1, 36.47, 32.64, 37.58, 38.03, 35.66, 36.44, 35.88, 35.11, 36.28, 37.22, 37.29, 37.05, 34.42, 39.24, 37.14, 34.61, 35.7, 33.71, 34.46, 36.08, 37.64, 34.95, 35.23, 34.97, 36.79, 38.33, 37.44, 33.58, 33.25, 34.09, 36.01, 36.66, 37.37, 35.67, 36.44, 35.75, 37.17, 37.91, 36.51, 35.47, 32.28, 36.29, 36.19, 36.78, 38.04, 35.17, 32.67, 36.52, 36.46, 32.85, 31.48], 'val_loss': [4.650148447911451, 3.887345323137417, 3.817755659674383, 3.629269693307816, 3.565342935786885, 3.4698012216835266, 3.4813393119034495, 3.363255850828377, 3.292058779175874, 3.2598858054276483, 3.283057338872533, 3.163461212899275, 3.1501641326649175, 3.1151354419198007, 3.1420461189974644, 3.0887340497059426, 2.9955666201889137, 3.0637396072885794, 2.9779944389489046, 3.072954580282709, 2.9757159796490034, 2.9312254805473765, 3.168472053139073, 3.1390526552868496, 2.9102647145083, 2.9859509597158738, 2.922333635342349, 3.0846346442107184, 2.9179368254485403, 3.095748243817858, 2.912787713822286, 2.9461116714842, 2.7807133744476706, 2.8704337701675997, 3.127746361835747, 2.887293400658164, 3.031959352979235, 3.0088391668477636, 3.0151296960320444, 2.9479479098775583, 2.9023259194793214, 3.054807742689825, 2.931177624471628, 2.828757362760556, 2.828470529264705, 2.795236565504864, 2.9513942458827023, 3.022538002129573, 3.0126209669052417, 2.752379283783542, 2.847342131244149, 2.878810786137915, 2.7816360171433465, 2.9605764484709236, 2.7358840871009096, 2.759546831155279, 2.911389913528588, 2.8795451707900708, 3.00416755904058, 2.811217043050535, 2.860491409423245, 2.7084762503387063, 2.671291714261292, 2.794008203372834, 2.669377016413743, 2.7303501352383073, 2.833617826935592, 2.8027292239438197, 2.708795595320926, 2.854643486867285, 2.764656288608624, 2.7250741780943173, 2.7980951010041935, 2.8004262527083137, 2.744267070369356, 2.7976292956406903, 2.723041952035989, 2.7199197096429812, 2.7813202924789135, 2.681315575435663, 2.8881208494210697, 2.7774566525866273, 2.635246720283654, 2.7233709772680976, 2.7047354924451015, 3.0087549124553705, 2.7211989251671325, 2.851816042972978, 2.6549349673994027, 2.7041685816588674, 2.742142754755202, 3.0255448605604234, 2.6008417196334546, 2.66547338218446, 3.0381593241053784, 2.792232210469094, 2.7151857493030036, 2.7084601441766045, 2.809103205705145, 2.7015190359893118, 2.772194209372162, 2.6640751787052035, 2.73126168046028, 2.933159184304013, 2.729225816240736, 2.761731352396072, 2.8168819231592166, 2.740173945001736, 2.640611190704783, 2.7946250582956207, 2.7566680065385856, 2.5960801308322106, 2.747208004544495, 2.606898035213446, 2.9600793799017646, 2.6153905835880598, 2.6666316940526293, 2.597374856092368, 2.831839614613041, 2.6741076335785494, 2.6679605078545348, 2.769055106837279, 2.6641661999331916, 2.752031712775018, 2.7539046235904574, 2.7389765359034204, 2.7800597316899878, 2.6025023878000346, 2.85199115230779, 2.879785607574852, 2.69027648989562, 2.7449440196820887, 2.720229116215068, 2.6188535485298012, 2.671740241111464, 2.59077138733712, 2.683580039413112, 2.858503372805893, 2.749706551527521, 2.800099268081082, 2.675276574815155, 2.668135955455197, 2.7470444008043615, 2.5679937798506134, 2.6790979599497122, 2.6082237564074764, 2.762469162606889, 2.778849809033096, 2.859772965786563, 2.7140002140573634, 2.7155101967465347, 2.67245113394063, 2.6661108815745944, 2.562013941965285, 2.6685026238678367, 2.5423729491841263, 2.6495453157242697, 2.7801194935087947, 2.644124594463664, 2.548334728380677, 2.598021543709336, 2.7831457359775618, 2.7025154145659913, 2.7719986089475595, 2.652306945460617, 2.718803293386083, 2.58513652519056, 2.7077623427293864, 2.5855053408890014, 2.9083233490871017, 2.63553453858491, 2.8035883337828764, 2.6151500849207494, 2.5979611000437646, 2.9663476890819087, 2.726701195072976, 2.800695399569858, 2.693370020693275, 2.6951895094221565, 2.638308712251627, 2.7202036046678093, 2.532037512511964, 2.6747670302725144, 2.617543723173202, 2.550914081038943, 2.7282098725343205, 2.7437649168026677, 2.6646060571549044, 2.7090769930250325, 2.5538201764890345, 2.9407233835025957, 2.644086972543388, 2.6744017813615737, 2.758097243916457, 2.6344223607117963, 2.6767172836194373, 2.6770028954098937, 2.935198704148554, 2.694659293077554, 2.608359798883936, 2.6203296845126305, 2.8309575015572226, 2.7830571949861613, 2.7623493887816264, 2.7487972135756427, 2.934092991670985, 2.6107419725436314, 2.9277893133983492, 2.9627065445966783, 2.5926794041493895, 2.6050569510004324, 2.8828550432897675, 2.652850529190841, 2.747027528513769, 2.68713353318014, 2.788661669773661, 2.8373485537850933, 2.662323881866066, 2.64839468791986, 2.601903093848259, 2.698015814374207, 2.63345733987298, 2.601562411921799, 2.638774896503254, 2.597795867616204, 2.583093546758032, 2.613049773273954, 2.8070917767324266, 2.7223802766982157, 2.6542787035559394, 2.6813456943839977, 2.57805987717999, 2.6811922109051114, 2.6343589960389835, 2.7160465276924666, 2.5829027792450727, 2.592767244691302, 2.7504062113488557, 2.601477601345937, 2.8660925323036825, 2.729879491648097, 2.8395999009442177, 2.7247947465841937, 2.730840104780379, 2.614876346982968, 2.6967129988275516, 2.707581234965355, 2.5648827575574256, 2.785478722897305, 2.651515596231837, 2.88517570267817, 2.568816370265499, 2.5655841470524003, 2.6587944554675156, 2.660139708382309, 2.7006073901607732, 2.7413813433829386, 2.629357179259039, 2.59315515475668, 2.5847066534552603, 2.611116225552407, 2.7417537915478847, 2.516293255007191, 2.61428954874634, 2.764088600304476, 2.7343007466595646, 2.7922234360579474, 2.7701103839145342, 2.668989713784236, 2.5795397591439024, 2.7309659187960778, 2.705175883071438, 2.708420167303389, 2.627170629182439, 2.5607222538844794, 2.606409301044075, 2.7994205602415048, 2.805349551188718, 2.783441204174309, 2.6529777080390105, 2.6023854693030097, 2.589037498850731, 2.6905411952620097, 2.6453508560065253, 2.714489053009422, 2.6036231813916735, 2.586901074002503, 2.6089523529550833, 2.680530201857257, 2.897200500129894, 2.6398587690037525, 2.6629315219867, 2.619553115717165, 2.5479522001971104, 2.672681972859012, 2.925384212451376, 2.6432517318968562, 2.6092182450993047, 2.9457266619251032, 2.9463346532195995]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal 1", optimizer="AdamW")
data.update({'train_metric': [8.672, 20.437, 28.451, 34.358, 39.062, 42.955, 46.597, 49.634, 53.092, 55.893, 58.777, 61.57, 64.038, 66.962, 69.315, 71.761, 73.92, 75.97, 77.772, 79.757, 80.875, 82.414, 83.405, 84.395, 85.164, 86.054, 86.6, 87.132, 88.078, 88.379, 88.689, 89.073, 89.54, 89.916, 90.276, 90.332, 90.561, 90.849, 91.301, 91.509, 91.297, 91.8, 91.909, 92.074, 92.289, 92.494, 92.609, 92.73, 92.718, 92.977, 93.038, 93.204, 93.396, 93.36, 93.584, 93.507, 93.838, 93.788, 93.874, 93.889, 94.037, 93.964, 94.352, 94.11, 94.322, 94.261, 94.491, 94.45, 94.513, 94.546, 94.618, 94.685, 94.722, 94.833, 94.869, 94.866, 94.888, 94.989, 95.102, 95.186, 95.211, 95.246, 95.071, 95.276, 95.285, 95.408, 95.258, 95.359, 95.534, 95.469, 95.552, 95.534, 95.488, 95.704, 95.761, 95.513, 95.71, 95.636, 95.933, 95.636, 95.822, 95.92, 95.843, 95.99, 95.862, 95.966, 96.016, 95.958, 96.015, 95.906, 96.119, 96.226, 96.035, 96.075, 96.323, 96.115, 96.189, 96.269, 96.069, 96.143, 96.045, 96.511, 96.402, 96.223, 96.305, 96.436, 96.328, 96.498, 96.401, 96.388, 96.32, 96.464, 96.514, 96.507, 96.457, 96.439, 96.584, 96.554, 96.654, 96.437, 96.531, 96.665, 96.608, 96.551, 96.711, 96.75, 96.676, 96.67, 96.742, 96.699, 96.701, 96.751, 96.833, 96.681, 96.738, 96.698, 96.872, 96.902, 96.66, 96.891, 96.873, 96.801, 96.746, 96.962, 96.757, 96.852, 96.874, 96.919, 96.899, 96.691, 97.077, 96.856, 96.985, 96.909, 97.022, 96.975, 97.074, 97.017, 96.892, 97.05, 97.005, 97.106, 96.979, 97.046, 97.152, 97.123, 97.089, 96.999, 97.111, 97.181, 97.018, 97.177, 97.119, 97.151, 97.218, 97.059, 97.103, 97.21, 97.188, 97.015, 97.235, 97.148, 97.257, 97.211, 97.097, 97.28, 97.166, 97.126, 97.275, 97.231, 97.18, 97.303, 97.366, 97.237, 97.178, 97.324, 97.4, 97.295, 97.296, 97.261, 97.416, 97.298, 97.279, 97.38, 97.389, 97.358, 97.383, 97.202, 97.441, 97.419, 97.343, 97.406, 97.43, 97.369, 97.325, 97.519, 97.379, 97.414, 97.472, 97.528, 97.415, 97.35, 97.46, 97.42, 97.44, 97.431, 97.486, 97.483, 97.449, 97.385, 97.593, 97.364, 97.52, 97.414, 97.546, 97.536, 97.498, 97.433, 97.536, 97.376, 97.484, 97.553, 97.585, 97.514, 97.62, 97.493, 97.48, 97.63, 97.514, 97.6, 97.493, 97.663, 97.526, 97.487, 97.573, 97.603, 97.605, 97.644, 97.495, 97.553, 97.618, 97.663, 97.596, 97.589, 97.692, 97.587, 97.509, 97.7, 97.682, 97.627, 97.6, 97.709, 97.593, 97.667, 97.686, 97.727, 97.621, 97.66, 97.733, 97.61], 'train_loss': [4.376793461584237, 3.506356142151455, 3.066097915134442, 2.7597490098334547, 2.5210939794492813, 2.3224519575854865, 2.147061831586573, 1.9818494544148217, 1.8304388652760024, 1.6938716026734482, 1.5564977145317038, 1.4284200086398378, 1.3059052286129766, 1.1839107474454, 1.07425397876662, 0.978180811657634, 0.88475993176492, 0.8019393331449305, 0.7342298700595161, 0.6638729747620745, 0.6177025649479697, 0.5689285699175629, 0.5279740692138367, 0.49059579094784883, 0.4698385815264244, 0.43865950868458453, 0.42012365498435245, 0.4018354829083782, 0.3782806563610002, 0.3638133441413242, 0.35235577328839673, 0.3397105271360162, 0.32701607460367016, 0.31510348877346983, 0.3045888342408991, 0.3044033751227748, 0.2938887760262419, 0.2863392519549498, 0.2711818300959817, 0.2646609329387917, 0.2691850102460697, 0.25672637479724175, 0.2572351620578453, 0.24835842108843767, 0.24263181746892645, 0.23442415184247822, 0.2352782432252085, 0.22934837059921664, 0.23029786989841222, 0.2236589306020443, 0.21995885415635502, 0.21367912283238746, 0.20758185697682513, 0.20965045153670447, 0.20200540893428118, 0.20737328472703942, 0.19505011773133746, 0.19503082019132123, 0.1948560110519158, 0.1963326059310427, 0.18920385797453837, 0.19253706008848934, 0.18073635196559945, 0.18865860592554815, 0.18023534978651135, 0.18452565702332646, 0.1742717266328256, 0.17545204356155447, 0.17483657607820627, 0.17223567294847084, 0.17145992479074368, 0.1703107180893159, 0.16668598335867918, 0.1643966300382028, 0.16402843536514733, 0.16615855152169343, 0.16270752264316637, 0.16020453469029325, 0.1545811576276579, 0.15609707201068942, 0.15437528757352836, 0.1540718667619693, 0.15795765715899687, 0.15112987278930257, 0.15027760040826538, 0.14843271239061845, 0.15522160891324915, 0.14947877452343522, 0.1450295929754688, 0.14913714940880765, 0.1445219847676061, 0.14285185879992557, 0.14486970160256593, 0.13786287384312684, 0.13763380911223883, 0.14240606043068305, 0.1396069828245398, 0.1418171786853325, 0.13010284342098163, 0.14138020401101342, 0.13410656941594032, 0.13157616151894203, 0.13530228359199303, 0.13092914770279254, 0.1344452241271191, 0.13277886659699425, 0.13042325802960433, 0.13351174381067613, 0.1319180328416223, 0.13432380530663832, 0.12677341404376027, 0.12657164010280053, 0.12986599069421037, 0.12895230130546906, 0.1205141025301759, 0.1283306083259854, 0.12587024231104066, 0.11986450558785713, 0.12665628163874362, 0.1270695008580011, 0.1290257916568587, 0.11610697296962201, 0.12030924357693125, 0.12293592850048743, 0.11943655660908052, 0.11733965728107497, 0.12227082512243422, 0.11713796060473342, 0.1190201479696412, 0.11948690097272559, 0.12251876099149846, 0.11430983232234153, 0.11427332596973738, 0.11814298025954763, 0.11737619381764712, 0.11542469503608861, 0.11618708471603148, 0.11455756254712869, 0.11085990206987947, 0.1156761591822941, 0.11663645298219621, 0.11072419937862404, 0.11362244195526171, 0.11405828221597109, 0.10835307923780775, 0.10918930914377084, 0.11106596611133614, 0.11040506973960666, 0.10865482274888015, 0.10887203237693698, 0.1107140095809207, 0.10980049909720518, 0.10650842915583605, 0.11089626782487659, 0.10660172298327755, 0.10973147400772534, 0.10393585541867427, 0.1029015893996002, 0.11041846272898498, 0.10425982933541521, 0.10417541038158265, 0.10787630758232777, 0.10981976232482943, 0.10178689840020312, 0.10734334831517511, 0.10459143888887247, 0.10741898427110806, 0.10200341977972284, 0.10565640643458818, 0.1104522208789404, 0.09716132428804389, 0.10670003483883181, 0.1008370819351721, 0.1040897297616172, 0.0994961039491526, 0.09848799414508777, 0.09977670381905097, 0.09992352276801023, 0.10609705866998513, 0.09961292003711175, 0.10166637947220833, 0.0977599650357897, 0.10419421674316047, 0.09672777193281305, 0.09622967375206427, 0.09861921248772054, 0.09755265240172271, 0.10175803382386425, 0.09996485279839348, 0.09574145103454106, 0.10040740952643067, 0.09542695158012632, 0.09974390591488184, 0.09617458939452427, 0.0917755006095624, 0.10006003686353822, 0.09705926511712731, 0.09455986877468218, 0.09565134349220489, 0.10043755577826069, 0.09294618846820601, 0.09664477086707365, 0.0911481179385157, 0.0961434809814737, 0.09921446627661971, 0.09302208992814295, 0.0961429107119143, 0.09687809380424484, 0.09385458786465539, 0.09459617562678503, 0.09542207605460876, 0.09274222887788053, 0.09104020363863564, 0.09414245121640044, 0.09707863729681224, 0.08984200879761259, 0.08952383100387645, 0.09287746712730474, 0.0950678737557172, 0.09459831089287685, 0.0878694997610681, 0.09228752814228089, 0.09131263336628527, 0.08978662290751889, 0.08774959654086231, 0.09085802689413805, 0.09002845952906559, 0.09611115494672008, 0.08855726985835548, 0.09043399304215329, 0.09171679802672798, 0.08882787408681206, 0.08868366889896911, 0.09067435883575788, 0.09120465755212648, 0.08516225525344408, 0.08915913259383251, 0.08940396133017384, 0.08681669355548415, 0.08532064844211694, 0.0851724594192039, 0.09189179552222314, 0.08734687808712166, 0.08845688832782403, 0.08688951723267291, 0.0868487178138487, 0.0869010694080148, 0.08653002661306038, 0.08729906729752558, 0.09131910792965012, 0.08487282642947352, 0.09147130801516046, 0.08606994386119378, 0.08758288794404574, 0.08436143004759414, 0.08507079305613258, 0.08654838377316837, 0.08950598463214626, 0.08448761508635522, 0.08988967103391224, 0.08634069728440416, 0.08294906929873885, 0.08356258422515513, 0.0882363914953329, 0.08307647865785082, 0.08775542307358997, 0.08848442724268231, 0.08267866914494024, 0.08529419620898505, 0.08386926939663865, 0.08577122730453791, 0.08434640448499112, 0.08531546649085013, 0.08653724483325557, 0.0848613499762887, 0.08234071595230927, 0.08203991164589407, 0.08068271986858916, 0.0880991631424705, 0.08520334051841598, 0.0838782134982145, 0.08324752379809519, 0.08280862243852018, 0.08509807289724824, 0.07976172395347318, 0.08301494175518544, 0.08718319299167691, 0.08025834862842485, 0.08268728476035664, 0.0816614195039364, 0.0842799868126908, 0.08086738537796688, 0.08349905024969433, 0.08009362017175138, 0.08276067104024716, 0.07850713017744088, 0.08460266137012361, 0.08234132503244104, 0.0773030829708472, 0.08516023594045759], 'val_metric': [14.83, 22.67, 28.84, 33.03, 35.06, 36.86, 39.37, 40.04, 41.4, 41.9, 41.91, 42.44, 41.27, 42.27, 41.61, 41.77, 41.26, 39.99, 40.7, 40.18, 39.49, 40.08, 40.74, 39.86, 39.84, 40.0, 40.81, 39.88, 39.97, 39.52, 39.99, 39.48, 39.95, 40.11, 39.77, 39.9, 39.67, 39.79, 40.61, 39.98, 40.43, 39.99, 40.87, 40.32, 41.01, 39.84, 40.3, 40.52, 39.64, 40.24, 39.64, 40.02, 40.32, 41.1, 41.1, 41.17, 40.22, 40.55, 40.86, 41.22, 41.17, 39.94, 40.56, 40.71, 40.8, 41.32, 41.27, 39.75, 41.51, 41.18, 40.57, 41.67, 41.59, 41.91, 41.15, 40.36, 41.4, 41.37, 41.21, 41.28, 40.91, 41.35, 40.82, 40.5, 41.24, 40.64, 41.01, 40.51, 40.34, 41.56, 41.09, 40.73, 41.05, 41.62, 41.5, 41.03, 41.17, 41.05, 41.46, 41.5, 41.17, 40.94, 41.08, 40.97, 39.75, 40.88, 41.22, 41.7, 41.74, 41.72, 41.0, 40.99, 41.64, 42.1, 41.57, 41.3, 41.42, 40.65, 41.22, 41.67, 41.66, 42.28, 41.62, 41.9, 40.72, 40.97, 41.27, 41.6, 41.13, 41.17, 41.18, 41.51, 41.63, 40.95, 41.01, 41.24, 41.21, 41.95, 41.54, 41.22, 41.74, 40.94, 41.45, 41.69, 41.72, 41.45, 41.76, 41.43, 39.96, 40.51, 41.39, 40.82, 40.95, 41.22, 41.0, 41.97, 41.85, 40.58, 42.34, 41.85, 41.76, 40.82, 41.45, 41.24, 41.37, 41.97, 41.14, 41.71, 41.69, 41.56, 41.68, 41.47, 41.05, 41.2, 41.46, 42.37, 41.37, 41.67, 42.11, 41.56, 42.31, 41.94, 41.85, 41.83, 41.99, 41.54, 41.27, 41.6, 42.48, 41.22, 41.85, 41.34, 40.97, 42.02, 41.35, 40.99, 42.42, 41.78, 41.76, 42.26, 42.27, 41.74, 41.56, 41.34, 42.24, 41.63, 42.05, 41.89, 41.78, 42.06, 42.11, 41.38, 41.49, 41.59, 40.89, 41.8, 42.33, 42.03, 41.63, 41.46, 42.06, 42.32, 41.49, 41.22, 41.64, 41.69, 42.31, 41.68, 41.37, 41.68, 41.96, 41.81, 41.65, 42.24, 41.56, 42.25, 41.89, 41.63, 41.6, 41.95, 42.25, 42.22, 41.63, 42.3, 41.62, 41.93, 41.48, 42.45, 41.79, 42.04, 41.65, 42.09, 41.88, 41.76, 41.82, 41.96, 41.97, 41.58, 41.68, 41.38, 41.49, 42.21, 41.42, 41.5, 42.11, 42.2, 41.34, 42.11, 41.87, 42.13, 41.54, 41.28, 41.71, 41.49, 42.05, 41.42, 41.48, 41.7, 41.95, 42.48, 41.68, 42.5, 41.6, 41.25, 41.77, 41.63, 41.83, 42.21, 42.67, 41.71, 42.05, 41.94, 41.74, 40.8, 41.94, 41.97, 41.94, 42.07, 41.48, 41.62], 'val_loss': [3.8996914632760795, 3.417867188241072, 3.0774261275674126, 2.865493943736811, 2.7365931864756687, 2.7249872805965936, 2.5702326434433083, 2.584997417820487, 2.522739676153584, 2.5722813887201297, 2.5749418306502565, 2.649861551014481, 2.7602201571130447, 2.848524331287214, 2.936047682336941, 2.959188288943783, 3.088691006040877, 3.3323625227448286, 3.379620866411051, 3.556045796461166, 3.655126769831226, 3.6992625590342625, 3.8670722420807855, 3.9806505084797075, 4.042270820611601, 4.119495927907859, 4.200888291286056, 4.273092922131727, 4.309096729679472, 4.387278079227277, 4.4967464519913785, 4.4581458940627465, 4.521594940477116, 4.623879292967972, 4.679398381406335, 4.852778776436095, 4.728153260650148, 4.766464140764468, 4.754494129472477, 4.824734275508079, 4.903000113311087, 4.959611793232572, 4.997891383565915, 5.000796711368925, 4.8361346949437625, 5.12116879262742, 5.162096615809544, 5.115160838054244, 5.09305776911936, 5.166346502152218, 5.284984953084569, 5.339115255957196, 5.125496164249007, 5.313832316428992, 5.2616806501036235, 5.422285010860224, 5.497408080252872, 5.426743701005438, 5.37273551552159, 5.337861610825654, 5.377876213401746, 5.393986081621449, 5.447206976307426, 5.576446960686119, 5.468402417602053, 5.611961720855373, 5.566704950514873, 5.52118781721516, 5.700768397112561, 5.579380743063179, 5.706994450016386, 5.767400106047369, 5.806680286006563, 5.597651007828439, 5.794447956571154, 5.695154756497425, 5.6255164017343215, 5.752889567879355, 5.68942206984113, 5.757179155471219, 5.73402277147694, 5.893219294821381, 5.920199832339196, 5.974955599019482, 5.899935824096582, 5.965722276906299, 5.809920537243983, 5.784062114490825, 5.830883777065641, 5.939043710945518, 5.855138802224664, 6.075646685946519, 6.050962662241261, 6.138065839269359, 5.9092862241587065, 6.130554805895326, 6.125145298660181, 6.069730124655803, 6.030050412864442, 6.304844438650046, 6.147585050315614, 6.139269734643827, 6.10989930219711, 6.211795826626432, 6.245150435502362, 6.063542659874934, 6.1993306146305835, 6.439128736022171, 6.221344076903763, 6.212576172154421, 6.302220842640871, 6.307754830190331, 6.188708883182258, 6.352719619015979, 6.30798496941852, 6.230585705702472, 6.22469280024243, 6.246781535968659, 6.401240477896041, 6.341905309895801, 6.356484044888976, 6.405067632912071, 6.386670224985499, 6.3796278716652255, 6.346376731896856, 6.526127967864844, 6.290659036605981, 6.458168240109826, 6.567929712830076, 6.620253140759316, 6.361300662824303, 6.470395434434247, 6.443282816060789, 6.4766957061305925, 6.525338705937574, 6.538609522922783, 6.4287495658655835, 6.388566734684501, 6.517664506936529, 6.661231844288529, 6.424928387259222, 6.570514823221098, 6.579599307600859, 6.61211619407508, 6.472171590586377, 6.720779487281848, 6.503801453645062, 6.702756637980224, 6.712483734082264, 6.931934551068932, 6.647455343774929, 6.800801117708729, 6.778306760605733, 6.810708193262672, 6.947268167119117, 6.7813528616716905, 6.910443801029473, 7.100500576815028, 6.793465983336139, 6.639823120870408, 6.660022105380988, 6.789950240949157, 6.859950442223033, 6.900250384762029, 6.878735065460205, 6.7224113242641375, 6.977082188721675, 7.010291273426858, 6.899450765293874, 6.773098966877932, 6.911628789962477, 6.89363342941187, 7.0652956780354685, 6.702541313353618, 7.14046414490718, 6.904694863945056, 6.957854126669039, 6.809738077175845, 6.930819198584101, 7.074170250801524, 6.92460575225247, 6.898015714754724, 7.021628802749002, 7.03125594831576, 6.916523773958729, 6.771413300447403, 7.309924667807901, 6.8336064481431515, 7.268059999320157, 6.934368936878861, 6.906867955141006, 7.012157601156052, 7.085021914949842, 6.959488124604437, 7.158195457640727, 7.0219078367682775, 7.061971556608844, 7.100104641762509, 7.189631992084965, 7.063602070899526, 7.202325210449802, 7.1609804121552, 7.019756222986112, 7.081947267435159, 7.059083599953135, 7.172651134478818, 7.104097622974663, 6.994446765085694, 7.020495793622011, 7.262570671215179, 7.078842021857097, 7.045209487532355, 7.200910148347259, 7.135465040328397, 7.340782835225391, 6.954682362307409, 7.049631984370529, 7.291541738494946, 7.13993660659547, 7.245377434287102, 7.11199012151949, 6.899752313164389, 7.09372395618706, 7.158197191110842, 7.291938022443444, 7.252734905595233, 7.3583910685435985, 7.246970105323062, 7.262390886142755, 7.458048851626693, 7.385746681006851, 7.196883894076013, 7.4818082220235445, 7.362123865230828, 7.4436899264147325, 7.498061278823075, 7.439571536270676, 7.393795900284105, 7.182310461238691, 7.285757843096545, 7.471253101233464, 7.4280974773844335, 7.1824963449672525, 7.239098599002619, 7.4124861067267735, 7.348071774100043, 7.477213927894641, 7.234439707106086, 7.371433209461771, 7.340494237887632, 7.490389968179594, 7.599478259208096, 7.3713729351189485, 7.348828967969129, 7.433753801758882, 7.502138045183413, 7.742551618320927, 7.40833465612618, 7.2827694302151915, 7.3327872373495895, 7.5758976002407685, 7.500550561649784, 7.545409810011554, 7.279449688401192, 7.246655379131341, 7.340635715776188, 7.334611011158889, 7.290381592550095, 7.582808814230998, 7.4478796242149015, 7.531678076762303, 7.476601360709804, 7.734377718275519, 7.448624152286797, 7.4746027023169646, 7.460743013460925, 7.365929012845276, 7.596620350886303, 7.4703504036945905, 7.269309472126566, 7.270953674984586, 7.4030322307234355, 7.816301342028721, 7.2785672108838515, 7.610534546481576, 7.551959996010847, 7.316520625618613, 7.470724101279192, 7.320917598760811, 7.600126797985879, 7.608360595763869, 7.8076806569555, 7.583265831516047, 7.360324782171067, 7.644726590745768, 7.3835260587133416, 7.428443345294636, 7.723686420993441, 7.478381565422009, 7.577660601609831]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="SGD")
data.update({'train_metric': [0.672, 0.767, 0.826, 0.904, 0.919, 1.011, 1.094, 1.189, 1.301, 1.362, 1.456, 1.599, 1.714, 1.798, 1.937, 2.062, 2.221, 2.431, 2.562, 2.731, 2.864, 3.025, 3.109, 3.165, 3.306, 3.497, 3.662, 3.897, 4.328, 4.551, 4.876, 4.961, 5.193, 5.471, 5.653, 5.892, 6.158, 6.411, 6.583, 6.82, 7.03, 7.24, 7.361, 7.536, 7.749, 7.979, 8.174, 8.324, 8.476, 8.581, 8.767, 8.953, 9.123, 9.326, 9.399, 9.673, 10.015, 10.143, 10.345, 10.543, 10.75, 11.002, 11.167, 11.367, 11.766, 11.918, 12.086, 12.339, 12.578, 12.784, 13.093, 13.194, 13.551, 13.851, 14.049, 14.365, 14.528, 14.909, 15.121, 15.399, 15.48, 15.868, 16.045, 16.351, 16.564, 16.723, 17.046, 17.174, 17.269, 17.674, 17.901, 18.184, 18.332, 18.438, 18.77, 18.976, 19.25, 19.489, 19.652, 19.771, 20.059, 20.24, 20.355, 20.705, 20.665, 21.015, 21.352, 21.393, 21.658, 21.851, 22.172, 22.197, 22.441, 22.467, 22.796, 23.068, 23.239, 23.299, 23.435, 23.763, 23.918, 24.033, 24.159, 24.49, 24.663, 24.856, 24.957, 25.172, 25.282, 25.41, 25.717, 25.983, 25.993, 26.214, 26.332, 26.611, 26.684, 26.842, 27.067, 27.168, 27.379, 27.42, 27.626, 27.912, 27.945, 28.229, 28.18, 28.553, 28.53, 28.816, 28.958, 29.101, 29.101, 29.348, 29.601, 29.775, 29.841, 30.025, 30.189, 30.172, 30.634, 30.791, 30.686, 30.905, 31.285, 31.335, 31.588, 31.49, 31.895, 32.151, 32.033, 31.919, 32.479, 32.503, 32.755, 32.946, 32.907, 33.279, 33.183, 33.629, 33.529, 33.617, 33.932, 33.953, 34.288, 34.396, 34.536, 34.738, 35.131, 35.116, 35.33, 35.12, 35.483, 35.755, 35.588, 35.961, 36.239, 36.056, 36.161, 36.465, 36.648, 36.774, 37.013, 36.956, 37.421, 37.268, 37.431, 37.646, 37.837, 37.697, 37.935, 38.087, 38.389, 38.629, 38.817, 38.605, 39.041, 39.047, 39.457, 39.362, 39.626, 39.658, 39.829, 40.118, 40.066, 40.205, 40.2, 40.536, 40.701, 40.954, 40.969, 40.997, 41.186, 41.124, 41.343, 41.838, 41.737, 41.783, 42.083, 42.365, 42.335, 42.717, 42.688, 42.742, 42.864, 43.047, 43.269, 43.283, 43.295, 43.569, 43.941, 43.786, 44.065, 44.264, 44.331, 44.423, 44.509, 44.707, 44.841, 45.096, 45.074, 45.094, 45.283, 45.509, 45.663, 45.681, 45.775, 46.044, 46.225, 46.5, 46.347, 46.645, 46.751, 46.857, 47.067, 46.973, 47.301, 47.192, 47.51, 47.51, 47.838, 47.879, 48.157, 48.326, 48.303, 48.668, 48.485, 48.597, 49.139, 48.839, 49.04, 49.086, 49.327, 49.359, 49.507, 49.666, 49.802, 49.777, 50.198, 50.24], 'train_loss': [5.296646775607489, 5.290545708234693, 5.280009399372572, 5.266210481591203, 5.253478445308146, 5.241742542593889, 5.229925218790827, 5.219237971290593, 5.207551300868878, 5.19423671143984, 5.180081304646576, 5.1657530930434294, 5.1483222310999155, 5.132548450889758, 5.114264271004568, 5.096756791732895, 5.07744661905944, 5.056542016418027, 5.034742013048988, 5.01172963427338, 4.988133856064985, 4.963407328246269, 4.93563439612654, 4.910284510653368, 4.885129668967356, 4.858091179255256, 4.831055394366088, 4.8049979569892125, 4.7756783527513385, 4.745561370312672, 4.717913551354973, 4.687096787048164, 4.659908618365658, 4.6335808841829795, 4.605714012397381, 4.578206386309897, 4.551172290943558, 4.52662037247202, 4.500937930453075, 4.477296568076731, 4.452067806106001, 4.428425345295755, 4.407480758195952, 4.385653886898771, 4.366074841371806, 4.34867811355542, 4.325780197815947, 4.308033160650798, 4.290903916819616, 4.272804593399253, 4.256367114287344, 4.238460165677891, 4.220863920103184, 4.204735649448134, 4.188275043886591, 4.17084231333937, 4.1522777676963685, 4.139100589282377, 4.1186870233568715, 4.103944096287625, 4.087705675486335, 4.071442899533135, 4.050782942421072, 4.036819495684965, 4.018356844163139, 4.00254921842979, 3.9853841021361447, 3.970243207476342, 3.951534937836006, 3.9364484376962263, 3.9183358394100507, 3.902474983640001, 3.8865003857334988, 3.8704188744074552, 3.8497710144069814, 3.837892510688084, 3.8202271046580525, 3.805683362201781, 3.786677007254163, 3.7701100171260626, 3.7585864825196853, 3.739671456684154, 3.7225492863951017, 3.7068679273242915, 3.693853350228708, 3.677315229036376, 3.662608716095859, 3.6474503126955917, 3.62890654287503, 3.6187178240475255, 3.6006543337345427, 3.586287660165544, 3.5717269494147614, 3.5597554237050884, 3.5426804419900084, 3.5301445226065256, 3.5157000792766335, 3.5010428180011206, 3.4852557337901873, 3.471855041695495, 3.4577175458882454, 3.447397962107692, 3.4324702574363215, 3.419025489175007, 3.409374659136176, 3.3912988682244727, 3.3818941660897517, 3.3692334358766916, 3.356583111727001, 3.342995997048767, 3.3305855152779333, 3.316928054000503, 3.305219719216218, 3.294102295499083, 3.282132082083099, 3.2691152834083814, 3.259986117491719, 3.2469852495407037, 3.2361429045769317, 3.2197369647498935, 3.2106189828458995, 3.200269611234171, 3.1885529469009857, 3.1765468677342588, 3.1665532189306353, 3.153650081699198, 3.1444722198173927, 3.1275578680056757, 3.124166236805443, 3.1157376577056377, 3.100240384777311, 3.089331556419036, 3.0839137352206962, 3.0681843342723103, 3.0586779956549157, 3.049040141200226, 3.038829832449222, 3.03243201570639, 3.0178096076272194, 3.007659331393105, 2.9981050410487295, 2.988722562637378, 2.9803123579373056, 2.9679166329494286, 2.9640311609074157, 2.946678777268813, 2.9422878575340268, 2.931507865969218, 2.924209228022619, 2.9096710273110555, 2.89947344794612, 2.894007858525311, 2.8870300171242564, 2.87657519082419, 2.8630491023024, 2.8562661486410286, 2.8488685364762865, 2.8389358355765606, 2.830045949169557, 2.821257201510214, 2.811647649613696, 2.7990359944253873, 2.7991249247079315, 2.785133611217799, 2.7744908988742742, 2.7706372094169613, 2.75690456254316, 2.7510118846014686, 2.7383629513030927, 2.7305103922530924, 2.7279480871907125, 2.720007060661731, 2.707522717943881, 2.701068697720099, 2.6893641349221373, 2.6819388429400104, 2.6744676925246713, 2.662127555141217, 2.6554264499068796, 2.645378652323688, 2.639510456598957, 2.6305200838539284, 2.6246010793452834, 2.611937063135402, 2.6077064605225986, 2.598241668287486, 2.5939489140849195, 2.583522517179268, 2.569516182555957, 2.5642697679180406, 2.553710159321893, 2.5534758181123496, 2.5398240216786636, 2.5317376455433918, 2.524243903480427, 2.514907605359742, 2.5031390259529793, 2.504385382489981, 2.493209498247426, 2.485165595932031, 2.4779624050424713, 2.473808753513329, 2.457010585378548, 2.4531231988185658, 2.445033676335999, 2.4407125744236944, 2.4309700417045743, 2.4196456127721993, 2.4176674680837973, 2.413024105479606, 2.40242181980526, 2.393994012019303, 2.386286000906468, 2.3726660129128554, 2.3617276509297787, 2.367544569956974, 2.3513567048772663, 2.346880651786399, 2.3331925509758507, 2.3355492174205916, 2.318277584385277, 2.3153431089124235, 2.305997587287571, 2.29570930750036, 2.2964160771074007, 2.286920802225612, 2.281968460583336, 2.276353224225328, 2.2633322918178633, 2.2561508205862895, 2.2499184677864754, 2.246019920247225, 2.242596297026138, 2.2332051844453478, 2.226577888981166, 2.2107848481192316, 2.210741298593776, 2.207198494607946, 2.195870077465104, 2.1864310615122204, 2.181218047822353, 2.1697434216680547, 2.164878678413362, 2.160623129025836, 2.153392736147553, 2.152418681542536, 2.1359115829089474, 2.1284522651787072, 2.1327183184071528, 2.1231125570304563, 2.106980070119017, 2.1106404850136675, 2.098894333275022, 2.0888073256743387, 2.0876549064388508, 2.078858489267199, 2.0766291486400865, 2.0669826303471073, 2.0577932378075783, 2.0497123968578346, 2.0475227497208217, 2.040997373851842, 2.034373460636639, 2.0256913854613643, 2.022470065743513, 2.0193433241438425, 2.013122396749788, 2.0012023604533953, 1.9928937610417548, 1.9832435953868786, 1.9872742599199311, 1.9793842330775198, 1.9706089808554963, 1.9697500640432268, 1.9561257566768087, 1.9514167689392372, 1.9468207293722162, 1.942120580892874, 1.9321332990703717, 1.930243161345474, 1.915526704534993, 1.9149539470672607, 1.905386547209434, 1.9016422860834434, 1.8982966064415294, 1.8854173974051205, 1.879918851840214, 1.8779232580541267, 1.8668128825576353, 1.8681712137607909, 1.8615665572313491, 1.8578758107037097, 1.8514537256037054, 1.8426355411818756, 1.8375928090767302, 1.8292055525081095, 1.8226112038068716, 1.8230429350071355, 1.8139936028576324, 1.8074173738158672], 'val_metric': [0.72, 0.74, 0.87, 0.95, 1.07, 1.27, 1.29, 1.35, 1.46, 1.43, 1.51, 1.58, 1.66, 1.81, 2.07, 2.34, 2.63, 2.32, 2.72, 3.04, 3.03, 3.23, 3.22, 3.53, 3.4, 3.59, 3.85, 4.26, 4.54, 4.73, 5.0, 5.21, 5.27, 5.64, 5.7, 5.77, 6.16, 6.37, 6.4, 6.68, 7.05, 7.29, 7.26, 7.34, 7.39, 7.73, 7.97, 8.02, 8.3, 8.71, 8.52, 9.0, 9.06, 8.82, 9.2, 9.21, 9.79, 9.73, 9.87, 9.96, 10.61, 10.42, 11.2, 11.25, 11.41, 11.69, 11.19, 11.62, 11.95, 11.98, 12.54, 12.54, 12.8, 13.14, 13.35, 13.3, 13.72, 13.88, 13.89, 13.69, 14.37, 14.45, 14.27, 14.58, 14.54, 14.96, 15.12, 15.23, 15.51, 15.25, 15.55, 15.59, 15.99, 15.49, 15.48, 16.05, 15.89, 16.13, 15.48, 16.21, 17.08, 16.89, 17.06, 16.63, 16.99, 17.26, 16.95, 17.12, 17.05, 17.7, 17.95, 17.59, 17.72, 18.23, 18.37, 18.29, 18.63, 18.35, 17.96, 18.84, 18.57, 18.45, 17.95, 18.51, 18.69, 18.9, 18.82, 18.43, 18.92, 19.16, 19.1, 18.98, 18.94, 19.17, 18.8, 19.29, 19.15, 18.95, 19.07, 18.94, 18.88, 19.41, 18.97, 19.24, 18.69, 18.7, 19.19, 18.63, 19.01, 19.23, 18.74, 19.21, 18.47, 18.88, 19.58, 19.38, 19.47, 19.68, 19.68, 19.54, 19.52, 19.84, 19.3, 19.81, 19.82, 19.44, 19.46, 20.04, 19.36, 19.67, 19.17, 18.87, 19.48, 19.15, 19.34, 18.9, 19.54, 19.29, 19.15, 19.65, 19.75, 19.7, 19.63, 19.29, 19.73, 19.67, 19.39, 20.06, 19.5, 19.05, 19.69, 19.48, 19.47, 19.54, 19.81, 19.17, 19.84, 19.76, 19.09, 19.4, 18.94, 19.19, 19.31, 19.33, 19.32, 19.34, 19.23, 19.03, 19.22, 19.27, 20.05, 19.75, 19.69, 19.06, 18.84, 19.59, 19.21, 19.54, 19.14, 19.19, 19.74, 19.28, 18.4, 19.58, 19.31, 18.26, 19.32, 18.4, 19.13, 18.91, 19.35, 19.28, 18.68, 19.08, 18.7, 18.58, 19.69, 18.47, 19.28, 19.39, 18.98, 18.41, 18.72, 18.54, 18.94, 18.42, 18.49, 18.96, 18.61, 18.94, 17.87, 18.66, 18.98, 18.63, 18.66, 18.6, 18.2, 18.99, 18.67, 17.82, 18.81, 18.92, 18.18, 18.25, 18.06, 18.12, 17.84, 18.43, 18.43, 18.47, 18.39, 18.48, 17.31, 18.04, 18.49, 18.41, 18.85, 18.31, 17.98, 18.0, 17.36, 17.93, 17.58, 17.53, 17.71, 17.63, 17.27, 18.44, 17.37, 18.41, 18.18, 16.74, 18.27, 17.36, 17.08, 17.36, 18.14, 17.51, 17.84, 17.92], 'val_loss': [5.293633260544698, 5.286358809015553, 5.279029017041443, 5.2705527961633765, 5.255202920573532, 5.241006053936709, 5.227432111266312, 5.21736497636054, 5.196359294235327, 5.186003861153961, 5.171348860309382, 5.1558415616393845, 5.136373475858361, 5.118914420437661, 5.099668025970459, 5.077444690048315, 5.073300753429437, 5.046722705197182, 5.020844079886272, 4.995389051498122, 4.963045559111674, 4.938272599202053, 4.924336393927313, 4.888057714814593, 4.862095928495856, 4.843252365756187, 4.8115021286496695, 4.791124310463098, 4.74386304047457, 4.718489754731488, 4.691443707532944, 4.652239507930294, 4.639631471816142, 4.6032333237350365, 4.581777221837621, 4.558525448392151, 4.523289070007907, 4.504766427787247, 4.489736292772232, 4.442605993550295, 4.43990393504975, 4.413108218247723, 4.390100290820857, 4.40031717537315, 4.372530694220476, 4.351889819096607, 4.322487418059331, 4.318315228079535, 4.291989201952697, 4.274224273718087, 4.286131457918009, 4.2407861545587044, 4.249479781290528, 4.242763352242245, 4.208479390022861, 4.185614989821318, 4.186490301873274, 4.177253434612493, 4.151059885693204, 4.135980554447053, 4.13595096017145, 4.15508719310639, 4.074613791362495, 4.0712484204845065, 4.091852435640469, 4.044684457171495, 4.072487534990736, 4.051468522685348, 4.010894119359885, 4.015681332084024, 3.988873222071654, 3.9851435825323605, 3.9832614819714975, 3.95423431457228, 3.9444897645598003, 3.932093225466977, 3.93892566185848, 3.9001080564632535, 3.9083457676468383, 3.906039140786335, 3.869203921336277, 3.866194365130868, 3.869141715347387, 3.837486936028596, 3.8522753837002313, 3.833579791579277, 3.8289841739994706, 3.813247243310236, 3.813109223250371, 3.817269856762734, 3.7727356549281224, 3.7924517324775646, 3.775408266456264, 3.783400254644406, 3.7730152029900035, 3.7623359230673237, 3.782886529424388, 3.759160716062898, 3.774577231923486, 3.7613200184645925, 3.7227184332100447, 3.7209091869888793, 3.722502306768089, 3.737861569520015, 3.7079524887595205, 3.7134719623881542, 3.6939266715080117, 3.7083269684178055, 3.6987359280798846, 3.6874376785982945, 3.696189205357983, 3.710949556083436, 3.6830818524026565, 3.678653616814097, 3.646472298415603, 3.6652614052887933, 3.656063638674985, 3.649478020941376, 3.696810371556859, 3.650048145822659, 3.679843273891765, 3.677444951549457, 3.712554810153451, 3.6710627215683083, 3.6841386002340135, 3.6487009092501013, 3.698338381044424, 3.6843717933460405, 3.6360460208479766, 3.6666585905536726, 3.640914731724247, 3.6410788168573074, 3.642191510291616, 3.6302558221634786, 3.6626701932044545, 3.642587254761131, 3.6437910780025895, 3.663418244404398, 3.691593413140364, 3.684005102534203, 3.6938163201520395, 3.6595508133529857, 3.7032812917308444, 3.667226679765495, 3.695898349118081, 3.7285006828368847, 3.669303165119924, 3.687823721557666, 3.74361266755754, 3.753965112054424, 3.730758536393475, 3.7022965728856954, 3.73586729225839, 3.727412290633864, 3.6987196875225967, 3.7238588826671526, 3.671396896338007, 3.6943150948566994, 3.732823306587851, 3.728402286578136, 3.735367134118536, 3.724454993654968, 3.7326533566614626, 3.7638506919715056, 3.7281725604063385, 3.757050465626322, 3.8343390416187844, 3.7261977545015372, 3.740614067976642, 3.7679672301954525, 3.8105873621193465, 3.826401877555118, 3.78141823316076, 3.787043153860007, 3.816727381602974, 3.890778064727783, 3.8631242293461114, 3.802480229906216, 3.907311702989469, 3.8167201834879103, 3.806373076833737, 3.8439167973342214, 3.8605734633791977, 3.8655137377939406, 3.874133735705333, 3.8926722714855413, 3.8531336541388446, 3.899817395362125, 3.8973942592645145, 3.944185237216342, 3.909172829549024, 3.862384721731684, 3.8978163618950328, 3.903208627822293, 3.9223570914784815, 3.915673692514942, 3.8974912834774917, 3.9449552107768455, 4.007772893662665, 3.9674006556249726, 3.9682784171620753, 3.9910748870509445, 3.976722503164012, 4.044057767861967, 3.976805736304848, 3.9933722869605774, 4.034966916794989, 4.00835802296924, 3.996869459273709, 4.004811614941639, 3.9852591639111754, 4.0386990726373755, 4.035597895361056, 4.030908537518447, 4.02528012634083, 4.075565242463616, 4.136186750071823, 4.068150432246505, 4.048024320298699, 4.121451617805821, 4.067156764352397, 4.101563795356993, 4.082516213131559, 4.097712387704545, 4.140434740455287, 4.1444825655335835, 4.134961009784869, 4.141021104375268, 4.104319182171184, 4.158514130647015, 4.195905781095954, 4.106787142480255, 4.187496311345678, 4.2093455609242625, 4.15784366115643, 4.244116886406188, 4.203368791349375, 4.260345352683098, 4.217637977782329, 4.193630696102312, 4.31421071860441, 4.256719595307757, 4.31170091963118, 4.305583432981163, 4.340685493627172, 4.3566080400138905, 4.32281531801649, 4.282149545706002, 4.3405387431952605, 4.333896598998149, 4.451050922369501, 4.412100284722201, 4.382086557947146, 4.425102886880279, 4.36506171439104, 4.386402995723068, 4.361790780049221, 4.3372887046473805, 4.442833176084385, 4.461704800842674, 4.406595149617286, 4.404870745482718, 4.44181620998747, 4.479800260750351, 4.446699014894522, 4.53181585232923, 4.56742815455054, 4.47164431954645, 4.49304241435543, 4.495414752109795, 4.582486959020043, 4.530695544686287, 4.56091790897831, 4.532312232217971, 4.579304962401178, 4.646940190321321, 4.589441414851292, 4.583100510250991, 4.757285652646593, 4.612169709175256, 4.66699874628881, 4.599434318056532, 4.554007190048315, 4.753701753677077, 4.714133649874645, 4.701899074445105, 4.80509947667456, 4.615685787929851, 4.791162232684481, 4.815775564521741, 4.683114010816927, 4.861872601660953, 4.7210617688051455, 4.80057758282704, 4.737745040541242, 4.795958569095393, 4.708422430002006, 4.72748566129405, 4.738622146047605, 4.7287847646482435]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="Adam")
data.update({'train_metric': [5.324, 10.72, 14.097, 16.361, 18.245, 19.445, 20.609, 21.585, 22.543, 23.533, 24.22, 25.163, 25.798, 26.257, 27.0, 27.302, 27.984, 28.344, 28.875, 29.107, 29.147, 29.607, 29.861, 30.286, 30.292, 30.888, 30.911, 31.238, 31.579, 31.511, 31.871, 31.932, 32.111, 32.484, 32.384, 32.585, 32.806, 32.903, 32.983, 33.161, 33.239, 33.355, 33.541, 33.576, 33.774, 33.8, 33.865, 34.207, 34.387, 34.426, 34.531, 34.523, 34.617, 34.88, 34.898, 35.054, 35.231, 35.143, 35.244, 35.15, 35.47, 35.437, 35.613, 35.607, 35.752, 35.874, 35.941, 35.903, 36.079, 36.047, 36.118, 36.111, 36.31, 36.271, 36.427, 36.542, 36.566, 36.573, 36.823, 36.76, 36.855, 36.718, 36.839, 36.87, 36.907, 36.805, 36.947, 37.124, 37.024, 37.085, 37.209, 37.213, 37.132, 37.11, 37.424, 37.446, 37.465, 37.545, 37.446, 37.389, 37.486, 37.575, 37.576, 37.579, 37.71, 37.733, 37.789, 37.905, 37.842, 37.893, 37.754, 37.891, 37.925, 38.004, 37.906, 37.915, 37.949, 38.137, 38.11, 38.094, 37.976, 38.132, 38.371, 38.226, 38.319, 38.257, 38.104, 38.344, 38.336, 38.319, 38.455, 38.539, 38.576, 38.472, 38.323, 38.365, 38.491, 38.57, 38.409, 38.626, 38.6, 38.629, 38.595, 38.728, 38.672, 38.903, 38.686, 38.723, 38.698, 38.63, 38.719, 38.774, 38.798, 38.899, 38.766, 38.977, 38.71, 38.862, 38.796, 38.958, 38.895, 38.979, 39.036, 39.017, 39.078, 38.985, 38.954, 39.039, 38.949, 39.346, 39.044, 39.049, 38.993, 39.14, 39.039, 39.242, 38.955, 39.087, 39.171, 39.182, 39.224, 39.201, 39.223, 39.078, 39.105, 39.357, 39.317, 39.231, 39.144, 39.367, 39.264, 39.175, 39.288, 39.327, 39.278, 39.322, 39.321, 39.218, 39.266, 39.234, 39.374, 39.433, 39.239, 39.514, 39.35, 39.298, 39.416, 39.313, 39.287, 39.243, 39.433, 39.282, 39.482, 39.49, 39.404, 39.34, 39.603, 39.662, 39.421, 39.419, 39.409, 39.332, 39.281, 39.55, 39.451, 39.44, 39.358, 39.416, 39.337, 39.349, 39.544, 39.629, 39.551, 39.55, 39.651, 39.639, 39.579, 39.626, 39.584, 39.648, 39.391, 39.683, 39.656, 39.801, 39.636, 39.587, 39.766, 39.484, 39.71, 39.506, 39.503, 39.477, 39.62, 39.758, 39.693, 39.723, 39.679, 39.863, 39.706, 39.749, 39.814, 39.652, 39.633, 39.808, 39.799, 39.946, 39.748, 39.861, 39.565, 39.754, 39.657, 39.864, 39.782, 39.827, 39.721, 39.806, 39.552, 39.709, 39.79, 39.786, 39.704, 39.811, 39.87, 39.734, 39.929, 39.865, 39.833, 39.732, 39.88, 39.85, 39.841, 39.749, 39.852, 39.837, 39.858, 39.82, 39.764, 39.824, 39.978, 39.787], 'train_loss': [4.570123968258624, 4.052817590558521, 3.806062189188815, 3.6498300875140854, 3.544754503479541, 3.4644246308832543, 3.404238261096537, 3.3439156420323912, 3.2896597034566613, 3.232149947093834, 3.199230044450961, 3.1543067171263983, 3.120776633612254, 3.09299353110203, 3.060120070201803, 3.0409923217575745, 3.0155009074769414, 2.9899337148330796, 2.9705288047143763, 2.952636797460164, 2.9393940063447275, 2.924667802592233, 2.907498358307324, 2.893671350759798, 2.8817307255013356, 2.8684408431318578, 2.856604330217846, 2.841044147427999, 2.8330182179532137, 2.825066910290367, 2.814481633180849, 2.805062070384059, 2.795662448715874, 2.7890215182411158, 2.7815446008571203, 2.7755056943789707, 2.7680506270738725, 2.761038529438158, 2.7526990815148626, 2.7497084344417257, 2.748483761563487, 2.7373354495982674, 2.73353237825102, 2.7258587316755905, 2.7198721727955744, 2.7154177052808435, 2.712195128488449, 2.7033642506797726, 2.6950920373296707, 2.688581322067759, 2.6876065843927504, 2.682159677500612, 2.676440737679153, 2.6719693717328084, 2.667108647882824, 2.664374649334015, 2.6570264628050957, 2.6571846893027433, 2.6583073618926076, 2.6520281345967827, 2.6424492361144387, 2.636814568413425, 2.6406999238393127, 2.6352883058713936, 2.63125718150929, 2.6264421269440605, 2.6248935570109753, 2.623124021028603, 2.616066767325862, 2.6179249959348945, 2.6165696213966902, 2.608814194197847, 2.6065759773217785, 2.6021794945630825, 2.597252454806503, 2.59855750594014, 2.5920788844426474, 2.594025779060271, 2.5927898807550043, 2.5904315093809873, 2.584484263939958, 2.583246283476275, 2.5803332701754433, 2.577736598745189, 2.5783043779475676, 2.578388801386779, 2.5710683958849234, 2.5691265866913553, 2.5718571746951864, 2.5670547632551775, 2.5638796929281944, 2.562700208760345, 2.5640098404899594, 2.561502040385895, 2.560013048250707, 2.5602802169376355, 2.554885894231741, 2.548266579383318, 2.5498332424340764, 2.551367164344568, 2.5474787156549845, 2.5463072265178366, 2.5442917848807913, 2.539851496254719, 2.5432648648074707, 2.540312535474488, 2.5440428115127185, 2.5382517371052593, 2.53629634011196, 2.5309203616030613, 2.53485432894506, 2.5310683399152847, 2.5301667889645674, 2.5255477567246842, 2.5278611714002497, 2.525637401118922, 2.527686118393164, 2.5221857515879345, 2.5247383538683636, 2.5266346365537538, 2.5228385368525332, 2.5212754718935497, 2.5161164114281527, 2.5173220240146934, 2.518365354699655, 2.514838141046574, 2.514606623106558, 2.5139005074345446, 2.5107778246709342, 2.5061972820827463, 2.508514011585018, 2.509121357898871, 2.5039135997141315, 2.5079702420335357, 2.502986837638469, 2.5040118105504576, 2.50298225086466, 2.502463231770106, 2.5022120368228995, 2.5013270147778783, 2.500175031270267, 2.504485105064841, 2.4988181622304446, 2.4970651569842377, 2.4997439986227112, 2.4951440140900516, 2.494226554228721, 2.4956451482065045, 2.4918486423699884, 2.4944324892297893, 2.4899010500386214, 2.491216969810383, 2.490978871334537, 2.48736613298637, 2.4891980920017946, 2.4916253160835455, 2.489117413396951, 2.4824786417314963, 2.486838778622701, 2.4842623679666893, 2.486668079385983, 2.4825463633619664, 2.48588094700626, 2.4800754106738823, 2.475349123174383, 2.4825926181679723, 2.4805935046951966, 2.4800321039906392, 2.480759205150055, 2.473664994203198, 2.4775856413752777, 2.4771842325031184, 2.476510350092511, 2.4744460629257574, 2.475644186758797, 2.4756775105220723, 2.475517469800899, 2.476727878826212, 2.4705715614942427, 2.473263397097816, 2.4713039137351536, 2.47120088075722, 2.472445228583372, 2.4721386882637986, 2.4724500219713628, 2.470438759066093, 2.471459609685765, 2.4683668921562774, 2.4669715148549924, 2.4659943482239735, 2.4717817572699246, 2.4691810057244084, 2.467212738093854, 2.4671085671743977, 2.4671499153321474, 2.4676618021570254, 2.4644868540138445, 2.4689664965780285, 2.468964719574038, 2.462544402707027, 2.4657917640488343, 2.4622327696415223, 2.464849381285147, 2.4642753767341814, 2.467099184114355, 2.464210727698362, 2.463031007247481, 2.4660269006734006, 2.464475456751545, 2.462196108246948, 2.4590103009993345, 2.4635992349910185, 2.459560749169275, 2.4592627524109276, 2.462590412840352, 2.464663946544674, 2.4528414251250634, 2.4565138029731854, 2.4595984209827026, 2.4593824948245566, 2.4586011984374228, 2.459810079860138, 2.4575791542757344, 2.4574283867101983, 2.458499018991901, 2.4553441888463854, 2.4578321131085707, 2.4578089000930103, 2.4579081767198754, 2.4542196719827976, 2.4557551131825073, 2.4524928605182765, 2.45275487414706, 2.4538352595486095, 2.4501766041159554, 2.451397326613418, 2.45046007015426, 2.4540717523218496, 2.448688988722217, 2.4558531395228185, 2.456597200129479, 2.4499562042917264, 2.4480654379532876, 2.4515610657970797, 2.4514641393397913, 2.4516617719439155, 2.448683863638001, 2.456402273187253, 2.4513156018193074, 2.456577004794501, 2.4510256640818056, 2.4496088401674845, 2.4517430263837787, 2.4490330776036435, 2.4489742641790664, 2.445360877616087, 2.445746967675056, 2.4502926794138764, 2.447353000222912, 2.445913418927257, 2.4480736652247357, 2.4472261113534732, 2.4457610248375303, 2.443456297109925, 2.449698524298152, 2.446522687080954, 2.4454685313077746, 2.4435940378381895, 2.4431979959772248, 2.4442111599239413, 2.4464937398163693, 2.443709010507385, 2.445280858208869, 2.4467847711065223, 2.4413263384379107, 2.4402317332672294, 2.447593094520056, 2.4432427443835647, 2.4416604617125546, 2.4443214960915878, 2.4412356559389763, 2.441121403406769, 2.444913466390096, 2.443337461769924, 2.4415664003967703, 2.4421144675842403, 2.4433085087088537, 2.441526732838314, 2.4419999738877505, 2.443495357410311, 2.4382403831945654, 2.4378276986337517, 2.440700799093289, 2.442937482265197, 2.441401905381977, 2.4398019062885474, 2.4406415997448443, 2.4371178783976872, 2.4402375604125526, 2.4380084403569473], 'val_metric': [4.41, 10.51, 13.99, 16.08, 18.45, 17.98, 18.98, 21.42, 21.44, 21.83, 22.1, 23.73, 23.78, 24.09, 25.33, 24.33, 27.62, 25.52, 23.05, 27.51, 26.06, 26.37, 27.66, 27.1, 25.66, 27.26, 28.41, 28.03, 27.0, 25.15, 29.14, 29.98, 23.19, 27.44, 28.45, 29.9, 26.25, 30.52, 29.0, 30.29, 26.3, 30.03, 28.15, 29.2, 27.94, 27.44, 29.65, 30.48, 31.14, 31.45, 32.3, 32.32, 30.23, 32.71, 30.92, 31.96, 27.67, 31.22, 28.25, 27.47, 29.07, 30.36, 30.51, 31.8, 29.91, 31.98, 31.48, 29.93, 31.21, 31.6, 31.32, 31.56, 31.09, 27.87, 29.68, 32.52, 29.12, 32.99, 31.33, 31.81, 30.27, 32.87, 33.43, 30.81, 34.07, 32.6, 30.44, 33.44, 30.64, 33.1, 32.25, 29.18, 33.11, 31.21, 28.98, 33.2, 32.02, 33.55, 31.48, 32.23, 33.07, 33.44, 31.2, 33.85, 35.26, 30.47, 34.16, 33.64, 32.7, 34.13, 30.59, 30.81, 33.61, 28.73, 29.66, 34.85, 32.49, 34.89, 33.63, 32.84, 32.5, 35.35, 35.33, 29.61, 33.18, 35.64, 32.25, 30.36, 31.91, 32.51, 30.42, 31.65, 33.16, 31.71, 32.82, 32.84, 30.72, 35.45, 29.81, 34.74, 33.84, 32.24, 34.38, 36.24, 35.48, 33.6, 30.21, 34.51, 29.18, 34.41, 34.31, 35.82, 36.07, 34.74, 33.1, 30.57, 35.23, 28.29, 33.39, 36.59, 34.05, 34.9, 35.16, 33.46, 33.3, 30.46, 33.65, 32.26, 34.35, 33.32, 32.66, 31.62, 31.75, 34.56, 35.29, 33.35, 36.45, 31.41, 29.0, 34.68, 31.57, 36.77, 33.22, 31.2, 28.3, 34.74, 31.14, 33.31, 32.0, 33.72, 35.61, 34.97, 33.72, 34.69, 35.99, 31.69, 33.98, 35.2, 33.7, 35.34, 34.25, 30.23, 34.2, 33.65, 32.65, 34.82, 31.18, 34.87, 31.89, 33.16, 33.41, 33.37, 34.0, 34.44, 32.88, 36.03, 36.48, 34.62, 32.53, 31.36, 34.47, 33.2, 33.44, 31.43, 32.08, 34.89, 31.78, 33.41, 38.11, 32.98, 31.28, 34.05, 34.38, 34.87, 33.56, 30.28, 30.63, 32.88, 33.14, 34.93, 35.19, 32.78, 35.16, 36.02, 31.91, 35.64, 35.08, 31.32, 32.97, 34.3, 31.3, 34.15, 33.11, 35.28, 34.44, 33.88, 34.04, 33.62, 36.48, 34.04, 33.75, 36.49, 33.9, 32.91, 31.46, 32.33, 33.15, 33.57, 33.27, 29.26, 35.91, 32.68, 35.78, 33.42, 35.54, 28.98, 31.14, 35.49, 35.4, 30.32, 34.16, 32.01, 34.37, 35.86, 31.19, 35.37, 33.19, 35.56, 33.9, 33.5, 33.02, 34.4, 33.37, 35.78, 29.47, 35.69, 31.41, 34.78, 30.4, 33.68], 'val_loss': [4.628913627308645, 4.040346865441389, 3.8538659438965426, 3.671914408920677, 3.54277871216938, 3.5680962147986053, 3.488905779875008, 3.3752930126372416, 3.3854785428684986, 3.3364493482431787, 3.353093985539333, 3.2777013884987802, 3.2093385640223313, 3.2126590909471937, 3.158282090903847, 3.212230541144207, 3.026833088534653, 3.109261055660855, 3.2947719734945116, 3.0393785169929455, 3.1420331062025326, 3.10782452905254, 3.0666272989503898, 3.071847808588842, 3.1855667237263576, 3.0506571493331034, 3.023454235617522, 3.009836389760303, 3.137477366028318, 3.17669935439043, 2.95578904516378, 2.930546119714239, 3.4415201700417097, 3.1102850900334156, 2.9980548734118226, 2.9490882029199295, 3.204115689180459, 2.91463994448352, 2.981749550552125, 2.927943940375261, 3.1467304108249152, 2.9559921709595214, 3.0464728402483994, 2.9911776386248836, 3.049877122708946, 3.123454380946554, 2.9546414188518644, 2.9216274922820413, 2.886091679524464, 2.8680751581860195, 2.8004678244803363, 2.842296282956555, 2.9320818295904028, 2.8092038802280546, 2.8788956092421416, 2.8670442780112007, 3.075600996898238, 2.9092892226140212, 3.0378051123042016, 3.125152959945096, 3.06937076407633, 2.901471093202093, 2.882468369356386, 2.8583069439906224, 2.9162612333419218, 2.8622574320264684, 2.8786196837759324, 2.953503516069643, 2.8828753426576115, 2.8436872207435075, 2.8684091378169456, 2.8542317539263684, 2.8630978939639533, 3.0800044863087357, 2.948416534502795, 2.8113284300846657, 3.015691060169487, 2.779110111248721, 2.8629972729713296, 2.859537621212613, 2.954275365847691, 2.780011795128986, 2.8045730856573505, 2.933588097049932, 2.737189907556886, 2.807470067291503, 2.909235391647193, 2.7689925925746843, 2.8668265502164316, 2.7789562553357166, 2.8357197046279907, 3.005269692961577, 2.7620929312554137, 2.9279728930467255, 3.0098948395176297, 2.7699615378288707, 2.873410284139548, 2.752392199388735, 2.8902972124184774, 2.83491915502366, 2.8114220397487566, 2.802612515771465, 2.8872057248832315, 2.793038417579262, 2.688292070558876, 2.900295555211936, 2.732627501153642, 2.7461912130853934, 2.8014138344746486, 2.764441047124802, 2.940455899117099, 2.93104670495744, 2.7922050861795995, 2.9845306243106817, 3.0394980672058787, 2.6612147609139702, 2.8421668091397376, 2.7034131037961147, 2.772414202143432, 2.7825207087644346, 2.84013855001729, 2.6485366726377206, 2.663928162520099, 3.029873185097032, 2.7727358493076006, 2.6856913019897073, 2.8395721107531506, 2.9560824617458756, 2.8909253208500565, 2.7838761829266883, 2.982847548594141, 2.867027046574149, 2.7762796430830745, 2.888181190581838, 2.8058905647059156, 2.7883450123155193, 2.947011047108158, 2.6680472777907256, 3.0899963720588928, 2.7004819645243847, 2.7635511896412845, 2.824407662176023, 2.7295051129760255, 2.6316788781220746, 2.709422108474051, 2.7710077276655065, 2.9547664830639104, 2.729534369365425, 3.034258152269254, 2.744545635144422, 2.7376330992218794, 2.6300318385385406, 2.6265283405401143, 2.7080161814477033, 2.8284250972377265, 2.9676144172431558, 2.6902407195158067, 3.1232214148636834, 2.7667327397947856, 2.6287074347210537, 2.7456583991931502, 2.710418235344492, 2.709131619353203, 2.770267345343426, 2.760598275312193, 3.00692884709425, 2.7363530146847865, 2.828331864563523, 2.719164009686488, 2.8244481838432844, 2.8270855352377438, 2.967256791652388, 2.888617176159172, 2.7265704839852205, 2.6593867213862716, 2.759781931615939, 2.63519816337877, 2.9138524471574527, 3.0854076735532967, 2.726400351828071, 2.8746975212340145, 2.6190353259918795, 2.7573860002930757, 2.8777323589203463, 3.0544064303112637, 2.7388057526509475, 2.9430415672101793, 2.7605137035345577, 2.856798051269191, 2.7826866192422854, 2.6898920619563693, 2.7034908867186043, 2.7640214262494616, 2.724826906517053, 2.675876095796087, 2.87335338638087, 2.7631599652539394, 2.7146877878031153, 2.753600269366222, 2.6546858085948193, 2.7789883476913353, 2.964716892333547, 2.7259249861832635, 2.7378134507282526, 2.803435117575773, 2.709399389613206, 2.9780434214385454, 2.6941608342395464, 2.883042850312154, 2.8075638523527013, 2.786669076248339, 2.8126366161237097, 2.777352743847355, 2.7642697931095292, 2.7924878688374903, 2.622208147671572, 2.6472011572995764, 2.7432821951094706, 2.838003405719806, 2.8795916920254943, 2.7345663430584466, 2.813402078713581, 2.801363498162312, 2.9506532162617725, 2.8465834893998068, 2.7085009524776678, 2.9228684378277725, 2.7866599928041933, 2.536255849394829, 2.7969344941673766, 2.9291000753451306, 2.7974219512028298, 2.7536037298524456, 2.6994413448746797, 2.7558976191623956, 3.0231988240199485, 2.9337990321930807, 2.8269641779030965, 2.8109466520843993, 2.6860982787077594, 2.6469604817165693, 2.8384750360136577, 2.708297450071687, 2.6600502915443127, 2.868376585328655, 2.670945949615187, 2.6996427903509446, 2.902033926575047, 2.7960322753638978, 2.753755762318897, 2.894033542863882, 2.7648876844697696, 2.822497193980369, 2.700328701241001, 2.7267121296779364, 2.7591518649629725, 2.743257331620356, 2.7778121035569794, 2.655753453446042, 2.7306650155668803, 2.783274217775673, 2.6214534773188793, 2.750498846837669, 2.848828767515292, 2.9452868126759864, 2.8767865577321143, 2.837963517684086, 2.769602118024401, 2.8177496954134313, 2.9707804242516778, 2.650367181012585, 2.836928868749339, 2.684523722168746, 2.7614667495344856, 2.6688150524333785, 3.072192598300375, 2.8939854650740413, 2.6665630940419094, 2.6723616252279587, 2.992140733891991, 2.7516687668053206, 2.864205060870784, 2.7307492092156864, 2.6538462623669083, 2.91620692781582, 2.7141417819223586, 2.7691840305449857, 2.6956380772742494, 2.749658704563311, 2.7863021815658375, 2.877488099465704, 2.7649342656894853, 2.8624233774318815, 2.6440319384738897, 3.0751038129162636, 2.67040270908623, 2.9226038169329334, 2.6828130681043976, 2.999208688735962, 2.7984546711490412]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="AdamW")
data.update({'train_metric': [7.978, 19.249, 26.788, 32.718, 37.372, 41.15, 44.81, 47.948, 50.938, 53.737, 56.469, 59.106, 61.655, 63.79, 66.122, 68.182, 70.213, 72.133, 73.818, 75.39, 76.693, 78.074, 79.183, 80.316, 80.995, 82.15, 82.983, 83.735, 84.446, 84.892, 85.298, 85.887, 86.446, 86.835, 87.247, 87.615, 88.043, 88.319, 88.162, 88.841, 89.101, 89.42, 89.516, 89.849, 89.972, 90.067, 90.315, 90.526, 90.794, 90.927, 90.785, 91.018, 91.255, 91.26, 91.489, 91.607, 91.967, 91.801, 91.858, 92.017, 92.125, 92.225, 92.556, 92.254, 92.66, 92.6, 92.737, 92.704, 92.826, 92.834, 92.994, 93.092, 93.101, 93.218, 93.156, 93.207, 93.408, 93.41, 93.409, 93.818, 93.499, 93.937, 93.642, 93.654, 93.853, 93.831, 94.046, 93.803, 94.242, 94.075, 94.106, 94.03, 94.4, 94.188, 94.357, 94.241, 94.333, 94.478, 94.474, 94.391, 94.638, 94.456, 94.513, 94.614, 94.455, 94.846, 94.663, 94.797, 94.937, 94.775, 94.79, 94.949, 94.803, 94.892, 94.941, 94.871, 95.078, 94.913, 95.211, 94.99, 95.101, 95.115, 95.119, 95.261, 95.235, 95.222, 95.245, 95.354, 95.322, 95.405, 95.266, 95.308, 95.444, 95.547, 95.482, 95.332, 95.439, 95.44, 95.427, 95.58, 95.432, 95.637, 95.494, 95.461, 95.677, 95.598, 95.669, 95.656, 95.793, 95.559, 95.917, 95.592, 95.695, 95.63, 95.89, 95.805, 95.898, 95.862, 95.659, 95.864, 95.834, 96.011, 95.828, 95.979, 95.793, 95.981, 95.983, 95.85, 96.114, 95.974, 96.032, 96.126, 96.001, 95.988, 96.049, 96.076, 95.993, 96.305, 96.047, 96.142, 96.192, 96.046, 96.247, 96.082, 96.255, 96.317, 96.256, 96.163, 96.068, 96.239, 96.238, 96.219, 96.17, 96.287, 96.341, 96.323, 96.373, 96.213, 96.515, 96.143, 96.246, 96.331, 96.267, 96.35, 96.374, 96.484, 96.452, 96.249, 96.496, 96.524, 96.364, 96.501, 96.523, 96.412, 96.538, 96.479, 96.424, 96.387, 96.594, 96.559, 96.489, 96.553, 96.448, 96.573, 96.555, 96.542, 96.648, 96.652, 96.57, 96.584, 96.563, 96.609, 96.627, 96.667, 96.699, 96.581, 96.612, 96.677, 96.679, 96.632, 96.666, 96.519, 96.637, 96.657, 96.672, 96.842, 96.58, 96.592, 96.76, 96.855, 96.668, 96.702, 96.631, 96.838, 96.687, 96.871, 96.788, 96.867, 96.775, 96.851, 96.751, 96.846, 96.894, 96.742, 96.881, 96.734, 96.963, 96.804, 96.87, 96.915, 96.81, 96.939, 96.736, 96.928, 96.857, 96.816, 96.854, 97.018, 96.966, 96.908, 96.988, 96.974, 96.879, 96.901, 96.886, 96.997, 97.026, 96.995, 96.827, 97.046, 96.993, 96.989, 97.026, 96.945, 96.898, 97.042, 97.091, 96.858, 96.906, 97.167], 'train_loss': [4.366441016310083, 3.535918966593532, 3.114079346507311, 2.8157736530535815, 2.581832195502859, 2.3908648884456585, 2.219970676125583, 2.0663495503094285, 1.9226701368678478, 1.7834073503430807, 1.6587161313091725, 1.5374368386854367, 1.4221632853579385, 1.315675202807172, 1.2167971406086697, 1.1254775448632561, 1.0392867207183947, 0.9629582377557029, 0.8898604041631605, 0.8257332802085791, 0.7729173662165991, 0.7254340721488532, 0.68170172406059, 0.6363955489237646, 0.6113103726546275, 0.5708689148694525, 0.5449751301103117, 0.519083601475639, 0.49248123951184775, 0.4748689886837988, 0.46253480428564037, 0.4405981060820593, 0.4269853930679637, 0.414399437844677, 0.39856861352977735, 0.391124643394944, 0.37557748594557866, 0.3676531058774876, 0.3671157668253511, 0.3502466788733532, 0.34471471786680163, 0.33179910922624406, 0.3302964166960347, 0.31707937299957356, 0.3145309555381822, 0.3144905816046229, 0.3038714844054201, 0.2970600221008158, 0.2896387000244661, 0.2863178586674305, 0.28966051758632244, 0.2824357621257342, 0.2741391661762238, 0.274361559165426, 0.2647088918023369, 0.26674368597271536, 0.25184473888716175, 0.2565842799422875, 0.25428671921321544, 0.2515823409445131, 0.24880786073976272, 0.24490648014107463, 0.23532524852705397, 0.2428729586542768, 0.2330178443380582, 0.23653408109197918, 0.23274532128757036, 0.22981337818983877, 0.22434524374388953, 0.22764177951587558, 0.21978016360290467, 0.21831527940085205, 0.2202027966134293, 0.2138410901623377, 0.21627580571528798, 0.2170534233907649, 0.20863162761438525, 0.2111497637032223, 0.21171005968640838, 0.1959281927689681, 0.20880727786737627, 0.1948774691101495, 0.20480467137414068, 0.20437324490107392, 0.1952138324265183, 0.1964412748161763, 0.1901742914627215, 0.19763618778563696, 0.18436011328211177, 0.18858960405425887, 0.18719557474319057, 0.19446639821555414, 0.17945231306792658, 0.18604599525740875, 0.17965199866839188, 0.1832311172908266, 0.18233569328609428, 0.1803341255504197, 0.17846772084411852, 0.18098707240663578, 0.1742816682039776, 0.1766538277002658, 0.17686722062346993, 0.17321628119119623, 0.18072588082762842, 0.16607512273633243, 0.174787281330744, 0.16568914637915785, 0.1615651030905583, 0.17012350767554735, 0.1672279566481128, 0.16404219236289241, 0.16591805435379853, 0.16525718070726345, 0.16434654442448066, 0.16556450687837446, 0.157568694159665, 0.1648147964003357, 0.1529684270388296, 0.16112543792527595, 0.16044386705474228, 0.1597909543747279, 0.1572796514301584, 0.1521839487619021, 0.15522660237426558, 0.15414344012899348, 0.1560461262410229, 0.15318605870542357, 0.15442657087240141, 0.14877266025525815, 0.1545067543282471, 0.1523687584830361, 0.14683793187683647, 0.14707342899384804, 0.14556875357241086, 0.15064942928314715, 0.1476163043499336, 0.14750019727830113, 0.1461841716998336, 0.1436733368483878, 0.14790405756680702, 0.14315066415288022, 0.14501908448739578, 0.1480200076064599, 0.14450058897540702, 0.14132551309385682, 0.1439289487533388, 0.14221666473329247, 0.14069981339535964, 0.1456398501755671, 0.13551471580620747, 0.14252684124172216, 0.14089519354280605, 0.14452151885620834, 0.1387000377479135, 0.13757861500739413, 0.1356116262705781, 0.1346679976920854, 0.1378205409530468, 0.1352310728151244, 0.13733036476810076, 0.13034193849555137, 0.13747241200285742, 0.13231664188819928, 0.13630771223349605, 0.132616630535941, 0.1313713187450437, 0.13725408251437235, 0.12969245340698057, 0.1333259578962719, 0.13122366638901814, 0.1294409750234717, 0.1300273833011938, 0.13236847604277543, 0.13027895111602542, 0.1279989081510579, 0.13197595489374347, 0.1228457405620689, 0.13326166490021668, 0.1264055318975796, 0.12433240962045992, 0.13007106930696746, 0.12346910781874151, 0.12692730807511868, 0.1258922756586393, 0.1243175409639806, 0.12195154656498185, 0.13024076820164682, 0.12917717929791853, 0.12655624694058937, 0.1252332079837722, 0.12421495255955178, 0.12489990143962526, 0.12270467943256877, 0.12131713533329212, 0.1225306368523807, 0.11957246452840778, 0.12350953408638796, 0.11561525317100524, 0.12743262555262508, 0.12203217613104414, 0.1241731379034864, 0.12438264138028633, 0.12241484100835949, 0.12167771269703762, 0.11494894470086695, 0.12179427960859546, 0.12267166780312656, 0.11757065362838631, 0.11350720860949992, 0.12005868623577182, 0.11481675625166791, 0.11652953442287711, 0.12151129049191195, 0.11685979922824336, 0.1170760839008169, 0.12151039691568338, 0.12201777039301627, 0.11303047865083392, 0.11482030904104949, 0.1208011634459109, 0.11730811702823701, 0.11830863834660255, 0.11668217247979142, 0.11507746575236968, 0.1175645528118688, 0.11259815888926245, 0.11345863245257491, 0.11496087890466936, 0.11380175382931348, 0.11383832653105391, 0.11449991482238793, 0.11192040527303497, 0.11238692110654706, 0.11212616402748793, 0.11422642894441969, 0.11272742839207454, 0.1111848346194996, 0.11224043822992695, 0.1132499233519233, 0.11132366196919764, 0.11751522879686471, 0.11240001388596668, 0.11283595921857978, 0.1113045712093458, 0.1073853295282726, 0.11436925469945355, 0.11234181237316757, 0.10841324878335167, 0.10485093225836718, 0.11181454002844993, 0.11017957224991168, 0.11252873703044608, 0.10777963213107154, 0.11114970909535703, 0.10403754952938969, 0.10758700968235282, 0.10517187870182222, 0.10946987717942278, 0.10594125176715573, 0.10934984999920658, 0.10752757319887672, 0.10517808758281737, 0.1103691695453462, 0.10556674606333397, 0.1102748334983403, 0.10409622352915296, 0.10835776252118526, 0.1061348119938388, 0.10555520524852238, 0.10829941549832067, 0.10189542015364975, 0.11053513329077193, 0.10234690905289882, 0.1064234688339443, 0.10901179882628195, 0.10408313937755737, 0.1000517198102926, 0.09994686455090837, 0.10603571077726807, 0.10124410904588069, 0.10125754859606227, 0.10539743491725609, 0.10319473596520053, 0.10635615477133757, 0.10192765141285015, 0.09977191315909269, 0.10192818844892616, 0.106152007099826, 0.10079003011739872, 0.10171998060229893, 0.10268745139919451, 0.10218952496011202, 0.1022172972671056, 0.1059769401385337, 0.10123733070671412, 0.0991034273545884, 0.10595803584823633, 0.10497645265342694, 0.09611786457768287], 'val_metric': [12.45, 22.76, 26.83, 32.96, 35.55, 38.07, 38.99, 40.55, 41.47, 42.47, 42.75, 41.58, 42.57, 41.9, 42.04, 42.1, 41.92, 41.95, 42.17, 40.49, 41.42, 40.86, 40.71, 40.79, 40.63, 40.68, 40.92, 40.71, 40.39, 39.96, 40.45, 41.48, 40.37, 41.11, 40.6, 39.98, 40.6, 41.06, 40.69, 40.68, 41.53, 40.21, 40.3, 40.23, 41.24, 40.78, 40.87, 41.06, 40.7, 41.05, 40.79, 40.87, 40.32, 40.51, 41.26, 41.03, 40.49, 39.49, 41.16, 40.75, 40.91, 41.08, 41.08, 40.6, 41.48, 40.63, 40.5, 40.53, 41.28, 40.56, 39.61, 41.28, 40.44, 42.17, 41.5, 41.88, 41.27, 40.99, 40.72, 41.58, 41.92, 41.63, 41.36, 41.79, 41.29, 41.43, 41.41, 40.99, 41.18, 40.77, 41.62, 41.48, 41.24, 41.36, 41.39, 41.47, 42.24, 41.61, 41.47, 41.41, 41.58, 41.53, 41.72, 41.47, 41.84, 42.02, 42.11, 41.31, 41.24, 42.01, 42.33, 42.19, 41.7, 41.49, 41.36, 41.85, 41.79, 41.96, 41.91, 40.87, 42.55, 42.33, 41.77, 41.2, 42.02, 41.95, 42.86, 41.47, 42.18, 42.08, 42.21, 41.66, 41.75, 41.48, 42.06, 41.89, 42.48, 42.35, 42.21, 41.65, 42.14, 41.8, 41.81, 41.85, 42.64, 42.23, 42.45, 43.07, 42.94, 42.19, 41.86, 41.92, 42.12, 42.55, 42.36, 42.12, 42.73, 43.14, 42.07, 42.47, 41.94, 42.82, 41.91, 42.62, 42.45, 41.37, 42.88, 42.2, 42.17, 42.37, 42.07, 41.98, 42.78, 42.43, 42.39, 42.5, 42.44, 42.5, 41.46, 42.15, 41.73, 42.56, 42.0, 41.26, 42.09, 42.6, 42.23, 42.56, 42.78, 42.79, 42.37, 42.17, 42.55, 42.93, 42.74, 42.4, 42.64, 43.12, 42.27, 42.73, 43.1, 42.39, 42.43, 41.89, 41.68, 42.53, 42.33, 42.35, 41.91, 41.83, 42.1, 42.76, 41.55, 42.13, 43.11, 43.1, 43.31, 42.35, 43.2, 42.88, 43.34, 42.13, 42.97, 42.48, 43.01, 42.81, 43.41, 42.63, 43.22, 42.85, 42.42, 42.88, 42.44, 42.87, 43.19, 43.44, 42.86, 42.98, 42.85, 42.74, 42.43, 42.12, 43.33, 42.86, 43.49, 43.82, 42.87, 41.9, 42.08, 43.15, 42.89, 42.67, 43.55, 42.91, 42.55, 42.9, 42.44, 42.87, 43.04, 43.37, 42.99, 42.77, 42.85, 43.54, 42.16, 43.16, 43.32, 43.21, 42.54, 42.56, 43.16, 42.7, 43.43, 42.95, 42.72, 42.69, 42.37, 42.82, 42.69, 42.73, 42.84, 43.44, 42.97, 42.47, 42.84, 43.18, 43.05, 41.81, 43.1, 42.71, 42.36, 42.29, 43.17, 43.26, 43.23, 43.16, 42.94, 42.63, 42.72, 43.05], 'val_loss': [4.190954993484886, 3.3352933879111224, 3.160648138659775, 2.8216876156011206, 2.7147924770974807, 2.6221101435886065, 2.5853059542406895, 2.5208096549769117, 2.5525295297810984, 2.5053243113171524, 2.541763877792723, 2.659587920851009, 2.6350540230228643, 2.775540071308233, 2.8323804724747967, 2.931834345410584, 2.969098321191824, 3.0797942003626733, 3.182886055320691, 3.4565016517213953, 3.4315929564700762, 3.553647029931378, 3.672857802384978, 3.7153130162293744, 3.7635472792728693, 3.860007787965665, 3.9142244735341163, 4.072871516464622, 4.114368851777095, 4.251306838290707, 4.218153390155476, 4.172033650859905, 4.34564146038833, 4.355718821476978, 4.469609566554902, 4.581251340307248, 4.5734374917996155, 4.543812096498574, 4.601883907986295, 4.635836793358918, 4.662361984799622, 4.757562532546414, 4.826654659714668, 4.850970153595991, 4.84464222230729, 4.719055648821934, 4.81887154518419, 4.835078199957587, 4.919907818933961, 4.950266412868621, 4.928635051296015, 4.942552204344683, 5.040181586696844, 5.023921820768125, 5.178568809655062, 5.1891743058611635, 5.179227923891347, 5.327349167720527, 5.162356523951148, 5.1934273675748495, 5.247411877486356, 5.188612847571161, 5.338254630945291, 5.302724864072861, 5.319421598106433, 5.3367487579394295, 5.4866525656098775, 5.544937208199957, 5.401573049794337, 5.467282290671282, 5.5938845379337385, 5.54599516558799, 5.449270553649611, 5.4060320428981905, 5.379791952242517, 5.377602721475492, 5.533601265804023, 5.551070148018515, 5.521537229513666, 5.5355274252071505, 5.430339249835652, 5.587529976656483, 5.627304994376602, 5.44802053263233, 5.6417886664153665, 5.552223852485608, 5.578534185506736, 5.509888942833919, 5.582501330952735, 5.77718294046487, 5.608840804191152, 5.718631468001445, 5.823131609874166, 5.77866259046421, 5.838289708848212, 5.881054837992237, 5.810231751697079, 5.675786382833104, 5.789767625984872, 5.6563585503086165, 5.853011313517382, 5.8557829264622585, 5.8010936831213105, 5.918664584493941, 5.863564722856898, 5.8877796245988, 5.778049528219138, 6.013646943553997, 6.002961359206279, 5.910878934677998, 5.869204182533702, 5.773456435294667, 5.75728290399928, 5.9970573361512205, 5.929012722270504, 5.980362438092566, 6.052952829439929, 5.9784125142796025, 5.986207873957931, 6.0410748985922265, 5.951445622049319, 6.009058659243736, 6.044283983813729, 6.016367488605961, 6.132643099803074, 6.111985182306569, 5.9571224853491325, 6.060881312485713, 6.042371718746842, 6.024826998923235, 6.09986841298972, 6.115338355872281, 6.073825679766904, 6.097961623197908, 6.049745544506486, 6.145441257270279, 6.093703365629645, 6.173767141475799, 6.2644490922332565, 6.165422447168144, 6.236081410365499, 6.2557001706141575, 6.15249743886814, 6.152060145785095, 6.293080745988591, 6.343902920461764, 6.197814809288948, 6.113548309180387, 6.27537795768422, 6.309924541764958, 6.283644173555313, 6.25219675234169, 6.226131899341656, 6.076102357001821, 6.270923568944263, 6.277646218135858, 6.080154643696584, 6.23896776794628, 6.16311221669434, 6.3158129263835345, 6.301895235754122, 6.198865400757759, 6.378691989145461, 6.2564999161252555, 6.249852534312351, 6.542259643032293, 6.199406051332025, 6.3037652999732146, 6.392481652035076, 6.430498552929824, 6.380130690374192, 6.329492707161387, 6.286225162493955, 6.222705421933703, 6.320962032694726, 6.246826466481397, 6.450039911422, 6.294266132792091, 6.408320199152467, 6.599815917622512, 6.31237886513874, 6.3001597626194075, 6.527237079705403, 6.742158238295537, 6.512219846628274, 6.4174919553623075, 6.592968889862109, 6.316101303525791, 6.480900284590994, 6.3612620086426945, 6.410482750576772, 6.550819748526166, 6.455176386863562, 6.495668638284039, 6.509818690597632, 6.425151495417213, 6.528883711547609, 6.389909344873611, 6.5612358386349525, 6.5514018171152495, 6.288738428407414, 6.451211472985092, 6.43256514543181, 6.568240095855324, 6.461621882809196, 6.612192017257593, 6.518857758515959, 6.389800573610197, 6.5905143971655775, 6.721217219237309, 6.562641985097509, 6.5280439382905415, 6.858179210857221, 6.628146745596721, 6.578789256940222, 6.517388639936022, 6.517477917823062, 6.662164218866142, 6.55737239084426, 6.478192876099022, 6.683943373382471, 6.681109935614713, 6.724396624382893, 6.6950795969386006, 6.316439921688882, 6.623042199262388, 6.442317671077267, 6.639695683861993, 6.581242980471083, 6.71256263696464, 6.624079807548766, 6.713292667060901, 6.634357610325904, 6.594554381765378, 6.657755326313578, 6.590350954395951, 6.7604632362438615, 6.606221596906139, 6.635515065709496, 6.5847640781645564, 6.647798040110594, 6.691850721456443, 6.7023366545415985, 6.459935077436411, 6.489470365700448, 6.7091394716007695, 6.805435692428783, 6.791946607030881, 6.83805072990952, 6.583327700378029, 6.645341467705502, 6.80941704579979, 6.73382807233531, 6.581419692677297, 6.635052741712825, 6.820540461570594, 6.870135217715221, 6.830919069849002, 6.782124420639816, 6.723305787250495, 6.595786509240509, 6.71048992606485, 6.745988472252135, 6.695984242068734, 6.619320679622091, 6.664705147409135, 6.743263657685298, 6.899674811940284, 6.717328262936538, 6.736361650904273, 6.654448766617259, 6.718828702428539, 6.87367268428681, 6.9435561082925, 6.881190890719177, 6.878860639159087, 6.920303179959583, 6.9362356966468175, 7.0040196704257065, 6.7478307280570835, 6.807555207780972, 6.766744098845561, 6.67248099624731, 6.930643913852181, 6.5838497284871, 6.763883943011047, 6.823443631457675, 6.962565593658739, 6.769846787118608, 6.81445452665827, 6.870643472975226, 6.951719631814653, 6.940228261765401, 7.080901062412626, 6.990440500769646, 6.94914736565511, 7.112085025021984, 7.076015967472344, 6.88238310054609, 7.00583665689845]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal 2", optimizer="SGD")
data.update({'train_metric': [0.645, 0.826, 1.004, 1.175, 1.307, 1.482, 1.617, 1.735, 1.847, 1.991, 2.145, 2.368, 2.442, 2.572, 2.775, 2.993, 3.127, 3.361, 3.503, 3.704, 3.918, 4.161, 4.32, 4.7, 5.011, 5.158, 5.542, 5.756, 5.995, 6.29, 6.52, 6.77, 7.131, 7.473, 7.643, 7.895, 8.09, 8.407, 8.643, 8.916, 9.218, 9.398, 9.599, 9.852, 10.038, 10.35, 10.589, 10.716, 11.001, 11.121, 11.434, 11.67, 11.851, 11.852, 12.153, 12.412, 12.648, 12.713, 13.178, 13.216, 13.679, 13.568, 13.755, 13.989, 14.276, 14.437, 14.743, 14.947, 14.988, 15.257, 15.449, 15.625, 15.681, 16.079, 16.244, 16.59, 16.555, 16.989, 17.063, 17.269, 17.465, 17.797, 17.979, 18.141, 18.267, 18.561, 18.773, 18.927, 19.21, 19.215, 19.498, 19.756, 19.964, 20.101, 20.332, 20.506, 20.877, 20.953, 21.361, 21.218, 21.576, 21.69, 21.963, 22.229, 22.47, 22.626, 22.96, 22.969, 23.224, 23.376, 23.623, 23.911, 24.058, 24.321, 24.475, 24.587, 24.915, 25.06, 25.161, 25.392, 25.716, 25.752, 26.111, 26.402, 26.291, 26.671, 26.833, 26.994, 27.328, 27.525, 27.599, 27.925, 28.037, 28.336, 28.554, 28.604, 28.667, 28.948, 29.235, 29.334, 29.405, 29.843, 29.973, 30.123, 30.244, 30.56, 30.903, 30.892, 31.229, 31.421, 31.587, 31.714, 31.94, 31.998, 32.289, 32.49, 32.804, 32.922, 33.204, 33.312, 33.475, 33.501, 33.94, 34.057, 34.355, 34.26, 34.637, 34.91, 34.96, 35.158, 35.478, 35.412, 35.701, 35.931, 35.973, 36.549, 36.536, 36.49, 36.796, 37.131, 37.236, 37.311, 37.557, 37.78, 38.267, 38.148, 38.486, 38.431, 38.793, 38.908, 39.083, 39.192, 39.403, 39.923, 39.849, 40.157, 40.321, 40.636, 40.738, 40.927, 41.134, 41.304, 41.301, 41.689, 41.786, 42.213, 42.225, 42.307, 42.448, 42.765, 42.667, 43.143, 43.217, 43.37, 43.91, 43.952, 44.094, 44.368, 44.195, 44.568, 44.866, 44.912, 45.202, 45.273, 45.657, 45.859, 45.893, 45.903, 46.518, 46.364, 46.603, 47.016, 47.171, 47.023, 47.277, 47.614, 47.699, 47.826, 47.978, 48.095, 48.489, 48.629, 48.838, 48.971, 49.095, 49.461, 49.657, 49.739, 49.976, 50.041, 50.337, 50.179, 50.474, 50.691, 51.09, 51.073, 51.353, 51.475, 51.758, 51.827, 51.87, 51.867, 52.369, 52.519, 52.466, 52.713, 52.969, 53.131, 53.275, 53.392, 53.651, 53.766, 53.88, 54.164, 54.462, 54.384, 54.547, 54.77, 55.029, 55.153, 55.199, 55.578, 55.684, 55.829, 55.685, 56.177, 56.215, 56.452, 56.348, 56.717, 57.004, 57.11, 57.364, 57.347, 57.859, 57.722, 57.833, 58.065, 58.058, 58.306], 'train_loss': [5.303329820443786, 5.269924601300436, 5.249439836844983, 5.2332437665357245, 5.219787277324186, 5.204857869859087, 5.18977262176006, 5.175050246433348, 5.157631239826986, 5.136850505933804, 5.111202324496884, 5.081939045618683, 5.053475541940348, 5.022462693834945, 4.992297390677268, 4.963244006790874, 4.936003053180697, 4.909265481884176, 4.885174454135614, 4.860066652450513, 4.835473009964936, 4.810139783284486, 4.781349164434373, 4.752801457659525, 4.721339656844478, 4.691457842987314, 4.660389857496578, 4.629446374706481, 4.598888660194167, 4.566529808788824, 4.536548629145704, 4.508594443076555, 4.479383364901357, 4.454061490484178, 4.425305069980145, 4.402307188365983, 4.377597859175786, 4.3516552501661385, 4.3301541074605145, 4.305508324219795, 4.284726374895238, 4.261900451346536, 4.2432141012666476, 4.2243877157368726, 4.20740008186394, 4.185526202599055, 4.16986103005998, 4.15408416779775, 4.138397143273039, 4.120792969174669, 4.104613373085847, 4.087812237760918, 4.072122127370657, 4.058963352643902, 4.045513832637765, 4.02925856984432, 4.016630348301971, 4.0041643227817, 3.987641936910511, 3.9720708637457207, 3.9606344088177914, 3.9448124302249647, 3.9311581713529407, 3.9195862897908316, 3.904586234614396, 3.8933832383049047, 3.8789989814038317, 3.866430451148455, 3.852804974417464, 3.839497719524918, 3.8229409525658333, 3.8135306665856183, 3.7997805274455727, 3.788360006444666, 3.774258261068616, 3.7576800899786287, 3.7446546664393567, 3.7294571271555896, 3.715796970512649, 3.7028060690073805, 3.6914175115025203, 3.674193240249302, 3.6621576922106116, 3.6521880693795663, 3.635296582870581, 3.6232321925904607, 3.6066903628680618, 3.594049712441628, 3.577944248209225, 3.5680149213213688, 3.552624061331868, 3.5423519088149606, 3.5277851730756704, 3.513319144856068, 3.495856423753237, 3.48490593987097, 3.473473630650106, 3.459814639100644, 3.446314311835984, 3.4348526450966843, 3.4211865674968873, 3.4073328776612772, 3.3949159279284533, 3.3813417502419734, 3.3706133194787338, 3.3578046435968125, 3.345180622523058, 3.330384251709863, 3.3206358520327206, 3.3055287775746236, 3.294952788722111, 3.284062251324693, 3.268394177644892, 3.2596693051143557, 3.2471488620406608, 3.2346658305487264, 3.222845359711943, 3.210046862305088, 3.200657743257509, 3.1893279807199977, 3.1746116178735893, 3.165891161044286, 3.153818697175839, 3.142558509542327, 3.131990013256793, 3.120200931911505, 3.1077012789989236, 3.099393594898021, 3.085102815164333, 3.0757440523085346, 3.0644436831361426, 3.052929556835026, 3.041696749813497, 3.0341535702166613, 3.022938162870157, 3.0106138894745578, 3.0020713966318375, 2.990295982330332, 2.9791595214311695, 2.967622214681585, 2.9563586692053465, 2.9444236540290993, 2.936853809686174, 2.9256265863576, 2.9163598248536053, 2.9014048199583455, 2.892605452223306, 2.8800034079121537, 2.871586609176543, 2.862362384185986, 2.851185500888739, 2.841484720098309, 2.8301778516934153, 2.821273537225168, 2.8104887461708055, 2.80043746123921, 2.788180984141959, 2.7760793459545092, 2.7665533297960376, 2.7560111921106634, 2.7517255708641986, 2.7382745825168953, 2.7219856105701936, 2.71539214476514, 2.7029908208304008, 2.6962500538340914, 2.6823412499516266, 2.671256923477236, 2.6613180052372254, 2.6568126515249983, 2.640621404806468, 2.6386031691111285, 2.6230100883708425, 2.6126539344140833, 2.6072765967819986, 2.59074352173491, 2.5830882983717185, 2.5738858407838787, 2.5617151245122067, 2.552451828390989, 2.541062744244962, 2.532651624081612, 2.523802524869898, 2.5124249528480966, 2.498074605917976, 2.4912195660254928, 2.478222217608627, 2.474127246718794, 2.462074627269176, 2.449405524796274, 2.441521111773285, 2.4311545378720996, 2.4229155724733515, 2.4077337040629665, 2.402764790270165, 2.3909038386891, 2.3793723581696047, 2.365597106978745, 2.361882305419834, 2.354621699431426, 2.3411850230631277, 2.3326745118686043, 2.32256867453904, 2.3134547628352675, 2.3054117408991623, 2.2870109506852336, 2.286168418629232, 2.2722475754474876, 2.264970516899192, 2.253802780302686, 2.244766243817481, 2.2358822275718206, 2.228661326056326, 2.2138147132181616, 2.202717335271439, 2.195272122677213, 2.1849953695817095, 2.175039521868383, 2.171052801479381, 2.1590754188029946, 2.148885717471288, 2.1396085008626096, 2.1343556480840924, 2.119504734177812, 2.1077179454948682, 2.100791239463894, 2.0934552825267545, 2.086183395205746, 2.07232996453403, 2.065864782110667, 2.0557926043591586, 2.0441975362470193, 2.039025690795059, 2.0315103139621056, 2.0228471140486266, 2.010135475062287, 2.0067078836133523, 1.9940343909132427, 1.986714808283444, 1.9743409741786682, 1.964624261062876, 1.9565376982960423, 1.9481743668716685, 1.9411890110905476, 1.9316838788284527, 1.917915020626627, 1.9114347639102167, 1.9025396048374383, 1.8947528987379312, 1.8877533973979401, 1.876340744972839, 1.8749840943689768, 1.8629026089581022, 1.8541089744805832, 1.841414805566052, 1.837301121563463, 1.831828090294919, 1.8231735754226617, 1.8116017161312579, 1.8073875287062682, 1.7980402912455953, 1.7943712425628535, 1.7823622469252223, 1.768850966058171, 1.7681862652034845, 1.7610025818501995, 1.7461093094435856, 1.7410348112279608, 1.7338525973598848, 1.7226506234435646, 1.7227051526555937, 1.7130573127640414, 1.7027400780090214, 1.6955664327948503, 1.684062010953614, 1.6830101281652372, 1.6733332373587961, 1.664150760941069, 1.6555490531146488, 1.6510855444562182, 1.643392711591812, 1.6330846739516987, 1.624598804911359, 1.6228928459964345, 1.6161160723795436, 1.6006144630703039, 1.5998467713766042, 1.592306822595578, 1.592236754532739, 1.57948606066115, 1.5673275708542065, 1.5614022332662507, 1.5524288779333166, 1.5507575892059755, 1.535754865587177, 1.5372725493314552, 1.5288398526299098, 1.5206509238241273, 1.5175146883447737, 1.5086954673817732], 'val_metric': [0.84, 1.06, 1.21, 1.26, 1.43, 1.58, 1.76, 1.94, 2.14, 2.33, 2.61, 2.59, 2.84, 2.95, 3.13, 3.41, 3.72, 3.84, 3.99, 4.14, 4.33, 4.6, 4.89, 5.23, 5.59, 5.77, 5.8, 6.33, 6.53, 6.81, 7.06, 7.45, 7.76, 8.12, 8.16, 8.54, 8.85, 9.12, 9.24, 9.27, 9.55, 9.83, 9.94, 10.38, 10.55, 10.71, 10.84, 11.25, 11.52, 11.72, 11.72, 12.1, 12.15, 12.31, 12.74, 12.74, 13.06, 13.07, 13.31, 13.24, 13.52, 13.96, 14.14, 14.23, 14.58, 14.54, 14.65, 14.88, 15.15, 14.93, 15.63, 15.5, 15.71, 15.86, 15.92, 16.24, 16.42, 16.67, 16.6, 16.84, 16.96, 17.62, 17.54, 17.83, 18.08, 18.19, 18.6, 18.57, 18.88, 18.86, 19.31, 19.24, 19.69, 19.63, 20.09, 19.88, 20.19, 20.4, 20.57, 20.79, 21.1, 21.19, 21.26, 21.4, 21.41, 21.55, 21.87, 21.78, 22.44, 21.77, 22.77, 22.47, 22.61, 22.83, 22.79, 23.29, 23.35, 23.27, 23.51, 23.35, 23.63, 23.58, 23.73, 23.59, 23.89, 23.95, 24.23, 23.83, 24.54, 24.53, 24.37, 24.76, 24.66, 24.67, 24.62, 24.66, 24.89, 25.03, 25.23, 24.91, 25.13, 25.43, 25.2, 25.39, 25.0, 25.32, 25.77, 25.9, 26.0, 25.86, 25.18, 25.76, 25.71, 25.99, 25.52, 25.77, 25.57, 25.98, 26.28, 25.54, 26.05, 26.35, 26.12, 25.89, 26.2, 26.12, 26.16, 25.97, 25.89, 26.14, 26.2, 26.12, 26.31, 25.59, 26.4, 25.88, 25.92, 25.81, 25.8, 26.21, 26.42, 26.47, 26.16, 25.56, 26.3, 26.19, 25.85, 25.99, 26.2, 26.05, 25.8, 25.87, 26.0, 25.93, 25.73, 25.81, 26.01, 26.09, 25.48, 26.23, 26.24, 25.84, 25.67, 25.29, 26.12, 25.82, 25.91, 25.76, 25.82, 25.43, 26.07, 25.26, 25.45, 25.8, 25.09, 25.44, 25.42, 25.21, 25.63, 25.3, 25.65, 25.27, 25.41, 24.49, 24.87, 24.7, 25.2, 25.05, 25.36, 24.94, 25.08, 24.84, 25.32, 25.24, 25.14, 24.71, 24.78, 24.74, 25.17, 25.28, 24.33, 24.37, 24.46, 24.73, 24.05, 24.75, 24.54, 24.27, 24.72, 24.73, 24.16, 24.75, 24.48, 24.36, 24.35, 23.98, 24.07, 24.65, 24.5, 24.1, 23.89, 24.12, 24.4, 23.97, 24.21, 24.12, 23.73, 23.91, 23.25, 22.83, 23.83, 24.06, 24.31, 23.24, 23.85, 24.24, 24.23, 24.02, 23.89, 23.5, 23.53, 24.09, 23.71, 23.79, 23.26, 23.67, 24.21, 23.59, 23.43, 22.98, 23.54, 23.16, 23.85, 23.84, 23.67, 23.94, 23.68, 23.7, 23.6, 23.69], 'val_loss': [5.277715913809029, 5.2555678270425, 5.238553520980155, 5.2223386415250745, 5.206157801257577, 5.192854910139825, 5.176512379555186, 5.159658722057464, 5.141042142916637, 5.116644886648579, 5.08833290968731, 5.055976348318112, 5.0243947111117615, 4.991034591273897, 4.960908049990417, 4.930486759562402, 4.903040481980439, 4.873344863296315, 4.850474457831899, 4.821909828550496, 4.794782858745307, 4.768499306053113, 4.74133211791895, 4.707568637884346, 4.67887240002869, 4.64255353295879, 4.611303259612648, 4.578453211268043, 4.546099160127579, 4.512570816999788, 4.482195514022925, 4.452798150906897, 4.42696363910748, 4.400301617421921, 4.376404988537928, 4.3439059713084225, 4.320144383011351, 4.306782684508403, 4.275751255120442, 4.258193948466307, 4.233068583117928, 4.21828725079822, 4.198498921789182, 4.182234662353613, 4.161405335566041, 4.1466774606401, 4.130707312541403, 4.114005410747164, 4.096928099917758, 4.080822596884078, 4.0688435560578755, 4.057814927617455, 4.053758862671579, 4.031431480577797, 4.012306839037853, 4.00019383430481, 3.988191018438643, 3.9731597809275243, 3.965262227756962, 3.9460764159062864, 3.9451949049712747, 3.934658038388392, 3.9097748470913833, 3.9001070557126574, 3.8888258098796675, 3.8895334331852616, 3.867640177915051, 3.8554520895526667, 3.8383647924775532, 3.8413272754401917, 3.8199999484286944, 3.810245778150619, 3.797558129972713, 3.7850150378646363, 3.7792332703900184, 3.7590935290998715, 3.752341473937794, 3.7487232259884, 3.738787227375492, 3.72261531034093, 3.7080541097434465, 3.68417950952129, 3.681784903167919, 3.6658435763826795, 3.6580214682658005, 3.6442579797878385, 3.6392660110619417, 3.624237771246843, 3.6145739099781986, 3.5979018591012166, 3.5909712967599274, 3.580428319372189, 3.5679867252422746, 3.556430581268991, 3.548947076129306, 3.5427627373652855, 3.5376953516796137, 3.517758082432352, 3.5038310843668166, 3.496701788750424, 3.4846446984892436, 3.4757284844756886, 3.4674968871341387, 3.465621926981932, 3.4486318993720277, 3.4408194016499123, 3.42878581536044, 3.421037709637053, 3.4220666057744604, 3.4137188879547606, 3.3927168975210495, 3.3846241379999054, 3.3790434522993245, 3.38035106886724, 3.375811341461862, 3.3663256608756487, 3.3626687496331087, 3.3595585238402057, 3.3471769367813304, 3.335487216141573, 3.3295670375702486, 3.326825939166318, 3.3353651822752255, 3.3177356036605348, 3.311453121482946, 3.312522292137146, 3.30470038219622, 3.312379124817575, 3.306140094805675, 3.2883469435819395, 3.28721729964967, 3.2758020838354804, 3.2756699938682994, 3.2749654198907745, 3.288762481349289, 3.270209438481908, 3.2720903955447445, 3.257020515241441, 3.2687927074493115, 3.2636446246675623, 3.2587994033363974, 3.258587567669571, 3.2469789161803617, 3.257386364754598, 3.263890349940889, 3.2635182301709604, 3.23413015778657, 3.2437318123070296, 3.2378443046739904, 3.2412913270816683, 3.2512442336720264, 3.239880558032139, 3.2437839546021383, 3.243068706457782, 3.267074868177912, 3.236547445036044, 3.233923621238417, 3.2271178231877125, 3.231545690518276, 3.2409555760158857, 3.242225375904399, 3.2383955603192565, 3.2342629523793605, 3.2558991362334817, 3.239916578219954, 3.2482607956904515, 3.23783055915954, 3.251997119302203, 3.253612739265345, 3.2492733138382057, 3.2576440808120046, 3.252663545547777, 3.2581637224574, 3.253916072238023, 3.269536396500411, 3.272786222445737, 3.280493792454908, 3.296186454736503, 3.2657544423060814, 3.2670149841126364, 3.266062439626949, 3.267050722602067, 3.287434823953422, 3.33525145129793, 3.285007190552487, 3.2863495235989806, 3.311651213153912, 3.3266275407402377, 3.2960225791688176, 3.324616410929686, 3.3197962151970835, 3.337902756253625, 3.329333997835779, 3.336996190866847, 3.375339684213043, 3.3545473646965758, 3.3704043368625034, 3.382005370346604, 3.3691100770500815, 3.375855190738751, 3.348802155749813, 3.400426844123063, 3.3961914648675613, 3.4455401069799048, 3.411054890626555, 3.4020796254941614, 3.398536977494598, 3.440622785288817, 3.415336035619116, 3.4560005778719667, 3.4351907976114067, 3.486881660807664, 3.45304874098225, 3.4402728840044348, 3.4775451452109465, 3.4735685245246644, 3.4759803547221386, 3.4761031501612085, 3.4849201676192556, 3.557984845653461, 3.5238761043852302, 3.521597647363213, 3.536505595134322, 3.603879372785046, 3.532678850137504, 3.5578768860762287, 3.5467088093423538, 3.566408672909828, 3.5789596100521694, 3.587312018795378, 3.616731427277729, 3.62196205983496, 3.635603417256835, 3.650353850073116, 3.622861300304437, 3.6203238705920566, 3.7084599885211627, 3.6506913437205517, 3.65452558143883, 3.615131300725755, 3.697426214339627, 3.693092886809331, 3.7181366211289815, 3.7309646409028656, 3.731530325427936, 3.7123818245663007, 3.741917894144726, 3.7407551449575243, 3.7464968778525187, 3.754084170244302, 3.7976891736316074, 3.7895034597178174, 3.715111204772998, 3.7630830228708354, 3.844505847639339, 3.8486733482142164, 3.816768815562983, 3.8045264824180847, 3.8209964431774845, 3.851963613443314, 3.8704074704722995, 3.869774692377467, 3.899211194105209, 3.8569588630822054, 3.9772375981519175, 3.9032845269342897, 3.922673721981656, 3.966420132643098, 4.03589977428412, 4.071815141447031, 3.95301136089738, 3.9696981937262663, 3.925828784894032, 4.003409230785006, 4.020014869179695, 4.032719344090504, 4.018643269872969, 4.065845241971836, 4.053561872737423, 4.096051257127409, 4.0959966919224735, 4.054646405444783, 4.175182918074784, 4.12616482206211, 4.1151756754346716, 4.13053138696464, 4.138807617175352, 4.166486488026418, 4.1181129634759985, 4.264454905394536, 4.178914262990284, 4.211395619781154, 4.18776296961839, 4.2084684508621315, 4.202025331509341, 4.233760035721359, 4.172736781417944, 4.220954819090047, 4.2356920318239055, 4.262616919863755]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal 2", optimizer="Adam")
data.update({'train_metric': [8.127, 14.502, 17.596, 19.675, 21.806, 23.28, 24.698, 25.557, 26.9, 27.506, 28.382, 28.993, 29.798, 30.461, 31.043, 31.359, 31.92, 32.201, 32.474, 33.165, 33.463, 33.748, 34.043, 34.205, 34.75, 34.95, 35.047, 35.479, 35.44, 36.034, 36.132, 36.292, 36.562, 36.684, 36.957, 36.973, 36.938, 37.178, 37.428, 37.31, 37.641, 37.465, 37.69, 37.862, 38.037, 38.126, 38.039, 38.263, 38.402, 38.651, 38.476, 38.482, 38.682, 38.807, 38.753, 38.774, 39.05, 39.232, 38.842, 39.097, 39.404, 39.179, 39.442, 39.449, 39.611, 39.539, 39.63, 39.642, 39.755, 39.798, 39.797, 40.104, 40.012, 40.075, 40.001, 40.068, 39.927, 40.032, 40.191, 40.238, 40.227, 40.186, 40.305, 40.354, 40.4, 40.536, 40.366, 40.601, 40.627, 40.666, 40.483, 40.592, 40.698, 40.697, 40.608, 40.724, 40.832, 40.883, 40.919, 40.872, 40.957, 40.865, 40.767, 40.854, 40.858, 41.198, 40.963, 41.04, 41.202, 41.128, 41.179, 41.293, 41.338, 41.155, 41.506, 41.48, 41.189, 41.403, 41.376, 41.407, 41.318, 41.279, 41.436, 41.531, 41.578, 41.356, 41.505, 41.578, 41.733, 41.766, 41.503, 41.714, 41.618, 41.588, 41.761, 41.766, 41.966, 41.785, 41.83, 41.972, 42.032, 41.829, 41.879, 41.906, 41.845, 41.871, 41.781, 41.996, 41.995, 41.9, 42.234, 42.014, 42.173, 42.044, 41.973, 42.091, 42.207, 42.135, 42.101, 42.077, 42.226, 42.138, 42.469, 42.315, 42.496, 42.363, 42.473, 42.25, 42.424, 42.306, 42.322, 42.385, 42.445, 42.339, 42.231, 42.359, 42.467, 42.464, 42.344, 42.489, 42.625, 42.402, 42.479, 42.32, 42.634, 42.459, 42.728, 42.387, 42.634, 42.646, 42.719, 42.553, 42.737, 42.524, 42.385, 42.571, 42.364, 42.626, 42.62, 42.662, 42.822, 42.675, 42.79, 42.811, 42.817, 42.575, 42.624, 42.666, 42.775, 42.802, 42.701, 42.902, 42.802, 42.898, 42.745, 42.724, 42.685, 42.764, 42.67, 42.877, 42.748, 42.871, 42.955, 42.823, 42.851, 42.763, 42.836, 42.627, 42.83, 42.993, 42.705, 42.99, 42.722, 42.805, 42.912, 42.626, 42.741, 42.692, 42.869, 42.848, 42.759, 42.755, 42.967, 42.949, 42.906, 42.887, 43.021, 42.877, 42.952, 42.954, 42.945, 42.947, 43.022, 43.079, 42.987, 43.054, 43.087, 42.997, 43.058, 42.84, 43.023, 43.079, 43.024, 43.001, 43.129, 43.052, 43.27, 43.011, 43.121, 43.139, 43.126, 43.023, 43.088, 43.067, 43.205, 43.188, 43.108, 43.183, 42.991, 43.078, 43.022, 43.424, 42.999, 43.02, 43.294, 42.994, 43.219, 42.974, 43.014, 43.126, 43.149, 43.074, 42.953, 43.268, 43.3, 43.177, 43.111, 43.244, 43.068, 43.126], 'train_loss': [4.393958336866138, 3.829054258911562, 3.6148976105112185, 3.473567121698547, 3.353025897069383, 3.268283985550405, 3.1956865602933817, 3.1408971231561855, 3.088866441431369, 3.0455173677309, 3.003145507262139, 2.966141080520737, 2.934231475615303, 2.8994063078708856, 2.8726565345158885, 2.8511144108140765, 2.8234146382666823, 2.802397354245567, 2.7847435407278556, 2.7633758372247637, 2.7447954079926355, 2.7305353344516425, 2.7115922215193873, 2.7017267024906033, 2.685523310763212, 2.675423589793063, 2.662241086468663, 2.649083318042206, 2.6373726170717413, 2.628377198944165, 2.6233455576503117, 2.611204687403473, 2.6035003216695265, 2.5979455898605854, 2.5849929698674403, 2.579440429816243, 2.5772555430425106, 2.567136501930344, 2.5618365157970007, 2.556728086407491, 2.5532705492494356, 2.5473224090904436, 2.544463685744097, 2.536251993112204, 2.5334565845423604, 2.5274209208924727, 2.528914264967559, 2.5217863112325785, 2.5174728930187165, 2.5075005370687387, 2.5091253847017247, 2.504842747882323, 2.5045927502143406, 2.4925452252648537, 2.4947684056012966, 2.4949103252901454, 2.4883552736299435, 2.4832948197635107, 2.483889713015834, 2.48261556362999, 2.474978905950535, 2.479819001483368, 2.4713012340048985, 2.46821678005116, 2.4651203657371146, 2.4643416641922387, 2.459607627249954, 2.4553374883385706, 2.455350710578401, 2.452458496819836, 2.4502141877617962, 2.4467190086117023, 2.4452469043829312, 2.4427003325824166, 2.4430606976122866, 2.4374120197308344, 2.4394782595503277, 2.4366192725401845, 2.4315935475507455, 2.4253363076189887, 2.4254065221193426, 2.4264634168842094, 2.426402287992696, 2.424534986969453, 2.419494432664726, 2.4188216718739604, 2.421865971745853, 2.4163084403109414, 2.416459948072354, 2.4129530703190114, 2.4145863652610657, 2.4117171255655956, 2.4149350675343704, 2.407503445714388, 2.4066401016841845, 2.404652502089834, 2.4060625535741647, 2.40055882701947, 2.3992184141242037, 2.3988827284680525, 2.3952372913854822, 2.3999633403138634, 2.394943330353525, 2.3931195149418643, 2.3948821963519524, 2.392542300663617, 2.39133761406555, 2.3852470269663852, 2.3858417991789502, 2.384703993949841, 2.383527739491893, 2.3829624426342018, 2.379261172359293, 2.3835923426134498, 2.3794124004403066, 2.3735295263377045, 2.3782332406468063, 2.37649371405862, 2.3713907535001395, 2.3728101071988017, 2.3742269864085386, 2.3729467947972713, 2.3690171144135244, 2.3697250201316193, 2.367403243645139, 2.3647260975547884, 2.3618854098188824, 2.3653551142565044, 2.3636138629852312, 2.364856881357048, 2.3633885306573723, 2.360726834945166, 2.362635859982447, 2.361222354403232, 2.3562290946871367, 2.361644782931547, 2.355865572174619, 2.3554327129478723, 2.354149342422217, 2.3531421196437843, 2.3503843150837485, 2.353947559077238, 2.347350734666762, 2.3512107467163244, 2.354024359681098, 2.3521433681993247, 2.3506394009214904, 2.345854255837351, 2.3434403158500268, 2.348102083971923, 2.3440571834243267, 2.3448468859349774, 2.3415920117384945, 2.3391650515493487, 2.3410595094860174, 2.3399475280550606, 2.3377398177743034, 2.340038151597641, 2.339353888063803, 2.337738165165931, 2.334801139926117, 2.337707914683732, 2.3315521583142225, 2.3337480770191625, 2.3337507623933633, 2.3359147848719903, 2.332687571425508, 2.3317461630814518, 2.328772550962403, 2.3320529917761523, 2.331994192232631, 2.3333529826546813, 2.3280564424553822, 2.3321845456719474, 2.3317025796923208, 2.3301716976568474, 2.3282760112467136, 2.32684889414794, 2.331961040304627, 2.3281138516814757, 2.324238566656717, 2.326975931437902, 2.3264249110938797, 2.3240733888770095, 2.32289425219318, 2.3175057871404245, 2.3223604220880274, 2.3262339012026407, 2.3218309959385994, 2.322076473568619, 2.3180889794251436, 2.3189929966276757, 2.3213611028168497, 2.3179370931380086, 2.3244717264511, 2.317905680460573, 2.3217800882330937, 2.315627394772003, 2.3182791021491043, 2.31462990970697, 2.3161097428620203, 2.3163831921929514, 2.3176319252887905, 2.315150500182837, 2.3138718400639138, 2.3180767503672506, 2.3177948866756926, 2.315063430343159, 2.3155918025238273, 2.3153934432998042, 2.3179545700740753, 2.313836805727416, 2.3145498051066773, 2.3145182038909415, 2.3121889009738075, 2.31251089586635, 2.311456094318983, 2.312020339267191, 2.3080782381602, 2.3109101650429626, 2.315385771728218, 2.3072474919597994, 2.306563488123742, 2.3132842347473956, 2.308951513475893, 2.306809837667132, 2.3115928109761468, 2.309796544655881, 2.309811934094664, 2.3061863132111933, 2.308666680306101, 2.3088326997201714, 2.304947979543275, 2.31021218519522, 2.3087817227619243, 2.3099812301244023, 2.3049076161015436, 2.3110759170560295, 2.305044700377886, 2.306528355323269, 2.307079700728067, 2.304664254798694, 2.3030115705381506, 2.3040406848098405, 2.3036053069341054, 2.3084122450475273, 2.3044589111916314, 2.307845295810272, 2.3050933449373594, 2.3030718214909998, 2.302826842587496, 2.304471290424247, 2.302690384255261, 2.2972280023499168, 2.295161472629906, 2.2996393616811175, 2.2998125797956326, 2.3038181206086317, 2.2985353578914074, 2.3005173676149706, 2.302237260669603, 2.300380384700846, 2.2969074524600614, 2.3013364881258016, 2.2981083730208285, 2.2986533469446027, 2.2944043653558936, 2.295677806350259, 2.3025855947135736, 2.2989525775915527, 2.2971811969541083, 2.296340683371458, 2.297225435803658, 2.3001434533014864, 2.2982126128116787, 2.298214693444704, 2.2997224301919674, 2.293932127098357, 2.298053128781871, 2.29516368589566, 2.299939804861199, 2.2896499305830043, 2.295471516695834, 2.294433619529104, 2.2923121702495632, 2.2958969774722138, 2.2935824417106936, 2.292399295384657, 2.295664094383718, 2.292464614219568, 2.2928603773153675, 2.29528472039155, 2.2977769111717503, 2.2914307687195614, 2.288924068345981, 2.2967757330593663, 2.294957031901647, 2.293158567111918, 2.2911453653586955, 2.2942592198468903], 'val_metric': [7.32, 14.99, 16.34, 19.04, 20.17, 22.73, 24.35, 25.54, 25.81, 25.6, 23.44, 26.71, 29.14, 27.28, 28.31, 30.21, 30.69, 27.71, 28.2, 31.48, 29.54, 29.31, 28.99, 33.3, 29.48, 29.84, 30.98, 32.33, 30.93, 29.75, 33.17, 33.09, 34.75, 33.59, 31.86, 34.7, 33.25, 32.99, 31.5, 31.16, 31.14, 34.49, 35.11, 33.8, 35.59, 32.39, 34.16, 35.69, 35.06, 31.86, 35.67, 34.65, 34.89, 34.85, 34.01, 34.05, 33.99, 34.41, 32.45, 32.35, 36.24, 34.21, 34.82, 35.48, 34.71, 35.45, 36.25, 34.55, 34.53, 35.96, 35.41, 34.46, 35.37, 36.01, 33.49, 36.07, 34.57, 28.09, 35.4, 34.25, 36.27, 34.54, 37.67, 35.01, 37.23, 36.39, 36.85, 35.38, 34.17, 34.48, 30.09, 36.15, 36.33, 34.58, 35.22, 32.21, 36.07, 33.96, 35.74, 33.67, 35.79, 36.62, 34.23, 37.06, 35.82, 33.52, 37.39, 37.69, 33.31, 34.91, 32.96, 36.02, 37.65, 35.89, 32.28, 36.23, 37.02, 35.29, 35.41, 36.55, 35.63, 35.68, 34.43, 33.15, 36.66, 39.05, 35.67, 34.95, 37.04, 35.72, 36.19, 35.23, 33.94, 36.71, 36.86, 35.32, 36.18, 37.21, 37.32, 37.79, 32.76, 36.46, 36.73, 38.27, 34.59, 36.76, 37.35, 33.7, 36.54, 33.04, 38.4, 34.63, 33.09, 34.86, 37.13, 34.13, 37.06, 37.94, 35.63, 38.18, 35.48, 34.54, 34.75, 36.12, 36.54, 36.41, 36.7, 32.76, 37.12, 38.54, 37.31, 35.79, 36.56, 37.58, 37.9, 36.23, 37.9, 35.03, 35.32, 36.72, 36.37, 36.87, 35.77, 35.64, 34.5, 37.5, 37.44, 38.03, 36.94, 36.92, 37.98, 35.97, 33.29, 37.8, 38.02, 35.84, 34.99, 37.66, 38.51, 35.28, 36.86, 37.31, 37.75, 36.08, 37.24, 34.72, 34.04, 34.61, 35.83, 38.09, 36.72, 35.9, 36.24, 37.02, 37.48, 36.35, 38.11, 38.27, 37.2, 37.75, 36.03, 38.96, 33.89, 34.84, 32.82, 36.17, 37.97, 36.13, 37.34, 35.15, 37.37, 36.32, 38.59, 38.38, 38.12, 38.12, 35.99, 37.78, 37.53, 35.74, 36.05, 38.88, 38.48, 37.24, 34.65, 37.6, 34.26, 35.87, 39.4, 36.66, 31.25, 36.38, 37.96, 37.45, 34.87, 38.24, 35.95, 36.58, 37.23, 37.32, 37.88, 37.5, 35.63, 36.92, 38.14, 35.86, 35.19, 36.2, 37.48, 33.62, 35.18, 36.94, 37.97, 34.48, 37.54, 34.41, 37.07, 37.18, 37.08, 35.11, 36.09, 37.37, 36.96, 36.59, 35.46, 35.37, 37.98, 37.92, 38.06, 38.32, 37.68, 33.22, 36.12, 37.95, 36.76, 36.42, 36.69, 36.23, 34.38, 34.55], 'val_loss': [4.375599451125807, 3.8159682978490355, 3.668288854277058, 3.519216803228779, 3.4638333563592023, 3.309978152536283, 3.2438190628768533, 3.1782932729478093, 3.1656306761844903, 3.1317398517754427, 3.3210032867018584, 3.099033642726339, 2.971476093219344, 3.0344599743557583, 3.0018376704234226, 2.941128986656286, 2.9097159569430504, 3.082187560713215, 3.025526299598111, 2.876434158367716, 2.985526025674905, 2.9771664681707977, 3.0437630247917906, 2.7911738696371673, 2.966363338907813, 2.9765613018327457, 2.9028981977207646, 2.8322241382234417, 2.90436027687826, 3.0028352638718427, 2.8103043015595452, 2.799484640170055, 2.7091105887844305, 2.743499156016453, 2.852842664263051, 2.728496105048307, 2.779433223851927, 2.7932704163205093, 2.890774974397793, 2.9213376774150097, 2.875887087955596, 2.7188246432383347, 2.749814476936486, 2.7687950961908716, 2.6800563532835358, 2.8025958295081073, 2.683446392891513, 2.659199912077302, 2.715966586854048, 2.8597857921746126, 2.694761314209859, 2.735599422910411, 2.7287697503521184, 2.7171304195549837, 2.776637172243398, 2.7572260950780976, 2.8015755764238395, 2.732745880533935, 2.8380337915602762, 2.877526667467348, 2.627628593308151, 2.7727579075819366, 2.7074499221364405, 2.6977549259829674, 2.720215582543877, 2.7224478960796525, 2.664104222492048, 2.7521070735469744, 2.7194449711757103, 2.645448206336635, 2.697966501970959, 2.7249063450819366, 2.7130387421626194, 2.664644905715991, 2.800490897552223, 2.6522668015425372, 2.769217566320091, 3.1352194737476906, 2.6953599612424326, 2.7705819333434865, 2.6303006463749394, 2.731167646730022, 2.6088829404988867, 2.7321060880733903, 2.6103455731823186, 2.6484615240886713, 2.615712496885069, 2.6952999473377397, 2.790101507666764, 2.7255277679224683, 2.992596089460288, 2.6629966296208134, 2.654023723997128, 2.7267107887632527, 2.719896238320952, 2.8795522079346285, 2.683595045736641, 2.7775617276027704, 2.7071865918530023, 2.7834228444251283, 2.685803124859075, 2.6147289530486817, 2.809105021179102, 2.596544947973482, 2.6608355880542924, 2.763117373369302, 2.567535277384861, 2.6368420162018698, 2.839662991511594, 2.7701597821180988, 2.8676166382564863, 2.67659071638326, 2.5955144083423978, 2.6855170772333814, 2.8860905436193867, 2.688526347184637, 2.6334231097227447, 2.7035071249980076, 2.701429585742343, 2.6753239308952526, 2.692132981719485, 2.7011117733967533, 2.844265650791727, 2.8202972981580503, 2.6270049666143525, 2.5495750562400574, 2.673539778229537, 2.752650781042257, 2.6297766687763726, 2.679593354653401, 2.6622426813575113, 2.7144176010872907, 2.8051084655864984, 2.6308295244623903, 2.6285632833553727, 2.730182336014547, 2.702279174783427, 2.632623632242725, 2.6138096204988517, 2.574003203659301, 2.859536974293411, 2.6375262786628335, 2.6241000716094, 2.5552104597638365, 2.8348000717770523, 2.6612350993854985, 2.605441079018222, 2.7766582836770706, 2.6709052969695657, 2.817603332221888, 2.560498178384866, 2.7454917886454586, 2.8442405743204104, 2.780107138263192, 2.6349207777885875, 2.7533367443236574, 2.5836869948988506, 2.5948949255001774, 2.706738302282467, 2.566609602065603, 2.7560843441896377, 2.749916533755649, 2.7297715654798376, 2.7034209465524954, 2.6984239866019815, 2.6518165723533387, 2.6249052597458955, 2.8743809169265115, 2.629950563239444, 2.560566370654258, 2.615783455645203, 2.6540088038535634, 2.668079783202736, 2.5834286979808927, 2.5892922475839115, 2.6481065218615685, 2.5767315975420035, 2.7347859804797325, 2.721594571307966, 2.6452785684804248, 2.6738868853089155, 2.6268648619105104, 2.723382137383625, 2.713197800763853, 2.793870572071926, 2.5899409412578414, 2.586292345812366, 2.5454180377304176, 2.6381712111697833, 2.6297917684931664, 2.550214883628165, 2.6706719284604308, 2.8531347050029003, 2.570059127109066, 2.601195216178894, 2.7240258796959167, 2.704481882654178, 2.601452112957171, 2.547849380666283, 2.7185854615679212, 2.625337783698064, 2.6122328711163467, 2.592969498057274, 2.643185366490844, 2.6243064039072412, 2.732492723282735, 2.8079266718998075, 2.7969375727282966, 2.6966632247730424, 2.5860197749107505, 2.638860637974587, 2.6816415031244802, 2.6383388687850564, 2.633909381878604, 2.5821681007458146, 2.652548830220654, 2.5736668398425837, 2.556470035747358, 2.6136335795092736, 2.5744225633372166, 2.6765071814227257, 2.5377404488575688, 2.7938673192528403, 2.787924758188284, 2.875125156846016, 2.6508427668528953, 2.5395893390011635, 2.6952291298064455, 2.615499563657554, 2.777888952167171, 2.6092152853680264, 2.6495068001139694, 2.5243096947669983, 2.586168848025571, 2.557827010655859, 2.53616891734919, 2.6911190915259584, 2.6147034570669674, 2.610801911657783, 2.7014037681992646, 2.6822689035136227, 2.5459168166111987, 2.535955313664333, 2.6204361330931354, 2.7928502244554507, 2.6183563235459055, 2.813993296425813, 2.673152067858702, 2.5472738332809155, 2.632250828348148, 2.8990202762518718, 2.701664836543381, 2.5591987496728352, 2.6104524401342792, 2.7096132533565447, 2.5786071567778377, 2.6564017754451483, 2.64628550732971, 2.6315620309987646, 2.6023303972687692, 2.579818189523782, 2.6089160279103907, 2.705069219990141, 2.644290135164929, 2.5550752832631396, 2.6780330728573403, 2.7611587715756363, 2.6759274651290506, 2.6151995381731896, 2.8396246668639455, 2.726836815761153, 2.669091220114641, 2.5748232462603573, 2.760180248576365, 2.6150753809388276, 2.761133255092961, 2.6573615909382036, 2.599350938371792, 2.6233576585532754, 2.6955299278733076, 2.7928346934591888, 2.635211523930738, 2.621745689659362, 2.670782973432237, 2.713571409510959, 2.7352688616248453, 2.5949227210063084, 2.606583985173778, 2.564390659711923, 2.568140474474354, 2.5981412514759477, 2.839187019949506, 2.657803230224901, 2.561142944226599, 2.646588472044392, 2.640541115384193, 2.6255253412920956, 2.6663396210427495, 2.7865995703988773, 2.718093880422556]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal 2", optimizer="AdamW")
data.update({'train_metric': [9.821, 22.138, 29.744, 35.387, 39.967, 43.459, 47.164, 50.431, 53.63, 56.549, 59.528, 62.415, 65.041, 67.523, 70.161, 72.55, 74.539, 76.26, 78.364, 79.597, 80.898, 82.138, 82.764, 83.94, 84.604, 85.278, 85.881, 86.69, 86.983, 87.699, 87.918, 88.251, 88.787, 88.967, 89.282, 89.741, 89.845, 90.206, 90.39, 90.695, 90.837, 90.85, 91.313, 91.315, 91.432, 91.699, 91.803, 91.884, 92.195, 92.25, 92.156, 92.585, 92.549, 92.708, 92.602, 92.935, 93.022, 92.996, 93.278, 93.14, 93.381, 93.436, 93.365, 93.464, 93.785, 93.712, 93.795, 93.964, 93.877, 93.91, 93.977, 94.126, 94.199, 94.177, 94.284, 94.37, 94.443, 94.377, 94.5, 94.507, 94.718, 94.666, 94.648, 94.854, 94.643, 94.666, 94.883, 94.888, 94.889, 94.991, 94.89, 95.009, 95.041, 95.03, 95.299, 95.19, 95.105, 95.256, 95.339, 95.244, 95.316, 95.359, 95.313, 95.444, 95.567, 95.4, 95.4, 95.541, 95.652, 95.566, 95.424, 95.711, 95.564, 95.759, 95.656, 95.799, 95.714, 95.702, 95.933, 95.703, 95.849, 95.873, 95.784, 96.013, 95.741, 95.968, 96.0, 95.955, 95.968, 95.959, 96.042, 96.015, 96.128, 95.913, 96.288, 96.05, 96.095, 96.093, 96.195, 96.13, 96.384, 96.175, 96.335, 96.18, 96.314, 96.383, 96.301, 96.196, 96.242, 96.335, 96.343, 96.304, 96.476, 96.344, 96.482, 96.455, 96.201, 96.427, 96.512, 96.364, 96.53, 96.68, 96.505, 96.472, 96.534, 96.513, 96.575, 96.581, 96.544, 96.665, 96.523, 96.533, 96.543, 96.497, 96.653, 96.65, 96.653, 96.682, 96.657, 96.718, 96.746, 96.741, 96.592, 96.876, 96.678, 96.906, 96.722, 96.684, 96.688, 96.756, 96.808, 96.832, 96.769, 96.899, 96.848, 96.849, 96.753, 96.924, 96.903, 96.883, 96.8, 96.915, 96.953, 96.971, 96.934, 96.918, 96.94, 96.947, 96.983, 97.038, 96.924, 96.901, 96.918, 96.992, 97.047, 96.861, 96.973, 96.993, 96.856, 97.039, 96.976, 97.078, 96.965, 97.072, 97.038, 96.966, 97.11, 97.061, 97.129, 97.041, 97.083, 97.044, 97.284, 97.032, 97.068, 97.112, 97.126, 96.998, 97.138, 97.096, 97.121, 97.11, 97.073, 97.053, 97.335, 97.244, 97.069, 97.151, 97.186, 97.3, 97.138, 97.264, 97.224, 97.337, 97.159, 97.196, 97.221, 97.232, 97.233, 97.222, 97.353, 97.313, 97.197, 97.303, 97.24, 97.291, 97.313, 97.267, 97.338, 97.233, 97.322, 97.328, 97.366, 97.19, 97.313, 97.444, 97.363, 97.355, 97.373, 97.4, 97.392, 97.364, 97.337, 97.35, 97.462, 97.409, 97.309, 97.484, 97.373, 97.358, 97.383, 97.488, 97.247, 97.466, 97.3, 97.356, 97.39, 97.447, 97.435, 97.394], 'train_loss': [4.269843841773611, 3.404150922406734, 2.9922124259531384, 2.700997492630971, 2.4769091081405707, 2.282890162708968, 2.114132943858112, 1.954394603721316, 1.8003759928719782, 1.6579712726943248, 1.5182786705550366, 1.3834121285229255, 1.2631964266338336, 1.1510959065120645, 1.0426313474783895, 0.9419261161440546, 0.860488515001646, 0.7920831977291437, 0.7192340596738108, 0.6663204968948053, 0.6170654266843869, 0.5751101132089026, 0.5491066066809213, 0.510270456987852, 0.48273142652677864, 0.45942440140403695, 0.44467475690943875, 0.42000632834163754, 0.4070894333962363, 0.38731640376155374, 0.38047713349958834, 0.36910904851942894, 0.35204365213597655, 0.34329487866008807, 0.3346839407798539, 0.3233731802426617, 0.3183935889908692, 0.31001295632446185, 0.3015329974631392, 0.2979655755839894, 0.29319419922761175, 0.2861854810713387, 0.275453523273317, 0.2710295955263798, 0.27213931921452417, 0.2637130977444098, 0.25558496862220903, 0.2565941687265765, 0.24617846478646715, 0.24450282293526182, 0.2466021159393278, 0.23485578764027423, 0.23276303317433583, 0.2315540063812833, 0.23602168475202048, 0.22200984072824434, 0.22167226083169114, 0.22185813488590073, 0.21416079252719955, 0.2186191170497232, 0.20928588697015857, 0.2099480737913555, 0.21059852136478963, 0.2044527574161679, 0.20211884082412385, 0.2007661677172416, 0.19959219737110692, 0.19223134762709407, 0.19675443506479665, 0.19370341059851318, 0.19111548432507103, 0.18877079960027912, 0.18443112479378626, 0.18710050332435643, 0.18435762649919224, 0.1815010718113468, 0.17862936179377067, 0.18038176379976312, 0.17983089288743123, 0.1756431291291582, 0.16966108924763484, 0.17021951203753485, 0.172284296587448, 0.16430513330563065, 0.17088885027682857, 0.17037252750249504, 0.16592903313020235, 0.1636841833183002, 0.16278718740894427, 0.1593813446010162, 0.1650291479321379, 0.15828335928323697, 0.16088553206066786, 0.16152845598162602, 0.1520424086613353, 0.1567416605146355, 0.1559789332436661, 0.15294256763174052, 0.15348846154029452, 0.15566937980776557, 0.15451789848322928, 0.15164999520228561, 0.15310896400532667, 0.14749901368722476, 0.14331637664477, 0.14960321379291883, 0.15251774957459552, 0.14257575206299772, 0.1413772325540916, 0.14474411522469086, 0.15006699915337768, 0.1411281369490369, 0.1447019743221715, 0.13808061032343277, 0.1429366112090242, 0.138257603987191, 0.140256682320862, 0.1404310840596722, 0.13293299205009532, 0.14097444181359103, 0.137147428915134, 0.13405404981703606, 0.13688591856572607, 0.13206569795462203, 0.14078806713774625, 0.133103831753183, 0.13119980699451395, 0.13277532945932868, 0.13138131457407765, 0.13392702688228264, 0.1300571542380703, 0.1317874604110056, 0.12801311101423463, 0.13506086553927804, 0.12228214914311064, 0.12995742884734485, 0.12825011129813388, 0.1295079237753013, 0.12288022232128716, 0.12707058834856447, 0.11983089848592525, 0.12706897177360177, 0.12476205162703842, 0.1299118602047159, 0.12162484612662383, 0.11957248929798334, 0.1209980239936759, 0.12411579068855014, 0.12409286759454866, 0.12216106900623246, 0.1218415618550434, 0.12430844439705446, 0.11532815057545678, 0.11963022012583512, 0.1142615111463096, 0.11689507558757721, 0.12803249848300147, 0.12150904588097879, 0.11678708909658604, 0.12289579484651374, 0.11659695708181862, 0.11061316081656408, 0.11728118670207847, 0.11752456384031693, 0.11541141247167058, 0.11513080067819542, 0.11534029416871326, 0.11502717107319006, 0.11825145342758477, 0.11226353200700108, 0.11656128721463929, 0.1147833830279686, 0.11375217709247314, 0.1171871120053101, 0.11140506197234391, 0.11204315645488244, 0.11177452057984348, 0.11038629123860623, 0.11220761636297098, 0.11210893723703753, 0.10743590797735908, 0.11245662081631834, 0.11185765141441432, 0.10543988132149205, 0.10979684033539558, 0.10400876781212164, 0.10867690252146421, 0.11181308567857337, 0.11018000499047471, 0.11142385170906151, 0.1079187239207055, 0.10742496618531347, 0.10792497780125007, 0.10435612266447535, 0.10717841685840905, 0.10516063865066833, 0.10752385395141235, 0.10254000832019249, 0.10467643557776002, 0.10341608130338947, 0.10960044625295078, 0.10527537091131223, 0.1020615807161589, 0.10303891599819603, 0.10180103246002534, 0.1049413843466516, 0.10443321808933767, 0.10325102694086809, 0.10207032464755673, 0.10014015726931483, 0.10420605308223267, 0.10537253929989863, 0.10439273032321197, 0.10374384039622447, 0.09871718018087147, 0.10551749703251492, 0.102285933172195, 0.10127897479142982, 0.107519177320988, 0.10037312765492684, 0.102350292029902, 0.09732516712997846, 0.10393994092136581, 0.098751357383028, 0.09839190919006896, 0.10401970357459095, 0.09778108410675779, 0.09863062136097778, 0.0977383646917406, 0.10074965393388798, 0.10004398533938869, 0.10120093047961241, 0.09126388333903647, 0.10116747711035551, 0.1002356783704837, 0.09876904751919567, 0.09718387736021834, 0.10207040514484148, 0.09596412404504696, 0.10085906285625154, 0.09709767478892684, 0.09689513302048933, 0.09895084077490196, 0.1000210454121078, 0.08913407216922, 0.09469975075900933, 0.09790240817874094, 0.09809561914338903, 0.09487809605671686, 0.0928015591716647, 0.09754906578404472, 0.09207438307951198, 0.09404107594666655, 0.09024269286288514, 0.09797894523937258, 0.0935573406568228, 0.0963082129444977, 0.0956866391567072, 0.09283595484957309, 0.09313990967780214, 0.08943008459421913, 0.09168426389336737, 0.09777719913888008, 0.09425816380143849, 0.09446436358561831, 0.09169635341382959, 0.09177049359396294, 0.09293291707571133, 0.09341717732660454, 0.09490017416550402, 0.09554854157743933, 0.09268517303371773, 0.08965360725603007, 0.09909788338679827, 0.09400016808923246, 0.0869355183111126, 0.09012897106568149, 0.09303136344918858, 0.09041060782024268, 0.09094248728419077, 0.08964005167941565, 0.08991591013629545, 0.09041914557135394, 0.0911420460302576, 0.0880508539789096, 0.0900376053422518, 0.09183535344818224, 0.08788582555818898, 0.08929377591942919, 0.09165980846120428, 0.08956847997164498, 0.08710418126066331, 0.09321630926326926, 0.08671916854699362, 0.09402715889612066, 0.09001279233375228, 0.08812904489984848, 0.08847428970846648, 0.08681692619971804, 0.09061145200461128], 'val_metric': [15.4, 23.63, 30.71, 33.08, 36.03, 39.3, 40.07, 41.07, 40.96, 42.41, 41.3, 42.02, 42.07, 41.35, 40.57, 41.73, 40.66, 40.52, 40.16, 39.81, 40.07, 40.83, 40.08, 40.69, 39.62, 40.84, 39.31, 40.81, 40.18, 40.41, 40.43, 40.39, 39.96, 40.72, 40.05, 40.61, 40.68, 40.43, 39.93, 40.77, 40.45, 40.15, 40.16, 39.87, 40.85, 39.54, 39.93, 40.43, 39.87, 40.42, 39.02, 40.23, 40.19, 39.74, 39.91, 40.6, 40.44, 41.1, 39.69, 41.68, 40.33, 40.01, 40.55, 40.45, 40.58, 40.02, 40.56, 40.87, 40.28, 40.09, 40.32, 41.06, 40.12, 41.14, 41.06, 41.73, 40.65, 41.47, 40.69, 41.39, 40.81, 40.57, 41.06, 40.38, 41.58, 41.19, 41.08, 40.82, 41.21, 41.68, 40.79, 41.38, 41.38, 41.27, 41.34, 41.83, 41.43, 40.77, 41.05, 40.52, 41.2, 41.38, 41.4, 41.27, 41.33, 41.14, 40.86, 41.21, 41.21, 41.24, 41.37, 40.92, 41.19, 41.43, 41.51, 41.31, 41.75, 41.02, 41.54, 41.71, 40.92, 41.78, 41.0, 41.15, 41.85, 41.58, 42.39, 41.28, 42.03, 40.44, 41.78, 41.36, 41.53, 41.59, 41.84, 41.55, 41.2, 41.56, 41.08, 41.99, 41.85, 41.91, 41.89, 41.51, 41.94, 41.47, 41.68, 40.81, 41.21, 41.25, 41.57, 41.91, 41.78, 41.46, 41.27, 41.51, 41.28, 41.41, 41.11, 41.93, 41.66, 41.95, 40.99, 41.66, 41.32, 41.65, 41.38, 41.37, 41.85, 41.68, 41.68, 42.39, 41.85, 42.15, 42.61, 42.16, 41.96, 41.41, 42.07, 42.38, 42.49, 42.01, 42.01, 41.96, 41.81, 41.66, 42.0, 41.5, 42.11, 42.25, 41.52, 41.52, 42.09, 41.2, 41.56, 41.42, 41.75, 41.94, 41.28, 41.91, 41.91, 41.77, 41.64, 41.69, 42.91, 42.23, 42.37, 41.73, 42.06, 42.44, 42.2, 42.08, 42.43, 41.68, 41.66, 42.27, 42.04, 42.35, 42.53, 40.85, 41.87, 42.03, 41.96, 42.18, 41.68, 41.08, 42.46, 41.97, 41.57, 42.36, 41.76, 42.51, 42.15, 41.69, 41.86, 42.7, 42.06, 41.76, 43.06, 41.18, 41.84, 42.24, 42.28, 42.02, 41.64, 42.32, 42.0, 42.05, 41.81, 41.52, 42.22, 41.45, 42.24, 41.89, 42.04, 42.05, 41.77, 42.29, 41.75, 41.76, 41.81, 41.33, 41.62, 41.82, 41.53, 41.5, 42.21, 42.13, 42.28, 42.66, 42.05, 41.83, 41.55, 43.05, 41.36, 42.74, 42.77, 42.14, 42.25, 42.59, 41.96, 42.54, 42.62, 41.81, 41.83, 42.17, 42.29, 42.82, 42.34, 42.95, 42.05, 42.16, 42.37, 41.98, 42.29, 42.97, 42.41, 42.45, 42.36, 41.68], 'val_loss': [3.8721572274615053, 3.375832841654492, 2.9770894437838513, 2.915811594884107, 2.7143264792527364, 2.6108918265931926, 2.559602303489758, 2.5083156706421237, 2.5889476184632367, 2.541367713053515, 2.6394881742775063, 2.743997619410229, 2.7759060070013546, 2.9167043638836807, 3.052982886126087, 3.1276902393170984, 3.2919600716062414, 3.4380661294718458, 3.5301954343820072, 3.6945880438871446, 3.744025702689104, 3.8498612429685655, 3.9592424585561083, 3.975845255669515, 4.143317991001591, 4.16988887528705, 4.371112503063906, 4.382954254271878, 4.402052640914917, 4.458909013468748, 4.588950421400131, 4.573921033531238, 4.52616822719574, 4.598896607471879, 4.723354281893202, 4.719194498791057, 4.852660903505459, 4.821957062004478, 4.768755584006097, 4.8552791859693585, 4.906934891536737, 5.029855395578275, 5.089245753683103, 5.064502609003881, 5.050911819858915, 5.004106030342685, 5.104540857539814, 5.139710508334409, 5.165777753113182, 5.2505405495880515, 5.3984538111717075, 5.2154340941435215, 5.337484324813649, 5.366070276612689, 5.452837784578846, 5.40427141660338, 5.437132303881797, 5.324277738856662, 5.462373829950952, 5.343123217297208, 5.567581166127685, 5.500042702741684, 5.644301321855776, 5.506444204385113, 5.588034295732049, 5.716868061928233, 5.550456485930522, 5.618777952376445, 5.6769922766715855, 5.687633550850449, 5.785400475666022, 5.6676974395278155, 5.705215009914082, 5.592614854217335, 5.694393275649684, 5.7495556295297705, 5.862618265637926, 5.594822068882596, 5.7491455078125, 5.679601033781744, 5.8833591042050895, 5.975010941742332, 5.880015156071657, 6.036814824790712, 5.959366995817537, 5.885815618144479, 5.72326252566781, 5.952176463831762, 5.901434936341206, 5.884383975320561, 6.105553174474437, 5.968838559594124, 5.953779096056701, 6.067279897677671, 6.004405102152734, 5.9563448793569185, 6.085740560179303, 6.041196121531687, 6.009247344010954, 6.093116960707744, 6.0671729130350105, 6.062352573795683, 6.108758994728137, 6.103042564574321, 6.287568936682051, 6.257558736072224, 6.037910693770002, 6.075922001698974, 6.115953741559557, 6.32134972104601, 6.235129276658319, 6.109293715969013, 6.185769392426606, 6.164091875598689, 6.156374749863983, 6.124270023054378, 6.242107073972179, 6.228525056960477, 6.218155379507952, 6.243959498253598, 6.397947571080202, 6.313439396536275, 6.377869631834091, 6.420728397217526, 6.3939033010203365, 6.229993937121835, 6.362142623609798, 6.392771124080488, 6.3312924454925925, 6.401627618036453, 6.567365679771277, 6.469745751399143, 6.440962568969484, 6.618621719870598, 6.38215254370574, 6.321942213993923, 6.417644145382438, 6.561884204293512, 6.55153657828167, 6.425405816667399, 6.432004954404892, 6.470372742148721, 6.389641576511845, 6.356612192597359, 6.329806727208909, 6.506527682018888, 6.45904430841944, 6.619332760002963, 6.546176963551029, 6.636930723858487, 6.714777859912556, 6.616923238061796, 6.739746461248702, 6.628447798407002, 6.814878644457289, 6.575499757839616, 6.596272401748949, 6.553483422394771, 6.516410718298262, 6.500774852789132, 6.780760652700048, 6.560368199257335, 6.72197722933095, 6.493288867792506, 6.84809056968446, 6.687096574503904, 6.615038521730217, 6.638568653422556, 6.677512670018871, 6.5968388918858425, 6.569817890786821, 6.741883898996244, 6.587886702482868, 6.905181689626851, 6.689687060702378, 6.909820857321381, 6.854010266103562, 6.689367235845821, 6.8051720166661935, 6.571891301756452, 6.796935418608841, 6.720900148343128, 6.737267807031134, 6.64039425485453, 6.758490361985127, 7.045361484691596, 6.7938832431841805, 6.87899337604547, 6.773508929902581, 6.746400409443363, 6.9620424136993995, 6.898137647634859, 7.004086300066322, 6.803973623142121, 6.84567408045386, 6.984566579199141, 6.881118892864057, 6.859110411565015, 6.779804463599138, 6.733348310373391, 6.80720858209452, 6.815638015224676, 6.938493478070399, 6.907953031503471, 7.101564430127478, 6.774397001144992, 6.855655741539731, 6.894940500806092, 7.007079074337224, 7.149007141210471, 6.78693793078137, 6.8595879085504325, 6.983047442831052, 6.8455180089185195, 7.058279267541922, 6.903057658748263, 7.032450880973962, 6.734731763791126, 6.882581092749431, 7.069040137491409, 7.24652080930722, 7.018037764130125, 7.0291399378685435, 6.913272496241673, 6.951313727980207, 6.974078775211504, 6.867573537644307, 7.250776132960229, 6.996387006370885, 7.091417643674619, 7.138008837487288, 7.017260261402009, 6.966355606249183, 7.0334095165228385, 6.993871793625461, 7.07150130211168, 7.117111878789914, 7.003145080463142, 7.234545770724108, 7.247823990074692, 7.1359352032849745, 7.329085699312246, 7.1045150969438495, 7.166638225506825, 7.4028968613618495, 7.442164068768738, 7.085421923619167, 6.8811907753063615, 7.21300501276733, 7.0180547161466755, 7.007353358967289, 7.196072001366099, 7.2145961803995124, 7.343390622716041, 7.28417865637761, 7.205181527289615, 7.086257398508157, 7.130850542882445, 7.257271525206839, 7.152167560188634, 7.142186193709161, 7.447094756326858, 7.334761397853779, 7.219486427914565, 7.228210218393119, 7.426554058767428, 7.132580748029575, 7.223917520729599, 7.263659279817229, 7.189684937714012, 7.482648430356554, 7.095736025245326, 7.574625728995937, 7.38946814445933, 7.244034076192577, 7.498027796957903, 7.008870029145745, 7.15013636904917, 7.386317183257668, 7.2781826174183255, 7.442044208004217, 7.129555867735747, 7.329633574576894, 7.2512177115033385, 7.374200605283118, 7.201510373194506, 7.430852811047985, 7.4222077579255314, 7.237940956832497, 7.319293327392287, 7.379251484658308, 7.269812760079742, 7.2872354589450135, 7.408542640649589, 7.251805559085433, 7.247810058533006, 7.146992467770911, 7.201426477189276, 7.362180983944303, 7.188891135962906]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="SGD")
data.update({'train_metric': [0.808, 1.439, 2.017, 2.264, 2.524, 2.704, 3.065, 3.323, 3.664, 4.002, 4.392, 4.621, 4.938, 5.328, 5.796, 6.141, 6.614, 6.834, 7.361, 7.693, 8.008, 8.426, 8.83, 9.101, 9.388, 9.7, 10.114, 10.39, 10.769, 11.239, 11.554, 11.864, 12.287, 12.544, 12.793, 13.167, 13.492, 13.678, 14.095, 14.441, 14.755, 15.026, 15.241, 15.668, 16.104, 16.228, 16.429, 16.801, 17.191, 17.568, 17.754, 17.937, 18.399, 18.682, 18.901, 19.238, 19.626, 19.688, 20.168, 20.375, 20.538, 20.789, 21.199, 21.337, 21.763, 21.994, 22.292, 22.332, 22.84, 23.018, 23.087, 23.558, 23.774, 24.028, 24.314, 24.461, 24.629, 24.807, 25.229, 25.464, 25.722, 25.892, 26.064, 26.237, 26.689, 26.837, 27.042, 27.049, 27.5, 27.729, 27.975, 28.126, 28.374, 28.494, 28.86, 28.864, 29.109, 29.403, 29.72, 29.809, 30.048, 30.135, 30.536, 30.524, 30.901, 31.05, 31.231, 31.465, 31.586, 31.827, 32.121, 32.247, 32.345, 32.529, 32.691, 33.071, 33.231, 33.322, 33.429, 33.759, 33.928, 34.235, 34.116, 34.283, 34.43, 34.672, 34.855, 35.09, 35.403, 35.704, 35.668, 35.809, 36.062, 36.113, 36.182, 36.416, 36.935, 37.028, 37.047, 37.175, 37.306, 37.537, 37.776, 37.926, 38.308, 38.397, 38.542, 38.505, 38.874, 39.097, 39.084, 39.127, 39.563, 39.602, 39.737, 39.874, 40.072, 40.288, 40.7, 40.542, 40.651, 40.817, 41.21, 41.273, 41.254, 41.724, 41.931, 42.113, 42.207, 42.056, 42.57, 42.453, 42.833, 42.985, 42.936, 43.381, 43.629, 43.406, 43.539, 43.842, 44.028, 44.252, 44.544, 44.282, 44.62, 44.805, 44.992, 45.257, 45.256, 45.302, 45.61, 45.875, 45.826, 46.028, 45.97, 46.032, 46.294, 46.44, 46.814, 47.064, 47.069, 47.539, 47.35, 47.717, 47.814, 47.84, 47.66, 47.799, 48.072, 48.338, 48.478, 48.512, 48.56, 48.621, 49.153, 48.841, 49.448, 49.262, 49.674, 49.681, 49.66, 49.989, 50.27, 50.456, 50.302, 50.306, 50.812, 50.77, 51.035, 50.993, 51.231, 51.538, 51.377, 51.517, 51.673, 51.964, 51.904, 52.104, 51.962, 52.21, 52.838, 52.882, 52.765, 52.889, 53.19, 53.158, 53.344, 53.124, 53.719, 53.827, 53.844, 54.125, 53.883, 54.1, 54.421, 54.374, 54.624, 54.881, 55.003, 55.145, 54.914, 55.235, 55.592, 55.402, 55.515, 55.905, 55.576, 56.046, 55.904, 56.418, 56.502, 56.442, 56.545, 56.637, 56.438, 56.907, 57.197, 57.229, 57.045, 57.187, 57.53, 57.541, 57.573, 58.026, 57.87, 57.999, 58.231, 58.235, 58.711, 58.65, 58.688, 58.501, 58.569, 59.192, 59.003, 59.093, 58.953, 59.245, 59.748, 59.384], 'train_loss': [5.287008736122898, 5.250721265776982, 5.209878530093515, 5.161189569240187, 5.10219751743651, 5.041570806533803, 4.978646242687203, 4.922615346585187, 4.869896632467259, 4.824097749291516, 4.781650456265616, 4.743920557787231, 4.704540740810597, 4.6677029610290335, 4.627531852465903, 4.588336904919918, 4.5473151814075745, 4.50815684964698, 4.466580025141466, 4.428653932121116, 4.394391358241925, 4.356478524528401, 4.322280787079287, 4.290591749867337, 4.257583477523032, 4.229115917830611, 4.196517634376531, 4.166014641885641, 4.143334399258107, 4.113703372916272, 4.087168094216443, 4.059790715908897, 4.033516814292278, 4.0096434918414, 3.985355508991029, 3.9613519379975166, 3.937802652174742, 3.917700137530697, 3.8916573931754437, 3.8710827983043474, 3.849808085979137, 3.82744246923382, 3.8031322914899652, 3.78469096020255, 3.7628531748868683, 3.744081460430465, 3.7242757116306766, 3.705401862956588, 3.68213839402812, 3.6606416110418887, 3.644194939162437, 3.628180024109814, 3.6081860664938783, 3.5924181622415494, 3.5704826567314867, 3.554460034184325, 3.5330567976182192, 3.5200338626014673, 3.502383834798597, 3.483943806194908, 3.46803664306914, 3.45312895793146, 3.437375200153236, 3.4230792951065228, 3.4033739078677927, 3.3866699806635605, 3.3679374541064826, 3.3575068038774467, 3.337981036742071, 3.3247170939479056, 3.310490650468657, 3.290826923330091, 3.276138015992353, 3.261846245059735, 3.2493020353298956, 3.2376366189406305, 3.2212638575833954, 3.2069411497427267, 3.194598548319274, 3.179697466140669, 3.1619525678784743, 3.1494229639179037, 3.138932457507152, 3.124888920280618, 3.110576280019105, 3.094608858466072, 3.080611078996957, 3.0749816461320267, 3.0577684763678357, 3.0445980664178185, 3.0285057113937897, 3.0193710655107457, 3.0074826714631007, 2.9919569669285417, 2.979948811399883, 2.972502162802776, 2.9582600318996555, 2.9424262165794444, 2.931453251182766, 2.9182481421008752, 2.908195530994535, 2.898178950533681, 2.8868270766941007, 2.874364506946644, 2.857796605550091, 2.850873223040551, 2.839782968211159, 2.8277453302345594, 2.818560327693429, 2.8061425620290765, 2.794777418204934, 2.7862026476356667, 2.7720197118098966, 2.765546182600718, 2.7517332199362707, 2.7431021166854537, 2.7275683544266323, 2.721367656765118, 2.712025676563773, 2.6993150576825182, 2.6905351219006404, 2.674483046650658, 2.6657576046917426, 2.6591539377595703, 2.6513801931343397, 2.640014921101102, 2.6292937999342163, 2.618904028957804, 2.608390586694997, 2.595856059001793, 2.5927996798653825, 2.5799257574520733, 2.5673244472352343, 2.559760740149578, 2.552704198880601, 2.5442174445034524, 2.523909402824104, 2.5235575414664915, 2.514744723674508, 2.504346381527296, 2.497883469953189, 2.48651405328066, 2.4774809255862342, 2.461553965710099, 2.4566070470608583, 2.4453146765038363, 2.4385725541215484, 2.4347924446342697, 2.4222083461643105, 2.4119329731661163, 2.405096117228327, 2.402823986796637, 2.3858658458968423, 2.3770140295827993, 2.370337190951435, 2.3600389641672086, 2.3514211335703874, 2.3403804411281017, 2.3267476342842506, 2.325110136127899, 2.3229149613560427, 2.3102424744986148, 2.296175091905771, 2.2940063723675808, 2.2855379609671145, 2.276779468289874, 2.260562076297084, 2.2551768903006213, 2.248416270800912, 2.25057185283473, 2.2342229809275973, 2.228697652093737, 2.218404753804588, 2.208807890642475, 2.2061831175480147, 2.197312551969454, 2.1827016050664567, 2.182229452993499, 2.1727690283335406, 2.165346862182202, 2.1540658102535852, 2.144584622813278, 2.1342465718129624, 2.138733391493311, 2.1289992157038555, 2.1186457401044994, 2.1095687650520683, 2.1010436711216767, 2.098663354408108, 2.0902844157954172, 2.084074458249166, 2.076495097755852, 2.0706198087351795, 2.0604851677565716, 2.0535753059295683, 2.049900366416438, 2.0422444477801283, 2.031129311996626, 2.0224812534171193, 2.0176866760790846, 2.011719992125713, 1.9947850621059318, 1.9955660219918592, 1.9849968945758891, 1.9794651970067088, 1.9741679728984527, 1.969782081233029, 1.9694956612754768, 1.9565036140491927, 1.9516800973786044, 1.9443305205322579, 1.9405286537861106, 1.9313215526036551, 1.9299822029057636, 1.917780181878054, 1.9125997242985515, 1.9039362784158094, 1.8981885672835914, 1.8872196633924068, 1.8791015813233223, 1.8821373266511747, 1.873475970630072, 1.8611105109206851, 1.8534509568205264, 1.8515863880772507, 1.8469717118195517, 1.8335940879045658, 1.8361150904183805, 1.8239581047992866, 1.8214130002721638, 1.8155278065078013, 1.8044157449968794, 1.8020776750487695, 1.79770262395428, 1.788544359759345, 1.7827667621794376, 1.7822730700632585, 1.7762005384808843, 1.772060996816468, 1.766272212554458, 1.7497486711234826, 1.7437312234614952, 1.7422686542979586, 1.7327248462102234, 1.7207689587305695, 1.7224555875884366, 1.7202953464772865, 1.7177726147804822, 1.7061813299425581, 1.7012509637663018, 1.6901206991570314, 1.6896787344913642, 1.689925265479988, 1.6846310071890276, 1.6719846391403286, 1.6698346781715399, 1.6587387816080739, 1.6508843282515318, 1.6493302788096822, 1.6389288180772876, 1.6415147670018544, 1.634623878748083, 1.6219143373266063, 1.6221345498328474, 1.616503893833319, 1.6046879059675025, 1.6110171083449096, 1.5961612221985693, 1.5965134629208693, 1.5879247498985138, 1.5777806682763615, 1.5750584635914555, 1.5781251731318537, 1.5677454937976365, 1.5704138930684393, 1.5569648442936492, 1.5511181298464594, 1.547866830288868, 1.5473070094315424, 1.546940161185774, 1.534734497524879, 1.531417585303977, 1.52832016690145, 1.516318276762886, 1.5164071456484511, 1.5081262480960926, 1.500377681799905, 1.4956310854382189, 1.482378211032101, 1.4875822692289615, 1.478238822211841, 1.484108701586647, 1.4738380621048555, 1.4654299597746274, 1.468473817702676, 1.4607862558261142, 1.462141266749291, 1.4560886099410224, 1.4458712748587321, 1.443732799739313], 'val_metric': [1.17, 1.88, 2.17, 2.55, 2.75, 3.14, 3.49, 3.68, 4.21, 4.71, 5.1, 5.23, 5.67, 6.2, 6.46, 6.61, 7.47, 7.49, 7.91, 8.38, 8.73, 8.98, 9.53, 9.87, 9.97, 10.25, 10.16, 10.99, 11.04, 11.32, 11.83, 12.35, 12.74, 12.65, 13.28, 13.48, 13.6, 14.31, 13.94, 14.85, 14.81, 15.33, 14.87, 15.53, 15.75, 15.17, 15.99, 16.16, 16.86, 17.01, 17.22, 17.2, 17.35, 17.89, 17.8, 18.04, 17.71, 17.76, 18.44, 18.67, 18.75, 18.4, 18.79, 19.3, 19.45, 19.21, 19.52, 19.48, 19.58, 19.7, 19.83, 19.91, 19.88, 20.33, 20.41, 20.23, 19.95, 20.46, 20.75, 20.65, 20.13, 20.45, 20.76, 20.84, 21.17, 21.26, 20.76, 21.63, 21.77, 21.17, 21.46, 21.29, 21.37, 21.3, 21.77, 21.79, 20.94, 21.35, 21.31, 21.13, 21.08, 21.45, 21.38, 21.91, 21.13, 21.76, 21.7, 21.51, 21.55, 21.92, 22.27, 21.94, 22.05, 22.32, 22.0, 21.77, 22.37, 22.06, 21.51, 21.61, 22.12, 21.75, 22.39, 21.41, 21.74, 21.41, 22.07, 22.07, 22.07, 22.04, 21.89, 22.26, 22.19, 21.7, 22.14, 22.05, 21.62, 21.85, 21.23, 21.59, 21.93, 21.55, 21.95, 22.19, 22.19, 22.24, 22.18, 22.47, 21.69, 21.69, 21.71, 21.97, 22.15, 22.12, 21.55, 21.69, 22.05, 21.37, 21.65, 22.18, 21.54, 21.59, 21.6, 20.52, 21.3, 21.41, 21.7, 21.5, 21.59, 21.27, 21.64, 21.04, 21.36, 21.69, 21.13, 21.15, 21.41, 21.37, 21.53, 21.09, 21.33, 21.34, 21.32, 20.76, 21.28, 21.53, 21.85, 21.3, 21.6, 20.98, 21.49, 21.16, 21.67, 21.28, 20.88, 21.56, 20.8, 20.93, 21.57, 21.01, 21.02, 21.13, 21.27, 21.34, 21.03, 21.29, 21.07, 20.96, 21.0, 20.68, 20.52, 21.0, 20.52, 21.09, 20.78, 21.21, 20.58, 21.29, 21.26, 20.31, 20.98, 20.65, 20.82, 21.2, 20.16, 20.61, 20.07, 20.34, 20.01, 20.57, 20.25, 20.78, 20.06, 21.08, 20.43, 20.39, 21.05, 20.59, 21.26, 20.95, 20.57, 20.14, 20.89, 20.32, 20.49, 20.42, 20.82, 20.19, 20.34, 20.43, 20.59, 20.32, 19.94, 20.05, 20.36, 20.34, 20.18, 19.72, 19.85, 20.57, 20.5, 20.18, 19.79, 20.59, 19.73, 20.04, 19.91, 20.93, 19.51, 19.8, 19.86, 20.49, 19.51, 19.59, 19.99, 19.84, 20.5, 19.85, 19.79, 19.52, 19.82, 19.9, 19.58, 19.63, 20.07, 20.08, 18.47, 19.66, 19.36, 19.58, 19.41, 19.85, 20.47, 19.71, 19.99, 19.96, 19.81, 19.74, 19.16, 19.05], 'val_loss': [5.265702675861918, 5.233204006389448, 5.1777749866437, 5.115777779536642, 5.0536729849068225, 4.995620609089068, 4.93539491732409, 4.874596383161606, 4.818984171387497, 4.782500790942247, 4.727568641589706, 4.691632342186703, 4.650480224828052, 4.613342637469055, 4.568862164855763, 4.535234273618953, 4.494350266304745, 4.493970559660796, 4.412526929454439, 4.390879890721315, 4.35022408187769, 4.31494735608435, 4.294333574878182, 4.245751200208239, 4.228147860545262, 4.1969261518709216, 4.208674168131154, 4.135235616355945, 4.112172955919982, 4.094991510840738, 4.069743126061312, 4.036694373294806, 4.01767728738724, 4.01716338601082, 3.9724827632782564, 3.952224869636973, 3.9428213326035033, 3.908013050723228, 3.9054899382743105, 3.876710507520445, 3.8706664264581767, 3.8417000284620153, 3.860672294713889, 3.8280620164931958, 3.812323626439283, 3.8107286775188083, 3.77645993384586, 3.7817897113265504, 3.7413930399402693, 3.7362277105355717, 3.71239625268681, 3.696078080280571, 3.7031758636426013, 3.6694423453823015, 3.676169940620471, 3.6510970243223153, 3.663072572392263, 3.666041936844018, 3.6441885551829247, 3.6254952690403934, 3.621719101432023, 3.6164403303413635, 3.6177168727680376, 3.5994568918920624, 3.596311970121542, 3.5901492416479024, 3.6000567742973377, 3.5743088448882863, 3.5622814111648853, 3.5590902984521953, 3.574113324189642, 3.5673297051411526, 3.5574421662433893, 3.5309379123578406, 3.5233518097810683, 3.5272771568055363, 3.5626223618817177, 3.518903993497229, 3.5145003529870587, 3.5295901078327447, 3.5285267290795685, 3.519713250694761, 3.518470109648006, 3.5240212162588813, 3.5032335922216915, 3.4873874544338057, 3.532318972478247, 3.481150279379195, 3.4946451809755557, 3.5004008947664005, 3.503983285776369, 3.5046333066976754, 3.506094259820926, 3.5230951559771397, 3.4881421426299273, 3.4907245886553624, 3.5321071421264842, 3.5316516775993785, 3.5203495701407173, 3.5354829609014424, 3.5460049536577456, 3.5315258556110845, 3.513754985894367, 3.5036325545827296, 3.521419817474997, 3.512035368354457, 3.507512041717578, 3.5412008078994264, 3.5395308391303772, 3.51527933026575, 3.5178714284471644, 3.5204583178659914, 3.5387871052808824, 3.5254587200796528, 3.5283743469578446, 3.560751154923895, 3.5383394918624003, 3.5552052130365066, 3.5914883613586426, 3.5593978204544943, 3.5864744429375714, 3.590716411353676, 3.593378531705042, 3.618797905126195, 3.6136393881147835, 3.6458761570559943, 3.5541800723713672, 3.5791779559129364, 3.6024087553570983, 3.602166570675601, 3.6358906319186945, 3.622428504524717, 3.6104720838510307, 3.6716764573079006, 3.6112172277110397, 3.63086576902183, 3.6743668841708237, 3.664742585200413, 3.6868253756480613, 3.6645220176429505, 3.6662896803230236, 3.7027891304842226, 3.6601117659526268, 3.673481578280212, 3.6899160830078612, 3.6901828179693528, 3.686072844608574, 3.70117239132049, 3.7396413383969835, 3.696998990265427, 3.7025414158584207, 3.70195090391074, 3.7331348034986265, 3.7496383068667853, 3.7177205936164612, 3.8256217446296836, 3.762570689438255, 3.7885772802267863, 3.793133169222789, 3.790474173369681, 3.7769330413478195, 3.7686273429044492, 3.8154684646873718, 3.86922091131757, 3.8263959413880757, 3.826053368817469, 3.827852448080755, 3.8202416577916236, 3.842078825470748, 3.8729564217245502, 3.85549123575733, 3.8976529160882256, 3.86912957118575, 3.903676414945323, 3.90351957880008, 3.939791030185238, 3.8693647050553825, 3.8900178632918436, 3.9336725602484055, 3.961990564492098, 3.9157681146245094, 3.9482974725164426, 3.9459218265144687, 3.993685544676082, 3.947957693391545, 3.9660248179344615, 3.9694831690211205, 4.032495756817472, 3.9791837771227407, 4.04804984475397, 3.9912949139904823, 3.979891849930879, 4.006337840086336, 4.037240678337729, 4.060440834920118, 4.0510332068060615, 4.091340388462043, 4.042240810242428, 4.082630644937989, 4.020432592197588, 4.070451695448274, 4.145838895421119, 4.096077270568556, 4.120485882850209, 4.094611292432068, 4.1025894751214675, 4.136801490358486, 4.1371863632445125, 4.159318744756614, 4.118307997466652, 4.19469275823824, 4.211082554926538, 4.214103247709335, 4.1399651302653515, 4.217643385480164, 4.126829435111611, 4.27934250102681, 4.294783337101055, 4.15611860858407, 4.22249618761099, 4.191330082097631, 4.312187032335124, 4.212317705154419, 4.268551521240526, 4.338725897157268, 4.302008624289446, 4.337946607049104, 4.345844179202038, 4.400881691343465, 4.3217471754475, 4.304837743188165, 4.379704680412439, 4.380656823990451, 4.299761907310243, 4.4066878382567385, 4.435459811216707, 4.334822923514493, 4.336884266251971, 4.415670624204502, 4.364009611925502, 4.364569890271326, 4.479085489443153, 4.4114753660882355, 4.375725939015674, 4.434298759812762, 4.439502954483032, 4.489252948457269, 4.4607733951252735, 4.429672642118612, 4.50130988989666, 4.426493864909858, 4.507575000167653, 4.503347109837137, 4.489847095149337, 4.439879204816879, 4.554468397881575, 4.530052753011132, 4.534203602250215, 4.618957672908807, 4.539701502793913, 4.559164267436714, 4.62396109180086, 4.605676026860619, 4.579336013004279, 4.674464421667111, 4.600488687017162, 4.616670449068592, 4.669108299692725, 4.611874238700624, 4.663728768658486, 4.65672423125832, 4.6315926229877835, 4.654849277180471, 4.703527259219224, 4.63453656852625, 4.679853853906036, 4.691951538347135, 4.696982111900476, 4.734986704625901, 4.801172598152403, 4.765856542405049, 4.780109921838068, 4.798430784492735, 4.807167806443135, 4.863190707127759, 4.8237131613834645, 4.900990868829618, 4.839962113435101, 4.698093295856646, 4.788051231651549, 4.850779859883011, 4.81327926125496, 4.791032291521692, 4.909452913673061, 4.856467242453508, 4.832646857401368, 4.871409422273089, 4.812693239017657, 5.025365411855613, 4.968772198743881]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="Adam")
data.update({'train_metric': [5.931, 12.756, 15.936, 18.244, 19.923, 21.511, 22.675, 23.915, 24.862, 25.921, 26.723, 27.281, 28.11, 28.558, 29.171, 29.615, 30.032, 30.488, 30.871, 31.084, 31.69, 32.083, 32.331, 32.529, 32.897, 33.246, 33.474, 33.746, 33.876, 34.263, 34.398, 34.597, 34.938, 34.845, 35.167, 35.191, 35.381, 35.6, 35.719, 36.131, 36.175, 36.012, 36.42, 36.418, 36.644, 36.666, 36.843, 36.961, 37.155, 37.123, 37.219, 37.433, 37.463, 37.645, 37.75, 37.556, 37.848, 37.996, 37.863, 38.063, 38.036, 38.316, 38.205, 38.46, 38.441, 38.51, 38.505, 38.585, 38.574, 38.616, 38.824, 38.72, 38.825, 38.932, 38.974, 39.194, 39.01, 39.042, 39.225, 39.108, 39.221, 39.229, 39.369, 39.558, 39.417, 39.453, 39.515, 39.748, 39.613, 39.64, 39.719, 39.798, 39.701, 39.726, 39.693, 39.84, 39.813, 39.887, 40.03, 40.053, 40.086, 40.053, 39.974, 40.063, 40.074, 40.189, 40.215, 40.066, 40.197, 40.151, 40.14, 40.231, 40.163, 40.305, 40.187, 40.26, 40.311, 40.508, 40.316, 40.237, 40.157, 40.36, 40.436, 40.349, 40.421, 40.424, 40.62, 40.436, 40.518, 40.454, 40.392, 40.724, 40.794, 40.645, 40.558, 40.464, 40.505, 40.617, 40.54, 40.589, 40.824, 40.743, 40.668, 40.614, 40.94, 40.717, 40.611, 40.503, 40.668, 40.737, 40.657, 40.671, 40.684, 40.961, 40.782, 40.779, 40.843, 40.878, 41.076, 41.054, 40.866, 41.108, 41.112, 40.852, 40.821, 41.039, 41.016, 41.008, 41.09, 40.788, 40.935, 40.75, 41.0, 41.151, 41.22, 41.242, 41.146, 41.054, 40.971, 41.129, 41.148, 41.194, 41.189, 41.144, 41.307, 41.081, 41.16, 41.218, 41.161, 41.355, 41.349, 41.168, 41.176, 41.127, 41.114, 41.208, 41.07, 41.203, 41.233, 41.279, 41.242, 41.197, 41.233, 41.413, 41.386, 41.209, 41.244, 41.334, 41.222, 41.426, 41.366, 41.333, 41.34, 41.469, 41.407, 41.253, 41.299, 41.321, 41.477, 41.37, 41.301, 41.313, 41.43, 41.245, 41.356, 41.42, 41.324, 41.252, 41.432, 41.542, 41.24, 41.458, 41.377, 41.663, 41.469, 41.533, 41.42, 41.439, 41.767, 41.541, 41.627, 41.536, 41.617, 41.547, 41.569, 41.592, 41.618, 41.532, 41.599, 41.554, 41.735, 41.56, 41.419, 41.557, 41.706, 41.664, 41.589, 41.557, 41.746, 41.52, 41.566, 41.795, 41.664, 41.682, 41.658, 41.526, 41.746, 41.625, 41.723, 41.699, 41.709, 41.919, 41.657, 41.594, 41.587, 41.601, 41.775, 41.77, 41.692, 41.687, 41.826, 41.86, 41.798, 41.644, 41.805, 41.666, 41.772, 41.837, 41.636, 42.016, 41.861, 41.799, 41.85, 41.675, 41.877, 41.844, 41.708, 41.864, 41.84, 41.844], 'train_loss': [4.554219322027644, 3.93675738714173, 3.705779946620695, 3.5585240735049743, 3.45451982709283, 3.3682037725405896, 3.298260639969233, 3.232220364013545, 3.182532573920828, 3.1264704192058446, 3.086736418845481, 3.0496059031495664, 3.016367206341627, 2.9852144907318623, 2.9593694377845474, 2.934129372057973, 2.912395278231425, 2.8841272083826732, 2.869824512448741, 2.851670727848778, 2.829700573392198, 2.8142630492580754, 2.798871709533174, 2.7820504250163385, 2.767241636378141, 2.7495929787575397, 2.7424516430743138, 2.721263048363586, 2.7165127750855564, 2.7015363788726767, 2.6958013133063043, 2.6819166919015114, 2.6723627960231924, 2.6684890032119655, 2.658088727174321, 2.6506329819855594, 2.642488856995937, 2.636011584248973, 2.6257876029627796, 2.615594357995749, 2.614818325122045, 2.61169037800604, 2.6061417917677474, 2.5989276079207144, 2.5892866791171745, 2.5859926635457855, 2.5828678657515876, 2.575560943102577, 2.5715472737109133, 2.5637304634904527, 2.559430795225362, 2.5522325060265385, 2.5532365406619686, 2.544907759491328, 2.54276481913361, 2.5434288276134205, 2.5313793510026987, 2.5258408741392695, 2.528418130090583, 2.5255738523474736, 2.522804124951744, 2.51584614917245, 2.509156547977767, 2.510493517074536, 2.50569918814639, 2.504433044468983, 2.5026848165941633, 2.4956408668921988, 2.4959094329133524, 2.4926063920623283, 2.4906976231381592, 2.4920653455621986, 2.485206450320785, 2.48100131631889, 2.4832840516486385, 2.476794804202694, 2.480610167987821, 2.4737778582484466, 2.4723636758533414, 2.4744425634504967, 2.4715450385863096, 2.4699600294318174, 2.4640793132995538, 2.4597685473589124, 2.461194650454164, 2.4634615792270202, 2.4594227490483074, 2.4498804862577033, 2.4563974889516067, 2.454257075708796, 2.4505833007552575, 2.4470365136690195, 2.4493328225360953, 2.4521696583552, 2.4449046256064757, 2.446217988899558, 2.442075935069064, 2.4441350907449606, 2.440590325945551, 2.4389547342988673, 2.4368804810677136, 2.4366946841231996, 2.4390252611992533, 2.4373273700761704, 2.4333004331405697, 2.4328584632885737, 2.431319381057339, 2.4360962267190467, 2.429356982672893, 2.4251705552245744, 2.4292542851283927, 2.4265393935482393, 2.4300829413146143, 2.4234304059718714, 2.426586409524245, 2.4227227196049705, 2.4233743334457194, 2.4229080103485536, 2.4192060308432013, 2.4182760597266832, 2.4211295186443658, 2.422639189739679, 2.419669627075537, 2.4191295933586163, 2.414333726302676, 2.417539955634607, 2.416369074289416, 2.4165542430780063, 2.415138929529367, 2.413696353281452, 2.413051215105917, 2.4099187262914, 2.40897658057344, 2.4072594569420405, 2.409861811139381, 2.4111458757941113, 2.4106376656949022, 2.4079471423087484, 2.4068753682720456, 2.4087727973648776, 2.4034677342733968, 2.4025217685193034, 2.4056272161212857, 2.4066917932117433, 2.400280568283945, 2.406465435592471, 2.402889774460405, 2.405241893715227, 2.4009194705856967, 2.3986717984986017, 2.402007473254921, 2.395839172452974, 2.4010256472414913, 2.394914873959693, 2.3962812162101534, 2.395325193020753, 2.398410151421223, 2.397459871709461, 2.3925241352424207, 2.3927725180554527, 2.3947407991094463, 2.3931250323565894, 2.3929271000131767, 2.393131396134389, 2.391366374393495, 2.3945235258024318, 2.392108165066134, 2.390586820765329, 2.3872097535386576, 2.391897909395678, 2.38948208372027, 2.3936796876305735, 2.3924249368681028, 2.3893378573354815, 2.387120369528626, 2.3862566123463296, 2.38404008980676, 2.382450483887148, 2.380418098133036, 2.389179988236894, 2.383992496546613, 2.3836451270835033, 2.380610812877282, 2.3863205642785617, 2.3839100204975425, 2.386004307219712, 2.3851345979625873, 2.377935879518798, 2.3836929085768728, 2.3816109295464902, 2.379133543904134, 2.381867412489649, 2.3829713716616787, 2.3807271385680995, 2.3793601687718717, 2.3795175335762675, 2.383503558692151, 2.3764987152811052, 2.381379208348153, 2.3753792715621773, 2.376458528403357, 2.3756529202616834, 2.376043583823562, 2.376498798719981, 2.3748238254493423, 2.3776170826080283, 2.3795259822581873, 2.3765160193903965, 2.380686313085501, 2.3744127898359633, 2.3755760540660193, 2.372888397728108, 2.3778163258722786, 2.3723871149623235, 2.37135140253654, 2.3751483377705607, 2.3791634794694065, 2.3707766522220215, 2.3723419260231258, 2.3715988691998686, 2.3716062112870464, 2.3723764773446328, 2.3717825524866467, 2.3701471754624457, 2.371868644779642, 2.3713302749589857, 2.3732446662447657, 2.369910295697564, 2.3728443545861957, 2.366292101865538, 2.369176924419342, 2.3668292836355844, 2.3664874973315424, 2.363730121253777, 2.370630610164586, 2.3642255792385938, 2.3661645588322626, 2.3683219860550384, 2.359844776658164, 2.3665266503756883, 2.364149462696229, 2.365015897549503, 2.3634967253289005, 2.3664144796205497, 2.3633443260528457, 2.3624165038305907, 2.364127160796582, 2.3646507054205057, 2.3648700150479436, 2.366276062762821, 2.3626602174224414, 2.3624416000173403, 2.363910738505085, 2.363903228014765, 2.360175387186647, 2.3615109475850296, 2.360680369025076, 2.3611832388074294, 2.3580508252709476, 2.361753072634921, 2.358072305061233, 2.3582900706118526, 2.359556457284011, 2.3566793052340196, 2.3607750890045995, 2.3611295900509592, 2.358979890229072, 2.3609414228779797, 2.3548809798650074, 2.357653940631576, 2.3544563308253323, 2.3530185996761555, 2.357504839784277, 2.358397529434868, 2.3593515743449647, 2.3588637209670784, 2.3581214421693897, 2.3552981584558714, 2.357474770442233, 2.3561787088178168, 2.355099377308758, 2.354212439754264, 2.353401387340353, 2.355637102270462, 2.355289270690215, 2.3567149353881565, 2.3527017663246688, 2.354143580563619, 2.3562621579136667, 2.35039115188523, 2.3541065954048515, 2.357754352377991, 2.3516777714169796, 2.3544852207352243, 2.357930223223344, 2.3558682635741133, 2.354642740137975, 2.3562754145510594, 2.3473810443341234, 2.3549237506022], 'val_metric': [5.54, 14.68, 15.23, 15.46, 17.42, 20.76, 21.93, 22.97, 24.42, 22.29, 23.81, 24.69, 25.14, 25.28, 27.49, 25.17, 27.29, 26.08, 28.97, 30.53, 26.44, 30.31, 27.83, 22.65, 29.58, 27.13, 32.38, 27.52, 33.35, 30.3, 28.45, 30.25, 31.25, 30.16, 27.4, 28.72, 31.29, 33.59, 31.43, 32.25, 31.27, 27.67, 32.89, 31.43, 32.99, 30.75, 32.58, 33.47, 32.79, 33.86, 32.59, 27.04, 33.86, 35.58, 33.96, 34.89, 32.67, 32.19, 33.91, 34.9, 32.98, 34.63, 35.81, 36.24, 34.55, 32.71, 32.41, 34.77, 35.85, 30.8, 34.38, 30.65, 32.02, 35.52, 35.48, 33.67, 35.39, 33.92, 34.83, 35.86, 30.57, 35.05, 35.54, 36.4, 35.85, 34.52, 34.8, 33.05, 34.02, 35.8, 28.54, 33.32, 31.65, 34.79, 35.05, 32.85, 35.7, 32.89, 34.01, 35.68, 33.11, 35.99, 36.1, 33.79, 36.09, 36.67, 35.26, 33.95, 35.73, 32.4, 35.21, 36.01, 32.93, 33.55, 36.0, 37.58, 34.85, 33.86, 35.1, 35.01, 36.86, 35.95, 34.72, 35.69, 33.84, 34.03, 35.14, 35.03, 33.19, 36.35, 32.09, 35.75, 36.62, 36.21, 36.37, 34.51, 31.41, 35.37, 34.97, 34.34, 34.98, 35.59, 32.67, 35.81, 35.64, 34.06, 35.31, 37.09, 30.26, 33.35, 35.04, 36.08, 33.75, 37.74, 38.12, 36.27, 34.31, 34.72, 33.51, 36.91, 35.48, 35.21, 32.47, 34.73, 34.55, 34.5, 35.75, 38.46, 33.78, 35.26, 33.95, 34.79, 37.27, 34.78, 26.25, 37.0, 35.33, 33.39, 35.91, 36.96, 35.04, 34.74, 34.43, 35.22, 35.14, 36.6, 36.55, 33.43, 34.78, 36.7, 33.3, 35.8, 35.98, 33.61, 34.94, 36.36, 32.11, 33.35, 34.38, 36.33, 34.26, 32.3, 34.65, 37.74, 36.03, 36.0, 33.44, 36.41, 35.4, 36.21, 35.73, 35.18, 36.39, 37.67, 33.63, 32.69, 36.85, 38.45, 33.39, 35.39, 37.18, 33.71, 34.43, 36.98, 36.6, 33.98, 37.18, 35.98, 37.73, 35.17, 35.9, 37.89, 32.74, 33.57, 36.44, 34.96, 37.46, 35.06, 31.81, 33.23, 37.03, 36.32, 31.46, 36.17, 33.13, 36.3, 37.21, 37.3, 34.13, 35.97, 35.87, 36.57, 35.54, 35.63, 34.19, 35.22, 36.53, 34.21, 32.04, 36.27, 35.43, 34.58, 34.6, 34.67, 35.13, 35.08, 35.02, 36.1, 34.54, 36.99, 37.7, 33.74, 35.47, 36.86, 35.37, 34.87, 37.34, 35.8, 36.32, 35.21, 35.99, 35.72, 33.61, 36.16, 36.48, 37.9, 32.04, 34.72, 34.87, 36.77, 33.99, 36.63, 38.3, 33.12, 37.69, 37.71, 35.6, 33.45, 34.81, 33.46], 'val_loss': [4.600437351093171, 3.83212179590942, 3.755249496478184, 3.7246390102775235, 3.573065701563647, 3.3942017335041315, 3.34951405160746, 3.2819689990608554, 3.1952849026698216, 3.3230412932717877, 3.241082304602216, 3.209694382491385, 3.1643851319695733, 3.1564177115252066, 3.0434910195648293, 3.165624336452241, 3.0463430608154103, 3.134451850964006, 2.9918279047984226, 2.8866916576008887, 3.16318899051399, 2.874376153490346, 3.041082800573604, 3.3531517944518168, 2.9545601446917105, 3.1222853675769393, 2.8059911773462964, 3.104546373816812, 2.7751920405466843, 2.915286998839895, 3.0113426484879415, 2.9276986190467884, 2.8763276460064446, 2.934073322897504, 3.0734178336562623, 2.981448768810102, 2.855486801475476, 2.781853508797421, 2.8712908473743757, 2.821539415675364, 2.884708131194874, 3.1408152390437523, 2.7933427778778563, 2.8462356093582835, 2.7459155059164497, 2.9247820605138304, 2.790806026215766, 2.7407316425044064, 2.791627038056683, 2.733300017703111, 2.859324430204501, 3.1708702137515803, 2.798682146011644, 2.653402796216831, 2.7749242828150464, 2.704778740360479, 2.8220024131665564, 2.899902024466521, 2.7773880753547524, 2.7060310563464074, 2.815206897486547, 2.710433872642031, 2.6443243360823128, 2.6469872734349247, 2.7336143801926047, 2.7891714899403275, 2.8557517072957035, 2.7202428146532385, 2.6512601193349075, 2.955740898278109, 2.732646801669127, 2.9220117201471023, 2.9014539559176016, 2.6924895237965187, 2.6747126632435307, 2.7836213430781274, 2.7074040641450576, 2.7622503300381314, 2.6627440536098117, 2.6775589201860366, 2.9645849079083484, 2.7065918908757007, 2.701294376591968, 2.6267610268228374, 2.6702433019686658, 2.7378984218949727, 2.7531451905608937, 2.792941872481328, 2.7745938942690564, 2.6696390642482006, 3.0564118692070057, 2.7954798914065027, 2.8631115255841784, 2.7410810878322383, 2.7194914073701115, 2.8296866113213217, 2.6817812399499736, 2.8182722132676727, 2.771808934819167, 2.700914818769807, 2.8205357858329823, 2.6678630257867706, 2.616043280644022, 2.7300587448344866, 2.6977147296735438, 2.6248213842416264, 2.6722954139587984, 2.7426541348931135, 2.6726856231689453, 2.824678982898688, 2.7539881012242313, 2.65931109220359, 2.8580428285963215, 2.784139042447327, 2.646062831210483, 2.598024788935473, 2.7020976888905666, 2.7745096129216966, 2.677437057920322, 2.7353430105622407, 2.5950202653362493, 2.711041897725148, 2.7374928369643583, 2.681825332580858, 2.772168001551537, 2.777950958081871, 2.692642103334901, 2.7163634960818444, 2.8210741813015785, 2.6694940708245443, 2.842792263456211, 2.6269938292776702, 2.646346131707453, 2.6246463781709126, 2.6273925543590715, 2.721611401078048, 2.9665238356134696, 2.7103280265619802, 2.7280074904678733, 2.7420754523793605, 2.705229298703989, 2.6392378731138386, 2.8092954401757306, 2.656501406317304, 2.6842658679196787, 2.8059154794474317, 2.6947850983613617, 2.6246782093291072, 2.9928127534829887, 2.80513919842471, 2.727659143460025, 2.649595707085482, 2.800465273629328, 2.5624954723248816, 2.557815408251088, 2.6213053609155548, 2.780787900374953, 2.7378080297427574, 2.8420477949889604, 2.6005356554772443, 2.6603865091967736, 2.6753764099376216, 2.839226568580433, 2.74071342929913, 2.718823050237765, 2.708878036896894, 2.6745561179082107, 2.520900356541773, 2.745125511649308, 2.672372773194769, 2.755730257672109, 2.7658891336173768, 2.5901020112311004, 2.6903345675984767, 3.2334708575230495, 2.5930205219110865, 2.672493232283623, 2.79315326168279, 2.644477001421011, 2.624243211594357, 2.7477194708623704, 2.7015090622719686, 2.746585366832223, 2.6836600053082607, 2.7124977081444612, 2.6389507040096696, 2.6221289391730243, 2.7811660910867584, 2.7216787201583763, 2.6332344910141767, 2.8233839334196347, 2.715604845125964, 2.6749274476318603, 2.7758698721600186, 2.710128293675222, 2.6436121357474356, 2.8571449290415285, 2.817944327547292, 2.738125493192369, 2.622783253906639, 2.7477322254970575, 2.8431988039593787, 2.7468192744406923, 2.5826185922713796, 2.6673995423468813, 2.6711417375856144, 2.800026658993618, 2.632259398508983, 2.6841963476436153, 2.65499510127268, 2.6936903216276957, 2.6858841028942426, 2.640385925010511, 2.55001676538188, 2.7735269092450476, 2.8524698877030876, 2.6048288163106155, 2.5400062009787105, 2.831937294097463, 2.701037793402459, 2.6022771213464675, 2.7514756261163456, 2.768163706846298, 2.632423493133229, 2.640430495997143, 2.739942956881918, 2.5670951528913655, 2.6706507760248366, 2.601311215929165, 2.6947875675881745, 2.681991782537691, 2.588718160702165, 2.8312948976352716, 2.7809230833296565, 2.618306487608867, 2.739577048903058, 2.5762585857111935, 2.717774664520458, 2.954079959802567, 2.7891409404718193, 2.631143503887638, 2.6317253583555766, 2.917203449899224, 2.654856859119075, 2.8687311820923145, 2.6493416143830415, 2.580467045686807, 2.565379928631388, 2.7256262579541297, 2.6560565062389254, 2.6427933251022533, 2.625344770729162, 2.6754427753436336, 2.6819290108741467, 2.75334636952467, 2.7368319323108454, 2.6543176436120537, 2.745104638634214, 2.8956010075891094, 2.6347989094485142, 2.680771444253861, 2.735324552104731, 2.749946813294842, 2.726621712089344, 2.695757520806258, 2.7149676069332536, 2.7162630683297566, 2.636973604275163, 2.7569878063384134, 2.6004156071668976, 2.573050643608069, 2.8142570579887196, 2.7034949823549597, 2.6410734964783784, 2.6832959576017537, 2.7390373611146477, 2.5729056490454703, 2.703771821252859, 2.606389145182956, 2.674025246292163, 2.6704539778126275, 2.6382368620793533, 2.790895326122357, 2.6343884217511317, 2.666640687899984, 2.5900240840425917, 2.927425386799369, 2.7447951644848865, 2.7066667102704383, 2.621105933644969, 2.749999605166684, 2.625724931431424, 2.5314965794800193, 2.832147788090311, 2.5751382568079952, 2.599793809234716, 2.7308341067308075, 2.8135870611591702, 2.7001878957080234, 2.815965041613123]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="AdamW")
data.update({'train_metric': [7.792, 19.097, 26.705, 32.311, 37.002, 41.093, 44.702, 48.086, 51.25, 54.246, 56.826, 59.835, 62.447, 64.83, 67.274, 69.594, 71.55, 73.569, 75.367, 77.016, 78.348, 79.806, 80.551, 81.792, 82.758, 83.45, 84.316, 85.123, 85.644, 86.239, 86.559, 87.064, 87.721, 87.925, 88.133, 88.773, 88.854, 89.391, 89.481, 89.71, 90.015, 90.216, 90.409, 90.581, 90.967, 90.93, 91.077, 91.476, 91.6, 91.555, 91.785, 91.836, 92.048, 92.154, 92.311, 92.249, 92.503, 92.783, 92.695, 92.61, 93.021, 92.781, 92.895, 93.29, 93.111, 93.396, 93.325, 93.537, 93.418, 93.564, 93.594, 93.731, 93.774, 94.024, 93.745, 93.988, 93.998, 93.963, 94.244, 94.128, 94.261, 94.213, 94.175, 94.414, 94.311, 94.629, 94.438, 94.468, 94.649, 94.531, 94.599, 94.708, 94.605, 94.733, 94.964, 94.877, 95.088, 95.015, 94.781, 95.046, 95.014, 95.073, 95.107, 95.126, 95.276, 95.099, 95.408, 95.237, 95.278, 95.474, 95.202, 95.269, 95.505, 95.398, 95.329, 95.551, 95.474, 95.694, 95.459, 95.441, 95.763, 95.628, 95.473, 95.532, 95.775, 95.658, 95.752, 95.89, 95.689, 95.719, 95.704, 95.953, 95.798, 95.975, 95.941, 95.82, 95.929, 95.911, 95.992, 96.05, 95.965, 95.918, 96.027, 96.129, 96.065, 96.212, 96.064, 95.962, 96.164, 96.2, 96.138, 96.103, 96.159, 96.217, 96.234, 96.243, 96.242, 96.286, 96.21, 96.271, 96.343, 96.242, 96.305, 96.418, 96.173, 96.467, 96.228, 96.366, 96.399, 96.358, 96.487, 96.332, 96.373, 96.392, 96.565, 96.396, 96.511, 96.444, 96.536, 96.511, 96.509, 96.503, 96.415, 96.608, 96.583, 96.556, 96.527, 96.753, 96.535, 96.661, 96.551, 96.608, 96.619, 96.688, 96.699, 96.631, 96.834, 96.609, 96.756, 96.64, 96.731, 96.74, 96.796, 96.768, 96.812, 96.868, 96.654, 96.752, 96.809, 96.771, 96.681, 96.789, 96.798, 96.709, 96.796, 96.927, 96.969, 96.828, 96.862, 96.784, 96.895, 96.903, 96.885, 96.873, 96.874, 96.793, 97.008, 96.904, 96.941, 96.941, 97.088, 96.879, 96.926, 97.013, 96.949, 96.986, 97.072, 96.979, 96.966, 97.089, 96.978, 97.053, 96.939, 97.102, 97.088, 96.913, 97.086, 97.087, 96.973, 96.988, 97.096, 96.998, 97.09, 97.104, 97.186, 97.143, 97.091, 97.211, 96.948, 97.149, 97.094, 97.147, 97.186, 97.178, 97.062, 97.17, 97.184, 97.04, 97.187, 97.164, 97.133, 97.257, 97.284, 97.13, 97.145, 97.254, 97.208, 97.14, 97.226, 97.226, 97.171, 97.303, 97.15, 97.198, 97.331, 97.244, 97.288, 97.229, 97.195, 97.213, 97.177, 97.395, 97.132, 97.358, 97.211, 97.332, 97.29, 97.279, 97.26, 97.274], 'train_loss': [4.432690776774918, 3.5820252108253583, 3.1561785468823316, 2.856292668322455, 2.6092179566717117, 2.403870567250389, 2.2293090605995096, 2.0629419828178177, 1.9099760180624037, 1.7686979735576411, 1.6374314767919285, 1.5029438031993458, 1.3857995680868815, 1.2711496346285156, 1.1667153113405444, 1.0680276321167375, 0.9822804878632075, 0.8962941103384271, 0.8301092962843443, 0.7678399326247584, 0.7133321324297807, 0.6578151086584849, 0.6247490205523759, 0.5811722157231066, 0.5501807660951267, 0.5277062780721327, 0.49691118386241007, 0.46989198285497613, 0.45262580872134983, 0.43241326839818606, 0.4234222562427103, 0.40718423294395645, 0.38558773651614225, 0.37861547482772584, 0.3719540112035174, 0.35273337562735924, 0.3506732965561532, 0.3316643826224944, 0.3285448852704834, 0.322433668819762, 0.3129987053678002, 0.306162482748429, 0.30070402497760546, 0.2967061069737393, 0.28357329188356856, 0.28828311871358275, 0.28231476364813396, 0.26945423960128007, 0.26516888065052696, 0.2660482048068341, 0.25749743010274356, 0.25633156102362803, 0.24880640409524557, 0.247433307084302, 0.24470292348998599, 0.24499479950699415, 0.23480453350660477, 0.22963825281368602, 0.23069530447929507, 0.23316234727500343, 0.21776578102859068, 0.22838152787020705, 0.21985034951946136, 0.21243555043834147, 0.21643610362986, 0.21038871792860697, 0.21264610952451607, 0.20574310147611666, 0.20960966742079723, 0.20480687982657172, 0.20248388116511917, 0.19843699349840044, 0.19511054496037145, 0.19110025757874863, 0.19883250678702952, 0.18949685681203735, 0.19261038547854867, 0.19197118247341383, 0.18611901872079303, 0.18644503025625256, 0.18299052774279317, 0.18406410220151223, 0.18739099277484916, 0.17852468031410293, 0.18186305733713407, 0.17333703948529253, 0.1784507187812691, 0.17679394633340592, 0.1683607539850253, 0.17369380603637963, 0.1729899009234455, 0.16938024935487156, 0.1749266166268578, 0.16815278574798115, 0.1628184346261691, 0.1641267936150004, 0.15887044440515183, 0.16041366088693285, 0.16780992061875746, 0.15923640107818218, 0.16225237378842236, 0.1590199022174301, 0.15829483240929815, 0.15644257598909922, 0.15292997785795645, 0.15842370931949926, 0.1474813328768703, 0.1565518720782769, 0.152751030674787, 0.1470203735991974, 0.1594888185615987, 0.1524168579298133, 0.147840328117736, 0.1509218392935382, 0.15115691648387444, 0.1449237987247158, 0.14686427386662304, 0.13969591240591048, 0.14625801182258458, 0.1484802054668528, 0.13759226565501312, 0.14386754565972357, 0.14806127782880674, 0.14300342270644453, 0.135388269755776, 0.14563034827566526, 0.1385945567115784, 0.13424991683555398, 0.13880494222332743, 0.140161116752876, 0.14012507715882758, 0.13267548122757275, 0.1395402666069901, 0.13325500057537112, 0.13264749755159913, 0.13453665018777028, 0.13403972106484038, 0.13611079899517323, 0.1329109129337719, 0.1268661880833815, 0.1311197802159275, 0.13840958345812518, 0.1290551818299786, 0.12806421852273175, 0.129245381283777, 0.12399341203900971, 0.13001468796604768, 0.1326531983710294, 0.12677556522644465, 0.12329593956186266, 0.1295717162117648, 0.1279321489962901, 0.12668186231746154, 0.12520050639134694, 0.12297210224915442, 0.12311731844415598, 0.12319443006968508, 0.12299413147566558, 0.12559482308211292, 0.12533086710874652, 0.11933340032155056, 0.12363714375369303, 0.12367901169408652, 0.11772492436564903, 0.12638786185240244, 0.11646383313622423, 0.12617611353351452, 0.11826825170150301, 0.11909788889148745, 0.12187181040316925, 0.11669076928717394, 0.12314364017715519, 0.1177797140153253, 0.12064615582595314, 0.11672835652568983, 0.11846960774773013, 0.11590092067209162, 0.12065982395907154, 0.117019352810508, 0.11614201120333299, 0.11786754050335126, 0.11572326283244402, 0.1175877895875508, 0.11089839867416885, 0.11411651800693771, 0.11503843746530291, 0.11622989864598945, 0.10771243314834351, 0.11657159517460186, 0.11111063078441047, 0.11792307878409912, 0.11220432293366865, 0.11492537287552052, 0.1123487109130532, 0.10921441878788578, 0.11478581053880535, 0.10485537700835164, 0.1139234052154683, 0.10994619098390672, 0.11436192310112514, 0.10807883795627118, 0.11098141509849117, 0.10857815728576092, 0.10963234705640443, 0.11088379514814782, 0.10300859915371448, 0.11257707659503408, 0.10801752488439728, 0.10753352687789768, 0.10933446332027866, 0.11103811979680235, 0.10898834571618615, 0.10762829908462669, 0.10942386734607419, 0.10846845283504022, 0.1054079973616894, 0.10145017260748723, 0.10994141427298795, 0.10610292761403232, 0.10682529741409949, 0.10564919808733213, 0.10174018661140018, 0.10520650426129782, 0.10368314713186937, 0.10627878530858763, 0.10898264427740104, 0.10089412728024445, 0.10480211866105342, 0.10789858961930875, 0.10367485672529395, 0.09950234711917101, 0.1039377463517412, 0.10471931628434354, 0.10043172905640468, 0.10484319324473333, 0.10156612509067015, 0.09725349640041878, 0.10185854564135166, 0.10151702351324435, 0.1015346783039551, 0.10107306285397881, 0.09732881704565738, 0.10194599135586382, 0.09972234061478666, 0.09999868525623765, 0.10392609326839158, 0.09882100546571367, 0.09742081285360038, 0.10158385824706609, 0.10219861796138134, 0.09793236823690703, 0.10181882919994885, 0.10002214344509151, 0.09753780471709493, 0.09552282207253775, 0.09596301579345327, 0.09892750736023954, 0.0965990247178645, 0.10248000309241742, 0.09801687731353434, 0.09926386032616528, 0.09668404783252733, 0.09513285252644671, 0.09527656016547337, 0.09982099075982215, 0.09576601345860414, 0.0967052503364871, 0.10051644168862976, 0.09513410319749507, 0.09757441010342384, 0.098074295945323, 0.09701915545476499, 0.09511150961143863, 0.09846662603571484, 0.09616590322599963, 0.09446652377577992, 0.09387560701243501, 0.09671762023226585, 0.09308064299273905, 0.09429968719605132, 0.09625295210503529, 0.09412064661487272, 0.09695719839478155, 0.09735868105943263, 0.09164915410341097, 0.09480915676763815, 0.09272390801238116, 0.09511792710392365, 0.09677292686035831, 0.0952500402373439, 0.09554411620699359, 0.0881201071020124, 0.09810591182277925, 0.09126800070787135, 0.09679003285157112, 0.08978833376481517, 0.09513576994932353, 0.09311553545283009, 0.09385058781468314, 0.09530905222044889], 'val_metric': [12.68, 21.42, 28.14, 32.24, 35.4, 38.19, 39.5, 41.33, 41.82, 42.55, 41.14, 40.45, 42.44, 41.76, 41.3, 42.01, 41.32, 41.44, 41.37, 39.99, 40.94, 39.9, 40.35, 40.53, 40.51, 40.74, 40.43, 40.38, 40.48, 39.59, 39.92, 40.18, 40.65, 40.62, 41.03, 41.08, 40.67, 40.51, 39.98, 39.65, 40.02, 40.04, 40.5, 41.43, 41.2, 40.9, 40.3, 41.36, 41.43, 40.6, 40.89, 40.83, 40.45, 40.18, 40.15, 40.81, 40.69, 41.5, 40.11, 40.9, 40.38, 42.06, 41.29, 40.32, 41.32, 40.82, 41.58, 41.16, 40.8, 40.44, 41.01, 41.29, 41.05, 40.5, 41.15, 40.9, 41.39, 41.29, 40.89, 40.77, 40.55, 41.41, 40.73, 41.3, 40.67, 40.63, 41.19, 41.38, 41.21, 41.3, 40.95, 41.5, 41.21, 41.6, 41.3, 41.42, 40.92, 40.78, 41.44, 41.73, 41.34, 41.18, 42.09, 41.86, 42.16, 40.94, 41.49, 40.2, 40.82, 41.25, 42.13, 40.99, 41.15, 41.85, 42.22, 41.95, 41.67, 40.64, 41.64, 42.01, 41.76, 42.06, 41.45, 41.88, 41.41, 42.65, 41.73, 41.37, 41.94, 41.54, 41.97, 42.11, 41.59, 41.95, 41.78, 42.18, 41.94, 42.81, 42.08, 40.72, 41.87, 41.36, 41.44, 41.75, 41.98, 42.25, 42.13, 41.34, 41.88, 41.71, 41.38, 42.45, 42.21, 42.34, 41.61, 42.47, 41.45, 41.7, 42.22, 41.71, 42.3, 42.5, 42.84, 42.43, 42.34, 42.52, 41.88, 41.55, 41.64, 42.61, 42.31, 41.72, 42.34, 42.32, 42.12, 41.8, 42.73, 42.51, 42.74, 42.17, 41.98, 42.57, 41.71, 42.01, 42.41, 42.34, 42.38, 41.55, 43.01, 42.72, 42.52, 41.48, 42.34, 41.98, 42.38, 42.2, 42.11, 42.42, 41.81, 41.31, 41.79, 41.02, 41.86, 41.98, 42.08, 42.49, 42.18, 42.02, 42.71, 42.28, 42.48, 42.22, 42.12, 42.49, 42.4, 42.63, 42.23, 43.09, 42.07, 42.86, 42.13, 42.07, 42.17, 42.75, 41.49, 42.45, 42.25, 42.42, 42.44, 42.71, 42.41, 42.02, 42.53, 42.91, 42.46, 42.89, 42.8, 41.67, 41.89, 42.39, 41.66, 42.29, 41.89, 42.19, 42.42, 42.85, 42.14, 41.85, 43.52, 42.66, 42.37, 41.92, 42.24, 43.19, 42.88, 42.66, 41.15, 42.02, 42.93, 42.47, 42.71, 42.23, 43.07, 42.75, 43.08, 42.72, 41.97, 43.19, 42.03, 42.35, 42.7, 41.75, 42.02, 42.35, 43.04, 42.33, 42.2, 42.07, 41.74, 42.92, 42.67, 42.95, 42.52, 42.18, 42.43, 43.21, 42.42, 42.26, 42.09, 42.62, 42.28, 42.11, 42.2, 42.66, 42.41, 42.08, 42.41, 42.75, 42.65, 43.01], 'val_loss': [4.032951245642012, 3.4589457139847384, 3.1422012307841305, 2.878748176204171, 2.753748861088115, 2.6285707312784377, 2.542423452541327, 2.482174826275771, 2.4914700521784985, 2.503233851900526, 2.6267355278039433, 2.7155753017231157, 2.6728606573335685, 2.749579060988821, 2.889314982541807, 2.9715509498195285, 3.084799057359149, 3.1746807546372624, 3.2693512819375203, 3.529282976107992, 3.516051539190256, 3.6263065330541817, 3.6431895608355287, 3.8022642940472644, 3.8416130740171783, 3.96066068615883, 3.9791547788935864, 4.135949577495551, 4.203981567340292, 4.299320950629605, 4.272244528980012, 4.404764812463408, 4.420834463872727, 4.560725999485915, 4.521256261570438, 4.5323298410245565, 4.581218748335626, 4.61609472438788, 4.788983063333354, 4.801594390990628, 4.878895095199536, 4.891331133569122, 4.847448091598073, 4.7576015724498, 4.879426626642799, 4.898512582110751, 5.0270728768816415, 4.887731773838116, 4.977681730962862, 5.090569191677555, 5.061953276585621, 5.094459452446858, 5.141761825342846, 5.2831685937893615, 5.264695784088913, 5.14276704059285, 5.175516675991617, 5.265515271265795, 5.371017430238663, 5.322130232100275, 5.402803191713467, 5.215931287237034, 5.495481986149102, 5.507525117533981, 5.336509698515485, 5.543966378375983, 5.392901407685249, 5.521940644379634, 5.369133063942004, 5.511816476560702, 5.571666992394028, 5.3198845705408955, 5.67660042434741, 5.63615331832011, 5.577255536036886, 5.6916817191300115, 5.5756033142660835, 5.715866237688976, 5.744725698878051, 5.7226068335733595, 5.787381023358387, 5.7693990962520525, 5.923950398803517, 5.993867417050016, 5.786368412576663, 5.874504589730767, 5.797313083508971, 5.757186566188836, 5.8362066852059336, 5.84451954152174, 5.984891548278226, 6.006492528945777, 5.785142035241339, 5.928816134003317, 5.902385890863504, 5.877496605466126, 5.905347708683864, 6.219971857253154, 5.933444068690014, 5.93045514556253, 6.043029826158171, 5.998671637978523, 6.155979962865258, 6.1006991088769995, 5.99646080679195, 6.069530816594506, 6.1076538987979765, 6.18994866662724, 5.974553393710191, 6.197580039880838, 6.071980995737063, 6.029766429002118, 6.00697922554745, 6.166809077475481, 6.011632844900629, 6.043920217046312, 6.100014261379363, 6.315211326453336, 6.32636451037826, 6.133837892751026, 6.066385574401564, 6.028751255600316, 6.294203228251949, 6.085025699275314, 6.35987691059234, 6.070867552119456, 6.267711307592453, 6.182094373520772, 6.2369368114289205, 6.357637085732381, 6.348797057084976, 6.196684321020819, 6.243939360235907, 6.332013484019383, 6.3570807040876645, 6.181125235405697, 6.400250761372269, 6.2256230305714215, 6.3614132085423565, 6.477887814971292, 6.618841657213345, 6.233154173869236, 6.405833879094215, 6.411195644147837, 6.310554852910862, 6.417030493165277, 6.500344182275663, 6.545326289097974, 6.565247710343379, 6.477567769919231, 6.441283658051947, 6.304648349239568, 6.402487120051293, 6.328412165307695, 6.500606179996661, 6.537615856547265, 6.481680537485014, 6.475850515304857, 6.323517366579384, 6.398405336270667, 6.5323574861903095, 6.344124704409557, 6.493419916766464, 6.487072842895605, 6.406344166227207, 6.53096552867039, 6.401824840314829, 6.498389124111005, 6.550944369310026, 6.427482390859325, 6.393996944852695, 6.66564431160119, 6.621169696188277, 6.61084790138682, 6.676016323126046, 6.6141947462300585, 6.572226049034459, 6.396667087154024, 6.419152953822142, 6.575039614537719, 6.616038779544223, 6.4979560413178366, 6.474031335229327, 6.717917638219846, 6.686149935813466, 6.540137169467416, 6.626874566837481, 6.658650471146699, 6.627500276656667, 6.803985015601869, 6.587022318202219, 6.582085484911682, 6.798858756472351, 6.684165673650754, 6.7267194219455595, 6.6778211730301, 6.734478019605017, 6.633013154291044, 6.800335273621188, 6.744650236360586, 6.67949652671814, 6.5566297230447175, 6.919738610079334, 6.651062166614897, 6.766500939988786, 6.842389486397908, 6.724434608866455, 6.954311618379727, 6.9968205819464035, 6.794536040087414, 6.684227536438377, 6.883086004074971, 6.759516637796049, 6.816412268170885, 6.916935847823027, 6.730391268517561, 6.862808115922721, 6.656531684717555, 7.001216273398915, 6.870717545983139, 6.901118633853402, 6.963850595389202, 6.888270104766652, 6.967502185493518, 6.929986569532163, 6.785249956094535, 6.849780595985947, 6.8444794454392355, 7.040690411427978, 6.6658925045827395, 6.917386621426624, 6.913345098495483, 6.845609951930441, 6.715082744883883, 6.846742298952334, 6.805549149300642, 6.806773523616183, 6.862230226492426, 6.755474294067189, 7.083623119220612, 7.033421812543444, 7.229555136079242, 6.987992673922497, 7.016437437883608, 7.1655982892224745, 6.971880648546158, 7.072786874831862, 7.073891917611383, 6.9385978165705495, 7.014609538825455, 7.24990425292094, 7.0814961855578575, 7.050377730351345, 7.067352591806157, 6.9383521763382445, 7.107221516833943, 7.0915853536812365, 7.110601516286279, 7.022271557218709, 7.185006627611294, 6.9482711318192205, 7.043909771427227, 7.136771367613677, 7.208472737081491, 7.103720019577415, 7.091898245416629, 7.348984516350327, 7.130657975081426, 7.081965171607437, 7.079216937350619, 7.050649775820933, 7.085201957423216, 7.499101778504196, 7.122020915815026, 6.997227767470536, 7.250296128023962, 7.143738186283476, 7.366976823017096, 6.959607025620285, 7.305824945686729, 7.061787784479226, 7.071180099134992, 7.255005593512468, 6.95405631308343, 7.199916452359242, 6.987297515960256, 7.167495743484254, 7.154484341099004, 7.129727854090891, 7.321546918647305, 7.204330006222816, 7.163363253235057, 7.266707449202325, 7.114184264164821, 7.067752340037352, 7.302083546188986, 7.298385286027459, 7.245178187728688, 7.182581623648383, 7.046016197295705]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="SGD")
data.update({'train_metric': [0.624, 0.848, 1.083, 1.248, 1.473, 1.779, 2.058, 2.119, 2.322, 2.598, 2.796, 3.002, 3.173, 3.419, 3.606, 3.926, 4.161, 4.411, 4.691, 4.958, 5.247, 5.433, 5.628, 5.933, 6.16, 6.268, 6.637, 6.809, 7.043, 7.317, 7.611, 7.865, 8.05, 8.302, 8.534, 8.977, 9.224, 9.702, 9.901, 10.215, 10.34, 10.821, 11.267, 11.433, 11.716, 12.105, 12.328, 12.629, 12.944, 13.156, 13.546, 13.825, 14.103, 14.187, 14.504, 14.933, 15.199, 15.457, 15.768, 16.044, 16.381, 16.488, 16.906, 17.228, 17.328, 17.861, 17.944, 18.162, 18.613, 18.83, 19.132, 19.39, 19.682, 19.945, 20.225, 20.523, 20.898, 20.958, 21.223, 21.457, 21.767, 21.963, 22.294, 22.604, 22.82, 23.131, 23.498, 23.658, 23.831, 24.228, 24.547, 24.773, 24.995, 25.163, 25.468, 25.616, 26.091, 26.168, 26.507, 26.73, 26.882, 27.084, 27.442, 27.758, 28.018, 28.067, 28.478, 28.675, 28.773, 29.229, 29.286, 29.624, 29.957, 30.036, 30.34, 30.64, 30.737, 31.089, 31.214, 31.453, 31.704, 31.996, 32.267, 32.47, 32.607, 32.916, 33.185, 33.47, 33.69, 33.795, 34.127, 34.424, 34.328, 34.773, 34.964, 35.242, 35.344, 35.6, 35.907, 36.074, 36.449, 36.683, 36.969, 37.173, 37.401, 37.554, 37.786, 38.021, 38.324, 38.66, 38.712, 38.915, 39.259, 39.539, 39.782, 39.901, 40.18, 40.35, 40.605, 40.808, 41.077, 41.312, 41.331, 41.599, 42.104, 42.024, 42.186, 42.621, 42.627, 43.123, 43.287, 43.333, 43.793, 43.75, 44.096, 44.459, 44.896, 44.877, 44.971, 45.304, 45.55, 45.615, 45.971, 46.175, 46.383, 46.482, 47.057, 47.16, 47.014, 47.816, 47.679, 47.925, 48.074, 48.576, 48.545, 48.695, 48.839, 49.297, 49.421, 49.617], 'train_loss': [5.296480697770952, 5.278023167901213, 5.263354977200753, 5.249553046467513, 5.233671873560031, 5.216495063315579, 5.195732127071876, 5.173557309256253, 5.145920870323938, 5.111237644615344, 5.070721763566908, 5.029690319349273, 4.991277768225984, 4.957957979508569, 4.924592606013048, 4.892811678497744, 4.863629054046943, 4.83338618995437, 4.802982228121083, 4.771401527365735, 4.739139889725034, 4.70710384792345, 4.6747882257877675, 4.641650801504261, 4.607147267744927, 4.576982561015045, 4.545991102244255, 4.515447938434603, 4.486797143920293, 4.459922461348013, 4.429396480760434, 4.4058366099459505, 4.378400066306175, 4.356557941711338, 4.328459826327255, 4.302723702145782, 4.27837372634629, 4.252517764566804, 4.228999312840741, 4.204166976900644, 4.181111828455617, 4.158299297075278, 4.133892539976807, 4.108726827288314, 4.089862039130389, 4.0652272301611045, 4.046232861886784, 4.024631185250945, 4.00310918419924, 3.9814695869587355, 3.9603368598379087, 3.943416276347233, 3.921210808091948, 3.9022564862068845, 3.883856201965078, 3.8610083381106133, 3.844281613331, 3.8226418458568845, 3.804639715913466, 3.786799641503635, 3.764303885586202, 3.7454816617800955, 3.728116096934674, 3.7084977876964627, 3.690988524937889, 3.6723784071775256, 3.6503613018028567, 3.6358249581020305, 3.6174178735155826, 3.5972769110918197, 3.5790061310019468, 3.564382563320704, 3.5452698681954002, 3.5257857767191743, 3.5123096329999597, 3.4908283513399248, 3.4764941339681945, 3.4607801330600574, 3.4427274621913466, 3.4262496586114417, 3.406248746586395, 3.3926340140979105, 3.373481770165822, 3.358729859765188, 3.344001843695906, 3.3277857831404596, 3.3094133380431057, 3.2979771100933246, 3.281833077727871, 3.2652009920058003, 3.251274657722322, 3.2357815163149257, 3.219549443122293, 3.2065915512260688, 3.1897174987134, 3.1776062987663773, 3.1625106950944155, 3.1457223880008787, 3.131971576316038, 3.12099643983066, 3.105174943871477, 3.0914384374539208, 3.0770268486008305, 3.062150887625384, 3.0471627727503665, 3.034000485811337, 3.018580141390888, 3.0053678376508386, 2.99582964765362, 2.9775370247304553, 2.9688437956689797, 2.9487783066828284, 2.938204671203213, 2.928308247604663, 2.9102119572713296, 2.899225713958057, 2.8854833816154906, 2.871000061108375, 2.8617495378468636, 2.8448304826452118, 2.8316344362150305, 2.8200945414874465, 2.8069253198168482, 2.7945176299687615, 2.783843895142763, 2.768084823208136, 2.754239236553434, 2.7425974211628743, 2.730349092855716, 2.716560912147517, 2.700324072413771, 2.689986181503218, 2.6778919504608623, 2.6649544541071566, 2.655340180363475, 2.640620296610065, 2.629106221867157, 2.6121805499779134, 2.602683298037133, 2.5905161891621193, 2.573551319687319, 2.565907866163126, 2.5515116906974535, 2.5320404630704942, 2.5235699181059426, 2.513088978412284, 2.502726794738306, 2.4855622408562414, 2.475364402556221, 2.456964362987096, 2.4499344697611805, 2.4355170531526102, 2.4259039930098347, 2.4073976822869563, 2.4002062494908856, 2.3871033971918294, 2.375155118056314, 2.3629343743058864, 2.3521457727490827, 2.3406480566782593, 2.329675800664564, 2.314720519826112, 2.3043317086255786, 2.2943230576036227, 2.275138752703017, 2.2685346289163055, 2.258244915032951, 2.242792845916382, 2.231745432983662, 2.220431071706712, 2.2090253845209964, 2.200545000252019, 2.1869060653032437, 2.1748946393367503, 2.160580359356417, 2.1483195973601923, 2.134864673657213, 2.1293158062893993, 2.117134714614712, 2.102347495459778, 2.0904568438337767, 2.087230735456646, 2.0719624756241335, 2.063882968399819, 2.05142988856603, 2.045613042757592, 2.025608673739418, 2.018900945334578, 2.013333495122381, 1.994746311917491, 1.9862972995980153, 1.9804585471186817, 1.9646178485488404, 1.952367045829026, 1.9474578691611897, 1.9348562878061393, 1.9262919662399927, 1.9176686775661476, 1.9052798756406006, 1.899082676691652], 'val_metric': [0.75, 0.89, 1.21, 1.37, 1.72, 2.05, 2.2, 2.43, 2.68, 3.02, 3.31, 3.41, 3.58, 3.68, 4.04, 4.11, 4.54, 4.98, 5.26, 5.39, 5.77, 6.03, 6.1, 6.4, 6.76, 6.69, 7.38, 7.53, 7.91, 8.26, 8.4, 8.76, 8.96, 9.25, 9.72, 9.83, 10.18, 10.42, 10.86, 10.82, 11.21, 11.56, 11.93, 12.02, 12.38, 12.6, 12.95, 13.33, 13.54, 13.98, 14.44, 14.41, 14.82, 14.95, 15.19, 15.25, 15.31, 15.36, 15.65, 15.95, 16.46, 16.62, 16.91, 17.26, 17.27, 17.35, 17.77, 17.75, 18.17, 18.55, 19.22, 19.29, 19.38, 19.79, 19.94, 19.85, 20.1, 20.43, 20.52, 21.16, 20.97, 21.51, 21.68, 21.89, 21.92, 22.04, 22.5, 22.89, 22.7, 22.73, 23.13, 23.4, 22.98, 23.7, 23.55, 23.72, 23.94, 24.29, 24.03, 23.78, 24.64, 24.58, 24.66, 25.24, 24.85, 25.01, 24.81, 24.96, 24.94, 25.02, 25.4, 25.58, 25.2, 25.39, 25.71, 25.11, 25.53, 25.4, 25.37, 25.94, 25.71, 26.24, 25.8, 25.54, 25.72, 25.83, 26.01, 25.57, 25.84, 26.04, 26.54, 26.45, 25.79, 26.47, 25.64, 26.05, 26.27, 26.85, 26.33, 25.94, 26.93, 26.08, 26.43, 26.44, 26.17, 25.93, 26.26, 26.1, 26.29, 26.56, 25.89, 26.38, 26.3, 26.44, 25.86, 25.76, 26.16, 25.79, 26.04, 25.45, 25.8, 25.99, 25.94, 25.86, 26.33, 25.64, 25.94, 25.4, 25.47, 25.14, 25.24, 25.33, 25.8, 25.09, 25.39, 25.56, 25.44, 25.53, 25.07, 25.53, 24.84, 24.98, 25.45, 24.71, 25.03, 24.26, 25.1, 24.91, 24.65, 23.92, 24.06, 24.01, 25.14, 24.53, 24.54, 24.42, 24.55, 24.27, 23.84, 24.97], 'val_loss': [5.28300966882402, 5.268027196264571, 5.255469310055872, 5.2399674312324285, 5.22371426965021, 5.203355295642925, 5.182650058892123, 5.15968464286464, 5.125078614350337, 5.085903720491252, 5.045759626254914, 4.998395022313306, 4.969114819909357, 4.925684582655597, 4.897391269161443, 4.868459848841285, 4.835127857839986, 4.799812576573366, 4.7670992255970175, 4.736429697389056, 4.697322011753252, 4.662275628679118, 4.62502047210742, 4.595877000480701, 4.556655534513437, 4.52706883211804, 4.491210311841053, 4.462741212480387, 4.431476378896434, 4.402120588691371, 4.379174923441212, 4.347981206930367, 4.3282237948885385, 4.301641921328891, 4.292149586282718, 4.26258972951561, 4.230479155376458, 4.212703566642324, 4.196051975724044, 4.172388792037964, 4.140751086982193, 4.119718304105625, 4.098906823783923, 4.078145493367675, 4.063431589466751, 4.038403933215293, 4.0146777067974115, 3.997329077143578, 3.98025697811394, 3.9560745142068074, 3.940016265128069, 3.9264787670913015, 3.9007628358853093, 3.8855932821893386, 3.8742373718577587, 3.868625882324899, 3.8296935710178057, 3.8292980907828946, 3.8082039234744514, 3.78211282924482, 3.7625813818281624, 3.7586637894818735, 3.724878215486077, 3.7070287610315216, 3.702493596228824, 3.6978473640551233, 3.665390373794896, 3.6735820588032913, 3.6441438835897264, 3.626043770723282, 3.618636448671863, 3.5948653380582285, 3.5866245623606785, 3.569342667889443, 3.559734326259346, 3.5487734253998773, 3.5272811613265116, 3.517072413377701, 3.5048702492076123, 3.4832289507434626, 3.4967003650726025, 3.470956835777137, 3.4512971707969715, 3.4369378704933604, 3.449347911366991, 3.4309062289584213, 3.4222827026039173, 3.411511877539811, 3.3929817646172395, 3.3888960393371095, 3.373669391984393, 3.3609176988054994, 3.377775441309449, 3.348292913406518, 3.3631291814670443, 3.359989936184731, 3.335175110276338, 3.3304042725046727, 3.3258270306192386, 3.3342280441029057, 3.2918161267687562, 3.3051285576668517, 3.2916309955013787, 3.2953345509851055, 3.2877312951786504, 3.277196576640864, 3.2750775221806423, 3.2707878123423098, 3.263672417895809, 3.2766492738845243, 3.257916363940877, 3.2505015962442774, 3.2669988126511784, 3.2652584633250146, 3.2546919303335202, 3.259709620931346, 3.2504838249485966, 3.262342844799066, 3.2592613719830847, 3.233008341424784, 3.2415062394111778, 3.239343707728538, 3.2438168662368874, 3.26413863251923, 3.24041883808792, 3.248497864243331, 3.234111930154691, 3.2647201027839805, 3.2581737102216977, 3.2459961137953837, 3.240866001244563, 3.249548953809556, 3.26386097737938, 3.2568070744253266, 3.2770381269940905, 3.264070854824819, 3.276277128298571, 3.2607476331625773, 3.282260887182442, 3.3208687662319014, 3.293448933370554, 3.309746537238929, 3.3133803704741656, 3.30121941019775, 3.3044238842217024, 3.3239893670294696, 3.3223316866880768, 3.339158112835732, 3.3248858163311223, 3.3574084225733567, 3.361035611219467, 3.342160623544341, 3.345275522037676, 3.352199124682481, 3.375791390989996, 3.3797606617022473, 3.3888952671342594, 3.4267848525077675, 3.3952447411361013, 3.420774445412265, 3.418601909260841, 3.4308682513085142, 3.430699769858342, 3.445096431264452, 3.4207453196215782, 3.4918263740600293, 3.4819459353282953, 3.4864307171220235, 3.50048949915892, 3.5266670034189893, 3.526489806782668, 3.5419643517512425, 3.5132812048978868, 3.5579827947981038, 3.5277648685844083, 3.5287164776188553, 3.543755166090218, 3.5505523438666278, 3.6184305924518854, 3.622439628953387, 3.640684523400228, 3.5879878990209786, 3.6555233874898048, 3.651716875422532, 3.690767976129131, 3.7127540225435975, 3.6720473265192313, 3.7142942490851043, 3.704902405951433, 3.742969232000363, 3.719218517564664, 3.755332041697897, 3.7664316657242503, 3.734223691521177, 3.7689301755018296, 3.7732782174067894, 3.784136963498061, 3.8302043091719318, 3.847154529231369, 3.853148346493958]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="Adam")
data.update({'train_metric': [7.748, 14.006, 17.125, 19.064, 20.69, 22.141, 23.412, 24.351, 25.576, 26.37, 27.331, 27.93, 28.687, 29.426, 29.702, 30.476, 31.0, 31.433, 31.947, 32.223, 32.465, 32.925, 33.167, 33.504, 33.872, 34.157, 34.31, 34.361, 34.78, 34.908, 35.063, 35.345, 35.419, 35.71, 35.744, 35.967, 36.303, 36.358, 36.526, 36.605, 36.758, 36.951, 36.999, 37.076, 37.23, 37.16, 37.354, 37.632, 37.66, 37.903, 37.81, 38.04, 37.92, 38.051, 38.357, 38.296, 38.475, 38.489, 38.417, 38.531, 38.708, 38.735, 38.671, 38.985, 38.833, 38.833, 39.092, 38.995, 39.089, 39.014, 39.197, 39.121, 39.359, 39.285, 39.274, 39.204, 39.321, 39.399, 39.402, 39.567, 39.588, 39.621, 39.694, 39.704, 39.755, 39.877, 39.747, 39.928, 39.978, 39.966, 39.979, 39.978, 40.012, 39.98, 40.116, 40.033, 39.972, 40.191, 40.04, 40.294, 40.278, 40.385, 40.051, 40.417, 40.32, 40.157, 40.325, 40.287, 40.414, 40.503, 40.506, 40.44, 40.588, 40.679, 40.478, 40.653, 40.434, 40.726, 40.622, 40.555, 40.573, 40.715, 40.507, 40.781, 40.824, 40.825, 40.857, 40.713, 40.959, 40.698, 40.787, 40.864, 40.958, 41.041, 40.93, 41.0, 41.003, 40.978, 40.952, 40.929, 40.945, 41.114, 41.163, 41.208, 41.093, 41.123, 41.135, 41.275, 41.005, 41.032, 41.226, 41.081, 41.197, 41.289, 41.023, 41.217, 41.316, 41.576, 41.319, 41.431, 41.371, 41.514, 41.43, 41.405, 41.419, 41.482, 41.455, 41.404, 41.386, 41.304, 41.514, 41.498, 41.405, 41.715, 41.492, 41.375, 41.601, 41.513, 41.569, 41.665, 41.684, 41.707, 41.403, 41.548, 41.59, 41.548, 41.712, 41.61, 41.432, 41.57, 41.823, 41.815, 41.831, 42.026, 41.85, 41.799, 41.815, 41.957, 41.843, 42.022], 'train_loss': [4.364064936140601, 3.830695052979775, 3.624503750261098, 3.503427493595116, 3.4058133803798993, 3.3220349802699367, 3.259058179263495, 3.1961341192534696, 3.14187269552503, 3.0919067243391782, 3.0451552248580747, 3.012367880962174, 2.977435983607804, 2.9451561898507905, 2.9168459135068967, 2.892462816134677, 2.8615281920515416, 2.8403052821498, 2.820158758303788, 2.7977954357462056, 2.782399549289003, 2.768355236858873, 2.7509048450321094, 2.7362670607087862, 2.7226789993730325, 2.7096471591249003, 2.69785467951403, 2.691021122615153, 2.6777148928614816, 2.6697093509209133, 2.65696633868849, 2.648526024833674, 2.641753274434969, 2.632479861235664, 2.6241953678796173, 2.6230782181196157, 2.6070542711519082, 2.5985304201861945, 2.5943084919368773, 2.590335182859893, 2.5831129987195602, 2.57321076894981, 2.5697180481957687, 2.5644562838554994, 2.5605956576836086, 2.554171138975152, 2.547709955211183, 2.544793468137925, 2.536820001352009, 2.5339061291036282, 2.52867047495363, 2.526849539327225, 2.523810334031733, 2.5190543907236305, 2.514971207367329, 2.510711372318134, 2.50532709652235, 2.5072193772687563, 2.5032429338187954, 2.4946192268065284, 2.4933780844365643, 2.4915588715102377, 2.4837475387392636, 2.4832016845124696, 2.4846817660774088, 2.481487664441153, 2.476713234159478, 2.473629991747367, 2.475111983711721, 2.474782105447082, 2.4691576315513117, 2.4693000430261485, 2.463452912109141, 2.4671936823783285, 2.4647013544654968, 2.461676406082402, 2.4579921140933143, 2.45666869748348, 2.4538276625686324, 2.4544810686672793, 2.4489918677988376, 2.4463042021560426, 2.452260785734356, 2.4476870304487184, 2.4448744115811163, 2.4415578035231364, 2.4446976794238586, 2.4352475863882006, 2.4364994385268393, 2.433332150972431, 2.433963780210938, 2.4346217539397403, 2.4318714654377005, 2.4301280080700103, 2.42883619633685, 2.4286388660651785, 2.4276880223401753, 2.4253501279645446, 2.425902086805245, 2.422281005179661, 2.42208146255747, 2.4199049960171193, 2.4211803333773037, 2.41658485019657, 2.4170139414030087, 2.4208616047277713, 2.4175064938997353, 2.419200330610391, 2.4147855888477747, 2.4131459411107343, 2.409784804226417, 2.412308787582627, 2.410441275597839, 2.40612210033036, 2.4095516722704153, 2.4053734838390533, 2.41014655377723, 2.4048674137105714, 2.4069362152713425, 2.402250491802462, 2.4052684072950905, 2.4009074361676674, 2.400708959412285, 2.40420628776172, 2.4011647679145263, 2.3940424165585372, 2.396658283582652, 2.4004558340830444, 2.390893830295106, 2.399092277653768, 2.3974223474013217, 2.3922928797458884, 2.393470319585013, 2.388109462885085, 2.392561865249507, 2.3878339213281583, 2.3850406894909595, 2.388426780471875, 2.386169488736627, 2.3846347523437275, 2.3844951878582448, 2.3864800052923494, 2.382596733855347, 2.380320685762514, 2.382835883401711, 2.3785198454817214, 2.3791896041508904, 2.3787353604707824, 2.3818787235673695, 2.3782040997338614, 2.379396627213203, 2.380826961208595, 2.3770799645993166, 2.3748657661451413, 2.378208081232609, 2.37505450587355, 2.3737954151454983, 2.37256348819513, 2.3712712646064587, 2.3691490585042816, 2.3736552376054605, 2.368207519968122, 2.3665290566796457, 2.3714048159252124, 2.3698585908991054, 2.369315736353283, 2.367443447225916, 2.36973383605137, 2.366075931644867, 2.366110108635476, 2.3670564502458578, 2.3655686061197723, 2.3698534434679144, 2.3660840121737214, 2.3619293093452525, 2.3659268747288222, 2.3606372060336445, 2.3622782024449442, 2.3609975819700586, 2.3603145704007042, 2.357767211193468, 2.3615169027106395, 2.3608454345360217, 2.3606645711629115, 2.3619775945989274, 2.356452332348375, 2.3594237378371195, 2.354962236287879, 2.3585418294197615, 2.3565420726896935, 2.353284157092802, 2.3524021107801167, 2.355694426837367, 2.3535466283388193, 2.352762572367223, 2.3531801820335216, 2.351453909642103, 2.350215947635648, 2.3489882484583693, 2.348822403243926], 'val_metric': [6.88, 14.12, 16.84, 16.11, 20.48, 21.42, 21.26, 23.91, 25.33, 23.81, 25.11, 22.46, 27.77, 25.46, 26.19, 22.86, 29.35, 25.82, 28.91, 28.86, 30.11, 31.09, 30.8, 31.08, 30.43, 32.37, 31.66, 32.26, 32.53, 33.69, 32.04, 32.15, 25.81, 32.92, 32.64, 29.27, 33.69, 32.04, 34.2, 34.12, 32.43, 30.54, 33.44, 34.01, 33.81, 34.08, 35.01, 33.41, 34.84, 33.2, 32.44, 30.64, 34.22, 35.38, 35.33, 34.11, 35.98, 33.77, 32.37, 33.76, 29.48, 34.35, 34.64, 35.3, 35.12, 33.52, 30.84, 35.57, 32.86, 33.78, 34.59, 35.52, 33.73, 35.89, 35.38, 29.5, 35.75, 36.61, 29.64, 35.64, 35.64, 33.5, 35.53, 32.66, 36.32, 34.59, 34.41, 33.46, 34.25, 35.93, 32.65, 35.89, 32.29, 33.75, 35.16, 35.22, 35.7, 30.79, 34.36, 32.62, 36.74, 35.11, 33.94, 35.71, 35.62, 35.82, 34.1, 35.38, 35.21, 32.02, 32.66, 36.84, 36.58, 33.23, 32.23, 36.0, 36.09, 34.24, 36.02, 36.49, 35.5, 32.01, 32.3, 35.55, 35.96, 33.57, 36.35, 35.18, 34.42, 32.83, 35.54, 34.67, 35.13, 31.42, 36.44, 34.02, 35.12, 38.0, 36.1, 34.3, 36.87, 38.57, 36.72, 33.32, 35.04, 35.02, 34.89, 34.3, 36.13, 37.61, 34.27, 32.76, 34.95, 35.1, 38.01, 35.5, 35.74, 36.5, 35.93, 36.39, 35.85, 37.34, 35.58, 38.18, 37.78, 35.57, 35.26, 35.19, 35.05, 35.29, 38.11, 38.33, 37.27, 35.56, 37.29, 37.53, 37.49, 35.02, 34.05, 34.41, 34.04, 36.61, 32.53, 37.08, 34.53, 34.15, 35.51, 36.57, 32.82, 32.6, 35.14, 38.54, 35.67, 34.71, 32.58, 38.11, 36.23, 34.82, 33.5, 33.48], 'val_loss': [4.36469620807915, 3.8370106098758185, 3.6652448245674183, 3.6915954844966814, 3.4089266919786003, 3.391810932736488, 3.322413275955589, 3.2393647083051644, 3.1529837034310506, 3.2644171168090432, 3.1366111753852506, 3.317059864663774, 3.053155697075425, 3.1790307950062355, 3.0866746287437, 3.3769130311953792, 2.9857920301947622, 3.193688750267029, 2.9950187699809954, 2.982816030265419, 2.9196534726270444, 2.8887002255506578, 2.874303743338129, 2.855327199219139, 2.955709614191845, 2.8452876897374537, 2.8555723664107595, 2.823825618264022, 2.840136681392694, 2.762994823941759, 2.830850530581869, 2.8415587081271374, 3.22725271723073, 2.819676522236721, 2.8037985950518567, 3.0383383627909764, 2.797600845622409, 2.887388011832146, 2.73138786197468, 2.7454199859291126, 2.8256370155674637, 2.941810242689339, 2.7970318763878694, 2.7387342794685607, 2.7875470515269383, 2.734746944372821, 2.6799505235283236, 2.779503650726027, 2.706148709461188, 2.776761841622128, 2.839427548608962, 2.9335088327432133, 2.7133623809571477, 2.6547562301538554, 2.666235423771439, 2.740907211592243, 2.6260291053231355, 2.760324838814462, 2.8390263068448207, 2.738556431357268, 3.020394792222673, 2.7100477347708054, 2.744229428327767, 2.693857038856312, 2.7107444405555725, 2.810281792264076, 2.9383616325961555, 2.6948140844417985, 2.8428291845473512, 2.7696805843122445, 2.733084817980505, 2.701622162654901, 2.7961655612204486, 2.6744381823357504, 2.681452871887547, 3.0213058678207885, 2.6635990150415214, 2.6238000962384946, 3.0045163464394347, 2.657332116631186, 2.644462153790103, 2.775226114282183, 2.6565985823892486, 2.8281779760008403, 2.622928116731583, 2.732725177600885, 2.7199999283833107, 2.7777224012241244, 2.7530655033269507, 2.669169093772864, 2.8888598976621203, 2.676512660494276, 2.8409431007257693, 2.7722867059100205, 2.7021505703591995, 2.696378924284771, 2.674523104527953, 2.9677934274551974, 2.7464608204592564, 2.840887985411723, 2.621125468023264, 2.7038822971331844, 2.7982825488801213, 2.7019925026377294, 2.685816376831881, 2.6488914816242874, 2.743019577803885, 2.700860249009102, 2.697678498401763, 2.835740376050305, 2.859067004197722, 2.6232868641804736, 2.595913380574269, 2.781763478449196, 2.8899741370207184, 2.643351781899762, 2.649975194293223, 2.776267833390813, 2.6412473218456194, 2.6295189007072692, 2.681703548522512, 2.9146189849088144, 2.924623925594767, 2.7321687116744413, 2.643497287847434, 2.7984917604239885, 2.6476169380412737, 2.7265564879034736, 2.741722812318498, 2.841631935280599, 2.681319715111119, 2.7524489627522266, 2.7232508750478175, 2.8721909534399677, 2.690518156738038, 2.7778387901129995, 2.7087213955107767, 2.573147071394951, 2.659779705819051, 2.7360099884355145, 2.612881748539627, 2.552528508909189, 2.631593215237757, 2.813689999139992, 2.711067552019836, 2.7266518864662026, 2.7252709379621374, 2.739603714958118, 2.6576769078613087, 2.5794280304270947, 2.7328984118570947, 2.854279485098116, 2.7387790998835473, 2.754992773578425, 2.5814617614077915, 2.6526158941779165, 2.6720561745819773, 2.6279076277070743, 2.6863633018390387, 2.6362929306212504, 2.694910880866324, 2.6131393446284497, 2.677429883343399, 2.5421764911360043, 2.5729514648959895, 2.657916821491946, 2.678762190660853, 2.722963421967379, 2.7074481332378024, 2.7212097622027063, 2.567478895946673, 2.5508044122890303, 2.617517172530958, 2.7051922348654194, 2.5877834436999763, 2.569533880728825, 2.5993533491329024, 2.70909087141608, 2.77629978337865, 2.7260662195788825, 2.743446958672469, 2.640366107415242, 2.859194700125676, 2.599671056316157, 2.74261378558578, 2.7497503920725195, 2.6614185575467006, 2.6726953022798914, 2.825704074209663, 2.896457998615921, 2.7136885823717543, 2.535355818879073, 2.6611173878050156, 2.7129657617799796, 2.80499199621237, 2.590701045124394, 2.660086717575219, 2.764666044787996, 2.7964879175659956, 2.7901617068393976]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="AdamW")
data.update({'train_metric': [9.875, 21.698, 28.752, 34.219, 38.652, 42.538, 46.037, 49.221, 52.251, 55.016, 57.788, 60.335, 62.733, 65.44, 67.945, 70.01, 71.888, 73.995, 75.595, 77.392, 78.684, 80.136, 81.339, 82.173, 83.364, 84.016, 84.759, 85.464, 86.004, 86.76, 87.14, 87.582, 88.032, 88.459, 88.621, 89.36, 89.404, 89.75, 90.003, 90.085, 90.531, 90.597, 90.634, 91.115, 91.11, 91.499, 91.351, 91.864, 91.913, 91.853, 92.054, 92.169, 92.598, 92.457, 92.565, 92.579, 92.802, 92.841, 93.124, 93.018, 93.248, 93.379, 93.427, 93.507, 93.507, 93.695, 93.728, 93.624, 93.787, 93.974, 94.032, 93.927, 94.072, 94.156, 94.172, 94.365, 94.223, 94.444, 94.492, 94.343, 94.604, 94.59, 94.518, 94.787, 94.735, 94.548, 94.659, 94.89, 94.917, 94.7, 95.058, 95.101, 95.0, 95.177, 95.054, 95.1, 95.043, 95.33, 95.279, 95.282, 95.402, 95.501, 95.269, 95.522, 95.487, 95.627, 95.227, 95.663, 95.378, 95.704, 95.581, 95.544, 95.699, 95.608, 95.717, 95.755, 95.758, 95.848, 95.661, 95.719, 95.915, 95.819, 95.885, 95.873, 95.937, 95.84, 96.004, 95.925, 95.965, 96.065, 95.988, 96.009, 96.045, 96.035, 96.048, 96.115, 96.165, 95.978, 96.282, 96.163, 96.145, 96.258, 96.209, 96.156, 96.367, 96.203, 96.211, 96.37, 96.284, 96.285, 96.276, 96.434, 96.249, 96.37, 96.389, 96.446, 96.405, 96.342, 96.474, 96.457, 96.409, 96.507, 96.512, 96.406, 96.493, 96.52, 96.518, 96.5, 96.658, 96.555, 96.543, 96.416, 96.681, 96.709, 96.562, 96.533, 96.686, 96.957, 96.429, 96.617, 96.808, 96.625, 96.697, 96.822, 96.748, 96.791, 96.674, 96.761, 96.823, 96.676, 96.745, 96.824, 96.773, 96.885, 96.801, 96.896, 96.867, 96.709, 96.784, 96.781], 'train_loss': [4.23848057433877, 3.407828077016087, 3.0223478874333454, 2.735709966640021, 2.513132331848755, 2.327662299324592, 2.154869773642649, 2.001683929831419, 1.860966326026526, 1.729659357020585, 1.6007073340092572, 1.4790261940397822, 1.3660279512405396, 1.2505822953549395, 1.1456660393789955, 1.047384290037747, 0.9712206675925471, 0.8822170730706445, 0.8179674610562303, 0.7482599654757511, 0.6978378505067648, 0.6441393981968373, 0.6016889611021953, 0.5690425514068957, 0.5293159483433227, 0.504965609457923, 0.4803292436223723, 0.4550077450471815, 0.43996878194736505, 0.4125980496196814, 0.4049982105835233, 0.3893205929483196, 0.373588576073915, 0.3615302573019544, 0.3553738496740011, 0.33421388064211405, 0.33494462987130375, 0.32387804577004353, 0.31253763443658694, 0.3124911988098043, 0.2980688638432203, 0.29543073166230893, 0.29197692649554574, 0.27877627683521006, 0.2777916467983945, 0.26857591700106764, 0.27205021045410854, 0.25622741459861104, 0.2531280706106892, 0.25578285024876174, 0.24840984380996464, 0.24678660310228057, 0.23497945915309382, 0.23687699286151565, 0.23360340789994863, 0.23209454945061578, 0.226101474331183, 0.22258150601260226, 0.21536386739855878, 0.22377648416808874, 0.21583240645593813, 0.2116712743246014, 0.2070170788145645, 0.20457847715997193, 0.20644833049209585, 0.20293910083046954, 0.19903314937447136, 0.20159492980133473, 0.19806090134993834, 0.19316797122902696, 0.1907422753999526, 0.19371879371854353, 0.18833521248978907, 0.18604923691123876, 0.18455446264286474, 0.177141628339553, 0.18502358239148853, 0.17558378052122542, 0.17652206175757412, 0.17703578210485263, 0.1720313593165464, 0.17318319469202914, 0.17426920377471206, 0.166245045039686, 0.1680646668678663, 0.17062261754942443, 0.17023301023098517, 0.1625432878928122, 0.16381530113242954, 0.16993079105033393, 0.1578431460968864, 0.158190206473816, 0.16148893824723082, 0.1571248735832104, 0.1596293036023933, 0.15697825587189548, 0.16090945996223727, 0.1509864739176918, 0.15451160462046434, 0.15340527044001417, 0.14912133693476984, 0.14630756568984724, 0.15304743341734528, 0.14722395134194183, 0.14795284133293665, 0.14176446122521927, 0.15506263752810548, 0.14039759720277617, 0.15106793213963404, 0.13938047604767043, 0.1441920241150321, 0.14650899782208981, 0.13978307964283226, 0.1423806997347666, 0.14072852048850712, 0.13652698194112936, 0.13706736685335755, 0.13589045789536097, 0.14148480192027027, 0.13900403225611507, 0.13121920544065724, 0.13850413362664823, 0.13426342881770276, 0.13386310833525353, 0.13181905248281334, 0.13799719239188263, 0.12811863307751148, 0.13185062473943732, 0.13243292780519747, 0.12755770647304843, 0.13053123912936695, 0.12800937784720695, 0.13092028300718225, 0.13011244756444604, 0.13070076298024041, 0.12698803907559716, 0.12517317494819022, 0.1320465129423411, 0.12282970656389973, 0.1264530923461836, 0.12613297961180064, 0.12445870118168906, 0.12360658885801469, 0.12549512567061463, 0.12032491458259327, 0.12679390032267435, 0.12487599634637214, 0.12075859950791319, 0.12293789332417648, 0.12168982163353473, 0.12374430803097343, 0.11762841024591873, 0.12247408366255923, 0.12108816242221473, 0.11720691959347436, 0.1190069312781217, 0.11976460185958406, 0.12011060498339635, 0.11595430536230196, 0.11712268807983603, 0.11973300527200885, 0.11696809551783469, 0.11636266836098688, 0.11764638336587331, 0.11628442025214306, 0.11576474154182136, 0.1143306918104548, 0.1156033053307455, 0.10951740696026109, 0.11438665691997334, 0.11392706015828928, 0.12004502668725929, 0.10821291907966496, 0.11050554345837381, 0.11347467546663781, 0.11521353434346156, 0.11167254061778525, 0.10351983099102274, 0.11665156043573477, 0.11254439713440535, 0.10587303368473766, 0.11413491478261925, 0.10881373035239285, 0.10637988168658344, 0.10777736285662515, 0.10905545416954096, 0.10882268232121076, 0.10976767611640856, 0.10442139301098995, 0.11271429846858873, 0.10873626735171939, 0.1057204124880466, 0.10795287101309668, 0.10511167010780119, 0.10470773609605277, 0.1049249447665135, 0.10525192447528522, 0.10858238737129187, 0.10782880726681816, 0.105573165505217], 'val_metric': [14.3, 23.26, 29.88, 34.34, 36.02, 38.53, 39.62, 42.97, 43.29, 42.51, 43.18, 42.89, 43.4, 42.63, 43.14, 42.85, 41.97, 41.91, 42.21, 42.58, 41.87, 41.64, 41.51, 41.64, 40.86, 41.03, 41.64, 41.79, 40.98, 41.48, 41.45, 42.02, 41.19, 41.04, 41.8, 41.54, 42.12, 42.45, 42.65, 41.36, 42.22, 41.61, 42.04, 41.55, 42.34, 42.45, 41.78, 41.08, 41.66, 41.56, 42.39, 42.01, 41.42, 42.23, 42.24, 42.16, 41.54, 41.84, 41.82, 41.62, 42.28, 42.01, 41.51, 42.43, 41.77, 41.98, 42.13, 42.3, 42.04, 41.96, 42.21, 41.67, 42.36, 41.64, 42.4, 42.57, 42.52, 41.93, 42.46, 41.65, 42.12, 42.33, 42.51, 42.55, 42.97, 42.75, 43.27, 42.7, 42.13, 42.75, 42.16, 42.21, 42.18, 43.14, 42.32, 42.28, 42.73, 43.0, 42.7, 42.39, 42.6, 42.57, 42.87, 43.25, 42.93, 43.56, 42.38, 43.76, 42.78, 43.29, 43.31, 43.31, 43.47, 43.61, 43.75, 43.52, 43.46, 42.54, 42.56, 42.96, 42.63, 42.45, 42.35, 42.23, 42.15, 42.48, 42.79, 43.45, 42.94, 43.25, 43.15, 42.91, 43.7, 42.85, 43.75, 43.5, 43.05, 43.4, 43.61, 42.9, 43.58, 43.14, 43.33, 43.57, 43.3, 44.21, 43.48, 42.9, 43.08, 43.42, 43.2, 43.58, 43.16, 42.98, 42.96, 42.9, 43.94, 43.28, 43.04, 42.72, 43.35, 43.44, 44.13, 43.46, 42.92, 43.06, 43.09, 43.76, 42.66, 43.77, 43.69, 43.78, 44.14, 42.91, 42.97, 43.78, 44.45, 42.77, 42.81, 43.5, 43.79, 43.92, 43.45, 43.62, 43.0, 43.61, 43.54, 43.32, 43.73, 44.01, 43.68, 43.87, 44.28, 43.09, 44.05, 43.51, 43.44, 43.54, 43.82, 43.9], 'val_loss': [3.9866400714133197, 3.386986916991556, 2.9728506416272205, 2.7680228963779037, 2.6739691070690275, 2.564203642356168, 2.5487286653488304, 2.470873956847343, 2.4481157333987533, 2.4806939697569343, 2.4890164987296814, 2.5589178508254373, 2.583594423570451, 2.677702862745637, 2.734950051186191, 2.863713232574949, 3.048569094603229, 3.070117585977931, 3.1751862976961074, 3.2678309549951248, 3.3885103851366956, 3.5056980628116876, 3.5733634719423426, 3.6499858563113365, 3.7928000529100943, 3.8251223974167163, 3.904664013795792, 4.055536956544135, 4.073138961366787, 4.134418634092732, 4.2330916277162585, 4.201004573494006, 4.210005500514036, 4.3989510650088075, 4.3177719078246195, 4.503636389780956, 4.403344493762702, 4.460928109041445, 4.472502196670338, 4.578784006416418, 4.565806763187336, 4.642946105853767, 4.574929293553541, 4.604582351484116, 4.695187677243712, 4.6859173334328235, 4.8103120448483025, 4.786178243388036, 4.877762852201037, 4.823440874458119, 4.863043123749411, 4.92240831730472, 4.967280873827114, 4.953565255092208, 4.991817434881903, 4.976175380360549, 4.9745632425235335, 5.073081688516459, 5.0470655457988665, 5.072305939759419, 5.130925642456978, 5.150183655653789, 5.21126021776989, 5.208947564386259, 5.120805836786889, 5.157800921209299, 5.273505936003035, 5.252247880218895, 5.163836177746961, 5.315778975274153, 5.3223317109855115, 5.316577773944587, 5.320120456112418, 5.4925841138621045, 5.361494642913721, 5.26091704702681, 5.307628683223846, 5.485339392522338, 5.379878315196675, 5.710459471508196, 5.421171122295841, 5.446454049675328, 5.515248316868096, 5.577185914774609, 5.426968061240616, 5.366656440078833, 5.520136343445747, 5.662683113365416, 5.611465943087438, 5.472155211837428, 5.656962727285494, 5.693085818533685, 5.691676514923193, 5.561540853445697, 5.787745957921265, 5.664478314150671, 5.637672873059655, 5.6096474440993775, 5.6855141433181275, 5.673418378374379, 5.800436066214446, 5.826770398267515, 5.796966701556164, 5.634288462104311, 5.731666719837553, 5.859946210672901, 5.682963781296068, 5.760619411043301, 5.92288481809531, 5.795606716423277, 5.8272644000448235, 5.828866979119125, 5.742311151164352, 5.826258862094515, 5.892234524344183, 5.807381046805412, 5.752416354835413, 5.932650592676394, 6.006174646365415, 5.831300863035166, 6.2412890309740785, 5.885063295911072, 5.998996235003137, 6.06897178728869, 6.074237206179625, 5.9287325462717915, 5.979668272528679, 5.932643265481207, 6.027499101723835, 5.950945059964611, 6.066115226715234, 6.107657903318952, 6.05067466094995, 6.138110366596538, 6.121564019257855, 5.9342516926443505, 6.181037106331746, 6.022735328431342, 6.008819241432628, 6.272149400346598, 6.200730882632505, 6.166832168390797, 6.0750728838003365, 6.128942080364106, 6.336019634441206, 6.156303970677079, 6.107595663921089, 5.987765760178778, 6.250767159613834, 6.049609814479852, 5.970883867543215, 6.22972499033448, 6.291452094248146, 6.206082123859673, 6.454173703861844, 6.343724064006927, 6.2788874298144295, 6.189330837529177, 6.189149362266443, 6.2578388209555555, 6.142503749792743, 6.2283303107425665, 6.141380196164368, 6.249970467986574, 6.448362361093995, 6.100985653081517, 6.506933592686988, 6.4847695994529, 6.6095500706107755, 6.442257611614884, 6.235028271462507, 6.147798767515049, 6.385405077296458, 6.267453438157489, 6.3654210210605795, 6.2854332589799435, 6.339219968030407, 6.376389770750787, 6.751993558968708, 6.357418182549203, 6.308377311487866, 6.3814003467559814, 6.385440535606093, 6.383465627955783, 6.5402957466757226, 6.529911237157834, 6.369987220521185, 6.428689177628535, 6.631207640763301, 6.503623834840811, 6.499632435999099, 6.7682934627411475, 6.543969008573301, 6.566774391824273, 6.638623649147665, 6.511174514035511, 6.518360784858655, 6.569250996705073, 6.3541373158716095, 6.372286963614688]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="SGD")
data.update({'train_metric': [1.195, 1.396, 1.567, 1.747, 1.834, 1.965, 2.157, 2.372, 2.497, 2.675, 2.907, 3.058, 3.342, 3.526, 3.784, 4.082, 4.299, 4.662, 4.923, 5.128, 5.459, 5.72, 5.965, 6.198, 6.606, 6.88, 7.044, 7.337, 7.585, 7.915, 8.14, 8.319, 8.612, 8.853, 8.978, 9.281, 9.494, 9.83, 10.044, 10.397, 10.599, 10.896, 10.966, 11.223, 11.683, 11.745, 12.031, 12.262, 12.427, 12.599, 12.748, 13.104, 13.203, 13.443, 13.64, 13.746, 14.072, 14.143, 14.353, 14.625, 14.817, 15.109, 15.111, 15.286, 15.543, 15.604, 15.861, 15.959, 16.117, 16.35, 16.476, 16.583, 16.764, 16.918, 16.925, 17.172, 17.387, 17.513, 17.706, 17.811, 17.984, 18.16, 18.303, 18.432, 18.405, 18.639, 18.807, 18.796, 19.108, 19.171, 19.364, 19.492, 19.572, 19.789, 19.908, 19.989, 20.095, 20.262, 20.471, 20.519, 20.797, 20.767, 20.894, 21.254, 21.178, 21.494, 21.491, 21.693, 21.74, 21.664, 22.045, 22.049, 22.338, 22.449, 22.566, 22.652, 22.83, 22.901, 22.909, 23.013, 23.117, 23.348, 23.497, 23.692, 23.591, 24.006, 24.133, 24.098, 24.287, 24.481, 24.406, 24.465, 24.847, 24.947, 25.05, 25.109, 25.217, 25.498, 25.466, 25.55, 25.827, 25.831, 26.051, 26.14, 26.144, 26.253, 26.485, 26.427, 26.675, 26.656, 26.681, 26.838, 27.054, 27.119, 27.267, 27.373, 27.498, 27.739, 27.904, 27.789, 27.87, 28.038, 28.038, 28.22, 28.346, 28.446, 28.519, 28.694, 28.729, 29.007, 29.051, 29.174, 29.507, 29.388, 29.337, 29.582, 29.756, 29.612, 29.874, 29.863, 30.07, 30.358, 30.326, 30.364, 30.414, 30.604, 30.527, 30.592, 30.886, 31.026, 31.057, 31.278, 31.319, 31.487, 31.455, 31.701, 31.781, 31.828, 31.83, 32.049], 'train_loss': [5.2644633636669855, 5.241461519545191, 5.222793920789098, 5.205168997105009, 5.187068246986648, 5.16731987316793, 5.141602841692709, 5.1144670604210365, 5.087152442639254, 5.057866756075556, 5.027425676298233, 4.9974039355990065, 4.963927487417894, 4.93263008711968, 4.902551313889614, 4.875191670232908, 4.84775061311435, 4.8220397307944465, 4.794451076253743, 4.7674082858548745, 4.738246661458958, 4.708516934249009, 4.681033865122633, 4.652776339842735, 4.623436144354705, 4.593984089825143, 4.564704593602313, 4.53570634252508, 4.510179109933357, 4.482445365407874, 4.456987375795117, 4.433092388676591, 4.410546385166512, 4.385744761596944, 4.3628011356922425, 4.339851205499982, 4.320291038056786, 4.298099800934795, 4.276923167118261, 4.2549122403389505, 4.234799252666881, 4.2139237691253255, 4.195228107907569, 4.1772606320969965, 4.156880641090359, 4.139421348455848, 4.11715578849851, 4.102849398678263, 4.08520465604937, 4.069914537138155, 4.054521472806436, 4.03942726135864, 4.0227294921569925, 4.009173048968813, 3.994158856775695, 3.9816975142051225, 3.9680444661883003, 3.9540661879250885, 3.9399652737344755, 3.9268272272379674, 3.9129465762728386, 3.899319266479746, 3.8908377343542058, 3.877861351670932, 3.867039567754578, 3.8530574735737884, 3.8420106313660294, 3.8317810299300414, 3.820978473869563, 3.809836342787788, 3.796574297427216, 3.7876552855137136, 3.7773059598162475, 3.764443376776658, 3.754932551679898, 3.7467088435448215, 3.737174089795416, 3.7254307088528544, 3.7158673110102813, 3.7063240922572747, 3.696401795285372, 3.688062506307796, 3.675967801631603, 3.6677625365388447, 3.6574207723560197, 3.651299609332533, 3.640985607109387, 3.6317482559023495, 3.6220202885296433, 3.614313560499263, 3.6042965105231266, 3.5948854115096256, 3.5868726484453686, 3.574372618761264, 3.5662230707023364, 3.559132770323555, 3.553935129827059, 3.5464354359180748, 3.5345317409805816, 3.5243468397485853, 3.5175297040628153, 3.5105933455115164, 3.5013933669887747, 3.4949355082716305, 3.4846879851718935, 3.4755396148903737, 3.467524996035692, 3.4619327091819874, 3.4517913677718344, 3.4472374281514093, 3.4378692810915252, 3.431450374448292, 3.420063416925822, 3.413443542792869, 3.406240707624439, 3.4009105366007, 3.392631297071851, 3.3823018246404803, 3.3751538888964223, 3.3713390839382082, 3.3607662152725384, 3.3526189498083756, 3.3454200336739413, 3.338192553681894, 3.332863528684249, 3.324332830163049, 3.317278740883484, 3.309785708813658, 3.2992755974704306, 3.2921176717896072, 3.2903719453268607, 3.2821917269981906, 3.274445436890127, 3.2667006791896416, 3.259129565416508, 3.2531364127297624, 3.2435287221150757, 3.239472686519855, 3.2298718409437592, 3.227181729794464, 3.217473035581739, 3.211327182888146, 3.2035433487944016, 3.1977572114667447, 3.193171200047528, 3.1868434997224226, 3.178256417114004, 3.170987637319095, 3.1659312199417475, 3.1590758790133933, 3.1529030243098086, 3.147208798946056, 3.140701579651006, 3.132798437269849, 3.1284542336955106, 3.121824226696676, 3.1173394215999317, 3.111654347360554, 3.1025365018264957, 3.0968831477833345, 3.087586636277856, 3.0831231782623996, 3.0798185356747854, 3.076369066384841, 3.066965542461959, 3.062342689194438, 3.0541788971889345, 3.049003866797293, 3.043862932703088, 3.0330642921682056, 3.0299773930130445, 3.0245046138153273, 3.0166717294081615, 3.013993138467663, 3.0059456266963323, 3.0015099524536426, 2.9953303695564, 2.9915527272361713, 2.9840122751905915, 2.980195923486125, 2.971622155098601, 2.9650616153111766, 2.9614052623033675, 2.953372405571428, 2.9511625235918157, 2.9444024881642368, 2.9384749596193673, 2.933646060378599, 2.9283877013969786, 2.921840338850357, 2.919387638301935, 2.9097512358667297, 2.9074853114874335, 2.8984381967985087, 2.895292590385969, 2.886316439164272, 2.8848774934989554, 2.877573081490632, 2.874914628759227, 2.866818937672611], 'val_metric': [1.2, 1.51, 1.76, 1.78, 1.91, 2.03, 2.15, 2.39, 2.76, 2.78, 3.1, 3.2, 3.46, 3.61, 3.73, 4.29, 4.46, 5.04, 5.41, 5.83, 5.76, 6.2, 6.46, 6.52, 6.79, 7.23, 7.34, 7.39, 7.82, 8.13, 8.05, 8.44, 8.76, 9.07, 9.23, 9.44, 9.71, 10.05, 9.96, 10.28, 10.6, 10.81, 11.08, 11.42, 11.48, 12.01, 11.97, 12.17, 12.61, 12.58, 12.86, 13.05, 13.24, 13.07, 13.4, 13.55, 13.72, 13.94, 14.08, 14.26, 14.34, 14.36, 14.4, 14.83, 14.94, 15.0, 15.22, 15.49, 15.67, 15.81, 15.79, 16.19, 16.39, 16.33, 16.43, 16.73, 16.81, 16.99, 16.86, 17.09, 17.54, 17.65, 17.68, 17.83, 17.63, 17.81, 17.95, 18.22, 18.63, 18.37, 18.44, 18.85, 18.73, 18.99, 18.99, 19.43, 19.38, 19.54, 19.63, 19.42, 19.92, 19.88, 19.97, 20.02, 20.1, 20.44, 20.51, 20.65, 20.58, 20.77, 20.81, 20.96, 20.98, 21.28, 21.3, 21.58, 21.77, 21.89, 21.57, 21.85, 22.0, 22.25, 22.17, 22.21, 22.54, 22.46, 23.03, 23.08, 22.87, 23.13, 23.25, 23.08, 23.3, 23.49, 23.63, 23.9, 23.6, 23.75, 23.79, 23.89, 24.16, 24.28, 24.44, 24.46, 24.8, 24.37, 24.72, 24.77, 24.95, 25.05, 25.09, 25.34, 25.11, 25.49, 25.33, 25.47, 25.4, 25.42, 25.89, 25.64, 25.82, 25.97, 26.07, 26.39, 26.21, 26.32, 26.39, 26.47, 26.6, 26.52, 26.35, 26.85, 26.75, 26.86, 27.0, 27.12, 27.14, 27.05, 26.91, 27.24, 27.21, 27.28, 27.5, 27.2, 27.37, 27.87, 27.95, 28.0, 27.84, 27.94, 27.96, 27.92, 28.1, 28.14, 28.4, 28.02, 28.3, 28.58, 28.47, 28.87], 'val_loss': [5.258648213307569, 5.2372068994364165, 5.223501621537907, 5.199984170828655, 5.182749389842817, 5.159461057869492, 5.137049910369193, 5.10571733857416, 5.07664390126611, 5.047640762511333, 5.017655287578607, 4.988939592033435, 4.95137975929649, 4.919780822316552, 4.89206431170178, 4.863032105622018, 4.837710778424694, 4.807710366643918, 4.7793836259538205, 4.751959735420859, 4.721226994399053, 4.692876401220917, 4.662251226461617, 4.636252125357367, 4.605458343104952, 4.576628440504621, 4.547651577907003, 4.5214493456919485, 4.495291998431941, 4.470334456984404, 4.445894382561848, 4.422032102657731, 4.39904930485282, 4.375230409537151, 4.350710733680968, 4.332445030759095, 4.312624729363022, 4.294820657960928, 4.273396153358897, 4.251460915158509, 4.23383472375809, 4.21364913624563, 4.196336753808769, 4.1770228549932975, 4.162087557422128, 4.141546414915923, 4.128130247638484, 4.111120205775947, 4.094561617845183, 4.079019734813909, 4.06359071792311, 4.049332992286439, 4.035269293815467, 4.020888352849681, 4.0099440152478065, 3.9956504812665807, 3.983188378583094, 3.971847265389315, 3.957734100378243, 3.9494179661866204, 3.934398880430088, 3.923343983425456, 3.911509652046641, 3.903976596844424, 3.890368655988365, 3.8790682698511016, 3.866678120983634, 3.8568021157744585, 3.846950316884715, 3.8349048742063485, 3.8257567912909636, 3.8131945011722053, 3.8039589307870076, 3.794584125470204, 3.7868313075630526, 3.7740994456467356, 3.770181590584433, 3.758870138484202, 3.7503735214281995, 3.7377478301904765, 3.732720437323212, 3.7213928714679305, 3.712988041009113, 3.706443578574308, 3.6985695787296176, 3.6894344994976263, 3.677764746793516, 3.6795918182202967, 3.663935942255008, 3.656191106055193, 3.6460478852508933, 3.6384395186308844, 3.63282224327136, 3.625237788364386, 3.6175397778772247, 3.611251706530334, 3.603778240786996, 3.5951057801580735, 3.5864357614213493, 3.5789381789553696, 3.571635914456313, 3.5631411485611255, 3.5553508594537235, 3.5485646314681714, 3.5402870648985454, 3.5321989871893718, 3.5249871524276246, 3.5203250213793127, 3.5182432794267204, 3.507601495001726, 3.504718926302187, 3.4945379229867535, 3.4901940139235963, 3.4787856575789724, 3.476216417968653, 3.4736898780628374, 3.466346405114338, 3.4565167275204023, 3.4534291279543736, 3.4450832734442063, 3.440765086252978, 3.4273172578993876, 3.4289954361642243, 3.4141304401835058, 3.4142780152096113, 3.412600556756281, 3.3996038922838343, 3.3922057516255957, 3.3938606385212795, 3.3848341323767497, 3.3763710829862363, 3.3714491880623396, 3.3628569028939412, 3.359137708214438, 3.357411620723214, 3.3475799226457146, 3.34556149753036, 3.339216479070627, 3.3356307213473473, 3.324233845540672, 3.32067571476007, 3.318797290704812, 3.311823794796209, 3.3085714107865742, 3.2983197224367955, 3.293769692159762, 3.295345130240082, 3.288322519344889, 3.285198131184669, 3.2776779065466233, 3.2739805172962746, 3.270548912370281, 3.260323574588557, 3.2572530120801013, 3.251463293269941, 3.253115016184035, 3.2464257114252466, 3.2436946820301613, 3.2397339966646426, 3.2317580371905286, 3.231579706167719, 3.2248147642536527, 3.219192141180585, 3.220301361600305, 3.210978514829259, 3.215791595209936, 3.2018889431740827, 3.203500870686428, 3.194883332131015, 3.1897204324697994, 3.197343696454528, 3.1806492820666854, 3.1776282658242874, 3.1755231763147247, 3.1744642105831464, 3.1690135488084925, 3.166721515594774, 3.165412084312196, 3.1652129668339044, 3.151883640107076, 3.151325114973032, 3.1486529704112156, 3.1416178759495925, 3.1499983102652678, 3.143145574126274, 3.1323016228949188, 3.1276449238418773, 3.124761367299754, 3.129611444321408, 3.1199171049579695, 3.1191680909721713, 3.1189049178627646, 3.1164008546027406, 3.108488312192783, 3.104143879975483, 3.1172519428714827, 3.1006708137548653, 3.0927364203580625, 3.094061255455017, 3.097321428311099]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="Adam")
data.update({'train_metric': [8.723, 16.281, 19.941, 22.379, 24.162, 25.636, 26.886, 28.054, 29.012, 29.892, 30.581, 31.375, 31.95, 32.543, 33.004, 33.652, 34.029, 34.642, 34.949, 35.199, 35.61, 36.265, 36.325, 36.667, 36.843, 37.082, 37.595, 37.669, 37.91, 38.15, 38.301, 38.59, 38.84, 38.813, 39.082, 39.227, 39.272, 39.46, 39.475, 39.766, 39.932, 39.987, 40.137, 40.186, 40.586, 40.518, 40.536, 40.736, 40.68, 40.889, 40.859, 40.896, 41.038, 41.227, 41.245, 41.287, 41.212, 41.42, 41.421, 41.604, 41.516, 41.687, 41.719, 41.574, 41.907, 41.668, 41.894, 41.99, 41.921, 41.998, 42.169, 42.112, 42.243, 42.314, 42.254, 42.447, 42.393, 42.285, 42.363, 42.39, 42.518, 42.643, 42.562, 42.578, 42.696, 42.776, 42.816, 42.849, 42.811, 43.031, 42.928, 42.88, 42.914, 42.867, 43.107, 42.907, 43.173, 43.147, 43.115, 43.231, 43.001, 43.115, 43.169, 43.398, 43.257, 42.997, 43.224, 43.201, 43.276, 43.249, 43.371, 43.399, 43.44, 43.232, 43.53, 43.453, 43.462, 43.295, 43.487, 43.526, 43.501, 43.394, 43.541, 43.676, 43.5, 43.633, 43.684, 43.643, 43.518, 43.598, 43.483, 43.625, 43.736, 43.682, 43.884, 43.797, 43.68, 43.824, 43.745, 43.78, 43.601, 43.79, 43.792, 43.928, 44.082, 43.839, 43.682, 43.914, 43.876, 43.83, 44.076, 43.94, 43.881, 43.851, 43.986, 44.148, 44.05, 43.874, 43.957, 43.996, 44.118, 44.049, 44.079, 44.011, 44.051, 43.906, 44.196, 43.965, 44.023, 43.986, 43.982, 44.147, 44.043, 44.057, 44.174, 44.147, 44.081, 44.049, 44.168, 44.179, 44.291, 44.292, 44.191, 44.253, 44.143, 44.125, 44.019, 44.517, 44.256, 44.202, 44.265, 44.211, 44.339, 44.431, 44.251, 44.32, 44.153, 44.312, 44.203, 44.303], 'train_loss': [4.3534061187212085, 3.724583436645, 3.4892885131250186, 3.3525692670679366, 3.242161736759862, 3.1555479425538904, 3.0836436292412794, 3.023182682249688, 2.974410576615971, 2.9288289917636514, 2.889250953153243, 2.8564165390536944, 2.821012683350996, 2.7933819304653564, 2.770878245261565, 2.7409214985652834, 2.724164469304637, 2.701705668495774, 2.6818463584206764, 2.6624852302207143, 2.646758141002057, 2.629273443136624, 2.6167282746986786, 2.596445513854634, 2.5907555687374133, 2.578993095927565, 2.5673682883391375, 2.5572675307896833, 2.543969474232357, 2.5312780725292416, 2.5226978693569766, 2.5175155759696692, 2.501786392008122, 2.4987543909807504, 2.4902165376140912, 2.4813697232852285, 2.474648291532305, 2.470063057070883, 2.463290182657907, 2.4543912789032998, 2.449114123141239, 2.4442056482294316, 2.435794235877478, 2.431797256167699, 2.4227339991833716, 2.4202377940322526, 2.414916502308251, 2.4107566470910706, 2.4085031351979085, 2.4032677022295736, 2.401429551958046, 2.396165513870278, 2.3898315791967812, 2.387071865000026, 2.3859876686384185, 2.377233244712278, 2.378845806085217, 2.3729334332129928, 2.3705482617144544, 2.3736440286526523, 2.3689918860516634, 2.3663907403451696, 2.360906356432006, 2.3632528544692604, 2.3530653434461763, 2.351241788800069, 2.354124663734924, 2.3490780268581877, 2.345558931792461, 2.3443764377235223, 2.3427829690568354, 2.3392572195500954, 2.3356323308542, 2.33549296444071, 2.3351890800552955, 2.331200380471755, 2.328069321634826, 2.3288306826135705, 2.331052366694196, 2.327665867594977, 2.324390989805137, 2.3199684017374205, 2.3209367578638265, 2.320241319622203, 2.3180513096709934, 2.313576015004422, 2.3114814819316414, 2.3083665287807364, 2.307613036987954, 2.3088954802285535, 2.308020158524858, 2.3008221980477477, 2.3053028387513894, 2.3054738184312025, 2.29860186950564, 2.3045904557062737, 2.2987412444613184, 2.2964915484857347, 2.2977199225721647, 2.2991979967228358, 2.2994183013626803, 2.295682303354821, 2.293974522589417, 2.289941093209304, 2.291276536533944, 2.2954746359674427, 2.291663519099059, 2.2894911934607927, 2.2890158552278406, 2.288998454179965, 2.285614134635364, 2.2847340980250332, 2.28360888207485, 2.286432728428758, 2.2834518907623877, 2.281233992701681, 2.2841576744330974, 2.283144543206013, 2.277534844626697, 2.277339776204476, 2.281847166999822, 2.2803791294628737, 2.2754622906808737, 2.274197539990328, 2.2801387612818145, 2.2725673283969297, 2.272025903676156, 2.2739807490881483, 2.2706071395410303, 2.2716775990722278, 2.2700482271301845, 2.2700132943084435, 2.2653239048678984, 2.2682170252424743, 2.263819380974968, 2.267870621656807, 2.265275851626161, 2.2667289400283757, 2.2667516529445684, 2.2614448063661863, 2.2635439224755696, 2.263379731928776, 2.259631138845506, 2.257851902826886, 2.258890500071715, 2.261270218145672, 2.261487029564358, 2.2568546652565074, 2.2607220850613206, 2.2570699718771876, 2.255560361950045, 2.260717007340488, 2.2563542166506716, 2.2563688605547103, 2.2556184170265956, 2.2557414774702513, 2.2564643343060884, 2.2547536673487873, 2.2533434194703936, 2.249905845025222, 2.25254205877935, 2.250367020691196, 2.2453725829920703, 2.253398755461607, 2.251737389973319, 2.2530025290283575, 2.2452058971347673, 2.250839619017227, 2.2468633542667957, 2.2509539247855725, 2.2496523507039514, 2.2448518496633567, 2.2513020024113524, 2.2484206641551707, 2.247808898326608, 2.249029884869215, 2.246051239341936, 2.2460044697165413, 2.244458421940843, 2.239810050029596, 2.243045143416045, 2.2435828171246186, 2.242051626121243, 2.2421960616371073, 2.2431608010619706, 2.242686820426814, 2.2430195143340264, 2.237371224893337, 2.239491478876662, 2.240663083943509, 2.2426756750828014, 2.2420504715529606, 2.2416868097722644, 2.240562762576498, 2.239025079204879, 2.2357726484556193, 2.2364242579490043, 2.236271758454775, 2.2362780389462382, 2.233985514421152], 'val_metric': [5.44, 14.45, 19.67, 20.03, 24.18, 25.13, 24.33, 25.99, 26.11, 27.86, 27.35, 26.09, 30.38, 31.43, 30.44, 30.43, 31.95, 30.5, 31.36, 32.29, 32.94, 34.62, 31.3, 32.74, 35.0, 28.93, 30.82, 33.35, 32.83, 35.02, 33.92, 33.8, 35.08, 34.09, 33.03, 34.04, 33.85, 34.51, 32.57, 33.92, 34.54, 33.65, 33.87, 33.86, 37.19, 35.18, 34.34, 36.03, 36.62, 35.94, 35.3, 34.83, 38.27, 38.3, 33.36, 35.44, 34.79, 34.17, 37.17, 35.54, 37.44, 36.86, 36.02, 37.93, 36.73, 33.61, 34.2, 34.96, 34.59, 33.6, 36.07, 36.52, 37.25, 36.18, 38.21, 35.04, 35.28, 36.19, 36.79, 36.71, 36.8, 36.46, 39.12, 36.37, 38.39, 38.59, 37.06, 37.57, 38.25, 38.02, 37.26, 36.81, 37.76, 37.48, 38.26, 34.97, 32.25, 36.73, 36.57, 36.69, 37.27, 39.04, 38.43, 37.94, 37.19, 37.58, 36.71, 36.95, 38.51, 37.42, 37.97, 38.74, 37.25, 38.69, 38.45, 38.6, 37.4, 39.47, 38.98, 36.33, 37.78, 35.41, 37.29, 38.59, 37.37, 36.17, 37.53, 37.21, 36.07, 36.02, 36.44, 38.62, 38.2, 37.19, 35.68, 37.98, 36.42, 35.92, 39.45, 36.19, 37.82, 36.77, 36.61, 38.25, 39.92, 37.92, 37.6, 39.61, 37.51, 38.24, 34.76, 37.27, 37.43, 37.15, 37.98, 37.63, 36.78, 38.42, 34.04, 37.18, 37.5, 38.06, 38.4, 31.99, 38.06, 37.18, 37.84, 38.87, 33.5, 36.12, 39.06, 39.42, 37.44, 38.19, 35.75, 38.32, 37.35, 37.58, 40.99, 38.63, 38.35, 37.95, 40.26, 40.23, 35.32, 38.93, 38.4, 38.47, 38.99, 37.94, 39.59, 38.41, 35.13, 35.69, 39.11, 38.88, 39.1, 38.72, 40.27, 37.85], 'val_loss': [4.546259618868493, 3.8408495164980554, 3.5245129576154577, 3.4525414507859833, 3.216888666912249, 3.1711074296076585, 3.2568190568571636, 3.1165071467685093, 3.131409390716796, 3.0585121994565245, 3.088618699152758, 3.241695299270047, 2.910198715841694, 2.889826226386295, 2.9032796742809808, 2.925892168169568, 2.8175056208470823, 2.936491042185741, 2.9171928607734148, 2.8245806397905775, 2.795933040843648, 2.7125521374356216, 2.8948643002540444, 2.825858160189003, 2.713805506943138, 3.0318673260652336, 2.9190190218056844, 2.8212167465003435, 2.8026205316470687, 2.72279320590815, 2.7792530652064427, 2.750760586018775, 2.7042159100247036, 2.7676007998217442, 2.8486611448275814, 2.7363220536784763, 2.767646097833184, 2.7417694804774726, 2.886414552190501, 2.789012786689078, 2.752788169368817, 2.7789425098212663, 2.74456419231026, 2.8101273137293044, 2.6116547842693936, 2.6683494247448674, 2.799850294924086, 2.683311676523488, 2.6512861418875917, 2.662663948004413, 2.744435221146626, 2.7274160802744, 2.5877879421422434, 2.5999767028602068, 2.8233937192115053, 2.698510641884652, 2.7339836761450313, 2.8052062623819727, 2.604853975545069, 2.6913514365056517, 2.6122444458068554, 2.6048279431215517, 2.6874003102824946, 2.5496704092450964, 2.6431627911367235, 2.8114599255239887, 2.7972251157851735, 2.7189697718164725, 2.742378428483465, 2.802180547623118, 2.671396924052269, 2.641500281300514, 2.6201289129105345, 2.6639712828739435, 2.5553054278063927, 2.729451803265104, 2.7229563650811555, 2.7083327523462333, 2.669953023931783, 2.6605872508067234, 2.65941869756978, 2.6569787294242033, 2.5080530677631403, 2.6773952724068026, 2.5605854399644645, 2.5383528835454565, 2.6414559039340655, 2.62155739213251, 2.619264141009871, 2.5875418117851208, 2.6379141519024114, 2.6590527713678447, 2.605145856833002, 2.5783186892794956, 2.633799409410756, 2.7525994443589714, 2.909647109402213, 2.6711423700782144, 2.657357320664035, 2.660245241632887, 2.594797135158709, 2.543280405603397, 2.5779819890951656, 2.585884759760207, 2.638051639696595, 2.60394940634442, 2.6383642056945025, 2.6724063841400634, 2.5554220828281085, 2.631758802256007, 2.6163294277373392, 2.535971656726424, 2.6689492631110414, 2.5410465127343587, 2.5693108378701908, 2.577268701450081, 2.635254346641006, 2.506296591014619, 2.55619444247264, 2.6768992865920826, 2.611164602124767, 2.738809733254135, 2.610937902881841, 2.5530510715618253, 2.617923187981745, 2.682847089068905, 2.595146256647292, 2.648480556573078, 2.660802501022436, 2.6916740642991037, 2.6878775585988524, 2.5419363163079427, 2.6043666487286803, 2.6234657745452443, 2.7718786289737483, 2.5648298935525737, 2.6474681022060906, 2.70721969483005, 2.504207330904189, 2.675694366169583, 2.6268772198136445, 2.6668858710367966, 2.667710960290994, 2.55606670630206, 2.5525827126897824, 2.5839199883163353, 2.605905809220235, 2.4935077406038904, 2.562020924820262, 2.5768099628436336, 2.764384345643839, 2.6122208902030994, 2.6147963780506402, 2.6399215983737045, 2.581384849396481, 2.5739558661819264, 2.628814523387107, 2.560859137279972, 2.8002019554946074, 2.643939016730922, 2.627254688815706, 2.582306457932588, 2.551976369824379, 2.9522701957423214, 2.6464882098185787, 2.619619846343994, 2.5979193623658197, 2.5244684283900414, 2.8735661871114355, 2.698699543430547, 2.563450536150841, 2.5067217995406716, 2.5925863578820683, 2.5570689242356903, 2.6947624163263164, 2.605337734814662, 2.6305210780186257, 2.612936697947751, 2.4583022313512815, 2.557182100168459, 2.5543806120088903, 2.591344212270846, 2.47292634028538, 2.477329547997493, 2.7341476959787356, 2.555417593877027, 2.5561564344509393, 2.5892372719801156, 2.5302565686262337, 2.5628874324689246, 2.5104709427068186, 2.555337185312988, 2.7458035163818653, 2.7463565035990087, 2.5204477682235136, 2.5388329507439, 2.535269306723479, 2.5297397386496234, 2.494746148586273, 2.612969584905418]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="AdamW")
data.update({'train_metric': [9.268, 21.897, 29.842, 35.762, 40.392, 44.817, 48.661, 52.407, 55.811, 59.428, 62.629, 65.714, 68.611, 71.438, 73.939, 76.061, 77.712, 79.43, 80.947, 82.335, 83.364, 84.372, 85.092, 85.979, 86.208, 87.259, 87.585, 87.969, 88.538, 88.608, 88.921, 89.428, 90.035, 89.933, 90.361, 90.683, 90.745, 90.657, 91.116, 91.347, 91.449, 91.642, 91.881, 91.872, 91.978, 92.35, 92.059, 92.446, 92.378, 92.619, 93.089, 92.871, 93.026, 93.165, 93.286, 93.109, 93.379, 93.401, 93.665, 93.534, 93.621, 93.783, 93.841, 93.909, 93.951, 93.889, 94.062, 94.029, 94.21, 94.332, 94.064, 94.352, 94.469, 94.306, 94.514, 94.544, 94.705, 94.593, 94.543, 94.918, 94.577, 94.846, 94.857, 94.708, 95.072, 95.068, 94.841, 94.984, 95.14, 95.262, 95.017, 95.166, 95.105, 95.183, 95.214, 95.109, 95.421, 95.316, 95.213, 95.428, 95.479, 95.381, 95.495, 95.525, 95.452, 95.624, 95.592, 95.794, 95.521, 95.693, 95.669, 95.67, 95.744, 95.656, 95.773, 95.778, 95.784, 95.899, 95.841, 95.911, 95.828, 95.928, 95.957, 95.793, 95.886, 96.025, 95.97, 96.004, 95.994, 95.995, 95.964, 96.225, 96.053, 96.048, 96.193, 96.204, 96.127, 96.132, 96.21, 96.257, 96.157, 96.257, 96.305, 96.214, 96.454, 96.269, 96.392, 96.22, 96.346, 96.291, 96.435, 96.266, 96.355, 96.254, 96.401, 96.365, 96.46, 96.408, 96.498, 96.538, 96.554, 96.379, 96.568, 96.487, 96.591, 96.407, 96.606, 96.461, 96.583, 96.623, 96.726, 96.485, 96.59, 96.55, 96.652, 96.489, 96.637, 96.686, 96.592, 96.709, 96.664, 96.702, 96.78, 96.702, 96.724, 96.848, 96.708, 96.817, 96.72, 96.801, 96.867, 96.79, 96.817, 96.791, 96.92, 96.703, 96.759, 96.805, 96.843, 96.767], 'train_loss': [4.372882971608021, 3.460045290237349, 3.004956238863183, 2.6974180769783063, 2.4500076417654504, 2.2310414484351093, 2.041559824559144, 1.8537572051040347, 1.6887234920732348, 1.5237248310734657, 1.3765612524896573, 1.2379878482907076, 1.108230105688842, 0.9887581390977592, 0.8886300377638311, 0.8054656970981444, 0.7374624754668655, 0.6674496819594695, 0.6170939089526599, 0.5676574877302996, 0.5318621193283426, 0.4966734349536957, 0.4712415864013054, 0.44422731369791013, 0.4354856872760708, 0.4021014366310831, 0.3952351268654974, 0.3760320154958365, 0.3630271138991596, 0.35377127722963947, 0.3470496131549336, 0.33527238874845755, 0.3152163612131346, 0.3141755509289769, 0.30268766970081123, 0.293253500983405, 0.2914497279708041, 0.29319842038670185, 0.27789656370067856, 0.27227377130751873, 0.26816952046929304, 0.2613083859565963, 0.2569547949700842, 0.25515872496999537, 0.2508181655196734, 0.24232198325863497, 0.24672125432680833, 0.2388280812824909, 0.23799115717673217, 0.23135264481698484, 0.21897609606599738, 0.22658114304011706, 0.2194223751749555, 0.22057134983673815, 0.21554454830692638, 0.21607855562852388, 0.20951941775507218, 0.20773864118590885, 0.20152082457788104, 0.20433095871081774, 0.2010477727382746, 0.1970190608895414, 0.19630160310942624, 0.19688462447351224, 0.1913831385123047, 0.1943378035206722, 0.1910493045935666, 0.18959453706858978, 0.18743362294668506, 0.1783404759019947, 0.19036252852094313, 0.1784967061129809, 0.1793481571038545, 0.18315066290710666, 0.17630728801875659, 0.17578410309389644, 0.16923475946000016, 0.17573327309215442, 0.17603078736224695, 0.16589344061806235, 0.17314814277629648, 0.16568023340463142, 0.16381124550862786, 0.1661629281706884, 0.15816965680128478, 0.15789023465914884, 0.16604866276173993, 0.1615016039714732, 0.15798863557674425, 0.1542703867046805, 0.15941605796132974, 0.15536993737862834, 0.1552581593152362, 0.15745226425575765, 0.15375513067693639, 0.15922310833639858, 0.14936481274828858, 0.15077162621857423, 0.15585107867613732, 0.14436814043359855, 0.14559591537423702, 0.14818728839059767, 0.14527436223575846, 0.14324837669662258, 0.14696837087821452, 0.1409254599382275, 0.1442756732730332, 0.13889079617326858, 0.14703768414864704, 0.13920735678250018, 0.13978916858260354, 0.13978768120131319, 0.13793193451078253, 0.14255770627869221, 0.13605011037955453, 0.1369842558217182, 0.13990192974523433, 0.1343612776350758, 0.1359564172765699, 0.13374376640329166, 0.13686684438477908, 0.13385212020942572, 0.13047797313396195, 0.13732535571517734, 0.13523906454195458, 0.1288424115119292, 0.1324805299241557, 0.1313621041401315, 0.1306881078796596, 0.1310164836166523, 0.13262050258223937, 0.12370333191774838, 0.13080373893634034, 0.12859450559675817, 0.12732258662569526, 0.12436283918923471, 0.1277646195834356, 0.12611749755624999, 0.12330506496999583, 0.12280073648876369, 0.12863423300505408, 0.12421066303003367, 0.12318934374566351, 0.12307840536044039, 0.12104018382802284, 0.12214926838882563, 0.12082033034097356, 0.12471002288846006, 0.12180284847700831, 0.12104457982058171, 0.1192438803604695, 0.12218582752278112, 0.1206706734657197, 0.12178705824701139, 0.1198063217245295, 0.12137810873788003, 0.11759992854966725, 0.12058420198470812, 0.11671370762644331, 0.11391702926548923, 0.11322394135726588, 0.12137912523304722, 0.112692906193838, 0.11619255490121697, 0.11254757024396643, 0.1206592857371122, 0.11285286777244193, 0.11897123856061828, 0.11508505471521913, 0.11161439278965485, 0.10938159850050927, 0.11770320903048515, 0.11079913089438403, 0.11568848520915977, 0.11013260991343846, 0.1198161674388735, 0.1104305960688298, 0.10992938079786328, 0.1134854037320165, 0.10940691139281626, 0.11299183657193247, 0.11116659266249142, 0.10823590168420325, 0.11299539135921846, 0.11168172637314845, 0.1056764827067642, 0.11075067413178041, 0.10990045346141365, 0.10793321774368278, 0.10667660757591674, 0.10511069862290881, 0.10957506880857126, 0.10816934522158232, 0.10642344130531445, 0.10405349545098867, 0.10925495238181904, 0.1095721287910111, 0.11054859170883832, 0.10664374226105702, 0.10864686159202078], 'val_metric': [14.72, 24.23, 30.88, 34.57, 36.28, 37.47, 39.88, 39.27, 40.29, 39.66, 39.55, 40.3, 38.43, 38.39, 38.1, 38.78, 37.77, 38.14, 38.72, 37.22, 36.98, 37.92, 37.06, 37.45, 37.41, 38.33, 37.88, 36.76, 37.04, 37.95, 37.73, 37.54, 37.75, 37.4, 37.48, 37.5, 37.38, 36.59, 37.28, 37.41, 38.3, 37.28, 37.8, 37.65, 37.11, 38.48, 38.3, 37.26, 37.08, 37.88, 38.33, 37.64, 37.56, 37.86, 37.36, 37.63, 37.84, 38.35, 38.54, 38.03, 36.86, 38.33, 38.02, 37.72, 38.27, 38.47, 38.21, 37.17, 37.6, 38.6, 38.85, 38.04, 38.27, 38.55, 37.83, 38.1, 37.78, 37.52, 37.62, 38.08, 38.79, 37.91, 37.73, 38.33, 38.17, 38.58, 38.12, 38.28, 38.12, 37.63, 38.1, 38.7, 38.03, 37.89, 38.29, 37.83, 37.75, 38.22, 37.78, 37.85, 38.44, 38.33, 38.68, 37.89, 38.72, 38.63, 37.96, 38.08, 38.61, 38.33, 38.18, 38.71, 39.14, 38.32, 39.45, 38.23, 38.03, 38.86, 37.87, 37.81, 38.4, 37.32, 38.38, 38.67, 38.67, 39.0, 38.64, 39.09, 38.55, 38.01, 38.13, 38.33, 37.72, 38.51, 38.45, 38.65, 38.41, 38.82, 38.87, 38.62, 39.09, 38.31, 38.22, 38.72, 38.69, 38.64, 38.24, 38.33, 38.58, 38.75, 38.88, 38.89, 38.46, 38.55, 38.51, 38.93, 39.12, 38.81, 38.18, 38.92, 38.78, 39.44, 39.13, 38.44, 39.05, 38.82, 38.62, 38.1, 39.01, 38.46, 38.68, 39.38, 38.37, 38.23, 38.72, 38.88, 39.01, 38.46, 38.88, 38.11, 38.51, 38.58, 38.73, 38.32, 38.7, 38.46, 39.01, 38.93, 39.08, 38.93, 38.68, 38.66, 38.55, 39.29, 38.33, 38.33, 38.34, 38.96, 39.04, 38.44], 'val_loss': [3.9517335770236457, 3.30988776000442, 2.997838682429806, 2.807205229807811, 2.7222666288637054, 2.6897767748043035, 2.6066526876893015, 2.658585180902177, 2.667630653472463, 2.7695453744025746, 2.825920460330453, 2.9438956657033057, 3.103098885268922, 3.2164588900888043, 3.3255005406725937, 3.501959509910292, 3.5251076396103875, 3.680474904692097, 3.706147097478247, 4.039961529385512, 4.023878923646963, 4.152231869424225, 4.154800482616303, 4.297207290199911, 4.3117164282282445, 4.407708326722406, 4.502655398314166, 4.654192298840565, 4.643521099333551, 4.551980164400332, 4.7287333937966896, 4.721102151901099, 4.746845428351384, 4.923169098082622, 5.068341869457512, 4.846781672945448, 4.893839289428322, 5.112908064180119, 5.128947957306151, 5.136372576093978, 5.1854613541038175, 5.029085592099816, 4.95786325491158, 5.088334496613521, 5.2544485687450235, 5.390100612761868, 5.308875955593813, 5.35368594242509, 5.352035926405791, 5.349559941868873, 5.481088978469751, 5.279722743732914, 5.653034972537095, 5.410939385936518, 5.4811306834980185, 5.519280139048388, 5.599014134164069, 5.644293493525997, 5.588103809174458, 5.5597815027662145, 5.7608208967621914, 5.717292335382693, 5.73077167826853, 5.681970928884615, 5.677691240978849, 5.776865895386714, 5.877177555090303, 5.905429165834074, 5.776451341665474, 5.9147915794591235, 5.852480844327599, 5.754184111668046, 5.9561110939949184, 5.973713653102802, 5.997931902575645, 6.063278044864631, 5.978624055340032, 6.022835221260217, 6.182453410640644, 5.988803234829265, 6.061380273217608, 6.017727249746869, 6.1072609728308995, 6.308953211565686, 6.188046171407032, 6.194270425541386, 6.210912445548233, 6.135452940965155, 6.061313069550095, 6.261459844127582, 6.201198987140777, 6.397062649392778, 6.272678765521687, 6.339654280121919, 6.3469896141890505, 6.314105533490515, 6.263976867031899, 6.166576822851874, 6.360943236928077, 6.31106850599787, 6.412515362356879, 6.470411747124545, 6.481304473937697, 6.465835155195491, 6.331802574692259, 6.434570247200644, 6.444098975248398, 6.4297827763162605, 6.432687914295561, 6.486416186496711, 6.534705098267573, 6.428299835533093, 6.42920882231111, 6.467908274595905, 6.537491646541912, 6.625618804032635, 6.6718362547030114, 6.522322897698469, 6.603994314837608, 6.316435002976922, 6.622978723732529, 6.607899111547288, 6.668292452575295, 6.5259658758807335, 6.437319524728569, 6.837624867250965, 6.766139929461631, 6.69264618074818, 6.644425890248293, 6.750999292750268, 6.597928071477611, 6.643625365700691, 6.748091939907924, 6.850675692224199, 6.809450893645074, 6.605433395713757, 6.669178229228706, 6.797342664876561, 6.931279598527653, 6.782249109000917, 6.942804216579267, 6.954997753641408, 6.6146377727484245, 6.796367887478725, 6.776402752870207, 6.852261379266241, 6.572264028962251, 6.9179655655174495, 6.805330109444394, 6.951296760777759, 6.8410917452186535, 6.800764961607137, 7.143263947432208, 6.8253837330326155, 6.852621104307236, 6.915007405979618, 6.953902326571714, 6.775197463430417, 7.084437686166946, 7.002509729118104, 7.0088146613661655, 6.932553909386799, 6.84275144452502, 7.218208361583151, 6.939023942704413, 6.841875651839432, 7.009503596907209, 7.079109656582972, 7.174934760780092, 7.243797680374923, 7.082714313154767, 7.01396159305694, 7.101811954170276, 6.940870201511747, 7.121806929065923, 6.861785557619326, 6.964652201172653, 7.103482314735461, 7.042422636299376, 7.330068779599135, 7.160100124444172, 7.171069122423791, 7.344519726030386, 7.238400188980589, 7.174804370874052, 7.371941314381399, 7.344797312074406, 7.213446762911074, 7.111765773433029, 7.18505119214392, 7.317091100534816, 7.009476959325705, 7.229775636818759, 7.454586396551436, 7.172492037912843, 7.294802986132871, 7.307552904080433, 7.025128929478348, 7.233772203421137, 7.277071334753826]})


# EfficientNetV2 Tiny-Imagenet
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal 1", optimizer="SGD")
data.update({'train_metric': [0.722, 0.771, 0.829, 0.883, 0.963, 0.902, 0.948, 0.946, 0.995, 1.011, 1.033, 1.103, 1.198, 1.312, 1.497, 1.603, 1.791, 1.944, 2.246, 2.429, 2.589, 2.88, 3.152, 3.457, 3.673, 3.939, 4.264, 4.636, 4.998, 5.26, 5.722, 5.966, 6.207, 6.631, 6.879, 7.184, 7.546, 7.714, 8.222, 8.522, 8.955, 9.244, 9.765, 10.037, 10.458, 10.987, 11.325, 11.842, 12.256, 12.586, 12.971, 13.235, 13.804, 14.136, 14.588, 14.932, 15.417, 15.702, 15.881, 16.257, 16.82, 17.108, 17.57, 18.039, 18.375, 18.579, 19.13, 19.466, 19.881, 20.38, 20.751, 21.207, 21.538, 22.165, 22.57, 23.206, 23.587, 23.96, 24.424, 24.925, 25.41, 25.902, 26.437, 26.893, 27.418, 27.911, 28.313, 28.965, 29.282, 29.756, 30.29, 31.008, 31.5, 32.158, 32.48, 33.007, 33.586, 34.025, 34.789, 35.111, 35.751, 36.197, 36.829, 37.308, 37.806, 38.685, 39.187, 39.558, 40.13, 40.495, 41.189, 41.679, 42.492, 42.998, 43.624, 43.827, 44.444, 45.173, 45.673, 45.996, 46.552, 46.944, 47.735, 48.102, 48.931, 48.94, 49.866, 50.079, 50.473, 51.487, 51.688, 52.073, 52.729, 53.253, 53.535, 54.173, 54.645, 54.925, 55.681, 55.837, 56.491, 57.036, 57.416, 57.883, 57.978, 58.683, 58.876, 59.133, 59.756, 60.182, 60.732, 61.222, 61.491, 61.801, 62.078, 62.27, 63.184, 63.757, 63.683, 64.138, 64.345, 64.819, 65.191, 65.21, 65.647, 65.755, 66.539, 66.706, 66.916, 67.219, 67.428, 67.742, 68.121, 68.217, 68.681, 68.668, 69.335, 69.449, 69.824, 70.125, 70.235, 70.423, 70.839, 70.715, 71.214, 71.339, 71.701, 71.897, 71.967, 72.077, 72.533, 72.702, 72.853, 73.114, 73.137, 73.416, 73.579, 73.884, 73.976, 74.078, 74.017, 74.737, 74.698, 74.95, 75.179, 75.162, 75.264, 75.629, 75.568, 76.053, 76.303, 76.018, 76.813, 76.517, 76.57, 76.83, 77.043, 77.381, 77.242, 77.283, 77.58, 77.724, 77.676, 77.84, 78.146, 78.35, 78.433, 78.459, 78.841, 78.574, 78.86, 78.979, 78.908, 79.138, 79.328, 79.654, 79.416, 79.401, 79.791, 79.781, 79.974, 80.212, 80.25, 80.371, 80.47, 80.463, 80.828, 80.843, 80.766, 80.899, 81.053, 81.037, 81.288, 81.487, 81.783, 81.608, 81.607, 81.618, 81.684, 81.874, 82.106, 82.503, 82.21, 82.471, 82.347, 82.533, 82.595, 82.839, 82.896, 83.028, 83.0, 83.023, 83.162, 83.039, 83.435, 83.103, 83.297, 83.398, 83.424, 83.763, 84.082, 83.846, 83.899, 84.121, 84.038, 84.354, 84.153, 84.116, 84.192, 84.3, 84.84, 84.678, 84.587, 84.925, 84.708, 84.979, 84.858, 85.018, 84.875, 85.253], 'train_loss': [5.310683485642505, 5.293253686896365, 5.283379693254018, 5.275880173544661, 5.266640123463715, 5.261743961658832, 5.251752654482597, 5.240931140255333, 5.230447893942005, 5.216795902410838, 5.207024333268042, 5.19424521457821, 5.179432950413387, 5.159118236827301, 5.130194929114382, 5.105688537151022, 5.077051256988877, 5.046103381683486, 5.012587575979593, 4.9789518794415475, 4.942445133865757, 4.904224895469973, 4.867504417324249, 4.833018815448783, 4.795726102510478, 4.759783065708036, 4.721964013934975, 4.6823012519172575, 4.645011303902282, 4.610918700046747, 4.576471058741183, 4.5415482142760215, 4.511278843162766, 4.480301857299707, 4.4525777215158335, 4.424039470790977, 4.391869815282156, 4.363166391124957, 4.333708165474451, 4.305045988036514, 4.2752200914969904, 4.24507868877223, 4.2134324155857525, 4.183113832620192, 4.153076501206869, 4.122370613589931, 4.095669885056948, 4.0622295038256215, 4.030315450270513, 4.002932105091849, 3.9755926147456058, 3.953662339037836, 3.9249975222162305, 3.897731810140823, 3.872868708288982, 3.8402452009729444, 3.8172127552239923, 3.792453473299189, 3.7722226523925917, 3.7434713401172073, 3.715494024395104, 3.690693824930368, 3.664592760766994, 3.634915396714165, 3.6108200684313734, 3.5858656094002557, 3.5556185227209225, 3.5309598308607164, 3.5098136578777703, 3.4788821329310853, 3.4531412765297915, 3.4233793547270928, 3.399762681563238, 3.369677942377287, 3.342348169533015, 3.314828403928077, 3.289531946944946, 3.2614708317447305, 3.2352233778110926, 3.2080250378228574, 3.175940459917084, 3.149127836495886, 3.1181195255128222, 3.0914565531473777, 3.0676545896975567, 3.0370072731205995, 3.012599155526091, 2.9834025839697604, 2.9573343294366996, 2.9338448441188762, 2.9030398517103433, 2.8761453062620057, 2.844744456401637, 2.814064107487313, 2.796784892268312, 2.7683355571822488, 2.7414715694298137, 2.714743096021529, 2.687717754415267, 2.663702321067805, 2.6233368932781356, 2.6120846604050083, 2.5846427290849934, 2.553278793452721, 2.53360309245414, 2.493705128143784, 2.4718909609874546, 2.455263084886323, 2.42897334216271, 2.402150464454524, 2.3781655413633116, 2.3536604900659084, 2.323515356692914, 2.298846311624128, 2.2717867031817396, 2.2590937275804164, 2.2400445286005795, 2.201340619372162, 2.1812717936699464, 2.167131874245554, 2.141423760662457, 2.119527293258345, 2.0941338169978967, 2.074799365594611, 2.046056025087719, 2.0379233180294416, 2.005312453762355, 1.9938463411953535, 1.9714591740341576, 1.940142262004845, 1.9280219179502451, 1.9109160480633502, 1.8858693879305057, 1.8693364013560827, 1.8548705585477296, 1.8273460340286323, 1.8048338476694783, 1.788591915418609, 1.767602037025886, 1.7555544730263952, 1.7409383729719918, 1.7132640093927267, 1.7009482849734912, 1.6799159982764256, 1.6691132200427796, 1.646042966263003, 1.641477828596314, 1.6239871922320306, 1.6047818917383083, 1.587257891607376, 1.569825541797694, 1.5465873816954503, 1.5386267243023493, 1.5263557141207, 1.5152636749654418, 1.5021411264392708, 1.4717884451169962, 1.4487346317702505, 1.4526192063104626, 1.4376040302402913, 1.4240425710333362, 1.4098197382303361, 1.3922885964104401, 1.3933271186823122, 1.3742754501710697, 1.3696992902594045, 1.341872972932597, 1.3315994465343477, 1.3269511777395173, 1.3105009528817233, 1.3029416838824102, 1.288540799916744, 1.277746391578584, 1.2724260110467653, 1.2562346982406791, 1.252311674929245, 1.2343618373266796, 1.22180530621467, 1.2161458276893875, 1.201988089107506, 1.1946661225969946, 1.1871871180589277, 1.1778418719196502, 1.1743656194370218, 1.1537172813790773, 1.1511234398156653, 1.1373981499016017, 1.133386308233172, 1.1235292844488616, 1.1218988027468906, 1.1064776755340269, 1.0978542551426878, 1.0932714428874215, 1.0825193446947075, 1.07842263634664, 1.0651145296377473, 1.0629136160025594, 1.0522643080599705, 1.0462640747380272, 1.0385504730679787, 1.0388739413354768, 1.0162281406970644, 1.0177805121709198, 1.010458833616053, 1.0009702335011097, 0.9988470499285199, 0.9903329455616073, 0.980467730390667, 0.9807748153319514, 0.9655140172153883, 0.9531165279033317, 0.9604379611562935, 0.9430945718318929, 0.945659947925398, 0.942872103253619, 0.9332265465288534, 0.9224068043289929, 0.9114680741547165, 0.9165378796962768, 0.9113555159884542, 0.9051252186336505, 0.8967273221401854, 0.8964957150029434, 0.885472267732663, 0.8795953051485622, 0.8740373750909047, 0.8690645160655218, 0.8682324737215072, 0.8522830861963222, 0.8624811558981241, 0.8520405056266089, 0.8460722057123788, 0.8474422853799943, 0.8343462987123051, 0.8238664970936412, 0.8186247398551275, 0.8274812996006135, 0.8248650811264626, 0.8114452550636982, 0.8092901159842961, 0.8043106332926588, 0.7946775832087736, 0.7975485018637419, 0.7886566045722059, 0.7856966890895206, 0.7842483322016337, 0.7736715193330213, 0.7718041014267097, 0.7700538927175568, 0.7618803268659595, 0.7592495110312564, 0.7583160217153058, 0.7541487810707824, 0.7463786761232927, 0.7368463618550938, 0.7362579367554348, 0.7353439202159167, 0.7318880004869084, 0.7301137610573991, 0.7264951952206958, 0.7173196389861238, 0.7028002631984608, 0.7112430331612426, 0.7035897168416055, 0.7066756018979078, 0.7020302012000263, 0.7001449396918389, 0.6900735428832085, 0.6866747861250195, 0.6852985326670258, 0.6844999659389391, 0.6796715594565952, 0.6799092117365705, 0.676938884889782, 0.6640763650776405, 0.6736921077345094, 0.6722280654438931, 0.6626186822937302, 0.6653372285195215, 0.6545250875707322, 0.6444562581568594, 0.6468983431962233, 0.6460167945804156, 0.6409342018500095, 0.6389704383266178, 0.6330163081105672, 0.6377074754703372, 0.633976145920964, 0.634573443513304, 0.6283258940459671, 0.6146761951485431, 0.6139138620096525, 0.6189937949447546, 0.6039447799677736, 0.6102769523954361, 0.5981343472072579, 0.6062307051089202, 0.5974697719685023, 0.6038042321963258, 0.5929124907888973], 'val_metric': [0.84, 0.97, 0.96, 1.01, 0.92, 0.99, 1.17, 1.03, 1.08, 1.18, 1.34, 1.3, 1.6, 1.47, 1.53, 2.16, 2.26, 2.4, 2.79, 3.03, 3.44, 3.57, 4.14, 4.56, 4.59, 4.89, 5.02, 5.18, 5.81, 5.83, 6.59, 7.11, 7.39, 7.55, 7.69, 8.09, 8.59, 8.88, 9.14, 9.44, 9.74, 10.11, 10.55, 11.19, 11.6, 11.58, 12.24, 12.23, 12.68, 12.85, 12.88, 13.77, 13.79, 14.49, 14.2, 14.61, 14.68, 15.27, 15.83, 15.38, 15.98, 16.26, 16.07, 16.44, 16.62, 16.96, 17.34, 17.42, 17.38, 17.76, 17.84, 17.81, 18.37, 18.1, 18.43, 18.62, 18.84, 18.3, 18.91, 19.01, 18.84, 19.04, 18.95, 19.24, 19.01, 18.6, 19.24, 19.17, 18.75, 19.16, 19.13, 19.34, 18.81, 19.85, 19.54, 18.78, 19.3, 20.16, 19.42, 19.12, 19.9, 18.67, 19.61, 19.12, 18.71, 19.61, 19.58, 19.73, 18.77, 19.43, 19.24, 19.39, 18.47, 18.87, 19.22, 19.55, 19.1, 18.64, 19.47, 19.24, 18.93, 18.91, 18.74, 19.05, 19.74, 19.08, 19.3, 19.11, 18.97, 18.65, 18.88, 18.56, 19.06, 19.38, 18.94, 18.64, 18.32, 18.58, 18.16, 19.23, 18.67, 18.61, 19.79, 18.35, 18.74, 18.79, 17.67, 18.66, 18.62, 18.47, 18.27, 18.75, 18.99, 18.78, 17.73, 18.53, 18.59, 18.78, 19.11, 18.7, 16.97, 19.44, 17.93, 18.73, 19.33, 19.36, 19.0, 18.52, 18.47, 18.83, 17.83, 18.46, 19.11, 18.99, 19.43, 18.67, 18.55, 18.34, 19.3, 18.78, 18.95, 18.53, 17.73, 19.08, 18.45, 18.41, 18.97, 18.35, 19.27, 18.05, 18.77, 19.01, 18.55, 18.93, 18.74, 19.24, 18.43, 19.1, 19.22, 19.06, 19.34, 19.32, 18.98, 19.63, 18.29, 18.72, 19.02, 17.17, 18.99, 20.13, 19.81, 19.21, 17.79, 18.92, 19.54, 18.39, 18.68, 19.62, 19.88, 19.43, 19.14, 18.57, 19.46, 18.46, 18.39, 19.34, 18.11, 19.54, 19.08, 19.39, 20.08, 18.8, 19.44, 19.74, 20.43, 18.78, 20.58, 19.05, 19.45, 19.62, 19.39, 19.41, 18.56, 19.4, 20.31, 19.45, 18.58, 18.95, 18.71, 19.89, 19.53, 18.59, 19.4, 19.46, 20.51, 19.73, 19.23, 19.27, 19.28, 19.73, 19.4, 19.48, 19.95, 20.33, 19.73, 19.18, 19.49, 19.61, 19.18, 19.65, 19.46, 19.26, 19.53, 19.21, 20.23, 17.88, 20.24, 18.75, 19.03, 20.07, 19.34, 20.09, 20.16, 19.59, 19.63, 20.61, 19.66, 20.24, 19.97, 20.85, 20.49, 20.46, 19.28, 20.38, 19.98, 20.34, 20.12, 19.0, 20.37, 19.87], 'val_loss': [5.279276128027849, 5.265847953261843, 5.260741464651314, 5.2553689100180465, 5.253980299469772, 5.239993362669733, 5.22963617713588, 5.218826485287612, 5.202620457691752, 5.192003459687445, 5.172404513996877, 5.163476816408194, 5.139826121603607, 5.116454586101945, 5.084414049318641, 5.053498284831928, 5.021606440756731, 5.001511907881232, 4.951629149685999, 4.903640941449791, 4.856880637490825, 4.820229109685132, 4.776916991373536, 4.744392674440031, 4.719987204120417, 4.662136554718018, 4.626433036889241, 4.590671710907274, 4.555701559516275, 4.525503433434067, 4.482123929224197, 4.450656537037746, 4.428685162477432, 4.402215943974294, 4.3735870889797335, 4.341851519930894, 4.319759194258672, 4.28147681047962, 4.259565096751899, 4.224226295568381, 4.217750231931164, 4.1949423058017805, 4.160645671710847, 4.118415410351601, 4.093273449855245, 4.067986703982019, 4.039259874137344, 4.032213955168512, 4.001064623996711, 3.9678561824142555, 3.9673913618561567, 3.9086388615286274, 3.9017198769150268, 3.8743897228483943, 3.8804700822587224, 3.8590858286353433, 3.8368498686772243, 3.813610770899779, 3.8103505441337635, 3.797915815547773, 3.783038597957344, 3.755315950721692, 3.756386000639314, 3.737462415057383, 3.7173550569327776, 3.704161501234504, 3.6842446828344064, 3.706403584996606, 3.7124321005146976, 3.6734487820582786, 3.68334066184463, 3.6708686761795337, 3.646742296826308, 3.6758517968426845, 3.651610354709018, 3.638643217694228, 3.641013474221442, 3.654273983779227, 3.618868575733938, 3.6038978821153096, 3.6164919957993136, 3.613237416668303, 3.6409126672015826, 3.61874912043286, 3.6308732146670106, 3.690737763787531, 3.628579743348869, 3.6524720199548515, 3.705466594665673, 3.6584960503183352, 3.670320111475173, 3.649957931725083, 3.687549797592649, 3.67514452660919, 3.650455074705136, 3.7106278808253585, 3.6982631372038726, 3.6504069984338847, 3.7019772567566793, 3.774615015193915, 3.689170053810071, 3.7522318613756993, 3.696407447954652, 3.7133971612164927, 3.7687266077964927, 3.741940480128975, 3.7632139738957595, 3.776683906081376, 3.8438928134881767, 3.7312653808836727, 3.8017993588356456, 3.7414634933896886, 3.9391259440950526, 3.820371202602508, 3.7964024946188473, 3.861914849585029, 3.912520886986119, 3.8775332011994283, 3.814023905498966, 3.8627029254937626, 3.946644715442779, 3.9195747952552358, 3.9465793242120437, 3.9052584133330424, 3.9233061325777867, 3.9608533230556806, 3.9016898338961754, 4.018076808589279, 3.953721786000926, 4.047566802638351, 3.9665502742597254, 4.021780826483562, 4.008496972405986, 4.000384101442471, 4.036381833872218, 4.088594363753203, 4.155605322236468, 4.113330191108072, 4.1616827568430805, 4.0639062003724895, 4.1575465749023826, 4.138562071854901, 4.087542309882535, 4.153871438305849, 4.149429630322062, 4.216716327485005, 4.379599185506249, 4.256910339282577, 4.303207061093325, 4.257008759838761, 4.299620467386428, 4.264321184462043, 4.181815554381935, 4.210964909784353, 4.324282463189143, 4.382808343620058, 4.325181428034594, 4.3435489463198715, 4.232288068267191, 4.359186827756797, 4.572231918383556, 4.276672610811367, 4.415640676097506, 4.348622529370011, 4.318401146846212, 4.354252167568085, 4.263496024593426, 4.457126195263711, 4.430076246808289, 4.363165224433705, 4.517752339885493, 4.573289137737007, 4.458608811068687, 4.408011945189943, 4.343458391298913, 4.43423145895551, 4.522502422332764, 4.643840174766103, 4.509326078329876, 4.559917372503098, 4.49246813233491, 4.566773927135832, 4.746842141364031, 4.547710317714959, 4.538914938641202, 4.500811082542322, 4.581524508773901, 4.537129449236924, 4.514555121683012, 4.623971406061938, 4.521133413740024, 4.564908099781936, 4.710594828721065, 4.63675603593231, 4.671901175930242, 4.594847141557438, 4.6914612275020335, 4.58275561271959, 4.65645034449875, 4.5810638764861284, 4.596351521030353, 4.626039423760335, 4.683368286509423, 4.623900121943966, 4.773310548940282, 4.715016677880743, 4.6718277900841585, 4.924489157215046, 4.771348942616942, 4.542874320297484, 4.692421779511081, 4.635082735377512, 4.8466972545453695, 4.740937439499388, 4.699688844620042, 4.839537508168798, 4.759107238927465, 4.662662325391344, 4.662226833355654, 4.686628539091463, 4.822186441178534, 4.794885425810602, 4.695530977978069, 4.813646100888587, 4.892979017488516, 4.859882875612587, 4.891754885388028, 4.702211477194622, 4.810526627643853, 4.747278328913792, 4.701930071897568, 4.868295230683247, 4.757182467515301, 4.789440081377697, 4.674038801982904, 4.8852473656842665, 4.650951765145466, 4.819443529578531, 4.770249107081419, 4.8199813745583695, 4.809616529258194, 4.886857652360467, 4.945171782924871, 4.829559913866079, 4.70314244783608, 4.794864127590398, 4.936892842031588, 4.82723785965306, 4.884100806181598, 4.821828082868248, 4.900208495984412, 4.945533082743359, 4.854175519032084, 4.867900974431615, 4.677936604068537, 4.962471782781516, 4.993857154420986, 4.829936209757617, 4.967769648618759, 4.951597338269471, 4.900188365559669, 4.93944647509581, 4.960961607611104, 4.827477465769288, 4.862316667653952, 4.9555960977153415, 4.985401519544565, 4.902295717008554, 4.943389230473026, 4.962180585618231, 4.96041230639075, 5.035885718218081, 4.925727303620357, 4.990927431993423, 4.884216133955937, 5.209698815254649, 4.892422581933866, 4.997945209976974, 4.986522606224011, 4.854560563518743, 5.021776799183742, 4.833536432047558, 4.887710551547396, 4.904425212532092, 4.884978675538568, 4.824708312939686, 4.917575078405393, 4.9357577858457145, 4.8885777626827265, 4.802511022349072, 4.857092240813431, 4.860990843195824, 4.9738873721687655, 4.922060890562215, 4.898212139773521, 4.89967604959087, 4.958137193303199, 5.046303422587692, 4.855848925888159, 5.015212716570326]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal 1", optimizer="Adam")
data.update({'train_metric': [3.471, 8.171, 10.705, 13.204, 15.39, 17.078, 18.593, 19.986, 20.884, 21.72, 22.799, 23.386, 23.971, 24.366, 24.827, 25.205, 25.622, 25.932, 26.229, 26.381, 26.849, 27.074, 27.517, 27.618, 27.871, 28.237, 28.363, 28.291, 28.765, 29.085, 29.062, 29.187, 29.173, 29.333, 29.58, 29.51, 29.805, 29.84, 29.971, 30.022, 30.105, 29.974, 30.277, 30.512, 30.626, 30.737, 30.574, 30.685, 30.719, 30.685, 30.791, 31.094, 31.199, 31.192, 31.233, 31.332, 31.216, 31.231, 31.334, 31.573, 31.504, 31.607, 31.285, 31.671, 31.637, 31.537, 31.725, 31.708, 31.8, 31.935, 31.629, 31.691, 32.0, 31.767, 31.973, 31.909, 31.855, 31.94, 31.977, 32.164, 32.298, 32.212, 32.18, 32.373, 32.374, 32.319, 32.3, 32.344, 32.376, 32.162, 32.377, 32.165, 32.425, 32.306, 32.361, 32.358, 32.481, 32.51, 32.576, 32.541, 32.703, 32.636, 32.431, 32.656, 32.54, 32.707, 32.726, 32.653, 32.621, 32.65, 32.856, 32.46, 32.645, 32.753, 32.825, 32.74, 32.838, 32.699, 32.772, 32.766, 32.744, 32.781, 32.895, 32.852, 32.776, 32.864, 32.877, 32.881, 32.691, 32.758, 33.002, 32.895, 32.945, 32.918, 32.882, 33.037, 33.045, 32.95, 32.967, 32.997, 33.137, 32.892, 33.098, 33.049, 33.075, 33.069, 33.002, 33.088, 33.107, 33.107, 33.163, 33.022, 33.041, 33.052, 32.998, 32.979, 33.332, 33.04, 33.03, 33.264, 33.14, 33.284, 32.974, 33.026, 33.261, 33.362, 33.191, 33.161, 33.293, 33.145, 33.247, 33.337, 33.22, 33.23, 33.177, 33.457, 33.195, 33.161, 33.333, 33.35, 33.412, 33.515, 33.558, 33.381, 33.395, 33.069, 33.572, 33.33, 33.449, 33.222, 33.372, 33.271, 33.457, 33.43, 33.629, 33.358, 33.401, 33.628, 33.398, 33.46, 33.408, 33.475, 33.274, 33.532, 33.419, 33.425, 33.407, 33.505, 33.606, 33.487, 33.58, 33.61, 33.379, 33.683, 33.557, 33.437, 33.421, 33.632, 33.456, 33.7, 33.494, 33.668, 33.529, 33.472, 33.557, 33.597, 33.608, 33.636, 33.494, 33.584, 33.677, 33.467, 33.577, 33.598, 33.725, 33.409, 33.56, 33.563, 33.702, 33.471, 33.67, 33.677, 33.593, 33.636, 33.644, 33.69, 33.578, 33.686, 33.554, 33.593, 33.339, 33.595, 33.652, 33.748, 33.835, 33.698, 33.624, 33.563, 33.615, 33.625, 33.544, 33.8, 33.612, 33.527, 33.623, 33.814, 33.666, 33.733, 33.521, 33.65, 33.725, 33.529, 33.753, 33.765, 33.662, 33.727, 33.783, 33.671, 33.522, 33.9, 33.925, 33.689, 33.957, 33.74, 33.807, 33.84, 33.745, 33.582, 33.869, 33.818, 33.926, 33.738, 33.874, 33.82, 33.964, 34.032, 33.918, 33.882, 33.8, 33.837], 'train_loss': [4.816044446831701, 4.2795162258892585, 4.08483488439217, 3.9053316595305714, 3.7722554272439948, 3.67783242803465, 3.5837972382284935, 3.50828862617356, 3.4513980425555815, 3.4001222108314986, 3.3575606998997625, 3.3219243905060734, 3.289662118150268, 3.2649682899811903, 3.2379019980390944, 3.2210136398015234, 3.201616001678291, 3.1856831860557553, 3.1718761937708253, 3.151725514455247, 3.134172301984947, 3.1255320145087753, 3.106656520090573, 3.096373130629937, 3.083804924367562, 3.0736027442714913, 3.0611299489143944, 3.0560690032619737, 3.043139784288803, 3.028004461271368, 3.0281757078945675, 3.0170304776763426, 3.0148772093552623, 3.007057041673422, 3.0005145063784666, 2.9960571892812173, 2.988075718083446, 2.9835963487167505, 2.9774579943866204, 2.9728958423673078, 2.9662647927638743, 2.9655343540494288, 2.960363475008798, 2.9547004118533144, 2.9517048398272316, 2.9497783931492996, 2.9509339622404815, 2.9415866135788207, 2.9392024380079236, 2.934888355562646, 2.932153527124982, 2.925570350843443, 2.9241798776735193, 2.920849022313104, 2.9166783374315335, 2.9134861490929347, 2.915878480623261, 2.9135398750341786, 2.908137680549158, 2.901814760646222, 2.9026276816638403, 2.9022927169836414, 2.900039510549983, 2.8967417984228447, 2.8938751007148418, 2.892468106189906, 2.891952031557177, 2.890797426512969, 2.888629547693908, 2.885844323395615, 2.888047388556365, 2.883680402660553, 2.8779151663899194, 2.877896983121651, 2.874640864130021, 2.875258153322333, 2.8800614064729757, 2.8727705519853766, 2.8727543201495345, 2.8705360988585213, 2.8700479268265013, 2.868371833690221, 2.8654698076266474, 2.8615973909467134, 2.8637199023558555, 2.862354204277922, 2.8598858187462532, 2.8597194487973807, 2.855932594489685, 2.8576121916011292, 2.8580675733752994, 2.857127593102092, 2.8533653145178115, 2.8541323046461557, 2.8547135169736406, 2.853862380645859, 2.850917370748001, 2.847905479633724, 2.8496368585758614, 2.85047477800268, 2.8450289530702224, 2.847482105439394, 2.8454012054704507, 2.844859569559323, 2.842519567017363, 2.8442735154889593, 2.847370717438535, 2.842959735001499, 2.841391350165896, 2.8417290547530163, 2.8387323443277936, 2.8410868624884276, 2.836213525563421, 2.837356714430484, 2.834402022953607, 2.836443052182042, 2.831939153006194, 2.832196199657516, 2.8314492421812227, 2.832405339428346, 2.834123077410883, 2.8362167485616028, 2.8310439641553473, 2.8298781139303, 2.832101704139246, 2.8317091083648642, 2.8326967659472504, 2.8293290104686033, 2.8285461449882425, 2.830084482981315, 2.827196878114726, 2.828443517764257, 2.819391970137183, 2.8264284649493217, 2.8278822488534625, 2.8268183847306556, 2.8234213751550676, 2.8223416150874687, 2.8244036838021405, 2.8208447349277432, 2.8214481603771313, 2.82415454950534, 2.8224932473207693, 2.8222268643626323, 2.821280037830521, 2.819974529384728, 2.8214863313746927, 2.818378496993755, 2.8168582823210926, 2.8194802609759115, 2.817158810846789, 2.8167450919184867, 2.819768971231452, 2.817352058028687, 2.816235293658666, 2.8150527385740958, 2.8106274449207502, 2.8130645822121103, 2.8122075828923223, 2.8159000660010967, 2.8168569857389287, 2.8124574794268957, 2.812145786108455, 2.8133070430767817, 2.8128079512297766, 2.812699794769287, 2.8108498034230123, 2.8082246574467753, 2.8086442085541905, 2.8140639091865114, 2.813709579060189, 2.812294394407071, 2.8105631768512787, 2.8096404475274945, 2.812681308863488, 2.8062621502256975, 2.809855910882077, 2.8128486457575765, 2.8052419334821646, 2.810084659170052, 2.8087185332200044, 2.807865274616029, 2.8054467653968893, 2.8076337132786455, 2.8063568744000453, 2.8112286647313387, 2.806457497870701, 2.8056507517875042, 2.804691809228957, 2.806079163127272, 2.8060848644278558, 2.803889016653587, 2.805441244855113, 2.8071158653791333, 2.7991899499813866, 2.800916272253084, 2.803012074069647, 2.802162493945541, 2.8037301267634884, 2.8015025288953432, 2.803554877819957, 2.8012900015366666, 2.801505843874589, 2.8001937817398432, 2.8015562379047494, 2.7997876036571374, 2.8005906769806805, 2.801284890562315, 2.7965886348802464, 2.798014862752464, 2.7968745053157695, 2.8015593385665905, 2.8010959510839832, 2.7978160666565213, 2.799148143901325, 2.7990187105275237, 2.8012067138271615, 2.800320874248951, 2.79640667055634, 2.792204349680124, 2.795087689859167, 2.798121013018998, 2.7959075517099174, 2.7960332073009098, 2.796508344258548, 2.7974128494335915, 2.7927454196712715, 2.7948524504995316, 2.795065877076074, 2.7950498822859595, 2.791033519130446, 2.7971750344516217, 2.7939677648794476, 2.7941600920981653, 2.790345232225883, 2.7961707397370636, 2.7960052290408184, 2.792502345752045, 2.7919690325255586, 2.793126766298798, 2.793652004564106, 2.7863014087567173, 2.7878086200831262, 2.7905990938612533, 2.7921504197941327, 2.7938877154220316, 2.7928055025260567, 2.794155971560048, 2.7905609398717997, 2.7847887830557307, 2.790256764899441, 2.792062915446891, 2.7903444820386967, 2.787655292377972, 2.786631126245168, 2.7872334307611406, 2.7915382263222646, 2.791799663048254, 2.792175726332271, 2.7877815337190244, 2.7900773446222793, 2.7881062590610655, 2.7881733242396125, 2.7871373375485664, 2.7857637025573205, 2.7828077941999476, 2.785644972240475, 2.7857664470404138, 2.7887857362237103, 2.782833653234627, 2.7846139523743516, 2.784776055805209, 2.7845299034948465, 2.785644058531397, 2.784807663580125, 2.7853201634595584, 2.782101729094639, 2.7809261993498504, 2.7833849843770206, 2.7817077058595645, 2.78157480328951, 2.783400810763993, 2.777508873246987, 2.7810954406943296, 2.7812970413737625, 2.7818459068744974, 2.7810660285669035, 2.785305192206658, 2.78095926600851, 2.779300321467931, 2.7825214489102743, 2.781659217500107, 2.779994745400954, 2.778987790709951, 2.779394645227199, 2.777150539732559, 2.7765207630811557, 2.776724304972897, 2.78108004568787, 2.7796156908256155], 'val_metric': [5.6, 8.21, 11.64, 14.32, 17.34, 19.34, 20.37, 21.33, 22.72, 24.05, 23.37, 24.8, 25.12, 26.12, 25.86, 26.49, 26.64, 26.81, 27.28, 26.99, 27.45, 27.66, 29.16, 29.01, 28.92, 29.18, 29.04, 29.88, 29.69, 30.21, 29.59, 30.43, 30.1, 29.72, 30.37, 29.91, 30.16, 30.61, 31.24, 31.61, 31.04, 31.78, 30.53, 30.71, 31.12, 30.9, 31.3, 31.46, 31.56, 32.63, 32.6, 31.88, 31.53, 32.08, 31.76, 31.16, 32.26, 32.6, 32.24, 32.29, 33.16, 32.4, 32.3, 32.77, 32.33, 32.24, 32.77, 32.29, 33.25, 33.14, 32.87, 32.64, 33.42, 33.58, 31.96, 32.9, 32.46, 32.7, 32.25, 33.12, 32.65, 31.55, 34.01, 32.15, 33.35, 31.98, 33.43, 33.47, 33.67, 33.85, 32.61, 34.03, 33.78, 32.42, 33.26, 33.27, 33.71, 32.92, 32.74, 33.91, 34.12, 32.96, 32.63, 33.36, 33.21, 33.73, 33.38, 33.5, 33.29, 32.36, 33.66, 33.4, 32.58, 33.03, 33.77, 33.22, 33.5, 33.04, 33.69, 33.32, 33.73, 32.84, 33.48, 33.73, 33.35, 33.47, 34.07, 33.83, 33.96, 33.7, 32.94, 33.71, 33.73, 32.93, 33.87, 34.36, 33.18, 33.43, 34.88, 33.75, 33.36, 34.28, 33.77, 33.1, 33.46, 33.16, 34.12, 34.28, 33.97, 32.65, 33.69, 32.45, 33.33, 33.67, 33.63, 33.6, 33.9, 34.38, 33.22, 33.52, 33.52, 34.89, 34.21, 33.4, 33.36, 34.2, 33.84, 34.02, 33.73, 34.12, 34.22, 33.67, 33.17, 32.9, 33.14, 34.56, 34.2, 33.44, 34.33, 34.36, 33.08, 33.31, 33.63, 33.29, 33.91, 32.97, 33.59, 33.82, 33.59, 34.29, 34.36, 31.72, 34.37, 34.22, 33.63, 33.34, 34.16, 33.84, 34.28, 34.04, 34.0, 34.25, 33.1, 34.52, 34.81, 34.05, 34.67, 34.38, 33.39, 34.53, 34.07, 34.29, 34.11, 34.38, 34.98, 33.52, 32.77, 33.18, 34.12, 33.32, 33.41, 34.0, 34.31, 33.9, 34.4, 34.06, 32.7, 33.39, 33.58, 32.21, 34.43, 33.58, 34.31, 34.48, 33.14, 33.22, 34.12, 34.78, 34.11, 34.54, 34.84, 32.97, 34.32, 34.18, 33.77, 33.89, 34.48, 34.07, 34.28, 33.74, 33.95, 33.06, 33.55, 33.62, 33.84, 34.27, 33.67, 34.51, 33.81, 34.54, 34.1, 34.26, 33.96, 33.1, 32.89, 34.13, 34.5, 34.36, 33.53, 33.75, 34.73, 33.57, 32.91, 34.03, 34.26, 34.22, 34.26, 34.58, 34.83, 34.83, 33.54, 33.77, 33.21, 33.92, 33.82, 33.22, 32.91, 34.23, 34.0, 32.84, 34.16, 32.52, 33.73, 33.74, 34.44, 34.16, 34.1, 33.9, 34.38, 33.48], 'val_loss': [4.427979522449955, 4.225246435517718, 3.972091143298301, 3.802723343964595, 3.6589450198373976, 3.5476895525197314, 3.47141695022583, 3.4336511299109005, 3.350842697605206, 3.2903148406630107, 3.2856921397956316, 3.228653045976238, 3.2126630043527884, 3.17598637046328, 3.1683924767621763, 3.1208074624371376, 3.139621771065293, 3.1134624898813335, 3.1167931404842695, 3.0931397957406985, 3.1027564007765167, 3.066594831503121, 3.029283664029115, 3.0165055473898628, 3.0376066607274828, 2.9877428552906986, 3.0138877440410057, 2.9935002531974937, 2.992068998373238, 2.970331966497336, 2.981531435516989, 2.9424060567928727, 2.97040493017549, 2.9988970703380122, 2.959081891995327, 2.9783945318999563, 2.9481937444893416, 2.932240044994719, 2.9070773959919145, 2.917699772081557, 2.9284692782505304, 2.895981713465065, 2.943547606847848, 2.929782969177149, 2.8917821273682223, 2.9008848765853106, 2.9064825279697493, 2.900229432780272, 2.8731274870550556, 2.8450006360461, 2.852055461543381, 2.881572906378728, 2.8872094928838643, 2.8771410952707765, 2.8874558476126118, 2.8884066624246585, 2.852686600320658, 2.8662817630038897, 2.8644131368892207, 2.8589966304742607, 2.830103641103028, 2.835171732173604, 2.85747995953651, 2.845183386164866, 2.864354710670034, 2.8559363138903477, 2.844160640315645, 2.828970549972194, 2.8130670869426364, 2.8232963350927753, 2.8342462846427967, 2.813375116913182, 2.810694923066789, 2.7983401019102447, 2.859737898893417, 2.815481461537112, 2.857452913454384, 2.82987663472534, 2.846435958412802, 2.8369094247271303, 2.833599600822303, 2.890727321813061, 2.794514274900886, 2.8530726979492576, 2.824931342131013, 2.8687385358628195, 2.787428720741515, 2.8299288575057013, 2.816363705191643, 2.79160222733856, 2.837596671596454, 2.7951351434561857, 2.790471202628628, 2.844476913190951, 2.832139510257988, 2.8025941377992085, 2.785317575096325, 2.8249947364163246, 2.826107660676264, 2.7983643401200604, 2.7687668557379657, 2.8154038699569215, 2.815270964507085, 2.806648629486181, 2.8102770155402506, 2.814494590091098, 2.7927903559557192, 2.7895218300971254, 2.804675143995103, 2.8554856040675167, 2.792372926025634, 2.792646218257345, 2.833087782950918, 2.811716012893968, 2.7987080698559996, 2.8140616773799727, 2.7916601220513604, 2.8290781481250837, 2.774046244894623, 2.830858709705863, 2.8040517143383146, 2.8002201379484433, 2.792522195038522, 2.7811501132454843, 2.800049830394186, 2.7998255810160546, 2.7761779446510753, 2.7783819353504544, 2.7748699841225983, 2.786104416391652, 2.8247464234661903, 2.796901586708749, 2.7709050102598347, 2.8349504136735466, 2.7923221140150813, 2.761615586888259, 2.8247318882851085, 2.798056585773541, 2.7524035789404704, 2.7939745263688884, 2.803499266600153, 2.761386396019322, 2.783495106514852, 2.805421295439362, 2.7742368840867546, 2.8133220201844624, 2.779697781915118, 2.740266761202721, 2.7746702500968983, 2.8417028651875293, 2.7838277702878234, 2.846980820795533, 2.8231210655467525, 2.7876097814292664, 2.7818845700306496, 2.7583970962815982, 2.7684871239267337, 2.7545947269269617, 2.8060508723471576, 2.793320051424063, 2.817140760695099, 2.74446294565869, 2.754326564491175, 2.802251897799741, 2.763287631949042, 2.7915321755561098, 2.779365446157516, 2.7527036780764345, 2.7793539770089897, 2.7798087072979873, 2.768575597720541, 2.7760542342617254, 2.8232706010721293, 2.835791443943218, 2.839140816099325, 2.745008982670535, 2.7524371906450598, 2.8137888938758024, 2.757806864513713, 2.755553837794407, 2.790892880433684, 2.8003105319988957, 2.7930966076577546, 2.8041150881226655, 2.7862985316355515, 2.7948253838119994, 2.7713748808879, 2.7660907582872234, 2.7880503250535127, 2.7699230020972574, 2.753056293080567, 2.870944618419477, 2.7591270542448494, 2.7750262561117767, 2.7997154035386007, 2.822646791767922, 2.7616968542147595, 2.779930741164335, 2.7617544558397524, 2.758936558559442, 2.7606801569082173, 2.759922025310006, 2.814303878006662, 2.779108237309061, 2.757895541798537, 2.7727768337650662, 2.7507773706108143, 2.7721048145537166, 2.812743996358981, 2.7610031837111064, 2.774120162247093, 2.7646123588464824, 2.7589654793405227, 2.7512158603425236, 2.7447026945223474, 2.7786146873121806, 2.828835669596484, 2.8049817783817366, 2.771168166664755, 2.778965982661885, 2.7882023282871127, 2.7509828828702307, 2.792235100345247, 2.759145786807795, 2.7572430341866365, 2.789242922120793, 2.847143670555892, 2.7975044174558796, 2.8042659091342026, 2.864548132677746, 2.763686095833019, 2.7854067512378573, 2.7715210770345795, 2.7414750118923794, 2.8273404730353384, 2.839684270749426, 2.7623772188356726, 2.775860355917815, 2.7615425996719654, 2.775523155358187, 2.7434128249526784, 2.8279993951700297, 2.756561664259358, 2.7525970875077945, 2.7709021909981018, 2.7756071910736666, 2.757244330302925, 2.766081058295669, 2.7633362833861335, 2.7945637103098973, 2.767593564501234, 2.8074372270304684, 2.79742080390833, 2.7587303356000574, 2.7739403376913376, 2.776402818928858, 2.778345897698858, 2.7543122920261065, 2.7883620269738945, 2.7194096548542097, 2.743573213838468, 2.761347387247025, 2.7715400206814906, 2.8224177459243, 2.8155326106745724, 2.7794251548256845, 2.7704759532478964, 2.754657128054625, 2.7906332160257232, 2.7681329660354907, 2.7355682090589197, 2.7657373057808847, 2.8210337283505, 2.7676396635687275, 2.752416584142454, 2.7577153611335024, 2.736392423605463, 2.7454370275424544, 2.735285346675071, 2.724719964774551, 2.784893489946985, 2.7803066543712736, 2.799487267330194, 2.7677371008380964, 2.7560081185808607, 2.7765177731301374, 2.7864228578130152, 2.760254168206719, 2.768846547527678, 2.8172168329263187, 2.768031312401887, 2.794299530375535, 2.7858328090351856, 2.7662822729463032, 2.7620516673774476, 2.7567413096215314, 2.763919304130943, 2.783126243360483, 2.7616969088839878, 2.8023780417290465]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal 1", optimizer="AdamW")
data.update({'train_metric': [4.794, 15.292, 24.123, 31.713, 38.401, 44.27, 49.484, 54.265, 58.868, 62.945, 67.868, 72.156, 76.014, 80.012, 82.626, 84.907, 86.934, 88.137, 89.443, 90.067, 90.835, 91.653, 92.033, 92.592, 92.996, 93.27, 93.705, 93.907, 94.188, 94.622, 94.664, 94.734, 94.957, 95.335, 95.375, 95.51, 95.51, 95.785, 95.858, 95.852, 96.226, 96.237, 96.292, 96.252, 96.611, 96.385, 96.584, 96.652, 96.766, 96.729, 96.93, 96.804, 96.998, 97.005, 97.222, 97.099, 97.224, 97.295, 97.259, 97.255, 97.345, 97.469, 97.468, 97.408, 97.519, 97.465, 97.626, 97.446, 97.631, 97.616, 97.727, 97.73, 97.708, 97.782, 97.809, 97.843, 97.891, 97.789, 98.026, 97.92, 97.898, 97.929, 98.001, 97.862, 97.957, 98.165, 98.048, 98.078, 98.07, 98.06, 98.084, 98.056, 98.229, 98.2, 98.102, 98.226, 98.209, 98.262, 98.162, 98.293, 98.353, 98.361, 98.205, 98.307, 98.373, 98.307, 98.276, 98.417, 98.396, 98.432, 98.354, 98.382, 98.519, 98.274, 98.378, 98.485, 98.46, 98.44, 98.672, 98.479, 98.447, 98.513, 98.541, 98.481, 98.614, 98.502, 98.566, 98.538, 98.538, 98.561, 98.554, 98.681, 98.699, 98.585, 98.6, 98.632, 98.603, 98.598, 98.567, 98.669, 98.727, 98.666, 98.627, 98.713, 98.7, 98.681, 98.666, 98.71, 98.643, 98.692, 98.686, 98.682, 98.696, 98.76, 98.749, 98.74, 98.724, 98.695, 98.705, 98.846, 98.798, 98.748, 98.812, 98.768, 98.84, 98.727, 98.751, 98.727, 98.845, 98.781, 98.787, 98.772, 98.945, 98.782, 98.805, 98.955, 98.798, 98.88, 98.848, 98.886, 98.822, 98.837, 98.925, 98.885, 98.829, 98.919, 98.853, 98.93, 98.942, 98.788, 98.955, 98.943, 98.78, 98.949, 98.912, 98.909, 98.97, 98.862, 98.916, 98.954, 98.888, 98.991, 98.879, 98.932, 98.968, 98.827, 98.953, 98.901, 98.928, 98.967, 98.94, 99.013, 99.003, 98.886, 99.012, 98.964, 98.946, 99.052, 98.945, 99.002, 98.895, 98.944, 98.968, 98.972, 98.912, 98.999, 99.091, 98.985, 98.943, 99.068, 98.943, 98.999, 99.049, 99.024, 98.954, 99.066, 98.977, 99.023, 99.092, 98.998, 99.028, 99.048, 99.02, 99.042, 99.095, 99.031, 99.038, 99.065, 99.084, 99.032, 99.02, 99.056, 99.05, 99.098, 99.084, 99.047, 98.946, 99.026, 99.171, 98.997, 99.102, 99.068, 98.964, 99.12, 99.004, 99.083, 99.1, 99.021, 99.086, 99.145, 99.108, 99.087, 99.104, 99.131, 98.999, 99.141, 99.167, 99.106, 99.163, 99.103, 99.085, 99.065, 99.159, 99.082, 99.096, 99.076, 99.143, 99.155, 99.08, 99.215, 99.07, 99.112, 99.127, 99.098, 99.172, 99.19, 99.099, 99.126, 99.138, 99.198], 'train_loss': [4.682663718630546, 3.7956652752649913, 3.2722004620752805, 2.878684344081183, 2.546139438641964, 2.2619606322839485, 2.0189480997397973, 1.7931441019836787, 1.5765504206668393, 1.3789526079986925, 1.1746631462987387, 0.9933890074548704, 0.8321153137711326, 0.682276498876698, 0.577419020774192, 0.4947364340492799, 0.4245209534505355, 0.38253748432154544, 0.3401179187004565, 0.31646093213722176, 0.2908916431082912, 0.2659911232799654, 0.253156211663345, 0.23469765090114858, 0.22577116137188136, 0.21335535686432133, 0.20121655281768994, 0.19424832888633986, 0.1860390904664002, 0.17218500425799566, 0.17011376959882482, 0.16769812955751642, 0.16068481537021853, 0.1495833527737715, 0.14701875535293374, 0.1409228327979053, 0.1419063000400537, 0.13349383755247998, 0.13270359672129306, 0.13162991190501994, 0.12163652775610154, 0.1196882617534737, 0.1181670756759128, 0.11939340854994357, 0.10682435537332709, 0.11305005479632607, 0.10800640460911731, 0.10626685166861373, 0.10293264233615065, 0.10285917701293797, 0.09754964891583995, 0.10177895289524598, 0.09643095182654134, 0.09442337163409989, 0.08919864945067182, 0.0915586745908087, 0.08802501286555174, 0.08499757097068737, 0.0889036631436872, 0.08664519571438813, 0.08349796217018339, 0.08048171465453749, 0.08064802958506684, 0.08204869911958293, 0.07883427843349608, 0.07930260298830849, 0.07511528357205005, 0.07979116570076387, 0.0744125377520876, 0.07698733576988147, 0.0711989407192246, 0.07194973248675565, 0.0730592644830328, 0.07027743264981524, 0.06889433711872961, 0.06830138511670025, 0.0677656565410915, 0.07013025833040556, 0.0635910571532107, 0.06546901793700154, 0.06579793530596172, 0.06656219344034968, 0.06410603771913947, 0.06671166128117856, 0.06467408979471274, 0.05761131683389239, 0.0626097672379615, 0.06057528148554814, 0.060081381077406855, 0.059855689664066826, 0.06016952761617845, 0.061332895585245815, 0.05664768215162788, 0.05761722343040109, 0.059458626381324764, 0.0560271060521019, 0.05681700138920371, 0.05515477971582878, 0.05769597481377899, 0.053943183652190245, 0.05254563487676209, 0.05298374132586909, 0.05614041326128385, 0.053441944964091825, 0.051126572650299254, 0.05274639622742097, 0.05355083799273877, 0.05009571182988739, 0.051836064223007024, 0.05021520067109581, 0.05295307855765315, 0.051866087819210815, 0.04629953524271492, 0.0546176859842922, 0.050114607055086385, 0.04892201999895649, 0.04945276402151814, 0.048967404673200744, 0.042221619544317794, 0.04978216392375703, 0.047489810580809574, 0.04681888667448595, 0.04636083188915407, 0.048407178544793907, 0.04458326707429938, 0.047979852757792055, 0.04423565579366305, 0.04765819124711358, 0.046415797032209404, 0.044637055267053714, 0.04570208196990789, 0.04384998330939606, 0.041260464630484155, 0.044931152838802765, 0.044820289699736414, 0.043281442651695555, 0.04503330877252604, 0.042916788472090885, 0.04539146200362048, 0.04193575490230035, 0.04138571770782943, 0.042918691402415894, 0.04425326691469999, 0.040323088391913696, 0.04146061125100439, 0.0414646563598882, 0.04205527890691315, 0.04046057116890714, 0.042271098032976115, 0.04066539397124682, 0.041956807733471706, 0.042326085754887395, 0.04046232017074871, 0.040101891414736655, 0.03945160504315532, 0.0397323416697558, 0.04016210101137813, 0.04114800122972001, 0.04042412060361638, 0.03713962010515947, 0.038790731112322556, 0.03916088883907749, 0.03766838610401117, 0.03973375796202614, 0.0369521934428176, 0.04025513832117726, 0.03858159620825634, 0.03990133959508482, 0.036929326965132465, 0.03808509691704261, 0.039458502725194855, 0.039102878538947235, 0.0340315147983028, 0.038601078269965214, 0.03706830395075033, 0.03526547125611655, 0.03745113451887245, 0.03727535711726059, 0.03619900111712918, 0.035256437984193444, 0.03782147428295524, 0.037410604993488965, 0.033734536380656016, 0.03521145300395067, 0.036210882165792484, 0.03405005200593415, 0.037483389764265664, 0.03398164658397493, 0.033625107507253574, 0.03789699180160428, 0.03386290331741453, 0.03463351254949378, 0.03825683545644329, 0.03273671301573283, 0.03519160680565432, 0.03468768584480306, 0.03310615700173239, 0.036097771110040984, 0.034588383996349885, 0.03385265627877041, 0.03481945553773648, 0.033950912864586506, 0.03507590615143642, 0.033030270588810966, 0.033674025539808486, 0.03840204515587211, 0.03285214548178407, 0.03435785436988775, 0.03452340592109578, 0.032586712733639396, 0.03378195287118466, 0.03080542030407149, 0.031174824947548415, 0.03563256324123265, 0.03207427047965093, 0.03351018466621829, 0.032686098940038585, 0.030184196565106274, 0.033493381450775, 0.032700835726577976, 0.03442729299105805, 0.032690524170942356, 0.03158062651632727, 0.03210069866285207, 0.03466904957128003, 0.031106641744916678, 0.02966263090748072, 0.03187621242021284, 0.03303078057377035, 0.03049184343752571, 0.03376070651906436, 0.03172184606960508, 0.029852300052973412, 0.03189372383569762, 0.033125982999121464, 0.029053894711911003, 0.031686904967777575, 0.03179719178524261, 0.029575111394415456, 0.03092063310157002, 0.031342074172618035, 0.030102366530427308, 0.030514820894391483, 0.030669939397466747, 0.02983940563084062, 0.03170382297639982, 0.03039365116865185, 0.029080060895159066, 0.02962120086094618, 0.030468749134646107, 0.03067860884158601, 0.0290956162046227, 0.03064698164560728, 0.029064229089884125, 0.02962248782340976, 0.03000330766382158, 0.03235135376456937, 0.03009992267931664, 0.02675735852572729, 0.03078985859328273, 0.029803546789538796, 0.0297073393223181, 0.03159550920249613, 0.02846065451889251, 0.031180095668514637, 0.02951507851454736, 0.02847397898889748, 0.030389877405327906, 0.028665145896973082, 0.027631405712626633, 0.027622569752850015, 0.029909138606327348, 0.02860577923575087, 0.027696423307149085, 0.032039379579435825, 0.027665090641734405, 0.027072865245223995, 0.02825376166031174, 0.02610504303900613, 0.027997474743471012, 0.0291055578272138, 0.02914057604820552, 0.027259424213142824, 0.02834558997895044, 0.028992642130691488, 0.028360616735657748, 0.027944931066146636, 0.027145817291569674, 0.02883274478059176, 0.02507558849795523, 0.028653283607111085, 0.02810485447300539, 0.02780718183848335, 0.028463812659476555, 0.02615133988761798, 0.02582426502218276, 0.02878270176907713, 0.027649216459192168, 0.027676613671168186, 0.025590922889317642], 'val_metric': [10.3, 20.71, 29.25, 34.38, 39.87, 42.65, 47.13, 48.87, 49.53, 50.54, 50.73, 51.09, 50.88, 51.09, 50.68, 51.25, 51.02, 50.75, 50.4, 50.99, 50.58, 50.74, 51.76, 51.11, 51.44, 50.99, 51.43, 51.3, 52.15, 52.19, 51.28, 51.87, 51.24, 51.88, 51.88, 51.35, 51.9, 52.07, 52.39, 52.14, 52.41, 52.11, 52.6, 52.44, 51.82, 52.09, 52.32, 52.35, 52.1, 52.12, 52.02, 51.75, 52.6, 52.76, 51.68, 52.57, 52.58, 52.62, 52.06, 52.45, 52.75, 52.45, 52.28, 53.15, 52.69, 51.84, 52.36, 52.31, 52.82, 52.37, 52.46, 52.6, 52.19, 52.78, 52.31, 52.76, 51.88, 52.78, 52.32, 52.72, 53.1, 52.89, 52.79, 53.11, 52.44, 53.16, 52.49, 52.43, 52.06, 52.21, 53.34, 53.15, 52.73, 53.23, 52.65, 53.31, 52.46, 52.64, 53.5, 53.02, 53.25, 52.69, 53.45, 53.28, 53.09, 53.27, 52.84, 53.39, 53.52, 53.02, 53.21, 53.69, 53.64, 52.87, 53.24, 53.48, 53.05, 53.09, 52.81, 53.75, 52.88, 53.85, 53.45, 53.39, 53.24, 53.67, 53.66, 53.85, 53.59, 54.02, 53.39, 53.53, 54.21, 53.94, 53.07, 53.15, 53.3, 53.03, 53.64, 53.74, 53.65, 53.5, 53.55, 54.59, 53.6, 53.39, 53.94, 53.61, 53.41, 53.52, 53.66, 53.57, 53.75, 53.69, 53.73, 53.88, 54.07, 53.86, 54.21, 53.91, 53.76, 53.99, 52.96, 53.66, 53.34, 52.8, 53.04, 53.56, 53.63, 53.78, 53.17, 53.96, 53.76, 53.72, 53.2, 53.89, 53.05, 52.92, 52.87, 53.47, 53.43, 53.74, 53.98, 53.59, 53.31, 53.75, 54.24, 54.41, 54.03, 54.09, 53.77, 53.41, 53.49, 53.71, 53.17, 54.17, 53.58, 53.68, 53.84, 53.22, 53.89, 53.49, 53.85, 53.88, 53.79, 53.19, 53.41, 53.54, 53.58, 54.23, 53.68, 53.66, 53.45, 53.79, 53.76, 53.96, 53.98, 54.59, 53.13, 53.77, 53.73, 54.21, 53.78, 53.62, 53.56, 53.82, 53.46, 54.13, 54.02, 53.94, 54.12, 54.67, 54.17, 53.99, 53.92, 53.97, 53.98, 54.21, 53.62, 53.77, 53.67, 53.89, 54.0, 54.21, 53.87, 53.73, 53.67, 53.19, 53.79, 53.65, 53.34, 53.72, 54.11, 54.13, 53.91, 53.52, 53.75, 53.85, 53.77, 54.05, 53.88, 53.78, 54.54, 54.53, 54.06, 54.45, 54.35, 53.92, 54.21, 54.83, 54.36, 54.18, 53.75, 54.09, 53.98, 54.02, 54.11, 54.42, 53.85, 54.44, 54.25, 54.03, 54.14, 54.39, 53.79, 54.76, 54.74, 54.27, 54.78, 54.19, 54.21, 53.92, 54.28, 54.48, 54.62, 54.55, 54.59, 54.48, 54.19, 54.35], 'val_loss': [4.144054095456554, 3.4770153616644013, 3.036427862325292, 2.7447436493673143, 2.493860308531743, 2.345770871183675, 2.1801213344950585, 2.109328506478838, 2.0951563753899496, 2.0579975175250107, 2.1176955593619375, 2.1816509825408836, 2.2746308442134007, 2.3517082363936552, 2.462868821089435, 2.52286753798746, 2.6054423307157624, 2.6827469752852324, 2.805789663913144, 2.813315143251115, 2.8994880443925313, 2.86313471710606, 2.8432565802221843, 2.9857525567340244, 2.9580296801913315, 3.071028989211769, 3.051052527442859, 3.0516417026519775, 2.9930589700200754, 3.0966907849737035, 3.1514062980178057, 3.0946210596212156, 3.1371767099495904, 3.18716208190675, 3.0987255592255076, 3.216949539959051, 3.148377071140678, 3.1561356684204878, 3.2204796523804875, 3.169561375098623, 3.2006413389922708, 3.222827840762533, 3.221577026661794, 3.2708455650669754, 3.2325338064485294, 3.205797451696578, 3.285315754686951, 3.3069570599847538, 3.315368788257526, 3.319524622267219, 3.3217016978628315, 3.3444852343030798, 3.3174090290525156, 3.314207528047501, 3.377134293887266, 3.34980476547958, 3.3658886350643864, 3.408343740329621, 3.4151201491143293, 3.4109238025489126, 3.383032570598991, 3.4308807903034673, 3.3961459189463574, 3.406855862231771, 3.4251332396914247, 3.462424017821148, 3.462241538390992, 3.4391280089973644, 3.44738657155614, 3.4763360547411972, 3.446743536147342, 3.4885610110440832, 3.450024153396582, 3.4818390691356296, 3.5037956727538138, 3.4756446408618027, 3.553782460795846, 3.4874804080671566, 3.530675546378846, 3.4879151799116923, 3.461505595286181, 3.4755188241885726, 3.513017426630494, 3.5518565147545687, 3.4933048145026917, 3.5147424852772122, 3.5417137483882297, 3.4996659979698763, 3.5869707309516374, 3.5537644852498533, 3.521668255329132, 3.5007110044455074, 3.4524441524675695, 3.473076938064235, 3.516905366235478, 3.5237348330248692, 3.4867080123561203, 3.5491001419960315, 3.513898881377688, 3.5974151792040296, 3.563361201696335, 3.572304379788174, 3.5298779435977816, 3.5404360385457423, 3.539861118717558, 3.582858256094015, 3.6285996102983025, 3.5722898654877002, 3.592817056710553, 3.5815832356738437, 3.639137496234505, 3.5778135585177475, 3.594629778224192, 3.602351988956427, 3.6127264545221998, 3.619916187729805, 3.6457678588332643, 3.6165641758851943, 3.6725150498614947, 3.6136071606046833, 3.614308134006087, 3.6212428231148204, 3.634216084601773, 3.663395065410881, 3.660388497030659, 3.679525487741847, 3.6579571036016865, 3.632587385405401, 3.647796967986283, 3.6545877008681087, 3.644940550160256, 3.6475631203621055, 3.70160111120552, 3.6141015313992835, 3.707978385269262, 3.661063091770099, 3.661515755258548, 3.6298652875954938, 3.615572853832488, 3.667479349549409, 3.641805733085438, 3.6379888122248802, 3.6136766277301082, 3.6507286949522175, 3.6536891938774447, 3.710524934112646, 3.7174670988587057, 3.733346540457124, 3.6779687875395366, 3.6635943480358004, 3.6331573071753143, 3.6493248837009356, 3.627166579483421, 3.692810630342763, 3.681551590846602, 3.6921710383360553, 3.6556616364770633, 3.7044175999939064, 3.651489853099653, 3.6528750172086584, 3.722103766574981, 3.6876900719988877, 3.749193489551544, 3.6780999428147725, 3.7748352441058795, 3.7556555423007647, 3.7211904487792093, 3.684764187047436, 3.744580052840482, 3.7159840335511856, 3.7946172694491733, 3.7766922545281187, 3.7725373776095688, 3.823161122905221, 3.8141956177486738, 3.744092851687389, 3.804836724973788, 3.769068579005588, 3.776800569455335, 3.7779533688429816, 3.8233796578304022, 3.6951269625098844, 3.839105308435525, 3.83939836473222, 3.793851031239625, 3.783061873381305, 3.7189285276801725, 3.76069705425554, 3.767984152599505, 3.727487382995095, 3.7975642028128265, 3.83318398712547, 3.7973320666392136, 3.8241679569718183, 3.763170279894665, 3.7143156699314237, 3.696262913904372, 3.7514063434995664, 3.732227901364588, 3.7602743248271335, 3.8498268135034355, 3.7577200786323304, 3.7627381857033746, 3.7852927066717936, 3.804641176940529, 3.799785781058536, 3.770618612599221, 3.7998212127928523, 3.821101266867036, 3.7843799735330474, 3.819525479511091, 3.8296287014226245, 3.835868739778069, 3.808571396359972, 3.822744121217424, 3.8024093231577782, 3.792216754263374, 3.787479202458813, 3.8178641583509507, 3.7934651382409843, 3.7937776507086056, 3.7910301100676227, 3.755044892335394, 3.7543711700257223, 3.7541846249513564, 3.77892466867046, 3.797808771300468, 3.749391376592551, 3.7722862267949777, 3.8238990496677956, 3.7596463807828866, 3.7678397742046674, 3.80082143947577, 3.79054297649177, 3.792494905982048, 3.833134970087914, 3.8091365827876293, 3.7882994967661086, 3.8342471107555802, 3.8737396688977626, 3.8068413901480898, 3.8051635247127265, 3.805328036569486, 3.8275581104740217, 3.8121536112135384, 3.8260574105438914, 3.8797782856947296, 3.86036008786244, 3.8420653517838494, 3.9395840555239636, 3.8677080360947143, 3.8563769873540115, 3.8841408764480785, 3.859175359367565, 3.8692846127376437, 3.8425299050701653, 3.8256899482884985, 3.863903928714193, 3.830905512639671, 3.856158180601278, 3.8245729921729703, 3.8076258229601914, 3.7997895862646165, 3.7864303763505, 3.8307472053606797, 3.8434395922976696, 3.784457259117418, 3.778491652695237, 3.782528486980754, 3.767391080309631, 3.7698499799534013, 3.7917193018706743, 3.800131183900651, 3.7805598647731125, 3.8042931526329866, 3.8513396879669966, 3.809132857307507, 3.8723424737620507, 3.8139682195748494, 3.8326619020692863, 3.865831355760052, 3.852370091684305, 3.850675034674869, 3.8481761847332026, 3.8967322969132927, 3.8194285281904183, 3.785114292885847, 3.836929646267253, 3.8096685523440126, 3.7925954434522398, 3.8451420288936347, 3.898010624442131, 3.8080032205885384, 3.8508136655874314, 3.8433366858275835, 3.7880892453679613, 3.784383172062552, 3.7834883670138706, 3.8412595641840794, 3.830326607272883]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="SGD")
data.update({'train_metric': [0.748, 0.978, 1.244, 1.533, 1.908, 2.29, 2.757, 3.248, 3.938, 4.457, 5.054, 5.583, 6.068, 6.628, 7.336, 7.901, 8.607, 9.337, 10.119, 10.646, 11.148, 11.864, 12.496, 13.111, 13.497, 14.156, 14.505, 15.273, 15.542, 16.074, 16.608, 16.919, 17.544, 18.141, 18.715, 19.033, 19.511, 20.19, 20.292, 20.835, 21.194, 21.705, 22.21, 22.63, 23.114, 23.323, 23.821, 24.262, 24.738, 25.068, 25.389, 25.916, 26.276, 26.808, 27.183, 27.473, 28.034, 28.307, 28.688, 29.248, 29.789, 29.987, 30.606, 30.924, 31.393, 31.898, 32.152, 32.596, 33.014, 33.464, 34.018, 34.381, 35.009, 35.412, 35.737, 36.338, 36.452, 37.083, 37.789, 37.963, 38.523, 39.057, 39.178, 39.954, 40.366, 40.76, 41.3, 41.808, 42.081, 42.494, 43.226, 43.722, 44.104, 44.652, 45.174, 45.621, 46.172, 46.506, 46.746, 47.421, 48.026, 48.364, 48.982, 49.77, 49.775, 50.572, 51.09, 51.406, 51.846, 52.608, 52.922, 53.136, 53.802, 54.243, 54.991, 55.386, 55.4, 56.105, 56.6, 56.769, 57.694, 57.968, 58.613, 58.707, 59.063, 59.587, 60.22, 60.145, 60.842, 61.296, 61.727, 62.223, 62.628, 62.944, 63.43, 63.622, 63.95, 64.418, 64.808, 65.161, 65.608, 65.734, 66.205, 66.543, 66.89, 67.373, 67.701, 68.126, 68.369, 68.689, 68.785, 69.027, 69.736, 70.271, 70.444, 70.23, 70.74, 70.829, 71.334, 71.739, 71.559, 72.091, 72.113, 72.674, 72.949, 73.206, 73.49, 73.667, 73.785, 74.117, 74.257, 74.565, 74.661, 75.113, 75.073, 75.285, 75.495, 75.861, 75.89, 76.041, 76.336, 76.665, 76.756, 76.845, 76.852, 77.385, 77.441, 77.546, 77.569, 77.845, 78.145, 78.075, 78.473, 78.661, 78.548, 78.732, 79.059, 79.231, 79.098, 79.374, 79.499, 79.433, 80.153, 80.005, 79.975, 80.333, 80.331, 80.46, 80.437, 80.875, 80.923, 81.071, 81.033, 81.385, 81.358, 81.351, 81.529, 81.758, 81.944, 81.993, 82.045, 81.853, 81.993, 82.244, 82.402, 82.489, 82.47, 82.596, 82.711, 82.64, 82.634, 82.623, 83.201, 83.031, 83.28, 83.407, 83.41, 83.517, 83.552, 83.556, 83.991, 83.561, 83.753, 84.078, 83.893, 84.067, 83.948, 84.224, 84.29, 84.614, 84.369, 84.639, 84.78, 84.511, 84.709, 84.904, 84.946, 84.774, 84.909, 85.189, 85.319, 85.263, 85.227, 85.257, 85.37, 85.597, 85.779, 85.538, 85.906, 85.929, 85.673, 85.811, 85.827, 86.105, 85.936, 86.241, 86.36, 86.304, 86.343, 86.16, 86.365, 86.43, 86.528, 86.482, 86.721, 86.499, 86.609, 86.711, 86.935, 86.68, 86.918, 86.684, 86.981, 86.865, 87.182, 87.108, 87.19, 87.261, 87.315, 87.285], 'train_loss': [5.284474825904832, 5.2525838789692765, 5.228152148172936, 5.200579034313511, 5.167900792315307, 5.120420527015828, 5.05385392427597, 4.976732320840436, 4.899605897780191, 4.830106973495532, 4.7684820581535, 4.703689002258535, 4.642345708223466, 4.580906871641895, 4.5207376522813485, 4.460176579554113, 4.403667609201969, 4.34732607733494, 4.295960838338616, 4.245219117772938, 4.200757956398045, 4.155496434073226, 4.110951601353046, 4.067425706336229, 4.03054437191915, 3.986720255377654, 3.949353135707511, 3.9096450186355405, 3.87546574985531, 3.8382498072418585, 3.7999666753062358, 3.7690538968020957, 3.734721128252631, 3.6987169469234202, 3.664820442425465, 3.6353084984606685, 3.6033929054811837, 3.5709887611049913, 3.5457757738104863, 3.514408611900442, 3.487247739170731, 3.460470719590678, 3.4313019406543814, 3.403455314770465, 3.3772246692703844, 3.3538618496573283, 3.3301282769506435, 3.303086077488163, 3.27654662959025, 3.255626629501753, 3.2292361433354997, 3.2052855372657705, 3.1800811643106237, 3.1564900348831735, 3.133128475395442, 3.112180326660703, 3.086374519119946, 3.0641724479099306, 3.041521835693242, 3.0164412041161963, 2.9956379783359464, 2.973895725651727, 2.9495912614115825, 2.927011177163054, 2.9003624463035598, 2.878393941709649, 2.855226540672268, 2.8326629587113668, 2.8093829290728043, 2.7855330523358504, 2.761825609573247, 2.7410749725554133, 2.7135567399682103, 2.6928863443019524, 2.6702622543598546, 2.6446137669295435, 2.6254623607420724, 2.60118947354022, 2.5724327086029493, 2.5564054612997476, 2.528606880267919, 2.504766517774615, 2.48857265905318, 2.4575962333898858, 2.436173602318962, 2.4167304454670453, 2.3921096352377687, 2.36589656185814, 2.3479854418387873, 2.3263361068848836, 2.306414775946014, 2.2798480192057538, 2.2552610563301383, 2.233214258995105, 2.2076151473966075, 2.18460054627917, 2.165358243885516, 2.1459682401143354, 2.1288056508593276, 2.101947629901742, 2.079255709721351, 2.0597076753126986, 2.035135566006085, 2.009459200991474, 1.9949908806434138, 1.970320036452471, 1.952556775230974, 1.9292434451676148, 1.9092075671435775, 1.8916668677589334, 1.8664239955421296, 1.847774650451089, 1.8288315820602445, 1.807906699043318, 1.7813667360972076, 1.769678896082111, 1.7564341057895165, 1.7338464482808373, 1.7111241623749736, 1.7059641830904393, 1.6782417272194334, 1.6562801803142233, 1.6358470830944816, 1.6271682987210083, 1.615292232233366, 1.5899098793894377, 1.5716667876210033, 1.564407347259961, 1.5453094788186457, 1.5263340578045665, 1.5087384991895978, 1.4907405208259992, 1.4789025189017762, 1.4585617202943666, 1.440064933158157, 1.4380743026885938, 1.4207165571107212, 1.406819012900308, 1.3900723263611796, 1.3792329832139263, 1.3614286972937948, 1.357017892831728, 1.3359579177750889, 1.3209689666427105, 1.3101641179961572, 1.2896932034407071, 1.281354052472862, 1.2649321298300267, 1.2585567570388585, 1.2445165402448415, 1.2365062391613053, 1.2268422649826518, 1.2060460138229399, 1.188365613430338, 1.1808521360902549, 1.1815751085888477, 1.1660510804587576, 1.1598672880931153, 1.1437198079631485, 1.133649234121912, 1.1293147696338246, 1.113745799754113, 1.107930870179709, 1.0940783472680466, 1.084985786725982, 1.0746077472631244, 1.0690199068854118, 1.0539342280541637, 1.0468485520882096, 1.0379506904043148, 1.0316424400701174, 1.0237486706814245, 1.0184608800473764, 1.0045447506854264, 0.9988026267736292, 0.9887291054197862, 0.9863795842868124, 0.9697090140993749, 0.967443121104994, 0.967429636593286, 0.9569879835870734, 0.9409442560000063, 0.934226689892401, 0.93405319024795, 0.934230094984107, 0.9135343945148429, 0.9097266017971173, 0.9059509922126433, 0.9016703228613389, 0.896073285673798, 0.8811705955807704, 0.8827506942925969, 0.8730741037593311, 0.8668920853278306, 0.8675790507863594, 0.8607852619119889, 0.8483997138165847, 0.8445743302351683, 0.8439173326420616, 0.835818249124483, 0.8278345643368121, 0.8271393971761983, 0.8086899906034586, 0.8156285070297585, 0.8107290363662607, 0.8007789380986646, 0.79678271012053, 0.7908136698769517, 0.7939008804025058, 0.7790494402745406, 0.7718979334762596, 0.7714595106345145, 0.76959127456579, 0.7616053603813576, 0.7593618791719621, 0.7601451028331456, 0.7515942870388409, 0.7461007766744988, 0.7360890612950023, 0.7357987864842723, 0.7290879099207358, 0.7344397191275257, 0.735101162071649, 0.7227807775645094, 0.7167677856833982, 0.7123485663458848, 0.7135943363891987, 0.7093136163949204, 0.7009647716449303, 0.7030756313565901, 0.7024557051640326, 0.7054232372279664, 0.6897555005832582, 0.689884144624532, 0.6835680263246853, 0.6774957245424323, 0.6770835789784513, 0.6702086285223812, 0.6681255492848307, 0.6727804815395475, 0.6554642950428349, 0.6694054218606161, 0.6612830821970527, 0.6534064223921917, 0.6548429450886568, 0.6480121609498046, 0.6526517240725034, 0.6449313199794682, 0.640252936652892, 0.6297515515364368, 0.636103159234986, 0.6325427465574602, 0.6223674785305274, 0.6261429400948935, 0.6198489091770816, 0.6168428541030627, 0.6100462815430556, 0.6199705251996058, 0.6156199770387899, 0.607426294495643, 0.6001973913235308, 0.6031141523855738, 0.6036367471867926, 0.5996683532811858, 0.596216335997548, 0.5895073053093958, 0.5829440163292339, 0.5920320066479788, 0.5779523077639569, 0.5806605859742436, 0.5813141898398054, 0.5770847856598028, 0.5756001926276902, 0.5726752373189097, 0.5739306005734476, 0.5653255428935348, 0.560229843929267, 0.561076541989565, 0.5588862380002144, 0.5640194558956192, 0.5564865772341279, 0.5539121483372178, 0.5514954349415774, 0.553604728725196, 0.5422861872006134, 0.5538635066378521, 0.5493824168210295, 0.5450584403956005, 0.5412191780366276, 0.5406999570661375, 0.5373612858417014, 0.541873300535055, 0.535812924953889, 0.5328165614597323, 0.5281892219321208, 0.5291796701216042, 0.5270892710675815, 0.518878385825029, 0.5206755578079364, 0.5219540948659582], 'val_metric': [1.01, 1.29, 1.84, 1.89, 2.64, 2.78, 3.31, 4.25, 4.69, 5.35, 5.88, 6.23, 6.93, 7.59, 8.31, 9.27, 9.63, 10.33, 11.22, 12.23, 12.02, 12.67, 13.62, 13.73, 14.74, 15.14, 15.63, 16.48, 16.75, 17.37, 17.18, 18.49, 18.73, 19.06, 19.41, 19.87, 20.36, 20.68, 20.54, 21.09, 21.06, 21.18, 22.22, 22.72, 22.52, 22.78, 23.43, 23.97, 23.57, 23.77, 23.9, 24.33, 25.12, 24.38, 24.19, 25.24, 24.69, 25.72, 25.74, 26.09, 26.48, 26.45, 26.04, 26.96, 27.47, 26.57, 26.59, 26.92, 26.71, 27.73, 27.6, 27.32, 27.08, 27.56, 27.9, 27.91, 27.25, 27.61, 27.57, 27.07, 27.89, 27.49, 27.79, 27.72, 27.76, 28.56, 27.97, 27.9, 27.38, 28.28, 28.3, 27.22, 27.95, 27.38, 27.13, 28.33, 27.04, 28.12, 27.28, 27.04, 27.32, 27.62, 27.59, 26.73, 27.67, 27.1, 27.62, 25.58, 27.13, 28.39, 26.93, 26.98, 27.36, 26.99, 27.11, 26.44, 27.16, 27.08, 25.82, 26.27, 26.53, 26.68, 27.12, 27.07, 26.76, 25.74, 26.63, 26.17, 26.72, 27.03, 26.37, 26.68, 26.72, 25.53, 26.65, 26.5, 27.18, 25.91, 26.77, 25.59, 26.09, 27.1, 25.85, 25.65, 26.44, 26.76, 26.06, 26.75, 25.76, 26.34, 25.72, 27.24, 26.08, 25.82, 26.12, 25.58, 26.21, 25.14, 25.52, 25.99, 26.02, 26.74, 26.53, 26.35, 25.22, 27.04, 25.83, 26.37, 25.45, 26.35, 26.49, 24.92, 25.56, 25.47, 25.11, 26.61, 25.24, 25.77, 25.51, 25.03, 25.91, 26.02, 26.43, 25.61, 25.79, 25.84, 26.77, 25.73, 25.95, 26.58, 26.15, 25.74, 24.65, 25.38, 25.69, 25.5, 24.75, 25.71, 25.35, 25.65, 25.47, 25.55, 25.64, 26.4, 25.46, 26.12, 26.46, 25.79, 25.54, 26.55, 25.66, 26.34, 26.11, 26.19, 25.5, 25.46, 26.03, 25.46, 26.25, 25.08, 26.39, 25.26, 26.1, 25.94, 25.79, 26.41, 26.88, 25.82, 25.64, 26.22, 26.06, 25.52, 26.77, 26.04, 25.32, 26.35, 26.11, 25.08, 26.07, 25.18, 26.16, 26.9, 25.99, 24.86, 26.38, 24.65, 26.27, 25.14, 26.87, 26.83, 26.21, 24.69, 25.7, 26.41, 26.69, 25.63, 26.74, 25.25, 25.46, 26.24, 26.11, 26.84, 25.71, 26.45, 25.54, 26.06, 25.93, 25.63, 26.55, 25.83, 27.03, 26.35, 26.34, 25.98, 25.46, 26.37, 26.53, 26.15, 25.96, 26.8, 26.34, 25.75, 27.04, 26.33, 26.91, 26.2, 25.42, 25.78, 26.86, 25.8, 26.39, 26.31, 26.35, 25.5, 26.86, 26.55, 26.11, 26.41, 26.34, 26.72], 'val_loss': [5.279759801876772, 5.284878794554692, 5.243849069449552, 5.200242821578008, 5.176108787014226, 5.074020833726141, 4.9854235968012715, 4.901350552868691, 4.808002409661651, 4.758073559232578, 4.679443493010892, 4.615981079210901, 4.544345738781486, 4.478493163540105, 4.4279424931593, 4.347221754159138, 4.273276991145626, 4.238855835738455, 4.180232890851938, 4.111180408745055, 4.103769885506599, 4.055128074755334, 3.9917163499601327, 3.9663978169678122, 3.9192455118628824, 3.8904174087913175, 3.8533549270812113, 3.808076089354837, 3.759494377549287, 3.7459813462700815, 3.7337140095461705, 3.6622409896486126, 3.6290784952746833, 3.615961864495733, 3.586768800285971, 3.5486610155956, 3.5373066838379876, 3.502861083692806, 3.505010700529548, 3.4680140641084902, 3.4649556905600676, 3.4690888436736573, 3.39933520289743, 3.365185677625571, 3.3974971816797925, 3.3504673379242043, 3.344479364194688, 3.301541604813497, 3.3242463048096673, 3.3176295271345007, 3.298093183784728, 3.268357796274173, 3.2361050221570737, 3.264254365757013, 3.276577166690948, 3.2389482609025992, 3.246080592179754, 3.1961801052093506, 3.2075534846372666, 3.211471738329359, 3.1683797532585776, 3.176481510423551, 3.179984132195734, 3.1544546268548177, 3.1329417874099343, 3.154751682737071, 3.191213119561505, 3.150480205845681, 3.1780302327149994, 3.1356747605998043, 3.120398207834572, 3.146484919414399, 3.1463834381407234, 3.153450930953785, 3.123456157696475, 3.1178224451223, 3.201678207725476, 3.1746261135028426, 3.183837996926277, 3.1740053216363213, 3.140674314681132, 3.1949429519616874, 3.154885812170187, 3.16993913756814, 3.1681104579548927, 3.1461310280356436, 3.1749053130483933, 3.1726794971781933, 3.214699003347166, 3.17557458361243, 3.196941510886903, 3.2357920112123915, 3.2240835238414207, 3.246330161762845, 3.2854887983601566, 3.17970860156284, 3.2779391533250264, 3.202948224013019, 3.28510765285249, 3.2978178741066317, 3.295830183727726, 3.2739642790168713, 3.2864212017909735, 3.3693728090091875, 3.331657793111862, 3.3353605528545986, 3.2963168059185053, 3.476758561316569, 3.354975661654381, 3.3016982693581065, 3.401407030737324, 3.3875906042232633, 3.385721997090965, 3.426841668262603, 3.3998217415657774, 3.4964377705458625, 3.402050509574307, 3.394649802499516, 3.504403844001187, 3.4989616309001947, 3.534766210871897, 3.4543539126207876, 3.429649183704595, 3.4644496805348974, 3.5056664526082906, 3.591556050974852, 3.5982181782935077, 3.6013709792665614, 3.535076585544902, 3.5376509617848, 3.546450639226634, 3.5309262435147715, 3.549848110812485, 3.714843390094247, 3.6223662066611513, 3.586271169838632, 3.557471952620585, 3.6484674146980236, 3.5781456376336944, 3.7322757449119712, 3.696308768478928, 3.613214725142072, 3.7348639942278528, 3.7292159643902143, 3.715370086347981, 3.732343031342622, 3.7703300122242824, 3.6441909788520475, 3.7788715415699468, 3.6998802780345748, 3.780725121498108, 3.657722136776918, 3.773353266867862, 3.8069015407258537, 3.833479033913582, 3.822509315363161, 3.7823271986785207, 3.902259794769773, 3.8221993955077638, 3.8859205967301778, 3.8657952311691965, 3.8241643860081957, 3.7680467678483125, 3.8604470795127237, 3.904195892583033, 3.7617074653601192, 3.874627521083613, 3.8886923250878693, 3.9891074447874812, 3.8895343420611823, 3.8792598900521638, 4.0498377092325, 4.01254680126336, 3.967063161977537, 3.973240023965289, 3.8610576748088667, 4.01436265714609, 3.9481312082071973, 3.9743557788763835, 4.091086342835882, 3.9521998159445015, 3.8879369679529954, 3.94555315166522, 4.019339883403414, 3.9744083494137805, 4.015484730908825, 3.8953969433049487, 4.02491958278, 3.988979543090626, 3.978124225974842, 3.9604413524554793, 4.1130958643688516, 4.100305162417661, 4.071892159000324, 4.064443226073198, 4.0480265419953945, 4.17850513746784, 4.04515378794093, 4.109547475341019, 4.081789063040618, 4.120765392188054, 4.171596390426538, 4.0765609551387225, 4.091473496643601, 4.045722283375491, 4.094006960559043, 4.083682745884937, 4.163579381195603, 4.105953401820675, 4.0947144859156035, 4.096465506371419, 4.051435810745142, 4.098279006921562, 4.121216505955738, 4.115833220967821, 4.157950440030189, 4.171492652528605, 4.177813483651277, 4.133168682930576, 4.299535842458154, 4.210354584797173, 4.193629510083776, 4.160248117082438, 4.191919743635093, 4.201407830426647, 4.1583614197506265, 4.135048144941877, 4.163179900236191, 4.2808463193808395, 4.113114985690754, 4.156041098248427, 4.205401434260569, 4.109958390521395, 4.154521591344457, 4.215343983310043, 4.181949962476256, 4.194011101297512, 4.263786051683365, 4.1464700212903844, 4.352949721798016, 4.176764492775984, 4.117638477094614, 4.16240600036208, 4.303369948818426, 4.159206930239489, 4.426024116528262, 4.18098748337691, 4.31400133242273, 4.177339700376912, 4.131831225316236, 4.250786528465854, 4.373747171110408, 4.21400958717249, 4.241843782412778, 4.200845700160713, 4.320002871713821, 4.2255872875262215, 4.350477946791679, 4.380209400395679, 4.265993848727767, 4.298425054094594, 4.206974120656396, 4.320695566523606, 4.179764997427631, 4.269601457437892, 4.270735537170604, 4.290025129439725, 4.268772321142209, 4.159281686612755, 4.220760801795182, 4.168710417808241, 4.304359811886101, 4.258088943305289, 4.338536932969549, 4.368303505478392, 4.322370337832505, 4.222248906542541, 4.271474343196601, 4.316773170878173, 4.219534214135188, 4.273066937543784, 4.353929361719994, 4.208643751539243, 4.249449800533854, 4.241977093326058, 4.3182704630930715, 4.3839030979545255, 4.425677708000134, 4.241830474252154, 4.327666810363721, 4.284597453038404, 4.329014497198117, 4.2466427383908805, 4.346335488519851, 4.238421982261026, 4.262264341305775, 4.309158776216446, 4.326309888226212, 4.330596267797385, 4.232058872842485]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="Adam")
data.update({'train_metric': [2.247, 5.436, 8.336, 11.103, 13.362, 14.638, 15.769, 16.676, 17.669, 18.58, 19.151, 19.86, 20.435, 21.033, 21.493, 21.647, 22.19, 22.363, 22.78, 22.931, 23.42, 23.699, 23.555, 23.947, 24.007, 24.176, 24.439, 24.45, 24.837, 24.886, 24.867, 25.067, 25.357, 25.306, 25.241, 25.66, 25.651, 25.65, 25.779, 25.902, 25.94, 26.033, 26.296, 26.136, 26.23, 26.299, 26.335, 26.467, 26.531, 26.496, 26.532, 26.678, 26.558, 26.697, 26.783, 26.922, 26.829, 26.89, 27.104, 27.06, 27.04, 26.849, 26.85, 27.02, 27.156, 27.095, 27.251, 27.23, 27.344, 27.337, 27.241, 27.494, 27.472, 27.41, 27.239, 27.589, 27.494, 27.418, 27.484, 27.6, 27.654, 27.596, 27.466, 27.602, 27.578, 27.637, 27.698, 27.676, 27.641, 27.705, 27.785, 27.663, 27.707, 27.69, 27.867, 27.873, 27.889, 27.742, 27.836, 27.794, 27.964, 27.967, 27.609, 27.874, 27.759, 27.662, 27.986, 27.915, 28.127, 28.134, 27.798, 27.997, 27.9, 28.083, 27.926, 28.001, 28.034, 28.103, 28.038, 27.928, 27.913, 28.092, 28.116, 28.121, 28.252, 28.009, 28.109, 28.066, 28.07, 27.98, 28.089, 28.094, 28.26, 28.174, 28.237, 28.154, 27.888, 28.09, 28.368, 28.303, 28.147, 28.279, 28.216, 28.271, 28.238, 28.152, 28.161, 28.281, 28.058, 28.238, 28.061, 28.327, 28.194, 28.327, 28.312, 28.23, 28.327, 28.202, 28.416, 28.236, 28.298, 28.197, 28.192, 28.359, 28.117, 28.241, 28.497, 28.313, 28.363, 28.34, 28.42, 28.408, 28.497, 28.462, 28.382, 28.34, 28.396, 28.416, 28.38, 28.361, 28.416, 28.4, 28.418, 28.453, 28.443, 28.416, 28.44, 28.483, 28.294, 28.378, 28.443, 28.531, 28.589, 28.526, 28.332, 28.598, 28.32, 28.284, 28.349, 28.53, 28.614, 28.537, 28.398, 28.738, 28.571, 28.504, 28.569, 28.718, 28.603, 28.533, 28.834, 28.54, 28.643, 28.635, 28.398, 28.514, 28.733, 28.499, 28.505, 28.611, 28.583, 28.727, 28.641, 28.899, 28.603, 28.568, 28.689, 28.57, 28.664, 28.577, 28.614, 28.649, 28.754, 28.608, 28.611, 28.716, 28.651, 28.604, 28.661, 28.696, 28.591, 28.627, 28.61, 28.785, 28.743, 28.684, 28.502, 28.681, 28.73, 28.653, 28.702, 28.694, 28.684, 28.688, 28.714, 28.7, 28.743, 28.729, 28.656, 28.896, 28.707, 28.691, 28.777, 28.712, 28.75, 28.754, 28.883, 28.863, 28.763, 28.478, 28.724, 28.745, 28.709, 28.707, 28.712, 28.846, 28.772, 28.783, 28.709, 28.916, 28.863, 28.736, 28.836, 28.772, 28.849, 28.662, 28.819, 28.748, 28.716, 28.849, 28.714, 28.898, 28.864, 28.798, 28.842, 28.866, 28.902, 28.763, 28.793, 28.905], 'train_loss': [4.9998683972154305, 4.576632452712788, 4.308141686866014, 4.114662012227132, 3.9514855337844623, 3.8581764716333256, 3.7890960160387834, 3.732098383241484, 3.676366021979412, 3.6219557672453018, 3.5889721595546944, 3.5473713784818837, 3.512588361860923, 3.479738270557621, 3.453972066280099, 3.4332765422413463, 3.4142049068223947, 3.400051337514866, 3.3818977762321136, 3.36730234773969, 3.353502966697141, 3.3417043219143507, 3.330246159760371, 3.3209433349675272, 3.310447558636705, 3.2996513573542208, 3.29053057529037, 3.2821770901109497, 3.2746284885125823, 3.2693345223339567, 3.2577735811796082, 3.250565562123148, 3.2449204947654056, 3.2399638725715194, 3.235282453450345, 3.2246447120502983, 3.2197514832668097, 3.2184784055442592, 3.216248722771994, 3.212114365071878, 3.2027494803652576, 3.2029081835475246, 3.197109619623869, 3.1923449138609628, 3.193516071073077, 3.1902953812653485, 3.18996717421885, 3.184313702720598, 3.181159022215918, 3.177259732421514, 3.173462870635059, 3.173019515041808, 3.1702704955886283, 3.165533491990083, 3.1639336589354397, 3.1601546516955397, 3.158322125768631, 3.1563163357366757, 3.1524827622177503, 3.1473909911633453, 3.148141953095517, 3.1496804520173174, 3.145491818167503, 3.142518669538443, 3.143943227412833, 3.1468471075889015, 3.142479534417639, 3.13766091722597, 3.135108984515824, 3.1354765574747527, 3.1345854531628006, 3.12875402698285, 3.127700081866137, 3.1280059709201162, 3.1281595501622097, 3.126109244803626, 3.127538792078722, 3.12943704755201, 3.1243376095784603, 3.1208851727932903, 3.1191005430691376, 3.1216930813768013, 3.1190443029788084, 3.116978450074687, 3.1157368410114135, 3.117117877656347, 3.1151729388185134, 3.120478181753567, 3.114031672553992, 3.109626931710954, 3.1107671694044723, 3.107230931768338, 3.112064337013474, 3.109639005178987, 3.1097198420431242, 3.1074783983553975, 3.1068847142803464, 3.1073445517209883, 3.1053145699064775, 3.104616087518742, 3.1026209174099444, 3.101551581405327, 3.1024105908850865, 3.101602350986698, 3.100088606259034, 3.104742316961746, 3.103092407310764, 3.1003850938720117, 3.098519290935055, 3.0964213907909333, 3.0978113459381476, 3.0976839915194425, 3.0950410779439252, 3.092010588044931, 3.0964617727661623, 3.0988607937452204, 3.095661892924489, 3.0953060775251626, 3.0962875685627766, 3.0950959653177095, 3.0942257056233218, 3.0940766964520083, 3.091316314210361, 3.092238384717867, 3.0890752878695515, 3.0911233516663827, 3.089502967936979, 3.087248266772894, 3.0882120744127994, 3.0955548983541576, 3.0902270744492135, 3.090836835075325, 3.0895800685089365, 3.087993860244751, 3.0873131154060975, 3.086209270333298, 3.090918080134035, 3.0860444974685737, 3.0878697913652493, 3.0836932848297627, 3.0834833396526, 3.084638693358604, 3.0864575958679064, 3.08130384666067, 3.086977179509588, 3.0850725096155265, 3.085378175047217, 3.0838575138926734, 3.084121217196825, 3.0802048652963765, 3.0835191652855656, 3.078092957183633, 3.0825629661423384, 3.0790693724834224, 3.0817643154605565, 3.0803254103706346, 3.0789899829100587, 3.0821286831158363, 3.077436024610308, 3.0763605543992036, 3.077885131994578, 3.0799938589048477, 3.079551552627915, 3.0784348846625917, 3.0816571671308344, 3.0816948153007053, 3.0780161011318175, 3.0765933012703024, 3.0778265171758807, 3.0755313348861666, 3.076599047112297, 3.074440015712306, 3.076887638997513, 3.075316648336839, 3.076880354341298, 3.0787982350347596, 3.0721320834437473, 3.0757854770409017, 3.0727083570134996, 3.0741553941142157, 3.0759171428241108, 3.075654907556047, 3.072053218147195, 3.0695938815387183, 3.073858213516206, 3.0704888383776274, 3.0743583281377305, 3.0702559777123763, 3.073961883802408, 3.0715575353960465, 3.070214853176915, 3.0724264021035728, 3.0705084100565845, 3.0673965167633175, 3.071720010869715, 3.0714403329106075, 3.071429973829273, 3.0741805378931573, 3.070740216676806, 3.0702127553069736, 3.0691283042051056, 3.0670070039562436, 3.066976930114297, 3.062159995123582, 3.066793399519136, 3.066695708917336, 3.063258526955212, 3.061381403178034, 3.064735379039059, 3.0639373813770705, 3.062972909887098, 3.061271892170531, 3.0614277831423533, 3.0661020768581104, 3.0658904449038222, 3.06600066994675, 3.065163142392823, 3.0631027897427803, 3.0658295917266924, 3.0628305297590415, 3.0651987576743998, 3.0615276951707484, 3.0609591920941743, 3.0632036058702914, 3.0630097316002436, 3.065715692169912, 3.0639601539360433, 3.0617146162519986, 3.0618143154883795, 3.06132728017757, 3.0654889876767752, 3.062395745641668, 3.0652452630258415, 3.0641807344428105, 3.062470966779644, 3.058070349220427, 3.0611390725817347, 3.060926625611152, 3.0612872844312866, 3.0608445888746267, 3.0626158830224135, 3.0608438228233763, 3.055574890137939, 3.0580506613066922, 3.0563820302295746, 3.0613710134363448, 3.060737085739009, 3.0574122649770628, 3.058314101023317, 3.059890853542589, 3.057357231013224, 3.058644581969854, 3.0603965954832444, 3.061325902292077, 3.0597196270545477, 3.057405884114886, 3.0567202514665524, 3.056783469304471, 3.057580069708504, 3.0537985393197125, 3.0588665664462957, 3.0578141120939932, 3.056899373377277, 3.0557835146317633, 3.0575766975271037, 3.055108458585489, 3.0539374157776837, 3.0518384687578686, 3.0561582431988463, 3.0596291718388398, 3.053899272313426, 3.0571743043202737, 3.0538905347224925, 3.0557820918997813, 3.057933755662299, 3.0518366503395185, 3.0525241245921424, 3.054670687905505, 3.055937426103969, 3.0560173225646894, 3.055896743169139, 3.0548364980359604, 3.0526462713267204, 3.051380142979491, 3.0514072416992577, 3.0557593051546745, 3.052745590893336, 3.0544101757188677, 3.05476031132562, 3.050976526714332, 3.0551913879807, 3.051376114727516, 3.052350324922392, 3.0515791826803413, 3.052267286004123, 3.0495428671382285, 3.0493402092104454, 3.0526926735617455, 3.0562059888455324, 3.050182991735613], 'val_metric': [2.98, 6.46, 9.59, 11.98, 14.49, 14.87, 16.97, 17.94, 19.46, 20.02, 20.57, 20.82, 21.69, 21.09, 22.95, 23.92, 22.82, 23.67, 23.95, 24.99, 24.49, 24.52, 25.18, 25.46, 25.44, 24.33, 26.1, 26.25, 25.7, 24.7, 25.58, 26.87, 25.86, 26.18, 26.03, 25.91, 25.58, 26.59, 26.46, 26.73, 28.16, 27.14, 27.42, 27.68, 26.68, 27.47, 27.88, 27.85, 25.92, 28.37, 28.55, 26.86, 27.94, 28.0, 26.66, 27.64, 28.32, 27.82, 28.8, 28.62, 27.57, 27.9, 28.0, 27.28, 27.7, 28.22, 27.53, 28.39, 28.33, 29.63, 27.54, 27.9, 28.43, 28.63, 28.4, 28.2, 28.59, 28.44, 27.83, 27.1, 28.63, 29.19, 29.36, 28.67, 28.18, 28.16, 28.37, 28.64, 28.79, 28.88, 28.91, 28.26, 27.85, 28.54, 28.65, 27.68, 28.54, 28.75, 28.91, 28.33, 28.46, 28.97, 28.8, 29.03, 28.82, 29.34, 29.14, 28.94, 28.72, 28.53, 29.43, 28.83, 28.28, 29.3, 29.7, 28.94, 27.85, 28.02, 29.6, 28.81, 28.87, 28.78, 28.03, 29.25, 29.09, 28.87, 28.85, 29.31, 28.92, 29.23, 28.94, 29.02, 29.44, 29.0, 28.38, 29.66, 29.5, 28.77, 29.0, 29.74, 29.61, 28.6, 29.78, 28.62, 28.42, 29.3, 29.18, 28.89, 29.46, 29.05, 27.84, 30.28, 28.72, 29.11, 27.37, 29.7, 29.98, 29.43, 27.7, 28.87, 30.07, 29.28, 28.86, 29.25, 29.11, 29.57, 29.59, 29.54, 30.22, 29.81, 29.32, 28.96, 29.13, 28.95, 29.02, 29.48, 29.34, 28.71, 30.45, 28.31, 29.56, 29.56, 29.26, 30.17, 29.05, 30.52, 28.86, 30.1, 29.16, 29.56, 29.92, 29.03, 29.78, 28.67, 29.49, 29.3, 29.45, 29.26, 29.63, 27.74, 29.17, 29.42, 29.14, 28.14, 29.5, 29.43, 27.84, 29.31, 29.54, 28.64, 30.32, 29.02, 28.55, 29.79, 30.2, 29.51, 29.92, 29.78, 29.72, 29.39, 30.22, 29.37, 30.16, 30.37, 29.11, 29.56, 29.97, 29.83, 29.37, 30.0, 30.3, 27.68, 29.82, 30.3, 29.55, 29.75, 29.26, 29.16, 28.64, 29.57, 29.41, 30.52, 29.72, 28.62, 30.13, 29.54, 29.27, 29.92, 29.86, 29.95, 27.22, 29.63, 29.81, 30.09, 28.82, 29.06, 27.58, 30.5, 30.07, 30.19, 29.97, 29.62, 30.65, 30.77, 28.26, 28.0, 28.42, 30.27, 28.86, 28.42, 29.98, 29.7, 29.83, 29.53, 30.48, 29.98, 30.11, 30.29, 29.88, 30.05, 27.62, 28.65, 28.67, 29.74, 30.32, 28.96, 30.46, 27.71, 29.58, 28.11, 28.47, 29.86, 29.11, 29.98, 30.17, 30.58, 29.6, 29.62, 30.14, 29.71], 'val_loss': [4.8382514965762, 4.463436120634626, 4.211785296725619, 4.0100766443143225, 3.885888295568478, 3.821019360973577, 3.716170292751045, 3.6820483602535954, 3.568782850435585, 3.523629464161624, 3.4960953733723636, 3.493215036240353, 3.4463792674860376, 3.4477696661736554, 3.361575890498556, 3.3536224312083736, 3.364144660864666, 3.3312691844952336, 3.2938266392726048, 3.2654592315102837, 3.288997511954824, 3.3013556481926303, 3.257802030842775, 3.2497601987449984, 3.241132496268886, 3.2866064575827045, 3.2131495240387644, 3.1938337474871594, 3.2454737493187, 3.238258169714812, 3.2121156173147214, 3.1766514322560306, 3.190904187548692, 3.179952559957079, 3.1955110313026767, 3.20095975156043, 3.2082399631002145, 3.1502929166623743, 3.1542374032318214, 3.1531691323420046, 3.126333380960355, 3.1489059833963964, 3.12254084143669, 3.1118644445565096, 3.1541816558048223, 3.1174826530893895, 3.1298741633725014, 3.110457731660005, 3.2408720573801904, 3.0960616905977774, 3.084181931368105, 3.154785255717624, 3.095908972867735, 3.1285495727684847, 3.132091543477052, 3.1036205117110236, 3.0974692637753334, 3.1290896758911715, 3.0601175742544187, 3.0591520663279637, 3.1189941615815373, 3.1094066770213424, 3.085743535855773, 3.116933925895934, 3.1022949947673046, 3.090740692084003, 3.117999402580747, 3.076045603509162, 3.0711136638738545, 3.015811694655449, 3.111293869413388, 3.1060114871164797, 3.0604750221701944, 3.051110670824719, 3.061352338001227, 3.0602101345730435, 3.0561354691815223, 3.0878283643418816, 3.086721358025909, 3.1333873188419705, 3.053760842153221, 3.0421902419655185, 3.0539528647805474, 3.0507603094076656, 3.067537301664899, 3.0736061099228587, 3.053493907497187, 3.0510861995113885, 3.0481581369023414, 3.041864065607642, 3.0485639017858324, 3.0860267308107607, 3.0825311029033298, 3.0465631652030214, 3.072078772411225, 3.0867834022849987, 3.0614813626951474, 3.062038357090798, 3.0484905508673115, 3.051417449477372, 3.055225350294903, 3.0351744275184194, 3.028103039522839, 3.0354014794537973, 3.0392369426739445, 3.0306484441089023, 3.0093160115989153, 3.043316664209791, 3.0375218133258213, 3.041131142598049, 3.0356977654110855, 3.0690724310601594, 3.0857934921410433, 3.006929552479155, 3.0170300508001047, 3.037716448686685, 3.097972185748398, 3.078238610249416, 3.031698521535108, 3.042990631358639, 3.0172474361529016, 3.036054838235211, 3.0664694013109632, 3.033756342663127, 3.0496476706425857, 3.053030092245454, 3.0203435899345736, 3.0293538737448915, 3.0417435518495597, 3.0008830720451987, 3.055352390192117, 3.0586887992871037, 3.010247686865983, 3.038306178560682, 3.0645719842546306, 3.0364613745622573, 3.0257089206367542, 3.044743884141278, 3.036730853615293, 3.0129636024973196, 3.0194109374550497, 3.0481257294393647, 3.0109289823823673, 3.040317876323773, 3.065457142842044, 2.999479162465235, 3.0319798197715904, 3.047257610187409, 3.0318344003835302, 3.027818609954445, 3.079829132480986, 2.984013750294971, 3.022730481093097, 3.0478567727811776, 3.1014144724341715, 3.006592785476879, 2.99622783463472, 3.022126996593111, 3.0854575254355265, 3.0345498027315565, 2.988278153595651, 3.0105650599595086, 3.0358068305216017, 3.016473783808909, 3.020244963609489, 3.013832689090899, 3.0000009468406628, 3.0069987014600428, 2.9966921312793806, 3.0015676264550275, 3.0232851808997476, 3.0494968238150237, 3.057650332238264, 3.024429899871729, 3.0315501317856417, 3.015420280444394, 3.009549032351014, 3.0471400030099662, 2.975423990541203, 3.0415882707401445, 3.0283309388312563, 3.0117506001405654, 3.01790379338963, 2.998164365246038, 3.0349577171787336, 2.9932038814398894, 3.067357518111065, 3.009371797749951, 3.034034113215793, 3.001936388623183, 3.001373212048962, 3.0423649169836833, 2.9904732058761985, 3.05391715970009, 3.0228044637449227, 2.9922899410223507, 2.99861699001045, 3.0260356558356314, 3.0218770914017017, 3.076836597387958, 3.022910629867748, 3.015695344110963, 3.032395479026114, 3.0850165087705963, 3.009845284140034, 3.03887199292517, 3.1001902651634947, 3.0168650294565094, 3.0149130502324195, 3.081379064329111, 2.983108044429949, 3.070839385318149, 3.0510752512391206, 3.020378734655441, 3.0022587145969366, 3.0218519891143605, 2.9890341728356233, 3.0051089593559315, 2.991020847278036, 3.015216607197075, 3.010378060067535, 3.035353632489587, 2.9866837893322016, 2.9744729266804493, 3.0464588722605614, 2.9983105537997687, 3.00483079396995, 3.014424843393314, 3.0459023919075157, 3.0093924369022345, 2.993278581625337, 3.087699664626152, 2.9947367010602526, 2.9972020767296956, 2.996607746288275, 3.011972337771373, 3.025674262623878, 3.0211029052734375, 3.102565490516128, 3.034640134519832, 3.005408932448952, 2.9700024416492243, 3.0112769178524137, 3.0663705335301197, 3.011460892713753, 3.014760511695959, 3.028867493009871, 2.988098290315859, 3.000577366276152, 3.009204732384651, 3.13714948657212, 3.0148823170145604, 3.019751448540171, 3.006940268407202, 3.031723686084626, 3.0468947614074513, 3.1441461971610973, 2.975900344787889, 2.994422586860171, 2.9945659857646674, 2.988103957692529, 2.9936719396311764, 2.9656854716076215, 2.9555914804434322, 3.0748520421374375, 3.0925491311747555, 3.0642534669037835, 2.9913723142283737, 3.0355280713670574, 3.0416610711699077, 3.0030186912816044, 3.0069090970762216, 2.9967277361329194, 3.032727847433394, 2.9791499885024537, 3.007927965966, 2.9850763416594, 2.9773572759263836, 2.991230902398468, 3.0058800156708734, 3.0850828271003286, 3.057085377395533, 3.036600028633312, 3.0129687960740106, 2.9881511569782426, 3.044423133704313, 2.9886929692736097, 3.0621483553746702, 3.023846059847789, 3.0839048684782284, 3.0837215291466684, 3.0079381302663477, 3.0200585217992213, 2.985417577111797, 2.99238212852721, 2.9640925705053243, 3.035838345813144, 3.0161817240866884, 2.9910241327467997, 3.0228223603242523]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="AdamW")
data.update({'train_metric': [4.521, 14.764, 24.199, 31.323, 37.275, 42.7, 47.407, 52.142, 56.401, 60.873, 65.04, 69.313, 73.237, 76.729, 79.79, 82.47, 84.7, 86.416, 87.63, 88.715, 89.471, 90.372, 91.084, 91.39, 91.973, 92.098, 92.815, 93.009, 93.493, 93.424, 93.979, 94.102, 94.225, 94.554, 94.949, 94.869, 94.945, 95.136, 95.303, 95.328, 95.538, 95.862, 95.674, 95.934, 95.941, 95.894, 96.234, 96.173, 96.315, 96.394, 96.441, 96.659, 96.377, 96.667, 96.719, 96.673, 96.636, 97.01, 96.922, 96.834, 97.027, 97.063, 97.06, 97.053, 97.182, 97.206, 97.255, 97.246, 97.278, 97.296, 97.388, 97.448, 97.247, 97.483, 97.497, 97.286, 97.749, 97.542, 97.666, 97.508, 97.581, 97.709, 97.734, 97.738, 97.76, 97.752, 97.799, 97.733, 97.823, 97.725, 97.89, 97.917, 97.772, 97.988, 97.849, 97.996, 97.882, 98.032, 97.869, 98.077, 98.035, 98.026, 98.07, 98.0, 98.065, 98.044, 98.15, 98.093, 98.137, 98.121, 98.237, 98.103, 98.286, 98.084, 98.242, 98.197, 98.246, 98.226, 98.341, 98.309, 98.2, 98.32, 98.259, 98.283, 98.264, 98.275, 98.359, 98.255, 98.477, 98.292, 98.443, 98.362, 98.353, 98.407, 98.403, 98.513, 98.268, 98.44, 98.369, 98.436, 98.415, 98.439, 98.34, 98.516, 98.498, 98.558, 98.435, 98.379, 98.554, 98.509, 98.635, 98.513, 98.495, 98.443, 98.62, 98.521, 98.628, 98.543, 98.602, 98.465, 98.596, 98.572, 98.651, 98.54, 98.649, 98.661, 98.499, 98.693, 98.558, 98.725, 98.706, 98.595, 98.634, 98.675, 98.627, 98.718, 98.599, 98.678, 98.643, 98.664, 98.648, 98.772, 98.602, 98.76, 98.561, 98.713, 98.718, 98.643, 98.721, 98.717, 98.702, 98.622, 98.742, 98.774, 98.737, 98.715, 98.699, 98.783, 98.74, 98.689, 98.69, 98.783, 98.735, 98.744, 98.8, 98.76, 98.814, 98.771, 98.783, 98.691, 98.918, 98.776, 98.839, 98.838, 98.823, 98.828, 98.779, 98.721, 98.828, 98.871, 98.74, 98.859, 98.741, 98.782, 98.932, 98.841, 98.744, 98.878, 98.799, 98.942, 98.893, 98.816, 98.779, 98.822, 98.892, 98.928, 98.878, 98.833, 98.92, 98.877, 98.922, 98.968, 98.889, 98.816, 98.824, 98.924, 98.979, 98.852, 98.922, 98.897, 98.963, 98.948, 98.852, 98.916, 98.869, 98.992, 98.867, 98.934, 98.93, 98.898, 99.013, 98.937, 98.94, 98.948, 98.977, 98.913, 98.924, 98.893, 99.005, 98.944, 98.932, 98.942, 98.966, 98.949, 98.959, 98.997, 98.961, 98.898, 98.966, 98.982, 99.012, 99.096, 99.065, 98.948, 98.984, 99.024, 98.927, 99.048, 98.946, 98.992, 98.963, 98.981, 99.025, 99.003, 98.927, 98.991, 99.076, 98.97, 99.012, 98.903], 'train_loss': [4.742311917278756, 3.844641852973748, 3.2839842474163152, 2.906002356162532, 2.5994595751423755, 2.3344279448343865, 2.104256932810187, 1.8897841326792273, 1.6827644001804554, 1.4815312471057236, 1.2928127165185437, 1.1122650599449166, 0.9454861770664662, 0.8014961902674238, 0.6832346954142826, 0.5817417795754974, 0.5060267200835302, 0.449123894908988, 0.4010564242456864, 0.36646844460006256, 0.34130796790122986, 0.31017347400434797, 0.28839681201488715, 0.2765833382171198, 0.2577690080985646, 0.2521686835318175, 0.23215418344486316, 0.22425718773907183, 0.20718207077426515, 0.20988611528865625, 0.19233194796662834, 0.1886872363325044, 0.18214708411385636, 0.17468459671393027, 0.1642310987354774, 0.1638392960177216, 0.16050144655109214, 0.15533128389653703, 0.14931921292542077, 0.14826874973363244, 0.14343649967051017, 0.13442067934172758, 0.13854496407972663, 0.1299753040522194, 0.1294180236313485, 0.1316431913115513, 0.12135697173828203, 0.12319376965882214, 0.1174083944834335, 0.11534125703343465, 0.11306055460330182, 0.10804513561681166, 0.11520289534807768, 0.10664823951356957, 0.10383038931143108, 0.10436938540272353, 0.10514295362269771, 0.0963233069898662, 0.09856662735901892, 0.1016944889518446, 0.09583807151832559, 0.09463506840557928, 0.09260131002380557, 0.09158509191925866, 0.08879221710647556, 0.08759182326269818, 0.08625005132275004, 0.08815702751591106, 0.08619860181959152, 0.08687169030958206, 0.08280011794457756, 0.08142276211050044, 0.08712872055907968, 0.07920333389499686, 0.07911722422729384, 0.08484670548921455, 0.07155609013833301, 0.07767549151899109, 0.0745691794688174, 0.07855772188109698, 0.07614351152719469, 0.07303430936521221, 0.07248273919177405, 0.07205404179654562, 0.07128694203356907, 0.07238913992937042, 0.06759139020633494, 0.07224845954381473, 0.06908001998881556, 0.07292290635277224, 0.06611197339627065, 0.06762394889258444, 0.07015096362564303, 0.0635478863955439, 0.06570725241852943, 0.06459530428175887, 0.06618241300460509, 0.06098021931859283, 0.06615052334261165, 0.061727839300256424, 0.06325501394307662, 0.06360687815275291, 0.06161083468258076, 0.06314313730495485, 0.061617271823447166, 0.061248521278067104, 0.05982368305390754, 0.060101489006227055, 0.05957942760697854, 0.05934909437966288, 0.05664495637110641, 0.059697707407253955, 0.054797614806630215, 0.06015963830515297, 0.05627795835594651, 0.05828592631948298, 0.0560924741812676, 0.05562141589055157, 0.05321796021262757, 0.054718685672399174, 0.05616844641961609, 0.05256419949157114, 0.05547553673036509, 0.05214680722060775, 0.055047115596702174, 0.05398733475181034, 0.05236374980807888, 0.05531888841706497, 0.04888795329775049, 0.0538711817619678, 0.049678624631792026, 0.05222762935437079, 0.05332480461181154, 0.050600709117143414, 0.050708793790716626, 0.04677647742999465, 0.054439741692828135, 0.04922617283538466, 0.05095366826312234, 0.049796347573452575, 0.049582485434458, 0.04909128202439744, 0.05136536928737639, 0.04784886564771774, 0.04791503762762269, 0.044687240181797795, 0.0501210987779058, 0.04962401175677093, 0.04605616618517455, 0.047831508343848944, 0.04441397147853034, 0.04850371393858636, 0.047154896496549324, 0.0485324138452663, 0.04425476295243405, 0.046536521382673966, 0.043302526408563635, 0.04631476467046279, 0.04415332367366641, 0.04655737450369589, 0.04486555844253097, 0.04540984116403639, 0.04291203464497589, 0.04613402040663142, 0.04369011008218217, 0.043564889880508884, 0.045905826278566396, 0.0420462384090657, 0.046157918084217375, 0.04045746910585548, 0.041332787315340944, 0.04496333096824379, 0.04358301895256622, 0.04239012401317731, 0.04394898900166493, 0.04159487679459051, 0.04476370189192342, 0.0427603422612825, 0.04288759894668088, 0.042387825389221666, 0.042378297502950926, 0.03902013432128017, 0.04470759054096368, 0.039770607298579855, 0.04474158542333808, 0.04111147689098589, 0.0400922361127341, 0.042569695435077375, 0.0397792481224801, 0.039597650868391385, 0.04098563901184786, 0.04351336044929409, 0.03984139234832204, 0.03908724695087636, 0.04074817257940022, 0.04060171701477916, 0.041505585597716756, 0.03852762681631978, 0.03934355356260316, 0.04152309445863749, 0.04090941011666881, 0.03880440707286308, 0.039912476118122805, 0.0391193972213786, 0.03766992631144446, 0.038424083344575484, 0.038904753794598824, 0.03913051291041256, 0.03783596400393102, 0.04018370580000854, 0.03591074254145543, 0.03871453406920976, 0.03785370178474501, 0.03711893403749887, 0.03881571188068215, 0.03675934461022367, 0.03854622758583238, 0.040691571559193014, 0.036982510067651825, 0.0357173192700434, 0.03804580664837325, 0.03665915980333804, 0.03934214142577512, 0.037846207791027424, 0.034068234497830224, 0.0360207668703098, 0.03978726196457183, 0.03437467005340828, 0.03853283496178813, 0.033949959858727174, 0.03585581105073285, 0.037200673077385386, 0.037713476509490855, 0.03730729445542482, 0.03619614241070894, 0.03530766730768535, 0.03582477094593648, 0.03715307777496135, 0.03376926171743785, 0.03524423722680572, 0.035302322627871516, 0.03397517488764463, 0.03530413541223786, 0.03759584159332447, 0.036623446722891806, 0.03384830888156435, 0.03225821980807164, 0.0368825853750685, 0.03395398592899487, 0.03428492248277668, 0.03349319186586309, 0.0335634774852999, 0.035898736082543264, 0.03356028433925051, 0.03499917888613769, 0.03298652417227413, 0.03658967383293556, 0.03441377058599525, 0.03350509099953119, 0.035371835996243856, 0.03235672716651218, 0.03421870760063688, 0.032982269114613896, 0.033076772586646166, 0.03306217648712942, 0.03471898336163993, 0.03441854252247065, 0.0352800229084608, 0.03168183946862669, 0.03411400886927329, 0.03354727969308186, 0.034196666761078236, 0.032490007696691595, 0.03390913697277504, 0.03317580379776061, 0.03175093093802248, 0.03344996191991505, 0.033944175834220075, 0.033238631744455086, 0.03220650027794485, 0.03253938692837475, 0.03024177631581207, 0.029982564472101042, 0.03426628827718974, 0.031294100331581615, 0.03165231011278724, 0.033357403828183446, 0.029893823915025784, 0.03423906262640088, 0.03207304361150454, 0.03301570300879499, 0.03127915992657237, 0.031000826405915442, 0.03189042167401719, 0.0331493170990925, 0.032944161353810966, 0.0300108859445318, 0.0331277744573907, 0.03110101382392487, 0.03501841408208085], 'val_metric': [9.89, 21.51, 29.37, 33.74, 38.41, 41.69, 44.53, 46.37, 47.5, 47.55, 47.9, 49.28, 49.0, 48.42, 48.91, 49.57, 48.5, 48.24, 48.21, 48.49, 48.45, 49.13, 48.62, 48.3, 48.81, 49.37, 49.17, 49.12, 48.06, 48.95, 50.07, 49.63, 49.98, 49.5, 49.98, 49.58, 49.68, 50.36, 49.23, 49.43, 50.02, 49.35, 50.09, 50.11, 49.76, 49.86, 50.07, 50.36, 50.62, 50.09, 49.53, 50.46, 50.48, 49.97, 50.61, 50.56, 50.12, 50.15, 50.5, 50.21, 50.61, 50.33, 50.6, 50.45, 50.74, 50.48, 50.62, 50.72, 51.1, 50.9, 51.38, 50.81, 50.63, 51.33, 50.42, 50.79, 51.59, 50.87, 50.87, 50.74, 50.76, 50.94, 50.71, 51.08, 50.27, 50.81, 50.64, 51.06, 51.23, 50.48, 50.63, 50.67, 50.7, 51.24, 50.53, 50.83, 50.7, 51.27, 51.26, 51.17, 50.67, 51.18, 51.13, 51.58, 50.73, 51.74, 51.25, 51.44, 51.01, 51.69, 51.52, 51.28, 51.38, 51.18, 51.55, 50.75, 51.85, 51.94, 51.99, 51.16, 51.63, 51.68, 51.55, 51.33, 50.96, 51.72, 51.42, 51.52, 51.23, 51.17, 51.52, 51.57, 51.74, 51.19, 51.31, 51.0, 51.78, 51.25, 51.78, 51.48, 51.4, 50.94, 51.84, 51.56, 51.79, 51.35, 51.81, 51.55, 51.37, 51.82, 51.39, 51.24, 51.25, 51.62, 51.48, 51.25, 51.85, 52.13, 51.81, 51.64, 51.47, 51.11, 51.49, 51.48, 51.81, 51.63, 51.4, 51.46, 51.31, 51.49, 51.54, 51.46, 51.7, 51.82, 52.06, 51.69, 51.76, 51.62, 52.05, 52.87, 51.6, 51.76, 51.74, 51.94, 51.5, 51.89, 51.88, 51.6, 51.72, 51.73, 52.06, 52.09, 52.24, 51.84, 52.03, 52.2, 52.52, 52.91, 52.15, 51.92, 51.39, 52.27, 51.87, 52.69, 52.43, 51.48, 52.34, 52.65, 51.94, 52.22, 52.2, 52.37, 52.19, 52.44, 52.68, 51.83, 52.07, 51.55, 52.26, 52.3, 52.13, 52.09, 52.41, 52.34, 52.32, 52.65, 52.39, 51.82, 52.31, 52.0, 51.97, 52.83, 52.03, 52.16, 52.22, 52.54, 52.67, 52.35, 52.4, 52.37, 52.73, 52.64, 52.84, 52.68, 52.41, 52.96, 51.82, 51.98, 51.63, 52.42, 51.66, 52.72, 52.7, 52.09, 52.48, 52.58, 52.68, 52.45, 52.64, 52.28, 52.4, 53.35, 52.41, 52.35, 52.26, 51.75, 52.74, 52.73, 52.42, 52.57, 52.72, 52.86, 52.52, 52.57, 52.09, 52.41, 51.72, 51.98, 52.18, 51.75, 52.37, 52.71, 52.52, 52.17, 51.89, 52.6, 52.71, 52.96, 52.48, 52.83, 52.55, 52.17, 52.75, 52.38, 52.71, 52.54, 52.05, 52.61, 52.32, 52.31], 'val_loss': [4.185602356673805, 3.441574616037357, 3.0589561644633103, 2.802880311467845, 2.5715892565478184, 2.4140381380251257, 2.3047679545013766, 2.2301696333915566, 2.18962531636475, 2.2258272987262457, 2.269688027299893, 2.3324159645730522, 2.3677291433522654, 2.510087026911936, 2.523268036401955, 2.6475777314726714, 2.75762172375515, 2.830574621060851, 2.9008147154644037, 2.9059680347230024, 3.010901310641295, 3.0231515385542704, 3.0293546755602407, 3.0912737603400164, 3.1691211021629866, 3.167841225293032, 3.215706445608929, 3.203602829936204, 3.2855956448111563, 3.217963962038611, 3.248464986397203, 3.2785299716481737, 3.352739376247309, 3.2590957257398374, 3.3130613789436922, 3.3226391175749956, 3.353555773473849, 3.34533768626535, 3.3556979010059576, 3.4134049992652455, 3.4064623166801065, 3.5801948559512, 3.3817104484624925, 3.433897555633715, 3.4154069932403077, 3.444604016793002, 3.492354053980226, 3.4451373697845797, 3.4449404109815123, 3.4755173568512983, 3.5597771983237783, 3.4702545222203445, 3.499404418620334, 3.5427135498660385, 3.605891270242679, 3.4771586838801194, 3.5845105059587272, 3.5538001838763047, 3.557972118353388, 3.6365768138770087, 3.578246254829844, 3.557691700139623, 3.5393796893441754, 3.535449733400041, 3.569252168676656, 3.571833571810631, 3.596849884576858, 3.5850694654094184, 3.6164147553929857, 3.634145995234228, 3.5528404143205874, 3.6306035514849766, 3.623416097300827, 3.5947013637822147, 3.672646907104808, 3.6142563785716986, 3.623419331517189, 3.5773427774951716, 3.589363592065823, 3.6173263169397973, 3.606906953890612, 3.6354463005521493, 3.647496433774377, 3.6932543295963556, 3.719200217040481, 3.6609512818087437, 3.733684281634677, 3.656230196451685, 3.6313847132549166, 3.7113353598649335, 3.702421423735892, 3.7957516487236993, 3.6735105339888556, 3.702904709205506, 3.9167435461548483, 3.7185112093664277, 3.755533147010074, 3.694619010208519, 3.7589961806679986, 3.7001792841656194, 3.800937400122357, 3.738785141212925, 3.7836588628732475, 3.770786970663982, 3.727732088915102, 3.699353753381474, 3.7292428756975067, 3.7919726037675408, 3.798022180605846, 3.7192629685827123, 3.7510900782172087, 3.784338355823687, 3.749908332232457, 3.765743398362664, 3.7639884838632716, 3.7416590717947407, 3.7150193270604324, 3.686641512403063, 3.758454919620684, 3.900303530844913, 3.7894573538166703, 3.7874501579126734, 3.756166877640281, 3.775586117984383, 3.7934515350943157, 3.7809845964620066, 3.814909325663451, 3.7775589694642715, 3.7972618842580514, 3.8140909360472564, 3.7704855245389757, 3.7904263746206928, 3.8166011754114915, 3.783496254188999, 3.7751414483519876, 3.881607866970597, 3.8003191777095675, 3.842166755989099, 3.848685734590907, 3.83554551441958, 3.807053644186372, 3.8193678107990583, 3.840143078451703, 3.827471004929512, 3.801845459421729, 3.873783754694993, 3.8675115765280026, 3.8325481691937537, 3.860687842414637, 3.8393614611048608, 3.903327484039744, 3.92631670016392, 3.8507060154228454, 3.892712250636641, 3.8700729798359474, 3.8389230993143313, 3.8848903706878612, 3.8251929040167743, 3.881589234254922, 3.873797470596945, 3.9141477756439502, 3.875256898676514, 3.881326256663936, 3.8101285472037687, 3.81347407808729, 3.8482202807809136, 3.889722632374733, 3.860625407118706, 3.9297302107142795, 3.8957057580067094, 3.9184563740803178, 3.8996936382761427, 3.8284015735243537, 3.827253297635704, 3.8624631713150412, 3.868740774643649, 3.90179807555144, 3.940359555991592, 3.8573475449707857, 3.8070665252436497, 3.851571824520257, 3.913479135674276, 3.821347295858298, 3.849215676070778, 3.8548643577630353, 3.8603216189487726, 3.8876416220027172, 3.9173935697336866, 3.8127596196095657, 3.8895048210575323, 3.887240628907635, 3.8884154250667353, 3.8611322527478453, 3.877867521374089, 3.881811849250915, 3.889472638345828, 3.8234846113593717, 3.862437602061375, 3.889266408932437, 3.9214864847766364, 3.909938262906044, 3.86342526772979, 3.904935736944721, 3.918658704514716, 3.910193272077354, 3.9121278478841113, 3.8411062014330724, 3.8659960627555847, 3.8940195027430344, 3.876490917555086, 3.8582096661731695, 3.901684169936332, 3.8835981082005104, 3.896003498013612, 3.934944432632179, 3.9147836831724567, 3.875021833902711, 3.882716024757191, 3.845830259809069, 3.8984226853984176, 3.870764805632792, 3.9099960323351963, 3.9156795907172426, 3.906380076696918, 3.936232587714104, 3.9213916796028236, 3.892413485961355, 3.9044422636366196, 3.898704479074782, 3.914587981002346, 3.9301874497134213, 3.871354135738057, 3.8704356448665544, 3.86868087045706, 3.8748712869966107, 3.929789516576536, 3.864320267917244, 3.875037575982938, 3.938919415899143, 3.9050757364862285, 3.948146353861329, 3.953856032365447, 3.8869335461573997, 3.934932490063321, 3.9454701125242146, 3.940773569094907, 3.9387660216374, 3.902078034012181, 3.9348952090664273, 3.9354584000672506, 3.9624981436000506, 3.9217887084195566, 3.9746376777150827, 3.9802161933510165, 3.9681256356512664, 3.947547384128449, 3.944352824596842, 3.90432378668694, 3.911813806576334, 3.947481076428845, 3.956077923820277, 3.88242242366645, 3.9453733643149116, 3.944551588623387, 3.943963840508917, 3.9430384601757025, 3.909252291652048, 3.9276091072969375, 3.9267487723356598, 3.928918675252586, 3.9875736335280596, 3.9450139380564355, 4.013075359687684, 3.985678840215039, 3.948349551410432, 4.0020576058679325, 3.973430368930671, 3.936802987080471, 3.939499404779665, 3.978340661449797, 3.935775993735927, 3.94005933005339, 3.960828205202795, 3.989516418450957, 3.9576605539413015, 3.9862742120293295, 3.916213868150286, 3.968182328020691, 3.911283482032217, 3.951827515082754, 3.8959092194107687, 4.023052663939773, 3.931077298085401, 3.9821583116130466, 4.00068077691801, 3.929580583693875, 3.9923493095264315, 3.96919495284937, 3.977987957987816, 4.0178791675598]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal 2", optimizer="SGD")
data.update()
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal 2", optimizer="Adam")
data.update({'train_metric': [3.588, 8.103, 10.615, 12.377, 14.571, 16.051, 17.425, 18.353, 19.538, 20.705, 21.409, 22.14, 22.831, 23.066, 23.665, 24.06, 24.344, 24.716, 24.914, 25.278, 25.581, 25.941, 26.092, 26.394, 26.629, 26.909, 27.092, 27.158, 27.391, 27.522, 27.634, 27.645, 27.969, 28.253, 28.328, 28.433, 28.604, 28.658, 28.617, 28.836, 28.925, 29.021, 29.087, 29.423, 29.285, 29.503, 29.47, 29.608, 29.75, 29.551, 29.762, 29.898, 29.958, 29.846, 30.098, 30.184, 30.147, 30.12, 30.464, 30.438, 30.429, 30.3, 30.417, 30.426, 30.368, 30.523, 30.437, 30.674, 30.654, 30.811, 30.57, 30.624, 30.793, 30.675, 31.073, 30.948, 30.667, 30.857, 30.917, 30.903, 31.084, 31.016, 31.009, 31.109, 30.923, 31.063, 31.147, 31.223, 31.34, 31.193, 31.088, 31.146, 31.227, 31.13, 31.199, 31.319, 31.316, 31.286, 31.27, 31.283, 31.26, 31.33, 31.569, 31.422, 31.625, 31.566, 31.351, 31.341, 31.588, 31.44, 31.509, 31.559, 31.458, 31.724, 31.549, 31.62, 31.59, 31.765, 31.596, 31.707, 31.758, 31.823, 31.649, 31.685, 31.693, 31.53, 31.573, 31.741, 31.825, 31.761, 31.769, 32.027, 31.922, 32.123, 31.902, 31.833, 31.908, 31.993, 31.872, 31.963, 32.1, 31.849, 32.008, 32.054, 31.922, 32.078, 32.058, 31.816, 32.02, 32.113, 31.964, 32.185, 32.026, 32.052, 31.9, 31.993, 32.093, 32.194, 31.988, 32.166, 32.04, 32.311, 32.135, 32.369, 32.102, 32.258, 32.302, 32.159, 32.182, 32.075, 32.332, 32.324, 32.35, 32.209, 32.201, 32.176, 32.146, 32.406, 32.43, 32.373, 32.271, 32.477, 32.538, 32.541, 32.38, 32.366, 32.499, 32.53, 32.321, 32.438, 32.437, 32.316, 32.557, 32.681, 32.388, 32.464, 32.549, 32.428, 32.373, 32.474, 32.581, 32.475, 32.445, 32.61, 32.519, 32.706, 32.58, 32.531, 32.557, 32.481, 32.538, 32.568, 32.528, 32.589, 32.588, 32.621, 32.534, 32.682, 32.65, 32.531, 32.502, 32.693, 32.677, 32.9, 32.535, 32.655, 32.777, 32.698, 32.549, 32.747, 32.64, 32.61, 32.618, 32.711, 32.643, 32.712, 32.735, 32.724, 32.816, 32.627, 32.745, 32.837, 32.778, 32.718, 32.833, 32.59, 32.725, 32.75, 32.837, 32.775, 32.742, 32.704, 32.895, 32.784, 32.974, 32.88, 32.844, 32.904, 32.647, 32.922, 32.712, 32.921, 32.859, 32.778, 32.789, 32.727, 32.712, 32.791, 32.865, 32.802, 32.968, 32.885, 32.976, 32.926, 32.845, 32.957, 32.711, 33.031, 32.87, 32.717, 32.88, 32.733, 32.925, 32.743, 32.936, 32.998, 32.87, 32.76, 33.104, 32.797, 32.728, 32.84, 32.977, 32.903, 32.808, 32.831, 32.857, 33.056, 32.932, 32.964], 'train_loss': [4.800513880419106, 4.300039448192008, 4.09327491460057, 3.9712992295651426, 3.8434428494478445, 3.7345680555928156, 3.660745038943495, 3.599914735963691, 3.5309934103557565, 3.4739535771648775, 3.430637295857806, 3.395775542378197, 3.3631478199498135, 3.3404407931380904, 3.314863959109257, 3.2966588771427126, 3.275938299170535, 3.257278734190069, 3.241124438232744, 3.230749176163286, 3.2110281849395594, 3.1953269953309764, 3.181113281695414, 3.175583929872177, 3.1603953228192077, 3.15580119876166, 3.143620332692269, 3.1310206565503043, 3.121640106385439, 3.1144251277335395, 3.1066210201285394, 3.099389657711876, 3.0870058089590042, 3.0837020849617187, 3.0734424238699183, 3.0725642875151533, 3.061561541456636, 3.0548018968341752, 3.0519250894462306, 3.0434106224557946, 3.0397558942942458, 3.041244005439988, 3.0335673332519435, 3.0279456727106604, 3.026549894841756, 3.019429390657734, 3.0163558821455454, 3.0100751813984954, 3.0109630268655825, 3.0084764586758017, 3.0018482488923857, 2.996141196822632, 2.991866187262215, 2.9938223305529537, 2.989266694545441, 2.9895668626212952, 2.984542562499385, 2.9817953362956082, 2.975491627774327, 2.971773942616683, 2.971782541702134, 2.971859346043202, 2.969511767800466, 2.9640561657232576, 2.966191225759661, 2.9599307858631234, 2.9613553821773615, 2.956160255677412, 2.953825664459248, 2.956731978853925, 2.954658258289232, 2.9528125517656614, 2.94925468408825, 2.948501798943381, 2.941747678218556, 2.9423830442068595, 2.939173510954766, 2.9420797271142765, 2.9369585614744396, 2.936750933365874, 2.936383262629396, 2.931955702855506, 2.934625643762502, 2.9346886475880942, 2.9305191962900485, 2.9277527513827413, 2.928209329215823, 2.9237032600190496, 2.922273899756863, 2.9278537790819534, 2.9226054564089785, 2.922428571026217, 2.921644348405678, 2.922253578653415, 2.915777502499249, 2.916597327130465, 2.9150215987585635, 2.9175891960117197, 2.9138590956985073, 2.916904650967013, 2.9121921533815236, 2.911275217491926, 2.910032526163893, 2.911468436149016, 2.9103991732106174, 2.9058303622503883, 2.9113460380300373, 2.9045602863443563, 2.907568097800989, 2.906750920485474, 2.903499142145851, 2.903599378014709, 2.9049836962328306, 2.9002716815860623, 2.902427896809593, 2.899086196683419, 2.8994130340815354, 2.8970121966671347, 2.8954027081939246, 2.895037722602839, 2.8967931391105237, 2.8936001256880512, 2.895449764516517, 2.8932938276768034, 2.8911710572563067, 2.893553190176409, 2.890919104037343, 2.889274991786564, 2.8910382468198557, 2.8904030750137983, 2.8891007973456793, 2.8846923086174923, 2.8842944522279237, 2.8865956947426725, 2.8859442148312344, 2.8874969537335944, 2.885834158191449, 2.8814604605761995, 2.8848677349639718, 2.8811023854629703, 2.883187111111993, 2.8794840504859245, 2.8768042026234224, 2.8786719423490537, 2.879215333069736, 2.877556248345744, 2.8754724214569696, 2.8761762684915437, 2.874946591645117, 2.8776029495573625, 2.8750221082512875, 2.873810550606716, 2.8765950363107926, 2.8738672383992396, 2.8739977149877958, 2.873738360115144, 2.8705343393202556, 2.8675375228803737, 2.871123910026526, 2.868542979332551, 2.8718468627331735, 2.870673037765122, 2.869391226265115, 2.8661120557205386, 2.866149088090151, 2.8676273633635967, 2.863941819104947, 2.8691267000126364, 2.8681812246716794, 2.866401363929265, 2.864475408732243, 2.8659607212435185, 2.8603544038454083, 2.8651588262080843, 2.8634686171055144, 2.8618482115630224, 2.8614693848810666, 2.8624373829219865, 2.8620680884070224, 2.862962624238076, 2.864005957363663, 2.8589458461, 2.8602312743015497, 2.8552525749743483, 2.857692020487038, 2.8577963505047523, 2.859240319281912, 2.855484968412403, 2.8571196250708075, 2.8583710513966096, 2.859161859739307, 2.860251671827076, 2.855241804647659, 2.855287587726566, 2.8551461772894293, 2.8536233636559545, 2.853094031851946, 2.8534524311107163, 2.858424124470599, 2.8548769369692812, 2.8528048697756563, 2.8501216392065576, 2.8526980402526076, 2.8509806967361264, 2.847571891008549, 2.8477084940240998, 2.850753804772463, 2.850423901338876, 2.8497583359689647, 2.8474607638495133, 2.8522826475130008, 2.844658264348084, 2.8505811210786085, 2.8473669112834576, 2.848122691772568, 2.848346295908942, 2.845140655301583, 2.845723530610097, 2.8489301959749835, 2.8483541274177515, 2.849353940686734, 2.8470243860038518, 2.8499647306465445, 2.838958400377309, 2.8434009695388687, 2.8452190841838325, 2.844962736161489, 2.846314042787558, 2.8452204851027263, 2.8452786211012575, 2.8447282433281016, 2.8462489650406595, 2.844386132039554, 2.844684176771441, 2.8436799755633375, 2.8436898550618097, 2.8390014316512464, 2.83811073507625, 2.8369250051958472, 2.8437916507952807, 2.8390714199361478, 2.839597974918778, 2.8379482298574614, 2.8423199549898914, 2.8381223597743155, 2.845081364124613, 2.8419559425981245, 2.840575902796066, 2.8373880767700235, 2.838925623085281, 2.8399935783061627, 2.8381194462779233, 2.839453295721736, 2.835777399712317, 2.838137690256745, 2.838870740897825, 2.840399349948495, 2.835187949969535, 2.8412982355839, 2.833554510572974, 2.8347565134747703, 2.83320050993106, 2.8331061002922913, 2.8349197560674626, 2.8387199450362894, 2.839446528172996, 2.83632719539635, 2.8350036722379697, 2.8338854122222883, 2.835237731707836, 2.8340948303769355, 2.834958834901347, 2.8337543623918764, 2.8356326996174213, 2.833750450832296, 2.837407186408113, 2.839190735545436, 2.8328666015839774, 2.8353258524654925, 2.8342694984516577, 2.8341650798087383, 2.835817645775227, 2.8370296930702392, 2.8335898990289414, 2.8340982474048246, 2.8322668787919016, 2.835737289256647, 2.833162061388647, 2.834054819681823, 2.8319684237298945, 2.834995112202523, 2.832158440439196, 2.831754754616218, 2.832116022524892, 2.832995108435418, 2.830607940657964, 2.8295130979839382, 2.8295878177259643, 2.8308036026860077, 2.832268427216084], 'val_metric': [5.57, 9.11, 12.53, 12.66, 16.77, 16.44, 18.62, 20.05, 20.36, 20.66, 22.39, 23.28, 23.5, 25.08, 24.08, 24.01, 24.91, 26.51, 26.76, 26.7, 27.21, 27.34, 26.74, 28.19, 28.31, 28.18, 28.76, 27.84, 27.93, 29.15, 28.84, 28.89, 29.65, 29.08, 29.69, 29.71, 29.04, 29.87, 29.7, 29.96, 31.1, 30.79, 30.27, 29.88, 30.89, 30.65, 30.73, 30.97, 30.88, 30.32, 29.54, 30.88, 31.14, 30.93, 30.35, 30.51, 31.44, 30.45, 31.28, 31.75, 31.86, 30.38, 31.3, 31.84, 30.48, 30.43, 31.3, 31.89, 31.83, 32.24, 31.46, 31.89, 32.56, 31.67, 31.83, 31.94, 31.95, 32.51, 32.11, 32.2, 31.9, 31.02, 32.31, 31.29, 31.33, 31.67, 31.49, 31.65, 31.68, 32.44, 31.97, 32.4, 32.56, 31.37, 31.4, 32.33, 32.21, 32.55, 32.75, 31.36, 32.51, 31.7, 32.04, 31.66, 32.27, 32.74, 32.39, 31.92, 32.66, 32.48, 31.92, 32.4, 32.62, 32.58, 31.78, 32.58, 32.23, 32.93, 32.4, 32.1, 30.99, 32.67, 32.03, 31.9, 31.88, 30.95, 30.82, 32.84, 32.95, 32.45, 32.81, 32.95, 32.55, 32.98, 32.68, 33.0, 32.27, 32.86, 32.84, 31.97, 33.03, 32.45, 32.92, 32.4, 32.96, 32.45, 32.68, 33.69, 32.9, 31.66, 33.54, 32.71, 32.81, 32.49, 33.46, 33.09, 32.96, 32.99, 32.0, 32.8, 32.31, 33.57, 32.94, 33.02, 33.39, 32.4, 32.71, 33.37, 32.2, 32.52, 33.02, 32.99, 32.67, 32.45, 33.02, 33.14, 33.27, 32.28, 34.11, 32.69, 32.6, 32.74, 32.91, 33.74, 32.5, 32.42, 33.09, 33.69, 32.94, 32.78, 32.62, 32.93, 32.57, 32.6, 32.89, 32.33, 32.92, 33.43, 33.01, 33.17, 32.72, 33.46, 32.52, 32.25, 33.31, 32.64, 33.7, 32.44, 33.1, 33.13, 33.74, 31.93, 33.71, 33.0, 32.22, 33.05, 33.08, 34.19, 33.55, 33.87, 33.25, 33.52, 32.96, 33.22, 32.74, 33.4, 32.85, 33.43, 32.3, 33.44, 32.68, 32.95, 32.5, 32.94, 33.95, 33.25, 32.79, 32.68, 33.29, 31.8, 33.8, 33.14, 32.86, 33.85, 33.74, 32.64, 33.18, 32.7, 34.53, 33.24, 32.87, 33.65, 34.47, 33.9, 33.33, 33.77, 33.52, 32.83, 32.93, 33.1, 33.26, 33.88, 34.19, 33.77, 33.02, 33.37, 32.98, 33.28, 33.17, 32.17, 33.91, 33.72, 35.04, 33.02, 32.39, 33.78, 33.14, 33.35, 32.38, 33.8, 33.5, 33.78, 33.25, 33.57, 33.49, 33.98, 33.58, 33.5, 32.78, 33.62, 33.25, 33.54, 34.0, 33.53, 33.57, 32.8, 33.42, 32.8, 34.38, 34.18], 'val_loss': [4.519263199180554, 4.222430089476761, 3.9823908107295916, 3.936482136416587, 3.710630402443515, 3.7383477619499157, 3.5688325812102883, 3.5081806311941452, 3.473496309511221, 3.4583021151791713, 3.381391963381676, 3.326620411721005, 3.3188810705379317, 3.2357976930156633, 3.253412680261454, 3.2694935821423865, 3.2396726289372535, 3.1801799467414806, 3.1830020110318613, 3.1516598538987957, 3.136626428859249, 3.1112298487098355, 3.129901676420953, 3.1011148531725454, 3.0810887547814922, 3.111957465767101, 3.0569304766928314, 3.0844423694975056, 3.080893786090195, 3.0324536083610196, 3.042101241221094, 3.048748132529532, 3.0067653785085984, 3.049812289559917, 3.0035439266520703, 3.0009088121402034, 3.031687678804823, 2.994481081415893, 2.999090790748596, 2.9969817187376084, 2.9387360697339293, 2.9764285277409157, 2.9719115055290755, 3.013609467797978, 2.9450515956635686, 2.9480179677343674, 2.966270043591785, 2.9545042886855497, 2.9475342111223064, 2.969952378303382, 3.012365765632338, 2.958876338733989, 2.9140096781360114, 2.938793215022725, 2.9737200881265533, 2.981015591484726, 2.934072199900439, 2.93783175945282, 2.928534412080315, 2.9091630460350375, 2.9279567026028968, 2.958155261483162, 2.931254395254099, 2.9054051493383515, 2.958262908990216, 2.9743469292950477, 2.9288121127778557, 2.9083704815548694, 2.873106954204049, 2.9035920071753725, 2.940841349826497, 2.8951429166611593, 2.8768077299093746, 2.8900810305479987, 2.8990256178910565, 2.894511037571415, 2.871632057390395, 2.906537151640388, 2.8926927557416784, 2.8862820962432085, 2.889256557081915, 2.920792728472667, 2.857328954775622, 2.894345605449312, 2.891474990328406, 2.9174461698835823, 2.8999659536750455, 2.8897089510206966, 2.8952824143087788, 2.879939219754213, 2.8854326283096507, 2.8685235240656857, 2.8503331659705777, 2.8995945484015593, 2.912270716041516, 2.8604241085659927, 2.8816337038756936, 2.873879286893614, 2.845299739746531, 2.9040644191632605, 2.8551929498174387, 2.882238258222106, 2.8715094882211867, 2.8921031124272925, 2.887720414787341, 2.8442805815654197, 2.883713965962647, 2.8940914292244395, 2.865942602704285, 2.8601942214236895, 2.849245051669467, 2.8678075401646317, 2.8776155611512007, 2.876385185369261, 2.892800161792974, 2.8564269481950504, 2.8681658863262007, 2.824221027125219, 2.843416399636846, 2.8851763852842294, 2.9016279308659256, 2.84720103755878, 2.868015693251494, 2.8916838347532186, 2.883694708726968, 2.9064851900574507, 2.9359087093620544, 2.8389869487968977, 2.825980309468166, 2.8161928137396552, 2.8377992142537596, 2.8439314623547207, 2.8598067259332938, 2.847314026705019, 2.8444882464257017, 2.8452707741670547, 2.840244915075363, 2.8588765396434033, 2.8315492291359385, 2.8424213403349468, 2.822003417713627, 2.8482470246636944, 2.8350202528534423, 2.836710540732001, 2.839013549932249, 2.85934841784702, 2.823261973204886, 2.8022169846637994, 2.8333164369984036, 2.88319270512101, 2.8308021733715276, 2.8494024967691702, 2.832491548198044, 2.8337931860784056, 2.82121165466916, 2.846439970526726, 2.8070306147739386, 2.830428751410952, 2.867985443704447, 2.8535950935570296, 2.884244569547617, 2.832849072802598, 2.8239432110148632, 2.8368058409660484, 2.836235978041485, 2.8660198336194274, 2.822768866636191, 2.8045147182835137, 2.869440375619633, 2.8330553496719166, 2.8346093391916556, 2.8030212335525806, 2.8417856738825513, 2.870942881152888, 2.7951995817718993, 2.8148042700093265, 2.8471262341092345, 2.848687439967113, 2.806581107674131, 2.856247494175176, 2.849437390922741, 2.848810830693336, 2.847892449919585, 2.8037853491534093, 2.841377155795978, 2.834976148453488, 2.822003839881557, 2.7750155743519973, 2.8223324049809935, 2.8294310216691083, 2.8403338801329303, 2.8461755157276323, 2.83877261277217, 2.8292761790524623, 2.8193052893231627, 2.861349505224046, 2.8116267957505148, 2.823565518780119, 2.808425551007508, 2.817877141533384, 2.824916823654418, 2.8271138599723766, 2.8399461727992743, 2.8515933387598413, 2.793585537345546, 2.822591691260125, 2.7936446188361783, 2.820264312871702, 2.807560987533278, 2.824784896176332, 2.808298945426941, 2.8498998766492125, 2.799026298674808, 2.799200617583694, 2.866958783690337, 2.81504513731428, 2.8160124918457807, 2.778718324223901, 2.7829658856057815, 2.786929647633984, 2.821869157681799, 2.7975977795898537, 2.816243679660141, 2.8156293812830735, 2.8108728448296807, 2.8029301098197887, 2.8397662707954456, 2.800319151513895, 2.858350527514318, 2.799306584771272, 2.8321486248332226, 2.8189960581481834, 2.8413077258760002, 2.8086810871294348, 2.783673394257855, 2.8071451544002364, 2.842874327282997, 2.837931113637936, 2.795957187938083, 2.864872703886336, 2.8055494651672945, 2.8175841547121667, 2.8179227151688497, 2.79336168280073, 2.792522734897152, 2.8255991290329368, 2.8008194053248996, 2.834472815701916, 2.810707295016878, 2.8131436360110142, 2.8224672618185638, 2.800155899327272, 2.7746809812108424, 2.7988424984512816, 2.808479871719506, 2.7904171632353667, 2.8062708878972726, 2.826449362335691, 2.813174291021505, 2.8014255967109825, 2.8110622706686614, 2.802452929460319, 2.7789522196836534, 2.7970711903966916, 2.8474135117925656, 2.8177664526708566, 2.8220575789737095, 2.8150413028753487, 2.7913690873771717, 2.8436214961823385, 2.77462337806726, 2.782002930428572, 2.7490662343942436, 2.847819846147185, 2.842934073915907, 2.804191350177595, 2.825073630946457, 2.833414566744665, 2.856993561337708, 2.7866102373524075, 2.807309548566296, 2.792794014997543, 2.802391449357294, 2.792835019196674, 2.7743667295783947, 2.7825408566529584, 2.8009212017059326, 2.7859746756826995, 2.8280558419075743, 2.792235561237214, 2.798438834536607, 2.8097229968210695, 2.771047396265018, 2.7955785594928035, 2.7844965108640634, 2.8144221586786258, 2.80523536463452, 2.8244570664539457, 2.761657709528686, 2.7702946936248973]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal 2", optimizer="AdamW")
data.update({'train_metric': [5.712, 17.326, 26.84, 34.206, 40.365, 45.544, 50.056, 54.665, 59.348, 63.533, 68.164, 71.881, 76.169, 79.448, 82.503, 84.608, 86.335, 87.963, 89.235, 89.882, 90.812, 91.487, 91.973, 92.358, 92.909, 93.052, 93.533, 93.903, 93.987, 94.336, 94.552, 94.687, 95.037, 95.01, 95.184, 95.412, 95.591, 95.625, 95.891, 95.855, 95.896, 96.055, 96.181, 96.322, 96.435, 96.325, 96.523, 96.591, 96.643, 96.618, 96.808, 96.972, 96.894, 97.074, 96.839, 97.174, 97.068, 97.084, 97.271, 97.202, 97.38, 97.315, 97.332, 97.33, 97.522, 97.502, 97.687, 97.431, 97.641, 97.639, 97.661, 97.714, 97.691, 97.587, 97.776, 97.782, 97.75, 97.826, 97.82, 97.818, 97.945, 97.95, 97.905, 97.972, 97.904, 98.035, 97.943, 97.901, 98.141, 98.07, 98.051, 98.078, 98.13, 98.109, 98.155, 98.17, 98.13, 98.195, 98.159, 98.222, 98.266, 98.169, 98.337, 98.265, 98.314, 98.312, 98.256, 98.338, 98.318, 98.4, 98.276, 98.323, 98.329, 98.437, 98.473, 98.307, 98.44, 98.449, 98.396, 98.525, 98.413, 98.595, 98.435, 98.509, 98.473, 98.557, 98.513, 98.467, 98.41, 98.56, 98.482, 98.513, 98.586, 98.569, 98.526, 98.628, 98.644, 98.5, 98.637, 98.564, 98.657, 98.581, 98.647, 98.675, 98.561, 98.638, 98.69, 98.626, 98.67, 98.689, 98.678, 98.783, 98.673, 98.692, 98.77, 98.645, 98.666, 98.818, 98.726, 98.766, 98.663, 98.684, 98.818, 98.697, 98.834, 98.687, 98.749, 98.804, 98.805, 98.753, 98.761, 98.811, 98.759, 98.764, 98.757, 98.806, 98.674, 98.756, 98.901, 98.779, 98.794, 98.826, 98.828, 98.831, 98.859, 98.85, 98.858, 98.846, 98.854, 98.817, 98.869, 98.839, 98.835, 98.921, 98.903, 98.822, 98.877, 98.839, 98.887, 98.806, 98.844, 98.924, 98.943, 98.855, 98.882, 98.88, 98.914, 98.909, 98.946, 98.941, 98.899, 98.883, 99.044, 98.905, 98.984, 98.863, 98.944, 98.968, 98.856, 98.958, 98.979, 98.894, 98.927, 98.854, 98.978, 98.883, 99.031, 98.95, 98.988, 98.992, 98.895, 99.077, 99.028, 98.944, 98.941, 98.992, 98.932, 98.995, 98.999, 99.033, 98.929, 99.091, 99.005, 99.032, 98.913, 99.084, 98.973, 99.047, 99.021, 98.98, 98.968, 99.025, 98.999, 99.052, 98.984, 99.108, 99.007, 99.002, 99.063, 99.059, 99.027, 99.006, 99.026, 99.074, 99.011, 99.04, 99.042, 98.991, 99.112, 99.034, 99.043, 99.093, 99.099, 99.024, 99.122, 99.113, 99.063, 99.105, 99.062, 99.045, 99.034, 99.116, 98.997, 99.08, 99.117, 99.063, 99.112, 99.049, 99.078, 99.07, 99.091, 99.056, 99.121, 99.116, 99.081, 99.148, 99.02, 99.103, 99.091, 99.109], 'train_loss': [4.6020800837018285, 3.670548213115504, 3.1425502852453913, 2.76538197962199, 2.450196260454101, 2.2050817321678498, 1.9858640733622468, 1.7711036988579913, 1.5591881754912402, 1.3586009021607715, 1.1630797640909694, 0.9921675588904629, 0.8282659661098695, 0.6979011747406907, 0.587340152652616, 0.5045731718617986, 0.4422290139925152, 0.38735921315310173, 0.34871819990991554, 0.3255113926230526, 0.2972635889255459, 0.27314074260617055, 0.2570916100721556, 0.2433303731087874, 0.2269339116119797, 0.21885224884246032, 0.20808116193581908, 0.19355599253402067, 0.1933853072319859, 0.17974653135405963, 0.17450827627096585, 0.16985441581539748, 0.15775949359941163, 0.15911190393806857, 0.15131427920098706, 0.14544821921223566, 0.1403353261419942, 0.138753078137873, 0.1312665501722657, 0.12977486267335528, 0.12925837349780595, 0.12374365052133227, 0.12195492183410647, 0.118647869352206, 0.11272577135492967, 0.11505537322809967, 0.11034092187591883, 0.10657756308824186, 0.10736830293493475, 0.10593334918273273, 0.09989042656069534, 0.09667721857755937, 0.09952971131838168, 0.09449937820323741, 0.10068242318549278, 0.0886774206865026, 0.09148932805106696, 0.09237751719212339, 0.08699689016147723, 0.08846909198189012, 0.08404900341354887, 0.08404473383432481, 0.08442853751573867, 0.08422428331432015, 0.07922579022973504, 0.08082222182873305, 0.0753233927886448, 0.0822976752239746, 0.07658291332601252, 0.07488884076491589, 0.07504077006866697, 0.07240175157515791, 0.07339403437216953, 0.07633612370544536, 0.07102921621198274, 0.0697074120480541, 0.07080897567966525, 0.0689203457449374, 0.06892303390141222, 0.07123367527263837, 0.06341998147708498, 0.06470253447521185, 0.06666082924198222, 0.06291505792079898, 0.06620582178262166, 0.06185246455390483, 0.06573391440029537, 0.06408636065958793, 0.060515251077094274, 0.06064126168618125, 0.061538895111982676, 0.061507698874390296, 0.06076034101295885, 0.060137814567921895, 0.05874256261543807, 0.05889123030884581, 0.06022063898481548, 0.05791057173798179, 0.05717828577716244, 0.055845459791389544, 0.05624080306184654, 0.05695763903573782, 0.05359015776947086, 0.056181187950007934, 0.05428091820653573, 0.05323492929690815, 0.05488959525208932, 0.05314376856662483, 0.052929011055082735, 0.04969663637936335, 0.05419109046066082, 0.053659498795684636, 0.05335454657039696, 0.050689330246622456, 0.04898906539978432, 0.05269563613319144, 0.049485018155752056, 0.05005883769979355, 0.05148336241890659, 0.047527012296133604, 0.050597803527310546, 0.04504950677757631, 0.049587885783801286, 0.04774442949838849, 0.04811035167835224, 0.046444866791372295, 0.04710956923245028, 0.048766314615315255, 0.04949458365450611, 0.045590142215539836, 0.04808769085306271, 0.04706423217091818, 0.0451588166643903, 0.045614998182549986, 0.04663068023602635, 0.04367696899057158, 0.0428788115322238, 0.0469081507374843, 0.04355084777352211, 0.044375836537058705, 0.04234130186054773, 0.045237145518878995, 0.042792486251632476, 0.04297229944862308, 0.0447590928194405, 0.04370787936125441, 0.04242369581689246, 0.04411502636797632, 0.042737246997827535, 0.04142874987510385, 0.04032194660630188, 0.039553337582918534, 0.04153195106085052, 0.04055333252720921, 0.039310115741360105, 0.04324561837571718, 0.042194068517769204, 0.037298809618131104, 0.04003119021473138, 0.039778004637415126, 0.040656836257235436, 0.04177373536309479, 0.037768890319114415, 0.04177540070222368, 0.036914510432552124, 0.041287508074424197, 0.04020527270679158, 0.038195828152931446, 0.03747777853115514, 0.039266941395654655, 0.040679307844578, 0.03717247081518451, 0.0398965141185754, 0.039667973123135976, 0.03847046061501253, 0.03747262443889228, 0.04098620352154879, 0.03867591595862544, 0.03486318071640942, 0.03946928479063345, 0.0393089388417627, 0.03719761887913012, 0.03742007566739207, 0.036712439714004994, 0.03550535772584265, 0.036541269615016816, 0.03697615623464737, 0.03616590876317635, 0.03618446169303023, 0.037045779968152344, 0.03487701803065288, 0.0362456629009149, 0.03777659388650791, 0.034350964192204805, 0.03455224420451657, 0.037354361268709446, 0.03672086810152406, 0.03665343390515754, 0.035435793510397155, 0.03699513610510659, 0.03578045209214487, 0.0348199971267837, 0.033402314152985974, 0.035790663828634016, 0.03633857981310186, 0.034913517619577096, 0.03489165518805861, 0.03432563727015706, 0.03307399920063729, 0.0340961936665538, 0.0352596113765998, 0.035163822391467266, 0.030918269586509246, 0.0354636711471883, 0.032436767356182804, 0.036116269793045545, 0.03286264528624606, 0.03203490858338245, 0.037147085816526884, 0.0332297580347759, 0.032031000804069516, 0.03394891648169432, 0.034259622126744486, 0.03562121565433933, 0.032469149382283846, 0.034881929917971115, 0.03169460926426578, 0.033381848041238844, 0.03316927459506371, 0.03152754497369117, 0.03470433389891868, 0.030098194892594046, 0.03166230028450578, 0.03340958674395918, 0.032875020058098815, 0.03368115030081692, 0.03376250013791594, 0.03290291671174802, 0.031754174090885975, 0.02928302632865239, 0.034351589431433106, 0.029131003474865167, 0.031827374447835906, 0.03148978105747699, 0.0335464494517228, 0.029626425753090015, 0.03129025911771924, 0.029623713302029796, 0.032080312077512524, 0.03172651151564897, 0.03272236188759337, 0.02990369000184628, 0.03225874653243521, 0.029659062177653756, 0.03243902690882329, 0.029198443943833346, 0.0323145782045221, 0.030596099441685735, 0.029990847609124294, 0.029946881371411875, 0.030058126420288968, 0.03145263405716802, 0.030205628806007884, 0.029828139695749153, 0.030941754393692927, 0.029387796552379133, 0.029499279539817642, 0.0312619200496023, 0.02900003085658982, 0.03093429683026219, 0.03135791399226952, 0.029498396395161625, 0.029345632400694013, 0.031078155427133325, 0.028349006922311678, 0.028931575252839722, 0.029733222441133904, 0.029218127076272447, 0.02921698822916993, 0.02943343133277384, 0.030834215522854144, 0.02805900945023098, 0.031095106865647912, 0.029513347592324612, 0.028507725219926578, 0.029887246446046734, 0.028161046654384455, 0.03023755879579009, 0.029274207942571146, 0.028880853320146678, 0.02908745807578532, 0.029537875894934902, 0.028221040640781726, 0.027439659540478906, 0.028700466147895145, 0.027536745173459255, 0.030806373070489904, 0.028861384164254905, 0.029003362250107283, 0.027786716669505742], 'val_metric': [11.21, 23.56, 31.34, 37.3, 41.32, 45.26, 46.83, 48.16, 49.24, 49.97, 50.01, 50.59, 50.22, 50.77, 50.5, 50.05, 50.33, 49.74, 50.72, 50.2, 50.03, 51.0, 50.63, 50.9, 50.21, 50.84, 50.88, 51.36, 50.8, 51.0, 50.56, 50.88, 51.59, 51.3, 50.82, 51.37, 50.92, 51.58, 50.68, 51.38, 51.83, 51.16, 51.37, 51.19, 51.54, 51.1, 50.96, 51.17, 51.19, 51.69, 51.79, 51.67, 52.13, 51.67, 51.7, 51.67, 51.53, 52.39, 52.86, 51.85, 51.97, 51.78, 52.09, 52.64, 52.61, 53.19, 51.91, 52.45, 52.43, 52.82, 52.43, 52.4, 52.55, 52.12, 52.09, 52.73, 53.04, 52.92, 52.36, 51.81, 52.2, 52.24, 52.12, 52.6, 52.17, 52.35, 52.16, 52.95, 52.7, 52.58, 52.79, 52.62, 53.34, 52.89, 52.94, 52.0, 52.43, 52.74, 52.18, 53.18, 53.29, 52.96, 52.51, 52.32, 52.87, 52.85, 52.73, 52.52, 53.03, 52.37, 52.74, 52.3, 52.43, 53.07, 52.9, 52.83, 52.71, 53.45, 52.71, 53.03, 52.92, 52.74, 53.18, 53.26, 53.22, 52.99, 53.12, 53.05, 52.78, 53.68, 52.98, 53.31, 53.34, 53.27, 53.26, 52.96, 52.43, 53.54, 53.12, 53.3, 52.83, 53.05, 53.15, 53.34, 53.21, 53.07, 52.99, 53.34, 53.39, 52.44, 53.18, 53.3, 52.75, 52.65, 53.64, 53.15, 53.19, 53.3, 53.26, 52.86, 53.27, 53.28, 53.51, 53.56, 52.85, 53.31, 53.4, 52.66, 53.23, 53.23, 53.55, 52.94, 52.93, 53.53, 53.27, 53.73, 52.83, 53.73, 53.46, 53.22, 53.46, 53.89, 53.73, 52.86, 52.93, 53.38, 53.21, 53.65, 54.38, 53.22, 53.62, 53.83, 53.71, 53.51, 53.35, 53.42, 53.4, 53.83, 53.2, 53.97, 53.54, 53.95, 53.75, 53.36, 53.41, 53.85, 53.69, 53.56, 53.94, 53.64, 53.19, 53.68, 52.66, 53.51, 53.12, 53.3, 53.38, 53.83, 53.15, 53.59, 54.05, 53.86, 53.49, 53.79, 53.79, 53.68, 53.67, 53.04, 54.35, 53.84, 53.75, 53.63, 52.92, 53.42, 53.44, 53.77, 53.45, 52.86, 52.87, 53.69, 53.31, 53.64, 53.63, 53.47, 52.99, 53.36, 53.44, 53.52, 53.56, 53.49, 53.77, 53.51, 54.25, 53.9, 53.82, 53.36, 53.88, 54.13, 54.24, 53.76, 53.44, 54.08, 53.17, 53.38, 54.15, 54.48, 53.7, 53.93, 53.49, 54.19, 53.42, 53.95, 53.64, 54.06, 53.79, 54.27, 53.72, 53.54, 53.78, 53.49, 53.77, 54.39, 53.82, 53.59, 53.62, 53.65, 53.91, 53.69, 53.49, 53.77, 53.42, 54.21, 53.84, 53.77, 54.24, 54.19, 53.89, 53.69, 53.9, 54.11], 'val_loss': [4.1060560949289115, 3.3317317522255476, 2.9291713738897043, 2.609757741545416, 2.4437233391840745, 2.263029931077532, 2.202843493716732, 2.1317526705705436, 2.1153628063050047, 2.1144025139747913, 2.2029597903512848, 2.2128623917603947, 2.354516090860792, 2.3991478032367244, 2.5318728970114592, 2.6224340769895322, 2.6865714237948133, 2.7719311824270116, 2.7959872818296883, 2.866119156977174, 2.9322956318308595, 2.9005339965698824, 2.9512671980128924, 2.976677194902092, 3.028565138008944, 3.064569900369948, 3.022096145684552, 3.0769917460004237, 3.1470586129814198, 3.152792938195976, 3.182322920507686, 3.11561081022214, 3.216210737349881, 3.180647977598154, 3.2126191417882395, 3.246107245706449, 3.2764163374141524, 3.2055174507153263, 3.2556875256975744, 3.199994583798062, 3.279262951225232, 3.2692634535443252, 3.2813072705724435, 3.3079464101487663, 3.316279537358861, 3.343297638710897, 3.428990941898079, 3.388382283745298, 3.394850052086411, 3.390289686288044, 3.3362253593031768, 3.373179661999842, 3.376605940092901, 3.4126998896052125, 3.4242446775649005, 3.4215138596334276, 3.446952100772007, 3.3738607794615874, 3.369823048069219, 3.4012811563576864, 3.4843018719345142, 3.47281898928296, 3.428649382226786, 3.467071183168205, 3.4473263025283813, 3.443969829826598, 3.515286406134344, 3.4708107763035283, 3.554086412593817, 3.500774691438979, 3.4792912181015985, 3.5129243473338474, 3.4631606659312157, 3.496765657595009, 3.541341457017668, 3.491481926410821, 3.455907852027067, 3.498470074811559, 3.5665077468392195, 3.546015276271067, 3.570814295179525, 3.5805956770660012, 3.5860909553849774, 3.5678054982689535, 3.5412796595294003, 3.5840594119327083, 3.5805215296471955, 3.5268807373229105, 3.5187536056633966, 3.553387843879165, 3.5577786090267693, 3.5779108173528296, 3.509672243883655, 3.6379797697826555, 3.578974758363833, 3.6258058585938375, 3.6147738395223192, 3.5585159602438567, 3.6136943872567193, 3.578067449247761, 3.6229271547050232, 3.665628081674029, 3.6398530355684318, 3.649462756457602, 3.6191988268475623, 3.5946455950949603, 3.6260323805414187, 3.6967156495258306, 3.6310816608416805, 3.710230288991503, 3.629492720980553, 3.6970740537734548, 3.6453127576287385, 3.63572955093566, 3.6660012614195514, 3.61886339050949, 3.668486344586512, 3.6069125263554276, 3.655270490676734, 3.6633721897556524, 3.6585676419507167, 3.656291681869774, 3.6435457350342135, 3.7086435142596055, 3.638170288246908, 3.659773453026061, 3.6858919137602397, 3.6891998348722033, 3.612684720640729, 3.6101958933909226, 3.6709875521386506, 3.62849521181386, 3.6883351780047082, 3.699499904729758, 3.6522976645998133, 3.6959650402615782, 3.7307615963516723, 3.6963894427961606, 3.74053071942299, 3.7122098169509012, 3.728462344901577, 3.7079663561407927, 3.7766637505999037, 3.7006216743949114, 3.721864544661941, 3.694979800540171, 3.6983498319698747, 3.6683039293167696, 3.7248480164321367, 3.777194807483892, 3.72623705218552, 3.708446268822737, 3.7674168325533532, 3.7551474191580607, 3.7211765776014634, 3.791793377915765, 3.7457062271749897, 3.746816561100589, 3.786600625818702, 3.790907307794899, 3.7157419806073424, 3.725334601797116, 3.7141035514272702, 3.706102326417425, 3.7472290981347394, 3.711999638824706, 3.7384143764046347, 3.7965435970361066, 3.733247500696, 3.7386478694381227, 3.7191525375007823, 3.817753123249977, 3.7818926303250016, 3.7173341022934885, 3.7507683179181095, 3.766623205819707, 3.7215459187319326, 3.7354651697122367, 3.8238743399358857, 3.7988716766333126, 3.788070431940115, 3.789274232023081, 3.728536305154205, 3.806352846182076, 3.7760995998504057, 3.7157550556644514, 3.783850870314677, 3.7794260447192345, 3.725137015436865, 3.7388410796025755, 3.7637689747628134, 3.717164144394504, 3.769273854365015, 3.809981397003125, 3.7935927855740688, 3.763631141109831, 3.774537870838384, 3.7869619586665157, 3.808332743538413, 3.7616139513671776, 3.791901790412368, 3.772781209581217, 3.829082043307602, 3.789287318089965, 3.78808791736129, 3.7596743691499066, 3.7593339036224753, 3.80790826317611, 3.812699659614806, 3.8241271460132236, 3.8301617002031607, 3.761179035256623, 3.826700112622255, 3.809681464152731, 3.811669413071529, 3.79324512990417, 3.790590014806978, 3.8700061220272333, 3.8588342594492966, 3.7834365052782046, 3.8180466613192467, 3.8192213394079997, 3.7881587690608516, 3.8279603066717742, 3.845447584322304, 3.810915668678891, 3.8340697888356106, 3.855391587421393, 3.8097644140765925, 3.8442635137564056, 3.8085181568838227, 3.8593933126729008, 3.9119060639363186, 3.9094361256641945, 3.84748443220831, 3.8431597203965397, 3.9182252542228455, 3.8810928949884547, 3.8699878719961567, 3.8837741043916933, 3.927711927207412, 3.8835821945196503, 3.9386279400746536, 3.8892825143352434, 3.8920651742607166, 3.873570845764913, 3.8901677841593507, 3.8733449566895795, 3.843026188908109, 3.924861149423441, 3.8201995943761933, 3.8181729886182554, 3.8781208931260807, 3.8601949074465756, 3.845145933187691, 3.9027363722491417, 3.8684198078076553, 3.8543564096377914, 3.8484248799882876, 3.860225621682064, 3.839714018022938, 3.8188236427914566, 3.8739287215433302, 3.8784249694484054, 3.89186686505178, 3.8239207761302874, 3.8225283410139146, 3.848359446237042, 3.835837981123833, 3.7713804051374935, 3.836079589880196, 3.8463277194150693, 3.8052067506085536, 3.838930696438832, 3.8308821905190777, 3.859888580194704, 3.873466733914272, 3.9081561143990533, 3.9157107013046364, 3.9236308863967846, 3.9288007453748377, 3.8788825960675624, 3.7798791015224094, 3.928141483835354, 3.863821860711286, 3.8446878597235226, 3.892650182080117, 3.9087867497638533, 3.8933398435070257, 3.8500466677033978, 3.866009874708334, 3.8265165189269243, 3.895694497284616, 3.896731130256774, 3.827662884809409, 3.8691312909885576, 3.9223549085058225, 3.881250140773263, 3.869086322890725, 3.87446511019567]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="SGD")
data.update({'train_metric': [0.775, 1.126, 1.728, 2.491, 3.238, 3.907, 4.797, 5.525, 6.299, 7.47, 8.302, 9.413, 10.525, 11.39, 12.275, 13.407, 14.371, 15.037, 16.042, 17.121, 18.185, 19.006, 19.934, 20.843, 21.779, 22.447, 23.233, 23.987, 24.87, 25.551, 26.663, 27.376, 28.029, 28.873, 29.753, 30.406, 31.251, 31.879, 32.587, 33.536, 34.18, 34.817, 35.519, 36.151, 37.165, 37.647, 38.382, 39.125, 39.751, 40.473, 41.058, 41.864, 42.776, 43.418, 43.84, 44.777, 45.417, 45.859, 46.837, 47.304, 48.064, 48.42, 49.384, 50.074, 50.784, 51.172, 52.022, 52.712, 53.375, 54.089, 54.675, 55.413, 55.827, 56.526, 57.31, 57.804, 58.368, 58.946, 59.522, 60.119, 60.772, 61.733, 62.194, 62.808, 63.3, 63.699, 64.605, 64.844, 65.447, 66.022, 66.235, 67.011, 67.354, 68.13, 68.82, 68.937, 69.361, 70.348, 70.819, 71.108, 71.331, 72.026, 72.469, 73.183, 73.458, 73.694, 74.057, 74.637, 75.16, 75.517, 75.983, 76.04, 76.604, 76.937, 77.231, 77.501, 77.879, 78.508, 78.676, 79.135, 79.607, 79.771, 79.896, 80.052, 80.751, 80.825, 81.092, 81.106, 81.688, 81.887, 82.332, 82.516, 82.799, 82.961, 83.093, 83.35, 83.71, 83.739, 84.171, 84.121, 84.353, 84.746, 85.113, 85.177, 85.182, 85.191, 85.372, 85.663, 85.767, 86.154, 86.425, 86.291, 86.68, 86.631, 86.779, 87.098, 87.212, 87.432, 87.377, 87.498, 87.741, 87.994, 87.89, 88.093, 88.267, 88.356, 88.439, 88.654, 88.693, 88.665, 89.1, 89.06, 89.129, 89.193, 89.536, 89.551, 89.644, 89.661, 89.952, 89.859, 89.729, 90.035, 90.031, 90.101, 90.385, 90.327, 90.506, 90.581, 90.756, 90.627, 90.83, 90.817, 90.716, 90.936, 91.176, 91.186, 91.019, 91.331, 91.456, 91.479, 91.366, 91.555, 91.51, 91.747, 91.838, 91.733, 91.776, 91.964, 92.159, 91.989, 92.144, 92.204, 92.262, 92.412, 92.368, 92.469, 92.346, 92.378, 92.592, 92.761, 92.654, 92.628, 92.675, 92.751, 92.877, 92.847, 92.825, 92.914, 92.877, 93.088, 92.969, 93.226, 93.267, 93.367, 93.294, 93.288, 93.372, 93.336, 93.546, 93.521, 93.477, 93.311, 93.327, 93.517, 93.59, 93.616, 93.623, 93.671, 93.874, 93.903, 93.77, 93.832, 93.86, 93.91, 93.949, 93.95, 94.064, 94.113, 94.048, 94.126, 94.23, 94.282, 94.03, 94.223, 94.274, 94.223, 94.449, 94.36, 94.359, 94.447, 94.445, 94.323, 94.528, 94.337, 94.433, 94.56, 94.277, 94.548, 94.608, 94.621, 94.746, 94.512, 94.859, 94.715, 94.719, 94.808, 94.842, 94.876, 94.625, 94.75, 94.647, 94.912, 95.098, 94.996, 94.775, 94.926, 94.896, 94.736, 94.85, 94.853], 'train_loss': [5.318828564153904, 5.261252801081193, 5.191596451586664, 5.1011453349088445, 5.006413890548188, 4.91155959029878, 4.806818352551012, 4.708709529173809, 4.611009058705219, 4.515343707262211, 4.42156076339751, 4.33658259752387, 4.254930978849464, 4.18102618478005, 4.106509419488205, 4.034831857193149, 3.965494591077779, 3.904637718841348, 3.8373005734905554, 3.772909182428322, 3.7169509814171477, 3.6587436765108214, 3.60229143056058, 3.5469330678135633, 3.498772040896132, 3.4521048503736615, 3.3976892820933045, 3.3533777295208402, 3.3041894660115014, 3.2607393829165097, 3.212227799994627, 3.169509915915042, 3.128168226584973, 3.0849377899999735, 3.0437818680981072, 3.005094278034154, 2.962904572563147, 2.925738534176876, 2.878503228484707, 2.8437246597507255, 2.804732247186027, 2.7686171121347125, 2.735142849991128, 2.695251134901724, 2.6551819009347675, 2.627123280892522, 2.585331108199429, 2.546138057476881, 2.51554026027101, 2.4783191957003936, 2.446190531407879, 2.4096898340828052, 2.3713707653132294, 2.3392698362860553, 2.310325520433681, 2.2782235003707507, 2.2421088705288623, 2.215148478536673, 2.1783727478386115, 2.1477181766556384, 2.113567656381574, 2.092938544654114, 2.05551713769892, 2.019398833495718, 1.9914600404500198, 1.967927129544742, 1.9317312759233451, 1.9029062048258572, 1.873687320387066, 1.8486492845086204, 1.818659012163593, 1.7909093018304212, 1.76561260002207, 1.732595902868211, 1.7035900185829695, 1.6820140481376527, 1.65079389019647, 1.6290261691301509, 1.5975419931966985, 1.5746468597318755, 1.553618680568971, 1.5165919079966677, 1.494817804161433, 1.4726580104535005, 1.448442549829062, 1.4241819188751934, 1.3977327931026426, 1.38236326772436, 1.3578711670328238, 1.334892939735664, 1.3178072116806656, 1.2911024399087434, 1.276505022840628, 1.2496477993954775, 1.225436401954463, 1.208906792633364, 1.1934741307967607, 1.1608083426227802, 1.1407011966253806, 1.1292344141830182, 1.1174111996258365, 1.0951154321260508, 1.0770288768748175, 1.056118690578585, 1.038749217415039, 1.0263946094500735, 1.0128410029929948, 0.9877063704086128, 0.9711560945593235, 0.9626513309609943, 0.9478434254477898, 0.936839338075024, 0.9188291186410803, 0.9039601036660273, 0.889666233376212, 0.8807524580529922, 0.8696041751464666, 0.8436908532394024, 0.837507755441385, 0.8261829650295291, 0.8032678108679051, 0.7941763739858921, 0.7898448989205229, 0.7839325536578722, 0.7581448253346649, 0.7563362114527099, 0.7407100748649714, 0.7430619043336796, 0.7204256918441007, 0.712020956261068, 0.7001958938645615, 0.6908307311554712, 0.6804969641388949, 0.6754823048680697, 0.6695168454412154, 0.6574428589467581, 0.6482158785093616, 0.6445998140465962, 0.6285944311418978, 0.6289584266971642, 0.6202257960672495, 0.6066643326654697, 0.5980842749296819, 0.5890911590458107, 0.5943832412524171, 0.5835205245994255, 0.5778070499671245, 0.5724298954582031, 0.5653626112147012, 0.5533658125350205, 0.5450678342001147, 0.5432793857554785, 0.5359049686448206, 0.5312380420135826, 0.5260104155853171, 0.5170962441848473, 0.512349705625938, 0.5033793984528009, 0.5019572613449792, 0.4968811945509468, 0.4914367565895912, 0.4790333922921429, 0.4839766600281858, 0.47633488615468306, 0.47015374774019153, 0.46688687170230647, 0.4607073540917895, 0.45403863693648855, 0.45268679705286974, 0.44978331857893, 0.44069492808344723, 0.43965631062719046, 0.43318935933207675, 0.43232193371842326, 0.42289557460974364, 0.41839493329679056, 0.4170752032037279, 0.41297718713814374, 0.4093573262399004, 0.40648822667540757, 0.41039266703758803, 0.4021761258033248, 0.3975817351027017, 0.3952767734564197, 0.38592343648236604, 0.3866267290087899, 0.385056986546791, 0.3796565998214525, 0.37415061061647714, 0.3772878679789493, 0.369938436535712, 0.36775481285704914, 0.3715397738306399, 0.3646332724061595, 0.3565089957060451, 0.35555813350474613, 0.3605800314698552, 0.35215959350935405, 0.3458740477589026, 0.3454826192659822, 0.34559483870015417, 0.34280138819818684, 0.34199411430117876, 0.3378335639491191, 0.33421407043647855, 0.3337927207646275, 0.3352622073084135, 0.32857221290612176, 0.32031968646871684, 0.3232218329618927, 0.3192418064976913, 0.3194006119955448, 0.31312120161469137, 0.30992013482008696, 0.30963588099848055, 0.3046027143853487, 0.309712007155574, 0.30905686397050636, 0.3003864947966254, 0.2995191184149594, 0.29664542878276634, 0.296817521645103, 0.2944424116432247, 0.2954955782767526, 0.2931409288800762, 0.2917598347803453, 0.29177727419656585, 0.28918471801322926, 0.29144809069975935, 0.2867481443728305, 0.28406884514095687, 0.27851828336429685, 0.2749375697995178, 0.27333825328032785, 0.27535704636775904, 0.27323025179954197, 0.2722375344339656, 0.2687627641807095, 0.26651270365341306, 0.26414867910489165, 0.2653709543166981, 0.2707001739637408, 0.2703808274618228, 0.2676957059825603, 0.26213365333704175, 0.2615199348080715, 0.2628515220306199, 0.25756731369101843, 0.2548053720662095, 0.2535363230770853, 0.2568847197641262, 0.25374277440386556, 0.25290459617305017, 0.24855791520916035, 0.24752598022773947, 0.24989842116041436, 0.2448149070105565, 0.24335318841445316, 0.2464471241043343, 0.2425709265383748, 0.24018352723043468, 0.2380369133694349, 0.24589523294093285, 0.24030126535922994, 0.23720798656935502, 0.2373917460327185, 0.22958646629599144, 0.23462405902829905, 0.23324019224719594, 0.23126006896249468, 0.2284802349685898, 0.23481578905652595, 0.22916875055544816, 0.23307773344080493, 0.22711554761697142, 0.22646996207292158, 0.2322698997968828, 0.22687112731395512, 0.22322301219336055, 0.22228863484256556, 0.22012217412687843, 0.22788030007330942, 0.2168287847284049, 0.22267184742581592, 0.22165790478855618, 0.2195825513669183, 0.21698346565062238, 0.21537588331526622, 0.22134301844586263, 0.22092064394259864, 0.22138975430053545, 0.21157797768483236, 0.20819836580841036, 0.21136799533103645, 0.21779896515306568, 0.21150012634651674, 0.21352186153380062, 0.2178361201903146, 0.21252851834328831, 0.21257986018454425], 'val_metric': [1.08, 1.83, 3.04, 3.41, 4.54, 5.15, 6.41, 7.55, 8.42, 9.67, 10.7, 11.91, 12.44, 12.99, 13.72, 15.31, 16.39, 16.98, 17.62, 18.37, 19.34, 20.34, 20.7, 20.89, 22.17, 22.44, 22.89, 23.76, 24.76, 24.62, 25.0, 24.94, 26.13, 26.22, 26.24, 27.53, 27.45, 27.82, 27.96, 28.91, 28.72, 29.05, 28.81, 29.6, 28.92, 28.71, 30.29, 30.25, 29.21, 29.48, 30.43, 29.86, 29.88, 30.71, 30.56, 30.98, 30.8, 31.33, 30.38, 31.33, 31.39, 30.58, 30.89, 30.73, 31.17, 31.68, 30.61, 30.98, 31.15, 31.47, 31.15, 31.18, 30.73, 31.74, 31.22, 31.26, 30.77, 30.61, 31.39, 30.55, 30.34, 31.46, 30.86, 30.64, 30.29, 30.78, 30.3, 31.18, 30.87, 30.76, 30.39, 29.98, 29.72, 30.91, 30.9, 29.8, 30.18, 30.8, 31.08, 30.86, 30.86, 30.11, 30.11, 30.14, 30.78, 29.57, 30.49, 30.27, 30.59, 29.55, 30.81, 30.49, 29.52, 29.96, 30.47, 30.25, 30.37, 30.48, 29.98, 30.74, 30.15, 30.18, 30.66, 29.53, 30.15, 29.95, 29.42, 30.51, 30.16, 29.91, 30.02, 29.9, 30.27, 30.94, 29.88, 30.55, 29.92, 29.86, 29.6, 30.44, 29.81, 30.11, 30.68, 30.23, 30.9, 29.33, 30.26, 30.17, 29.4, 30.49, 30.75, 29.96, 29.65, 29.62, 29.9, 30.47, 30.17, 30.19, 30.24, 29.25, 30.35, 30.45, 30.32, 29.97, 30.33, 30.82, 30.31, 30.05, 30.49, 30.5, 30.1, 30.34, 30.7, 30.8, 30.23, 30.74, 29.54, 30.72, 30.63, 29.93, 29.71, 30.34, 30.19, 30.63, 29.92, 29.99, 30.03, 30.72, 30.33, 31.14, 30.67, 30.35, 30.51, 29.9, 29.92, 30.35, 29.79, 30.28, 30.33, 30.79, 30.9, 31.15, 30.43, 30.09, 29.81, 30.66, 30.8, 30.73, 29.62, 30.71, 30.46, 30.59, 30.85, 30.63, 30.44, 30.68, 30.82, 28.97, 30.62, 30.85, 29.84, 30.64, 30.58, 31.09, 30.83, 29.62, 30.0, 30.19, 30.82, 29.97, 30.45, 30.73, 30.45, 30.14, 30.02, 30.59, 30.84, 30.49, 29.63, 30.42, 30.47, 29.97, 30.4, 30.73, 30.25, 29.91, 30.77, 29.92, 29.96, 31.11, 29.62, 30.21, 31.57, 30.69, 30.6, 31.07, 30.64, 30.38, 30.35, 30.31, 30.55, 30.12, 30.13, 30.39, 30.32, 30.12, 29.85, 30.41, 30.67, 30.62, 30.05, 30.39, 30.67, 30.28, 30.78, 30.41, 30.69, 30.61, 30.93, 30.77, 30.15, 29.87, 30.28, 30.72, 30.57, 30.62, 31.05, 30.6, 31.33, 30.39, 30.63, 30.63, 30.12, 30.62, 30.53, 30.5, 31.2, 31.26, 30.45, 30.64], 'val_loss': [5.622411785611681, 5.223980384267819, 5.094697344834637, 5.0038105818876035, 4.894518891717218, 4.792921169548277, 4.680285677029069, 4.562470553027596, 4.450888849367761, 4.385196334996801, 4.2819356037552945, 4.1938249806689605, 4.120648836633961, 4.044370749953446, 3.9903056105231025, 3.9072443649267696, 3.8340095775142595, 3.8079497039697734, 3.7231047244588282, 3.6792601992370217, 3.6155794760224165, 3.5559104491191307, 3.523448171129652, 3.4867267342889385, 3.4443235313816434, 3.4293875299441585, 3.3849343668883014, 3.3513500181732665, 3.3185017298740944, 3.288926925628808, 3.2676986569811586, 3.280849706595111, 3.2174764795667805, 3.20139999602251, 3.2004155528013873, 3.157158106755299, 3.1304685925222504, 3.116256134525226, 3.1341947696770833, 3.0798794889146355, 3.0679992839788937, 3.0639498560291947, 3.113302302208676, 3.055004791848978, 3.060709427116783, 3.0785446569418453, 3.0070688420799887, 3.0255084804668546, 3.0610773639314495, 3.064322683461912, 3.0143094715798737, 3.0237357024174587, 3.054985568781567, 3.0105265211907164, 3.02595865422753, 3.0150106370828715, 3.045026416991167, 3.009341413048422, 3.0453329450765234, 3.0303422037962897, 3.0080091247133387, 3.079084324229295, 3.04322577282122, 3.052512820359248, 3.0700088663465657, 3.009696884519735, 3.0930698897428575, 3.0899953553631048, 3.0782508417299597, 3.076329513719887, 3.1037579972273224, 3.1155938995871573, 3.1310523087811317, 3.1050509024577537, 3.1116679291816274, 3.141862110727152, 3.1871360297415667, 3.185463912927421, 3.1318417051035885, 3.1940590530444104, 3.220328933114459, 3.194553621255668, 3.253871342938417, 3.2306190068554725, 3.2902975955586524, 3.294607495046725, 3.324247133200336, 3.2681242971663265, 3.268461035315398, 3.317852774243446, 3.3325387186305537, 3.4419001090298793, 3.4335289274810985, 3.308466105704095, 3.3638444470751816, 3.3761287122775037, 3.4020466971549257, 3.3954459504716716, 3.3948724983604093, 3.4257343832854255, 3.415143810260068, 3.4879100284758646, 3.497085039782676, 3.5066581554473584, 3.427946231167787, 3.576294080466981, 3.4596148881183306, 3.5354838758517224, 3.525790554702662, 3.5821779753751817, 3.5938130943638504, 3.554845812973703, 3.6009502661455968, 3.598437200685975, 3.5591469479214615, 3.58376196994903, 3.6233160875405477, 3.59181179286568, 3.6757391266002775, 3.6169199267770074, 3.6423780834598904, 3.709234362195252, 3.65288824184685, 3.7286115155857837, 3.6823133549113183, 3.7640449590743725, 3.763998424171642, 3.7366617735783767, 3.6924190133999866, 3.695500720078778, 3.7000829596428355, 3.750277946709068, 3.72454442719745, 3.7190019241563834, 3.8172358479469444, 3.7586947398580564, 3.7810156261845, 3.8055920821086615, 3.8299773832794966, 3.790804469661348, 3.815350228813803, 3.8461745180142155, 3.763334645587168, 3.8409795070150095, 3.744789320951814, 3.859256282733504, 3.805333337966044, 3.825823197698897, 3.889541655589061, 3.8466531949438107, 3.804554536084461, 3.863694341319382, 3.883376908909743, 3.8398974139219635, 3.8665467872741117, 3.847168874588742, 3.885501259451459, 3.9180113768122, 3.8577287166741243, 3.934500458134208, 3.887690254837085, 3.9154264767458486, 3.8883932699823074, 3.931310767580749, 3.945468138737284, 3.903706409369305, 3.9192276319880395, 3.935383480825242, 3.920062592075129, 3.916640389497113, 3.9822351689551287, 3.923293507782517, 3.927936956381342, 3.8762475441975197, 3.950982988260354, 3.8981805842393524, 4.011777427545779, 3.9225195957596894, 3.930417454166777, 3.9870092633423533, 4.035030159980628, 3.9455462344892465, 3.95732003108711, 3.9900319477555097, 4.009907155279901, 4.040954742462012, 4.008818290795491, 3.936990705265361, 3.9786848262616785, 3.9768354216958306, 3.9978761376848646, 3.98903341581867, 3.982772594804217, 4.033380197871263, 4.02748584367667, 4.054155270764782, 4.048776947768631, 4.0124200521760685, 4.007682463925356, 4.012383043386374, 3.9784420979250767, 3.952030373227065, 4.0038085484960275, 4.0307239438318145, 4.00712080897799, 4.0183809745083945, 3.9912531588487563, 4.027215654682961, 4.037382072703854, 3.98149468336895, 4.023401071311562, 4.01875849742039, 4.018052150489418, 4.023191963031793, 4.010419501620493, 4.007417933956074, 4.033540583719873, 4.132971678569818, 4.050593960816693, 4.046160245397289, 4.089493984629394, 4.009364923853783, 4.007299014717151, 4.029111058089383, 4.01918962958512, 4.116834481051014, 4.125578121015221, 4.026483336831354, 4.012811842238068, 4.0738745487419665, 4.066496524081868, 4.056229459252327, 4.079315037484381, 4.115617140842851, 4.094343126959102, 4.02778982812432, 4.067809740449213, 4.039569779566139, 4.130649457311934, 4.081167898360332, 4.075338919451283, 4.100282536950081, 4.085287734961054, 4.054929108376715, 4.092598650106199, 4.080673739408991, 4.069707834037247, 4.127206825906304, 4.114271800229504, 4.019139642928057, 4.147094508644882, 4.112433865571478, 4.033675021426693, 4.090921805163098, 4.116734047604215, 4.03298296898034, 4.105759158255948, 4.116265419941799, 4.131586922961436, 4.070570976111539, 4.114400513612541, 4.129239413388975, 4.129967690273455, 4.054357407958644, 4.092018081883716, 4.1048976015892755, 4.131401336117155, 4.081855192305936, 4.0477501655080514, 4.120061197098653, 4.145528340035943, 4.131661204775428, 4.087798130740026, 4.125407603895589, 4.07034863863781, 4.159083829563894, 4.084908726109061, 4.160790885330006, 4.085634739535629, 4.097233377444517, 4.099125007155594, 4.212632240763136, 4.12554463459428, 4.092972857177637, 4.10505709982222, 4.094654422656745, 4.058762457720034, 4.159819848218541, 4.048545566334087, 4.1137218991662285, 4.073544872034887, 4.091703607777881, 4.124462969743522, 4.107501513639074, 4.0681432781705436, 4.12541380353794, 4.068712959623641, 4.059803125205313, 4.068811355123095, 4.081020764484527]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="Adam")
data.update({'train_metric': [3.919, 9.854, 13.325, 16.212, 18.475, 20.04, 21.58, 22.793, 23.599, 24.577, 25.174, 26.023, 26.672, 27.241, 27.606, 28.293, 28.71, 29.022, 29.454, 29.683, 29.802, 30.146, 30.531, 30.783, 30.901, 31.039, 31.292, 31.545, 31.87, 31.662, 31.939, 32.217, 32.166, 32.452, 32.334, 32.407, 32.543, 32.648, 32.761, 32.827, 32.862, 32.968, 32.993, 33.168, 33.236, 33.412, 33.308, 33.272, 33.523, 33.634, 33.35, 33.645, 33.645, 33.619, 33.804, 33.903, 33.92, 33.987, 33.883, 33.927, 34.037, 33.952, 34.22, 34.21, 34.18, 34.135, 34.219, 34.363, 34.409, 34.335, 34.194, 34.34, 34.617, 34.574, 34.47, 34.486, 34.471, 34.65, 34.673, 34.63, 34.824, 34.788, 34.772, 34.688, 34.774, 34.934, 34.755, 34.738, 34.922, 34.781, 34.981, 34.901, 34.954, 35.009, 34.97, 35.043, 35.117, 35.059, 34.993, 35.123, 35.044, 34.876, 35.023, 35.194, 35.203, 35.266, 35.27, 35.078, 35.383, 35.132, 35.396, 35.199, 35.266, 35.192, 35.306, 35.325, 35.433, 35.379, 35.229, 35.358, 35.334, 35.498, 35.677, 35.405, 35.496, 35.444, 35.259, 35.485, 35.356, 35.497, 35.6, 35.516, 35.369, 35.384, 35.442, 35.401, 35.533, 35.574, 35.591, 35.804, 35.63, 35.746, 35.63, 35.565, 35.712, 35.74, 35.777, 35.524, 35.788, 35.806, 35.862, 35.644, 35.673, 35.641, 35.776, 35.795, 35.747, 35.882, 35.769, 35.774, 35.763, 35.77, 35.865, 35.817, 35.805, 35.94, 35.685, 35.808, 35.864, 35.717, 35.896, 35.808, 35.958, 35.942, 35.748, 35.881, 35.869, 35.926, 35.739, 35.759, 36.072, 35.912, 36.014, 35.918, 36.023, 35.804, 35.921, 35.939, 35.944, 36.114, 35.883, 35.972, 36.132, 36.014, 36.016, 35.817, 36.267, 35.914, 35.927, 35.995, 36.133, 36.151, 36.024, 36.252, 36.072, 36.133, 36.034, 35.997, 36.038, 35.89, 36.26, 36.102, 36.082, 36.14, 36.245, 36.162, 36.176, 36.068, 36.129, 36.195, 36.21, 36.294, 36.242, 36.346, 36.173, 36.327, 35.967, 36.306, 36.16, 36.345, 36.512, 36.013, 36.041, 36.039, 36.372, 36.212, 36.037, 36.118, 36.325, 36.274, 36.179, 36.28, 36.238, 36.301, 36.237, 36.365, 36.531, 36.347, 36.315, 36.313, 36.273, 36.165, 36.403, 36.34, 36.26, 36.33, 36.146, 36.29, 36.248, 36.451, 36.296, 36.372, 36.288, 36.4, 36.664, 36.362, 36.336, 36.249, 36.258, 36.371, 36.399, 36.503, 36.533, 36.287, 36.391, 36.244, 36.481, 36.509, 36.478, 36.521, 36.534, 36.32, 36.543, 36.413, 36.576, 36.426, 36.474, 36.497, 36.522, 36.51, 36.479, 36.437, 36.524, 36.585, 36.387, 36.378, 36.596, 36.67, 36.473, 36.497], 'train_loss': [4.809216335806721, 4.16354387079533, 3.907742426476262, 3.7337987287183334, 3.602146969646959, 3.5073430667835708, 3.4229271436302002, 3.3599539610032307, 3.3050923371879395, 3.259816557188028, 3.218286558213481, 3.185090426214978, 3.1542264152168085, 3.119928961904554, 3.095691357982975, 3.0711362535802507, 3.0469343567687734, 3.030853769524465, 3.009995093196154, 2.996113309323292, 2.9878856177827293, 2.973998941981632, 2.9561894724022784, 2.9455307651161005, 2.934888682606429, 2.9273810754276894, 2.9186271978965266, 2.9085416844161136, 2.900930546829507, 2.8909596400770408, 2.886237101606734, 2.8793296151945245, 2.8734581386593008, 2.8635712702611436, 2.8641058669514328, 2.8560385890138202, 2.8508213196362124, 2.846213976694694, 2.840899405384857, 2.8363933766109395, 2.8325608884075555, 2.828574453342899, 2.8308117719163364, 2.8244289262738658, 2.8181624873204636, 2.8144070441648124, 2.8098220031992107, 2.8086842088766457, 2.8026124184816523, 2.8025979086594632, 2.802847931954011, 2.801938024454977, 2.7933992180241582, 2.796492677274913, 2.786909176154695, 2.7877508896860035, 2.7874353124175557, 2.7838639491502857, 2.781963199510532, 2.780401480785182, 2.7752269389762074, 2.7743340667363396, 2.77253899906815, 2.7706139440042272, 2.76598027205513, 2.7674460647507346, 2.766194225959265, 2.7639930123941148, 2.7611628734981566, 2.7597979079128763, 2.760341300311488, 2.7595319531319316, 2.7528900006453503, 2.755815771246902, 2.7522164451259874, 2.750912105129532, 2.7445029879867153, 2.74285823476673, 2.7440454099548983, 2.7414237102940535, 2.7376763721497794, 2.736988756951047, 2.7393354612974044, 2.736973930915349, 2.7366330511510486, 2.733757722400658, 2.7380386158509356, 2.737037447424783, 2.7316271248949846, 2.733442189978699, 2.7312752488173513, 2.7308109212364053, 2.7283760490740865, 2.7206920213754255, 2.7251486987237814, 2.7234446877328846, 2.7234757262319613, 2.7263283438203585, 2.723021971820946, 2.7202172073430155, 2.7209609988707424, 2.7219990909061442, 2.7187128966997163, 2.7157673738129384, 2.7139557460448107, 2.715619118184671, 2.716302174142897, 2.7177691833376505, 2.712281469267603, 2.7132051122241956, 2.7084241573885324, 2.710034747956582, 2.7099700573538636, 2.7087000133284986, 2.707414806461151, 2.7083911802703113, 2.7055962843843093, 2.7052656145181246, 2.7078115207143725, 2.703771348649389, 2.7050269823080444, 2.7034978941321297, 2.6987973946603687, 2.700871193935226, 2.701662700449284, 2.700472941096593, 2.7013119036618973, 2.69985177031863, 2.701394336542409, 2.701361898878638, 2.701637707073873, 2.700141527678672, 2.697555312573414, 2.6985613901647176, 2.6967582145869082, 2.698909194669278, 2.695567088484078, 2.6988049120912168, 2.692476773368801, 2.6912057149585666, 2.691706297569983, 2.6922687767258227, 2.690478936724379, 2.6902955550073586, 2.6874145996242627, 2.687158193331991, 2.6854803409625227, 2.6889983075441495, 2.6902763367614453, 2.685078322696747, 2.6869132759780054, 2.6893428009592104, 2.6879831173445883, 2.6863003911227654, 2.685698988189014, 2.6845774312699673, 2.6885761090447637, 2.6853922829746972, 2.6846234661146227, 2.6841627247274036, 2.685394793081497, 2.6888036303846636, 2.6814922251460342, 2.683294781491456, 2.6811373291760017, 2.6817831887851065, 2.686038512033449, 2.6801803922012533, 2.6755914480657204, 2.683120025706764, 2.6779117636091803, 2.681937281778815, 2.6789235399079034, 2.678614537149992, 2.6806610432940268, 2.67725556993515, 2.6794499655373953, 2.677876583521593, 2.6792570391451784, 2.6795062851005844, 2.676094116801569, 2.6779203791688517, 2.6735153173835937, 2.675717169248516, 2.677041515660301, 2.6775082395538945, 2.6772246623908718, 2.6777948190673224, 2.6756541953205835, 2.6721542678577963, 2.67389419692034, 2.675818441315332, 2.6731076054441876, 2.6765774930812425, 2.6713569037668687, 2.675133275970464, 2.670351718224094, 2.6744823240730446, 2.6736942491543574, 2.6740189516918065, 2.6697881160755608, 2.670080907971754, 2.677852832882052, 2.6680779708781763, 2.6668552641523853, 2.6700624184050166, 2.667592107982721, 2.6672269192095834, 2.6720699000953485, 2.6727974381266844, 2.6652963154604246, 2.6675191952948833, 2.670692869126606, 2.6698398337635716, 2.6684712465192293, 2.667020195657751, 2.6673421442546834, 2.6680067549587747, 2.6671392138921064, 2.6645926150921744, 2.665966789080253, 2.665977682353439, 2.6628460779604968, 2.6595626698040613, 2.665823958199221, 2.659479384611451, 2.6650103742467692, 2.6632060649985316, 2.6648650256319835, 2.6633751433550055, 2.6604330255980684, 2.667453739022263, 2.663882652232072, 2.6662016935403425, 2.662034669444108, 2.6616923553548557, 2.6617913778516167, 2.664127633881279, 2.6605698802268285, 2.6591046959333364, 2.6630599032741897, 2.658662898069151, 2.658144866817667, 2.661838140078866, 2.6611157146235422, 2.6569180763156766, 2.659810706200847, 2.660875191081432, 2.6596316528564374, 2.659460800165407, 2.6579126202747445, 2.6614814152415565, 2.6579434199738943, 2.656372604275543, 2.659480229754213, 2.657429334832092, 2.659962079041445, 2.65912148011318, 2.657804526736625, 2.656176424529868, 2.655596964723852, 2.6575345021368064, 2.6587852716293385, 2.6574902307201635, 2.6534297714916772, 2.657882982694561, 2.6572421388754233, 2.6559146239981772, 2.655916257005278, 2.6524762189014552, 2.656849912703228, 2.6543561712107593, 2.6509229988908434, 2.6553217957436237, 2.6533087038185377, 2.656564032123856, 2.653571623224367, 2.6545437504981315, 2.649833002078251, 2.6510511210387286, 2.65273103459249, 2.654782076791091, 2.649433908901837, 2.653624557564065, 2.6528066914583426, 2.6521056947689825, 2.6513715174742716, 2.6547373938240155, 2.650904995313609, 2.6508915331298084, 2.64904575842127, 2.6514069680136436, 2.6507055821970953, 2.6488265402410094, 2.6508416231824126, 2.6529473379797763, 2.650466332585096, 2.648953649224338, 2.652162787857837, 2.6492477159658763], 'val_metric': [6.49, 11.74, 14.11, 17.22, 18.91, 19.68, 22.32, 24.16, 24.2, 25.87, 25.79, 26.49, 26.19, 28.72, 28.74, 27.82, 28.38, 29.61, 30.32, 30.44, 31.32, 31.32, 30.44, 31.3, 32.46, 32.54, 32.03, 32.06, 30.99, 32.89, 32.89, 32.26, 31.79, 32.02, 32.21, 33.13, 33.66, 33.11, 33.48, 33.84, 33.29, 34.08, 33.55, 33.55, 33.81, 33.49, 33.19, 34.17, 34.83, 34.07, 34.18, 32.74, 34.58, 34.88, 33.81, 35.06, 32.96, 35.4, 34.93, 33.91, 33.42, 34.94, 34.83, 34.59, 34.79, 34.75, 35.73, 33.9, 34.88, 35.42, 35.47, 35.86, 34.19, 34.82, 35.23, 35.46, 34.9, 35.28, 35.37, 35.15, 35.13, 34.32, 35.3, 34.85, 35.44, 35.27, 35.18, 35.77, 35.4, 35.04, 35.13, 35.18, 35.93, 34.45, 35.04, 35.67, 36.47, 35.12, 35.28, 36.0, 35.36, 36.05, 35.51, 35.61, 35.69, 35.91, 36.2, 35.48, 35.44, 35.41, 35.77, 36.04, 35.7, 35.19, 35.34, 34.83, 34.77, 35.41, 35.91, 34.67, 36.32, 35.76, 34.75, 35.93, 34.92, 36.82, 34.3, 35.24, 36.45, 36.46, 35.26, 35.33, 35.49, 35.62, 35.73, 34.69, 36.47, 36.22, 35.14, 34.59, 35.99, 35.11, 35.8, 36.05, 36.33, 35.25, 35.74, 36.51, 35.25, 35.89, 35.98, 36.11, 36.25, 35.99, 36.58, 36.39, 35.59, 35.62, 36.46, 36.55, 36.83, 36.93, 36.69, 36.03, 36.26, 36.33, 36.13, 36.01, 35.77, 35.79, 36.77, 36.4, 37.04, 35.98, 35.85, 37.0, 36.74, 37.12, 37.22, 35.91, 35.94, 36.4, 36.26, 35.51, 36.3, 36.3, 36.03, 36.34, 35.83, 37.35, 36.09, 36.32, 36.83, 36.69, 36.51, 35.37, 36.27, 37.13, 36.33, 37.18, 35.6, 36.11, 36.05, 35.8, 35.74, 36.99, 36.36, 35.88, 36.04, 36.63, 36.3, 36.65, 36.99, 37.14, 36.35, 36.33, 35.4, 36.45, 36.24, 36.23, 36.26, 37.22, 34.21, 36.74, 36.65, 36.49, 36.05, 36.44, 35.53, 36.02, 37.25, 36.92, 36.46, 35.34, 36.45, 34.72, 36.47, 35.79, 36.83, 37.43, 37.35, 36.39, 36.93, 36.63, 36.24, 37.51, 37.04, 36.37, 36.63, 36.46, 36.79, 36.71, 37.18, 36.55, 36.52, 37.36, 36.72, 36.64, 36.27, 36.74, 36.1, 36.59, 37.75, 35.32, 37.47, 37.16, 36.77, 36.99, 37.5, 36.62, 36.83, 36.23, 36.58, 36.62, 35.83, 36.85, 37.56, 37.34, 36.66, 35.89, 37.17, 37.47, 36.77, 36.5, 36.94, 36.77, 36.25, 36.36, 36.53, 37.27, 33.2, 37.78, 37.14, 37.29, 37.04, 36.18, 36.81, 36.25, 37.03, 36.96], 'val_loss': [4.42318685798888, 4.016302725312057, 3.9009910913030055, 3.6566110287502314, 3.5187618861532517, 3.49493363632518, 3.358245179152033, 3.273146579220037, 3.2424924107873516, 3.1839430408113323, 3.1751870843255596, 3.1345327431988563, 3.1837757188043776, 3.044190042337794, 3.019388955110198, 3.058213518683318, 3.0161079441665843, 2.9900692739304464, 2.968548265991697, 2.9636039604806594, 2.9148598841041515, 2.9130646157416566, 2.9518232846715646, 2.9447377639211667, 2.8531558756615705, 2.85681030552858, 2.868270754814148, 2.8638743465873087, 2.884670169490158, 2.8358316953015175, 2.819902322094911, 2.8268590072158037, 2.8720061832172856, 2.847361200174708, 2.863708546966504, 2.828048520027452, 2.799318668948617, 2.8220724915243256, 2.796091561864136, 2.7761302070253215, 2.8253460902317316, 2.7840408405680566, 2.8163652002431783, 2.8048090577884843, 2.7811485954151034, 2.7910880693204843, 2.797841298352381, 2.7867852775913895, 2.758328358838513, 2.7724233972039194, 2.760329335358492, 2.8060485830732214, 2.748600927887449, 2.7466440626010775, 2.7645399494535603, 2.7483130382124785, 2.809310910048758, 2.732179824713689, 2.7264547758041675, 2.780515805930848, 2.7810659044107813, 2.7373869867081853, 2.7353026764408037, 2.75434956277252, 2.7410912551697653, 2.7351782018211996, 2.6984060941987735, 2.787678407256011, 2.7249147595873304, 2.713916352600049, 2.713807402143053, 2.7070045979918946, 2.7709323813201516, 2.7443703952108978, 2.7087134145627356, 2.7294289200169266, 2.7133577264797917, 2.7388608425286165, 2.728161967484055, 2.7192269890171707, 2.7085930967027214, 2.7335588499239294, 2.710104984462641, 2.7437817032929437, 2.731044644003461, 2.715719147852272, 2.7354816312243226, 2.703687754406291, 2.720868234421797, 2.7227611822687137, 2.7232056301870164, 2.6960517494541825, 2.6884215105870726, 2.7607544850391945, 2.7320233507520832, 2.6932581807397735, 2.6788937179905594, 2.7040213445189654, 2.71457009862183, 2.6767947389062043, 2.7095564960674117, 2.6987787075103467, 2.6987410905254876, 2.7080906591597635, 2.700106602185851, 2.675215338445773, 2.6999555951470784, 2.700919843403397, 2.7094037100008337, 2.7041434079978117, 2.6813152155299096, 2.694215091550426, 2.6976778940030726, 2.6860341509436347, 2.686846831042296, 2.7351272212471933, 2.7167767407787835, 2.6958655210057643, 2.68734160426316, 2.7305396081535678, 2.6910132298803635, 2.7072590263026535, 2.72381952461923, 2.671894917822188, 2.7072226120408174, 2.6465282827425916, 2.7528219526740396, 2.6874607657171357, 2.6926458310929076, 2.6667350940643604, 2.7141345071185166, 2.699926201704961, 2.710617279550832, 2.7059923687558265, 2.690251450629751, 2.7399049754355365, 2.6634862612766823, 2.6617943838143803, 2.703843945150922, 2.716662317324596, 2.6781713966351406, 2.711423026528328, 2.6923394529682816, 2.6684503479368367, 2.6822406547084734, 2.6824622465546724, 2.684768984272222, 2.647354386414692, 2.6935372975221865, 2.690585571489516, 2.6765918659556442, 2.6543103547612574, 2.685358929785953, 2.692427850832605, 2.662169922688964, 2.671008693944117, 2.6959338909501485, 2.6973853361834386, 2.633552107082051, 2.667672109452023, 2.6375439155633282, 2.6532740820744993, 2.631318276854837, 2.659843500252742, 2.6721066130194693, 2.663570188033353, 2.6798552768245623, 2.689361098465646, 2.6850011348724365, 2.655351064007753, 2.6754560633829443, 2.6600278965227164, 2.651657083231932, 2.680490477069928, 2.663637177959369, 2.6329005698489536, 2.647984494828874, 2.656587165632066, 2.638979752352283, 2.673776448911922, 2.676108613895003, 2.6579211616212395, 2.6688393828975165, 2.685920305312819, 2.687029851470024, 2.6804647992370993, 2.677167021165228, 2.6524084510317274, 2.675634420981073, 2.6109483181291324, 2.657987642440067, 2.673306875927433, 2.637082375538577, 2.651900363575881, 2.638472084027187, 2.7054834844200473, 2.6870848084710968, 2.6346552466890616, 2.659498766349379, 2.6290995171115656, 2.6830157732507987, 2.66194541628953, 2.6505916559012834, 2.6775385353975234, 2.691408984220711, 2.625798859414022, 2.66821936703032, 2.6679123594502734, 2.6659944292846, 2.666716446542436, 2.657354138839017, 2.656310663861074, 2.649413417858683, 2.635166999640738, 2.6397479074016497, 2.6548065281218025, 2.7050856150639286, 2.6456712848821264, 2.6752024919364104, 2.6504237146893885, 2.640885785127142, 2.635491483530421, 2.7739907920740214, 2.6471325333710687, 2.6222771960458937, 2.6405627028957293, 2.6672072463734136, 2.664825078028782, 2.6866456133544823, 2.6687184841769516, 2.63323377272126, 2.6491110453939744, 2.6453365277332863, 2.716565657573141, 2.667230204412132, 2.736990158724937, 2.6613116135263137, 2.6528550652182026, 2.6336528803132904, 2.638497650243674, 2.6243032611859074, 2.6217877500376123, 2.6314721358050206, 2.6556522937337306, 2.66249400605062, 2.6011578998747904, 2.631157169676131, 2.6593775688462955, 2.646006479384793, 2.6598966421595045, 2.6298284766021047, 2.646176215190037, 2.6290033683655367, 2.6216117716898584, 2.662552753071876, 2.621107428696505, 2.656007092469817, 2.6541330913069903, 2.6781103223752063, 2.647150548400393, 2.653433366945595, 2.6273576123699263, 2.5889661357660962, 2.6882090291399865, 2.6102867259341442, 2.6475460206627086, 2.6539505089923834, 2.628129337243973, 2.626518771906567, 2.662642212430383, 2.642456210342942, 2.647214713369965, 2.668533603856518, 2.6385237977003597, 2.6678561754287426, 2.649888739464389, 2.61838210046671, 2.62326054284527, 2.647855169454198, 2.670556341007257, 2.6477792817316237, 2.6222841435936606, 2.6557496307761808, 2.678869764516308, 2.6293765314065727, 2.643821729975901, 2.6639852508617814, 2.6637377420048804, 2.6636282820610484, 2.6254073510504075, 2.8437910581090646, 2.633244532688408, 2.645303412227874, 2.6414451758573008, 2.6680225786889435, 2.660604339116698, 2.6497804776878113, 2.66986318150903, 2.64062174338444, 2.623922516585915]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="AdamW")
data.update({'train_metric': [4.798, 15.603, 24.619, 32.208, 38.563, 44.377, 49.77, 54.48, 59.097, 63.525, 68.26, 72.426, 76.31, 79.831, 82.997, 85.143, 86.827, 88.437, 89.435, 90.164, 90.985, 91.665, 92.181, 92.536, 93.198, 93.307, 93.763, 93.956, 94.221, 94.506, 94.623, 94.911, 95.032, 95.104, 95.371, 95.398, 95.585, 95.609, 95.931, 95.916, 96.038, 96.235, 96.265, 96.461, 96.366, 96.546, 96.526, 96.786, 96.763, 96.861, 96.784, 96.985, 97.036, 97.083, 96.919, 96.987, 97.303, 97.088, 97.36, 97.372, 97.346, 97.358, 97.486, 97.421, 97.337, 97.494, 97.501, 97.517, 97.744, 97.81, 97.594, 97.778, 97.651, 97.732, 97.767, 97.785, 97.887, 97.828, 97.885, 97.886, 98.042, 97.831, 97.974, 97.973, 98.083, 97.973, 98.03, 97.996, 98.118, 98.086, 98.1, 98.09, 98.16, 98.094, 98.095, 98.23, 98.238, 98.23, 98.204, 98.209, 98.231, 98.217, 98.303, 98.368, 98.241, 98.311, 98.345, 98.304, 98.426, 98.306, 98.355, 98.439, 98.482, 98.375, 98.534, 98.336, 98.446, 98.404, 98.487, 98.492, 98.475, 98.385, 98.467, 98.471, 98.572, 98.532, 98.478, 98.566, 98.472, 98.508, 98.535, 98.511, 98.648, 98.547, 98.605, 98.527, 98.732, 98.516, 98.601, 98.733, 98.56, 98.667, 98.671, 98.663, 98.664, 98.558, 98.614, 98.746, 98.581, 98.63, 98.623, 98.656, 98.722, 98.707, 98.663, 98.721, 98.615, 98.806, 98.663, 98.732, 98.682, 98.811, 98.719, 98.73, 98.751, 98.745, 98.742, 98.77, 98.782, 98.773, 98.853, 98.739, 98.807, 98.778, 98.818, 98.76, 98.892, 98.906, 98.76, 98.8, 98.811, 98.892, 98.869, 98.746, 98.811, 98.732, 98.897, 98.847, 98.785, 98.877, 98.9, 98.841, 98.913, 98.893, 98.796, 98.856, 98.873, 98.895, 98.938, 98.796, 98.946, 98.909, 98.841, 98.952, 98.926, 98.902, 98.883, 98.907, 98.943, 98.912, 98.995, 98.91, 98.969, 98.903, 98.967, 98.94, 98.942, 98.819, 98.966, 98.949, 98.875, 98.88, 98.945, 99.041, 98.857, 98.952, 98.855, 99.081, 98.961, 98.971, 99.007, 98.887, 99.055, 98.979, 98.978, 98.961, 99.008, 99.001, 99.067, 99.021, 98.948, 99.034, 99.054, 98.971, 99.031, 98.921, 98.999, 99.04, 99.062, 98.956, 98.964, 99.09, 99.002, 99.066, 99.049, 98.992, 99.017, 99.089, 99.044, 99.067, 99.103, 99.05, 98.989, 99.054, 99.105, 99.029, 99.051, 99.104, 99.087, 99.037, 99.051, 98.978, 99.125, 98.943, 99.065, 99.133, 99.008, 99.114, 99.06, 99.142, 99.044, 99.053, 99.04, 99.158, 99.065, 99.042, 99.105, 98.993, 99.092, 99.163, 99.079, 99.231, 99.042, 99.172, 99.034, 99.111, 99.149, 99.073, 99.118, 99.099], 'train_loss': [4.700914292783975, 3.7686333926305204, 3.2498084481374163, 2.858520185497428, 2.5378250916341294, 2.2547188747867284, 2.0039233386478434, 1.7796957716298119, 1.5687890481048872, 1.365894083670142, 1.1597251927936527, 0.979769650420087, 0.8172291529651491, 0.6829497351127027, 0.5688028973947331, 0.4849742507598984, 0.4280441485521737, 0.37282803898256556, 0.3407917192976824, 0.31582733973746335, 0.28919716739951995, 0.26596721656434597, 0.2506452414444907, 0.2389297476223052, 0.2173939436764688, 0.21467848711302093, 0.20167651446565976, 0.19247105275891527, 0.18412435127592627, 0.17551665026539615, 0.1693495456165064, 0.1627713570434431, 0.15699409304445303, 0.15501883741937725, 0.14744179848562486, 0.14490575443652964, 0.1399261625727933, 0.13723902565785234, 0.12888260743230992, 0.12911255531813245, 0.12543291240866206, 0.12042573035795358, 0.11718138349102482, 0.11372397075122984, 0.11504949552977668, 0.11028044416300442, 0.11000476709745172, 0.10153984946901276, 0.10324107607235501, 0.09963605590629429, 0.1020526681205595, 0.09571455556974387, 0.09345916600633349, 0.09454738876680466, 0.09559058804009575, 0.09400761799775993, 0.08784265151713341, 0.09104516048500373, 0.08431378256941306, 0.08530414258281838, 0.08389192627878742, 0.08310016870298449, 0.07968902059187796, 0.08299585217693421, 0.08201621605487933, 0.08035539095161896, 0.07908721578833888, 0.07751140315490831, 0.07166752022154355, 0.0701480193822737, 0.07720767597010048, 0.07077804108249215, 0.07443643495561442, 0.07078023045949437, 0.07055997709482693, 0.07053440630731488, 0.06762003587799889, 0.06889685549876359, 0.0685101345384652, 0.06624325004514123, 0.06368518929293358, 0.06808785280949833, 0.06354327265993094, 0.06452867297440176, 0.061570813306902256, 0.06310705913840635, 0.06097960132469143, 0.06392767456586884, 0.05881800705204126, 0.060132242513222287, 0.059881075820081196, 0.05996604651841022, 0.058988724263083735, 0.0600543519711719, 0.05902150906889055, 0.057489297750427715, 0.05624622571915484, 0.056845828789393274, 0.05736382835226161, 0.05617794513172677, 0.05545729277772419, 0.057196344203739924, 0.054011521907694586, 0.051655151406231545, 0.05494656091524969, 0.05238861488382453, 0.05177494738074002, 0.052192094001632425, 0.04999555164774712, 0.05313173879641963, 0.05207540742443399, 0.050564471749737515, 0.04860006088781381, 0.052722804080599574, 0.04665289508002755, 0.05302137504850907, 0.04814095344884984, 0.05029665874031278, 0.04777871299777028, 0.0470763822517406, 0.04885121993463569, 0.0508103218430195, 0.04730514204945154, 0.047224869873343865, 0.045799512043342074, 0.046948869584094895, 0.047581168232525575, 0.044962406924159176, 0.047123401542417634, 0.04700102268309873, 0.04631936665665643, 0.04531565420223062, 0.04296219422670691, 0.04657483813348414, 0.04468514042766735, 0.04714015169509806, 0.040007976499859536, 0.04715754560426772, 0.04472442256891981, 0.04144821696848741, 0.045920477615551986, 0.0416098523646148, 0.04288488932101977, 0.042750554246315825, 0.0425251193745455, 0.045533516230539826, 0.04299466614689653, 0.039843718612537396, 0.043705516695446395, 0.042932888537109665, 0.04298637344131402, 0.04184869873635822, 0.04127369193957629, 0.04110847325604609, 0.041649807564551056, 0.040388081537943064, 0.04333037865271891, 0.03874247245049916, 0.04205889306009195, 0.04027181492916708, 0.04042607760148466, 0.03860682829145982, 0.04139622363415088, 0.03964862626344839, 0.03927261304945306, 0.03965581974585321, 0.04026069358300393, 0.037895647990552325, 0.039253501919135736, 0.03877152951543892, 0.037480421575426554, 0.039133041161821557, 0.037947539146870414, 0.038400274247523354, 0.0378994259446882, 0.03831754783884952, 0.03665976398898946, 0.03534139893411719, 0.03810101787077641, 0.03885723698139764, 0.03705275228380891, 0.035606041302518224, 0.03566063767970809, 0.038407772674575485, 0.036682922303519, 0.038788892711204094, 0.035579533927021664, 0.03716403655075857, 0.03914817600657074, 0.03611974526553189, 0.03678380920720717, 0.03755776622823171, 0.033987393199576554, 0.03486961697347275, 0.03779676380111498, 0.035545136587904626, 0.03562953494341272, 0.03496506939178034, 0.03399028915525016, 0.03704510220803615, 0.03329008712825389, 0.03520647412983737, 0.036547836471972114, 0.03329950891697621, 0.03412622655617113, 0.035352069696155126, 0.034381272341269954, 0.03488678906716548, 0.032931920498997895, 0.03387082239596328, 0.03211207944813845, 0.0346611980768159, 0.03296882820269704, 0.03516041275251575, 0.032808783406279865, 0.03316755719194728, 0.03314652047385191, 0.0372629595085091, 0.03214727121771047, 0.033831044020194094, 0.034831964800400794, 0.03492994084949793, 0.0331331437825814, 0.030687715357134278, 0.03557504749567825, 0.03296906329157821, 0.035652578434579474, 0.029565749648748106, 0.032723807717975284, 0.03331547314770556, 0.03147142712862969, 0.03468950047516783, 0.031091295241489823, 0.031770539976624705, 0.03200505053120938, 0.03337127850329151, 0.031472876063815866, 0.03179379845079506, 0.029808670115189107, 0.03084719516393524, 0.03391309782402141, 0.03014473445967578, 0.030126343036622528, 0.03323707257088596, 0.030089746766914886, 0.033726338332414514, 0.03034014703945888, 0.03068091448887504, 0.029922660890375565, 0.03270767781946341, 0.032008588905890145, 0.029177951673319477, 0.03250998310456219, 0.030058844757848195, 0.030506811973095523, 0.031785252654988445, 0.029881819344678495, 0.029767212723892763, 0.03044173745055433, 0.029474435415627196, 0.02870352226042088, 0.030691446985373714, 0.03102991678614772, 0.030382599361054734, 0.027783467012015277, 0.03136568836643358, 0.029796085592188517, 0.02781141467563902, 0.028977194346265157, 0.03147246798790191, 0.029699976075166105, 0.032572017519650116, 0.027560065824313758, 0.033369651036252855, 0.028906803522258252, 0.026888935094672748, 0.030966659072787762, 0.026765059948272756, 0.030235811324487872, 0.027089613575656434, 0.03104464744348186, 0.029000135390063384, 0.029674598129588104, 0.027452032771606908, 0.02955399266342963, 0.029850437327122965, 0.028209724403452725, 0.031188663619924076, 0.0289500477648625, 0.02666669211764073, 0.02884338364547392, 0.024668931373862325, 0.03144893735851543, 0.025692836037593116, 0.02972813336761035, 0.028755981943396944, 0.026797556633447978, 0.030078760004935753, 0.02697990587062288, 0.02913727714931028], 'val_metric': [9.88, 21.21, 29.68, 36.34, 40.43, 44.89, 47.0, 48.88, 51.09, 50.95, 52.33, 52.31, 52.43, 53.12, 52.55, 52.8, 52.32, 52.65, 52.47, 52.26, 52.09, 52.22, 52.68, 52.78, 52.9, 53.49, 53.26, 53.3, 52.95, 53.11, 53.47, 53.43, 54.42, 53.68, 53.45, 53.61, 53.27, 53.81, 53.63, 53.89, 53.51, 53.78, 53.76, 54.26, 53.17, 53.61, 53.41, 53.58, 54.16, 53.5, 53.74, 53.71, 54.01, 53.79, 54.05, 53.99, 53.83, 53.28, 54.19, 54.1, 54.26, 54.21, 54.04, 53.79, 53.76, 53.83, 53.41, 53.95, 54.9, 54.41, 55.29, 54.5, 54.59, 54.67, 54.53, 54.2, 54.53, 54.37, 54.24, 54.23, 54.37, 54.68, 54.22, 54.41, 54.6, 53.86, 54.77, 54.15, 53.9, 54.24, 54.33, 54.82, 54.26, 54.38, 54.51, 54.81, 55.03, 54.86, 54.54, 54.68, 54.21, 54.46, 55.63, 54.6, 54.47, 55.21, 55.41, 54.43, 54.77, 54.64, 54.57, 54.33, 53.9, 54.95, 54.13, 54.71, 54.26, 54.76, 54.84, 54.39, 54.08, 54.76, 54.56, 54.01, 54.57, 55.06, 55.15, 54.59, 54.58, 54.73, 54.84, 55.55, 54.66, 55.11, 55.11, 54.89, 54.96, 54.72, 55.17, 54.96, 54.22, 54.37, 54.77, 55.11, 54.86, 55.28, 55.37, 54.04, 54.82, 54.49, 55.09, 55.32, 54.28, 55.23, 55.01, 55.08, 54.91, 54.62, 55.54, 54.98, 54.86, 55.16, 55.66, 55.46, 54.9, 54.65, 54.79, 55.13, 55.13, 54.9, 55.13, 55.23, 55.54, 55.29, 55.08, 55.34, 56.16, 55.67, 55.21, 55.51, 55.46, 55.73, 55.26, 54.71, 55.18, 55.37, 55.23, 54.82, 55.22, 54.78, 55.36, 54.75, 54.89, 55.07, 55.24, 54.98, 55.68, 54.45, 55.49, 55.12, 55.36, 55.07, 55.38, 55.45, 55.9, 55.79, 55.71, 55.5, 55.99, 55.12, 55.21, 55.85, 54.72, 55.48, 55.45, 55.08, 54.76, 55.96, 55.74, 56.04, 55.48, 56.03, 55.69, 55.49, 55.57, 54.94, 55.79, 55.85, 55.53, 55.45, 55.44, 56.01, 55.51, 55.74, 55.09, 55.25, 55.49, 55.68, 55.59, 55.09, 55.44, 55.71, 54.94, 55.16, 55.45, 55.4, 55.53, 56.09, 55.64, 55.17, 55.1, 54.97, 55.3, 55.73, 55.77, 55.74, 55.73, 55.14, 55.06, 55.72, 55.94, 55.78, 55.22, 55.0, 55.52, 55.63, 55.46, 55.71, 55.73, 55.4, 55.33, 55.62, 55.51, 54.94, 55.72, 55.65, 55.23, 55.35, 54.85, 55.33, 55.64, 55.13, 55.57, 55.66, 55.64, 54.82, 55.56, 55.34, 56.06, 55.4, 55.35, 55.12, 55.54, 55.68, 55.83, 55.53, 55.41, 55.14, 55.85, 55.34], 'val_loss': [4.127026843417222, 3.4504810837423725, 2.9990267077828667, 2.6716508721090424, 2.474953704578861, 2.2780829933798237, 2.181429686819672, 2.099366358700831, 2.0213918503682327, 2.0393366282153282, 2.061226214572882, 2.1128276225867544, 2.1621326834532866, 2.2402503543598637, 2.3570409261496965, 2.3608760343995066, 2.4912521599963973, 2.588355798630198, 2.6231257008139495, 2.687064337502619, 2.728919465450724, 2.7649514614396793, 2.7816544216909227, 2.755450233532365, 2.8223385795666154, 2.8316925908349884, 2.8538397132970723, 2.900367502573949, 2.894328415014182, 2.914297921262729, 2.9724706958053977, 2.9480411136985585, 2.9894709719973767, 3.0361139542737585, 2.9674433553294772, 2.9821533655664725, 3.0156085650632334, 3.090509695991589, 3.0879782328180445, 3.038703986034272, 3.066556916874685, 3.0434739308752072, 3.080259014846413, 3.108436119024921, 3.1020019559343908, 3.1371745865815766, 3.17004333635804, 3.180041649539, 3.1272621063669774, 3.183945339576454, 3.1756078718574186, 3.1685259247281747, 3.191921649844783, 3.2533020980798515, 3.214846213152454, 3.2486457467838457, 3.2255372667008904, 3.215252954868754, 3.2065558699285908, 3.1968225328026305, 3.1925099716065035, 3.242210524477017, 3.3300503036778446, 3.261298420702576, 3.249806092043591, 3.243475823645379, 3.3065568231473303, 3.2598912055325355, 3.292725522047395, 3.3137965152977378, 3.205700464309401, 3.32647265864026, 3.2751474574113346, 3.31253020209112, 3.330269596379274, 3.3125747943380075, 3.348108617363462, 3.344444523191756, 3.369235837915141, 3.377859754167545, 3.333320269159451, 3.3401457063711373, 3.3798080265142354, 3.3771230437953, 3.319164557821432, 3.3650583317325373, 3.364557171703144, 3.3639972346603493, 3.4604986044251995, 3.4002298412808947, 3.438120991181416, 3.3963810987533276, 3.4176874883994937, 3.430223849928303, 3.4145154094999763, 3.330891578060806, 3.3811888876993943, 3.4167465066454215, 3.4372093563626525, 3.358849036086137, 3.5190570122876745, 3.4167612327891552, 3.3725946196325265, 3.3977555287112096, 3.457482332636596, 3.435108846160257, 3.477252304933633, 3.470683060254261, 3.474376169359608, 3.5140720681779705, 3.4608383736792643, 3.4543308137328763, 3.49690371941609, 3.5045903943906165, 3.525229188287334, 3.43968191230373, 3.485611248927511, 3.46117343150886, 3.432246532789461, 3.5100608714826547, 3.5410308712606975, 3.469858639179521, 3.5441605961246854, 3.5406444342273056, 3.475112471610877, 3.5302674804523493, 3.4886637979252324, 3.5382033358713625, 3.5381942888733686, 3.489041926754508, 3.4754658375576044, 3.495974892263959, 3.50713816266151, 3.5043126925541337, 3.4998592464787186, 3.5218404451752923, 3.5078149585966854, 3.53529250128254, 3.5041383330229743, 3.5198227774565387, 3.568229512424226, 3.5887101562159835, 3.5575726484037506, 3.5704513469319434, 3.5809749725517954, 3.4735692359839274, 3.5248364365784224, 3.512189494956071, 3.5475690573643726, 3.5202411056324174, 3.495344318781689, 3.512563391855568, 3.6090715421233206, 3.5144427775577376, 3.550146146564727, 3.478390816290667, 3.4936204484313915, 3.5476315921279276, 3.489603582461169, 3.5303174223110174, 3.6386080790477195, 3.5637123083612723, 3.5133567585307324, 3.549952043849192, 3.5600246768088857, 3.608475277378301, 3.5932778218749224, 3.5699208807793394, 3.5453747647583107, 3.539801432448588, 3.562539298822925, 3.5678918224990746, 3.571500150261411, 3.5789894617287215, 3.583818326330489, 3.5646616505209807, 3.5548912977716727, 3.571920766192637, 3.6867391797387676, 3.6272119663323568, 3.580062197272185, 3.555027695218469, 3.5845170806927285, 3.6439212909929313, 3.6263499871181075, 3.632665822840041, 3.641346166847618, 3.610251541350298, 3.622605276715224, 3.657683894702583, 3.577497908264209, 3.576195809871528, 3.6972860018159173, 3.6323661481498912, 3.6041810132895304, 3.592482885737328, 3.5839178053436767, 3.6049291783836996, 3.6306441141541597, 3.613537451264205, 3.6518283759712413, 3.6607652219237794, 3.623517580852387, 3.596484804229372, 3.6182398067158497, 3.594975184103486, 3.63503858504022, 3.651035178998473, 3.60577144136854, 3.669184341172504, 3.6609269350197664, 3.6070651009583927, 3.656009332769236, 3.6265535202755292, 3.645898870222128, 3.6397484229628447, 3.6866405613862785, 3.6130300942499924, 3.619178907886432, 3.6048696350519824, 3.6181311576989046, 3.5541116023898884, 3.5888144696594044, 3.6418057209367207, 3.6428679679609406, 3.69039183968951, 3.6596079597807236, 3.6865989972072044, 3.6664717410020766, 3.6562197944920536, 3.6633736688619964, 3.6780059493271406, 3.630437240858746, 3.6451214084959336, 3.6911470525583643, 3.6612384376252534, 3.6718864495966845, 3.6550769851465894, 3.701589820870928, 3.707474588208897, 3.7135176134716934, 3.693405846881259, 3.7572500307089203, 3.683091330680118, 3.6897610507573293, 3.7321543325284483, 3.692181745532212, 3.743786452682155, 3.6394976088955144, 3.69254516644083, 3.6693555507690285, 3.7430118645072743, 3.766822383661938, 3.7336529390827105, 3.7423472036221983, 3.6552478906455312, 3.7193527764575496, 3.7221500042137827, 3.7586029471865126, 3.647877542456244, 3.6660011809342987, 3.74015448579363, 3.7032060721877276, 3.7323375522710713, 3.749531553429403, 3.7184596300884416, 3.6986140555636897, 3.665064470403513, 3.730938827915556, 3.7588702049225, 3.720109287720577, 3.7102631243171205, 3.763314857984045, 3.7546793343914544, 3.679881370750962, 3.7484781149845974, 3.6924613843298264, 3.6969353230136215, 3.698513744743007, 3.709085918156205, 3.7504834635242537, 3.6944863200187683, 3.7149668540924217, 3.7412564921530946, 3.713948130607605, 3.750028168699544, 3.707120606853704, 3.6813440918922424, 3.6638520044885623, 3.664181112483808, 3.696798678036708, 3.78822083116337, 3.690174291847618, 3.7550708047903267, 3.74093488712979, 3.7713146217309745, 3.7314119039067797, 3.7079219510600825, 3.7718056470725188, 3.688393093598117]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="SGD")
data.update({'train_metric': [0.797, 1.059, 1.244, 1.641, 2.078, 2.602, 3.083, 3.607, 4.16, 4.578, 5.048, 5.609, 6.124, 6.708, 7.253, 7.931, 8.432, 9.019, 9.599, 10.231, 10.87, 11.557, 12.046, 12.73, 13.289, 14.024, 14.413, 14.981, 15.566, 16.102, 16.822, 17.26, 17.902, 18.529, 18.991, 19.35, 19.823, 20.564, 21.136, 21.6, 22.084, 22.607, 23.031, 23.523, 24.183, 24.638, 25.237, 25.663, 26.278, 26.709, 27.338, 27.711, 28.385, 28.733, 29.373, 30.038, 30.367, 30.794, 31.358, 32.364, 32.44, 33.17, 33.685, 34.361, 35.031, 35.179, 36.016, 36.555, 37.211, 37.895, 38.472, 38.896, 39.783, 40.311, 40.902, 41.619, 42.124, 42.948, 43.558, 44.126, 44.799, 45.451, 45.797, 46.9, 47.348, 48.241, 48.736, 49.223, 50.127, 50.888, 51.395, 52.125, 52.921, 53.491, 54.05, 54.827, 55.501, 56.068, 56.772, 57.576, 58.139, 58.759, 59.245, 60.206, 60.583, 61.145, 61.837, 62.572, 62.855, 63.488, 63.873, 64.715, 65.262, 65.867, 66.449, 66.942, 67.392, 67.819, 68.35, 68.993, 69.36, 69.935, 70.191, 70.764, 71.315, 71.472, 71.976, 72.498, 72.914, 73.007, 73.807, 73.88, 74.444, 74.588, 74.964, 75.265, 75.366, 75.838, 76.175, 76.589, 76.834, 76.983, 77.575, 77.638, 77.914, 78.303, 78.514, 78.762, 78.975, 79.178, 79.509, 79.831, 79.806, 80.184, 80.349, 80.831, 80.712, 81.097, 81.343, 81.394, 81.523, 81.963, 81.928, 82.032, 82.56, 82.596, 82.769, 83.022, 83.068, 83.116, 83.346, 83.535, 83.528, 83.869, 84.094, 84.232, 84.181, 84.438, 84.674, 84.771, 84.988, 85.188, 85.077, 85.363, 85.235, 85.368, 85.539, 85.726, 85.966, 85.815, 86.237, 86.255, 86.213, 86.413, 86.717, 86.954, 86.974, 86.831, 87.04, 87.124], 'train_loss': [5.28679898543306, 5.239476534928256, 5.21706200164629, 5.182799006759243, 5.132641404848105, 5.057958123017333, 4.979468271813176, 4.9155346515921545, 4.8526697424231475, 4.797410400266153, 4.740543922856307, 4.678661715580116, 4.618669160573206, 4.5612786900745474, 4.502574600932389, 4.451376431734228, 4.40153864416951, 4.350875309775139, 4.3015161192119695, 4.256260085243182, 4.206814823589947, 4.164278143579504, 4.118582680525875, 4.077204191143209, 4.035535146087237, 3.9952102793536732, 3.954380479136569, 3.9142148386417714, 3.872954563078633, 3.8384448086231546, 3.798923760747879, 3.761674216368682, 3.7266336629883416, 3.6920101728030525, 3.653967627484449, 3.624490296466947, 3.586174421865667, 3.5510966053240893, 3.5209762192809726, 3.4884470894179587, 3.455062724502134, 3.423656351964442, 3.39157361871374, 3.3624736496979657, 3.330117495946219, 3.2988247244463316, 3.2687554646209502, 3.2439210536917735, 3.210005990526879, 3.181463266288479, 3.1521637940055958, 3.1227368072905146, 3.094225384528562, 3.0662183207879825, 3.0403141401855898, 3.0043576616395837, 2.9832378702901985, 2.9528195124288743, 2.9251777105276506, 2.8900695176591342, 2.8632295673197077, 2.8340791995450614, 2.8053349068129743, 2.7733713654623684, 2.7440339388636845, 2.719150159729648, 2.684663391998008, 2.6563735842628504, 2.6226322081938664, 2.5956457365954906, 2.5615613173004608, 2.534796960599439, 2.4996517120228314, 2.477093965825711, 2.4397944503309477, 2.4117870069968723, 2.3791143875128173, 2.3440938994278913, 2.315790370573848, 2.2891657439394786, 2.256056074598853, 2.2277818726791607, 2.19903620213785, 2.163079318676861, 2.1382001328758147, 2.1023707164836405, 2.077983120352659, 2.047940476232054, 2.0105564337240454, 1.980975041310145, 1.9543229102020605, 1.926220787997743, 1.8932993283122301, 1.8660690120909356, 1.8333772828925212, 1.8046104184191576, 1.7804262675464115, 1.7518898615528968, 1.7322479677139302, 1.6962341493471114, 1.6676528096122767, 1.6475050095480677, 1.6175086247333714, 1.589535445108371, 1.5679916839376902, 1.5489902965395557, 1.5204344786136332, 1.4921860101889588, 1.4680753478009352, 1.4461577372984176, 1.4335264536103451, 1.406715760418641, 1.3786714337837673, 1.3550085964602534, 1.3359942386643062, 1.31273990884776, 1.2963698529236147, 1.2819970409151689, 1.2604285980826833, 1.2321266578468695, 1.2184010728459593, 1.1969564216913355, 1.1842439979677084, 1.1698071482237988, 1.1435928917129454, 1.1363293796644254, 1.1191768815177263, 1.1038093723247087, 1.0814334088155877, 1.0683184337021063, 1.049535868759729, 1.0445316407974912, 1.0239556624732258, 1.0135577433931469, 1.000774418933988, 0.9905930642355579, 0.9790719618647814, 0.9624189027821644, 0.9539392639716618, 0.9403869682256792, 0.9249270919874854, 0.9183609809771762, 0.8976139287992845, 0.8942484503667932, 0.8876371004607383, 0.869497606446174, 0.8613092710364727, 0.8507564292263695, 0.8419575204814159, 0.8336129688293752, 0.8187584672802469, 0.81126471940173, 0.8076853354390584, 0.7930136273781306, 0.7868761020597554, 0.7670223771648688, 0.7695855016481091, 0.7595312807587424, 0.7512959717216967, 0.7436095214927342, 0.7369234124895707, 0.7283335632798921, 0.7273849702499192, 0.7186319234656433, 0.7016260518298573, 0.6958772467636406, 0.6931919688722374, 0.6840878379741542, 0.6784466867140296, 0.6718467884885906, 0.6683094913541546, 0.6608529589493459, 0.6579460149686914, 0.6480659762980155, 0.6356966151728969, 0.6335372443162548, 0.6286397591776673, 0.6220052392530044, 0.6152461118524225, 0.6120203538861553, 0.5993126283375331, 0.5941104890632081, 0.5974864511251907, 0.5890276762665805, 0.5883200924917436, 0.5827096428195406, 0.5751466392059777, 0.572285771093457, 0.5658543255568619, 0.5662903737655757, 0.5504932326436882, 0.5526105333465228, 0.5511946127114201, 0.5444512067890594, 0.5318624619406458, 0.5263343754957978, 0.5267493428039154, 0.5270881636148069, 0.5228219496101733, 0.5154156108277773], 'val_metric': [1.25, 1.52, 2.14, 2.5, 2.85, 3.54, 4.18, 4.99, 5.0, 5.57, 6.45, 6.49, 7.28, 8.34, 9.17, 9.44, 9.98, 10.71, 11.34, 11.98, 12.73, 13.02, 13.68, 14.68, 14.59, 15.35, 15.87, 16.64, 16.89, 17.44, 18.33, 18.24, 19.03, 19.42, 20.3, 20.17, 21.02, 21.42, 21.52, 21.9, 22.3, 23.26, 23.52, 24.36, 24.17, 24.42, 24.84, 25.12, 25.08, 26.23, 26.43, 27.2, 26.56, 27.05, 27.52, 27.84, 28.07, 27.95, 27.98, 28.59, 28.67, 28.45, 29.81, 29.15, 28.91, 29.35, 29.54, 30.19, 30.06, 29.61, 29.6, 30.53, 29.82, 30.28, 30.63, 30.73, 30.25, 30.15, 29.77, 29.56, 29.7, 30.44, 30.67, 30.72, 30.07, 30.06, 30.92, 30.29, 30.82, 30.84, 31.05, 30.98, 30.37, 30.36, 29.84, 29.42, 30.49, 30.77, 30.03, 30.29, 30.33, 30.33, 30.25, 31.04, 29.48, 30.46, 29.51, 30.51, 29.74, 30.42, 30.22, 30.51, 30.57, 30.63, 30.53, 29.66, 30.31, 29.7, 30.07, 30.54, 30.52, 31.05, 29.05, 30.91, 30.61, 30.13, 29.68, 30.58, 30.8, 31.15, 31.1, 31.02, 30.89, 30.99, 30.6, 30.55, 30.97, 30.49, 29.55, 29.74, 30.06, 30.6, 31.08, 30.94, 29.93, 31.54, 29.61, 29.6, 29.99, 31.4, 30.38, 30.91, 28.58, 30.56, 30.74, 30.53, 30.77, 30.87, 30.29, 29.11, 30.52, 31.02, 30.94, 30.89, 30.76, 29.83, 30.68, 31.42, 31.35, 30.38, 30.85, 31.09, 30.39, 30.23, 31.33, 29.86, 29.93, 29.18, 30.26, 30.16, 30.3, 28.12, 30.48, 30.74, 30.69, 30.84, 30.87, 30.57, 30.64, 30.7, 29.96, 30.75, 31.08, 30.54, 31.02, 31.25, 30.67, 31.29, 31.56, 31.39], 'val_loss': [5.241364612700833, 5.21055831726949, 5.178959521518391, 5.1474673201324075, 5.075622125795692, 4.9690739835143845, 4.901386572297212, 4.8184576611609975, 4.766212077657128, 4.705798656317838, 4.631328992782884, 4.578741463885945, 4.508968924261202, 4.423491210694525, 4.3695778512651, 4.326308505550312, 4.2878047949189595, 4.225931431837143, 4.176335986252803, 4.126579863250635, 4.0702482742868415, 4.033853717670319, 4.000999963966904, 3.9518446816000967, 3.919194882083091, 3.8675195062236423, 3.837831360519312, 3.791537013023522, 3.7615728864244593, 3.7436810086487204, 3.6959939655984284, 3.6520864788893683, 3.625196206341883, 3.5965643771894418, 3.5482898844275503, 3.5323962535068487, 3.4918812536130286, 3.4868309672471063, 3.4619337677196333, 3.443214778687544, 3.405829917093751, 3.38723209253542, 3.358455163658045, 3.3162168393469162, 3.3060660681147485, 3.295662794902826, 3.295079558518282, 3.2413248970250415, 3.2647369538143183, 3.207028499074802, 3.191243628787387, 3.1642114409975184, 3.1901716669653633, 3.1582978743656427, 3.1362624783424815, 3.1174151138135584, 3.112086278617762, 3.1014103502224963, 3.106941008264092, 3.0641693377950388, 3.0776264485280227, 3.091348347390533, 3.0383846532007692, 3.043195305356554, 3.0456675093644745, 3.0413299373760343, 3.032777452165154, 3.018443235925808, 3.0289335911441, 3.0264836709210825, 3.048241063288063, 2.989165168658943, 3.023004711813228, 3.0069771845629263, 3.0166418802965977, 3.0161715556102195, 2.9997285862637173, 3.048716870083171, 3.072573301898446, 3.081783930207514, 3.072356193688265, 3.0532584554830176, 3.06370976548286, 3.044704425107142, 3.0833523858125043, 3.085726354532181, 3.0291917551854612, 3.077522250497417, 3.0904398275788423, 3.0680306162803794, 3.0626103149098194, 3.1285634997544016, 3.127927921380207, 3.162978791127539, 3.176169321035883, 3.239808421985359, 3.1437082655110937, 3.1484961357845624, 3.2255469803597516, 3.2113526679907634, 3.219520025192552, 3.2154763466233662, 3.2523970019285846, 3.1769879703309125, 3.32243966524768, 3.2538518480434537, 3.3831598880184686, 3.2753917571086033, 3.3568596847497734, 3.281276062795311, 3.3349915097473533, 3.317715749619113, 3.326109629148131, 3.326367717639656, 3.3596560780409797, 3.409305995057343, 3.4147151746567648, 3.4301069143471445, 3.3806643417686413, 3.3879695287935294, 3.4124969023807794, 3.4231701656511633, 3.508554620727612, 3.443288414341629, 3.4792842394227437, 3.4883919293713417, 3.5299989918994297, 3.47647058318375, 3.464157213830644, 3.4484284641636407, 3.4783299493182236, 3.509651029945179, 3.5015364593001688, 3.498416795852078, 3.5338031828023824, 3.5499846008932514, 3.563610450477357, 3.633802477721196, 3.6966027598472158, 3.710776158199189, 3.667203581257231, 3.634347630154555, 3.56894614058695, 3.5724728183381877, 3.6947237602464713, 3.545918757748452, 3.710871283415776, 3.7575628347457593, 3.7154361768892614, 3.629476515730475, 3.713213993865214, 3.681844149425531, 3.915702246556616, 3.712513899347585, 3.74818428534611, 3.7099672115532454, 3.721311031633122, 3.763686257183172, 3.779520386343549, 3.9411601488757286, 3.781433427409761, 3.7015397890358215, 3.736769109015252, 3.7568162983390176, 3.781056351722426, 3.795392772953981, 3.7482630766121443, 3.7489858918888554, 3.7403810692440933, 3.7797794026933658, 3.7557731601083355, 3.783117841003807, 3.8285221977598347, 3.844910417392755, 3.8018890520569624, 3.903164346506641, 3.8946570150411812, 3.9157057643696, 3.8603066760263625, 3.9288764371993437, 3.909085033805507, 4.011820947288707, 3.8930255918745784, 3.854851158940868, 3.870553189402173, 3.8114114908655736, 3.8877651091593846, 3.92240904394988, 3.876576860998846, 3.9181657072844778, 3.9461730711019722, 3.8886753335879867, 3.896166161367088, 3.9097399256031986, 3.887490088013327, 3.8852153279978756, 3.904424210262906, 3.854526996612549, 3.8686341351004923, 3.869664792042629]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="Adam")
data.update({'train_metric': [2.502, 5.592, 8.334, 10.834, 12.924, 14.296, 15.392, 16.668, 17.544, 18.624, 19.126, 19.757, 20.159, 20.69, 21.218, 21.496, 21.791, 22.257, 22.445, 22.857, 23.113, 23.344, 23.359, 23.764, 24.133, 24.098, 24.39, 24.697, 24.689, 24.979, 25.168, 25.218, 25.277, 25.3, 25.378, 25.524, 25.596, 26.054, 25.883, 25.86, 26.101, 26.208, 26.033, 26.292, 26.216, 26.271, 26.334, 26.364, 26.649, 26.512, 26.715, 26.618, 26.742, 26.913, 26.802, 26.948, 27.034, 27.195, 27.219, 26.949, 27.156, 27.178, 27.223, 27.158, 27.42, 27.24, 27.28, 27.449, 27.28, 27.692, 27.443, 27.512, 27.58, 27.807, 27.688, 27.686, 27.65, 27.817, 27.675, 27.555, 27.718, 27.724, 27.869, 27.877, 27.871, 27.952, 27.946, 27.783, 27.879, 28.165, 27.864, 28.116, 28.042, 28.188, 28.095, 27.899, 28.305, 27.903, 27.981, 28.178, 28.049, 28.239, 28.115, 28.279, 28.028, 28.17, 28.091, 28.348, 28.176, 28.219, 28.081, 28.361, 28.263, 28.272, 28.188, 28.344, 28.409, 28.46, 28.305, 28.331, 28.31, 28.29, 28.523, 28.297, 28.191, 28.41, 28.42, 28.299, 28.512, 28.432, 28.414, 28.405, 28.346, 28.386, 28.492, 28.457, 28.381, 28.513, 28.521, 28.513, 28.713, 28.468, 28.321, 28.673, 28.578, 28.432, 28.46, 28.505, 28.551, 28.404, 28.354, 28.44, 28.643, 28.565, 28.608, 28.631, 28.658, 28.682, 28.499, 28.554, 28.671, 28.551, 28.478, 28.497, 28.33, 28.614, 28.656, 28.526, 28.539, 28.658, 28.763, 28.58, 28.708, 28.524, 28.628, 28.802, 28.641, 28.741, 28.598, 28.713, 28.724, 28.532, 28.987, 28.646, 28.611, 28.645, 28.719, 28.562, 28.801, 28.658, 28.878, 28.922, 28.561, 28.752, 28.874, 28.925, 28.845, 28.658, 28.882, 28.757], 'train_loss': [4.9440657895723366, 4.546375788409819, 4.3020748318728925, 4.118369432877671, 3.9700383471893486, 3.879940043941798, 3.8084714441671634, 3.737650628739721, 3.673354842307395, 3.6176447555642057, 3.579328491073042, 3.549126636501466, 3.51979341479501, 3.4900072954132706, 3.464394946626113, 3.4431253515293565, 3.4197776534201925, 3.4017185923844213, 3.388529452008463, 3.3698741764268734, 3.351727342575083, 3.341503542352775, 3.3327234247443163, 3.319326500517393, 3.3069049854425003, 3.293780196269811, 3.2855466769737687, 3.2737503465138715, 3.2683621261337974, 3.25969985426807, 3.2515486490245973, 3.2473460558661267, 3.2378148160069244, 3.2360064896420644, 3.229097666224835, 3.2245909745160843, 3.2167323677492536, 3.2126962888416233, 3.2113919706582568, 3.210326515857943, 3.2012352015982817, 3.1972077780630217, 3.1943570644673978, 3.189179095715494, 3.1861075714316343, 3.1845349847545856, 3.1823608422233596, 3.176632467173493, 3.1760711717208987, 3.1719630135226846, 3.1681164511487183, 3.1633981688237696, 3.161896502140311, 3.1589456727240837, 3.155547431853057, 3.152229374673835, 3.1476872455440725, 3.1497460804302877, 3.1452358501200024, 3.1423989573275515, 3.1449102283057995, 3.141450243581966, 3.1347639403584213, 3.1349055120293636, 3.132474987154501, 3.1342812594281355, 3.131587983245508, 3.124728048602816, 3.12763973786445, 3.120308046987708, 3.1236164531719965, 3.1202075124473354, 3.1169566830533175, 3.1113992072951695, 3.1118170121352184, 3.1149018800800152, 3.1128405423326058, 3.107239335756308, 3.1092516733756526, 3.1099076582236846, 3.1082146864858715, 3.1014870439518436, 3.0990832056361595, 3.1011919122587317, 3.0992950331455456, 3.1001631115311166, 3.0992358307463195, 3.0954417865091766, 3.0958578479801013, 3.092788766914656, 3.089998725584815, 3.092071986549265, 3.0925433805030047, 3.0867110332921004, 3.088684105827346, 3.090999559950386, 3.084376597206179, 3.0821942575299732, 3.087046893529227, 3.084816347843397, 3.0847152357900747, 3.080939648628845, 3.082392630024896, 3.080521562354197, 3.080435147288512, 3.0784558772430617, 3.0808241631232693, 3.0771333910453342, 3.0766992436260727, 3.0773669857286294, 3.0748356046084786, 3.0739820599937318, 3.077183703619627, 3.077137126574818, 3.076184259952831, 3.0784253303774336, 3.074109619424958, 3.070519978010113, 3.0739128466073473, 3.070393037429927, 3.0705790542595217, 3.072023957033456, 3.0692833671948123, 3.072587933207809, 3.069201744906962, 3.0709407753617963, 3.0667664673720427, 3.0696132756621886, 3.064735610898458, 3.0730165643563883, 3.0631446626349588, 3.0651941261303706, 3.0635166123976556, 3.0670492699416263, 3.06529861921236, 3.066581648736906, 3.06521260379906, 3.064291675351632, 3.0618830108520547, 3.0614440467063235, 3.0626351107257492, 3.062123822769292, 3.0640677168669797, 3.064283643253934, 3.0600357716158273, 3.061161760481519, 3.0614342526297347, 3.064315767678708, 3.0623797096660024, 3.0606955229587762, 3.064705465210605, 3.0588013463804375, 3.06038355918855, 3.0539507292358827, 3.0576881026123397, 3.0557376519274575, 3.057937576003511, 3.0569404884553153, 3.0608201399111854, 3.0559150551803893, 3.0562181553624033, 3.055634888333536, 3.0586898203317125, 3.059573251470418, 3.0579152386385284, 3.0527101948104147, 3.0570980399675425, 3.056150107832193, 3.057642869741888, 3.0524777112217647, 3.0529133039487912, 3.0542314301220483, 3.0540476821587013, 3.0557313723512283, 3.054331364573688, 3.0504385284026005, 3.0481127623328015, 3.052641860964355, 3.052914250758849, 3.054002314443704, 3.056859769808964, 3.0552519585029176, 3.0505481283403864, 3.0501610502095384, 3.0519176103026915, 3.0489030598221265, 3.050735301568732, 3.05083596775033, 3.0480414001284237, 3.047582964826988, 3.046238903044434, 3.0475747356488014, 3.047767222270856, 3.048186861393319, 3.0455715930850857, 3.0474920774680716, 3.047272910235863, 3.04604632703448, 3.0461786787859233, 3.0485887251370087], 'val_metric': [3.62, 6.47, 10.23, 12.74, 13.93, 15.75, 17.15, 18.22, 18.23, 19.17, 20.15, 21.11, 20.96, 21.64, 21.83, 21.84, 23.64, 24.06, 22.43, 23.83, 24.25, 24.71, 24.02, 24.6, 25.79, 25.55, 25.45, 26.13, 26.19, 25.97, 26.59, 25.8, 26.96, 26.37, 26.59, 26.9, 27.16, 26.45, 27.82, 26.97, 26.46, 27.46, 27.72, 27.71, 27.35, 28.54, 28.29, 28.36, 28.17, 28.34, 28.39, 28.22, 28.11, 28.41, 28.84, 28.54, 29.12, 29.45, 28.92, 28.09, 28.9, 28.47, 28.72, 29.26, 29.37, 29.04, 28.46, 29.36, 28.65, 29.01, 28.87, 28.76, 28.86, 29.11, 27.92, 28.87, 28.68, 28.56, 29.09, 27.67, 29.32, 28.56, 28.4, 28.54, 29.37, 28.65, 29.04, 29.71, 28.74, 29.44, 29.33, 30.07, 28.98, 29.64, 29.04, 29.52, 28.82, 29.44, 29.68, 28.8, 29.71, 28.48, 29.93, 29.88, 28.9, 29.22, 30.0, 29.65, 29.5, 29.51, 30.25, 29.7, 29.79, 29.36, 29.6, 28.91, 28.91, 30.01, 29.5, 30.59, 30.05, 29.62, 30.06, 29.57, 29.82, 30.01, 29.51, 29.58, 29.71, 28.8, 29.86, 30.28, 30.22, 29.59, 30.11, 29.3, 29.52, 29.21, 29.63, 29.63, 29.77, 29.78, 29.54, 30.18, 29.58, 30.41, 29.78, 29.66, 30.04, 29.09, 29.37, 29.02, 30.0, 29.87, 29.32, 30.0, 29.6, 30.64, 30.27, 30.59, 29.56, 29.37, 29.43, 29.24, 30.45, 29.55, 30.39, 28.87, 29.2, 29.55, 29.62, 30.38, 29.32, 29.65, 29.33, 30.05, 29.02, 29.87, 29.93, 29.94, 29.99, 30.44, 30.02, 29.86, 29.52, 29.96, 30.6, 29.84, 30.01, 30.07, 30.15, 29.81, 29.82, 29.69, 29.49, 29.05, 28.88, 29.92, 29.14, 29.14], 'val_loss': [4.723332915336463, 4.4435178216095945, 4.189916105027411, 3.9879672618428614, 3.8809655168253903, 3.7824059641285306, 3.708161612225186, 3.651200932302293, 3.6228875536827525, 3.5576466451025315, 3.4987658376147035, 3.4301216807335044, 3.4519149748383056, 3.4207497531441367, 3.383290392578028, 3.416995971825472, 3.3238360950141956, 3.302580502382509, 3.3608642938030755, 3.3016289252384454, 3.2742952860085066, 3.2708069496094043, 3.278764169686919, 3.275631665424177, 3.2232869840731286, 3.202714401445571, 3.1854232959686573, 3.1867804208378883, 3.1921240508936015, 3.2020497618207506, 3.1598642707630327, 3.22114906037689, 3.1578349953244444, 3.183200317583266, 3.157644670480376, 3.154807238062476, 3.137454258408516, 3.2009718342191853, 3.1121962693086855, 3.1525078723385076, 3.180015689248492, 3.132466130195909, 3.1167494779939107, 3.140483237375879, 3.1227735303769446, 3.0890943472552452, 3.104191372349004, 3.095190794604599, 3.1065060902553, 3.086936003083636, 3.083182646210786, 3.095618525128456, 3.0986245392234464, 3.097659663789591, 3.0655448922685755, 3.071247002121749, 3.0720054981814826, 3.0528144244175808, 3.0601694386476166, 3.064386010169983, 3.0642801629509897, 3.0944898280368487, 3.071917953764557, 3.051058192921292, 3.0479029561303985, 3.058290852103264, 3.0667884266300565, 3.0456524806417478, 3.069927604335129, 3.059104797187125, 3.0557860910512837, 3.0361367782969384, 3.055145657745896, 3.0533396271383686, 3.096804782843134, 3.0481922565751773, 3.061353972003718, 3.0953352109641785, 3.0307739039135586, 3.0846116246691175, 3.0176711226724517, 3.067533475578211, 3.058833206535145, 3.0700640139306428, 3.0587107489822776, 3.0699988292281035, 3.049884337528496, 3.0268402722231142, 3.0541298389434814, 3.0511069533171926, 3.0263707265732394, 3.0092848258413327, 3.049816336601403, 3.014720748943888, 3.0404825506696276, 3.0284000817377854, 3.0465507385837043, 3.0517023201960667, 3.003590075833023, 3.0374645860331833, 3.0167135934161533, 3.05759273516904, 3.006749345998096, 3.0175901043946576, 3.0467901184300707, 3.056482621818591, 3.0157423831854655, 3.0115726502837648, 3.0239258784397394, 3.0189798850162775, 2.991348671305711, 3.002635876843884, 3.0030203199690315, 3.011088140451225, 2.992003870617812, 3.0406983703564685, 3.024064504416885, 3.018254740223004, 3.003152626335241, 2.9900283054181727, 3.006458216412052, 3.0310462827135805, 3.0013755514363574, 3.0361835463031843, 3.0274441196660327, 2.997370977310618, 2.9956675798270354, 3.0227025701741503, 2.984021760855511, 3.0261121851623436, 3.0306005409568737, 3.003547869670163, 3.0000889369636585, 3.021361995654501, 2.990412763729217, 3.0256706331945527, 3.0151276474545714, 3.0119369508354525, 2.988885803587118, 3.005760715265942, 3.02091894635729, 2.992210766312423, 3.001223831419732, 2.988578706030633, 3.0240902976625286, 2.979908602252887, 3.0041506487852447, 2.9918590776479927, 2.979355624526929, 3.006400483429052, 2.9768805974608012, 3.0296851738243347, 3.0044739102102387, 2.9996559338964475, 3.0047975991182265, 2.98023470419987, 2.9958566199442385, 2.9834493952951613, 2.9849213126358713, 2.9883242306435944, 2.992150415280822, 3.011811863085267, 3.012509021789405, 3.0013042673183854, 2.9704260742588406, 2.99798003275683, 2.968681568552734, 3.009221266029747, 3.0146541876398074, 3.003788614728648, 3.0295434802960437, 2.996085843462853, 3.012355815832782, 3.031955624082286, 2.9969491913060473, 2.999671737099909, 3.0262429683831087, 2.983596793405569, 3.0029973163726225, 2.9841443808974732, 2.9964270781559548, 2.977669606542891, 2.9761712444815664, 2.9880468784623844, 3.011962183721506, 2.987397479023903, 2.97104493068282, 2.9714049434965584, 2.9955647515643173, 2.987701421330689, 2.979248664181703, 2.982273586996042, 2.983592392532689, 2.9879251187014733, 3.001416470594467, 3.0317055510867172, 3.012325415945357, 3.0080965696626407, 3.006959543865957, 3.017672989019163]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="AdamW")
data.update({'train_metric': [4.632, 15.093, 23.831, 31.188, 37.316, 42.372, 47.406, 51.845, 56.175, 60.45, 64.387, 68.656, 72.609, 76.214, 79.526, 82.045, 84.252, 85.962, 87.66, 88.487, 89.497, 90.435, 90.973, 91.677, 92.067, 92.37, 92.89, 93.166, 93.356, 93.515, 93.834, 94.545, 94.247, 94.612, 94.765, 94.837, 95.3, 95.218, 95.375, 95.653, 95.628, 95.751, 95.838, 96.013, 95.967, 96.209, 96.262, 96.341, 96.44, 96.394, 96.577, 96.486, 96.651, 96.778, 96.782, 96.733, 96.905, 96.999, 96.978, 97.03, 97.073, 97.211, 96.916, 97.436, 97.141, 97.458, 97.304, 97.292, 97.484, 97.314, 97.63, 97.374, 97.606, 97.478, 97.652, 97.61, 97.48, 97.642, 97.715, 97.685, 97.695, 97.785, 97.817, 97.762, 97.985, 97.742, 97.959, 97.858, 97.914, 97.976, 97.921, 97.941, 97.867, 97.987, 98.033, 98.014, 98.109, 98.059, 98.079, 98.025, 98.113, 98.124, 98.094, 98.125, 98.353, 98.16, 98.148, 98.142, 98.269, 98.204, 98.223, 98.247, 98.311, 98.148, 98.409, 98.179, 98.369, 98.231, 98.399, 98.389, 98.347, 98.345, 98.344, 98.401, 98.468, 98.351, 98.376, 98.414, 98.455, 98.48, 98.499, 98.461, 98.563, 98.545, 98.483, 98.53, 98.393, 98.571, 98.522, 98.523, 98.44, 98.506, 98.556, 98.519, 98.608, 98.487, 98.676, 98.423, 98.594, 98.66, 98.575, 98.65, 98.566, 98.651, 98.455, 98.655, 98.589, 98.65, 98.603, 98.627, 98.668, 98.629, 98.64, 98.73, 98.667, 98.718, 98.683, 98.649, 98.701, 98.569, 98.804, 98.727, 98.67, 98.765, 98.82, 98.655, 98.843, 98.808, 98.679, 98.726, 98.701, 98.758, 98.784, 98.756, 98.734, 98.886, 98.784, 98.701, 98.704, 98.848, 98.776, 98.76, 98.885, 98.742, 98.816, 98.812, 98.829, 98.9, 98.774, 98.857], 'train_loss': [4.732618023170086, 3.837009495218366, 3.303636210741176, 2.9162444661080036, 2.6102483579918734, 2.35137929355038, 2.1113562946203652, 1.9034441642401239, 1.6983560709639078, 1.501836548649342, 1.3140563661672333, 1.1307402454502522, 0.9696635115589001, 0.8196774184208074, 0.6917678799983102, 0.5969258051649241, 0.5173240576656827, 0.45705384119954234, 0.4026908167731434, 0.36730907017976444, 0.33799160448733007, 0.3091700930846706, 0.2915964995201627, 0.2667016276526035, 0.25500580096904546, 0.2455825961968949, 0.22917890205254787, 0.21740676392100974, 0.21520310129805856, 0.205673031077523, 0.1986409930210806, 0.17311373461092694, 0.18209599255943482, 0.17203718418003996, 0.16554157964537256, 0.16225085192391564, 0.15068647427030352, 0.15269555422100248, 0.14716356100427536, 0.13954499640197554, 0.1393739470135914, 0.13609240597479821, 0.13231162346306036, 0.1267253485235957, 0.12863788018461916, 0.12096185347879268, 0.11826254574371048, 0.11817707311301251, 0.11234925859843119, 0.11422019781394649, 0.10833220253266532, 0.11148996399006475, 0.10575503376965016, 0.10198995208853924, 0.10340837339000086, 0.10412448788149925, 0.09810488301037941, 0.09555452353823322, 0.09606356156086615, 0.09498219757867585, 0.09269915055304823, 0.08851892941438319, 0.09697304070901581, 0.08181372417228788, 0.09198502776354804, 0.08058428362751366, 0.08568276059548642, 0.0861352871771561, 0.08067701196700802, 0.08613770151384513, 0.07745730407045641, 0.08328720502929568, 0.0744105813736114, 0.08062492288395545, 0.0741011889240873, 0.07669324850505939, 0.07892138172764515, 0.07557215063084305, 0.07319421069151337, 0.07225877768702696, 0.0739623775627638, 0.0707695698663497, 0.0703790559117895, 0.07061802165243734, 0.06590981996988833, 0.07151409076243431, 0.06349367843908083, 0.06761271939332873, 0.06546633114566953, 0.06343452500586677, 0.06594923066251786, 0.06549203731891266, 0.06651636300684785, 0.06291862163363204, 0.06066547398233783, 0.06393249317129535, 0.06132274655275457, 0.06067794305600121, 0.06114760873960218, 0.06266886004668637, 0.059590738921924374, 0.06035010492004888, 0.060689583262404805, 0.05971300051820489, 0.05300128179051456, 0.05885625864654393, 0.05863192462807304, 0.06133633705148901, 0.0534089224587921, 0.05686913337491474, 0.057192922669468224, 0.05503541301518514, 0.053440196434410094, 0.05848385009091462, 0.05160101716129868, 0.057150932076893285, 0.05102714938956601, 0.05584967904038947, 0.051384327532330284, 0.052873028780933126, 0.05246768004753432, 0.05214827802556288, 0.052365857262900384, 0.05184441720199862, 0.04837568244822049, 0.053191665196684036, 0.05285064615644035, 0.04972174256390243, 0.04856179714995667, 0.04932078070203748, 0.04875925646737928, 0.050052618895974954, 0.047044544609296905, 0.047026082374129, 0.0484978699827938, 0.04764719910923986, 0.05157985565709475, 0.0455818447133649, 0.04699744367539276, 0.04554822205006361, 0.04901024588381947, 0.04696501808287784, 0.04644845231971421, 0.04724760122216764, 0.04481186228379743, 0.04782091425882232, 0.04117747099580177, 0.049106424134062285, 0.044710916179827115, 0.04401199149371331, 0.045462756289564095, 0.044048683931163235, 0.04692669601677348, 0.04319325230333576, 0.04695678100963966, 0.042260564832734535, 0.04402171409153662, 0.042779315127177794, 0.04419658530410558, 0.0441840099241115, 0.04250341285000301, 0.04280655950753844, 0.043132940174756135, 0.040964572609316435, 0.041430806528137236, 0.041188135079879215, 0.041347916264674475, 0.0431902252277836, 0.04051134067052625, 0.04398012631448398, 0.03827387652300615, 0.039455664331637594, 0.04303911197570082, 0.04008040688832881, 0.037710260656457904, 0.04249879849492349, 0.03625092613535367, 0.038930501967760295, 0.04216071377494637, 0.040890968007777244, 0.041099238990779524, 0.03952094578911206, 0.03886420811218569, 0.03923791633885811, 0.040445404888594896, 0.036827190037644265, 0.038715501181526266, 0.04077233530352189, 0.04098035720009708, 0.0368164446480782, 0.0386089197325643, 0.0396656532096818, 0.0356937078955148, 0.04198693911110302, 0.036940265491534646, 0.036931044227133496, 0.03766117273725677, 0.035766234265493496, 0.03854017711711674, 0.037248457466502106], 'val_metric': [10.18, 20.63, 28.62, 33.44, 38.87, 42.3, 44.5, 45.91, 47.37, 48.25, 47.78, 48.35, 48.64, 48.24, 48.5, 48.92, 48.48, 48.89, 48.52, 48.86, 49.51, 49.48, 49.23, 49.52, 49.6, 49.25, 49.12, 48.97, 50.74, 49.63, 50.45, 49.26, 49.4, 50.64, 49.37, 49.66, 49.99, 49.97, 49.34, 50.15, 49.4, 49.85, 50.25, 50.08, 49.65, 49.31, 49.92, 50.47, 49.85, 49.77, 50.66, 50.09, 50.01, 51.08, 49.46, 50.84, 50.19, 50.81, 50.3, 50.45, 50.07, 50.92, 50.84, 50.33, 50.94, 51.17, 50.44, 50.02, 50.4, 50.87, 49.89, 50.32, 50.66, 50.98, 50.63, 50.96, 50.99, 51.16, 50.87, 51.32, 50.66, 50.46, 51.55, 50.92, 51.71, 51.47, 50.98, 50.81, 51.41, 50.91, 50.99, 51.37, 51.5, 50.68, 51.04, 51.25, 51.27, 51.19, 51.44, 50.68, 50.86, 51.49, 51.47, 50.82, 51.47, 51.62, 51.0, 51.23, 51.78, 51.36, 52.01, 51.9, 51.79, 51.63, 51.55, 50.93, 51.88, 51.47, 51.36, 51.13, 51.22, 51.1, 50.95, 51.26, 51.78, 50.63, 51.32, 51.38, 51.53, 51.56, 51.54, 51.24, 52.2, 51.43, 51.81, 51.6, 51.68, 51.51, 51.73, 51.62, 51.62, 51.63, 51.75, 51.66, 51.12, 51.66, 51.62, 51.82, 51.56, 51.3, 51.11, 52.43, 52.26, 51.99, 51.3, 51.87, 52.14, 52.32, 51.47, 51.92, 52.25, 52.39, 52.26, 52.18, 51.44, 51.96, 51.75, 52.42, 51.49, 52.57, 51.72, 51.33, 52.23, 51.89, 52.53, 51.49, 52.6, 52.1, 51.77, 52.26, 52.21, 51.51, 52.39, 52.17, 52.26, 52.53, 52.59, 52.64, 52.0, 51.75, 51.97, 51.96, 51.51, 52.01, 51.87, 52.47, 51.77, 52.03, 52.46, 52.33], 'val_loss': [4.155144642872416, 3.4760255616181976, 3.0528167600085023, 2.8072206715869297, 2.5343799196231136, 2.3872334983698122, 2.29174277812812, 2.2223417500781406, 2.2063083451264984, 2.2029359033153315, 2.2515892743305037, 2.2763107627820056, 2.3464909613512126, 2.4708192236484234, 2.5361539581019406, 2.605528296179073, 2.7538433974715555, 2.792878375691213, 2.856342786816275, 2.876577492352504, 2.9499322882123815, 3.008299588018162, 3.049083854742111, 3.109117484776078, 3.0830218624916808, 3.1404042756481534, 3.167250411146006, 3.245663550628978, 3.1164535978797137, 3.2232938382276304, 3.123378391858119, 3.2803032140063633, 3.268281535737833, 3.245649062524176, 3.3159585591334446, 3.300858881822817, 3.2820482683029906, 3.3758431407296734, 3.3515588569033676, 3.349676922248427, 3.3850666747731006, 3.399772790207225, 3.413526945812687, 3.3991642252654786, 3.428160581619117, 3.487150806530266, 3.4561948449748336, 3.4231206102735676, 3.47397564778662, 3.477116117052212, 3.4147630671786655, 3.487475205378927, 3.5192891378311595, 3.4689539249535577, 3.5989073241592213, 3.4780162595639563, 3.4958409594882065, 3.5264381473990762, 3.4979907437494604, 3.4946238926261852, 3.5691893609466065, 3.5302274690312183, 3.5002562870645217, 3.5899184418332046, 3.537055731578997, 3.570720785362705, 3.6388734244996574, 3.6424595918625022, 3.6274931495356713, 3.565669477365579, 3.645692820002319, 3.6642542971167593, 3.697098011803475, 3.6723012779928315, 3.641013586597078, 3.615912958315224, 3.617185760455526, 3.6151476999756635, 3.652369750912782, 3.6477714625133832, 3.6666782657811594, 3.705400138523928, 3.6301203419448465, 3.6381955678295936, 3.6255966512260924, 3.67827575146013, 3.964861224791047, 4.017418283945436, 3.6502543163906997, 4.058486811294677, 3.849061124264055, 3.7730496931987205, 3.657746048110306, 3.688899126402132, 3.70118728564803, 3.722269688062607, 3.7045109378304453, 3.682860689558041, 3.6910154629664818, 3.8233025324572423, 3.974473794554449, 3.7401212271611404, 3.740928265699156, 3.722122119490508, 3.7579256964337295, 3.7350299851909563, 3.7568764265175836, 3.6678481481637166, 3.768069913432856, 3.7962463820815846, 3.709798229348128, 3.7030529649394333, 3.767607392019527, 3.711196309062326, 3.7705023656984804, 3.785187952837367, 3.738127908129601, 3.8013323389800493, 3.8158053827893204, 3.8397207742284056, 3.806489633906419, 3.8354496826791458, 3.7596919081013676, 3.7952433870096876, 3.7537987110721076, 3.848316994442302, 3.8629335339661615, 3.8186594057994285, 3.8488086769535284, 3.7825894788571985, 3.793389841249794, 3.82152760636275, 3.7903997298258885, 3.817480377710549, 3.812692967569752, 3.813071140058481, 3.7736024454141117, 3.8550736972480824, 3.8560603211639792, 3.8661503996818687, 3.84623070583222, 3.78617000427975, 3.8071780941288944, 3.7968922135936225, 3.8913753746421476, 3.850919366262521, 3.8600478794924014, 3.8849508451048735, 3.9085027609661127, 3.858762707300247, 3.8566394285031946, 3.814949914148659, 3.866777182384661, 3.8228413474028278, 3.8981434000525503, 3.8271993345515742, 3.823091029741202, 3.8004404579757884, 3.859751614036074, 3.832587334760435, 3.8486308094802175, 3.871706398809032, 3.8640471605738256, 3.8597827914414133, 3.8420219998450795, 3.796103212484129, 3.9206087904371274, 3.8846735969470565, 3.8522531872342345, 3.861181190059443, 3.8792950833679005, 3.9349649590291795, 3.8576200889174346, 3.8534476198967855, 3.8461800046787142, 3.8874274385962515, 3.826181959954037, 3.973541209652166, 3.98267132119768, 3.887461848319716, 3.861559626403128, 3.8705231444850847, 3.858390007049415, 3.8917423304478835, 3.88808529468099, 3.8621841540002517, 3.8718309880821566, 3.886681822834501, 3.8839310502550406, 3.9208719282393245, 3.928670901401787, 3.8676295466483777, 3.9497483239811695, 4.023665704165294, 3.8869255200313155, 3.8853342590058686, 3.904488135295309, 3.89199039890508, 3.843072326319992, 3.9504053213034465]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="SGD")
#data.update({'train_metric': [2.923, 7.969, 12.889, 17.413, 21.489, 24.945, 28.133, 31.344, 33.974, 36.686, 38.995, 41.542, 43.985, 46.158, 48.752, 50.425, 52.559, 54.658, 56.57, 58.551, 60.451, 61.985, 63.761, 65.254, 67.21, 68.798, 70.1, 71.801, 72.96, 74.507, 75.334, 76.665, 77.666, 78.654, 79.387, 80.682, 81.222, 81.997, 83.114, 83.391, 84.079, 84.85, 85.364, 85.85, 86.316, 86.76, 87.328, 87.718, 88.065, 88.517, 88.712, 89.309, 89.502, 89.742, 90.013, 90.219, 90.388, 90.805, 91.029, 91.094, 91.462, 91.554, 91.643, 91.931, 92.053, 92.367, 92.376, 92.627, 92.695, 93.022, 93.1, 93.144, 93.251, 93.403, 93.505, 93.604, 93.895, 93.905, 93.908, 94.055, 94.036, 94.091, 94.012, 94.191, 94.43, 94.696, 94.634, 94.8, 94.78, 94.921, 95.104, 95.078, 95.078, 95.068, 95.131, 95.249, 95.351, 95.358, 95.499, 95.5, 95.719, 95.648, 95.645, 95.64, 95.837, 95.804, 95.875, 95.935, 95.9, 96.071, 95.972, 96.049, 96.032, 96.203, 96.201, 96.2, 96.186, 96.144, 96.322, 96.419, 96.341, 96.492, 96.444, 96.468, 96.217, 96.331, 96.531, 96.48, 96.635, 96.559, 96.661, 96.65, 96.769, 96.828, 96.77, 96.763, 96.87, 96.766, 96.832, 96.883, 96.875, 96.973, 96.927, 96.974, 97.012, 96.936, 97.006, 97.217, 97.004, 96.894, 97.088, 97.05, 97.198, 97.297, 97.029, 97.14, 97.294, 97.259, 97.373, 97.372, 97.32, 97.306, 97.36, 97.423, 97.298, 97.355, 97.444, 97.382, 97.43, 97.385, 97.514, 97.3, 97.425, 97.46, 97.377, 97.501, 97.497, 97.544, 97.516, 97.531, 97.427, 97.508, 97.558, 97.542, 97.625, 97.677, 97.585, 97.633, 97.674, 97.638, 97.575, 97.682, 97.627, 97.543, 97.774, 97.647, 97.752, 97.791, 97.818, 97.657], 'train_loss': [5.007517597420583, 4.447556939250143, 4.0600041225028205, 3.756176028889261, 3.511490281544964, 3.303963253456892, 3.1284893905971574, 2.968347872211166, 2.8207010647767032, 2.6878731565756135, 2.5638351397718746, 2.44357360446598, 2.329064052911882, 2.2236845079173055, 2.1161179335851052, 2.0246211206463003, 1.929622119539301, 1.8334968151073003, 1.7540812717365746, 1.6701979422065896, 1.5934930221056678, 1.5190278740548508, 1.449817888948754, 1.383577428181356, 1.3090682796614337, 1.2463418981316603, 1.1881295177925877, 1.126676733724139, 1.0784986989283059, 1.0247822741781833, 0.9839487526939987, 0.9371495652641155, 0.8934069168697316, 0.8567577519061393, 0.8236100911827173, 0.7831364846046507, 0.7552088770398099, 0.7299414170871388, 0.6893731474647595, 0.6744975348504323, 0.6455670554746211, 0.6205885873169603, 0.601066595609571, 0.5819927600813614, 0.5594771294470254, 0.5461740641069961, 0.5248172656321327, 0.5107438193439904, 0.49638769918157744, 0.47901090725026524, 0.4697688879470221, 0.4482770291129062, 0.4429707792006619, 0.4319908135282787, 0.42044053185237806, 0.408997640719188, 0.40204067823792294, 0.38683891641506385, 0.379723505196248, 0.3790662564380155, 0.36511224224658173, 0.35821260576227576, 0.35186431392483886, 0.34086454477130185, 0.33734167964982437, 0.32965755243848244, 0.32777905685354025, 0.31612508804540335, 0.31374863780181683, 0.3029721135789587, 0.2988102079934595, 0.2949390722222993, 0.2903954585679281, 0.28428189538452614, 0.28075802727094157, 0.2790024209379082, 0.26810189337491686, 0.2659690751004738, 0.26431995343776826, 0.25769191960817867, 0.26098902427704, 0.2561122155626157, 0.2599565333097468, 0.25402634498149024, 0.24596718100024123, 0.2359287637909749, 0.23550792713664467, 0.22830843977911383, 0.2295632509136917, 0.22460625817855046, 0.22133918632577415, 0.21951337342441882, 0.21734121364141532, 0.21981022497418592, 0.21607891160825546, 0.211759512053513, 0.20816203295917596, 0.20731216941507902, 0.20079824699759866, 0.20094201711893692, 0.19514334244952702, 0.19542425778180228, 0.19821914735649995, 0.1953237008253352, 0.1892002995911883, 0.18911813417603324, 0.18567357760254039, 0.18295167213971997, 0.18508376579872324, 0.18035198551717663, 0.18154973040858638, 0.18081435914165075, 0.17890411688542793, 0.1742313659117722, 0.17381234824423863, 0.17454894609935223, 0.1738890577238756, 0.17443535335943514, 0.16849189544107315, 0.16542238886786895, 0.16742476050623395, 0.1635546485630656, 0.16251445602165607, 0.16268701110838393, 0.17355902819290653, 0.1688782323118459, 0.1621809265017986, 0.16223413664645023, 0.15754563462820978, 0.15795141735345944, 0.15790045961990276, 0.15511920334329152, 0.15410015317930179, 0.150547818803322, 0.15108817277484535, 0.15030131504530717, 0.14928182341177457, 0.14855958015603737, 0.15112534191480867, 0.14639384333003772, 0.14636532384461307, 0.1453638076412796, 0.1453411412244795, 0.14214244494673464, 0.1427639620777837, 0.1432192337759416, 0.14173203770140388, 0.13524863294904346, 0.1414210389744221, 0.14418657090436246, 0.13775682114589047, 0.1379736646144667, 0.13560289383037058, 0.13160663145407797, 0.14030122272884776, 0.13552884087181022, 0.130274877775644, 0.131990786274789, 0.12836796491003463, 0.1276023709003695, 0.1295262678447093, 0.1296162928442065, 0.13001933979539063, 0.12712747546095193, 0.12896313709317112, 0.12711445365389493, 0.12547911734630665, 0.12691569048911333, 0.12549595892791632, 0.12702353850069026, 0.12207344626639603, 0.1293213498514677, 0.12562855220473088, 0.1251548837574815, 0.12694335878562713, 0.12263502775836242, 0.12496576838252527, 0.12257559681269539, 0.1225283168821154, 0.12048658249412335, 0.1256811353017505, 0.12274520528775068, 0.12006900139315077, 0.12103370886941942, 0.11807865369349947, 0.11560929564953384, 0.11787954175169088, 0.11625433878368929, 0.11793900964518274, 0.11878552946475973, 0.11954389085906175, 0.11611049157529783, 0.11802413367731253, 0.12242065672539246, 0.11491679333827279, 0.11880402751767476, 0.11340122237701449, 0.11178001655493305, 0.11296205236549683, 0.11643465314005631], 'val_metric': [6.43, 11.81, 18.01, 22.02, 25.36, 27.39, 30.57, 31.8, 33.3, 34.7, 35.97, 36.67, 37.15, 38.07, 38.73, 38.79, 39.68, 40.04, 40.21, 39.74, 41.14, 40.57, 40.04, 41.01, 41.47, 41.42, 41.42, 41.32, 41.72, 41.96, 40.75, 41.59, 41.53, 41.01, 41.08, 40.75, 41.25, 41.49, 40.72, 41.9, 41.51, 40.87, 41.48, 41.57, 41.06, 40.97, 41.06, 41.33, 41.09, 41.75, 41.42, 40.93, 41.14, 41.31, 41.58, 41.57, 40.94, 41.24, 41.8, 41.2, 41.37, 41.39, 41.4, 41.07, 41.59, 41.38, 41.84, 41.46, 41.28, 41.32, 41.37, 41.23, 40.58, 41.12, 42.03, 40.9, 41.22, 41.49, 41.43, 41.06, 41.46, 40.97, 40.87, 41.7, 41.21, 40.91, 41.04, 41.51, 41.14, 40.89, 41.33, 41.23, 41.48, 41.64, 41.47, 40.97, 41.37, 41.22, 41.71, 41.82, 41.26, 40.95, 40.98, 40.95, 41.08, 41.48, 41.18, 41.34, 41.32, 41.35, 41.52, 41.11, 41.49, 41.05, 41.73, 41.99, 41.2, 41.05, 41.83, 41.82, 41.61, 41.32, 40.73, 40.98, 41.85, 41.18, 41.05, 41.56, 41.45, 41.28, 41.43, 41.5, 41.2, 40.91, 41.04, 41.5, 40.0, 40.99, 40.07, 40.83, 41.55, 41.03, 41.58, 40.87, 41.13, 40.96, 41.18, 40.96, 40.7, 41.16, 40.38, 41.93, 41.52, 41.62, 41.75, 41.47, 41.19, 41.11, 40.73, 41.82, 41.91, 41.73, 41.11, 41.02, 41.24, 41.75, 41.17, 41.45, 40.87, 41.27, 41.04, 41.51, 41.3, 40.9, 40.44, 40.77, 41.09, 40.93, 41.06, 40.71, 41.1, 40.58, 40.49, 39.75, 40.97, 40.81, 41.16, 41.05, 41.22, 40.91, 40.82, 41.25, 40.99, 40.67, 41.31, 41.45, 40.85, 40.67, 41.03, 40.85], 'val_loss': [4.638643730977538, 4.119215626625499, 3.725263322234913, 3.4863845102346627, 3.2932973712872546, 3.1828942169809036, 3.049163813044311, 3.0602028984932383, 2.899490664719017, 2.84095781699867, 2.8279076723536107, 2.748232737468306, 2.740182509847507, 2.7175129424234865, 2.705286075355141, 2.7998121156813993, 2.6733076230735535, 2.649212954909938, 2.663115182499977, 2.7132290662473935, 2.617503501807049, 2.71309218588908, 2.8694941853262055, 2.6994260663439515, 2.7794730875902114, 2.7767722105524344, 2.7994090888150938, 2.851842600828523, 3.0229128720654046, 2.777938244449105, 3.2493485576787573, 2.9844956291708975, 2.9556810498996904, 2.9881449763182624, 3.053542965536664, 3.10094524986425, 3.1243337514294183, 2.9846937200825687, 3.4511350264215164, 3.1436933901659243, 3.187221098477673, 3.1652528374058426, 3.0951782859814396, 3.3391414854177244, 3.2571278256215868, 3.390751170124977, 3.8797478486018577, 3.6954870922550276, 3.352362499874868, 3.4338820041364926, 3.334745719554318, 4.591519076353426, 3.6132193841751974, 4.307585697265187, 3.648688171319901, 3.539064682972659, 3.841665706816752, 3.6515226257834463, 3.4536946374139967, 4.07248393213673, 3.5300012781361865, 4.305466408182861, 3.940503861494125, 3.5666077114214563, 3.7596974524722735, 3.575094746935899, 3.4352974147553654, 3.4060310653060863, 3.6431886003275586, 3.289138988704438, 4.175245408799238, 4.026227957503811, 5.592927764175804, 4.447548962702417, 3.5031515337099695, 5.015789604111082, 4.242151130536559, 3.6470267939719423, 5.276638343455685, 4.911139875840229, 4.609040107696679, 7.1630933269573625, 5.457294007015836, 4.459570062388281, 4.8317732082051075, 6.719197803242191, 6.700178017282182, 3.5559380995999477, 5.892485455342919, 5.3479485564930425, 4.787845996534749, 4.199735944438133, 4.341818179294562, 5.473711270435601, 4.719281524609608, 5.310932280151707, 4.388165051390411, 4.769046419745038, 4.1370734895110886, 5.43567532908385, 6.599989094551961, 4.8551069801780065, 6.984173359764609, 6.341195277347686, 5.501972662795121, 4.364640938248604, 5.6341262729304615, 4.170335690686657, 4.827847396492198, 4.684948062061504, 3.9371438861652543, 6.050820489218281, 3.9982160041286687, 5.896700972204755, 8.312308519509187, 3.8734120426663927, 5.336355913976195, 13.501871052061675, 4.6484690266809645, 5.475277374504478, 5.4493637555723735, 6.672297534669281, 5.54962326766579, 7.558174933597541, 4.438758608641898, 8.21946642429206, 4.6663854630889405, 5.181731940075091, 5.152106680687825, 10.297548903021843, 4.024491222041427, 6.60509483297919, 5.135910627188956, 9.301211839268921, 7.850203342498488, 5.962026147326087, 9.316509510301481, 7.44812681720515, 17.34553224873391, 9.000470698259438, 4.587483098552485, 7.746027209956175, 4.669818541806215, 9.139533777145823, 5.811353508074572, 7.402748000090289, 5.770093154755368, 4.538250649051302, 7.232401446931681, 6.918218608874424, 8.981656398742821, 4.645845271979168, 5.487861764658788, 9.142149898656614, 4.765173987977824, 6.331925801410796, 10.655403815257321, 5.231213811096872, 12.260938008120105, 4.955754484340644, 5.856754922563104, 5.4550209971749855, 8.567839540493717, 6.720292401921218, 6.848372227067401, 6.056126167060463, 6.719655489465993, 10.736922167668677, 6.672725085999556, 5.958864878697001, 15.440226655856819, 8.53158056204486, 5.1559590814979215, 7.434098122226205, 10.865720183226713, 5.139762546606125, 7.1504414446035005, 13.416744502486697, 5.7588440658180575, 10.04889974168911, 11.811626952924547, 17.69648994837597, 9.84930297523547, 31.66044716242772, 8.862839757257207, 9.554859861446793, 11.487637312548935, 5.703246383909967, 5.5278292218591, 7.719293093985053, 6.274844827925324, 8.45871327409319, 10.464329856975823, 12.52627933784655, 6.917145868775192, 11.245053157684909, 11.75811170696453, 26.67615116174054, 6.3665547674628575, 16.524299033887825]})
data.update({'train_metric': [0.696, 0.833, 0.961, 1.057, 1.284, 1.432, 1.706, 1.97, 2.311, 2.663, 3.026, 3.279, 3.591, 3.911, 4.317, 4.565, 5.066, 5.535, 6.068, 6.739, 7.399, 8.017, 8.604, 9.286, 9.923, 10.263, 10.628, 11.144, 11.615, 12.094, 12.573, 12.992, 13.346, 13.974, 14.218, 14.583, 15.23, 15.582, 16.12, 16.55, 17.091, 17.385, 17.852, 18.328, 18.782, 19.137, 19.62, 20.16, 20.385, 20.707, 21.246, 21.648, 22.17, 22.385, 22.75, 23.482, 23.729, 24.195, 24.418, 25.029, 25.333, 25.844, 26.143, 26.681, 26.982, 27.345, 27.801, 27.982, 28.766, 29.058, 29.663, 29.918, 30.05, 30.892, 31.002, 31.672, 31.934, 32.592, 33.028, 33.141, 33.944, 34.423, 34.628, 35.216, 35.631, 36.135, 36.642, 36.701, 37.654, 37.902, 38.283, 39.004, 39.489, 39.756, 40.54, 40.704, 41.349, 41.913, 42.241, 42.766, 43.317, 43.798, 44.36, 44.759, 45.291, 45.978, 46.516, 46.973, 47.439, 47.946, 48.464, 48.958, 49.307, 50.226, 50.496, 51.126, 51.564, 52.011, 52.755, 53.081, 53.89, 54.146, 54.703, 55.224, 55.571, 55.968, 56.455, 57.139, 57.404, 58.431, 58.512, 59.135, 59.306, 59.76, 60.535, 60.775, 61.116, 61.853, 62.47, 62.972, 63.073, 63.355, 63.926, 64.279, 64.663, 65.156, 65.908, 65.979, 66.681, 66.756, 67.148, 67.664, 67.738, 68.25, 68.986, 68.842, 69.369, 69.673, 69.974, 70.2, 70.797, 70.887, 71.306, 71.714, 71.867, 72.157, 72.468, 72.569, 73.063, 73.566, 73.74, 74.098, 74.253, 74.511, 74.792, 74.87, 74.796, 75.536, 75.592, 75.991, 76.014, 76.179, 76.446, 76.927, 77.075, 77.397, 77.614, 77.62, 77.981, 78.066, 78.384, 78.442, 78.604, 78.614, 78.827, 78.937, 79.239, 79.507, 79.59, 79.55], 'train_loss': [5.354372072433403, 5.304670630093194, 5.279702244244244, 5.254411926806469, 5.222546708942299, 5.196877941136473, 5.162641475845893, 5.108980168155273, 5.056135589772894, 5.0076519538405915, 4.9617928546434475, 4.912526245080578, 4.870246089122575, 4.833810733665813, 4.787173756863624, 4.743063851037395, 4.68405228887547, 4.62734619257775, 4.572465408786473, 4.510928664387454, 4.45574057323385, 4.400824667929993, 4.351860419649843, 4.301468145214283, 4.258635359853792, 4.2208602971475395, 4.18406262553356, 4.146028733146702, 4.113213348449688, 4.075899270277945, 4.0423488729822274, 4.006297355268678, 3.976241033502824, 3.94446941498984, 3.9159970426895034, 3.8840165867381424, 3.852919961577871, 3.8227783239429303, 3.788563622012782, 3.7614656412975194, 3.7288603357069783, 3.7038349243439854, 3.673611033977184, 3.645482955799603, 3.6172028352111405, 3.590115789564771, 3.565064151547921, 3.5342523537609565, 3.5088479197032925, 3.4853708194908393, 3.455605238123117, 3.429446021181913, 3.4030956309801175, 3.3817313049972935, 3.355555125558064, 3.32716876165423, 3.30599161744194, 3.280298114661902, 3.2603078977617788, 3.231789421592854, 3.2122770608119757, 3.1876187081986793, 3.164054182196609, 3.1429796944958084, 3.124285406129755, 3.098762384684362, 3.0756205798568286, 3.056054733536294, 3.026300854966645, 3.009050107353098, 2.981100342781667, 2.96200343194255, 2.940413599661048, 2.9143268416801935, 2.8975552328869996, 2.8704910188322255, 2.8459679781437224, 2.8210525923025433, 2.79843313802303, 2.779117383487089, 2.7504494664612595, 2.7301936166376466, 2.7099968827236025, 2.680887318313389, 2.6640139213374083, 2.637121870665694, 2.6110272749981966, 2.5954764906748395, 2.567141016858248, 2.541875206730111, 2.5201603867499704, 2.4920374161908816, 2.4728596224208252, 2.448819094945892, 2.4203653670394565, 2.4023315215522634, 2.383225289889047, 2.3507497208437247, 2.334375837409031, 2.3072660677873853, 2.287763836974146, 2.2588439368316933, 2.238229697130463, 2.2143884071995643, 2.1924153342128028, 2.1632620479842446, 2.1407266935322884, 2.1197514006974068, 2.0953560297258833, 2.074171916231923, 2.0531926472371613, 2.0310452377193644, 2.01292323250078, 1.9782480642113713, 1.9604078490842403, 1.9336358257691524, 1.9191785726650967, 1.895843964696922, 1.868806547563349, 1.8540186082714274, 1.8299696308942612, 1.8062902111619692, 1.7887893566624598, 1.7695708203147942, 1.750756764289895, 1.7289261716493642, 1.7109156871406375, 1.682348168514054, 1.6765337415940473, 1.6446279813598077, 1.6336914695232096, 1.6007097351650206, 1.5999266431641288, 1.5819729459034046, 1.5550521192684894, 1.5425858589905772, 1.5226621917250518, 1.505450583732212, 1.4788273952744058, 1.4622931127279748, 1.4495204457547217, 1.4387890256831681, 1.416214676179416, 1.4084237637385602, 1.3888348963728945, 1.3730799257945954, 1.3487756275551028, 1.340552503415887, 1.3199089525299659, 1.3082878383702372, 1.2950306249519075, 1.2772955346854924, 1.2688462649944572, 1.2509765869062526, 1.2337541552361813, 1.2290778266109874, 1.2115407237545923, 1.1954115401531593, 1.1838296894377345, 1.1766762774263677, 1.160004062371916, 1.1529999592711508, 1.138023627925514, 1.1277049869890023, 1.1200448271180297, 1.107584660852558, 1.0962702892410854, 1.0850217520466083, 1.0704156084847771, 1.0579965615684377, 1.0477484021519363, 1.031853055885337, 1.0283952193312056, 1.0198989879299416, 1.0077161958640155, 1.005213246883983, 0.9963193539236878, 0.9797007458109316, 0.9758004399727951, 0.9675091284631692, 0.9602521307103846, 0.9528276913454345, 0.9407319000571185, 0.9311557866904648, 0.9242649205357923, 0.9103436722674586, 0.9058738621625089, 0.9027160597167866, 0.8901400269984588, 0.8872128588910753, 0.8731030990355914, 0.8699480422932753, 0.8667937178529385, 0.8610063819876101, 0.8561678857698093, 0.8494260937299625, 0.8393323003330524, 0.8261813855827121, 0.8302992191058431, 0.8261589509931346], 'val_metric': [0.8, 1.13, 1.27, 1.42, 1.71, 1.83, 2.18, 2.59, 3.12, 3.33, 4.2, 4.39, 4.8, 4.84, 5.62, 6.02, 6.59, 7.14, 8.0, 8.43, 9.04, 9.67, 11.06, 11.28, 11.6, 11.85, 12.83, 12.56, 13.31, 13.84, 14.12, 13.9, 15.06, 15.19, 15.69, 16.43, 16.41, 17.31, 16.85, 17.25, 18.02, 18.41, 18.52, 19.29, 19.28, 20.05, 20.52, 20.31, 21.02, 21.24, 21.71, 22.18, 22.23, 22.43, 22.95, 23.36, 23.63, 23.98, 23.67, 24.08, 24.98, 24.4, 25.4, 24.83, 25.15, 25.22, 25.18, 25.95, 25.95, 26.15, 26.15, 26.74, 27.03, 25.99, 26.95, 26.81, 26.94, 26.99, 27.51, 27.27, 27.26, 27.75, 27.03, 27.75, 27.6, 27.26, 26.98, 27.3, 27.5, 27.61, 27.49, 27.56, 26.54, 27.67, 27.66, 27.79, 27.23, 26.72, 28.26, 27.53, 26.83, 27.24, 27.33, 27.31, 26.6, 27.09, 26.9, 26.94, 27.2, 27.15, 27.09, 26.38, 26.46, 26.72, 26.42, 26.54, 26.19, 26.84, 26.4, 26.83, 26.5, 27.1, 25.9, 26.09, 25.61, 26.11, 25.8, 26.36, 26.67, 26.07, 26.07, 26.17, 26.29, 25.22, 25.96, 25.15, 25.43, 26.5, 25.5, 25.35, 25.79, 25.71, 25.92, 25.9, 25.86, 25.86, 24.71, 24.74, 25.36, 25.57, 25.53, 25.24, 25.44, 25.93, 24.88, 25.51, 24.61, 25.49, 25.11, 25.27, 24.9, 24.75, 25.18, 25.46, 25.21, 25.02, 24.57, 23.95, 25.08, 24.48, 24.66, 24.67, 23.57, 25.2, 25.01, 25.49, 25.14, 24.53, 24.91, 24.91, 24.39, 24.09, 24.11, 24.5, 24.7, 25.5, 24.62, 24.61, 25.08, 23.72, 24.88, 24.48, 24.69, 25.3, 25.09, 24.13, 24.59, 24.31, 24.79, 24.26], 'val_loss': [5.356314470813532, 5.278241585774027, 5.3360051106495465, 5.248352512432511, 5.198759124537182, 5.253492349272321, 5.124294643948792, 5.0552242667811695, 4.992587053092422, 4.942845748488311, 4.889124238566988, 4.8319026801236875, 4.784263493908439, 4.748628115198414, 4.687901439180799, 4.633841964089946, 4.562705858497862, 4.509724746084517, 4.430263012078158, 4.372119599846518, 4.305931253797689, 4.274633339256238, 4.182386425650044, 4.167168190524836, 4.118630354571494, 4.086359002787596, 4.034325426551187, 4.05250157216552, 3.967076134529843, 3.9319972217462626, 3.906571377614501, 3.9305730473463703, 3.8385450308489952, 3.8261354911099574, 3.7845080369597026, 3.7491337402611022, 3.74369576630319, 3.6907217122946574, 3.6844313858420987, 3.6618280213349945, 3.6311162398878936, 3.610380028463473, 3.581847493815574, 3.558628338157751, 3.552356387399564, 3.518902635878059, 3.482663177380896, 3.5118976383452205, 3.4528367876247237, 3.444254599559079, 3.417762585506318, 3.3965942305364427, 3.3876924719780113, 3.358391443635248, 3.3412960942383783, 3.3209396388120713, 3.3158509716106828, 3.323355972387229, 3.3070831465872987, 3.2720858747032797, 3.2449090738964688, 3.2512961580495165, 3.2222203874284294, 3.2478875757023027, 3.2121921625866254, 3.212953187098169, 3.2214396106209726, 3.1811198459309376, 3.192623818755909, 3.1827976620121365, 3.1767665018701248, 3.1510327614037092, 3.145580220374332, 3.178160291568489, 3.142558479005364, 3.1373500003936186, 3.1516133691095245, 3.1703831534476796, 3.123906922947829, 3.1349769663658873, 3.1410269782801343, 3.1345339276988033, 3.1752379745434802, 3.1348800150452147, 3.1422052223970938, 3.157696018553084, 3.1722604157818353, 3.1558634232563576, 3.1686643756878605, 3.1422355979870837, 3.1841196603835766, 3.1556205620431594, 3.215503042670572, 3.2027924341760623, 3.1751538659356964, 3.1677459250589846, 3.1979104409551926, 3.241923981411442, 3.1863270064068447, 3.189865947529009, 3.2162434105660505, 3.2489745138557096, 3.2357777167277733, 3.264395247599122, 3.2872056664934584, 3.269410079451883, 3.276870374466963, 3.2909689686100956, 3.2779909797534823, 3.316628768945196, 3.3020316051070098, 3.345595762987805, 3.334579446513182, 3.3371402511171473, 3.3695784670532127, 3.364922547036675, 3.44088651344275, 3.3647310650272733, 3.4105528699364633, 3.365076297407697, 3.4280711609846466, 3.3981869440929144, 3.4557611684131015, 3.4728766581055464, 3.5566672200609926, 3.4926849140483105, 3.5169547044547502, 3.525258839510049, 3.533325477010885, 3.5539087588620033, 3.544160148900026, 3.5704317533286516, 3.574704062407184, 3.6480948439069616, 3.576380036439106, 3.6579856416981693, 3.657646576310419, 3.6150780976957577, 3.6615323678703064, 3.6804384258901996, 3.660248360056786, 3.7318870816261147, 3.71105390141724, 3.6909960136292086, 3.728237376850881, 3.739429774557709, 3.8195780986433574, 3.815198919575685, 3.7744829745808985, 3.731490145063704, 3.7990226297621517, 3.8307546005127535, 3.8710027105489355, 3.793354156670297, 3.8621737045846927, 3.8300735426556534, 3.914131689223514, 3.8434232556895846, 3.937161082674743, 3.8784188389018843, 3.9561613441272905, 3.965377711946038, 3.9397288264742323, 3.8977360664659244, 3.9593008088458115, 3.984502416507454, 4.037396220644569, 4.015371345410681, 4.0376345260887385, 4.0683408223899304, 3.9927280503473463, 4.053269798588601, 4.1620194061546565, 4.007745742038557, 4.034853232893974, 3.9940566902707335, 4.023757191220666, 4.056029044139158, 4.077163817016942, 4.099693239874141, 4.132057703984011, 4.194569220968113, 4.1880757337922505, 4.148410985424261, 4.1436464763750696, 4.12422319810102, 4.144578529770967, 4.166071279793028, 4.117923304533503, 4.264915079068226, 4.252100878460392, 4.184640514622828, 4.195851154388136, 4.147620974072985, 4.172710556892833, 4.244602169200873, 4.187096730159347, 4.300667466631361, 4.177127428874848, 4.2528994015068005]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="Adam")
data.update({'train_metric': [3.251, 8.587, 11.934, 13.717, 15.565, 17.554, 19.04, 20.504, 21.559, 22.384, 23.059, 23.856, 24.255, 24.895, 25.343, 25.851, 26.181, 26.676, 26.815, 27.058, 27.354, 27.679, 27.977, 28.383, 28.455, 28.44, 28.504, 29.021, 29.063, 29.101, 29.488, 29.425, 29.575, 29.787, 29.713, 29.798, 30.005, 30.215, 30.185, 30.115, 30.33, 30.328, 30.474, 30.501, 30.645, 30.62, 30.681, 30.832, 30.689, 30.943, 30.879, 31.036, 31.043, 31.025, 30.959, 31.215, 31.278, 31.432, 31.355, 31.35, 31.463, 31.477, 31.535, 31.422, 31.543, 31.635, 31.637, 31.656, 31.811, 31.767, 31.862, 31.84, 31.841, 31.907, 31.957, 31.752, 31.794, 31.982, 32.094, 31.946, 32.155, 32.028, 31.972, 32.259, 32.229, 32.282, 32.183, 32.24, 32.24, 32.042, 32.18, 32.294, 32.353, 32.502, 32.339, 32.331, 32.284, 32.437, 32.483, 32.419, 32.435, 32.38, 32.611, 32.596, 32.459, 32.602, 32.452, 32.641, 32.627, 32.494, 32.55, 32.758, 32.713, 32.63, 32.716, 32.739, 32.639, 32.751, 32.644, 32.862, 32.817, 32.637, 32.749, 32.706, 32.816, 32.791, 32.953, 32.825, 33.03, 33.028, 32.854, 32.845, 33.01, 32.907, 32.829, 33.182, 32.898, 32.84, 33.003, 32.914, 32.936, 32.79, 32.965, 33.075, 32.943, 32.954, 33.178, 33.033, 33.213, 32.859, 33.056, 33.004, 33.105, 33.008, 33.187, 33.039, 33.122, 33.164, 33.173, 33.173, 33.128, 33.113, 32.983, 33.085, 33.062, 33.202, 33.26, 33.051, 33.126, 33.067, 33.104, 33.256, 33.189, 33.241, 33.25, 33.077, 33.317, 33.128, 33.377, 33.394, 33.218, 33.277, 33.167, 33.334, 33.328, 33.184, 33.302, 33.339, 33.344, 33.411, 33.311, 33.2, 33.127, 33.178, 33.269, 33.218, 33.33, 33.326, 33.47, 33.371], 'train_loss': [4.894067776485353, 4.277079956240175, 4.021081842524992, 3.889559419660025, 3.7680938591655067, 3.6595090742837293, 3.568336168162272, 3.4872877282052945, 3.428664213254981, 3.381836088666226, 3.343224023140476, 3.3078150894118665, 3.278664128260207, 3.2502419388759463, 3.2239652814883417, 3.2009506728964894, 3.1749019172812454, 3.16363184694594, 3.1422842479408053, 3.127787169019, 3.116440839974909, 3.1008510420281232, 3.087327893849603, 3.0744507244132073, 3.0666427581796873, 3.0598643340137017, 3.046461065381441, 3.0398914682811755, 3.029501059233799, 3.0270815498159243, 3.012848724254186, 3.0146843163691037, 3.005120421859292, 2.997207195729837, 2.995564778035677, 2.985487314194956, 2.978636111041635, 2.9742348116327384, 2.9753366997817046, 2.969698307839099, 2.9632621770933203, 2.9613802475724857, 2.955731274299109, 2.9539409937648076, 2.9468479542418007, 2.952419680200627, 2.9443038512099955, 2.938343052061719, 2.935808769648302, 2.9332623596154797, 2.9289264973355498, 2.929784880642394, 2.9268665309144533, 2.9218444545072235, 2.9230014679909364, 2.9171450414187774, 2.915141948430262, 2.9113360018739316, 2.912155896673123, 2.9056646860492434, 2.9029780907731597, 2.9018723131827797, 2.89889342618614, 2.8994780999303855, 2.8982230991563656, 2.896836090148906, 2.8916663165589744, 2.8918157836526004, 2.888771601388337, 2.888225551911523, 2.8894927869297646, 2.8804439061433933, 2.8838406472502043, 2.8815656501516194, 2.879281992951953, 2.8773463332187803, 2.879286855440146, 2.873660223497768, 2.8768933106139922, 2.8752313110207566, 2.8696445379971087, 2.8724210713814253, 2.874443222602361, 2.8694787719504466, 2.866275460157193, 2.8665888355240483, 2.8633591712321977, 2.8631236914709755, 2.8618711860837345, 2.8601183807399893, 2.86123700334106, 2.85898950247908, 2.8577956706380814, 2.8551737089151, 2.8552332249346715, 2.8550966208513473, 2.8510345910805124, 2.8562528591314646, 2.8497399360952054, 2.849661547178194, 2.850421001113384, 2.8478078000147375, 2.848191816381209, 2.8476844680515225, 2.847827050293857, 2.846186260954356, 2.845741115162484, 2.8432425424523333, 2.8431099245201983, 2.841486331826208, 2.841977197240731, 2.843091844825964, 2.8387768732304, 2.837271295444979, 2.8414063899088426, 2.8403531309129026, 2.8390273619216755, 2.8379087632692404, 2.838337617277413, 2.83552368634493, 2.835837312943647, 2.837639076924827, 2.833559556108061, 2.835751241701044, 2.8322479572344954, 2.831130173750894, 2.8286272548973295, 2.8292450424348097, 2.830938904390683, 2.8285848946427965, 2.829964509165905, 2.83141178300727, 2.826646282973384, 2.8259738155152654, 2.829088960484061, 2.8252238273925685, 2.827830698272012, 2.827757317212935, 2.8236818039028293, 2.826388823551317, 2.8266504334091604, 2.8257894406163073, 2.822440755573207, 2.823189199695355, 2.8218029671118035, 2.8246572292240018, 2.817985372976546, 2.822694725511323, 2.8213500873598623, 2.824690350186573, 2.8197057756642385, 2.8144006552180647, 2.816952718959279, 2.818586465417004, 2.8167580691805574, 2.8182188950512397, 2.8136788059791082, 2.816047726116803, 2.8136175397261547, 2.8155683989106883, 2.814533258658987, 2.8137839499453436, 2.8156078191270297, 2.8135652969223677, 2.8134522685162624, 2.8116727866809184, 2.8184541472241578, 2.8118438724516603, 2.8139128275430134, 2.8120282271391903, 2.816092795007746, 2.8107625303250128, 2.8087962439482745, 2.810741078754457, 2.8134868289901136, 2.813675456831109, 2.8117927971057073, 2.8105578611694844, 2.8073165720270294, 2.8067912893728497, 2.807680483590466, 2.8104786316096133, 2.8120686702825894, 2.8084317159744234, 2.8107999162802084, 2.806336399537207, 2.8103865963941344, 2.8081122567389762, 2.810549227533932, 2.808699637289163, 2.8074090793509554, 2.809863487574357, 2.8090353956301857, 2.8114354979587683, 2.8112759387271953, 2.810174706648804, 2.807970676983463, 2.8055385820238694, 2.804947269168788, 2.806087310086895], 'val_metric': [5.8, 10.2, 12.22, 15.56, 16.65, 18.78, 20.69, 22.41, 21.61, 23.71, 23.4, 25.23, 25.41, 26.27, 25.91, 25.8, 27.88, 26.77, 27.4, 27.33, 28.42, 28.74, 29.27, 28.47, 30.42, 29.64, 29.58, 30.17, 30.31, 30.75, 30.37, 30.09, 30.39, 30.37, 31.09, 31.0, 31.53, 30.5, 31.46, 31.59, 31.27, 30.25, 31.32, 31.62, 32.21, 31.12, 31.75, 31.79, 30.69, 32.7, 31.85, 32.23, 32.58, 32.39, 32.25, 32.42, 32.17, 32.58, 32.11, 32.56, 31.93, 32.17, 32.53, 32.81, 31.82, 32.77, 32.9, 32.13, 33.03, 32.63, 33.11, 33.68, 31.87, 32.54, 32.61, 32.56, 32.06, 33.4, 32.98, 33.95, 32.78, 32.84, 33.53, 33.49, 33.01, 33.47, 32.27, 33.02, 32.69, 31.79, 32.87, 32.92, 32.32, 33.71, 33.28, 33.07, 33.64, 33.7, 34.13, 32.66, 33.08, 32.88, 33.55, 33.7, 33.02, 33.61, 31.37, 32.87, 33.79, 33.74, 33.22, 33.95, 32.56, 33.54, 33.92, 34.06, 33.34, 33.97, 32.51, 34.27, 33.45, 33.7, 33.49, 34.38, 32.61, 33.85, 34.6, 33.22, 33.53, 34.44, 33.7, 34.0, 32.96, 33.64, 33.82, 33.16, 33.72, 34.23, 33.77, 33.98, 33.51, 33.74, 33.45, 33.71, 34.54, 33.32, 33.56, 34.46, 33.9, 33.41, 33.75, 33.01, 34.11, 33.57, 33.77, 33.56, 32.99, 33.6, 33.44, 33.5, 33.53, 34.42, 34.11, 34.29, 34.02, 33.07, 33.62, 33.34, 34.13, 33.46, 33.23, 33.61, 34.26, 33.34, 33.07, 34.16, 34.75, 33.44, 33.14, 33.24, 34.07, 33.83, 32.94, 35.12, 33.94, 34.08, 33.95, 33.72, 33.29, 33.11, 33.95, 34.37, 34.82, 33.25, 34.88, 34.79, 34.53, 33.57, 33.98, 33.89], 'val_loss': [4.50652725073942, 4.143336579298517, 3.951015628826846, 3.780644867830216, 3.6862349540564665, 3.5689986617701828, 3.4529406857338683, 3.3821194703411903, 3.373771773022451, 3.3248097744716962, 3.306677460670471, 3.2279613367311515, 3.2170993317464354, 3.1789165477084507, 3.1882037136964736, 3.1648544262928566, 3.1037808694657247, 3.143251006770286, 3.099969772776221, 3.113080588116008, 3.059787269610508, 3.035463669497496, 3.020618089445078, 3.027156274030163, 2.9842023378724507, 2.99755536067258, 3.012956916146977, 2.9713377246431483, 2.979086905528026, 2.9549666400168353, 2.9650952064307634, 2.971807143490785, 2.9706092184516275, 2.9750354616505326, 2.93110550133286, 2.9223408357352967, 2.907962178728383, 2.947446352357318, 2.9064460681502227, 2.923213870662033, 2.9228814255659747, 2.9447452239929492, 2.91445093777529, 2.9060818192305837, 2.8872929834256507, 2.9053771207287054, 2.8899395625302744, 2.8852911405502613, 2.9537867019130926, 2.850797953119703, 2.876697853871971, 2.8857085393492583, 2.8581614889157048, 2.8630690529088305, 2.8784751231503334, 2.8897121719493986, 2.8637227009815773, 2.8506083928855364, 2.880824373026562, 2.848990999968948, 2.866002694816346, 2.870865370817245, 2.862247792019206, 2.8481980311642787, 2.8923127119708214, 2.8450064537631476, 2.8240817549881663, 2.8774277761483646, 2.850443485436166, 2.8443922859847923, 2.843520297366343, 2.825249781654139, 2.8534338595760858, 2.853467311828759, 2.8334115143794163, 2.8630803291964684, 2.8706989812243515, 2.812478657740696, 2.8292994726994993, 2.7935499203432896, 2.842200944378118, 2.833533315142249, 2.806351146120934, 2.8264980794517856, 2.8263373876073556, 2.8014033144446695, 2.8504038912475487, 2.8261446846518545, 2.8648411726496024, 2.8434558347531946, 2.8249205798859807, 2.8465683407084956, 2.8554290905120268, 2.8196623241825467, 2.827948223253724, 2.8204169053180963, 2.8031315082197734, 2.7913596155537164, 2.8008853910834928, 2.833213837283432, 2.810306186129333, 2.845407314361281, 2.814049576498141, 2.8136303424835205, 2.82095805323048, 2.8019989615033385, 2.8804976670605362, 2.81718076034716, 2.793914115352995, 2.7990506986144243, 2.8268823259195703, 2.804121026567593, 2.825507931648546, 2.8051439994459697, 2.798288813062534, 2.7830455644874816, 2.795676068895182, 2.7869367379291803, 2.8271764797769534, 2.7724384031478007, 2.792730890261899, 2.783179661270919, 2.7669444000645047, 2.7748841510456836, 2.8444917832210566, 2.777441162212639, 2.76383411276872, 2.817407028690265, 2.8240990046482937, 2.785622599777902, 2.792993327614608, 2.7848578828155617, 2.8049362449888973, 2.8360127780088193, 2.783095110753539, 2.781350087967648, 2.7895508063067296, 2.773590002849603, 2.7810933362146852, 2.7613710066315473, 2.7952376475000076, 2.779323617364191, 2.785927687480951, 2.798977298341739, 2.7513663852290744, 2.797927006794389, 2.8118116073547657, 2.776180690261209, 2.7668240829637853, 2.773463000917131, 2.7723062448440845, 2.8188242380786095, 2.7737406788358263, 2.7713065033505675, 2.8030701936430233, 2.80394999616465, 2.8262457741293936, 2.778609978165596, 2.7867729489211066, 2.81691509599139, 2.7921255583975726, 2.769024882346961, 2.791192190662311, 2.761268246705365, 2.761884614160866, 2.7919068101105418, 2.79581014062189, 2.8105786833793496, 2.7790258993768386, 2.8081544630087105, 2.8015476875244434, 2.7811369311278034, 2.769678477648717, 2.8105940013934094, 2.789326198541435, 2.787029821401948, 2.7485093827460223, 2.806723797397249, 2.8099864341650798, 2.822746112088489, 2.781600804845239, 2.7756028456293094, 2.824693386721763, 2.733971901000685, 2.789351596194468, 2.782969931887973, 2.7762831639332375, 2.80290666051731, 2.8076277811815786, 2.8024129472720394, 2.7685360111248722, 2.7809033720356644, 2.7539488654227773, 2.8129508297914154, 2.7480838731595667, 2.7598993854158245, 2.7678050539296146, 2.8055888983854063, 2.7944601181965725, 2.7936565382465437]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="AdamW")
data.update({'train_metric': [5.448, 16.497, 26.372, 34.125, 40.698, 46.57, 52.088, 57.365, 62.492, 67.785, 73.001, 77.48, 81.244, 83.942, 86.305, 87.729, 89.117, 90.203, 90.903, 91.489, 91.908, 92.517, 92.922, 93.223, 93.685, 94.018, 94.319, 94.524, 94.687, 94.846, 95.04, 95.366, 95.412, 95.563, 95.672, 95.844, 95.722, 96.136, 96.098, 96.278, 96.33, 96.412, 96.602, 96.552, 96.672, 96.673, 96.993, 96.806, 96.858, 96.972, 97.053, 97.026, 97.087, 97.254, 97.218, 97.297, 97.243, 97.362, 97.416, 97.406, 97.571, 97.526, 97.667, 97.589, 97.523, 97.698, 97.764, 97.639, 97.803, 97.812, 97.788, 97.729, 98.042, 97.908, 97.763, 97.9, 97.992, 97.955, 98.021, 98.018, 98.012, 98.021, 97.999, 98.172, 98.062, 98.142, 98.23, 98.119, 98.127, 98.164, 98.183, 98.319, 98.226, 98.125, 98.367, 98.266, 98.316, 98.321, 98.253, 98.394, 98.298, 98.322, 98.417, 98.351, 98.362, 98.413, 98.502, 98.361, 98.442, 98.448, 98.521, 98.51, 98.494, 98.51, 98.511, 98.456, 98.645, 98.541, 98.467, 98.559, 98.604, 98.536, 98.529, 98.519, 98.674, 98.551, 98.599, 98.558, 98.684, 98.515, 98.674, 98.575, 98.64, 98.629, 98.611, 98.66, 98.646, 98.684, 98.718, 98.66, 98.616, 98.651, 98.755, 98.759, 98.609, 98.777, 98.698, 98.771, 98.773, 98.666, 98.732, 98.749, 98.792, 98.754, 98.734, 98.853, 98.665, 98.753, 98.861, 98.817, 98.785, 98.864, 98.865, 98.766, 98.841, 98.834, 98.765, 98.861, 98.895, 98.909, 98.843, 98.834, 98.891, 98.953, 98.814, 98.746, 98.958, 98.883, 98.887, 98.911, 98.891, 98.814, 98.932, 98.916, 98.986, 98.915, 98.896, 98.939, 98.876, 98.954, 98.984, 98.873, 98.936, 98.91, 98.963, 98.906, 98.966, 98.927, 98.954, 98.983], 'train_loss': [4.6672568254110836, 3.7401310119275015, 3.172986289437429, 2.7711410987705127, 2.4377745315194206, 2.151731183722625, 1.8843707410631771, 1.6396875257531727, 1.4033198406203617, 1.1722647130527484, 0.9552658906901257, 0.7736321356116543, 0.6269948650401751, 0.5315708564457341, 0.4460228492892559, 0.3939132609583976, 0.3518881997330709, 0.31504619959563074, 0.2923271679846774, 0.27338562179559406, 0.25931004251538753, 0.23638971909041673, 0.22516290245962617, 0.21613248317637698, 0.20139196091906542, 0.19101497455382682, 0.18113655946262167, 0.174310360114571, 0.16906047215112036, 0.1634516362124674, 0.1585536618596056, 0.14836586241849287, 0.1489331028057752, 0.1412435713059061, 0.13830285363187697, 0.13285351864056888, 0.13530703123434853, 0.12265096810870993, 0.12375617269200159, 0.1176209034820064, 0.11675798469351191, 0.11424327353301553, 0.10815952006329425, 0.11097332057546078, 0.10697415865116389, 0.10517175568795631, 0.09765476223736196, 0.09982988712814608, 0.1018981771696757, 0.09504763201631067, 0.093827148246676, 0.09268618760343286, 0.09329752467354606, 0.08806048632853844, 0.08868233374005954, 0.08500309442455975, 0.08654776698256046, 0.082643724693449, 0.08185867065747321, 0.0819432695522268, 0.0770868708675111, 0.07946346879305743, 0.07549021610160736, 0.07666620505143669, 0.07835320933940638, 0.07382767984342076, 0.07249498534834421, 0.07488346765945789, 0.06983890854759513, 0.07077379802406378, 0.06936362099865664, 0.0714584295818152, 0.06366396574967886, 0.06649215670008456, 0.07007258426727547, 0.06635945512700447, 0.06417672242269144, 0.06443030046601794, 0.06316062416440389, 0.06373979140910835, 0.0635016631199165, 0.06205473186143436, 0.06426428361680843, 0.058050825576234186, 0.0617948691312984, 0.05928524584464445, 0.05711920496833976, 0.059928958093740346, 0.059052990522866944, 0.05828387722674214, 0.057314507727900324, 0.05435223823497566, 0.056566021570957296, 0.06027118986015466, 0.051596030469203844, 0.05563623887178103, 0.05297945570859102, 0.05289581089758124, 0.05468256054861711, 0.05227842583736347, 0.05285785450023299, 0.0514217111805212, 0.051941632811335806, 0.05103770928314076, 0.05278673653959482, 0.05197065370812803, 0.04923367663752139, 0.051767972848164805, 0.04904304342064351, 0.048959652694452835, 0.04746754831205148, 0.04800088498765968, 0.04832363835775329, 0.04771128542247864, 0.04734075121356321, 0.04971018523163363, 0.04433326346019756, 0.04658902090596827, 0.04930613370964117, 0.04514199903156484, 0.04446699832465085, 0.045734887646159045, 0.04649164158775235, 0.046902853003647944, 0.042726040695572266, 0.04631844269774195, 0.045231481186658415, 0.045881378236150766, 0.04178279433383053, 0.048115332609303806, 0.04194712961507158, 0.04477502329150478, 0.04214698535414629, 0.04406666020907894, 0.04312682239701781, 0.04196699663407693, 0.0428899681254532, 0.042022007048764326, 0.04205088439276555, 0.0427920782646593, 0.04295379300358224, 0.042323783366575655, 0.04026408518693736, 0.04016782668629892, 0.04207326890150325, 0.039389518226155155, 0.04163227767824486, 0.039999777512512096, 0.039047036989526766, 0.04242461667465695, 0.04047024540916134, 0.039402675894026476, 0.038128227953604224, 0.04017161482925198, 0.03998903988558747, 0.035530066868832234, 0.041667067846769684, 0.03889076792915374, 0.0362712360908802, 0.038093066547660044, 0.03907574462698849, 0.03616247126607743, 0.03597889364610938, 0.03854271572502047, 0.03691097867786625, 0.03662399065489979, 0.038752529940364426, 0.036786725880292595, 0.035910948168080284, 0.03609582287236981, 0.037104015114123715, 0.037251658296446874, 0.03655017969590002, 0.034353094817231866, 0.0367099550176886, 0.039702201707146216, 0.03402114512939355, 0.035479547586473766, 0.03494482092624211, 0.03468448358290514, 0.03507554338077382, 0.037756346443579225, 0.03415948296813694, 0.03512014314847041, 0.03406314554502333, 0.03457903122296706, 0.03587145094230531, 0.03372882563082376, 0.036207839812665864, 0.032599879837445514, 0.03233702191512469, 0.035773531392243346, 0.03389295496498127, 0.03509856480392622, 0.03310572624798526, 0.03500546821337881, 0.03403093914447891, 0.034285272377014626, 0.0335821233142268, 0.032501609105883005], 'val_metric': [10.68, 22.98, 31.74, 37.23, 41.63, 45.39, 47.08, 48.51, 48.8, 48.61, 48.94, 49.54, 49.91, 49.15, 48.94, 48.94, 48.95, 48.77, 49.6, 49.13, 49.51, 49.27, 48.79, 49.5, 49.09, 49.79, 49.57, 49.23, 48.83, 49.38, 49.67, 49.39, 49.29, 49.79, 49.41, 50.72, 49.75, 50.44, 49.82, 49.7, 49.62, 50.12, 50.03, 49.91, 49.63, 50.1, 49.8, 49.99, 50.02, 50.28, 49.84, 50.65, 50.76, 50.25, 50.78, 49.73, 50.53, 50.79, 50.77, 49.94, 50.65, 50.63, 50.59, 50.46, 50.42, 50.51, 51.16, 50.19, 50.2, 50.52, 50.97, 50.52, 50.67, 50.32, 51.0, 50.77, 50.5, 50.78, 50.41, 50.82, 50.84, 51.04, 50.66, 51.31, 50.99, 51.39, 51.02, 50.96, 51.1, 50.87, 50.56, 50.6, 50.89, 50.54, 50.95, 50.62, 50.76, 50.64, 50.94, 51.54, 50.94, 51.64, 50.95, 50.51, 51.23, 51.54, 51.12, 51.17, 51.03, 51.07, 51.21, 50.5, 50.85, 51.91, 51.16, 51.45, 51.0, 50.97, 51.1, 50.74, 51.13, 50.69, 51.61, 50.74, 51.22, 51.53, 51.42, 51.45, 51.3, 51.18, 51.1, 51.11, 51.4, 51.24, 50.91, 51.14, 51.65, 51.2, 51.0, 51.27, 51.61, 51.62, 51.32, 51.32, 51.58, 51.43, 51.63, 51.47, 51.52, 51.03, 51.28, 51.03, 51.2, 51.51, 51.17, 51.33, 51.0, 51.15, 51.63, 50.98, 51.23, 51.1, 51.49, 51.38, 51.38, 51.17, 51.07, 51.5, 51.62, 52.12, 51.48, 51.58, 51.19, 51.7, 51.62, 51.58, 51.29, 51.96, 51.18, 51.17, 51.78, 51.47, 51.93, 51.62, 51.3, 52.19, 51.4, 51.71, 51.77, 51.66, 51.66, 51.57, 52.13, 52.61, 52.33, 52.15, 51.97, 52.47, 52.11, 52.06], 'val_loss': [4.069531379991276, 3.346311159194655, 2.8947961064660626, 2.6307421789807117, 2.419346366338669, 2.2514770141073095, 2.1980638997569963, 2.1864799428138, 2.168964302463896, 2.23680154722967, 2.331057284667993, 2.433015390186553, 2.519634153812554, 2.644437998343425, 2.7445260020577984, 2.8552610528696873, 2.8812642031034845, 2.953357679069422, 2.9905118099443473, 3.086015610748036, 3.0717200458429423, 3.141433667984738, 3.1378859308115237, 3.138219285922445, 3.2423613899073023, 3.188860976771944, 3.2748467052818104, 3.279720589993106, 3.3514173262438196, 3.32559024870016, 3.3075189077930087, 3.3898907254455954, 3.3461082797900885, 3.39983052394952, 3.3982141860731088, 3.39729903799713, 3.3910743148083897, 3.4555076588490965, 3.4346063592631344, 3.4070239818779524, 3.426894774102861, 3.450039943692031, 3.5347098947330644, 3.5409914801834494, 3.521962477902698, 3.5438271525559153, 3.5386766681245936, 3.6048420158920775, 3.5562302914394697, 3.565844651240452, 3.5602793048141868, 3.5681025670592192, 3.545550678945651, 3.58376058727313, 3.563538772665012, 3.6820679234850937, 3.618628262714216, 3.56222182853966, 3.6317123216428575, 3.5994816767941615, 3.5977145824462746, 3.6180460380900437, 3.6115234118358344, 3.611323833465576, 3.6169468965500022, 3.680638785574846, 3.5702590528567124, 3.639124265901602, 3.660586094021038, 3.6814767498119623, 3.6153085239373954, 3.703241182740327, 3.728852518804514, 3.711060052464722, 3.7318487326810312, 3.7290933591545006, 3.7219547655931704, 3.6880071375780044, 3.8378054313598926, 3.6820934840068698, 3.715068000137426, 3.7051137685775757, 3.6858774165439, 3.7091783759700263, 3.7142664167531736, 3.724836686614213, 3.7422344927575177, 3.7427475634653855, 3.717323276647337, 3.7697116349153457, 3.731458265690287, 3.7614191425074437, 3.77274990689223, 3.7994326717534643, 3.821837708069261, 3.8188239146190086, 3.8068491114172964, 3.841596595800606, 3.7809254713119214, 3.7541372449534713, 3.8285074856630557, 3.7918390688622834, 3.8045356638112646, 3.8360133182470966, 3.8165873224568214, 3.748176558002545, 3.831655486753792, 3.8662602590147856, 3.8495230887346206, 3.8151933611578244, 3.8416349891644375, 3.9092641134930264, 3.893214624398833, 3.831317520824967, 3.9033356005219138, 3.835215841888622, 3.841256174691923, 3.8977037056995805, 3.7934456144928173, 3.9515623412314493, 3.8572953058655854, 3.9084972177341486, 3.8417147743474147, 3.8766011274923944, 3.901522450386339, 3.924666097589359, 3.8830283124735403, 3.856749736199713, 3.94855489130992, 3.8671508883215058, 3.8760117793538766, 3.9338051524891218, 3.8696374278159658, 3.842602547566602, 3.944161752227006, 3.939151794287809, 3.8748648231196556, 3.919066697359085, 3.878081435610534, 3.8454901302696034, 3.8754054326919993, 3.8495366444253616, 3.8824096481511545, 3.9276688038163883, 3.924081912086268, 3.941717360429703, 3.9102706142291903, 3.914015982940698, 3.995644985490544, 3.9500683994050236, 3.938543448782271, 3.9249587613306227, 3.9498865376612184, 3.893878207464886, 3.940679896029697, 3.9365706663982123, 4.007076089169569, 3.946728995271549, 3.9338410037338356, 4.0490625428546005, 3.9401603299341383, 3.995539080565143, 3.961943700055408, 4.015648698730833, 3.9574297378017644, 3.942315068973857, 3.9276905614099684, 3.936208073500615, 3.921993781806557, 3.973779986618431, 3.937659850925397, 3.9064356160771316, 3.9324996813087707, 3.919390517435256, 3.9254633624842215, 3.954109923854755, 3.976605695144386, 4.002089225182868, 4.011496204479485, 4.030596725500313, 3.998385711840004, 3.9733334195082355, 3.9747282813309104, 3.9613668698414117, 3.9606402064584625, 3.997937169803935, 4.021209634793032, 4.0520370359633375, 4.053515373901197, 3.9754606952333145, 4.033423545633911, 3.9162448075166933, 3.93840426168624, 3.8717840575868157, 3.9202604927834432, 3.961947452490497, 3.9116363084999617, 3.892735067066873, 3.9953881471779695, 3.9638285135767264]})

# BERT-mini WikiText
# data = database.get_data(model="BERT B16", dataset="WikiText", initialization="Sinusoidal", optimizer="SGD")
# data.update({'train_loss': [10.3223, 10.2807, 10.2391, 10.2601, 10.1847, 10.1134, 10.0433, 9.9702, 9.8982, 9.8279, 9.7547, 9.6803, 9.6074, 9.532, 9.4628, 9.3924, 9.3188, 9.2501, 9.1842, 9.1217, 9.0642, 9.0041, 8.9618, 8.9108, 8.8647, 8.8202, 8.7823, 8.7436, 8.7073, 8.6636, 8.6404, 8.6168, 8.5812, 8.558, 8.5211, 8.4966, 8.4749, 8.4469, 8.4238, 8.4117, 8.3862, 8.3826, 8.3624, 8.3484, 8.3338, 8.3192, 8.3025, 8.2914, 8.2706, 8.264, 8.2515, 8.2463, 8.2252, 8.2181, 8.2066, 8.1915, 8.1845, 8.1769, 8.1714, 8.1474, 8.143, 8.1347, 8.1334, 8.1193, 8.1258, 8.1034, 8.0975, 8.0846, 8.0783, 8.0647, 8.0676, 8.0694, 8.0536, 8.0452, 8.0412, 8.0444, 8.0207, 8.0235, 8.0233, 8.017, 8.0134, 7.9971, 8.002, 7.9936, 7.9926, 7.9918, 7.9797, 7.959, 7.9723, 7.9629, 7.9581, 7.9626, 7.9514, 7.9627, 7.9462, 7.9403, 7.9407, 7.9308, 7.937, 7.9279, 7.9313, 7.9258, 7.924, 7.912, 7.912, 7.909, 7.9063, 7.9015, 7.9086, 7.902, 7.8944, 7.8959, 7.8852, 7.8922, 7.8855, 7.8818, 7.8728, 7.8725, 7.8724, 7.8586, 7.8734, 7.8698, 7.8607, 7.8559, 7.8554, 7.8589, 7.8626, 7.8607, 7.8452, 7.8423, 7.8431, 7.85, 7.8305, 7.8388, 7.8329, 7.8256, 7.83, 7.8306, 7.8196, 7.8217, 7.8257, 7.8128, 7.8199, 7.8054, 7.8161, 7.8067, 7.802, 7.7988, 7.8084, 7.8133, 7.8024, 7.8049, 7.795, 7.7932, 7.7927, 7.7817, 7.7913, 7.7773, 7.7819, 7.7795, 7.7837, 7.7789, 7.783, 7.7696, 7.7759, 7.7737, 7.7657, 7.764, 7.7638, 7.7572, 7.7584, 7.76, 7.7616, 7.758, 7.7492, 7.7474, 7.7455, 7.7547, 7.7478, 7.7432, 7.7442, 7.7358, 7.7511, 7.7348, 7.7312, 7.745, 7.7359, 7.7255, 7.7255, 7.727, 7.7258, 7.719, 7.7227, 7.7274, 7.7226, 7.7241, 7.7172, 7.7124, 7.7176, 7.7198], 'val_metric': [0.0003942218342578774, 0.04999861346052522, 0.06344155661307127, 0.06408585452427831, 0.06229196386115074, 0.060897885841030966, 0.06129256428075052, 0.0594847643876321, 0.06080035996512838, 0.061138816104584, 0.060474308300395255, 0.060850955987948076, 0.06128734216199569, 0.06140597737378286, 0.06278203971119134, 0.06311894959385779, 0.06436710809978442, 0.06272099680080821, 0.06131084148289293, 0.06333267887695285, 0.06213872832369942, 0.060445090583762025, 0.0603052798398532, 0.06191654482060511, 0.061891612975204305, 0.06122506322000562, 0.06032774071075626, 0.06247908066495593, 0.06122966049208941, 0.06147849088124739, 0.06174597855227882, 0.05881527231892195, 0.06172565758474459, 0.062034739454094295, 0.06302755883503829, 0.06127581100338093, 0.06183469876832191, 0.06174246636272657, 0.06278353402408289, 0.06185652837519079, 0.06030504277352544, 0.06240953123260216, 0.05988896988728761, 0.06365439964232822, 0.06164171978994074, 0.06126110970925505, 0.06355519706283203, 0.06256839033697147, 0.06137103107820038, 0.061733269409740825, 0.06077845997859517, 0.06516206616886204, 0.06332327306574902, 0.06307342667152999, 0.06112496847061461, 0.06167908873576149, 0.06238577836849975, 0.06065750014145872, 0.06297613001523616, 0.06268323328214435, 0.06236486676213742, 0.05959229265568277, 0.0618154663126452, 0.06183593641634533, 0.061906901005109774, 0.06287992861524734, 0.06214055267218404, 0.06281231679740934, 0.06265349408350078, 0.06325064541474913, 0.061105640737093826, 0.061584152867669724, 0.06075542911866603, 0.06049692908989392, 0.061165994156115205, 0.061558173867023204, 0.0638499748841882, 0.06232905097389142, 0.06098551699022327, 0.062269129287598944, 0.06199089938767485, 0.06145392357509322, 0.06195568294565097, 0.0636919452546556, 0.06092530657748049, 0.06181725802354137, 0.05896716250350828, 0.06267175144411419, 0.06209260507979321, 0.06320789185239313, 0.06288004035082803, 0.06244394618834081, 0.06203501706102814, 0.06303385673610916, 0.06137294796399008, 0.062106940212985996, 0.06249478050274197, 0.060755659027797254, 0.06323118991909339, 0.06207838546022189, 0.0634080414461088, 0.061295825060820444, 0.06368142485087792, 0.06354997454607161, 0.06208637128435222, 0.060711382683521595, 0.06224371482706937, 0.06088679297932305, 0.06118905598028508, 0.061024004440127656, 0.06077875410572415, 0.06276325699461631, 0.05974114056406488, 0.0607364362742359, 0.06345049003041568, 0.061905963495668265, 0.06146355036171397, 0.06356241234221599, 0.062067419508504494, 0.060601815532892525, 0.06147369594914829, 0.06253672458658609, 0.06259293549925651, 0.06222422079199573, 0.06161177295024527, 0.06251933515200946, 0.061908107348601936, 0.06308683815767542, 0.06042780748663101, 0.06279452360371414, 0.06194591580708113, 0.06090367814812754, 0.06115451632848069, 0.060925666199158485, 0.06003208465846725, 0.060151422368201306, 0.06120960502692998, 0.06177873945849978, 0.06073067024728656, 0.061759044882702445, 0.06227658623771224, 0.06069339899823712, 0.061926025781447465, 0.06040155694323878, 0.06275473172910502, 0.05889856358493705, 0.06209850107066381, 0.061317586477107354, 0.0638922272841611, 0.06034721837040324, 0.0635075584314163, 0.06151693254967814, 0.05900516505726477, 0.06215195186791661, 0.06094575799721836, 0.06059336238685887, 0.06071009881091945, 0.06112978965795104, 0.06307288173485356, 0.06144061860869078, 0.06101157355886935, 0.061573609718043015, 0.06136630016185746, 0.06519848506101837, 0.06159795981279601, 0.06118493131443661, 0.058971071846608325, 0.06118503992155764, 0.06357523115718688, 0.06308548418668382, 0.06181058495821727, 0.06372137297721037, 0.06205951448707909, 0.06290551641558297, 0.06249824285192162, 0.062292405562123516, 0.06132500281436452, 0.0606665729152018, 0.061788754952372925, 0.06166592526690391, 0.06158365976496922, 0.0625034957212372, 0.06254017046248428, 0.06149304091100801, 0.06045107738744026, 0.06275007044237814, 0.06039180616493866, 0.061307748579145796, 0.06244374437443744, 0.06124859392575928, 0.06218954340078017, 0.06158119178543373, 0.06181544550040411, 0.06163199286072843, 0.06290083176968102, 0.06199053194767361, 0.060460274589241504, 0.06430742630859757, 0.06111793611793612, 0.06079307456017872]})
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="Sinusoidal 1", optimizer="AdamW")
data.update({'train_loss': [7.6534, 7.1816, 7.1289, 7.059, 6.9514, 6.8292, 6.6976, 6.5559, 6.4342, 6.3145, 6.1848, 6.0676, 5.9148, 5.7595, 5.6326, 5.5254, 5.4269, 5.3421, 5.2554, 5.1881, 5.1149, 5.0535, 4.9908, 4.9309, 4.8779, 4.8146, 4.7706, 4.7277, 4.6859, 4.6424, 4.6091, 4.561, 4.5252, 4.4883, 4.4467, 4.4144, 4.3798, 4.3433, 4.3109, 4.2835, 4.2559, 4.217, 4.1879, 4.1592, 4.1348, 4.0967, 4.0765, 4.0475, 4.0287, 4.0002, 3.9705, 3.9467, 3.9277, 3.8978, 3.8799, 3.8655, 3.8319, 3.8249, 3.7898, 3.7732, 3.7494, 3.7388, 3.7155, 3.7024, 3.6761, 3.6607, 3.6337, 3.6253, 3.6196, 3.6014, 3.5687, 3.5674, 3.5546, 3.5322, 3.5179, 3.5139, 3.4875, 3.4798, 3.4606, 3.4584, 3.4471, 3.4252, 3.4121, 3.3921, 3.3806, 3.366, 3.3543, 3.3557, 3.3367, 3.3181, 3.3168, 3.3013, 3.2944, 3.2835, 3.2634, 3.2528, 3.2573, 3.2473, 3.234, 3.2198, 3.2071, 3.2021, 3.1948, 3.1856, 3.1817, 3.1706, 3.1642, 3.1549, 3.1426, 3.1388, 3.1308, 3.1167, 3.1105, 3.1009, 3.0796, 3.0873, 3.068, 3.0701, 3.0575, 3.0432, 3.0497, 3.0339, 3.0288, 3.029, 3.0275, 3.0144, 2.9974, 3.0015, 3.0021, 2.9783, 2.9747, 2.962, 2.9537, 2.9518, 2.9493, 2.9484, 2.9412, 2.9297, 2.9181, 2.9174, 2.9042, 2.9013, 2.8945, 2.8965, 2.8866, 2.8843, 2.8859, 2.8694, 2.8693, 2.8606, 2.8514, 2.8465, 2.8393, 2.8368, 2.8269, 2.8285, 2.8246, 2.8276, 2.8092, 2.81, 2.7875, 2.8017, 2.7902, 2.7903, 2.7884, 2.7706, 2.7724, 2.7632, 2.7663, 2.7531, 2.7553, 2.7473, 2.739, 2.741, 2.737, 2.7288, 2.7191, 2.7236, 2.7181, 2.7098, 2.702, 2.6956, 2.7022, 2.694, 2.6865, 2.6898, 2.6848, 2.6803, 2.6705, 2.6731, 2.6831, 2.6672, 2.6651, 2.6563, 2.6481, 2.6478, 2.6512, 2.6428, 2.6361, 2.6304], 'val_metric': [0.06344155661307127, 0.06691439505282716, 0.07272523845486839, 0.0780806918044754, 0.10104239054899236, 0.11156897373363608, 0.12910936753002053, 0.15166370833685516, 0.1686335403726708, 0.1813138850561766, 0.1944172242909539, 0.20771655316035345, 0.22145758122743683, 0.23492266607321688, 0.24288154100288378, 0.2502104731436269, 0.2508196261944125, 0.26354023504333435, 0.2694253001333926, 0.2732050995303064, 0.278004837767954, 0.28244854347092563, 0.2839371879619558, 0.2898286035403203, 0.2844141704586695, 0.30143367176168695, 0.29713980068689827, 0.30277043018237504, 0.30674709562109026, 0.30828186412128017, 0.3093766456941711, 0.31240415981264114, 0.3141874895186986, 0.31682919332755877, 0.31909379702390905, 0.3184977204492383, 0.32497899747969755, 0.33173303732482307, 0.3310164340387213, 0.3342055450395279, 0.33194078393988674, 0.3352334646659402, 0.33668454604173104, 0.3393893515013878, 0.3457569604761772, 0.3492326253472125, 0.34797486817008866, 0.3506021038382357, 0.35684109727933305, 0.3580557653046956, 0.35523086903788115, 0.3581549339244129, 0.3576693478321796, 0.36231191112361133, 0.3658691920031013, 0.36674588355117976, 0.36626036905366516, 0.36917492670668717, 0.36801168112767807, 0.36729963697291257, 0.37340966921119595, 0.3700446639110051, 0.3724802066370936, 0.37591322291004403, 0.37727591527563475, 0.38055889003657073, 0.3807211431123019, 0.37683802895947915, 0.3799971866647911, 0.3819170098904486, 0.3832633861927161, 0.38204913456169737, 0.38399888687908723, 0.38352692125567533, 0.3835184461684434, 0.3866279872910623, 0.3865310810432249, 0.3907235106235245, 0.3885455873265547, 0.3858252263870588, 0.3867544923382207, 0.3947161768005385, 0.3912486064659978, 0.396138538402913, 0.39469548133595284, 0.39711737984409173, 0.39916273319847156, 0.3943958854444788, 0.3977358702048365, 0.39358183856502243, 0.3988644627174582, 0.39776731598680687, 0.39950946235959756, 0.40123543058389466, 0.39974389666787297, 0.40192420969958764, 0.39988160009020945, 0.4018230473323298, 0.4030296204527537, 0.40278515701462486, 0.40507995183287127, 0.4014367328468805, 0.4083567021873247, 0.3987162597752053, 0.39981900964339245, 0.40720484415548175, 0.40384776946988155, 0.4036631053142778, 0.40838269560090956, 0.40561800887054034, 0.40371898490928615, 0.40431357210705704, 0.40278247155570573, 0.4064261081672134, 0.40717863105175295, 0.4072931276297335, 0.40773951298792277, 0.41129664910904407, 0.4100476734784912, 0.4115951761381124, 0.41388210868894315, 0.4093476855625193, 0.408801681850035, 0.41046769974969766, 0.4118870310524097, 0.4144119117728763, 0.41317196735153394, 0.4151632601845914, 0.4118762196821857, 0.41792332090590395, 0.4155752908774099, 0.4142215988779804, 0.41324477217078043, 0.4103434838278684, 0.4167975763016158, 0.4150299600532623, 0.4158274588788184, 0.41108538592429356, 0.41071827524575516, 0.416822900635196, 0.41533968039992136, 0.41914256111562265, 0.41801127798559545, 0.4154234374133437, 0.4158120139749803, 0.4161815739393261, 0.41840185583722295, 0.41921348937822395, 0.4224354325877169, 0.42244612370556955, 0.4171064450931956, 0.4227228207639569, 0.41827538247566065, 0.4207453346742653, 0.4172109641042818, 0.4185445552807802, 0.4167505030181087, 0.4216513041772897, 0.4222957934564879, 0.4219324640588432, 0.41608528213428586, 0.4228924112778791, 0.4210688563181347, 0.41933308986712364, 0.4219385764450142, 0.420675164588878, 0.42146259456430374, 0.41911579653812814, 0.4235097493036212, 0.4207962371913321, 0.42566282581944287, 0.41856255743365545, 0.4250611487531277, 0.42304227889433094, 0.41922773837667454, 0.41788777949655465, 0.4233049537778527, 0.4198732206405694, 0.4235310576385003, 0.42412886626768836, 0.4260723766941456, 0.42322508083790245, 0.42142477851373633, 0.42510566356720203, 0.42039515971271274, 0.4225423442687525, 0.4242799279927993, 0.4202755905511811, 0.42042488704290965, 0.4265514532600157, 0.4216158969928374, 0.4232528306096269, 0.42075783459825805, 0.42390543152469257, 0.4218996173756471, 0.425137343484202, 0.4227161045342864, 0.42189332588662387, 0.42490699040532603, 0.42246318084006373]})
# data = database.get_data(model="BERT B16", dataset="WikiText", initialization="Default initialization", optimizer="SGD")
# data.update({'train_loss': [10.3412, 10.2544, 10.172, 10.0945, 10.0197, 9.9505, 9.8827, 9.8162, 9.7489, 9.6821, 9.6156, 9.5519, 9.4872, 9.4204, 9.3544, 9.2935, 9.2332, 9.1767, 9.1175, 9.0754, 9.025, 8.9802, 8.9372, 8.8983, 8.8593, 8.824, 8.78, 8.7548, 8.7315, 8.6939, 8.6692, 8.6294, 8.6018, 8.5777, 8.5476, 8.5213, 8.505, 8.4774, 8.4706, 8.4471, 8.431, 8.4147, 8.398, 8.3803, 8.3682, 8.3452, 8.3375, 8.3245, 8.3182, 8.2971, 8.2882, 8.2766, 8.2598, 8.2545, 8.2452, 8.2401, 8.215, 8.2107, 8.2018, 8.2004, 8.1857, 8.1918, 8.1694, 8.1624, 8.1485, 8.1429, 8.1286, 8.1304, 8.1314, 8.1156, 8.1059, 8.1022, 8.104, 8.079, 8.0817, 8.0802, 8.0737, 8.0693, 8.0533, 8.0562, 8.0477, 8.0449, 8.0427, 8.0312, 8.0098, 8.0221, 8.0127, 8.0063, 8.0096, 7.9981, 8.009, 7.9915, 7.9847, 7.9854, 7.9749, 7.9808, 7.9711, 7.9734, 7.9673, 7.9656, 7.9527, 7.9535, 7.9496, 7.9465, 7.9421, 7.9477, 7.9416, 7.9331, 7.9345, 7.9235, 7.9292, 7.9224, 7.9182, 7.9093, 7.9094, 7.908, 7.8946, 7.9089, 7.9053, 7.8954, 7.8915, 7.8904, 7.8931, 7.8961, 7.8935, 7.8795, 7.8753, 7.8769, 7.8828, 7.8628, 7.8708, 7.8661, 7.8582, 7.862, 7.8619, 7.8508, 7.854, 7.8572, 7.8447, 7.8503, 7.8358, 7.8467, 7.8375, 7.8322, 7.8287, 7.8381, 7.8434, 7.8317, 7.8344, 7.8241, 7.821, 7.8222, 7.8089, 7.8188, 7.8064, 7.8101, 7.8071, 7.8113, 7.806, 7.8099, 7.7961, 7.8028, 7.7997, 7.7914, 7.7891, 7.7895, 7.7816, 7.7834, 7.7851, 7.7861, 7.783, 7.7748, 7.7714, 7.7699, 7.7773, 7.7702, 7.7657, 7.7667, 7.7582, 7.7726, 7.7552, 7.7522, 7.7664, 7.7574, 7.746, 7.7454, 7.747, 7.747, 7.7386, 7.7426, 7.7469, 7.7415, 7.7437, 7.7361, 7.731, 7.7356, 7.7389, 7.7265, 7.735, 7.7319], 'val_metric': [0.0, 0.004215080003327694, 0.0392156862745098, 0.04570850998118876, 0.051758165392633775, 0.05505564432483952, 0.057453809162237406, 0.06037810272448088, 0.059655561829474874, 0.060822797285501086, 0.06128734216199569, 0.06149040355715653, 0.0627538357400722, 0.06309113163458328, 0.06436710809978442, 0.06266487062917439, 0.06131084148289293, 0.06333267887695285, 0.06213872832369942, 0.060445090583762025, 0.0603052798398532, 0.06191654482060511, 0.061835829637687224, 0.0611688676594549, 0.06041149046648614, 0.06250697311168135, 0.06114520578796239, 0.0615341779200891, 0.06174597855227882, 0.058646827624929815, 0.06153163890351728, 0.06220202414475702, 0.06308345910894964, 0.061247869457095754, 0.061918296828846905, 0.06165906816412765, 0.06286754410529263, 0.061606771194671846, 0.06005177847816299, 0.06249304086404632, 0.0597768182582852, 0.06357057031883086, 0.06147322306158556, 0.06100877561891945, 0.06369426751592357, 0.06251227518868718, 0.06114663973970605, 0.06167686623988268, 0.060834788486453, 0.06521799927286964, 0.0632104476348969, 0.0631575993939564, 0.06115299459095877, 0.06187596681198144, 0.06246884864595448, 0.060572624908051834, 0.06280683934315219, 0.06265531202010331, 0.06225254822677112, 0.05953644233454342, 0.061898440092930634, 0.06191916109523678, 0.06207535515750463, 0.06290781328425632, 0.06253331463038295, 0.06278440021216605, 0.06293257423532038, 0.0636154450555618, 0.06138697425798284, 0.061808299010955144, 0.061234261893361126, 0.06102735901730877, 0.061694726589675807, 0.06244951394111584, 0.06440810403527376, 0.06348943224202237, 0.06238619491834048, 0.06340786001944175, 0.06342340317959665, 0.0625192744399899, 0.0633061404475802, 0.06512227956024232, 0.06256967670011149, 0.06325683800491151, 0.06093179904574796, 0.06463462509113342, 0.06405933917734322, 0.06464124111182935, 0.06526186006108667, 0.06460201793721973, 0.0639928399619623, 0.0650353790206636, 0.06349117868390981, 0.0647063756044386, 0.06597444534142471, 0.06398137499649378, 0.0656837594790404, 0.0648659196075152, 0.06633629913278523, 0.06448365537876458, 0.06706992634909967, 0.06685898523672154, 0.06503084688726865, 0.06432715755248479, 0.06498684991940273, 0.06434690403772637, 0.06480159063541405, 0.06449285416955737, 0.06440021335728924, 0.06616642026276884, 0.06364098796134064, 0.06442420517405152, 0.06702714881153543, 0.06641994000056074, 0.06566499721758487, 0.06805049088359046, 0.06615854512847816, 0.0646923680376555, 0.06515375393794084, 0.06667785892162624, 0.06708189546334484, 0.06694584188190326, 0.06634898388227049, 0.06696290463199933, 0.06617957004355768, 0.06761486749263848, 0.06594427244582043, 0.06767420461199565, 0.06671313074993031, 0.06546658505369762, 0.06528581867236243, 0.06591865357643759, 0.06585798316962653, 0.06633079107053387, 0.06712859066427289, 0.06655015534842432, 0.06576591697437618, 0.06731010627911518, 0.06685656836461126, 0.06567423118896382, 0.06684078973235599, 0.06527400520847917, 0.06716542906593713, 0.0644722977095003, 0.06714189113039558, 0.06655099070860852, 0.06914670616842282, 0.06603250318930612, 0.0686394823450661, 0.06784214945424014, 0.06459128677296205, 0.06856023506366307, 0.0674547983310153, 0.06662755615152531, 0.06721375537319266, 0.06752063130400957, 0.0693326626425218, 0.0680805760233099, 0.06843979523703539, 0.06851108882202162, 0.0681754758051013, 0.07125824098751578, 0.06812767984754645, 0.06801134927100598, 0.06526949445404381, 0.06832889760470655, 0.07133650882600168, 0.06965688878946338, 0.0688857938718663, 0.07091662467103421, 0.06916321736212104, 0.07045195065578792, 0.07073575304338048, 0.0694139503462253, 0.06898007429922323, 0.06890732667697932, 0.0692629745145972, 0.06958963523131673, 0.06902630106323447, 0.07058560322165669, 0.07061617996367194, 0.06922536201321523, 0.06796903384477795, 0.06979430825584672, 0.06813291227677948, 0.06938270215519667, 0.06947569756975698, 0.06853205849268841, 0.06937389498498583, 0.06862305016272023, 0.06881079122655444, 0.06860393775447599, 0.0696222029293976, 0.06893750525224797, 0.06763448120639208, 0.07116762876823113, 0.06974536519991066, 0.06732756213348227, 0.06920473299952447, 0.06919486907190565]})
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="Default initialization", optimizer="AdamW")
data.update({'train_loss': [7.542, 6.9915, 6.877, 6.7997, 6.7326, 6.67, 6.6159, 6.5586, 6.5112, 6.4657, 6.4242, 6.3995, 6.3787, 6.3386, 6.3107, 6.2872, 6.2681, 6.238, 6.2161, 6.2025, 6.18, 6.1663, 6.144, 6.1219, 6.1058, 6.079, 6.0691, 6.0531, 6.0446, 6.0256, 6.0154, 5.9953, 5.9845, 5.9745, 5.9567, 5.9406, 5.935, 5.92, 5.9096, 5.8965, 5.889, 5.8782, 5.8643, 5.8553, 5.8448, 5.8308, 5.8296, 5.8102, 5.8093, 5.7906, 5.7611, 5.7462, 5.721, 5.6948, 5.687, 5.6556, 5.5902, 5.5571, 5.4983, 5.4339, 5.3596, 5.3029, 5.2272, 5.1715, 5.1056, 5.0556, 5.0085, 4.9698, 4.944, 4.9016, 4.8533, 4.8258, 4.7952, 4.753, 4.7221, 4.6964, 4.6581, 4.6354, 4.5985, 4.5856, 4.5546, 4.5218, 4.4901, 4.4638, 4.4266, 4.38, 4.3234, 4.2868, 4.2428, 4.1913, 4.1605, 4.1216, 4.0777, 4.049, 4.0109, 3.9741, 3.9578, 3.9303, 3.8968, 3.8706, 3.8374, 3.8208, 3.7958, 3.7733, 3.7559, 3.7322, 3.7093, 3.687, 3.668, 3.6502, 3.6278, 3.6077, 3.5898, 3.5713, 3.5417, 3.5407, 3.5039, 3.5046, 3.4851, 3.4585, 3.4578, 3.4344, 3.421, 3.4116, 3.3998, 3.3849, 3.3602, 3.3562, 3.3495, 3.3234, 3.31, 3.2935, 3.2792, 3.2737, 3.2623, 3.2539, 3.2392, 3.2228, 3.2131, 3.2054, 3.1885, 3.181, 3.1674, 3.1606, 3.1497, 3.1445, 3.1392, 3.1213, 3.1109, 3.1019, 3.0894, 3.0797, 3.07, 3.0621, 3.0461, 3.0474, 3.0343, 3.0348, 3.0162, 3.0129, 2.9874, 2.9917, 2.9829, 2.9784, 2.9705, 2.9556, 2.9512, 2.9407, 2.9435, 2.9249, 2.9225, 2.9111, 2.9017, 2.8998, 2.8962, 2.8843, 2.8718, 2.8715, 2.8609, 2.8572, 2.847, 2.8362, 2.8404, 2.8334, 2.8234, 2.8218, 2.8118, 2.8068, 2.7948, 2.793, 2.7961, 2.7834, 2.7813, 2.7687, 2.761, 2.7558, 2.7565, 2.7398, 2.7395, 2.7326], 'val_metric': [0.09348689019680023, 0.09944260865449905, 0.10673827677965164, 0.11149170249700546, 0.11635858565568924, 0.12238948047161102, 0.12826569378376007, 0.1305609494447708, 0.1345285177230835, 0.13597837090492249, 0.13839347660541534, 0.13927505910396576, 0.1420069932937622, 0.14537665247917175, 0.14340509474277496, 0.1463770568370819, 0.14686019718647003, 0.14559784531593323, 0.14742663502693176, 0.14566092193126678, 0.14863625168800354, 0.14748059213161469, 0.14952723681926727, 0.14818769693374634, 0.14857207238674164, 0.15126073360443115, 0.15001970529556274, 0.15046638250350952, 0.14798368513584137, 0.14915777742862701, 0.14964383840560913, 0.1523405909538269, 0.15188103914260864, 0.1514711230993271, 0.1495848000049591, 0.15320248901844025, 0.15004201233386993, 0.15224087238311768, 0.1527465134859085, 0.15440930426120758, 0.15123647451400757, 0.15133987367153168, 0.14777162671089172, 0.14985841512680054, 0.15230995416641235, 0.15027636289596558, 0.15454953908920288, 0.1555035412311554, 0.1591562032699585, 0.1596890091896057, 0.1633148044347763, 0.16618500649929047, 0.17073512077331543, 0.17567148804664612, 0.17674586176872253, 0.18516947329044342, 0.19005699455738068, 0.200837641954422, 0.20439165830612183, 0.21466070413589478, 0.22491426765918732, 0.2354981005191803, 0.23830647766590118, 0.24273604154586792, 0.24929162859916687, 0.25429216027259827, 0.2579258680343628, 0.2615613341331482, 0.26037418842315674, 0.2682468891143799, 0.26541417837142944, 0.269877165555954, 0.2736886143684387, 0.2757861912250519, 0.2790645658969879, 0.2810056507587433, 0.274420827627182, 0.2874600887298584, 0.2837200164794922, 0.2865513563156128, 0.2877576947212219, 0.2926015257835388, 0.29269787669181824, 0.29624298214912415, 0.3006174564361572, 0.3087039291858673, 0.3108563721179962, 0.3141845464706421, 0.3172023296356201, 0.3180493414402008, 0.3221457600593567, 0.32421842217445374, 0.32785195112228394, 0.332252562046051, 0.33215489983558655, 0.33278170228004456, 0.33377498388290405, 0.3394937813282013, 0.33869242668151855, 0.33860909938812256, 0.34531912207603455, 0.34314724802970886, 0.34848570823669434, 0.33943435549736023, 0.3434009253978729, 0.34760165214538574, 0.34873560070991516, 0.35043707489967346, 0.35108503699302673, 0.35214370489120483, 0.3505341112613678, 0.3501145541667938, 0.3521459996700287, 0.35523033142089844, 0.3559821844100952, 0.3573632538318634, 0.357917457818985, 0.3603328466415405, 0.36326631903648376, 0.36209744215011597, 0.36335325241088867, 0.3653354346752167, 0.3643728196620941, 0.3679163157939911, 0.36512574553489685, 0.37115952372550964, 0.3690965473651886, 0.3679279386997223, 0.37159186601638794, 0.3726559579372406, 0.3741498589515686, 0.372117817401886, 0.3732515275478363, 0.36931470036506653, 0.37682339549064636, 0.3753051459789276, 0.3756573796272278, 0.3746547996997833, 0.37494415044784546, 0.37770378589630127, 0.37689217925071716, 0.3821231424808502, 0.37987828254699707, 0.3767123222351074, 0.38101544976234436, 0.3775327503681183, 0.3850303292274475, 0.38121914863586426, 0.38548001646995544, 0.38508257269859314, 0.3835897147655487, 0.3869875371456146, 0.38442280888557434, 0.38627222180366516, 0.3859487473964691, 0.3864791989326477, 0.38394254446029663, 0.38789117336273193, 0.38891053199768066, 0.3906720280647278, 0.3810068666934967, 0.3898162543773651, 0.387943834066391, 0.38958337903022766, 0.38980546593666077, 0.39170750975608826, 0.39198654890060425, 0.38967031240463257, 0.3914763331413269, 0.39156728982925415, 0.39431145787239075, 0.39049315452575684, 0.395906537771225, 0.39236053824424744, 0.38978949189186096, 0.38976234197616577, 0.39250892400741577, 0.39018017053604126, 0.39476776123046875, 0.39627495408058167, 0.3981277048587799, 0.39696329832077026, 0.3947625756263733, 0.4005635380744934, 0.3957465887069702, 0.3939845860004425, 0.40166518092155457, 0.39460068941116333, 0.3965705931186676, 0.40020760893821716, 0.39639362692832947, 0.4001338481903076, 0.3971770107746124, 0.39936691522598267, 0.3970853090286255, 0.4029671847820282, 0.39889994263648987, 0.39701199531555176, 0.4019692838191986, 0.4003297686576843], 'val_perplexity': [1174.2099609375, 1012.9158935546875, 952.616943359375, 879.58056640625, 851.3609619140625, 812.3502807617188, 763.5390625, 717.3893432617188, 702.6370849609375, 687.3735961914062, 674.25537109375, 657.6511840820312, 632.5193481445312, 609.5951538085938, 607.2874755859375, 594.828857421875, 599.7003173828125, 593.5569458007812, 579.316162109375, 572.177978515625, 563.70703125, 555.4869995117188, 557.422119140625, 546.3578491210938, 571.23583984375, 531.463134765625, 544.0752563476562, 532.0257568359375, 538.944580078125, 518.6178588867188, 523.8938598632812, 517.655517578125, 520.4468383789062, 520.3748779296875, 501.5356750488281, 524.4375, 527.0851440429688, 494.5812683105469, 518.9000854492188, 501.6234436035156, 513.1350708007812, 508.7920837402344, 508.93768310546875, 510.97906494140625, 495.55364990234375, 495.9112854003906, 501.60430908203125, 494.8539733886719, 472.68743896484375, 476.8252258300781, 484.1129455566406, 463.5309143066406, 461.10662841796875, 439.2032775878906, 441.131103515625, 429.2122802734375, 404.8316650390625, 379.71380615234375, 352.541748046875, 327.2327880859375, 293.7922058105469, 272.8728332519531, 253.4473876953125, 243.4262237548828, 226.48040771484375, 219.23837280273438, 210.71665954589844, 202.1103973388672, 198.63938903808594, 193.97657775878906, 193.06541442871094, 183.8351287841797, 175.6916961669922, 175.5159454345703, 172.62088012695312, 163.47633361816406, 169.09178161621094, 156.23406982421875, 157.2568359375, 157.99139404296875, 155.78111267089844, 146.9600067138672, 146.9943389892578, 141.67160034179688, 132.9800567626953, 124.99745178222656, 118.81388854980469, 116.94982147216797, 109.88551330566406, 111.46161651611328, 106.37095642089844, 101.6239013671875, 101.67799377441406, 96.08206939697266, 95.77952575683594, 91.24170684814453, 91.84777069091797, 88.47384643554688, 87.6022720336914, 87.9504623413086, 84.78585815429688, 81.83890533447266, 81.10559844970703, 84.77146911621094, 83.5498275756836, 78.60997772216797, 77.4630126953125, 77.46205139160156, 75.20535278320312, 78.31111907958984, 76.70232391357422, 75.37677764892578, 77.11271667480469, 74.32478332519531, 72.6151351928711, 73.3587646484375, 73.56547546386719, 71.60233306884766, 68.5986557006836, 69.71099853515625, 69.4231948852539, 67.42569732666016, 67.59728240966797, 68.54685974121094, 67.26782989501953, 64.93771362304688, 66.90717315673828, 66.02178955078125, 64.66425323486328, 64.79576873779297, 64.23143768310547, 63.785213470458984, 63.85237503051758, 63.508460998535156, 61.929969787597656, 62.57032012939453, 61.97040939331055, 63.460296630859375, 63.14552688598633, 61.036041259765625, 61.043724060058594, 58.81554412841797, 59.41853332519531, 60.76959228515625, 60.71082305908203, 60.10879135131836, 59.52765655517578, 59.22993469238281, 58.13292694091797, 59.88789749145508, 58.7415771484375, 58.22203063964844, 58.63615798950195, 57.70135498046875, 57.638492584228516, 59.4763298034668, 57.94131088256836, 57.32659912109375, 56.40648651123047, 55.05681228637695, 59.16862487792969, 56.657073974609375, 56.68645095825195, 56.04588317871094, 56.589359283447266, 55.30586242675781, 56.439151763916016, 56.52431106567383, 55.5106201171875, 55.38519287109375, 53.99789047241211, 54.6581916809082, 53.13441467285156, 54.1355094909668, 55.52872848510742, 56.793697357177734, 54.35725402832031, 56.12689208984375, 54.257476806640625, 54.60330581665039, 53.72523880004883, 53.96030044555664, 53.235042572021484, 52.5318717956543, 54.42055892944336, 54.34231185913086, 52.714534759521484, 54.912139892578125, 53.79193878173828, 53.37251281738281, 53.81927490234375, 52.80289840698242, 53.69326400756836, 52.66344451904297, 54.49839782714844, 52.006553649902344, 52.09104919433594, 54.627811431884766, 52.455726623535156, 52.925533294677734]})
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="Sinusoidal", optimizer="AdamW")
data.update({'train_loss': [7.6572, 7.1901, 7.1377, 7.069, 6.9591, 6.8563, 6.7434, 6.6159, 6.5041, 6.4036, 6.2977, 6.2222, 6.1336, 6.0404, 5.9422, 5.8297, 5.7224, 5.6225, 5.5194, 5.4359, 5.3496, 5.2829, 5.2125, 5.1486, 5.0899, 5.0217, 4.9741, 4.9252, 4.8807, 4.8329, 4.7962, 4.7437, 4.7062, 4.6639, 4.6221, 4.5837, 4.5458, 4.5081, 4.4739, 4.4424, 4.4183, 4.3727, 4.3433, 4.3119, 4.2825, 4.2447, 4.229, 4.1968, 4.1749, 4.1493, 4.1175, 4.0942, 4.0749, 4.0403, 4.026, 4.0129, 3.9762, 3.9664, 3.9317, 3.9164, 3.8881, 3.8824, 3.8573, 3.8437, 3.8181, 3.8, 3.778, 3.7666, 3.7645, 3.7474, 3.7162, 3.71, 3.7006, 3.6781, 3.6585, 3.6564, 3.6312, 3.6235, 3.6003, 3.6041, 3.5899, 3.5675, 3.5506, 3.5392, 3.5238, 3.515, 3.5016, 3.4996, 3.4836, 3.4668, 3.463, 3.4482, 3.4375, 3.4276, 3.4103, 3.3981, 3.4033, 3.3914, 3.3765, 3.3655, 3.3507, 3.3468, 3.3377, 3.3338, 3.3284, 3.3155, 3.3037, 3.2962, 3.2837, 3.2858, 3.2721, 3.2662, 3.2513, 3.2505, 3.2281, 3.2307, 3.2098, 3.2181, 3.201, 3.1862, 3.1985, 3.1775, 3.1724, 3.1746, 3.1733, 3.1589, 3.1405, 3.1491, 3.1449, 3.1229, 3.1174, 3.1054, 3.0997, 3.0958, 3.0919, 3.0907, 3.0826, 3.0716, 3.0663, 3.0659, 3.0515, 3.0492, 3.042, 3.039, 3.027, 3.0294, 3.0323, 3.0127, 3.0138, 3.0055, 2.9971, 2.9915, 2.9867, 2.9807, 2.9694, 2.9733, 2.9633, 2.9729, 2.9545, 2.9531, 2.9303, 2.9429, 2.9388, 2.9327, 2.93, 2.913, 2.9134, 2.909, 2.9126, 2.8983, 2.8986, 2.8906, 2.882, 2.8831, 2.8794, 2.8721, 2.8601, 2.8646, 2.8583, 2.8541, 2.8448, 2.8403, 2.8444, 2.8373, 2.8304, 2.8282, 2.8215, 2.824, 2.8104, 2.8163, 2.8194, 2.8039, 2.8086, 2.7947, 2.7875, 2.7873, 2.7851, 2.7798, 2.7755, 2.7689], 'val_metric': [0.06344155967235565, 0.0671917051076889, 0.06718692928552628, 0.08369598537683487, 0.09306462854146957, 0.10461694002151489, 0.12137573957443237, 0.13478714227676392, 0.15152455866336823, 0.16151832044124603, 0.17299884557724, 0.17901165783405304, 0.18947428464889526, 0.1995382159948349, 0.21177589893341064, 0.21967783570289612, 0.22184549272060394, 0.23512747883796692, 0.24377501010894775, 0.2516774833202362, 0.2545666992664337, 0.26116296648979187, 0.2649429738521576, 0.26737847924232483, 0.2648446559906006, 0.2817973792552948, 0.27951690554618835, 0.2855631411075592, 0.290214478969574, 0.2922515571117401, 0.2932453751564026, 0.29843589663505554, 0.30216893553733826, 0.30327755212783813, 0.3051886558532715, 0.30587679147720337, 0.31044524908065796, 0.31844040751457214, 0.3179311156272888, 0.32120588421821594, 0.320949912071228, 0.32447537779808044, 0.3267993927001953, 0.3322679102420807, 0.3360220193862915, 0.3355124592781067, 0.3367272615432739, 0.3378831744194031, 0.34447699785232544, 0.3487708568572998, 0.34448423981666565, 0.3475211262702942, 0.35125136375427246, 0.35423991084098816, 0.3555130958557129, 0.3559384346008301, 0.35850122570991516, 0.36060309410095215, 0.3611041009426117, 0.3582518994808197, 0.36533355712890625, 0.3622492849826813, 0.36209219694137573, 0.36514973640441895, 0.3689717948436737, 0.37159767746925354, 0.37343713641166687, 0.37347063422203064, 0.3712758421897888, 0.37446415424346924, 0.3734050691127777, 0.3749581277370453, 0.3776262700557709, 0.37505918741226196, 0.37894177436828613, 0.3811299800872803, 0.3786872923374176, 0.3860574960708618, 0.3827874958515167, 0.38010597229003906, 0.3815731406211853, 0.3863585293292999, 0.38534003496170044, 0.3854122459888458, 0.38812798261642456, 0.38999494910240173, 0.39031243324279785, 0.38745397329330444, 0.39176732301712036, 0.38808855414390564, 0.39315879344940186, 0.3909452259540558, 0.39153823256492615, 0.3937445878982544, 0.3926175534725189, 0.3931446671485901, 0.3936797082424164, 0.3932653069496155, 0.3969760239124298, 0.39406058192253113, 0.3992270827293396, 0.39413994550704956, 0.399915874004364, 0.39355888962745667, 0.39410650730133057, 0.39830341935157776, 0.39754682779312134, 0.3991119861602783, 0.3993149995803833, 0.4000948369503021, 0.3955519199371338, 0.3978320360183716, 0.39833277463912964, 0.39790281653404236, 0.39880356192588806, 0.40193548798561096, 0.40017372369766235, 0.40241509675979614, 0.4032730162143707, 0.40471193194389343, 0.4034172296524048, 0.4051319658756256, 0.40448492765426636, 0.40577101707458496, 0.40384992957115173, 0.40941163897514343, 0.4062763750553131, 0.40626829862594604, 0.40683022141456604, 0.4087696969509125, 0.4075937271118164, 0.4072931408882141, 0.40567392110824585, 0.40396925806999207, 0.411215215921402, 0.4078728258609772, 0.4077710509300232, 0.40639907121658325, 0.405607670545578, 0.412233829498291, 0.41022831201553345, 0.41132983565330505, 0.4103902578353882, 0.40929511189460754, 0.4133044183254242, 0.40772977471351624, 0.41376227140426636, 0.4096178412437439, 0.4154627025127411, 0.40945982933044434, 0.4107343256473541, 0.41460752487182617, 0.4118497967720032, 0.41244831681251526, 0.4143080413341522, 0.41190364956855774, 0.4096244275569916, 0.4151514172554016, 0.414227694272995, 0.4160537123680115, 0.40880170464515686, 0.4149530231952667, 0.4108959436416626, 0.41242238879203796, 0.414023756980896, 0.4131110906600952, 0.41647520661354065, 0.41201308369636536, 0.41668522357940674, 0.4130130410194397, 0.41802772879600525, 0.41357800364494324, 0.4192414879798889, 0.4191015064716339, 0.41095349192619324, 0.4113064408302307, 0.4143696129322052, 0.4128669798374176, 0.4201734662055969, 0.41688573360443115, 0.4217129945755005, 0.41900745034217834, 0.416170597076416, 0.4191039800643921, 0.4152251183986664, 0.41818130016326904, 0.4187106192111969, 0.41557928919792175, 0.4132966697216034, 0.4192851483821869, 0.41921907663345337, 0.4187907874584198, 0.41585683822631836, 0.416034072637558, 0.4154850244522095, 0.4212052822113037, 0.4186117947101593, 0.4165596067905426, 0.41900473833084106, 0.4161473214626312], 'val_perplexity': [1380.8294677734375, 1307.633544921875, 1277.6865234375, 1147.733154296875, 1044.65625, 944.0477905273438, 810.0610961914062, 704.0094604492188, 641.6773071289062, 576.354736328125, 529.6739501953125, 482.27154541015625, 443.6580810546875, 403.5230712890625, 361.23260498046875, 321.41717529296875, 305.4492492675781, 272.7865905761719, 247.0054473876953, 227.6531219482422, 209.97544860839844, 198.04147338867188, 189.50994873046875, 182.5464324951172, 181.05364990234375, 161.56253051757812, 160.7646942138672, 152.84701538085938, 147.44410705566406, 140.90162658691406, 139.0000762939453, 132.80162048339844, 128.59703063964844, 126.88126373291016, 118.01233673095703, 118.80839538574219, 115.48258972167969, 107.93653106689453, 109.65980529785156, 105.24665832519531, 103.72208404541016, 102.3960952758789, 98.49270629882812, 97.55777740478516, 93.75238037109375, 90.2511978149414, 90.3825454711914, 88.81110382080078, 83.81178283691406, 83.1412353515625, 83.54237365722656, 81.22398376464844, 81.57389068603516, 77.9935073852539, 77.62870788574219, 77.19365692138672, 77.05762481689453, 74.23390197753906, 73.90412902832031, 74.04553985595703, 71.6315689086914, 72.22374725341797, 70.0416259765625, 69.59043884277344, 67.61339569091797, 66.97235107421875, 66.15884399414062, 66.63472747802734, 67.1021957397461, 65.24937438964844, 65.67135620117188, 64.76957702636719, 62.79799270629883, 63.36142349243164, 62.97911834716797, 62.42194366455078, 63.24897766113281, 59.43196487426758, 60.53079605102539, 63.1522102355957, 62.04370880126953, 59.289306640625, 59.612815856933594, 58.52621078491211, 58.08975601196289, 56.74429702758789, 55.923057556152344, 57.222408294677734, 56.5907096862793, 57.5630989074707, 55.782039642333984, 55.487281799316406, 57.02228927612305, 54.50697708129883, 55.431400299072266, 53.7933235168457, 53.841915130615234, 53.667884826660156, 53.588802337646484, 54.91590881347656, 53.45683288574219, 52.531494140625, 52.671165466308594, 54.519493103027344, 55.08864212036133, 51.459896087646484, 51.66039276123047, 51.97254180908203, 50.94147491455078, 53.86963653564453, 53.38080978393555, 51.843017578125, 52.8150749206543, 51.659088134765625, 51.814483642578125, 51.74940872192383, 52.37957000732422, 51.68398666381836, 49.324493408203125, 50.041542053222656, 50.327430725097656, 49.35348129272461, 49.52060317993164, 50.92298126220703, 49.768489837646484, 48.90388488769531, 50.45526123046875, 49.385108947753906, 49.17724609375, 49.127044677734375, 49.32094192504883, 48.8317985534668, 49.09886932373047, 49.45146560668945, 47.398704528808594, 48.290077209472656, 47.66664123535156, 49.73612976074219, 49.70304489135742, 47.686954498291016, 48.3816032409668, 47.04188919067383, 47.549232482910156, 47.97683334350586, 48.494503021240234, 48.25261688232422, 47.89315414428711, 47.42280197143555, 46.6263542175293, 48.63564682006836, 47.75651931762695, 46.75868606567383, 47.07343673706055, 47.15951919555664, 47.82465744018555, 48.79609298706055, 47.77814483642578, 47.28565216064453, 46.72807312011719, 45.83784103393555, 49.04845428466797, 46.98475646972656, 47.129913330078125, 46.94622802734375, 47.60676574707031, 46.62861251831055, 47.47598648071289, 47.145423889160156, 46.131412506103516, 46.938270568847656, 45.82461929321289, 46.490562438964844, 45.06277847290039, 45.478851318359375, 47.45380401611328, 48.07178497314453, 46.52799987792969, 47.74141311645508, 46.34543228149414, 47.18833541870117, 45.25800704956055, 46.01504898071289, 46.451908111572266, 45.56705856323242, 47.60716247558594, 46.20203399658203, 45.33020782470703, 47.18974304199219, 46.208499908447266, 45.653507232666016, 46.737586975097656, 46.008216857910156, 46.26005554199219, 46.45561599731445, 46.5937614440918, 45.8140869140625, 45.3917121887207, 47.43642044067383, 46.25888442993164, 46.20219802856445]})
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="Sinusoidal 4", optimizer="AdamW")
data.update({'train_loss': [7.6544, 7.1858, 7.0973, 6.9692, 6.8303, 6.6834, 6.5507, 6.4291, 6.3235, 6.226, 6.1295, 6.0576, 5.9498, 5.8271, 5.7231, 5.6253, 5.5331, 5.4504, 5.3668, 5.2963, 5.2204, 5.1567, 5.088, 5.02, 4.9636, 4.894, 4.8475, 4.7997, 4.7511, 4.7023, 4.6665, 4.6164, 4.5787, 4.536, 4.4943, 4.4564, 4.4209, 4.3815, 4.3507, 4.3176, 4.2923, 4.2498, 4.2196, 4.1911, 4.1631, 4.1267, 4.1071, 4.0767, 4.0567, 4.0291, 3.9965, 3.9729, 3.9539, 3.9197, 3.905, 3.8879, 3.8532, 3.8455, 3.8119, 3.7922, 3.7686, 3.7581, 3.7343, 3.7187, 3.6971, 3.6782, 3.6511, 3.6411, 3.6344, 3.6179, 3.586, 3.5797, 3.5676, 3.548, 3.5299, 3.5202, 3.4995, 3.4911, 3.4703, 3.4651, 3.4544, 3.4331, 3.42, 3.3967, 3.3802, 3.3722, 3.3587, 3.3593, 3.345, 3.3238, 3.3192, 3.3038, 3.294, 3.2834, 3.2645, 3.2541, 3.2608, 3.2455, 3.2298, 3.2174, 3.2025, 3.1966, 3.1888, 3.1809, 3.1759, 3.163, 3.1531, 3.1446, 3.135, 3.1288, 3.1188, 3.107, 3.0963, 3.0917, 3.0698, 3.0767, 3.0569, 3.0578, 3.0456, 3.0306, 3.0382, 3.0192, 3.0151, 3.0141, 3.0122, 3.0008, 2.9786, 2.9871, 2.9809, 2.9631, 2.9561, 2.941, 2.9352, 2.9324, 2.9285, 2.9253, 2.9176, 2.9071, 2.9008, 2.8977, 2.8829, 2.8816, 2.8747, 2.8753, 2.8604, 2.8616, 2.8636, 2.8485, 2.8429, 2.8351, 2.8263, 2.8211, 2.8135, 2.8097, 2.7957, 2.802, 2.7962, 2.8005, 2.7826, 2.7822, 2.7628, 2.7682, 2.763, 2.76, 2.7541, 2.7418, 2.7387, 2.7284, 2.7389, 2.7192, 2.7245, 2.714, 2.7044, 2.7042, 2.705, 2.6972, 2.6863, 2.6887, 2.6809, 2.6785, 2.6662, 2.6595, 2.6652, 2.6583, 2.6537, 2.6529, 2.6437, 2.6442, 2.6367, 2.6358, 2.6398, 2.6236, 2.6279, 2.6147, 2.6091, 2.6072, 2.6062, 2.6004, 2.5973, 2.5868], 'val_metric': [0.06344155661307127, 0.07356978453176562, 0.09398338508013762, 0.10930173793413257, 0.12550382209867963, 0.14285313822779133, 0.16021260440394836, 0.17349899980277803, 0.18145115753811406, 0.1861290231746121, 0.19368927961474927, 0.19907694039511453, 0.21198104693140793, 0.22424056971180595, 0.22930257300445167, 0.2367963181231408, 0.23891052764311935, 0.25234903093708805, 0.259532014228546, 0.2644542607917692, 0.26730058108821975, 0.2727477224159262, 0.2739519705463978, 0.2797695982017421, 0.2735825353842718, 0.2901651232846145, 0.2883565114576882, 0.29249617151607965, 0.3002960232350313, 0.3023301516002246, 0.30083982372016965, 0.30498787185992693, 0.3079266588406283, 0.3109055855151024, 0.3115142395363094, 0.31260424774824863, 0.3180621674600952, 0.32460108228111556, 0.3238968932913102, 0.3256597260884089, 0.3247070038692312, 0.32785648419817254, 0.33087140891347694, 0.33378192727170775, 0.34005507189942424, 0.34112398642013414, 0.34143947043644113, 0.34177500775543584, 0.34943389849602885, 0.3507564952317029, 0.34724846980509405, 0.35133694340787297, 0.3517278103192175, 0.3560118126845732, 0.3587528382344797, 0.3587676116109319, 0.3594605270582924, 0.3629764065335753, 0.36236767472552156, 0.3589500139625803, 0.3684035844673083, 0.367797597580936, 0.3670896737604582, 0.36894205565779936, 0.3704025810071539, 0.37450099103877615, 0.3759767805313686, 0.3733864631271748, 0.3740610493740329, 0.37723795914936536, 0.37757372616398616, 0.37878280290340594, 0.38004730763879224, 0.3798501434500432, 0.380867332700787, 0.38074319657411243, 0.38157268117769, 0.38641855297875294, 0.38739396663108816, 0.3817600717710056, 0.3877191048395987, 0.3939028494503029, 0.38913043478260867, 0.3904648996528072, 0.3921975863036767, 0.3935281251752566, 0.3951730726005844, 0.3935246339338411, 0.3971754420377168, 0.3906670403587444, 0.39508866140851373, 0.39649874552476533, 0.39446472867136767, 0.3988316516197557, 0.39757258580853494, 0.3983057978738324, 0.3961040791587968, 0.3995372693315493, 0.40134024101813265, 0.4014988395179106, 0.40401579433756196, 0.400446857853951, 0.40448681996634883, 0.39753903074810104, 0.39984728938661235, 0.40620029578368727, 0.40303564927609287, 0.4041903704731511, 0.4062771960360462, 0.4020753716979553, 0.40304075057932515, 0.40260937587305134, 0.40461304494761746, 0.40373454453696694, 0.40425709515859765, 0.4070406732117812, 0.4051895648275282, 0.4075423063991931, 0.4067021662159525, 0.4073980805282744, 0.41071178071430575, 0.4087855878137208, 0.4072039243167484, 0.40908963073375143, 0.40702543206407193, 0.41357853214067447, 0.41089220377146074, 0.413936368959652, 0.41354892667967663, 0.4132491235880029, 0.41315833848575123, 0.41178120617110797, 0.4141172497255917, 0.40889606413182655, 0.4152266606822262, 0.41297714158899246, 0.41289023162134947, 0.4122011771597534, 0.41119302949061665, 0.4171586870750203, 0.41587328334316287, 0.41768642715129795, 0.4178158673440902, 0.4145360767566968, 0.4161219429730644, 0.41469830963841936, 0.42019061460633333, 0.4156081868101392, 0.42009259775757235, 0.4170445004198153, 0.4180608578486414, 0.41888904435427454, 0.4169123783031989, 0.42027042127612024, 0.4176854798191258, 0.4178221123120954, 0.4164431030628214, 0.4188496343820917, 0.41781660360560874, 0.4224618299342472, 0.41415973656304067, 0.4215738532753542, 0.42101280721912393, 0.42023204202601344, 0.4208795496349144, 0.42157164869029273, 0.4198655085458112, 0.4189200525712368, 0.4199442896935933, 0.4209362226328462, 0.4237890144311444, 0.41669683383921363, 0.42452697573729936, 0.4221133817485785, 0.4173421141506248, 0.416509632963015, 0.42083227963696646, 0.4179826512455516, 0.4238947957470621, 0.42211533083505787, 0.4232779097387173, 0.42297202305637566, 0.42106145719795424, 0.427134404057481, 0.42240728837716235, 0.42034775758257836, 0.4255738073807381, 0.41898200224971877, 0.42115454775068056, 0.4250925822017731, 0.4241799280956495, 0.42330860616877686, 0.4224101716750217, 0.42104820863329506, 0.42021156875984694, 0.4262249365570707, 0.42293946839401386, 0.42127897235409106, 0.4247671263531847, 0.4197803426207976]})
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="Orthogonal", optimizer="AdamW")
data.update({'train_loss': [7.6705, 7.1964, 7.1551, 7.0994, 7.0337, 6.9608, 6.9012, 6.8418, 6.7908, 6.744, 6.6993, 6.6715, 6.6327, 6.5724, 6.4999, 6.4106, 6.2995, 6.172, 6.0418, 5.9404, 5.8205, 5.6884, 5.5546, 5.4271, 5.3244, 5.2171, 5.1392, 5.0646, 4.9935, 4.9245, 4.872, 4.8052, 4.7514, 4.6998, 4.6505, 4.6004, 4.5554, 4.511, 4.474, 4.4329, 4.3986, 4.3536, 4.3178, 4.2838, 4.2514, 4.2114, 4.1881, 4.157, 4.1298, 4.0972, 4.0667, 4.0403, 4.0179, 3.9832, 3.9638, 3.9473, 3.9082, 3.8998, 3.8627, 3.8439, 3.8172, 3.808, 3.7801, 3.7643, 3.7381, 3.7189, 3.6901, 3.681, 3.6764, 3.6568, 3.62, 3.6161, 3.6014, 3.5783, 3.5608, 3.5545, 3.5342, 3.5254, 3.4975, 3.4965, 3.4809, 3.4603, 3.4444, 3.4241, 3.415, 3.4049, 3.3838, 3.3818, 3.3701, 3.3507, 3.3468, 3.3284, 3.3184, 3.3101, 3.2888, 3.2775, 3.2776, 3.2673, 3.2521, 3.2409, 3.227, 3.2202, 3.2107, 3.1992, 3.1959, 3.1848, 3.1753, 3.1667, 3.156, 3.1546, 3.1381, 3.1245, 3.1152, 3.1101, 3.0864, 3.0933, 3.0722, 3.0759, 3.0661, 3.0518, 3.0537, 3.0375, 3.0305, 3.0308, 3.027, 3.016, 2.9965, 3.0031, 3.0012, 2.9751, 2.9713, 2.9569, 2.9526, 2.9537, 2.9472, 2.9438, 2.9318, 2.9225, 2.9155, 2.91, 2.8994, 2.8957, 2.8897, 2.8876, 2.874, 2.8766, 2.8816, 2.8592, 2.8581, 2.8506, 2.8413, 2.8355, 2.8289, 2.8261, 2.8122, 2.8141, 2.8095, 2.8135, 2.7958, 2.7968, 2.7782, 2.7814, 2.7755, 2.776, 2.7705, 2.7545, 2.7565, 2.7479, 2.7518, 2.7344, 2.7367, 2.729, 2.7168, 2.7211, 2.7188, 2.7123, 2.6997, 2.7015, 2.696, 2.6922, 2.6813, 2.6711, 2.6808, 2.6776, 2.6678, 2.6676, 2.6584, 2.6584, 2.6468, 2.6504, 2.656, 2.643, 2.6403, 2.6302, 2.625, 2.6244, 2.6205, 2.6112, 2.6127, 2.6035], 'val_metric': [0.06344155967235565, 0.06408585608005524, 0.06517300009727478, 0.06923659890890121, 0.07594162970781326, 0.07554733008146286, 0.0805421993136406, 0.0888625904917717, 0.0911349505186081, 0.09664066880941391, 0.09751658886671066, 0.09897562861442566, 0.10675203055143356, 0.11455435305833817, 0.12610241770744324, 0.13995060324668884, 0.15854512155056, 0.1787787824869156, 0.19077923893928528, 0.19917245209217072, 0.21519726514816284, 0.23228545486927032, 0.24279697239398956, 0.2522337734699249, 0.25172385573387146, 0.2704172730445862, 0.270902544260025, 0.27829596400260925, 0.28387510776519775, 0.2889949381351471, 0.29199811816215515, 0.29495078325271606, 0.30174967646598816, 0.30104222893714905, 0.3062754273414612, 0.3050428032875061, 0.31190142035484314, 0.31869015097618103, 0.3170869052410126, 0.3202037513256073, 0.32120227813720703, 0.32078686356544495, 0.3268555700778961, 0.3316230773925781, 0.3353823125362396, 0.3346426784992218, 0.3372882306575775, 0.33681154251098633, 0.34357574582099915, 0.34712085127830505, 0.3442586064338684, 0.3455570936203003, 0.3484487533569336, 0.35193362832069397, 0.3538793921470642, 0.3543541133403778, 0.35610294342041016, 0.36088231205940247, 0.35964393615722656, 0.3574141263961792, 0.36602500081062317, 0.3624989688396454, 0.3636925220489502, 0.36297473311424255, 0.3683265447616577, 0.3719885051250458, 0.37234872579574585, 0.371366024017334, 0.37051624059677124, 0.37297919392585754, 0.37317973375320435, 0.37498605251312256, 0.37787672877311707, 0.37561628222465515, 0.3778534233570099, 0.38046690821647644, 0.3766983151435852, 0.38319677114486694, 0.3817763030529022, 0.3781434893608093, 0.3826204240322113, 0.3871718645095825, 0.384782612323761, 0.38702118396759033, 0.3891383707523346, 0.3873310387134552, 0.39084625244140625, 0.3890559673309326, 0.3917953372001648, 0.38878923654556274, 0.3921799063682556, 0.39190369844436646, 0.39229074120521545, 0.3947228789329529, 0.39233916997909546, 0.3933129608631134, 0.394525408744812, 0.3956068456172943, 0.39568081498146057, 0.39551466703414917, 0.399815171957016, 0.3934328854084015, 0.401205837726593, 0.39212939143180847, 0.39560532569885254, 0.401205450296402, 0.399367094039917, 0.3982239365577698, 0.4011959135532379, 0.4014616906642914, 0.4006386697292328, 0.3986142873764038, 0.39751604199409485, 0.39742618799209595, 0.40153032541275024, 0.3997195065021515, 0.4012385606765747, 0.40319958329200745, 0.4014887511730194, 0.40479588508605957, 0.40420278906822205, 0.4016750454902649, 0.4018220007419586, 0.40627723932266235, 0.40452438592910767, 0.4091338515281677, 0.4059104919433594, 0.40710482001304626, 0.40869808197021484, 0.41002169251441956, 0.4090832471847534, 0.4057784080505371, 0.40651825070381165, 0.40469297766685486, 0.41166403889656067, 0.409509539604187, 0.40796688199043274, 0.40572959184646606, 0.4044068455696106, 0.41052690148353577, 0.41022831201553345, 0.41379407048225403, 0.41184189915657043, 0.40760356187820435, 0.4121773838996887, 0.40669429302215576, 0.41566282510757446, 0.40992289781570435, 0.41387292742729187, 0.41060733795166016, 0.41053783893585205, 0.41502729058265686, 0.41237831115722656, 0.4125600755214691, 0.41165634989738464, 0.4110422730445862, 0.41149675846099854, 0.41621607542037964, 0.41208547353744507, 0.41555219888687134, 0.4102528393268585, 0.4171973764896393, 0.4126895070075989, 0.41247856616973877, 0.41433030366897583, 0.41330718994140625, 0.41655924916267395, 0.41232067346572876, 0.41643452644348145, 0.4130130410194397, 0.41702091693878174, 0.4133552312850952, 0.42095646262168884, 0.4142318367958069, 0.41047507524490356, 0.4102376699447632, 0.4142572283744812, 0.4118382930755615, 0.41684386134147644, 0.41657811403274536, 0.42025989294052124, 0.41304653882980347, 0.41751208901405334, 0.4174133539199829, 0.41561636328697205, 0.41356703639030457, 0.4189356565475464, 0.4128515124320984, 0.413240522146225, 0.4184715449810028, 0.4158189594745636, 0.4158904552459717, 0.4137844145298004, 0.41614609956741333, 0.41179946064949036, 0.4194484055042267, 0.414786696434021, 0.41480034589767456, 0.41685083508491516, 0.41438671946525574], 'val_perplexity': [1388.87646484375, 1328.0814208984375, 1302.264892578125, 1223.7252197265625, 1160.0657958984375, 1102.766845703125, 1029.6473388671875, 972.6255493164062, 947.600341796875, 907.8381958007812, 892.9541015625, 864.0932006835938, 818.663818359375, 767.1820068359375, 702.5667724609375, 623.0748291015625, 554.4007568359375, 477.7995910644531, 419.53497314453125, 380.8903503417969, 327.003173828125, 286.0456237792969, 254.95077514648438, 228.05848693847656, 222.4997100830078, 188.85421752929688, 182.87261962890625, 169.86167907714844, 161.49783325195312, 150.89466857910156, 147.20428466796875, 139.72607421875, 133.3627166748047, 131.15957641601562, 121.2209701538086, 121.73777770996094, 117.23004913330078, 108.97673797607422, 110.91600036621094, 105.59270477294922, 105.04521179199219, 103.13533782958984, 98.7304458618164, 98.11643981933594, 93.0895767211914, 90.6268081665039, 91.39981842041016, 88.35542297363281, 83.86859130859375, 82.60124969482422, 84.0732421875, 81.71844482421875, 81.45895385742188, 78.00908660888672, 77.4157485961914, 77.24089813232422, 76.58697509765625, 73.983642578125, 73.48915100097656, 73.70282745361328, 70.73542022705078, 71.27694702148438, 68.89348602294922, 68.87952423095703, 66.34529876708984, 66.04508972167969, 65.87125396728516, 65.9488525390625, 66.08421325683594, 64.67556762695312, 64.18383026123047, 63.62821960449219, 61.76431655883789, 62.6340217590332, 62.483795166015625, 61.626434326171875, 62.05781936645508, 59.05078125, 59.373924255371094, 62.336517333984375, 60.96995162963867, 57.582149505615234, 59.121891021728516, 57.66109085083008, 57.13949203491211, 56.3690071105957, 55.458526611328125, 56.435543060302734, 55.01799774169922, 56.573307037353516, 55.2919921875, 54.78944778442383, 56.27433776855469, 53.18181228637695, 54.56379699707031, 53.293434143066406, 53.41246032714844, 53.11429977416992, 52.713539123535156, 53.47150421142578, 52.152400970458984, 51.81108856201172, 51.794673919677734, 54.550514221191406, 54.05539321899414, 51.059993743896484, 51.19662857055664, 51.33052444458008, 50.1349983215332, 53.10564041137695, 52.0584831237793, 50.762367248535156, 52.364810943603516, 51.10159683227539, 50.678504943847656, 50.95294189453125, 51.71926498413086, 50.93043518066406, 49.40549087524414, 49.74459457397461, 49.53277587890625, 49.80926513671875, 49.64947509765625, 50.31442642211914, 49.41991424560547, 48.28371047973633, 49.95935821533203, 48.92123794555664, 48.779178619384766, 49.19411087036133, 49.036014556884766, 48.53179168701172, 49.085750579833984, 48.54328155517578, 47.231422424316406, 47.89567565917969, 48.13901901245117, 49.64841079711914, 49.32500076293945, 47.67042541503906, 47.428863525390625, 46.6431770324707, 47.41501235961914, 48.4671630859375, 48.849185943603516, 48.42177200317383, 47.56956100463867, 46.937767028808594, 46.535343170166016, 48.370635986328125, 48.1348762512207, 46.275089263916016, 47.23245620727539, 47.30669403076172, 47.5652961730957, 49.439945220947266, 47.547203063964844, 47.01407241821289, 47.252685546875, 45.939796447753906, 48.90364074707031, 47.13813781738281, 47.455467224121094, 46.979705810546875, 47.56534194946289, 47.139095306396484, 47.42117691040039, 47.826114654541016, 46.81855773925781, 47.11988067626953, 45.41490173339844, 47.05680847167969, 45.146732330322266, 46.15324783325195, 47.29239273071289, 48.24068832397461, 46.83194351196289, 48.1022834777832, 46.38026428222656, 47.59989929199219, 46.01881408691406, 46.92345428466797, 46.757022857666016, 46.11283874511719, 47.60204315185547, 46.67823028564453, 46.33839416503906, 48.262306213378906, 47.389122009277344, 46.35554504394531, 47.76847457885742, 46.4599723815918, 47.078914642333984, 46.895965576171875, 48.098236083984375, 46.5505256652832, 46.5621223449707, 48.35628890991211, 47.269195556640625, 47.44924545288086]})
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="LSUV", optimizer="AdamW")
data.update({'train_loss': [7.4331, 6.8725, 6.7721, 6.7117, 6.6618, 6.6249, 6.5932, 6.5576, 6.5267, 6.4918, 6.4634, 6.4436, 6.43, 6.3959, 6.3751, 6.3551, 6.3418, 6.3157, 6.296, 6.2882, 6.2687, 6.2583, 6.2374, 6.218, 6.2041, 6.1785, 6.1716, 6.1583, 6.1517, 6.1341, 6.1257, 6.1061, 6.0958, 6.0864, 6.0682, 6.0529, 6.049, 6.0344, 6.0224, 6.0098, 6.0029, 5.9899, 5.9783, 5.9698, 5.9587, 5.9421, 5.9429, 5.9254, 5.9284, 5.9133, 5.8937, 5.8964, 5.8795, 5.8658, 5.8681, 5.862, 5.8442, 5.845, 5.8277, 5.8193, 5.8074, 5.8122, 5.8013, 5.7978, 5.7749, 5.7732, 5.7609, 5.7652, 5.7659, 5.76, 5.7463, 5.743, 5.7402, 5.7231, 5.7215, 5.7172, 5.709, 5.7073, 5.6929, 5.7028, 5.6908, 5.6804, 5.675, 5.67, 5.6584, 5.6593, 5.6489, 5.6503, 5.6456, 5.6309, 5.6405, 5.6301, 5.6177, 5.6225, 5.6085, 5.6065, 5.6087, 5.6081, 5.5963, 5.5901, 5.5833, 5.5805, 5.5789, 5.5772, 5.5758, 5.5711, 5.563, 5.5579, 5.5599, 5.5517, 5.5482, 5.5399, 5.542, 5.5344, 5.5221, 5.5248, 5.5102, 5.5208, 5.5118, 5.5039, 5.5108, 5.5026, 5.5, 5.5007, 5.494, 5.4873, 5.4794, 5.4832, 5.4798, 5.4654, 5.4642, 5.4612, 5.4544, 5.4608, 5.4558, 5.446, 5.4401, 5.4393, 5.4368, 5.43, 5.4167, 5.4185, 5.403, 5.4052, 5.3975, 5.3974, 5.3953, 5.3744, 5.3753, 5.3698, 5.3528, 5.352, 5.3421, 5.3288, 5.3088, 5.308, 5.2817, 5.271, 5.2493, 5.2401, 5.2079, 5.204, 5.1796, 5.1612, 5.1428, 5.11, 5.0913, 5.0607, 5.0493, 5.0137, 4.9928, 4.9569, 4.9282, 4.9103, 4.8909, 4.8579, 4.8286, 4.8081, 4.7819, 4.7629, 4.7385, 4.7162, 4.7028, 4.6757, 4.6551, 4.6387, 4.6204, 4.6069, 4.5826, 4.5693, 4.5596, 4.5346, 4.5242, 4.4985, 4.4802, 4.4676, 4.452, 4.4323, 4.4229, 4.4089], 'val_metric': [0.10866443067789078, 0.12359613180160522, 0.1364157646894455, 0.14147739112377167, 0.142515629529953, 0.14147955179214478, 0.1470794975757599, 0.14876173436641693, 0.14627328515052795, 0.14617182314395905, 0.1482207328081131, 0.14819610118865967, 0.14996051788330078, 0.15247023105621338, 0.15132850408554077, 0.15291576087474823, 0.1528848111629486, 0.15120747685432434, 0.15387395024299622, 0.15197941660881042, 0.15439152717590332, 0.15330109000205994, 0.15326471626758575, 0.15422871708869934, 0.15211747586727142, 0.1553051471710205, 0.15294747054576874, 0.15313935279846191, 0.15175379812717438, 0.15190903842449188, 0.1523878127336502, 0.15563052892684937, 0.15291519463062286, 0.15404173731803894, 0.15301232039928436, 0.15498165786266327, 0.15166619420051575, 0.1541556864976883, 0.15587010979652405, 0.15582896769046783, 0.1538720279932022, 0.15262524783611298, 0.14970934391021729, 0.1502789705991745, 0.15403443574905396, 0.1508655697107315, 0.15359586477279663, 0.15496771037578583, 0.15670591592788696, 0.1556897908449173, 0.1549375206232071, 0.1547936350107193, 0.15391945838928223, 0.15890873968601227, 0.15439996123313904, 0.15472754836082458, 0.1551831215620041, 0.1537344753742218, 0.15544885396957397, 0.1521083563566208, 0.15482906997203827, 0.15590757131576538, 0.15433208644390106, 0.1537560671567917, 0.15340159833431244, 0.15435080230236053, 0.1546662151813507, 0.15290717780590057, 0.15116049349308014, 0.15421254932880402, 0.14984649419784546, 0.15321049094200134, 0.1546124964952469, 0.15152780711650848, 0.15418317914009094, 0.15435834228992462, 0.14939631521701813, 0.15839466452598572, 0.1542048156261444, 0.15464408695697784, 0.15072759985923767, 0.1538030058145523, 0.15217390656471252, 0.15482541918754578, 0.15338198840618134, 0.15459032356739044, 0.1573387235403061, 0.15286248922348022, 0.15322105586528778, 0.15291479229927063, 0.15508754551410675, 0.15205931663513184, 0.15226176381111145, 0.15029208362102509, 0.15368983149528503, 0.15508681535720825, 0.15214388072490692, 0.15119585394859314, 0.15302962064743042, 0.14856967329978943, 0.15433083474636078, 0.15170541405677795, 0.15487942099571228, 0.14866720139980316, 0.15073102712631226, 0.15302620828151703, 0.15228654444217682, 0.15124185383319855, 0.14918166399002075, 0.1505202353000641, 0.14785508811473846, 0.14759457111358643, 0.15044496953487396, 0.14983037114143372, 0.1530328392982483, 0.15326787531375885, 0.14809314906597137, 0.149165078997612, 0.14820596575737, 0.15291418135166168, 0.1515585035085678, 0.15064220130443573, 0.14976873993873596, 0.14947830140590668, 0.1509343832731247, 0.14964720606803894, 0.14810019731521606, 0.15040570497512817, 0.14770002663135529, 0.1494908481836319, 0.15179023146629333, 0.15023842453956604, 0.15107934176921844, 0.1488058716058731, 0.15086400508880615, 0.14885708689689636, 0.15030211210250854, 0.1519707590341568, 0.14773234724998474, 0.1524190604686737, 0.15488527715206146, 0.15639439225196838, 0.15487688779830933, 0.15598136186599731, 0.1567113697528839, 0.15327997505664825, 0.1608205884695053, 0.15982583165168762, 0.16369163990020752, 0.16395185887813568, 0.16062204539775848, 0.16625156998634338, 0.16862308979034424, 0.1701028048992157, 0.1753195971250534, 0.1764705926179886, 0.1805555522441864, 0.18454599380493164, 0.18581682443618774, 0.18892788887023926, 0.19247083365917206, 0.1973067820072174, 0.19776923954486847, 0.19802230596542358, 0.2014380395412445, 0.20462249219417572, 0.21269263327121735, 0.21618522703647614, 0.21824511885643005, 0.22576852142810822, 0.23131781816482544, 0.23430146276950836, 0.23708847165107727, 0.23315317928791046, 0.2376730889081955, 0.2374912053346634, 0.24235016107559204, 0.24096418917179108, 0.24804140627384186, 0.2523910701274872, 0.25720274448394775, 0.2527484893798828, 0.2556384801864624, 0.2602986693382263, 0.2574406862258911, 0.2603680193424225, 0.26485148072242737, 0.26195162534713745, 0.26250946521759033, 0.2668892443180084, 0.2691396474838257, 0.2690891921520233, 0.2696110010147095, 0.2713521420955658, 0.2718883752822876, 0.2779497504234314, 0.2762731611728668, 0.2761240005493164, 0.2818540334701538, 0.27739429473876953], 'val_perplexity': [994.1739501953125, 876.3578491210938, 837.5892944335938, 777.56591796875, 778.8558349609375, 760.704833984375, 726.62109375, 694.4014892578125, 692.3436889648438, 678.4428100585938, 678.5372924804688, 667.333984375, 645.7101440429688, 621.7247924804688, 622.8552856445312, 616.0419311523438, 618.9719848632812, 612.962646484375, 598.2379760742188, 592.2567138671875, 588.1234741210938, 579.4490356445312, 581.990478515625, 569.9100341796875, 592.5892333984375, 560.5631103515625, 567.51171875, 559.2476806640625, 560.1895751953125, 545.0838012695312, 547.7377319335938, 539.724365234375, 542.5384521484375, 545.1838989257812, 527.25634765625, 548.71728515625, 548.906982421875, 515.399169921875, 533.12841796875, 521.0485229492188, 531.1770629882812, 527.72314453125, 529.5274658203125, 525.8560791015625, 514.7770385742188, 511.0679931640625, 520.0308227539062, 518.1978759765625, 493.80035400390625, 499.849853515625, 515.9962158203125, 498.47412109375, 509.8033142089844, 486.6273193359375, 500.94293212890625, 516.2655029296875, 512.3167724609375, 506.73455810546875, 503.47918701171875, 512.00927734375, 500.9512939453125, 500.48284912109375, 494.9143981933594, 500.2843322753906, 490.3448486328125, 491.03204345703125, 491.4932861328125, 493.34375, 501.9366455078125, 508.0466003417969, 505.25262451171875, 494.6268005371094, 493.3270568847656, 494.25994873046875, 491.05194091796875, 489.19769287109375, 505.8294982910156, 484.43853759765625, 496.36053466796875, 507.41033935546875, 509.8052673339844, 488.99407958984375, 505.41094970703125, 501.3589782714844, 492.4522705078125, 484.7985534667969, 477.9009704589844, 503.37091064453125, 484.39581298828125, 498.46484375, 494.7386169433594, 498.9402160644531, 512.2273559570312, 502.9594421386719, 494.31793212890625, 492.8107604980469, 505.24371337890625, 495.5299987792969, 495.2767639160156, 514.4175415039062, 502.029052734375, 494.3580017089844, 489.45550537109375, 516.2955322265625, 514.7772827148438, 494.9618225097656, 504.0408020019531, 508.27508544921875, 498.1816101074219, 514.5868530273438, 521.5125732421875, 518.3923950195312, 530.125, 509.9667053222656, 501.17205810546875, 513.8950805664062, 518.5508422851562, 521.5431518554688, 515.4443969726562, 516.7610473632812, 506.02874755859375, 502.58880615234375, 509.67401123046875, 519.8372192382812, 516.1596069335938, 515.4819946289062, 525.3838500976562, 521.1578369140625, 521.3634033203125, 514.919677734375, 525.5860595703125, 525.834228515625, 521.9544067382812, 534.5213012695312, 509.408203125, 526.82080078125, 520.709716796875, 520.6099243164062, 544.441650390625, 534.8121948242188, 516.5853881835938, 503.6734313964844, 504.61578369140625, 525.3656005859375, 512.491943359375, 516.7979736328125, 499.7357177734375, 504.9286804199219, 495.0576477050781, 506.5410461425781, 508.55340576171875, 503.7476501464844, 484.65478515625, 487.21868896484375, 480.9778747558594, 481.0989990234375, 449.6778564453125, 438.706787109375, 430.047119140625, 410.3688659667969, 414.030029296875, 390.34576416015625, 392.32037353515625, 394.5762023925781, 376.2539978027344, 360.01495361328125, 348.14935302734375, 336.69744873046875, 321.725830078125, 304.5982360839844, 287.2310485839844, 279.8775634765625, 271.00677490234375, 275.8238525390625, 269.7674865722656, 259.0009765625, 245.206787109375, 251.6451416015625, 235.23838806152344, 229.15982055664062, 218.64642333984375, 220.01365661621094, 213.8594970703125, 209.57582092285156, 214.0057830810547, 207.53578186035156, 197.9703826904297, 197.49111938476562, 195.0354766845703, 194.4865264892578, 190.6980743408203, 186.3846893310547, 184.6108856201172, 180.79010009765625, 185.75244140625, 177.589599609375, 175.6230926513672, 175.3957977294922, 168.96612548828125, 171.89002990722656]})


# ViT Imagenet-1k
data = database.get_data(model="ViT B16", dataset="ImageNet-1K", initialization="Sinusoidal 1", optimizer="SGD")
data.update({'train_metric': [1.9442430221821199, 5.243422598302954, 7.82271163712459, 9.921735417787064, 11.794325017737734, 13.485205285493617, 15.081562356819994, 16.684007627420936, 18.234390988840644, 19.82231824578685, 21.342026449323157, 22.77798288591573, 24.16695091272254, 25.5161114827341, 26.8139126280961, 28.094932198534618, 29.29992733187789, 30.521157663286676, 31.707263768111417, 32.79494398466398, 33.85436871227561, 34.94259530568615, 35.95066060864821, 36.94693978224541, 37.889752077598004, 38.78666871688078, 39.661418066497184, 40.538821246566606, 41.36626997104983, 42.13720771765117, 42.82041295162926, 43.57847181514978, 44.25098367347895, 44.919827001476, 45.494693509901516, 46.12591488853522, 46.65910064808101, 47.20165286804921, 47.6780154343657, 48.20214694883649, 48.67819729980557, 49.06432963071949, 49.45990647589268, 49.8572785593135, 50.305541744362756, 50.63898773540061, 50.98281488673998, 51.354116988651754, 51.66976670488703, 51.96114167786089, 52.2911533000772, 52.5539605687627, 52.86024382457556, 53.10681589519555, 53.42582192641552, 53.706347416066755, 53.99210251278717, 54.24788493615586, 54.43396528321444, 54.83063488210358, 54.97480031877187, 55.19631710776191, 55.530933906352566, 55.74667471141545, 56.05803146662379, 56.30991119814981, 56.54485324707864, 56.829515590083105, 57.06078910867982, 57.44169183252457, 57.62870882562539, 57.92148876766261, 58.21426870969983, 58.55622256895471, 58.8210592373984, 59.0449956953309, 59.413409805279095, 59.72718622942989, 60.007789772917974, 60.362466407579966, 60.67351094744089, 60.95708053672941, 61.2777256985233, 61.640207716870634, 62.010417065066456, 62.33957009507738, 62.679884823758336, 63.00380824670008, 63.37862277127026, 63.769672493905944, 64.11560709884036, 64.42688580021183, 64.79662682538654, 65.26120326233817, 65.58489252376934, 65.94940394187486, 66.37378265284697, 66.74875328509086, 67.11513799528086, 67.57354817912106], 'train_loss': [6.409100566426502, 5.747913364668792, 5.391068612739002, 5.139914235449902, 4.936877873020219, 4.765164879415352, 4.612804800392572, 4.469683577746527, 4.330406943998694, 4.199330809709582, 4.0785695889701294, 3.9673305945985, 3.862518555718206, 3.7640041742263377, 3.671723886638786, 3.5827914112496373, 3.498620959677368, 3.4169146840737543, 3.3395147641833307, 3.2647288316041356, 3.1934187322225274, 3.125148635649218, 3.060006196446826, 2.996968879102559, 2.9368840420935096, 2.8794079246837314, 2.8245026332002117, 2.7717901717902502, 2.7217586832426663, 2.674708508505522, 2.630077986916353, 2.585261332481985, 2.544178720376632, 2.5046116694792375, 2.4677649733724754, 2.432406350374043, 2.3985373598753115, 2.366861769616994, 2.337976608137501, 2.3076793687720776, 2.280819665862223, 2.255414301951667, 2.2310561685987547, 2.206688184642644, 2.185595318078816, 2.163647519011164, 2.1440062020302437, 2.1243477545213767, 2.1057831443828734, 2.0884392340704068, 2.071114135631714, 2.054120846832958, 2.037774777024161, 2.022923646716842, 2.0074352583177046, 1.9929084099936756, 1.9785167204165972, 1.9622122781296591, 1.950462190818701, 1.934763825810916, 1.9221058247585898, 1.9095386740889688, 1.8948923764787857, 1.8818996991303258, 1.867624529824461, 1.8539043863405982, 1.8410304855398796, 1.8255104565157139, 1.8138076496967468, 1.7977935135039995, 1.7836370085066888, 1.769886248285248, 1.7563528191662792, 1.7387464019741186, 1.725507838649584, 1.7109445556875436, 1.6965492630917158, 1.6805226817653398, 1.6652724341505611, 1.6494208243684136, 1.633254432983109, 1.6174753717570116, 1.6022500819727354, 1.5844025304281864, 1.568018951945517, 1.5499772978380395, 1.5336850756912646, 1.517070015429307, 1.4984500315218063, 1.4801395281824035, 1.4632558226853354, 1.4460379429568337, 1.4283913938105597, 1.4079783791053235, 1.391232589663275, 1.3729846052871229, 1.3545003850248418, 1.3358764853967677, 1.3172591054134064, 1.298040980654131], 'val_metric': [3.41, 5.576, 7.774, 9.266, 10.658, 12.324, 13.296, 14.674, 15.1, 16.488, 16.614, 17.636, 19.754, 20.37, 21.842, 22.288, 22.602, 23.12, 22.772, 23.684, 25.12, 24.734, 26.406, 25.69, 27.282, 27.092, 26.772, 28.05, 27.038, 25.63, 25.668, 27.466, 28.416, 26.34, 26.136, 29.4, 28.118, 25.854, 26.828, 29.374, 28.354, 27.506, 28.738, 28.836, 28.842, 27.616, 30.124, 28.99, 27.706, 29.362, 29.516, 29.224, 28.498, 26.064, 29.564, 30.468, 28.77, 27.964, 28.374, 29.532, 29.574, 29.714, 29.75, 29.116, 29.688, 29.854, 26.116, 27.984, 29.238, 28.538, 29.44, 25.748, 30.106, 29.198, 30.362, 29.99, 30.518, 31.394, 28.066, 30.464, 28.99, 29.262, 29.81, 25.676, 28.584, 30.542, 28.158, 27.772, 30.086, 29.342, 27.448, 28.08, 27.41, 30.146, 26.482, 29.676, 25.506, 28.874, 30.924, 26.87], 'val_loss': [6.049753085731545, 5.722221225119003, 5.3794207292444565, 5.188610702829288, 5.037124233020236, 4.869913463092521, 4.784419478205463, 4.636754936726807, 4.6090162175390725, 4.456842367225291, 4.4470362443753215, 4.374944662041677, 4.18368682593031, 4.132515075718961, 4.022074120657523, 3.9860490164183595, 3.964726779695667, 3.9256030382097835, 3.970991357513096, 3.8842675178252217, 3.772279520969257, 3.8114741428581347, 3.6838684734862173, 3.7350026330222255, 3.6331435381756414, 3.6113922491936425, 3.651188167083599, 3.5552533662989925, 3.6403274975926676, 3.734541555614118, 3.7355392408340484, 3.6301339183698227, 3.5277659775274794, 3.7011831102468777, 3.727673602092754, 3.4798497196734712, 3.5601213859475176, 3.794196374352326, 3.6895034903531796, 3.4991008656294755, 3.562060335255645, 3.6687010317049977, 3.54980827890851, 3.5670421381130852, 3.5531343413359675, 3.6514917482119387, 3.4536438805749046, 3.556509502937117, 3.674566738593304, 3.5236648442342764, 3.510934425818036, 3.5696306544953904, 3.610386730993495, 3.865711649265283, 3.539164151453301, 3.4852260250950713, 3.609718519365391, 3.7117435310197915, 3.649579138516465, 3.5809543741785963, 3.5786541558211415, 3.5988801476138326, 3.5735028516262997, 3.622869486089253, 3.578054544108603, 3.563352745588478, 3.9255617124878843, 3.7543242771340455, 3.631449131633315, 3.720762636731653, 3.6279315511360193, 4.0154536326827905, 3.5817043163892253, 3.6780260603903505, 3.605159254673192, 3.5970951477470607, 3.5659687325853824, 3.5252013511174476, 3.815503689372326, 3.60764159723316, 3.761608762166384, 3.7070515181707298, 3.664919960224415, 4.124032780794841, 3.8022934687145225, 3.6587818168732515, 3.8296824824398437, 3.8975574789602128, 3.6808631917476045, 3.7993585793944575, 3.979302325593236, 3.9091059557350394, 3.955305981704646, 3.7320347045312454, 4.111196508340519, 3.8127627478498023, 4.2165268481235065, 3.8691350303952348, 3.7245401423376845, 4.148658654802595]})
data = database.get_data(model="ViT B16", dataset="ImageNet-1K", initialization="Sinusoidal 1", optimizer="Adam")
data.update()
data = database.get_data(model="ViT B16", dataset="ImageNet-1K", initialization="Sinusoidal", optimizer="SGD")
data.update({'train_metric': [1.4848181384628234, 3.990580462968528, 6.08499906725665, 7.807959461959292, 9.31517905159905, 10.685882480582157, 11.934665816400203, 13.182668613849717, 14.403196460726821, 15.637930105911252, 16.832700186626724, 17.93154210184933, 19.020783395138963, 20.038761535381415, 21.046982946017184, 22.01797267647387, 22.922772753278846, 23.80048814869568, 24.74650065135927, 25.564504861583227, 26.421301828723344, 27.258741444323807, 28.01211707763313, 28.793045715351706, 29.53838180346512, 30.332033216590812, 31.028897872018245, 31.706951552764004, 32.39312283254252, 33.01786574271738, 33.63995482243923, 34.280386553821636, 34.88998701964693, 35.47367361163689, 35.9937463265913, 36.565490681542684, 37.07721163595378, 37.54241250360023, 38.034229729613706, 38.44908587248969, 38.99811656091673, 39.40547953545479, 39.84554706763443, 40.27882391600783, 40.60711835381336, 41.05382046212554, 41.41247784246706, 41.77730147592, 42.14415450913113, 42.49110381394463, 42.80831460691697, 43.17579207082293, 43.470679466455195, 43.7621324932659, 44.104554675541905, 44.34566297758216, 44.72898537036936, 44.97017172624646, 45.26880570604769, 45.54644320373534, 45.80886020323658, 46.111318821043625, 46.38146315039335, 46.67869216113122, 46.962652019604, 47.25551001547807, 47.561715217454086, 47.80906782644261, 48.090920231320354, 48.36239147589659, 48.60420226246851, 48.909626926075994, 49.24799030883562, 49.52914022918168, 49.83776510010014, 50.162000738389295, 50.43682829795023, 50.795251516781185, 51.092870796703316, 51.42467765716725, 51.74235677316072, 52.09110131622185, 52.44125082834634, 52.74207031557947, 53.05451982450375, 53.401313021643546, 53.79447019787428, 54.15039569392593, 54.48977377656465, 54.88074544536349, 55.27936639017396, 55.59946517510988, 56.08706749393327, 56.47374620170516, 56.819758860476426, 57.26177773857741, 57.67156038205792, 58.104837230431315, 58.542719255179065, 58.995353455092115], 'train_loss': [6.42693154745999, 5.867115647412065, 5.5475926722457904, 5.325077336784533, 5.149656957736578, 5.002747146142066, 4.871801987544206, 4.749256875954568, 4.63002423977148, 4.517564286932709, 4.412610981918023, 4.3162671911698, 4.22699177874344, 4.143164826854315, 4.064810497395505, 3.9902756600370655, 3.9200006613537735, 3.8526411793485518, 3.787873722717439, 3.7248927555072795, 3.6657494696068285, 3.608128387781878, 3.5523594659110445, 3.4982629633728717, 3.4458106968737403, 3.3951852112611016, 3.3459413164151086, 3.299848955431103, 3.254743146529546, 3.211364248617229, 3.1699922357419434, 3.12980716346033, 3.091110745809719, 3.0536244839863067, 3.018252298342335, 2.983383534605957, 2.950623267829177, 2.9178326152422502, 2.886960386092551, 2.855725567332157, 2.8269293390732138, 2.799044976427848, 2.7721457257304873, 2.7464540960357717, 2.7202863669834914, 2.696359382159727, 2.672491962285992, 2.6498364783695836, 2.627864761657187, 2.6069641515707134, 2.5853937224578627, 2.565836443119315, 2.545542971482494, 2.5267244543108145, 2.507431402282166, 2.4892464462970842, 2.4715150656581755, 2.4526154494165295, 2.435906641341225, 2.418476737571576, 2.4014748735166798, 2.3853944251692027, 2.368609312477571, 2.350747466569681, 2.3339615305008734, 2.3166067267614703, 2.300013570585679, 2.28340360639431, 2.266792447579253, 2.2499310433182766, 2.2330746808617294, 2.2150708844155673, 2.198225708850775, 2.1805622291300546, 2.1634807728394336, 2.145984971652504, 2.128102472650343, 2.109576345948979, 2.0910130226264925, 2.0730720571159416, 2.0548225390136383, 2.0361677215163527, 2.0158974690015476, 1.9973070879095447, 1.978815550588336, 1.959335073068715, 1.9386921604915042, 1.9188553846717875, 1.8983201586323022, 1.878323967547688, 1.857214576417115, 1.8363665285551845, 1.814291076161977, 1.7930735696314264, 1.7716022278022872, 1.7496186742964095, 1.7272704358349564, 1.7049199805684163, 1.6824933061936058, 1.6593321939348145], 'val_metric': [2.604, 4.698, 6.452, 7.518, 8.758, 9.804, 10.524, 11.892, 10.188, 13.898, 14.722, 15.412, 15.744, 17.268, 16.282, 18.8, 19.562, 19.596, 18.12, 21.112, 21.294, 22.424, 21.882, 23.874, 22.7, 23.954, 23.952, 25.192, 24.956, 22.202, 26.084, 26.11, 25.458, 27.584, 27.336, 26.704, 26.616, 22.48, 27.922, 29.206, 27.938, 29.64, 28.94, 29.506, 28.668, 28.0, 29.89, 30.162, 30.262, 29.69, 30.418, 30.094, 26.828, 31.434, 31.548, 30.628, 30.602, 27.564, 29.698, 30.692, 28.31, 30.98, 31.006, 30.454, 28.582, 28.538, 31.134, 30.344, 29.118, 30.004, 29.674, 30.27, 28.646, 30.206, 31.594, 32.1, 28.844, 31.148, 27.524, 30.84, 25.42, 30.84, 31.314, 30.644, 30.59, 29.932, 32.968, 28.606, 30.752, 30.704, 29.23, 30.328, 29.776, 31.748, 30.59, 29.316, 30.194, 30.93, 30.674, 29.3], 'val_loss': [6.137430698365507, 5.760170145412845, 5.51728729610248, 5.371489239775616, 5.221977804322987, 5.102518867043888, 5.020766393150515, 4.8549745422799875, 5.115228084347132, 4.670808521225629, 4.594284750006693, 4.52445015395084, 4.490608370974851, 4.354365984039843, 4.450079484821281, 4.2257192767489595, 4.167063633346801, 4.180728347100261, 4.2957592132451285, 4.037167243640441, 4.029268819200413, 3.933468278411709, 3.9916938743780337, 3.831351495993412, 3.9353985699546308, 3.825158660445372, 3.817147108447521, 3.7328363353639005, 3.7551341796165234, 3.965360301534843, 3.666970737907283, 3.6810572380604953, 3.7298721801441954, 3.5613665932889487, 3.5656726282575857, 3.6433183138480274, 3.636495565743093, 3.9900402935874433, 3.5471436997203876, 3.453232196743226, 3.5549663724496847, 3.4323248651326463, 3.476762203761684, 3.441352955673052, 3.5227119331926944, 3.543602757517944, 3.4189907079538724, 3.385039970583623, 3.3640942238557066, 3.4232432401698567, 3.3772662533518605, 3.4100951755519415, 3.6589873356892326, 3.3093936963535637, 3.305480361670789, 3.3896238732215997, 3.3901078625560723, 3.6206478561891617, 3.4597452706693077, 3.3830632995766448, 3.54134375855441, 3.3615403631916436, 3.3558454353105076, 3.396384011677769, 3.549014110089568, 3.5412612155727716, 3.3540222060954785, 3.4057594059068528, 3.500238147919135, 3.4504853280456476, 3.4990191764538854, 3.419537429385783, 3.555256797986872, 3.4552639031501684, 3.352825117499932, 3.3498166147476574, 3.560063503511117, 3.401792142549744, 3.6931627831800515, 3.434106746674194, 3.864661352256375, 3.441850828773835, 3.4019701017062074, 3.4472024044036256, 3.4545808364363277, 3.5478219251955867, 3.2948881852276184, 3.6196499752723956, 3.4677008201399118, 3.486701083915008, 3.636418555703614, 3.526389176850124, 3.584401574547943, 3.4326343699870514, 3.538797756899958, 3.659571674092651, 3.588239547732236, 3.548968314362304, 3.54280321620157, 3.6822303948576187]})
data = database.get_data(model="ViT B16", dataset="ImageNet-1K", initialization="Default initialization", optimizer="SGD")
data.update({'train_metric': [1.6572390640720531, 4.498867048558073, 6.589070745656109, 8.321241493107456, 9.958186559597618, 11.44550242083975, 12.943667765404511, 14.420056089487163, 15.89269782940085, 17.31398014466498, 18.64737383963215, 19.971323020340048, 21.205120019482237, 22.418779128716242, 23.592396619644433, 24.7073956790957, 25.832385629664206, 26.948165227484004, 27.996116041078174, 29.06225339865919, 30.06969427092643, 31.03740574023527, 32.02400623806264, 32.955188511724074, 33.84039707547884, 34.72006381681701, 35.578656022204754, 36.42171551405867, 37.207873758846425, 37.995280865023844, 38.713532271749116, 39.47728906535994, 40.15229864646842, 40.86633514600361, 41.59855818952564, 42.177951820488666, 42.84585850244348, 43.50424261630217, 44.1225070580182, 44.7651242968325, 45.30806678598496, 45.88769457845854, 46.45163354972459, 47.00831351416326, 47.57053530101852, 48.09575956920526, 48.60404615479481, 49.199909145333905, 49.65184086071527, 50.28829184641815, 50.822414252006176, 51.33803789825995, 51.83672386191652, 52.44195331287803, 52.96085522027963, 53.47374698224353, 54.08857705513801, 54.651033003503834, 55.29880179555046, 55.80662005811889, 56.4746828477474, 57.094976689221625, 57.67905355039585, 58.34532110177674, 58.98528451013802, 59.6750462664118, 60.34365543289829, 61.07829814536278, 61.7520588650816, 62.41067714045086, 63.17396561104056, 63.89744662483501, 64.63154296044154, 65.38616745514051, 66.15328056373603, 66.89159180653263, 67.67447179017255, 68.36961926118921, 69.14742574543365, 69.95645376441946, 70.68235444715638, 71.55132781284563, 72.23585996205023, 72.92569977216085, 73.66104496915703, 74.46624835013702, 75.1595225290692, 75.69567433441541, 76.40416901153401, 77.11664443433213, 77.66247491544819, 78.30704350018382, 78.89221311507399, 79.45779121691396, 80.02282294189594, 80.50113685413378, 81.02714166068904, 81.42443569027301, 81.83304752620072, 82.20622292019698], 'train_loss': [6.476288809548356, 5.86359729435282, 5.52434337236193, 5.286257179105333, 5.091158416055765, 4.923162814161805, 4.769300705307912, 4.62239494326827, 4.484345298718784, 4.357224579475293, 4.241376134171293, 4.134677172481278, 4.034102187146435, 3.9403307361079714, 3.8514335503724197, 3.766603357016942, 3.6846637708084753, 3.607047016967563, 3.530884660465197, 3.458101501164245, 3.387930537682622, 3.319579656750918, 3.2540424048016314, 3.1912864474448965, 3.1309630585107433, 3.0725136926180237, 3.015781530592431, 2.961980855109433, 2.9095375970012687, 2.858969041616256, 2.8106006942218618, 2.763052405644811, 2.7178628453722937, 2.6726068705670203, 2.630430708202343, 2.5892814327545164, 2.5491144498082106, 2.509227425957174, 2.471992059445678, 2.4350136128557605, 2.3994139280047198, 2.3644408577299183, 2.3293417777278647, 2.2969411326202636, 2.263622977194226, 2.2316637487760214, 2.1998735667585083, 2.1675274430755684, 2.1382302373872486, 2.1059035810058697, 2.0757349425485496, 2.0446902733117325, 2.0143415326391865, 1.9825555434313453, 1.9525983859813978, 1.922631519539527, 1.8905255324274006, 1.8591465071916307, 1.8280071074285316, 1.7961631140882088, 1.7645872121204047, 1.7329547021098295, 1.6988491247932813, 1.6661735263831419, 1.6333883444063253, 1.5996902667503685, 1.5658270402312893, 1.5313307186470944, 1.498474746223477, 1.464018446822476, 1.4284279771549993, 1.3944673743410672, 1.3601572875453969, 1.3261932656775584, 1.2905952435308203, 1.2573762566835243, 1.2232515267829414, 1.190487734176603, 1.1573708419813142, 1.1229900176264653, 1.0919615873176798, 1.0578982352829343, 1.0279307979894057, 0.9984223824622445, 0.9683006764071597, 0.9367252578327566, 0.9103172855486409, 0.8860788479989996, 0.8588260618550263, 0.8338775691246973, 0.8109102066139389, 0.7868274563428578, 0.766169686305296, 0.7439561513130157, 0.7243023010375432, 0.705141805435038, 0.6871740598388054, 0.6721494547749942, 0.6572047963740675, 0.6434738324312452], 'val_metric': [2.8, 4.992, 6.486, 8.222, 8.698, 10.952, 12.07, 12.806, 14.408, 15.18, 13.866, 16.52, 18.342, 18.912, 19.102, 19.74, 20.482, 22.204, 20.642, 23.268, 21.444, 23.754, 24.298, 23.026, 24.324, 24.902, 24.886, 26.224, 26.032, 24.496, 25.018, 26.406, 25.658, 26.75, 25.102, 26.64, 24.904, 26.264, 27.444, 24.694, 27.438, 26.862, 24.708, 25.48, 28.01, 27.226, 26.198, 23.492, 28.136, 26.088, 27.792, 27.542, 25.35, 27.294, 28.604, 24.642, 26.82, 27.194, 27.398, 25.966, 26.858, 26.238, 26.834, 26.418, 25.64, 26.556, 22.938, 26.388, 26.918, 27.462, 27.794, 27.434, 24.5, 24.468, 26.928, 26.654, 24.95, 24.746, 26.846, 26.112, 27.384, 26.806, 27.024, 23.738, 26.798, 24.714, 24.092, 23.498, 26.172, 24.028, 23.482, 23.39, 24.57, 26.134, 22.292, 26.608, 26.114, 24.642, 24.276, 25.064], 'val_loss': [6.167014826594106, 5.7528502474660455, 5.499846223370193, 5.299615209974596, 5.195802359020009, 4.966810856938667, 4.827435275020502, 4.770233704001092, 4.611100134642228, 4.539265552750024, 4.652866247060049, 4.412345983030851, 4.250706343089833, 4.210803879496386, 4.206526228229103, 4.1424791308315205, 4.072979085738091, 3.9404445654900786, 4.076530134129097, 3.8708967555057057, 4.021124087636124, 3.8200675433554, 3.787177972019176, 3.8907571197928066, 3.820375049891679, 3.764686340108857, 3.7734832041098945, 3.679278037279768, 3.6780031702250167, 3.809821614828866, 3.7531627468440845, 3.677042907918506, 3.730477920502348, 3.6526390112498226, 3.7991436631478313, 3.6803550242310594, 3.825645943889228, 3.7169510350202963, 3.635005965485902, 3.8763280732704857, 3.650513649825245, 3.719941999372619, 3.8975749221604192, 3.8411875176612678, 3.6211913793212007, 3.7091483267219476, 3.8084516353009605, 4.021807071421763, 3.6619242995291414, 3.887841653259819, 3.6952416617852037, 3.722867168581394, 3.9313038642449145, 3.779046294481858, 3.6814584507204384, 4.077970112681084, 3.8537210161271305, 3.854450328842453, 3.807585816249213, 3.9617776308218233, 3.899377594442319, 3.96199003883335, 3.948987122615585, 4.023263294754735, 4.099033301748583, 4.006751053473529, 4.402179132946921, 4.107772527638908, 4.0359690993490736, 3.9456303478278163, 4.004807732827828, 4.025865400424394, 4.374439659402194, 4.363301507652263, 4.15252143883949, 4.253964060560212, 4.380665136496429, 4.4149533670271754, 4.265102017489846, 4.380813432052312, 4.243182162342169, 4.326049333033354, 4.336707536255003, 4.751693486252709, 4.37345927252489, 4.682277965385591, 4.806436913702494, 4.810553981329474, 4.583276780136406, 4.92187507587778, 4.935959349202988, 4.902598917636725, 4.831192340692291, 4.714507226139078, 5.209043311798359, 4.647836658503393, 4.708745172414024, 4.939059478471346, 5.0196861530585055, 4.915119174000857]})
data = database.get_data(model="ViT B16", dataset="ImageNet-1K", initialization="Default initialization", optimizer="Adam")
data.update()
data = database.get_data(model="ViT B16", dataset="ImageNet-1K", initialization="Orthogonal", optimizer="SGD")
data.update({'train_metric': [], 'train_loss': [], 'val_metric': [3.79, 5.98, 7.64, 9.03, 10.57, 11.70, 12.71, 13.39, 14.53, 15.23, 16.65, 16.74, 17.76, 19.10, 20.01, 20.66, 19.45, 19.72, 22.42, 23.13, 21.29, 23.49, 24.35, 24.99, 24.46, 23.95, 25.28, 25.94, 25.62, 25.50, 25.77, 26.28, 25.51, 26.27, 25.31, 28.08, 26.84, 26.17, 27.04, 28.15, 28.06], 'val_loss': []})
data.update({'train_metric': [2.1680233724409073, 5.398983895151842, 7.787041033682573, 9.726834987164047, 11.492412776788662, 13.003457004434239, 14.496158580419259, 15.91884586474675, 17.32162942067662, 18.731515875760145, 20.095428620937007, 21.430227284967533, 22.71998888513363, 23.864726456426055, 25.01906464965145, 26.140854392908967, 27.24063295417381, 28.28358832220936, 29.260197928919492, 30.215732999679197, 31.20280182052769, 32.0775511701441, 32.98734669250769, 33.81120494049566, 34.670109361230814, 35.44807195314897, 36.16273288337898, 36.90221493372839, 37.540226996168336, 38.24739475806042, 38.82538341996008, 39.38760520681535, 39.94202160998527, 40.52360074838019, 41.0049587602553, 41.4452604539455, 41.87065386479671, 42.323053903199195, 42.748993690908364, 43.14488275142897, 43.51969727599915, 43.85657763585856, 44.252856965563424, 44.532367755335564, 44.870418922747774, 45.15851563457379, 45.43716783214054, 45.77771672233206, 46.0379482144014, 46.30223850598712, 46.56855819733103, 46.89521350456263, 47.09932428793436, 47.332236937105, 47.6314172937642, 47.90351296903526, 48.11613162062401, 48.334370148466206, 48.591479487061406, 48.770456934966326, 49.06503211525117, 49.33541060611146, 49.49261103353427, 49.75331084862473, 50.00854689513545, 50.339261001883436, 50.48810966876293, 50.70798732717905, 50.94527099121348, 51.181227740021406, 51.478768966106685, 51.7247946598687, 51.99009965133351, 52.183751220566876, 52.49456159891724, 52.72802062494585, 53.03243058867423, 53.23021901126083, 53.56928487855213, 53.803836658296696, 54.08108388680008, 54.36746341421532, 54.575242727919154, 54.93928582300356, 55.239012556520734, 55.546622727560106, 55.777505976972556, 56.11430828299511, 56.37906689760195, 56.675515369971286, 56.992648109106774, 57.350681058753466, 57.64759785414392, 57.95138338717747, 58.26195960401728, 58.637554666955985, 58.92892963992985, 59.27072739151102, 59.67356324351158, 59.94987382597272], 'train_loss': [6.349180839523711, 5.711829903040133, 5.388201759032921, 5.1583148662464815, 4.973844844398515, 4.816144451584585, 4.676409729342044, 4.547790440963314, 4.425583082179374, 4.305357136469846, 4.189455197892279, 4.082218566756189, 3.9852899541578077, 3.8946694349622124, 3.808382190103339, 3.727972474379987, 3.6511413301583846, 3.576939767685748, 3.5067198409907463, 3.4402391497884826, 3.3758403892741944, 3.3150711014909255, 3.2563700801134408, 3.2001343792305046, 3.146516701076955, 3.0950653984613474, 3.046575244784659, 3.0013612564451253, 2.9572710693641775, 2.9156740232688025, 2.876426615172663, 2.8389143318472296, 2.803872588208532, 2.769949202384141, 2.73897143450345, 2.709769071491277, 2.681206211783418, 2.6539133267479107, 2.6285511276761135, 2.6038858746784395, 2.579913794687682, 2.5587334365621777, 2.537765930187119, 2.516234319898071, 2.497241503500761, 2.4777072602632226, 2.460075066082365, 2.4414394868885676, 2.4255480849782787, 2.4081353822414964, 2.3919884180044866, 2.375879066629637, 2.3608819594767443, 2.3459587013443377, 2.3304007915690907, 2.316004705801848, 2.300277947837629, 2.2873742304110753, 2.2730246041524813, 2.260000477227079, 2.245061968684024, 2.230865529905613, 2.2179384330774474, 2.203213707723903, 2.189957078332471, 2.176582007446491, 2.1623634094720097, 2.14897141561885, 2.133238851562581, 2.120053049615168, 2.1051054679523893, 2.0913683901872124, 2.077010923608854, 2.063695389143421, 2.0484005217071273, 2.0339567722679464, 2.0186417794345504, 2.004039626163443, 1.988919734674958, 1.97417139714638, 1.959046715616769, 1.9436641951661062, 1.9290201796941797, 1.9124316726747264, 1.8966102831680116, 1.8797224308145208, 1.8642587208463224, 1.8483615340133237, 1.8331329698084808, 1.8152096315610242, 1.7982060870654315, 1.7812046886764745, 1.764338803312758, 1.7482443120972762, 1.7308132327563446, 1.712641290984994, 1.6951580038496568, 1.6767383988393438, 1.6586296972120145, 1.6424231223486254], 'val_metric': [3.792, 5.984, 7.638, 9.034, 10.572, 11.702, 12.706, 13.392, 14.526, 15.228, 16.654, 16.736, 17.758, 19.098, 20.01, 20.656, 19.454, 19.716, 22.422, 23.132, 21.294, 23.49, 24.346, 24.986, 24.464, 23.954, 25.28, 25.94, 25.616, 25.502, 25.772, 26.28, 25.512, 26.268, 25.306, 28.076, 26.836, 26.17, 27.036, 28.154, 28.058, 24.744, 26.254, 26.976, 27.564, 27.32, 26.538, 27.6, 27.158, 27.146, 28.472, 28.258, 29.42, 26.592, 26.7, 28.34, 28.692, 30.562, 24.274, 28.256, 27.378, 29.522, 27.296, 25.164, 30.384, 30.556, 26.326, 29.11, 28.086, 29.82, 27.02, 27.076, 28.932, 31.348, 31.142, 30.702, 29.218, 29.538, 28.64, 29.306, 32.1, 29.098, 28.868, 30.006, 29.518, 29.382, 31.576, 31.184, 27.948, 29.33, 31.032, 28.084, 30.09, 30.172, 28.388, 28.634, 31.236, 30.216, 29.33, 30.066], 'val_loss': [5.993627945480444, 5.613727226891481, 5.3960519105272216, 5.207147121734327, 5.055323584305356, 4.927729014088126, 4.8194047505288475, 4.756034408079084, 4.636586379669511, 4.578407862454729, 4.439479538851686, 4.450469581519856, 4.358438085564567, 4.268690239209349, 4.179830668984777, 4.114387385832989, 4.209833215264713, 4.191883410944048, 3.9717989631778443, 3.9222504822037103, 4.066497229859042, 3.8813247800330677, 3.837218986371594, 3.7842889294752378, 3.8280990146615013, 3.85266462959292, 3.7496008192334336, 3.7006478787535597, 3.7290288504127345, 3.7398651283415383, 3.7420094555906016, 3.6887863346225465, 3.7672241035934606, 3.692371000795413, 3.7798456904833273, 3.536983875210023, 3.656470607156339, 3.694385695335505, 3.613460941159207, 3.551929238507205, 3.5774691243610723, 3.8607203986333762, 3.7188682425052613, 3.646341070952013, 3.589046342141183, 3.6255496291019726, 3.675987423838252, 3.6299659046523103, 3.657564484516678, 3.6358284310764057, 3.5456499686021634, 3.5680875713410587, 3.4764998700765086, 3.6821025574527435, 3.7012914953481815, 3.583059431265687, 3.5215533221774087, 3.406721204824155, 3.927083231108573, 3.5832876200261325, 3.6557562164485913, 3.4909542039837067, 3.6320237736110492, 3.867243149320183, 3.4090623742021866, 3.4263530316407724, 3.739158524157446, 3.5563489911348922, 3.615549739364468, 3.478229104786578, 3.692392100611001, 3.6847821804111267, 3.540778118935997, 3.374732584828306, 3.3892440649364, 3.4270455414224466, 3.534747314956182, 3.5030146608572177, 3.588657050105312, 3.548780814232424, 3.333069984489085, 3.575797503256737, 3.613020193584435, 3.507206719282948, 3.541031176324391, 3.553735787968349, 3.3848662396602314, 3.4175022830591177, 3.6713545817472135, 3.605759372796549, 3.4480787535076556, 3.7233497552249744, 3.5651961124080525, 3.5240621559912593, 3.6723090510843965, 3.6661217942109805, 3.4871329403746767, 3.583801860394685, 3.648890246134585, 3.596668245161281]})
data = database.get_data(model="ViT B16", dataset="ImageNet-1K", initialization="LSUV", optimizer="SGD")
data.update({'train_metric': [2.0674119767368344, 5.124078281754057, 7.374916775096455, 9.220890016680105, 10.869387051024574, 12.440298571536731, 13.892412152357968, 15.319080182364985, 16.746762912251096, 18.208945438026426, 19.633349906764693, 20.989613375929913, 22.4042611150615, 23.688637000484714, 24.996429036963956, 26.25949622492618, 27.46347665839036, 28.688843843152377, 29.86410046465449, 31.014770127547774, 32.15935159116649, 33.2766142118865, 34.35563045254834, 35.48054234927999, 36.491339536531925, 37.58323466027458, 38.64898174867133, 39.640187422873055, 40.64146204202887, 41.61518365677542, 42.556434875390956, 43.53195172838514, 44.47421764687976, 45.39470654489227, 46.32424968797979, 47.24278724007097, 48.10817012926496, 48.882230029340434, 49.769546046690245, 50.58598918017714, 51.34451636671878, 52.1981131265479, 52.924794347653354, 53.674423396793706, 54.341393432706276, 55.03092102746949, 55.72926870579714, 56.33699587953795, 57.04517834130913, 57.63284567897862, 58.309338282987305, 58.927602724703334, 59.48155080485214, 60.05821255152529, 60.65813434158076, 61.18304639442009, 61.762361971546255, 62.37773842129871, 62.88305896108782, 63.5285641918657, 64.03146506271236, 64.62287898455081, 65.16449455847676, 65.81288778121822, 66.3799489059584, 66.95153715323607, 67.55419082758142, 68.13662855818173, 68.69057663833053, 69.28776654409613, 69.89471317946841, 70.48495629375405, 71.15411183709853, 71.69838124147749, 72.348881917814, 72.84085525150117, 73.41837559037971, 73.9834073153617, 74.56998189931524, 75.12463246399571, 75.72713003066735, 76.24821744550086, 76.73457090293459, 77.30280283522757, 77.77229666390096, 78.23242403215194, 78.74484747109472, 79.1221597184442, 79.66229226946994, 80.03812149391922, 80.46023664362258, 80.83309982227141, 81.24381911179417, 81.54307752229022, 82.01171275875822, 82.24415708490774, 82.6468368292346, 82.93212360293388, 83.20695116249482, 83.4352586352911], 'train_loss': [6.401968378663164, 5.752617833164047, 5.43102989683166, 5.209986747898144, 5.028422725540673, 4.87010096115001, 4.7270614919119875, 4.5933717067106254, 4.464176219709806, 4.33807146036516, 4.218308521142414, 4.10715823702548, 4.001833316883107, 3.9035051956574063, 3.8085926485869117, 3.7183771312424363, 3.6315657039212446, 3.548297558507753, 3.4685328812548923, 3.3908297067708237, 3.315740495397647, 3.242486323478535, 3.1713542307975606, 3.102367974237531, 3.036122077983054, 2.9698208630564356, 2.906141895610461, 2.8437237226615877, 2.7826759331864346, 2.7240484234300477, 2.66610345624771, 2.6092141257209422, 2.5540288557949835, 2.499917634616419, 2.4463406898108473, 2.3945584339876147, 2.34445521625357, 2.2949642104650927, 2.247059919187564, 2.200752548502366, 2.1552466396101884, 2.11068293760573, 2.0685904467878204, 2.025459626991853, 1.987290397468781, 1.9480635599571006, 1.910147217652843, 1.8735505187360788, 1.8380093476805537, 1.8032273044625007, 1.7698092651067066, 1.735242814770933, 1.7044234199953625, 1.6726027501380327, 1.6418653669500929, 1.6116389123833355, 1.5825189143259975, 1.5532308401353554, 1.5245301646186824, 1.493980127541155, 1.4673618531690396, 1.4375039894946824, 1.4111396410801211, 1.3820799210074064, 1.353900539401028, 1.3276743542821121, 1.2993509931916618, 1.2723012355617416, 1.2449067309904458, 1.2180788725386775, 1.1895038733693637, 1.1660003725766266, 1.137516783899203, 1.1130682715193436, 1.085489011524238, 1.0635972844118649, 1.0367558229618052, 1.0130240951044003, 0.9886001504020165, 0.9660588755859016, 0.9415018923204377, 0.9199339482171998, 0.900395039435694, 0.8771324651785215, 0.8578737591004787, 0.8396615717989729, 0.8189010554480323, 0.8015000008266583, 0.7834518002093543, 0.7683839047459481, 0.7520372377726884, 0.7369572245277923, 0.7217452540384067, 0.7085241627641667, 0.692331976551795, 0.6817633432506283, 0.668301567453129, 0.657492516529126, 0.6469702739570934, 0.637541565778545], 'val_metric': [3.464, 5.876, 7.328, 8.716, 10.078, 11.214, 11.984, 13.76, 14.104, 13.796, 17.052, 17.282, 17.224, 19.088, 20.22, 18.868, 22.104, 21.726, 22.894, 23.504, 24.022, 23.97, 25.938, 25.064, 26.666, 26.65, 25.562, 27.764, 27.418, 26.924, 27.668, 28.346, 28.234, 26.958, 27.472, 29.236, 27.858, 28.766, 28.89, 27.774, 27.732, 29.628, 27.806, 28.194, 25.202, 28.37, 28.32, 29.302, 28.262, 28.664, 27.1, 27.764, 29.748, 28.42, 27.324, 27.614, 27.572, 28.012, 25.796, 26.908, 27.738, 24.194, 25.356, 25.732, 26.036, 27.44, 26.75, 26.622, 24.85, 23.988, 27.098, 27.61, 26.534, 26.69, 25.274, 25.282, 26.15, 26.058, 21.754, 26.724, 26.118, 24.918, 25.26, 27.588, 24.41, 25.884, 24.976, 24.334, 21.054, 23.772, 25.294, 25.952, 27.23, 23.976, 27.222, 24.438, 27.39, 27.046, 24.834, 26.102], 'val_loss': [6.042204588270553, 5.635931872345907, 5.452552232595965, 5.257978891472682, 5.109950689708485, 4.996311070821474, 4.911820241100038, 4.721182163719021, 4.693283312460956, 4.727478833454649, 4.429247327777736, 4.387752500031611, 4.422143126845055, 4.245128632354005, 4.151907044146067, 4.2564211263681, 4.002879850547332, 4.030107531248761, 3.967841062475653, 3.914725957243034, 3.8323553475120185, 3.8566989193639487, 3.7219645698052233, 3.7864769154497426, 3.6631692753118625, 3.6593394631619955, 3.7435758103190175, 3.589058977182564, 3.6266390678217952, 3.6531161429632046, 3.598633327333214, 3.561761792282314, 3.562246732547155, 3.6545315425261817, 3.64301649715437, 3.49064335752936, 3.608694156188794, 3.560166467181252, 3.547404476443825, 3.641771668134748, 3.637591073770657, 3.5018305744966276, 3.660302069714612, 3.6246421465940792, 3.876703106533841, 3.6020690965682953, 3.620621248965373, 3.565880383676885, 3.638986076197356, 3.6296800104400995, 3.739058714045588, 3.7213950859921057, 3.566383616050796, 3.6780945743288833, 3.818506425375219, 3.7780652671976163, 3.8532663578038937, 3.7352816079126296, 3.993577081224193, 3.8670268995149057, 3.835846773232036, 4.170634306605209, 4.089488429943923, 4.009304739858793, 4.022499937611773, 3.900778122684535, 3.970899362355242, 4.004723727131439, 4.136788029576201, 4.307353112398816, 3.96248811170878, 3.947697336487758, 4.067333490418656, 4.088190444290181, 4.239388253206335, 4.276213335106744, 4.241121490898035, 4.211625484744911, 4.729830615127178, 4.17631343628287, 4.222674706250506, 4.355157951869623, 4.335419569326484, 4.155265122652054, 4.515007130820733, 4.311676573494206, 4.4112730646682214, 4.533143806411787, 4.893922688711025, 4.628955543917768, 4.491175058095352, 4.4503636142177045, 4.32540185871484, 4.664196774020525, 4.329417415592067, 4.5770770431022205, 4.346600204629971, 4.387038062162259, 4.588499327297406, 4.48429283133858]})

**Experiments 2**

In [4]:
'''
    data[model, dataset, initialization, optimizer]
    data[train_metric, val_metric, train_loss, val_loss]
'''
# ResNet-50 CIFAR-100
database2 = DataBase()
database = database2
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Default initialization", optimizer="SGD")
data.update({'train_metric': [1.79, 2.678, 3.778, 4.492, 5.288, 6.058, 7.014, 7.824, 8.646, 9.384, 9.92, 10.526, 11.148, 11.754, 12.414, 13.052, 13.252, 14.04, 14.444, 14.898, 15.468, 15.888, 16.112, 16.742, 17.132, 17.436, 18.08, 18.372, 18.77, 19.334, 19.688, 20.102, 20.568, 20.796, 21.17, 21.748, 22.108, 22.44, 22.884, 23.194, 23.696, 24.034, 24.572, 24.798, 25.294, 25.656, 26.12, 26.462, 26.622, 27.11, 27.536, 27.822, 28.278, 28.332, 28.904, 29.258, 29.522, 29.902, 30.282, 30.718, 30.804, 31.158, 31.458, 31.942, 32.184, 32.428, 32.768, 33.134, 33.578, 33.774, 33.878, 34.164, 34.306, 34.812, 35.194, 35.422, 35.828, 36.052, 36.366, 36.604, 36.812, 37.242, 37.436, 37.6, 38.052, 38.612, 38.792, 38.868, 39.24, 39.518, 39.69, 39.918, 40.364, 40.7, 40.712, 41.108, 41.492, 41.778, 41.982, 42.432], 'train_loss': [4.535412340822732, 4.455949684543073, 4.405216245700027, 4.3497567469506615, 4.283412515659771, 4.211402347325669, 4.1521698434639465, 4.106029899528874, 4.068188859678595, 4.022348077095988, 3.9761365091099456, 3.9303700131223636, 3.8855312983398242, 3.842918266115896, 3.79819005438129, 3.7585529271903857, 3.7198173868686646, 3.6868513377426226, 3.6575832095597405, 3.6285451800012223, 3.5969928914628677, 3.5736805478020397, 3.547397411997666, 3.5196932590830965, 3.4956761211385508, 3.4739957343586876, 3.4536229137264556, 3.428568032696424, 3.4033023829350384, 3.3821210382539597, 3.355537769434702, 3.3354763042591418, 3.3113507129957, 3.2921576134079253, 3.267921009636901, 3.241782242684718, 3.225925642206236, 3.2019958127185206, 3.1815551559028723, 3.15978949698036, 3.1387721028779167, 3.119266435923174, 3.0958608827932412, 3.0779993241400363, 3.0587440021812458, 3.037990520067532, 3.0132135878438535, 2.9963399373052066, 2.9731373994246773, 2.958611383157618, 2.9363809878868823, 2.9178257293408483, 2.8991317053889984, 2.8846351972321416, 2.8666497209797734, 2.8473451235105314, 2.8286209502793334, 2.808170647877257, 2.7970232966610844, 2.777208748375973, 2.764159523312698, 2.740538912051169, 2.7238344329092508, 2.710986950208464, 2.693632587752379, 2.672775658805047, 2.6640909833981254, 2.641766797855992, 2.6280071528061577, 2.6133269882568007, 2.6009282512433085, 2.587553584362235, 2.578983719086708, 2.5559852800100966, 2.539911763137564, 2.5247689160849434, 2.5136557055251374, 2.496059620319425, 2.4815180841309337, 2.4695960742128475, 2.452864079981509, 2.4356814151834647, 2.4296570604719467, 2.4186954879394884, 2.3975348625036763, 2.3837095183484696, 2.3648853382795973, 2.358502093788303, 2.3404551157561104, 2.3327849077446685, 2.315238912239709, 2.304606675339477, 2.287468734757065, 2.274900238227356, 2.261258733242064, 2.2484474293411236, 2.2332692501490072, 2.2213585329482624, 2.206185885707436, 2.1922477511188867], 'val_metric': [1.99, 3.78, 4.31, 4.86, 5.3, 6.66, 7.65, 8.31, 9.27, 9.9, 10.8, 11.22, 11.9, 12.24, 13.04, 13.12, 14.15, 14.7, 15.17, 15.65, 15.77, 16.14, 16.82, 17.01, 17.14, 17.68, 18.39, 18.7, 19.16, 19.35, 20.24, 20.61, 20.84, 21.0, 21.47, 22.27, 22.44, 23.15, 23.13, 24.0, 23.85, 24.22, 24.48, 24.93, 25.1, 25.96, 25.7, 26.16, 26.27, 27.09, 27.14, 27.34, 27.68, 28.24, 28.17, 28.41, 28.51, 29.35, 28.99, 29.23, 29.38, 30.44, 30.36, 30.9, 30.77, 30.96, 31.2, 31.67, 31.98, 32.07, 32.31, 32.09, 32.16, 32.55, 33.06, 32.89, 33.41, 33.75, 33.51, 34.12, 34.64, 35.18, 33.95, 34.68, 34.46, 35.02, 35.35, 35.8, 35.31, 36.17, 36.25, 35.93, 36.03, 36.56, 36.9, 37.1, 37.18, 37.18, 37.19, 36.52], 'val_loss': [4.4806275884057305, 4.421662093727452, 4.358965050642658, 4.308065012002447, 4.251326980104872, 4.168444407214025, 4.106009519783554, 4.09249243159203, 4.028292744023026, 3.9876250901799293, 3.946443750600147, 3.8914973553578567, 3.8563657915516263, 3.8049155253513605, 3.757296029169848, 3.7272586397304655, 3.695725796328988, 3.6591946744615105, 3.624963336689457, 3.605623694741802, 3.5832028237118085, 3.557924007913869, 3.530293060715791, 3.5066868788117818, 3.479446947194968, 3.45896376470092, 3.4352171512166407, 3.413954008916381, 3.393664797400213, 3.3633151646632298, 3.3451087201476857, 3.3246095636088375, 3.3030846483388525, 3.2846073345014246, 3.267856830244611, 3.237011066667593, 3.2166871022266945, 3.1908160288622427, 3.179186631160177, 3.1554180467204684, 3.1410194810029046, 3.132608513923208, 3.1120793956100563, 3.0877040313307647, 3.0689749991058544, 3.0483145273415144, 3.0365007439996026, 3.0126819458736738, 3.0088092734099954, 2.9928832934920195, 2.9742527509191232, 2.9537616231638912, 2.94869674239189, 2.919325571910591, 2.9157613128613513, 2.8964921243631156, 2.8924741896854083, 2.8650096708042607, 2.865809234084597, 2.8426800867554487, 2.8395947878527794, 2.814105232809759, 2.793270947826896, 2.7878472182401426, 2.786421420467887, 2.7730341382846713, 2.7468908273490373, 2.749423617769958, 2.7305480234182564, 2.7188138445471504, 2.702659915207298, 2.6965342403217485, 2.702859321217628, 2.6804341920621835, 2.6606690215457016, 2.670835195832951, 2.663306816368346, 2.6456717305881963, 2.6432528480602677, 2.6123523803273585, 2.6085332928189806, 2.584549326805552, 2.609908554964005, 2.5913208685103495, 2.5868023823780617, 2.5560528244941856, 2.5572109480572354, 2.5438417568328275, 2.541956892438755, 2.513151631993093, 2.5177618379046205, 2.5360440524520387, 2.5140773612222853, 2.4957565137535145, 2.4727642703208192, 2.4644485507041787, 2.4594284850320998, 2.4523996539935946, 2.448825950835161, 2.494603362812358]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Default initialization", optimizer="Adam")
data.update({'train_metric': [7.062, 13.702, 18.678, 23.912, 28.804, 32.754, 35.39, 37.874, 39.884, 41.87, 42.942, 44.102, 45.194, 46.03, 46.702, 47.634, 48.276, 49.152, 49.618, 50.376, 50.374, 51.162, 51.468, 51.514, 52.368, 52.626, 53.042, 53.068, 53.246, 53.67, 54.08, 54.144, 54.254, 54.466, 54.72, 54.992, 55.188, 55.306, 55.468, 55.868, 55.802, 55.972, 56.21, 56.378, 56.21, 56.608, 56.888, 56.836, 57.15, 57.164, 57.032, 57.222, 57.524, 57.448, 57.646, 57.596, 57.77, 58.124, 57.806, 57.968, 58.398, 58.228, 58.298, 58.078, 58.584, 58.336, 58.55, 58.632, 58.838, 58.768, 58.914, 59.098, 58.81, 59.174, 59.17, 58.972, 59.296, 59.334, 59.482, 59.476, 59.724, 59.58, 59.656, 59.682, 59.634, 59.73, 59.684, 59.608, 59.836, 59.686, 59.982, 59.988, 59.91, 60.05, 59.732, 60.024, 59.924, 60.006, 60.172, 60.23], 'train_loss': [4.09087639818411, 3.632722424424213, 3.348301089328268, 3.0531171002351414, 2.7915676954152335, 2.5829152394743526, 2.450406131841947, 2.33761358322085, 2.242562581663546, 2.167974682415233, 2.113817427012012, 2.060814641015914, 2.013711387848915, 1.976504072356407, 1.9343420244238871, 1.902173456785929, 1.874919677481932, 1.8480748921404104, 1.8244184629081766, 1.8002891557295915, 1.7832502330965398, 1.7632724654948926, 1.746461870267873, 1.7287723245218283, 1.7120838912246783, 1.7039126549535395, 1.6883338305651379, 1.681743830671091, 1.6674509703960565, 1.6576283043607727, 1.6429278916105285, 1.6413797588299608, 1.6316340085490586, 1.6209360588237147, 1.6153326016252914, 1.6048417954188783, 1.5982683038772525, 1.5874339800966366, 1.5842795298837336, 1.570242334052425, 1.5713218915492981, 1.5652131918446182, 1.5606228748855688, 1.5484206938682614, 1.5498837288993095, 1.5419134334530062, 1.5346324596258685, 1.5313275441184373, 1.5204426832211293, 1.5206193114485582, 1.5146359231923243, 1.512382415401966, 1.505515929073324, 1.5059667664110814, 1.4988155103552983, 1.4973546659854977, 1.4958513923313306, 1.4887796638109494, 1.4840452475925845, 1.4856613309639495, 1.4811178954970805, 1.4751611608068655, 1.4699196034990003, 1.475576211043331, 1.46565965824115, 1.4678791316269, 1.4598294088755117, 1.45920174566986, 1.4573608100261835, 1.458331454440456, 1.445279067205956, 1.4431511349690236, 1.452909814427271, 1.4445634030777474, 1.441307289444882, 1.4408730206739566, 1.4415677730232248, 1.4362271599604954, 1.4303676471533373, 1.4316286711436708, 1.424548994549705, 1.4266322576786246, 1.4175041521449223, 1.42030414969415, 1.4235947518549916, 1.4188642078043554, 1.42150062955249, 1.417757567313626, 1.4133142899826665, 1.4166633040856218, 1.4114536386926462, 1.4115549483719994, 1.409165711277891, 1.4102323647502744, 1.4075153248999126, 1.4082142366167834, 1.4050403744973186, 1.402817449682509, 1.3968501907327902, 1.3999227414198239], 'val_metric': [10.91, 13.59, 19.46, 24.89, 29.84, 32.87, 35.45, 36.12, 38.38, 42.87, 39.95, 40.73, 42.31, 45.17, 43.99, 44.2, 44.45, 45.7, 46.33, 46.28, 48.34, 48.11, 49.51, 50.19, 49.05, 49.0, 49.34, 48.56, 49.35, 49.86, 50.93, 47.49, 50.85, 49.17, 50.83, 50.12, 52.25, 47.87, 38.59, 49.85, 52.21, 51.47, 51.66, 48.99, 49.37, 51.45, 52.32, 51.35, 51.24, 52.09, 52.13, 52.3, 51.73, 54.22, 51.89, 53.37, 53.77, 53.8, 52.52, 51.92, 51.88, 52.37, 51.7, 54.04, 52.68, 54.49, 52.74, 53.78, 54.13, 53.25, 52.51, 52.9, 53.92, 54.94, 53.48, 51.68, 50.5, 54.18, 54.44, 55.19, 54.45, 54.63, 54.21, 52.15, 53.4, 52.01, 53.9, 50.32, 54.03, 52.63, 53.64, 54.56, 55.11, 54.76, 52.76, 54.06, 52.44, 53.96, 54.33, 51.23], 'val_loss': [3.785049619188734, 3.651114633888196, 3.318335493658758, 3.0407369774617967, 2.706539858678344, 2.5785837127904223, 2.42495624502753, 2.4207697271541426, 2.3118856454351144, 2.127967873196693, 2.2297561214228345, 2.1769644743318013, 2.152277097580539, 2.041285698580894, 2.070723667266263, 2.0844886219425565, 2.068405758803058, 1.9915061536108611, 1.9698859969521785, 1.9810881880438251, 1.8940853176602892, 1.8922974622932969, 1.831290682409979, 1.8376244906407253, 1.8538196375415583, 1.8566603698548239, 1.8338098571558668, 1.8685141444965532, 1.8662401589618367, 1.8022244751073753, 1.7590396761134932, 1.9131712207368985, 1.7876129810977135, 1.8360821441480308, 1.7771693969228466, 1.8091107113346172, 1.7080353354192843, 1.9182121533497123, 2.462887130725156, 1.8019321982268315, 1.7286158511593084, 1.733209260709726, 1.728140677616095, 1.891800106710689, 1.8462670851664937, 1.7227617745186872, 1.7322479949635305, 1.7491481926790469, 1.760860028540253, 1.7203218473750315, 1.7430742925899043, 1.7119683733411655, 1.7580599025556236, 1.6382664753373262, 1.751949330803695, 1.675844856128571, 1.6542520181388611, 1.642802591536455, 1.7313980031165348, 1.7335128609541874, 1.7393998120241105, 1.7433587958098977, 1.7392012549054092, 1.6618490211523262, 1.7199300504793786, 1.6338664323660979, 1.6848059513007, 1.6642130552583438, 1.6482895331777585, 1.6774498732985965, 1.6965323891609339, 1.682801070486664, 1.651158974428845, 1.625505265916229, 1.6632112014065883, 1.7484437561338875, 1.794003391721446, 1.6285840356425874, 1.6269653815372733, 1.6111083645729503, 1.6750161503530612, 1.632708431049517, 1.6315653164675281, 1.7473933233576975, 1.6739596291712135, 1.7732721589932776, 1.6601121395256868, 1.8310108587240717, 1.6669642849332968, 1.7054784017004025, 1.6734900740301533, 1.6322397289762072, 1.58753216722209, 1.6101729345929092, 1.69322077028311, 1.637021780773333, 1.7345224839107247, 1.6721026768350298, 1.6334031425463926, 1.7576436730706768]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Default initialization", optimizer="AdamW")
data.update({'train_metric': [7.662, 17.374, 26.882, 37.008, 44.676, 50.322, 55.268, 59.648, 62.886, 66.528, 69.34, 72.276, 75.118, 77.738, 80.246, 82.55, 84.89, 86.682, 88.438, 90.31, 91.122, 92.31, 93.228, 93.744, 94.036, 94.776, 95.238, 95.338, 95.756, 95.802, 96.214, 96.372, 97.008, 96.356, 97.062, 96.814, 97.438, 97.316, 97.466, 97.468, 97.73, 97.764, 97.69, 97.958, 98.004, 97.838, 97.996, 98.252, 98.168, 98.144, 98.414, 98.088, 98.342, 98.284, 98.56, 98.454, 98.376, 98.55, 98.61, 98.56, 98.942, 98.392, 98.676, 98.834, 98.852, 98.62, 98.96, 98.778, 98.668, 98.67, 99.148, 98.882, 98.722, 99.1, 98.964, 98.826, 99.08, 98.898, 98.988, 98.988, 99.168, 98.93, 99.08, 99.08, 99.228, 98.972, 99.002, 99.23, 99.154, 99.166, 99.184, 99.182, 99.33, 99.092, 99.206, 99.306, 99.08, 99.154, 99.358, 99.19], 'train_loss': [4.054311121516215, 3.4185559219106687, 2.892543634795167, 2.378326653519555, 2.0336578067611244, 1.788587994130371, 1.5905284595763898, 1.4219663945762702, 1.2828478665303087, 1.1515261998871709, 1.0328993808735363, 0.9225264154660427, 0.8188657497658449, 0.7187761664390564, 0.6270579048015578, 0.5487156605247951, 0.47038862374051454, 0.4098776034877428, 0.35105838476087126, 0.2977265888143836, 0.2695886397834324, 0.23488096184933277, 0.2077313276684231, 0.18788715740403786, 0.17857321273640295, 0.1597384363317581, 0.1457922881507241, 0.141217435548163, 0.13075576725837482, 0.125742625267676, 0.11374413274535838, 0.1093592709549667, 0.09536385014736572, 0.10754027245613887, 0.09080802303765093, 0.09673689495381492, 0.07872671216173703, 0.08487519282309333, 0.0795374659659422, 0.08108750164277775, 0.07309975803536756, 0.07276996178314318, 0.07290236339511355, 0.06430654579956951, 0.061994457321748846, 0.0652165008676679, 0.06482045398489156, 0.05688727523594418, 0.058512740126868135, 0.05696938311695443, 0.04907562122832093, 0.056701828190601905, 0.05274147559683818, 0.05502159102394691, 0.04514531034004429, 0.049288273220459033, 0.04990364500718034, 0.046317060506167816, 0.043766356514566736, 0.0465146504136224, 0.03567087268153125, 0.049667523202040924, 0.04301042576877834, 0.039382932311527744, 0.037243223877574845, 0.04366509974493509, 0.034567552288491614, 0.03879712377351833, 0.04210082240863596, 0.04059529006394827, 0.0292091983579048, 0.03598358738355224, 0.04081860323965936, 0.030122820689441766, 0.034649586445405424, 0.03784501410657814, 0.02950940101832935, 0.03462601607755336, 0.03216869088315257, 0.03057229832921456, 0.026442587258965027, 0.033927042259137404, 0.029683933140102016, 0.029546393328593434, 0.025428530599591753, 0.03090582877043825, 0.031033455270349675, 0.024450247671277694, 0.028180227670988277, 0.02800387249967885, 0.025734100512171145, 0.02724688780441037, 0.02287008270157276, 0.030035229376800088, 0.02585126890733456, 0.022350531245563642, 0.029151029294476393, 0.028081805964284563, 0.020535670022261057, 0.02561882785302611], 'val_metric': [12.46, 22.55, 31.69, 40.66, 46.64, 51.77, 55.12, 56.56, 58.65, 59.68, 61.75, 62.84, 63.92, 64.36, 64.53, 65.4, 65.36, 65.16, 65.4, 65.81, 66.35, 65.6, 64.98, 66.82, 66.96, 65.34, 65.57, 66.11, 66.28, 66.71, 67.29, 67.09, 65.92, 66.0, 66.97, 66.97, 66.32, 66.44, 67.14, 67.03, 67.62, 67.17, 67.26, 66.77, 66.33, 66.56, 68.02, 67.13, 68.08, 68.9, 67.47, 68.01, 68.0, 67.21, 67.93, 67.17, 67.02, 68.23, 68.25, 67.48, 67.88, 68.02, 68.49, 66.4, 68.53, 67.92, 67.95, 67.95, 67.66, 67.63, 68.3, 68.01, 68.53, 68.03, 68.4, 68.47, 68.74, 67.77, 69.12, 68.3, 69.23, 67.48, 69.26, 67.82, 68.84, 69.26, 67.5, 69.8, 67.91, 68.97, 68.48, 68.87, 69.44, 68.64, 69.74, 67.97, 69.23, 68.17, 68.76, 67.75], 'val_loss': [3.6631127056802155, 3.1311474302012448, 2.6430974219255385, 2.2608212483157017, 1.9732510891689616, 1.7651931875070948, 1.6185415876898797, 1.5504549832860375, 1.4409437528841056, 1.4734157403563237, 1.387556482272543, 1.4156220486968945, 1.3089476489717033, 1.3426980182623407, 1.356868218084809, 1.3376402539812076, 1.4434938480140298, 1.5130617686897327, 1.4765756213740937, 1.5020471386089447, 1.554961955091756, 1.6150865144790358, 1.7082246822916018, 1.6871046146769433, 1.6284919195114427, 1.7696864931446732, 1.7952552282126846, 1.7587980703943094, 1.7855237236448154, 1.8003790033091405, 1.7753509146392725, 1.7340185319542125, 1.8506983173121312, 1.885141438739315, 1.8047738959834834, 1.815458937435393, 1.9025045126486735, 1.8091092234963824, 1.9660221019368262, 1.8469406916837023, 1.827157827319613, 1.832129741550251, 1.9085708822414373, 1.9213157994731975, 1.9015488377801932, 2.0722173258757137, 1.8597049557479324, 1.8846182052496891, 1.848322137905534, 1.819661263068011, 1.9058841402363624, 1.9043519899343988, 1.9271338544074137, 1.8774554831966472, 1.9092307432441955, 1.9985811763508305, 1.9998734889516405, 1.91351814938199, 1.9276711120726957, 1.908395386425553, 1.988141669589243, 1.9009055354792601, 1.897747406154681, 2.1144998339331074, 1.9145927277340251, 2.009332696533507, 1.9984723944572886, 1.9731965721792477, 1.9605077000180626, 2.005277419545848, 1.9642603606175466, 2.098242774131192, 1.9368092410124031, 2.003966844765244, 1.931145057936383, 1.9585821601995237, 1.9600230345300809, 2.032859050544204, 1.9270190945856132, 2.0376477586995265, 1.9829335356973539, 2.099145428769907, 1.914837070331452, 2.016760161348209, 2.00336934654576, 1.9675722589158708, 2.1282178573547657, 1.9724834511993796, 2.123320656976882, 2.031323095795455, 2.0516547534116514, 1.943582843823038, 2.0095967608652296, 2.0156137320646055, 1.9538795720240114, 2.0957399705413042, 1.946718950939786, 2.0342274858693408, 2.0259542742352576, 2.1136799435706655]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal", optimizer="SGD")
data.update({'train_metric': [2.942, 6.584, 9.6, 12.452, 14.792, 17.078, 19.126, 21.064, 22.64, 24.348, 26.016, 27.292, 28.906, 29.918, 31.094, 32.076, 33.254, 34.436, 35.278, 36.234, 37.38, 38.366, 39.44, 40.142, 40.932, 41.704, 42.75, 43.458, 44.146, 45.068, 46.01, 46.384, 47.558, 48.064, 49.062, 49.53, 50.492, 50.994, 51.7, 52.374, 52.808, 53.788, 54.284, 55.122, 55.778, 56.186, 57.024, 57.628, 58.216, 59.01, 59.74, 60.18, 60.416, 61.482, 61.87, 62.434, 62.982, 63.702, 64.156, 64.736, 65.23, 65.96, 66.658, 67.036, 67.648, 68.394, 68.854, 69.314, 69.912, 70.584, 71.184, 71.658, 72.09, 72.744, 73.39, 73.814, 74.054, 74.972, 75.364, 75.968, 76.234, 77.122, 77.432, 78.096, 78.37, 78.736, 79.318, 80.036, 80.5, 81.174, 81.412, 82.07, 82.378, 82.934, 83.218, 83.714, 84.18, 84.698, 85.176, 85.486], 'train_loss': [4.527204561111567, 4.171814040149874, 3.949492697825517, 3.7882611788142366, 3.652792446143792, 3.5269481184537455, 3.4053258066592007, 3.29975218144829, 3.2030417736229078, 3.1074775859827883, 3.02147712518492, 2.951780429276664, 2.881967009790718, 2.8148551938478903, 2.7577398314195523, 2.698169938743572, 2.648439903698309, 2.5908648746702676, 2.548579008530473, 2.496461332454096, 2.4514131093269116, 2.411142606869378, 2.3608900316231085, 2.3208618432359622, 2.2812109033165076, 2.2451631509129655, 2.2038911474330347, 2.1726443484006333, 2.1404726508328373, 2.1020094395598488, 2.068850720172648, 2.0311807462626406, 2.0037368302759915, 1.969346731520065, 1.9342945664739974, 1.9090817590503741, 1.8771498177362524, 1.8463082752569253, 1.8184729761174878, 1.7871065397396722, 1.7651049683770865, 1.7327037863719188, 1.704198656179716, 1.6812210345207272, 1.655077592796072, 1.631600262106532, 1.600812883328294, 1.5778310105318913, 1.553839753808268, 1.5256785565934827, 1.4995697441003513, 1.478789906520063, 1.4589918559926855, 1.4299167608056227, 1.4112382461042965, 1.3915033942598212, 1.3649166865879312, 1.3427839042890408, 1.3201006647875853, 1.303154562364149, 1.275675349985547, 1.25185863380237, 1.2330503552923422, 1.2135492640230663, 1.188841755432851, 1.166467098704994, 1.1410442009149, 1.1271171547720193, 1.1052777474493627, 1.0855193873653022, 1.0642473148872784, 1.0435947779651797, 1.026644009915764, 1.003240753317733, 0.9815508445815357, 0.9592965112622741, 0.9460075088321706, 0.9230904076105494, 0.9064433692056505, 0.8877607909462336, 0.8690978976924096, 0.8486491733652246, 0.8365288480087314, 0.8167056254566173, 0.8006664773311152, 0.7833821762858144, 0.767178707072497, 0.7472066796191817, 0.7339790151704608, 0.7078957784435024, 0.6984948539139365, 0.6774365714062816, 0.6645445562613285, 0.6485318764091452, 0.632400581637002, 0.6190146008111022, 0.6065405746707526, 0.5872174645476329, 0.5755346033274366, 0.5603276739644882], 'val_metric': [5.0, 8.55, 11.1, 13.84, 15.36, 18.62, 20.32, 21.66, 23.57, 25.18, 25.91, 26.59, 28.54, 29.64, 30.44, 30.93, 32.09, 32.56, 33.84, 34.65, 34.89, 35.44, 37.2, 36.01, 38.11, 38.02, 37.59, 39.24, 39.55, 39.66, 41.11, 42.52, 42.45, 42.83, 43.39, 40.3, 43.79, 44.9, 42.15, 45.33, 44.4, 45.26, 45.76, 46.32, 47.53, 46.37, 44.26, 46.26, 47.33, 47.68, 48.29, 47.88, 47.72, 48.06, 48.3, 47.48, 49.64, 49.49, 47.47, 44.52, 49.92, 49.3, 50.68, 48.49, 50.4, 51.19, 51.44, 50.66, 51.17, 50.76, 50.38, 51.06, 50.71, 50.72, 51.98, 52.28, 51.89, 49.99, 50.54, 52.35, 52.41, 51.72, 52.28, 49.73, 50.43, 49.57, 50.63, 42.23, 48.44, 51.45, 52.02, 51.61, 51.34, 50.21, 50.44, 51.43, 52.77, 50.77, 52.68, 52.13], 'val_loss': [4.33316042772524, 4.02178303906872, 3.837242654934051, 3.705236179813458, 3.6082366287328633, 3.445857909075014, 3.35464423629129, 3.2558630499870156, 3.1612097564016937, 3.078117162558683, 3.0031352559472344, 2.9631114886824492, 2.8912656975399917, 2.8426993105821547, 2.796928559139276, 2.7417523481283976, 2.694496329422969, 2.658433804846114, 2.606389912830037, 2.5703981196045116, 2.56070164206681, 2.54342433449569, 2.4543310608833457, 2.520168857210001, 2.402683372710161, 2.390746855432061, 2.4235616155490756, 2.344597357853203, 2.3443716521475726, 2.3155096769332886, 2.2669612685586236, 2.2255506629397157, 2.2063737173748623, 2.18254900965721, 2.1644216024192278, 2.3392816303641935, 2.154043858218345, 2.105477401405383, 2.2392900840492005, 2.0736588543387735, 2.11587269290997, 2.0763372395448623, 2.063720520134944, 2.04619672799566, 1.9984518449017956, 2.0320937960011185, 2.151493295742448, 2.028367486728984, 1.9823178455328485, 1.976952262744782, 1.9545022636462168, 1.964127305966274, 1.973363189180945, 1.9524596891585428, 1.9484443565842453, 2.0195880528468235, 1.9047422037003146, 1.93603766268226, 2.035898575357571, 2.2174852167724803, 1.9010539753421856, 1.9111796351754742, 1.877954640206258, 1.9850668132684792, 1.8901639781939756, 1.8351932802018087, 1.8336148444254687, 1.910453639971982, 1.8409213822358732, 1.8955377606069965, 1.9029018803007285, 1.8858134731365617, 1.9047236047732603, 1.8965933178640475, 1.8429425666286687, 1.8272044036039121, 1.8279149767699514, 1.9537184056203076, 1.9014880277548627, 1.839485403838431, 1.849569578079661, 1.8699984094899171, 1.847549919110195, 1.9733297498362838, 1.9809201462253643, 2.0278112865557336, 2.0074797830763895, 2.801989104337753, 2.1285766120169574, 1.9730636337000853, 1.8781730422548428, 1.9703645546724842, 1.923946982736041, 2.0401777826297054, 2.048991828207757, 1.9715718987641062, 1.8619988655588429, 2.0914775079982295, 1.9174591940679369, 1.9320302017175468]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal", optimizer="Adam")
data.update({'train_metric': [12.904, 26.63, 34.012, 38.366, 41.876, 44.498, 47.142, 48.72, 50.452, 51.964, 52.986, 54.248, 55.272, 56.214, 57.078, 57.524, 58.36, 58.966, 59.342, 59.698, 60.452, 60.826, 61.19, 61.528, 61.788, 62.202, 62.548, 62.806, 63.448, 63.3, 63.504, 63.692, 63.83, 63.96, 64.288, 64.49, 64.782, 64.796, 65.178, 65.332, 65.28, 65.542, 65.856, 65.586, 66.064, 65.972, 65.996, 66.1, 66.158, 66.068, 66.492, 66.3, 66.446, 66.718, 66.342, 67.188, 66.93, 66.946, 67.292, 67.416, 67.298, 67.374, 67.322, 67.72, 67.724, 67.686, 67.67, 67.61, 67.842, 68.068, 68.002, 68.078, 67.938, 68.322, 68.076, 68.06, 68.346, 68.06, 68.448, 68.49, 68.278, 68.434, 68.566, 68.492, 68.936, 68.368, 68.78, 68.734, 68.588, 68.886, 68.928, 68.944, 68.948, 68.904, 69.094, 69.01, 69.206, 69.314, 68.974, 69.266], 'train_loss': [3.772447855881108, 2.8725189616917954, 2.5027881717437976, 2.300260136956754, 2.1364538311348547, 2.0239071233193284, 1.9237686317899954, 1.8565929971082742, 1.7883120561804613, 1.7191502820805211, 1.6713039324716534, 1.6259033856794352, 1.5884865991904606, 1.5508184902503361, 1.5202548822478565, 1.5011039067564718, 1.4727846688931556, 1.4426587856639073, 1.4332946647920877, 1.4121058658718149, 1.3881383706694064, 1.3783387011274353, 1.3651101737833389, 1.3480354400394519, 1.3375813500655582, 1.327472801479842, 1.3109926244486934, 1.3022327583159328, 1.2861111097018738, 1.2854688035710085, 1.2780908955942334, 1.2709686216490959, 1.2670605845006226, 1.2523833437039114, 1.2499925199219637, 1.2397245430885373, 1.233770164458648, 1.2272324606280802, 1.223196591257744, 1.209031507944512, 1.2151573522164083, 1.2086164054968167, 1.201230298984996, 1.191955461831349, 1.1872417789590939, 1.1901647479790252, 1.1844469292084578, 1.1778581113461644, 1.1782421802773195, 1.1757842429611078, 1.16877256238552, 1.1744659456145732, 1.169139531719715, 1.1579888437105261, 1.1628970682163677, 1.1500714654507844, 1.1471891895584438, 1.1508245452895494, 1.142465302553933, 1.1378976207255098, 1.135234381689135, 1.1350023363099988, 1.1379900676819978, 1.130746032270934, 1.1233522610743638, 1.131261413664464, 1.1274124078281091, 1.122922174613494, 1.116782438419664, 1.1158810158824677, 1.1176055011237065, 1.114996578077526, 1.1109722020376065, 1.1053668088315394, 1.1073039046028996, 1.1064555805052638, 1.0985357304820624, 1.1055797707394261, 1.095742611628969, 1.0951224470230014, 1.0985026022662288, 1.092472974982713, 1.0924584279432321, 1.0916678100595694, 1.0846405475950607, 1.0961603815750698, 1.0857949189060485, 1.085026146734462, 1.0886510833145102, 1.0829247528177393, 1.0822399192301513, 1.0792790966875412, 1.080568775953844, 1.0786397133946724, 1.078259483246547, 1.0817030672831913, 1.0699682956004082, 1.0654297964194852, 1.0744765908516887, 1.0670851889778585], 'val_metric': [19.3, 29.13, 32.31, 35.72, 40.01, 42.65, 43.43, 45.79, 50.74, 48.65, 51.54, 52.11, 53.87, 54.74, 52.57, 49.11, 55.09, 55.45, 57.28, 55.22, 55.59, 52.86, 54.2, 58.41, 59.26, 58.11, 58.38, 58.18, 57.64, 57.92, 58.54, 57.1, 58.19, 58.39, 57.16, 56.51, 59.15, 59.51, 58.66, 59.81, 60.39, 58.22, 59.3, 59.27, 60.44, 59.65, 59.5, 60.66, 59.8, 60.95, 60.75, 58.87, 58.11, 55.98, 60.75, 60.65, 60.28, 59.41, 58.17, 61.63, 60.55, 59.62, 59.5, 60.24, 59.88, 59.84, 60.99, 59.47, 62.1, 60.76, 60.23, 60.73, 61.35, 59.91, 61.49, 61.74, 60.98, 58.32, 61.78, 59.48, 60.35, 60.87, 61.01, 58.63, 61.16, 61.15, 60.05, 59.01, 61.4, 61.18, 60.57, 59.96, 60.0, 59.47, 62.8, 61.58, 61.46, 60.89, 60.8, 60.93], 'val_loss': [3.284364352560347, 2.760063936755915, 2.6078536495281632, 2.427596907706777, 2.258164508327557, 2.1345006743813775, 2.0687447338347225, 1.971539123802428, 1.796806812286377, 1.905643714461357, 1.7394716648539161, 1.7018894131775875, 1.659558700148467, 1.6002415069349252, 1.6816052198410034, 1.847559178710743, 1.6175550397034664, 1.587699191585468, 1.526314444982322, 1.5970730705625693, 1.5718647970515451, 1.7314171722740124, 1.6443173980257313, 1.4698662306093107, 1.4446624642724444, 1.4779808703501514, 1.456809451245958, 1.4901685691942834, 1.4993737465257098, 1.5105050316282138, 1.457033406776987, 1.502130998168022, 1.49910338555172, 1.4795239283021089, 1.5361721306849436, 1.51905971499765, 1.426246244056969, 1.4280978524760835, 1.4568803302801339, 1.416162610813311, 1.3963317407923899, 1.4676576993268007, 1.437630136301563, 1.4618812526107594, 1.4063865329809249, 1.41882946119187, 1.411752412653273, 1.3783968633906856, 1.418901919938956, 1.3866477464414706, 1.4033721279186808, 1.4675545172326883, 1.4734441508912737, 1.6148419076470053, 1.3862743241012476, 1.3798102933889742, 1.4015011924087621, 1.435986248550901, 1.4810536082383174, 1.349083706451829, 1.3732690048065914, 1.4390396020215028, 1.4495342700344742, 1.4116271354590253, 1.4242422675630848, 1.4006026941499892, 1.3749581829757447, 1.4475761116689938, 1.3280755843326544, 1.3914838649664716, 1.3865082017175712, 1.3970180332281028, 1.3454938682780904, 1.4063795823959788, 1.3556609111986342, 1.3391492154188216, 1.3826421605553596, 1.4798103510194522, 1.348379542873164, 1.4354446439226722, 1.3962640394071104, 1.378267154192469, 1.3785148710961554, 1.507038383726861, 1.380490691039213, 1.3661503974039844, 1.4172637367704113, 1.4431061554866231, 1.3704789075881811, 1.371073169693066, 1.399901685821023, 1.4259757502063823, 1.4286609083224253, 1.4228786852709048, 1.3020072545215582, 1.3667162527703935, 1.3559238872710306, 1.366580083491696, 1.3573679168512867, 1.3979575353063596]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Sinusoidal", optimizer="AdamW")
data.update({'train_metric': [13.986, 30.454, 41.562, 48.86, 54.752, 59.484, 62.74, 66.48, 70.112, 72.548, 75.178, 78.184, 80.388, 82.268, 84.928, 86.94, 88.552, 89.774, 90.994, 91.972, 93.132, 93.694, 94.206, 94.834, 94.894, 95.758, 95.326, 96.062, 96.238, 96.702, 96.76, 96.966, 96.984, 97.102, 96.986, 97.548, 97.552, 97.694, 97.628, 97.91, 97.878, 97.76, 98.164, 98.066, 98.242, 98.142, 98.26, 98.252, 98.268, 98.688, 98.552, 98.17, 98.6, 98.76, 98.596, 98.37, 98.86, 98.664, 98.736, 98.756, 98.746, 99.004, 98.646, 98.806, 98.964, 98.904, 98.966, 98.932, 98.964, 98.984, 99.032, 99.13, 99.024, 98.9, 99.118, 99.07, 99.104, 99.19, 99.056, 99.33, 98.926, 99.166, 99.348, 99.014, 99.306, 99.194, 99.214, 99.126, 99.198, 99.308, 99.134, 99.444, 99.094, 99.358, 99.352, 99.342, 99.314, 99.182, 99.444, 99.344], 'train_loss': [3.7211459722665263, 2.683971514939652, 2.1729799743808442, 1.844897810303037, 1.6083450347871122, 1.4221029233597124, 1.2868262785474966, 1.1404045474193896, 1.011393484328409, 0.9093377346272968, 0.8191857817761429, 0.7076395928402386, 0.6274769683856793, 0.5602757328230402, 0.47313601314030646, 0.4063818843658928, 0.35363252403790996, 0.31137495537948273, 0.2711765283761579, 0.24153605223540456, 0.20773221514737972, 0.19241838797907848, 0.1773656123482129, 0.16085162489672603, 0.15555995149428353, 0.13112304837126143, 0.14265978761741419, 0.12163134130156215, 0.11209820889417663, 0.10388741923955834, 0.10080615613106972, 0.09286284519959707, 0.09113655205282485, 0.09123691731868573, 0.09332447501537783, 0.07504105095006049, 0.07448175945051272, 0.07463456388346165, 0.07366880865426986, 0.0662803528949504, 0.06604416675084387, 0.07090401035302397, 0.0591103179419599, 0.06202868771854826, 0.05499430063038902, 0.05864950207561073, 0.055662086414223266, 0.05455691055299433, 0.05444548409991919, 0.045130439280696655, 0.048476681286824, 0.0543017135132128, 0.04484335506228311, 0.03908852887470299, 0.04675155112509737, 0.051428007861406395, 0.038443587392138656, 0.040911038147906066, 0.04069414503022056, 0.04066184944870091, 0.039668307390898855, 0.03222870412479152, 0.043399317503568555, 0.03869671135640029, 0.034108996724017925, 0.035528630558478755, 0.034472082537236264, 0.035437219324098934, 0.03416787532617903, 0.03214065447215067, 0.03204344602319135, 0.02812000574118903, 0.03094264267020218, 0.03624572633536976, 0.028386350292230116, 0.030633162267704416, 0.03020314452849636, 0.027893258246418702, 0.030143463258322477, 0.022586189935033273, 0.03517357727280184, 0.026943943348711433, 0.022062262145603192, 0.03247834981963708, 0.02205253875626208, 0.025922915523523726, 0.025871685608123404, 0.02709383314082106, 0.02486389701377036, 0.022116273295429364, 0.026902109892811697, 0.018428300507366657, 0.029885209386955703, 0.02057404553125728, 0.02184514036995199, 0.02194737762891148, 0.022908455295384923, 0.02549209376349581, 0.019190535464928018, 0.02181754335324885], 'val_metric': [24.42, 36.8, 46.06, 49.04, 52.99, 45.66, 58.05, 62.42, 62.15, 62.81, 65.13, 65.4, 66.56, 67.52, 64.61, 68.14, 68.15, 68.81, 67.72, 68.54, 68.55, 68.18, 69.71, 69.79, 70.28, 65.08, 68.85, 70.0, 70.74, 70.06, 70.81, 69.29, 70.01, 68.57, 69.9, 69.15, 69.81, 70.51, 70.07, 70.44, 69.37, 70.24, 69.71, 70.75, 70.03, 68.75, 69.84, 70.56, 69.92, 70.96, 69.25, 71.32, 70.84, 70.57, 69.53, 71.92, 71.06, 70.04, 71.43, 69.99, 71.22, 69.31, 70.84, 71.1, 71.33, 71.35, 71.45, 70.71, 71.16, 71.33, 70.95, 71.72, 70.57, 72.17, 71.2, 71.22, 72.0, 70.71, 71.77, 71.36, 71.33, 71.7, 72.17, 71.71, 71.81, 71.19, 70.85, 70.35, 71.52, 71.93, 71.53, 72.42, 71.02, 70.34, 71.95, 71.21, 71.16, 71.67, 72.28, 70.9], 'val_loss': [3.003699774954729, 2.382886131098316, 1.9940778601701092, 1.8304196801155237, 1.7157423716441842, 7.4876653700117854, 1.5037669774832998, 1.348924980801382, 1.3721087806543726, 1.339696239513956, 1.2397796268675738, 1.2805833554571602, 1.2182678561301747, 1.2011435760813913, 1.457119678995412, 1.2086419926327505, 1.3203429025449571, 1.2933205795136227, 1.4406287579020118, 1.3613972200709543, 1.4050765610804223, 1.4639293839977046, 1.4023111660009737, 1.4281199657992951, 1.4359992875415049, 2.2726582060953615, 1.627214292052445, 1.4862866044803789, 1.4940700515819962, 1.568773526674623, 1.5189984797672103, 1.6182261720584457, 1.6301612846410958, 1.719840176166243, 1.6149266502659791, 1.7005486803449643, 1.6040494240772951, 1.584511149081455, 1.6115721559068958, 1.6309281777424418, 1.7563114166259766, 1.6477097865122898, 1.6881478201052187, 1.6367920508050615, 1.7418280991779012, 1.8147897439397824, 1.6356449514437632, 1.6711479288757227, 1.7256505850014414, 1.675943182532195, 1.7864987121266165, 1.684639397320474, 1.6635575085688548, 1.70614617037925, 1.770778160945625, 1.6025526622298416, 1.646201069187966, 1.7316351737945703, 1.6160279455458282, 1.715680453807685, 1.648532043217094, 1.8539885248348211, 1.7719323805942657, 1.692453850986092, 1.733153083521849, 1.6432098639998467, 1.644906628663373, 1.7280485588274184, 1.7446454410340375, 1.6859973896840574, 1.7156708923874386, 1.714184411012443, 1.768997812726695, 1.6628281766441977, 1.6702128823395748, 1.7138052328377014, 1.6733694839629398, 1.7683832053166286, 1.7455311649164575, 1.737627272013646, 1.7346383747021863, 1.6868707507279268, 1.6845851916416434, 1.777859542020567, 1.772790558398909, 1.7452062041896164, 1.7935480684231802, 1.8029779894336773, 1.7794837275887752, 1.760946460590241, 1.7698941860988642, 1.69249902713071, 1.7727021537009318, 1.8174826859668562, 1.7153991863226434, 1.841598699806602, 1.8298165524841115, 1.7368302914747007, 1.7399199768236489, 1.818368012738076]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Orthogonal", optimizer="SGD")
data.update({'train_metric': [2.54, 4.56, 6.89, 8.928, 10.576, 12.236, 13.69, 14.858, 16.028, 17.336, 18.254, 19.278, 20.31, 21.106, 22.078, 22.932, 23.89, 24.866, 25.71, 26.54, 27.396, 27.956, 29.066, 29.688, 30.314, 30.74, 31.66, 32.318, 32.874, 33.516, 34.098, 34.676, 35.304, 35.762, 36.448, 36.974, 37.474, 38.394, 39.062, 39.442, 40.004, 40.668, 41.17, 41.75, 42.55, 42.736, 43.374, 43.946, 44.602, 45.222, 45.572, 46.73, 46.664, 47.404, 47.95, 48.42, 48.942, 49.746, 49.974, 50.818, 51.482, 52.128, 52.316, 53.064, 53.714, 53.966, 54.678, 55.298, 55.806, 56.298, 56.93, 57.566, 57.83, 58.518, 59.116, 59.678, 60.27, 60.692, 61.534, 61.802, 62.842, 62.908, 63.784, 63.784, 64.534, 65.394, 65.732, 66.204, 66.78, 67.28, 68.036, 68.408, 69.178, 69.754, 70.016, 70.246, 71.104, 72.032, 72.45, 73.016], 'train_loss': [4.514228321402274, 4.358207890444704, 4.196165576005531, 4.055724035138669, 3.942486602631981, 3.829968813740079, 3.730949453990478, 3.650494656599391, 3.5760482333200363, 3.512809719271062, 3.453718003409598, 3.398520980344709, 3.34778265087196, 3.296651896613333, 3.246866978647764, 3.198409256118033, 3.1534376110872038, 3.1028824123885017, 3.059036494520924, 3.01529461343575, 2.9674664491887595, 2.92686341302779, 2.8841904435316317, 2.845070585875255, 2.805468077549849, 2.7763806081488918, 2.7371458229811294, 2.699769752104874, 2.6733231727424487, 2.635575809442174, 2.612068354931024, 2.5817118482211665, 2.5554535266993295, 2.5239713406928663, 2.4951356117377808, 2.4663776985519683, 2.4417103231715425, 2.409507620517555, 2.380728821010541, 2.354760835542703, 2.323383274895456, 2.2991335181628956, 2.271001414417306, 2.2440293705676826, 2.2141490645725708, 2.1936157159793104, 2.1614616083367095, 2.139642519871597, 2.1158542447077955, 2.0904911262604893, 2.0687007053428905, 2.034282753839517, 2.0144707763286505, 1.986964150462919, 1.9657181173639224, 1.936082103520708, 1.9126490633505995, 1.8866393000573454, 1.8692746156316888, 1.8350791954018575, 1.8142675217765067, 1.7939131807183366, 1.7762256034499848, 1.74482688787953, 1.723655044117852, 1.7011317090914988, 1.6792719772709606, 1.6535092514494192, 1.6325849473019085, 1.6105811079139905, 1.5896369464257185, 1.5639769052300612, 1.543395001565099, 1.5222726074021187, 1.500410976159908, 1.4728574504327896, 1.45657998071912, 1.4297555556230228, 1.410856925084463, 1.396831504676653, 1.365117794869806, 1.3494677981147376, 1.3192014490704402, 1.306462141940051, 1.2805333699259307, 1.2603131546388806, 1.2375173837022708, 1.22451850771904, 1.1998330410331717, 1.1764796291623274, 1.1558474059147603, 1.1373063031669772, 1.1195902371650461, 1.0932257851523817, 1.07902438279308, 1.057682785612848, 1.0397186789213848, 1.0110969212658876, 0.9923100286280103, 0.9752288956166534], 'val_metric': [4.04, 6.28, 7.9, 9.75, 11.51, 13.44, 14.91, 15.94, 17.07, 18.14, 19.48, 20.69, 21.41, 21.96, 22.67, 23.38, 24.8, 25.34, 26.03, 26.65, 26.82, 27.65, 28.41, 28.95, 29.24, 29.88, 30.69, 31.03, 30.8, 32.08, 31.88, 32.86, 32.77, 33.53, 34.19, 34.42, 34.35, 34.98, 34.92, 36.71, 35.69, 36.84, 35.78, 37.39, 38.2, 38.39, 37.14, 36.03, 35.92, 31.59, 39.68, 38.92, 40.51, 40.82, 41.27, 41.05, 39.59, 41.1, 39.67, 42.99, 40.65, 41.21, 43.16, 41.79, 41.81, 44.4, 44.26, 44.93, 42.09, 43.61, 44.68, 44.44, 42.26, 43.81, 46.06, 43.2, 42.56, 43.37, 34.92, 45.18, 42.72, 46.8, 45.1, 45.36, 45.72, 45.67, 45.94, 45.41, 42.1, 44.24, 45.37, 45.36, 45.93, 43.91, 41.58, 45.21, 43.62, 46.52, 41.71, 46.74], 'val_loss': [4.404414146569125, 4.273868266184619, 4.100755705195628, 4.002354152642997, 3.8705448345014246, 3.758307873063786, 3.6723401531292375, 3.599749419339903, 3.528934594172581, 3.471627982558718, 3.4173325201508344, 3.3730815170676847, 3.3197793262020037, 3.2815570223862958, 3.2340488039004573, 3.1917383959338923, 3.137651416146831, 3.0956088949920266, 3.0507127919774146, 3.0197487834152903, 2.98156302597872, 2.949214458465576, 2.9395179292958256, 2.88273431996631, 2.8509823957066627, 2.8160654754395695, 2.7940187879428744, 2.754549687075767, 2.7827337972677437, 2.710183854315691, 2.715193687730534, 2.676989605472346, 2.6753095714909256, 2.633294494288742, 2.617659363777015, 2.592647232067813, 2.6049710565311894, 2.5563373261955893, 2.5826790165749327, 2.520069918055443, 2.5263335811104746, 2.4776701722175454, 2.549903909871533, 2.473612342670465, 2.4178171757679836, 2.397046470338372, 2.4805199279906645, 2.528538637859806, 2.5984048084088953, 2.963633496290559, 2.3532073831861946, 2.3970420140369684, 2.3197475352864356, 2.3049403413845475, 2.2822016211831646, 2.2853828273761043, 2.345369405047909, 2.30539942926662, 2.3747808470088207, 2.2131448452639733, 2.311817521502258, 2.294229264471941, 2.1896685847810877, 2.299784284488411, 2.284999042559581, 2.136017885937053, 2.1635486229210144, 2.1222139505823705, 2.2929130663537673, 2.206847175671037, 2.1320852064023352, 2.1558159551802714, 2.30759648304836, 2.217441144262909, 2.087094671407323, 2.235067451076143, 2.305297262349706, 2.241319970720133, 2.816876342342158, 2.1526022443346156, 2.3162847438435645, 2.061273622664676, 2.1858627970810907, 2.1778474279270053, 2.1116476195633034, 2.1428212944869025, 2.126004873567326, 2.186220185012574, 2.391411916465516, 2.2250284085607834, 2.179886050285048, 2.179268250040188, 2.151223584345192, 2.2092989083308323, 2.538192729281772, 2.1688911390912002, 2.371291245624518, 2.149249262870497, 2.6419857861889398, 2.147845373032199]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Orthogonal", optimizer="Adam")
data.update({'train_metric': [8.046, 14.536, 20.406, 25.894, 30.796, 35.242, 38.368, 41.192, 42.834, 44.248, 45.46, 46.454, 47.492, 48.298, 49.218, 49.52, 50.11, 50.776, 51.286, 51.67, 51.946, 52.306, 53.032, 53.34, 53.69, 53.952, 54.18, 54.438, 54.738, 55.304, 55.196, 55.458, 55.68, 55.972, 56.032, 56.054, 56.436, 56.484, 56.728, 56.492, 56.95, 57.126, 57.154, 57.618, 57.67, 57.666, 57.59, 57.966, 58.068, 58.154, 58.286, 58.538, 58.502, 58.852, 58.818, 58.734, 58.598, 59.168, 59.048, 59.142, 59.322, 59.16, 59.198, 59.688, 59.446, 59.728, 59.598, 59.912, 59.784, 60.03, 59.906, 60.256, 60.072, 60.05, 60.198, 60.296, 60.198, 60.364, 60.414, 60.37, 60.622, 60.46, 60.48, 60.63, 60.506, 60.792, 60.994, 60.856, 60.686, 60.938, 60.8, 61.164, 61.062, 61.108, 61.07, 61.18, 61.164, 61.264, 61.408, 61.322], 'train_loss': [4.071236934198443, 3.567337679741023, 3.246814753698266, 2.9385689804925943, 2.6770884448000234, 2.4649298494429233, 2.3102738438055033, 2.1959218454482916, 2.112637604136601, 2.057441793896658, 2.0000818079084994, 1.9558387058775137, 1.9148407997682577, 1.8823406153627673, 1.8508529153931172, 1.8306354279713253, 1.804636392751923, 1.7761448004361613, 1.7609374610054525, 1.7436689178047278, 1.725509951486612, 1.7104558420303229, 1.6901532821643077, 1.6765058551298078, 1.6629602569143485, 1.649942418650898, 1.6426391022284623, 1.6302477752461153, 1.6194407755456617, 1.6062895155623746, 1.6022660215492444, 1.591847090312587, 1.583526043940688, 1.5746890705869632, 1.5713041363774662, 1.56249147485894, 1.551660850987105, 1.547982422592085, 1.5469676068676708, 1.534283540895223, 1.5311155679738124, 1.5252759663955024, 1.5204260183112395, 1.5131203143492988, 1.5061617144538313, 1.5074938203367736, 1.5020054866895651, 1.493797914603787, 1.4881509412889895, 1.4851622751454259, 1.4807424344065245, 1.4718734286630246, 1.4645454302010938, 1.4667344840286334, 1.4643338751000212, 1.4611032948926892, 1.4630214233532586, 1.4480445614403776, 1.449373379082936, 1.4426756168113035, 1.4446287162773443, 1.439217359132474, 1.4421303194502126, 1.4288704796977665, 1.4369241540389293, 1.424266197339958, 1.4274321959908967, 1.41965235110439, 1.421129903829921, 1.4122204102213731, 1.4174465591187977, 1.4112969418163495, 1.4165704694702803, 1.4072038961950775, 1.4080331988651733, 1.4001209535409727, 1.3958576941276755, 1.397051110901796, 1.3959930478154545, 1.3906355113019724, 1.390455299021338, 1.392613853258855, 1.3880769033413713, 1.382196882268047, 1.3878994289109163, 1.3808441934988016, 1.3799309053689317, 1.3814950530486338, 1.3820478216461514, 1.3774338267038546, 1.3808813984589199, 1.376832374069087, 1.3683554421148032, 1.3682971350524737, 1.3698946200978115, 1.363965811098323, 1.3636155043873945, 1.3640677784867299, 1.3595808085120853, 1.3623812964657689], 'val_metric': [11.93, 14.33, 21.29, 26.36, 28.51, 32.13, 38.48, 38.48, 42.8, 41.18, 44.44, 45.25, 46.74, 46.05, 47.48, 45.57, 48.24, 48.24, 50.42, 48.45, 48.25, 49.18, 46.72, 51.1, 50.63, 49.82, 49.86, 51.14, 52.29, 51.01, 52.32, 53.05, 52.91, 52.63, 52.21, 52.4, 51.46, 52.72, 52.91, 50.34, 51.66, 54.2, 53.99, 52.73, 53.44, 52.34, 51.39, 53.39, 54.41, 54.4, 54.98, 54.37, 53.36, 54.23, 55.46, 55.47, 56.28, 54.27, 53.53, 55.55, 55.58, 55.97, 54.55, 56.1, 54.49, 55.54, 55.59, 55.97, 56.42, 55.03, 55.67, 56.33, 54.81, 55.43, 55.33, 56.7, 56.25, 54.22, 52.77, 56.96, 54.83, 55.45, 53.85, 55.03, 54.87, 54.82, 54.46, 54.75, 55.92, 56.91, 54.82, 57.34, 55.87, 54.91, 55.14, 55.25, 56.62, 55.84, 55.17, 55.71], 'val_loss': [3.7366056320773566, 3.5441638147755032, 3.1669029976911607, 2.8861105457233016, 2.793435423237503, 2.649501300921106, 2.299880154573234, 2.2891246657462636, 2.126614630601968, 2.16772685840631, 2.021596342135387, 1.999603626834359, 1.9347933720631205, 1.955375398040577, 1.8976010152488758, 2.01392435268232, 1.8948911101954757, 1.893095986858295, 1.8152013728573064, 1.864575668505043, 1.8823571850539773, 1.8841377815623193, 1.9489723147859999, 1.7600821355345901, 1.7753458448276398, 1.8034166910086469, 1.783836064065338, 1.7779993107364436, 1.6974576415529676, 1.7937765470735587, 1.7307907282167179, 1.6892288217119351, 1.6961783527568648, 1.7116531877760675, 1.7112518321177004, 1.726349636247963, 1.7588155345552285, 1.6917133619830866, 1.6890180308348055, 1.7810673379594353, 1.7298683406441075, 1.6401760805943968, 1.664241684470207, 1.6844455315049287, 1.6717699910425077, 1.7230049493206534, 1.8019417091539711, 1.7104241711318873, 1.6674047693325456, 1.6432674720788458, 1.622401584485534, 1.657991666702708, 1.6953716027508876, 1.6541158006449415, 1.5852740175405127, 1.6008854292000934, 1.5624024427620469, 1.6387812020672354, 1.6544632888903283, 1.621121196610153, 1.5921352536055693, 1.58708978990081, 1.6587219701451101, 1.5781853609024339, 1.6222868702214235, 1.6089090640377846, 1.567167124171166, 1.5690417358070423, 1.5554164192479127, 1.626030145177416, 1.5787919543351336, 1.5796862643235807, 1.6421624839685525, 1.5958143586565734, 1.58525416957345, 1.558613491286138, 1.571834157986246, 1.625076183088266, 1.7168325519865486, 1.541857029222379, 1.6226112413558231, 1.5763008100971294, 1.6339382092664196, 1.617657166377754, 1.610373803005097, 1.6108658890815297, 1.6261553992131712, 1.6235828308542823, 1.5643950545104446, 1.5398501450089133, 1.6144280695611504, 1.527917395731446, 1.5689773544384416, 1.6017442949258598, 1.5963174636196937, 1.605276763059531, 1.529677557717463, 1.6051715771863415, 1.6234408799250415, 1.5816759493700259]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="Orthogonal", optimizer="AdamW")
data.update({'train_metric': [8.256, 18.514, 27.824, 35.79, 42.942, 48.58, 54.058, 58.394, 62.404, 65.78, 68.628, 71.996, 74.952, 77.178, 80.236, 82.37, 84.774, 86.654, 88.166, 89.552, 90.758, 91.948, 92.628, 93.328, 94.068, 94.306, 95.244, 95.158, 95.494, 96.092, 95.97, 96.474, 96.638, 96.834, 96.89, 97.054, 97.044, 97.426, 97.644, 97.552, 97.678, 97.56, 97.876, 97.772, 97.886, 98.174, 97.782, 98.254, 98.204, 97.972, 98.406, 98.31, 98.202, 98.444, 98.44, 98.5, 98.734, 98.612, 98.368, 98.64, 98.794, 98.75, 98.58, 98.734, 98.65, 98.808, 98.796, 98.844, 99.092, 98.684, 98.854, 98.886, 98.978, 98.962, 98.802, 98.928, 99.098, 98.862, 98.974, 99.302, 98.978, 99.194, 98.828, 99.238, 99.116, 99.214, 99.18, 99.068, 99.162, 99.248, 99.112, 99.292, 99.048, 99.412, 99.048, 99.326, 99.216, 99.138, 99.38, 99.286], 'train_loss': [4.066269853840703, 3.3564587090631277, 2.853015504834597, 2.4556198757322854, 2.1273073721724702, 1.870818975788858, 1.6541689445295602, 1.4718946563770703, 1.3098879541887347, 1.1729884144595213, 1.0573818944299314, 0.9388306735421691, 0.8284448336838456, 0.7357284322647792, 0.6381354366841219, 0.558645228405133, 0.479794652089286, 0.4135573590198136, 0.36061264035265767, 0.3186219948655961, 0.28201835271914294, 0.246923517745436, 0.21767284666828793, 0.19924852346329738, 0.18022922964771385, 0.17055241429649504, 0.14707337151331557, 0.14500247518224713, 0.13619143097325587, 0.11582061016450987, 0.12416501950396372, 0.11031621987359774, 0.1039742459007122, 0.09982565353336313, 0.09477669221546758, 0.09214095193463975, 0.09075464917134846, 0.08083082513724123, 0.07668510044369932, 0.0791617943006127, 0.07483467833339082, 0.0759312315630224, 0.06779889804650517, 0.0682737654488286, 0.06571734618113312, 0.05756298711766367, 0.06935133452555808, 0.05483545612691499, 0.057104957255753964, 0.06345094074620186, 0.05095963315595218, 0.05197009099769475, 0.056622105170055614, 0.04914316478217511, 0.050818911256671165, 0.04874462662163474, 0.04183987370478418, 0.0450013346853666, 0.05003617319684533, 0.0438952220525221, 0.039412566440422896, 0.03940385194850223, 0.04573558802079872, 0.03941790452343114, 0.04396442758857899, 0.03694968939634681, 0.039243420816885304, 0.03827335833370283, 0.030616605745615377, 0.04129211719454828, 0.03523306002113325, 0.03491070810208202, 0.03442871371973985, 0.03279297282260395, 0.03814294132867904, 0.033459718157971975, 0.028743390213785087, 0.036290429672166766, 0.032816003262371427, 0.024727843603474454, 0.03283185944492009, 0.027158096592943007, 0.03664329593799903, 0.024370544715775558, 0.029782946378214742, 0.02511282377552403, 0.026871672418439532, 0.030490862045441863, 0.02691960119960301, 0.02654081403297614, 0.029268273976324793, 0.023056111200843627, 0.02933445435253751, 0.019332074123425642, 0.03167324111802275, 0.021897815936680467, 0.02531213841303978, 0.02835145940325837, 0.021840445337380294, 0.023520883630893892], 'val_metric': [13.47, 23.91, 30.67, 38.78, 43.03, 48.49, 52.44, 56.52, 58.77, 60.4, 61.42, 62.72, 63.29, 63.61, 64.29, 59.93, 64.54, 57.16, 65.05, 65.12, 65.72, 64.18, 62.8, 64.22, 64.83, 63.06, 67.11, 67.37, 65.24, 65.95, 67.16, 66.88, 62.94, 67.01, 65.44, 66.88, 63.9, 66.1, 66.87, 67.47, 67.25, 68.06, 67.95, 67.39, 67.78, 67.81, 67.43, 67.47, 67.8, 67.65, 67.71, 67.4, 67.07, 67.63, 67.79, 68.5, 63.65, 68.69, 66.23, 68.1, 68.63, 67.4, 68.44, 68.22, 67.53, 68.42, 63.41, 69.42, 67.77, 68.12, 68.06, 69.11, 68.83, 68.53, 69.18, 68.87, 69.22, 69.12, 68.9, 63.74, 66.95, 68.51, 68.18, 68.28, 68.67, 69.07, 69.18, 67.38, 69.23, 69.38, 68.54, 69.61, 68.97, 68.78, 68.52, 69.38, 64.85, 68.91, 69.45, 69.67], 'val_loss': [4.037032546511122, 3.288210135356636, 2.6992992777733287, 2.3478897696088072, 2.1344133167509822, 2.5028134743878794, 1.7155358745793627, 1.555872917175293, 1.496165526141027, 1.4385605234249381, 1.3708124289846724, 1.3196348616272022, 1.3345217792091855, 1.3719517440552924, 1.3664546346968147, 4.437953456192259, 1.424977106273554, 10.95638131336042, 1.4727919887585246, 2.4385160401368595, 1.81211604851826, 2.0262114910563085, 2.721743002818648, 2.188517707927971, 2.587581364971817, 2.504363643135994, 1.6911324703010024, 1.6860578477762307, 2.329613489709842, 1.759015421199191, 1.7284628881770334, 1.805286279529523, 3.4177688071682195, 1.7499858759770728, 2.0872621790618653, 1.798776484978427, 3.7561095186099887, 1.9173621660584856, 1.9022090267983212, 1.8603515731301277, 1.8927526568910877, 1.79165688024205, 1.7857319640505844, 1.875982565105341, 1.8195935559880203, 1.8982061117318025, 1.876645152735862, 1.8969536977968398, 1.8452268322561955, 1.845145147317534, 1.8767240643501282, 1.8813038164642966, 1.86902035801274, 1.952384022011119, 1.8952638780235485, 1.9183054668888164, 4.011606012180352, 1.8796832159066656, 2.3892305709753825, 1.9320038978461247, 1.9141418607371627, 2.0240226255101, 1.9023601158409362, 1.9136124150768208, 1.978141155971843, 1.9490630641864364, 4.507587461714532, 1.8701953717098114, 1.9921880641560645, 1.9945016256563224, 1.9694352187928121, 1.864182743297261, 1.9120046333142906, 1.9453450331262723, 1.8905194215713792, 1.9095287763389053, 1.9029008484190437, 1.9081273120679674, 1.8920392971129933, 5.108197604774669, 2.26941415724481, 1.9676421193560218, 2.004159858651981, 1.9781311496048217, 1.9424637654784378, 1.9951349929639488, 1.97053772628687, 2.023857986851103, 1.9269099812598744, 1.9427434509726846, 2.037654080588347, 1.951744734101994, 1.9449862674543053, 2.0271189592446492, 1.973553965425795, 1.9070847755784441, 5.101100898092719, 1.9309164114818451, 1.9449737652851518, 1.9378721186309864]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="LSUV", optimizer="SGD")
data.update({'train_metric': [2.238, 3.776, 5.176, 6.478, 7.94, 9.436, 10.556, 11.542, 12.432, 13.418, 14.128, 14.84, 15.524, 16.262, 16.87, 17.68, 18.294, 19.024, 19.632, 20.182, 20.848, 21.202, 21.956, 22.678, 23.128, 23.844, 24.21, 24.864, 25.398, 26.094, 26.672, 27.312, 27.546, 28.222, 28.93, 29.292, 29.558, 30.252, 30.71, 31.516, 31.57, 32.084, 32.5, 33.108, 33.53, 33.776, 34.358, 34.87, 35.14, 35.774, 35.914, 36.44, 36.688, 37.448, 37.844, 38.144, 38.728, 39.114, 39.436, 39.814, 40.308, 40.83, 41.38, 41.376, 41.866, 42.316, 42.68, 43.466, 43.994, 44.182, 44.592, 45.184, 45.258, 45.95, 46.212, 46.604, 46.822, 47.478, 47.766, 48.398, 48.674, 48.87, 49.426, 49.602, 50.436, 50.98, 51.094, 51.59, 52.268, 52.242, 52.604, 52.936, 53.368, 53.878, 54.672, 54.67, 55.142, 55.566, 56.186, 56.15], 'train_loss': [4.5245122238803095, 4.417800429531985, 4.3354900810114865, 4.231751560859973, 4.1311466385946245, 4.04014198920306, 3.950187093156683, 3.8694472050727784, 3.7975227104123594, 3.741302138094402, 3.6843278697689477, 3.6400533734685014, 3.595154450372662, 3.552201581123235, 3.5132739126224957, 3.4733118648114414, 3.4406438318969648, 3.4061519133160485, 3.373221258678095, 3.3410430491123053, 3.306653143499818, 3.276217295995454, 3.2422369056955325, 3.210857260562575, 3.1842094854930476, 3.150543174780238, 3.1220660712712864, 3.08882660481631, 3.0584285420834867, 3.0305154140648023, 2.9988645867008685, 2.9680911881844407, 2.942460631165663, 2.9100512296647367, 2.8819982359171523, 2.8586592101075157, 2.831713028576063, 2.8060939034537586, 2.778165729454411, 2.750209653773881, 2.7314336345628702, 2.7025188963736415, 2.680001025004765, 2.657875330094486, 2.633011404510654, 2.6177376432491997, 2.590597257894628, 2.567678065708531, 2.5461224671215046, 2.5210584121591904, 2.5069158586394757, 2.4852987570530924, 2.463138743282279, 2.4421097210910925, 2.4223726231728673, 2.4030141414278914, 2.3831185637532597, 2.357827031703861, 2.3418246338434536, 2.3191904200007545, 2.298249661922455, 2.2843378561232095, 2.2582164589706286, 2.243139176722378, 2.2252239318150084, 2.204342700026529, 2.1871615701624196, 2.163696371380935, 2.142926955162107, 2.1317534977212893, 2.106335589647903, 2.0829248486272514, 2.071242110808487, 2.0499829526447577, 2.0333891489621623, 2.011215273681504, 1.997588724583921, 1.9751340240773643, 1.9601010135982349, 1.9385815968598856, 1.91892636005226, 1.9061132157245255, 1.8818253643067597, 1.8719450108840336, 1.8481272984953487, 1.835022400102347, 1.8151460722889132, 1.8018782739444157, 1.779614490316347, 1.7641739505331229, 1.7534018046105915, 1.7304230350667558, 1.7155162484749504, 1.7015145174072832, 1.6789152093250732, 1.6654510263286892, 1.6450487968257017, 1.6343461168391624, 1.617190241051452, 1.6042563614942837], 'val_metric': [3.12, 4.74, 5.72, 6.75, 8.41, 9.8, 11.52, 12.46, 13.43, 14.08, 14.82, 15.69, 15.97, 16.83, 17.34, 18.9, 19.29, 19.52, 20.24, 20.63, 21.75, 21.97, 22.26, 22.82, 23.29, 24.12, 24.41, 24.6, 25.53, 25.71, 26.96, 27.01, 27.29, 28.23, 27.91, 28.38, 29.3, 29.67, 29.84, 30.45, 30.31, 30.87, 31.39, 32.4, 31.85, 32.13, 33.1, 33.57, 33.24, 33.25, 34.11, 33.97, 35.01, 35.18, 35.31, 34.31, 35.75, 35.35, 36.68, 36.64, 36.76, 37.91, 37.32, 37.57, 38.9, 36.54, 38.33, 39.12, 39.38, 37.64, 39.19, 38.28, 39.26, 40.8, 34.95, 41.25, 38.33, 41.51, 42.14, 41.88, 42.55, 41.49, 42.82, 42.0, 42.42, 43.48, 42.48, 43.3, 45.07, 42.33, 44.21, 43.24, 44.75, 44.71, 44.54, 42.77, 44.51, 43.62, 44.47, 45.26], 'val_loss': [4.4618912289856345, 4.362162629510187, 4.281647858346344, 4.191506754820514, 4.094492655650825, 3.9881456824624615, 3.89349313602326, 3.8115282271318374, 3.7492588674946195, 3.692820474600336, 3.6444620584986014, 3.5995639023507477, 3.5580318551154653, 3.5292665487641743, 3.4904019832611084, 3.449434950093555, 3.4127677595539456, 3.388700626458332, 3.3501164943549284, 3.3233655395021864, 3.285108279270731, 3.270154825441397, 3.2341503246574645, 3.207659710744384, 3.1822337922017288, 3.1474277760572495, 3.1169947044105286, 3.10328993979533, 3.067806980412477, 3.049007581297759, 3.0044755844553563, 2.9917227538527955, 2.9583237778608966, 2.9325004884391834, 2.9195022795610366, 2.896286167156924, 2.8635325219221177, 2.843113926565571, 2.8318380337611884, 2.811116538989316, 2.79278775233372, 2.769405276912033, 2.759064183872976, 2.710271612094466, 2.74030885423065, 2.710896227769791, 2.673065673014161, 2.6498220300978157, 2.6698359835679364, 2.6610589072962476, 2.6254373902727846, 2.6176918783005636, 2.5697269211908815, 2.554814410057797, 2.553489238593229, 2.5999911332586008, 2.5184889766061382, 2.5365722498316674, 2.496636530396285, 2.492127837648817, 2.4750216280578807, 2.4246158774491327, 2.4641372899340976, 2.4280650972560713, 2.430125984416646, 2.504551731856765, 2.3950531475103585, 2.3948943303648833, 2.3566500183882986, 2.464658516987114, 2.36919671942474, 2.3982301472098966, 2.334602908723673, 2.2888701987114684, 2.59782790302471, 2.280768811323081, 2.3862894393835856, 2.248491021478252, 2.2303362348277096, 2.2606346857775548, 2.2208216425719534, 2.2849220546187867, 2.232258704817219, 2.2183258009564346, 2.214152764362894, 2.1915481758725113, 2.227842687801191, 2.2391762505671022, 2.1520398991882423, 2.2502242721569767, 2.1624171870529274, 2.1957123454209344, 2.139849248965075, 2.133491260990216, 2.153878903692695, 2.244097254078859, 2.1444765367325704, 2.2148104101229626, 2.12020159830713, 2.1213336438889714]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="LSUV", optimizer="Adam")
data.update({'train_metric': [7.976, 15.05, 20.18, 25.548, 30.48, 34.278, 37.82, 40.382, 42.444, 44.082, 45.72, 46.888, 48.102, 48.848, 49.792, 50.582, 51.308, 51.804, 52.456, 53.06, 53.346, 53.83, 54.434, 54.672, 55.106, 55.352, 55.922, 55.914, 56.226, 56.594, 56.42, 56.696, 57.25, 57.414, 57.646, 57.878, 57.76, 57.908, 58.268, 58.352, 58.628, 58.762, 59.018, 59.108, 59.3, 59.244, 59.602, 59.762, 59.618, 60.038, 59.886, 59.994, 60.152, 60.172, 60.622, 60.51, 60.496, 60.566, 60.768, 60.636, 60.688, 60.784, 61.192, 60.854, 61.1, 61.322, 61.428, 61.488, 60.968, 61.752, 61.552, 61.642, 61.834, 61.892, 61.926, 61.97, 62.188, 62.036, 62.2, 62.052, 62.34, 62.246, 62.43, 62.708, 62.498, 62.618, 62.276, 62.56, 62.854, 62.742, 62.918, 62.832, 62.724, 62.956, 63.106, 63.338, 62.722, 63.298, 63.12, 63.072], 'train_loss': [4.032285703722473, 3.5341921340474083, 3.247744646828498, 2.945580739499358, 2.698624687121652, 2.5033009934913166, 2.3375375911098004, 2.215794831285696, 2.120347144048842, 2.0564796366655003, 1.988113615061621, 1.9347894139911817, 1.889188876847172, 1.856302844593897, 1.81619614987727, 1.7895620532352905, 1.7582458408592303, 1.7366433064346118, 1.7080384027927429, 1.6947697263849362, 1.6786517223433766, 1.6593839693862154, 1.6429380341563993, 1.623483271702476, 1.6118044572718002, 1.6028898146451283, 1.5825402384523846, 1.5782074143209726, 1.5620438098297704, 1.5542637487811506, 1.551729143732954, 1.5406253220480117, 1.5247473088676666, 1.5216643993202073, 1.5099267390987756, 1.5023186476639165, 1.495149449924069, 1.4926664278940167, 1.4843402839530155, 1.4789326261071598, 1.4699140799319959, 1.4651603994467068, 1.4589111509225559, 1.4496019766916095, 1.4447739063321476, 1.439283253134364, 1.435495647399322, 1.4293477516954818, 1.426694357867741, 1.4244074326037142, 1.4181624732511429, 1.4193562405646, 1.4123720087663596, 1.4121776130955543, 1.3978770736538235, 1.400001488606948, 1.4001530693924946, 1.3939021334166417, 1.3867324112778734, 1.3908678967019785, 1.3862935989866476, 1.3824140866241796, 1.371196001844333, 1.3762323699338967, 1.371927764790747, 1.3705109390608794, 1.3647758580382219, 1.3615108572156227, 1.3662520143229637, 1.3561881573303887, 1.3540715163626025, 1.352098982657313, 1.3527096613593723, 1.3445663574863882, 1.3436290229982732, 1.3428998500337381, 1.3410144482579682, 1.341412378546527, 1.3356148748446608, 1.3334125503707115, 1.3276565480415168, 1.328841596918033, 1.319143984400098, 1.3178767782190572, 1.3193110594206758, 1.3198717032247187, 1.3142221353547958, 1.3151668420685527, 1.3081721161942348, 1.3186363274484034, 1.3124863272127898, 1.311296302720409, 1.3114064804580816, 1.3070305928854686, 1.2996010596642409, 1.2986379909088543, 1.3071774636083247, 1.2959520961622448, 1.295350589791832, 1.2964679856434502], 'val_metric': [11.64, 15.78, 21.09, 26.19, 31.6, 31.36, 38.76, 38.48, 40.27, 42.82, 43.48, 40.99, 46.57, 45.79, 46.75, 48.57, 49.67, 50.13, 47.01, 49.62, 48.52, 49.97, 51.15, 51.12, 51.13, 50.37, 51.61, 51.72, 50.65, 54.14, 53.2, 53.09, 52.11, 53.59, 54.04, 50.14, 53.1, 52.11, 55.23, 52.69, 50.7, 54.7, 54.01, 55.02, 54.08, 52.15, 49.91, 53.54, 52.87, 53.96, 54.28, 52.35, 55.23, 55.38, 55.04, 55.39, 57.12, 55.19, 55.31, 53.16, 54.78, 56.28, 51.31, 56.66, 56.42, 54.59, 56.94, 54.69, 55.26, 55.21, 55.93, 55.92, 53.38, 55.96, 55.63, 53.57, 56.12, 54.3, 55.97, 55.68, 56.03, 56.24, 57.59, 55.79, 58.38, 55.91, 56.17, 55.87, 58.08, 51.37, 56.69, 57.55, 56.48, 55.27, 58.63, 55.05, 55.16, 57.37, 55.75, 56.6], 'val_loss': [3.745501629106558, 3.497502809876849, 3.1906767924120474, 2.9192636119332285, 2.6437079784976447, 2.6068353698511793, 2.300331904629993, 2.3090315085307807, 2.2300246885627697, 2.084906825594082, 2.075164671916111, 2.191370391541985, 1.956163914340317, 2.0078739049328362, 1.9309393721780959, 1.869614118223737, 1.8158351927046563, 1.7897958368252798, 1.907270481632014, 1.8076688195489774, 1.882026293475157, 1.7708340177110806, 1.7591135782800662, 1.7768638855332781, 1.7581226491624382, 1.816833636563295, 1.7356733994878781, 1.7174954558633695, 1.779751753351491, 1.6667778848842452, 1.6795020361614834, 1.685942600487144, 1.7279778643018882, 1.6478231780847925, 1.6413224950717513, 1.8164820534408472, 1.6701294009093266, 1.7379575094599633, 1.6186674370127878, 1.6882674010695926, 1.78376494851082, 1.6110840443592922, 1.658804217341599, 1.6294980838799933, 1.6653873874882983, 1.7572115257287482, 1.8022165116231152, 1.6377089805663771, 1.6873281252611974, 1.6590500910570667, 1.6435592022671062, 1.7034579629351378, 1.6049748157999317, 1.5944978856736687, 1.619484277287866, 1.5954421416969056, 1.543159978404926, 1.6230126300435157, 1.5971881842157643, 1.7140620119252783, 1.6324077085324913, 1.5537477800041248, 1.7875825850067624, 1.5252902652048002, 1.5329263897458458, 1.6394670199436747, 1.5334534246450777, 1.651793444991871, 1.585367947627025, 1.6038792778731912, 1.592210082491492, 1.5829487203792403, 1.6597906275159995, 1.5899328395819208, 1.5593939419764622, 1.709602295972739, 1.5694778398343712, 1.6598080791485537, 1.5702337588474249, 1.5967357545901255, 1.6137081832642768, 1.5636434243742827, 1.514995166450549, 1.5901693940921953, 1.4972583962853547, 1.5843803438411397, 1.595618302655068, 1.616473781834742, 1.4970751180770292, 1.7848621925730614, 1.5497282189168748, 1.5191529799418844, 1.5615790353459158, 1.6200459254015782, 1.497383413800768, 1.6258156982956418, 1.599162262716111, 1.5102302261218903, 1.6020351207939683, 1.550196419475944]})
data = database.get_data(model="ResNet-50", dataset="CIFAR-100", initialization="LSUV", optimizer="AdamW")
data.update({'train_metric': [9.032, 20.97, 32.422, 42.598, 50.168, 56.148, 60.48, 64.472, 68.086, 71.512, 74.314, 77.3, 79.996, 82.362, 84.77, 86.006, 88.47, 89.934, 91.55, 92.482, 93.462, 94.04, 94.662, 94.972, 95.52, 95.86, 96.264, 96.446, 96.594, 96.824, 97.232, 97.03, 97.18, 97.436, 97.702, 97.562, 97.802, 97.916, 97.92, 97.908, 98.08, 98.108, 98.222, 98.266, 98.266, 98.546, 98.472, 98.33, 98.368, 98.776, 98.446, 98.56, 98.692, 98.572, 98.764, 98.694, 98.772, 98.938, 98.826, 98.89, 98.806, 98.908, 99.016, 98.868, 98.844, 98.882, 99.302, 98.88, 98.964, 99.156, 99.152, 98.776, 99.23, 99.062, 99.24, 99.086, 99.41, 99.128, 99.1, 99.218, 99.206, 99.264, 99.368, 99.084, 99.206, 99.324, 99.162, 99.284, 99.284, 99.348, 99.416, 99.206, 99.28, 99.362, 99.342, 99.368, 99.278, 99.588, 99.25, 99.35], 'train_loss': [3.9607857087688982, 3.2073139558972605, 2.621264906642992, 2.135722316134616, 1.811540845257547, 1.566540926008883, 1.379217557894909, 1.2221575579832278, 1.086544133132071, 0.9586495149623403, 0.8529181890475476, 0.7443344728339969, 0.6489126835485248, 0.561951392999543, 0.47887948671798874, 0.4344721966901856, 0.35214897442389936, 0.3124920879788411, 0.26060543491331206, 0.23239165401119558, 0.1981221035818386, 0.18496733517064462, 0.16211390069893103, 0.15480638159643811, 0.13789761560442654, 0.12981334048063706, 0.11771939618184286, 0.11180382749289655, 0.10577098670346505, 0.10112108482772013, 0.08657990590981243, 0.0924607744092679, 0.08844047343677572, 0.08212266123527304, 0.07355346224840036, 0.07620769624343461, 0.07060149085351869, 0.06740219560165978, 0.06655958708604355, 0.06724616543144521, 0.062177342096226926, 0.05912941377437281, 0.05581491205977071, 0.05706806908718303, 0.05532095900765332, 0.05036445468892831, 0.049178847838240815, 0.05221619239509287, 0.05161205306440792, 0.040525933412615865, 0.05027874181171536, 0.0456909852157778, 0.04331667120392789, 0.044695885612151785, 0.04186411865372354, 0.04195054021992666, 0.03962890626511315, 0.037192534714433435, 0.03961116546164136, 0.03645656570030586, 0.03835504699964076, 0.035512525946005485, 0.03163825325272463, 0.03548148518060203, 0.03728120959178685, 0.035661070472727555, 0.02449075244103625, 0.03689023289044061, 0.03335340483226668, 0.027940991469641047, 0.02853871489357849, 0.03902999848528418, 0.02596947349011874, 0.030990136731544014, 0.02597362847130536, 0.030356962356003016, 0.02152033031391411, 0.028172517085330005, 0.028613708693298327, 0.02646297762644432, 0.0268223976927495, 0.02466318349587991, 0.021277929228433357, 0.02948574986803206, 0.02636486609883802, 0.022231062126244403, 0.027997694645814485, 0.023929013089319127, 0.02382866275826376, 0.02196092520258538, 0.018893602696541265, 0.025664914924594218, 0.023428726619939604, 0.021340895383453528, 0.020361051404909136, 0.02081776845592397, 0.023797625443264556, 0.01500469134430728, 0.02414401041352294, 0.021305379404997826], 'val_metric': [13.33, 28.15, 37.42, 46.23, 50.41, 54.06, 56.79, 59.15, 61.76, 63.29, 64.03, 65.88, 66.6, 66.67, 66.82, 67.0, 67.69, 67.17, 67.87, 67.3, 67.47, 67.81, 67.78, 68.29, 68.17, 67.95, 67.97, 67.93, 68.18, 68.54, 67.74, 68.19, 67.89, 68.8, 69.28, 68.07, 68.74, 67.52, 68.86, 69.0, 68.78, 66.9, 69.03, 69.69, 68.62, 69.64, 69.48, 69.77, 69.41, 69.17, 67.88, 69.76, 68.19, 67.88, 68.59, 69.72, 69.0, 67.92, 70.03, 69.23, 70.04, 69.84, 69.5, 69.83, 69.34, 69.62, 70.07, 69.38, 70.83, 70.36, 69.47, 67.89, 70.2, 69.01, 69.07, 70.11, 70.29, 70.12, 69.62, 70.23, 69.7, 69.48, 70.85, 69.35, 69.03, 71.03, 70.35, 70.03, 69.91, 69.62, 70.28, 69.85, 70.42, 71.41, 70.21, 69.89, 71.31, 69.98, 69.11, 70.75], 'val_loss': [3.6555422354655662, 2.83356752668976, 2.383231882077114, 1.98436790287115, 1.8289184608277242, 1.6640037844894797, 1.5360492429915507, 1.4494351228331304, 1.356097623041481, 1.317059827458327, 1.2979248496377545, 1.2377320687482312, 1.213230976253558, 1.271909270316932, 1.280959388253036, 1.2921467248801213, 1.3473936524360803, 1.3821298673654059, 1.426090758317595, 1.4793542494439775, 1.4763275259619306, 1.4870650616421062, 1.5874284904473905, 1.50992301467118, 1.5463118921419619, 1.567922873102176, 1.6832832671274804, 1.597733010152343, 1.5545197239347324, 1.5912317842434927, 1.7305217510575701, 1.6656902098351982, 1.6744376561444276, 1.6543247904747156, 1.6291771663981638, 1.7366077877154016, 1.6934139709563771, 1.766230543327939, 1.6769887167177382, 1.712520517361392, 1.7795357776295608, 1.8174144762336828, 1.6840401720848812, 1.721000660756591, 1.7465893702142556, 1.7168340390654886, 1.7187416826843456, 1.6741311515972113, 1.697691643313997, 1.7562013636728762, 1.8350964336638238, 1.6909223096385884, 1.8158450904925159, 1.8288459231139749, 1.8275340324754168, 1.7295393332554276, 1.7464436865915918, 1.860153814789596, 1.707145281658051, 1.8145420817053242, 1.7209333800206519, 1.7212970533948035, 1.8306755871529792, 1.7503801603226146, 1.8264542525740945, 1.812772618357543, 1.8089917894381626, 1.829369979679205, 1.7237795986187685, 1.7821911064682492, 1.8318527094118155, 1.9226578215884556, 1.7301733406486026, 1.7791175003264361, 1.8739477885756524, 1.832988418211603, 1.8010065453067707, 1.802853491275933, 1.8691610469939604, 1.7763736988328824, 1.8275307838324528, 1.8105866871062357, 1.81255770944486, 1.822230757801396, 1.8811155075480224, 1.7209627089227082, 1.7430769599926699, 1.8227243514577294, 1.830581390174331, 1.8410012725811855, 1.8687792511502648, 1.7965621803976168, 1.8138933500666528, 1.7648720737475498, 1.8575260365844533, 1.8442643565736758, 1.7315883750368835, 1.9508032920254263, 1.9529242025818794, 1.797692577930013]})


# MobileNetV3 Tiny-Imagenet
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="SGD")
data.update({'train_metric': [0.526, 0.723, 0.774, 0.849, 0.916, 0.995, 1.125, 1.281, 1.51, 1.573, 1.631, 1.618, 1.72, 1.807, 1.867, 1.98, 1.998, 2.14, 2.159, 2.3, 2.396, 2.526, 2.671, 2.791, 3.021, 3.348, 3.516, 3.679, 3.86, 4.074, 4.177, 4.244, 4.544, 4.616, 4.862, 4.999, 5.15, 5.313, 5.566, 5.74, 5.905, 6.217, 6.417, 6.779, 6.968, 7.093, 7.416, 7.631, 7.917, 8.139, 8.406, 8.657, 8.9, 9.088, 9.344, 9.567, 9.797, 10.005, 10.332, 10.498, 10.649, 10.868, 11.238, 11.375, 11.599, 11.818, 12.1, 12.312, 12.418, 12.735, 13.034, 13.126, 13.433, 13.431, 13.707, 14.03, 14.157, 14.195, 14.585, 14.701, 14.89, 14.814, 15.165, 15.314, 15.577, 15.725, 16.045, 16.075, 16.274, 16.423, 16.616, 16.874, 17.059, 17.198, 17.541, 17.565, 17.841, 18.112, 18.182, 18.181, 18.419, 18.64, 18.747, 19.167, 19.162, 19.465, 19.576, 19.633, 19.784, 19.972, 20.102, 20.527, 20.654, 20.654, 20.789, 21.087, 21.256, 21.395, 21.637, 21.85, 22.049, 22.214, 22.351, 22.331, 22.565, 22.807, 22.933, 23.206, 23.437, 23.543, 23.686, 23.871, 23.93, 24.444, 24.403, 24.606, 24.473, 24.929, 24.992, 25.342, 25.298, 25.513, 25.675, 26.026, 25.939, 26.137, 26.323, 26.53, 26.698, 26.976, 26.784, 27.284, 27.474, 27.56, 27.437, 27.98, 27.907, 28.267, 28.409, 28.503, 28.652, 28.739, 29.0, 29.09, 29.404, 29.453, 29.528, 29.848, 30.09, 30.201, 30.236, 30.29, 30.772, 30.651, 30.685, 31.024, 31.151, 31.236, 31.354, 31.422, 31.658, 31.828, 31.935, 32.221, 32.248, 32.633, 32.631, 32.553, 32.756, 33.062, 33.075, 33.296, 33.4, 33.534, 33.68, 33.848, 33.781, 34.123, 34.197, 34.245], 'train_loss': [5.297552117611915, 5.2919997744886675, 5.284817128172305, 5.278353198399852, 5.271559194807661, 5.261790013511594, 5.247729072644019, 5.230639195945579, 5.2105561193562595, 5.185189332248153, 5.157250807976311, 5.128213997460754, 5.101924422758936, 5.076679728539114, 5.052995036522395, 5.031181084064818, 5.011272366658587, 4.9886822322203574, 4.969554516114414, 4.946889428854446, 4.927257413064831, 4.9075844536511015, 4.889401205823121, 4.870350524087175, 4.852957207807271, 4.835167635882884, 4.81618200657235, 4.799577875009196, 4.781230639740205, 4.760964457682135, 4.742240028967098, 4.722650129827108, 4.7023590988481345, 4.6823922375113405, 4.662841189464391, 4.6400315891224375, 4.617912324208597, 4.594484120245096, 4.573365650799361, 4.548950009519903, 4.524328456654125, 4.497348175549156, 4.471953147051049, 4.445314763374841, 4.419830336451759, 4.394189308487446, 4.369760562880864, 4.345275868838671, 4.3226493800670305, 4.298540360639283, 4.278328282361754, 4.25811628584517, 4.238025720654888, 4.2189831428625455, 4.198478225706787, 4.182067919753716, 4.161849734879272, 4.144574432592703, 4.125655786547231, 4.109523503046652, 4.093113754013754, 4.074162784480011, 4.055493474235462, 4.041071200477566, 4.021368870732117, 4.001983356262276, 3.9848178915693757, 3.9721715332831775, 3.9530346057999233, 3.9375871275757186, 3.916614953783637, 3.90467255220761, 3.8860551206560676, 3.872406584287559, 3.85675773236207, 3.840835909773277, 3.827264048697776, 3.8120544981819195, 3.7944364105366164, 3.7814745445397904, 3.7673397476064494, 3.75119673710028, 3.738023841220907, 3.7278215406189648, 3.711563288974823, 3.699439034580956, 3.6826620809710033, 3.677871798675791, 3.6612354359105086, 3.6475594605075496, 3.6354171622661315, 3.623815300978687, 3.609518189347866, 3.597361132035405, 3.584308848500023, 3.5738374146603653, 3.5633846117911703, 3.54776167183142, 3.5378666645582104, 3.525012831312681, 3.51433682838313, 3.5045902611579334, 3.4912457913064987, 3.477311308080389, 3.466890946154555, 3.4559934676189266, 3.441756414894255, 3.4317079818332643, 3.4225076767243565, 3.4129066166783173, 3.401191234131006, 3.385839968710013, 3.3788531180459267, 3.368694379401375, 3.359704447189204, 3.3465010299182287, 3.336144174129171, 3.3254612236547683, 3.3148133541175517, 3.3034963838122096, 3.2932227011758095, 3.281999235647425, 3.2692274069526754, 3.2654973307406374, 3.2522766760962174, 3.237201206057177, 3.231010500773053, 3.218047681559528, 3.207006486348441, 3.195795751578977, 3.1884064642344234, 3.179770613967495, 3.1643770924761596, 3.1555845127300963, 3.1474837976164034, 3.13219493089848, 3.128400100474928, 3.116695488711923, 3.1082231246425946, 3.0904052060762432, 3.084921787125288, 3.0750939208425474, 3.0645934440200326, 3.0575248060208136, 3.047317117281015, 3.0399759857607282, 3.027932105885052, 3.0156637353158806, 3.0098521256401076, 2.9966476184774193, 2.9894983141527525, 2.9811774476247193, 2.9682268066735733, 2.96079860188148, 2.952513791167881, 2.9452550000894244, 2.9350440001838574, 2.920653207593443, 2.9148780794686715, 2.9046833359577375, 2.8961128560076594, 2.888402662441964, 2.879721769368275, 2.8703849715295697, 2.861578245156862, 2.852052507382208, 2.8468285993513813, 2.8344132070425454, 2.8237518605252374, 2.8191093983592244, 2.8058049250167683, 2.795651793708728, 2.7866994787009953, 2.7821255436480543, 2.7771421848621722, 2.7648097567274568, 2.7589462753754734, 2.7475162388343346, 2.7422637614544887, 2.7367039028834017, 2.7188550017998145, 2.714620495025576, 2.707105559716984, 2.699932252910758, 2.6937555400362703, 2.684412247846314, 2.6740436235148404, 2.669236993606626, 2.6611375680582996, 2.6510778172078684, 2.6431968061114, 2.6368866277976593, 2.630144612726613, 2.6213158253897326, 2.6147349330758103, 2.6057815272611298, 2.603812773717342, 2.591965840630095, 2.5825266739686024, 2.5777235694672918], 'val_metric': [0.76, 0.8, 0.92, 0.89, 0.94, 1.1, 1.24, 1.61, 1.64, 1.63, 1.63, 1.65, 1.73, 1.95, 1.94, 2.03, 2.08, 2.22, 2.24, 2.39, 2.59, 2.77, 3.09, 3.26, 3.65, 3.84, 3.85, 4.04, 4.33, 4.37, 4.36, 4.73, 4.61, 4.95, 4.62, 5.35, 5.21, 5.69, 5.86, 5.85, 6.01, 6.29, 6.51, 6.83, 6.63, 7.3, 7.59, 7.77, 8.09, 7.98, 8.55, 8.97, 9.26, 8.96, 9.5, 9.85, 9.84, 9.8, 9.78, 9.86, 10.23, 10.32, 10.63, 10.85, 10.69, 11.45, 11.4, 11.3, 11.76, 11.76, 12.3, 12.19, 11.82, 12.21, 12.0, 12.7, 12.46, 12.8, 13.05, 13.24, 12.84, 13.41, 13.41, 13.32, 13.54, 13.41, 13.64, 14.16, 13.95, 13.28, 14.03, 14.51, 14.56, 13.94, 14.53, 14.74, 14.86, 14.42, 14.73, 14.88, 15.04, 14.83, 14.74, 15.5, 15.15, 15.09, 15.3, 14.77, 15.43, 15.35, 15.35, 15.91, 15.86, 15.61, 16.15, 15.97, 16.06, 15.67, 16.07, 16.52, 15.86, 16.04, 16.03, 16.66, 15.33, 15.93, 16.36, 16.42, 16.59, 16.79, 16.92, 16.98, 16.48, 16.52, 16.81, 17.0, 17.2, 17.18, 17.23, 17.08, 16.25, 17.33, 17.1, 15.93, 17.08, 17.56, 17.11, 16.56, 17.15, 16.52, 17.36, 17.44, 17.29, 16.86, 17.54, 17.06, 16.41, 17.51, 17.5, 17.4, 17.36, 17.45, 17.16, 17.11, 17.71, 17.23, 17.22, 16.7, 17.8, 17.87, 17.83, 17.42, 17.22, 18.04, 17.67, 17.25, 17.88, 17.43, 17.69, 17.78, 17.89, 17.09, 17.21, 17.11, 17.96, 17.35, 17.5, 17.94, 18.0, 17.98, 18.04, 17.42, 17.88, 17.4, 17.87, 17.32, 17.66, 17.58, 17.52, 17.58], 'val_loss': [5.29249564249804, 5.290506856456684, 5.300896252796149, 5.296515076023758, 5.28205629518837, 5.269596925966299, 5.240988316809296, 5.22113243030135, 5.196735254518545, 5.173361928599655, 5.144773126407793, 5.119458231956336, 5.091925695443609, 5.061695563565394, 5.040837790556015, 5.025251763641455, 4.993851418707781, 4.970982439199071, 4.954142404969331, 4.928634997386082, 4.913526196388682, 4.894295640811799, 4.877582188624485, 4.858387821039576, 4.837781749713193, 4.829930231070063, 4.821326579257941, 4.78911243274713, 4.768762638614436, 4.760835199599054, 4.748411327410656, 4.715425029681747, 4.706931496881376, 4.673376795592581, 4.682168763154631, 4.645797601930655, 4.632167053830092, 4.608230312918402, 4.573490460207508, 4.569380441289039, 4.52273126468537, 4.53998177370448, 4.478883638503445, 4.446347329267271, 4.446925284756217, 4.4052480269389545, 4.380906653252377, 4.367288247794862, 4.339663198799085, 4.3300828554068405, 4.317977057900398, 4.286028398829661, 4.271035912689889, 4.270907599455232, 4.252100195854333, 4.2210339254634395, 4.243478205553286, 4.198840352380351, 4.211233837589337, 4.192616341220345, 4.168440732227009, 4.153625325792155, 4.144370753294344, 4.121260023420783, 4.115700856895204, 4.108877488761951, 4.083945713225444, 4.1167328706972155, 4.061166698006308, 4.066641640511288, 4.024574879628078, 4.037975126011356, 4.04841095019298, 4.022644422616169, 4.033233224965964, 4.009909316992304, 3.9852694280587944, 3.971642383344614, 3.990980136166712, 3.972272280674831, 3.996693810080267, 3.9395623723412774, 3.9317888727613317, 3.938888534618791, 3.9286027258368814, 3.928451091620573, 3.9110428087270943, 3.9170715915169687, 3.917639853848014, 3.9646580811518772, 3.884097822152885, 3.881223239716451, 3.8738881691246276, 3.9220910148256145, 3.8648876794584237, 3.877620009100361, 3.8660657329923787, 3.881956990357417, 3.90514649555182, 3.876056873115005, 3.862439945245245, 3.882220295584126, 3.8666473482824433, 3.8699619557447495, 3.8394829938366155, 3.869152532261648, 3.861623492210534, 3.8806077128003356, 3.847813756602585, 3.8706091528485533, 3.8799503973335217, 3.8540132106489438, 3.8436281802547967, 3.8362146805805764, 3.8347371611625527, 3.8332783697517057, 3.8307298125734754, 3.888488096796024, 3.8277084265544916, 3.8491000969698477, 3.86420965953997, 3.854099247865616, 3.8291844607918124, 3.8608929564239114, 3.8803024033831943, 3.8420444795280506, 3.8434196004442347, 3.8342002440410057, 3.8274279885990605, 3.8590262862527447, 3.8420115489109308, 3.8228397749032186, 3.885652911131549, 3.876336793231357, 3.824208300584441, 3.830554017595425, 3.863545508901025, 3.826978809514623, 3.8345789013394884, 3.852833468443269, 3.9039754381605016, 3.8260306157883566, 3.867018344296012, 3.9362019748444768, 3.854389284826388, 3.8543128876169774, 3.8696471870325175, 3.91984617786043, 3.875888423555216, 3.884818373971684, 3.8538088069599903, 3.8687066120706546, 3.8838279140982657, 3.9027660697888416, 3.953591963288131, 3.9300043172897046, 3.9652548990431864, 3.910113655078183, 3.9481445892601257, 3.8496630579043347, 3.8863714318366567, 3.907594169021412, 3.9304065916948256, 3.91401607975079, 3.93570567088522, 3.9671586759530815, 3.9492287195412215, 3.929378955986849, 3.9157352918272563, 3.946719971432048, 3.926782328611726, 3.9881851004946762, 3.9918913476786035, 3.9823433927669645, 3.969409438455181, 3.961025230444161, 4.018874377961371, 3.98748273454654, 4.004763246341875, 3.9970776169163407, 4.004430725316333, 4.106138715318814, 3.993606565864223, 4.057750615344685, 4.031805501621999, 4.043367928000772, 4.043745189715343, 4.040703076465874, 4.047458080729102, 4.083124320218517, 4.077111965531756, 4.083286863983057, 4.071297218845149, 4.107875318284247, 4.10532503978462, 4.156900565335705, 4.089459161090243, 4.149831278308941, 4.179594367932362, 4.154021190989549]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="Adam")
data.update({'train_metric': [5.809, 11.676, 14.813, 16.566, 18.39, 19.728, 20.909, 22.003, 23.126, 23.987, 24.816, 25.523, 26.087, 26.637, 27.25, 27.55, 28.204, 28.413, 28.898, 29.39, 29.416, 29.827, 30.255, 30.53, 30.917, 31.238, 31.358, 31.654, 31.879, 32.148, 32.44, 32.745, 32.933, 32.87, 33.288, 33.447, 33.533, 33.685, 33.695, 33.911, 34.307, 34.377, 34.516, 34.679, 34.544, 34.876, 35.182, 35.216, 35.301, 35.545, 35.585, 35.678, 35.834, 35.897, 35.938, 36.008, 36.11, 36.159, 36.445, 36.485, 36.373, 36.642, 36.581, 36.642, 36.887, 36.827, 36.901, 36.993, 36.94, 37.057, 37.012, 37.039, 37.255, 37.035, 37.398, 37.125, 37.354, 37.283, 37.448, 37.639, 37.464, 37.636, 37.555, 37.627, 37.729, 37.632, 37.601, 37.634, 37.825, 37.603, 37.887, 37.86, 37.856, 37.945, 37.873, 37.877, 38.063, 37.981, 38.254, 38.031, 37.917, 38.319, 38.29, 38.328, 38.097, 38.356, 38.374, 38.465, 38.37, 38.344, 38.421, 38.717, 38.434, 38.744, 38.834, 38.68, 38.73, 38.642, 38.812, 38.627, 38.902, 38.881, 38.829, 38.832, 38.687, 38.869, 38.956, 38.95, 38.839, 39.048, 38.942, 38.989, 39.15, 39.019, 38.965, 39.144, 39.148, 39.241, 39.26, 39.217, 39.314, 39.477, 39.332, 39.364, 39.195, 39.368, 39.413, 39.416, 39.185, 39.37, 39.399, 39.406, 39.401, 39.606, 39.475, 39.436, 39.47, 39.533, 39.568, 39.741, 39.472, 39.475, 39.553, 39.56, 39.617, 39.582, 39.515, 39.642, 39.581, 39.685, 39.571, 39.681, 39.912, 39.657, 39.693, 39.891, 39.795, 39.783, 39.765, 39.765, 40.05, 39.652, 39.788, 39.787, 39.981, 40.181, 39.862, 39.846, 39.957, 40.012, 39.967, 39.98, 39.725, 39.975, 40.086, 39.982, 39.924, 40.043, 40.016, 40.021], 'train_loss': [4.525078131766634, 3.975579113359262, 3.7425775593545905, 3.6310306708933218, 3.532135993795218, 3.448303701476416, 3.383016719470326, 3.31309449832865, 3.2577850624909406, 3.207692245604209, 3.1690912487715845, 3.134971033359901, 3.099012771784611, 3.073456138200815, 3.045862819625259, 3.0226544609301684, 3.0034325461470006, 2.98128330379591, 2.9589898040183296, 2.941965750067644, 2.9246879217339417, 2.907034616018821, 2.8879888149232187, 2.87942345128636, 2.86232884374095, 2.847543157680021, 2.832120761914049, 2.8267772954317216, 2.8100079904362243, 2.798787450149741, 2.786150317732066, 2.7769229196998757, 2.765811109878433, 2.759276373906541, 2.7517547078111275, 2.7406143859953582, 2.7333367624423173, 2.7264922235687803, 2.720478506936374, 2.710549502058511, 2.6981652705850925, 2.690011220487813, 2.685037913569562, 2.679790234764036, 2.6772232026681637, 2.6693555210464974, 2.6620056388169164, 2.654511561930675, 2.652136948958316, 2.6408032872168894, 2.6363547408115537, 2.635372689802984, 2.629273056144983, 2.623664534328385, 2.6242297276883115, 2.61608590869208, 2.6142059963480144, 2.6097418059924435, 2.6034768418631185, 2.598202390771452, 2.6041268174341683, 2.5943247696869816, 2.5943669989104463, 2.588855688341596, 2.5874091573655416, 2.586244436616098, 2.5807064102768362, 2.5782143385533867, 2.5736898777199646, 2.572690636853873, 2.5696585310321547, 2.5670656630875435, 2.5655740925080486, 2.566651408365729, 2.559370459003168, 2.5604535486937636, 2.556471862704496, 2.5564824280949336, 2.552689655728624, 2.5511810840587166, 2.5470418642517547, 2.544131648822694, 2.539859492353194, 2.541387170076523, 2.541785703739598, 2.54180298626461, 2.538364012044588, 2.536823475200704, 2.5325187282995465, 2.535112415302738, 2.5301439857452404, 2.5298163097635418, 2.529913836767181, 2.5291565500309434, 2.5262189893026954, 2.527363195605409, 2.5203410355768674, 2.520253883442357, 2.519730828316335, 2.5203938382753406, 2.5209471695863965, 2.5139791093723787, 2.517780686675625, 2.513978938452342, 2.5143295836158845, 2.5123665013987515, 2.5095191133838393, 2.506053958271683, 2.503615802934516, 2.504627357899037, 2.4998126433434122, 2.497726584807925, 2.501283457244121, 2.496286138920775, 2.4947676397788547, 2.4974166326467913, 2.4903643817681953, 2.4935578964340785, 2.4905000551343344, 2.494165618070333, 2.4900862883087616, 2.49178838462915, 2.4893656464166085, 2.484530885480416, 2.4906467812532656, 2.4863445438487517, 2.4813505664362943, 2.481511414394269, 2.481140972632898, 2.479954466709935, 2.479801589376409, 2.4780872642269367, 2.473381116233113, 2.4789507083227753, 2.4757337646459967, 2.471564138843246, 2.4751009592549282, 2.472914480430837, 2.472335875530084, 2.4745969615223617, 2.4698270563887084, 2.4692205176930053, 2.4670967019984755, 2.46728364885883, 2.4642085812447245, 2.4652328480533203, 2.467412180879218, 2.465270234618672, 2.465666089924344, 2.4634208626573235, 2.4604577307356372, 2.460630298423523, 2.4608630475827082, 2.4536004190786636, 2.4593331112132497, 2.4587377071075536, 2.4582499518885648, 2.457097585965484, 2.460790957766928, 2.453678943984263, 2.456784588819273, 2.453051724040348, 2.452948499397063, 2.4550889227074535, 2.4553149539693684, 2.451298190627583, 2.4553728118891605, 2.4509211443054775, 2.4529221097552005, 2.4505511172978602, 2.449621903537865, 2.4482514492152063, 2.4469639649394224, 2.449612206628669, 2.4448459553398236, 2.445161197251108, 2.445280766761692, 2.4480135282185773, 2.445357817987258, 2.4437731982650317, 2.443824025658713, 2.4448579108951494, 2.4406664333355708, 2.4440851007145485, 2.4452221468329354, 2.4398233664012916, 2.4406133650665627, 2.441445962099867, 2.4362205215089228, 2.4394257472099894, 2.4368734173643536, 2.4341220218099546, 2.4385166991923914, 2.4354197777621804, 2.4334227253974285, 2.433511096319173, 2.43605716146114, 2.4348675192584612, 2.4330175008365, 2.4336590762330568], 'val_metric': [5.56, 12.6, 15.2, 15.18, 16.78, 17.59, 17.74, 19.91, 19.92, 22.17, 20.71, 23.31, 25.0, 26.81, 23.98, 24.59, 24.35, 25.79, 25.87, 26.97, 25.55, 27.11, 25.71, 25.25, 25.29, 27.58, 27.56, 29.43, 29.17, 28.75, 29.32, 28.49, 31.71, 28.2, 29.51, 30.24, 30.06, 29.58, 26.0, 30.0, 29.76, 31.16, 28.84, 31.22, 31.25, 30.37, 32.79, 29.9, 29.17, 31.95, 31.08, 31.39, 31.35, 32.9, 33.46, 31.5, 32.02, 33.61, 27.65, 29.23, 31.93, 30.64, 31.88, 32.78, 30.65, 28.66, 31.27, 31.92, 30.53, 30.78, 31.35, 35.49, 33.18, 31.09, 30.72, 31.32, 32.16, 32.39, 31.63, 33.8, 32.6, 34.96, 33.7, 34.88, 32.27, 35.87, 31.72, 32.92, 33.26, 31.44, 27.86, 30.21, 30.91, 35.32, 34.24, 33.64, 33.59, 31.48, 32.8, 35.15, 34.53, 32.12, 32.66, 32.94, 30.67, 33.83, 33.01, 36.13, 33.62, 34.08, 33.68, 34.74, 33.79, 33.42, 32.99, 34.71, 34.85, 32.9, 31.52, 33.46, 32.9, 33.58, 33.25, 33.84, 32.52, 34.13, 33.63, 32.57, 34.19, 34.59, 30.28, 34.74, 34.63, 34.36, 35.5, 31.97, 33.93, 33.77, 32.26, 34.03, 28.28, 34.8, 32.88, 35.74, 29.86, 36.46, 35.26, 33.72, 34.01, 34.01, 34.77, 34.09, 30.53, 33.11, 32.76, 35.62, 35.48, 36.52, 35.58, 34.88, 35.12, 34.97, 32.62, 35.81, 32.76, 32.68, 34.69, 35.37, 38.11, 34.89, 34.24, 33.44, 32.55, 30.98, 36.01, 34.15, 31.18, 35.28, 34.18, 33.51, 33.63, 34.05, 37.76, 35.15, 32.76, 33.7, 33.96, 35.68, 34.5, 33.16, 34.98, 35.49, 37.76, 31.25, 36.36, 34.29, 35.09, 35.1, 33.27, 34.23], 'val_loss': [4.505256177513463, 3.9124395892878248, 3.7188593498460807, 3.7226533244369895, 3.6359358828538544, 3.5699912590585696, 3.5920051845015992, 3.4808334028644925, 3.4370654996033685, 3.3520366667182584, 3.4528600273618273, 3.2616868330414888, 3.180702406889314, 3.0722239700851928, 3.239877088814025, 3.1836464837857874, 3.2113262004913037, 3.122003519610994, 3.1420680876750096, 3.075645174190497, 3.1376649422250735, 3.0822479208563545, 3.1099495470144185, 3.249220595238315, 3.1694738249869863, 3.0297583508643373, 3.0561940950952518, 2.9530275414703757, 2.9669703533695, 2.970889505307386, 2.9509699822990756, 2.9826297046272616, 2.837098623536954, 3.0192085736116785, 2.9525614027764386, 2.9082205994113997, 2.9440609452071462, 2.94211848222526, 3.2167553104412785, 2.944658950635582, 2.962214474465437, 2.8804946887265346, 2.9857054288220253, 2.877071077656594, 2.8656657934188843, 2.9021782449855924, 2.810776831238133, 2.9701568958865607, 2.9922432200923845, 2.8242425113726575, 2.846514808144539, 2.8555735433177585, 2.8460431296354645, 2.7810797349662537, 2.7433803855993184, 2.88492512095506, 2.870753121224179, 2.761730437066145, 3.1098409860756746, 3.071888703069869, 2.829832111194635, 2.8917932373702904, 2.8429657311955836, 2.76666238733158, 2.936555355977101, 2.967836060341756, 2.8990039255968325, 2.850854081712711, 2.9753320892904975, 2.9427099645517436, 2.876989844498361, 2.6678757333451775, 2.7950175370380377, 2.867198554573545, 2.9254401671658656, 2.8493617848985515, 2.8508890593887135, 2.7809008709184684, 2.857742424983128, 2.794266027250108, 2.8408452143335037, 2.692625843035947, 2.765739806138786, 2.6921008369725223, 2.845360644304069, 2.654314730577408, 2.8650080075689184, 2.7878856059092625, 2.757913894714064, 2.903852184107349, 3.131778710966657, 2.9490278936495447, 2.8724337056943567, 2.6824374290028956, 2.758878998695665, 2.813408552461369, 2.790858545121114, 2.925199116111561, 2.7969657881244734, 2.696978397050481, 2.7332774188108506, 2.8595053534598867, 2.816384852312173, 2.803706447030329, 2.9243196271787024, 2.7428715312556857, 2.815242840226289, 2.6563908814624617, 2.7723312962586713, 2.7567363567413037, 2.7656705592088637, 2.700357282996937, 2.751301069168528, 2.806922252011147, 2.794393767976457, 2.7043293767673955, 2.6906503211161135, 2.7660352545938673, 2.8375311467298276, 2.7828215174614246, 2.7907889033578765, 2.7939764937018134, 2.7811489917670085, 2.7678140352486045, 2.7848706245422363, 2.7346336689724287, 2.7655862528047743, 2.833497565263396, 2.763057606994726, 2.697341266711047, 2.9728243449690996, 2.710653646736388, 2.7294871955160884, 2.7142674467366215, 2.670781468130221, 2.8411555704037856, 2.7825250048546275, 2.7656131284252092, 2.7969980513214305, 2.787126785630633, 3.0907399760689707, 2.7145911436172048, 2.8318417417775295, 2.660344294302023, 3.022962145744615, 2.647252942726111, 2.7097180132653302, 2.7892752118930697, 2.748480170016076, 2.743500603991709, 2.7059502776261346, 2.7475614995713444, 2.9338817676161506, 2.807369507042466, 2.8195162153547737, 2.7103135433925947, 2.7097548580473396, 2.61719752572904, 2.671143537873675, 2.7023776648150886, 2.6897768397240123, 2.7166068645039942, 2.8517817653668156, 2.640670805979686, 2.785406020796223, 2.8133675812915633, 2.727020207484057, 2.7015034021085995, 2.5447623851192986, 2.7122662135749867, 2.7130792323191453, 2.777392991788828, 2.8250894128896626, 2.91098423064894, 2.6593493944520405, 2.7406618431874903, 2.9580462396524516, 2.673022915603249, 2.7634778159439186, 2.803088065165623, 2.770714321713539, 2.7559103483607057, 2.5808921665143054, 2.689608198442277, 2.8259609736454716, 2.7821214483801726, 2.763839315836597, 2.664822526038832, 2.7398582355231995, 2.7864424696393835, 2.711390738274641, 2.6696762203410933, 2.56739388985239, 2.912347102620799, 2.6242039970531583, 2.741662036081788, 2.6793328967823347, 2.6890375591387414, 2.7907105980405382, 2.7735554048210194]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="AdamW")
data.update({'train_metric': [9.678, 22.389, 30.491, 36.2, 41.045, 45.123, 49.07, 52.546, 56.142, 59.746, 63.051, 66.037, 69.075, 71.566, 73.85, 76.134, 77.858, 79.801, 81.208, 82.34, 83.45, 84.372, 85.009, 85.787, 86.395, 87.135, 87.596, 87.983, 88.416, 88.88, 89.123, 89.323, 89.761, 90.123, 90.255, 90.543, 90.844, 90.975, 91.116, 91.49, 91.382, 91.521, 91.608, 91.867, 92.24, 92.243, 92.544, 92.323, 92.639, 92.501, 92.931, 92.875, 92.925, 93.224, 93.105, 93.33, 93.273, 93.44, 93.48, 93.502, 93.633, 93.765, 93.688, 93.956, 93.962, 94.048, 94.06, 94.105, 93.998, 94.251, 94.295, 94.406, 94.371, 94.44, 94.588, 94.595, 94.488, 94.774, 94.763, 94.634, 94.8, 94.656, 94.75, 94.799, 94.882, 95.073, 94.998, 95.169, 94.946, 95.045, 95.025, 95.195, 95.2, 95.232, 95.337, 95.284, 95.385, 95.323, 95.292, 95.342, 95.381, 95.46, 95.46, 95.431, 95.338, 95.577, 95.613, 95.737, 95.639, 95.693, 95.671, 95.713, 95.777, 95.569, 95.794, 95.825, 95.855, 95.873, 95.75, 95.732, 95.867, 96.047, 95.975, 96.023, 95.906, 95.824, 96.115, 95.983, 95.948, 95.934, 96.072, 96.219, 95.937, 96.168, 95.985, 96.072, 96.225, 96.32, 96.092, 96.311, 96.195, 96.238, 96.43, 96.138, 96.388, 96.33, 96.289, 96.291, 96.32, 96.356, 96.304, 96.316, 96.46, 96.437, 96.329, 96.5, 96.331, 96.478, 96.475, 96.429, 96.426, 96.427, 96.506, 96.475, 96.53, 96.595, 96.562, 96.473, 96.634, 96.613, 96.512, 96.507, 96.73, 96.613, 96.682, 96.642, 96.463, 96.772, 96.698, 96.673, 96.715, 96.611, 96.797, 96.72, 96.749, 96.739, 96.783, 96.753, 96.855, 96.688, 96.723, 96.719, 96.703, 96.758, 96.812, 96.751, 96.812, 96.803, 96.782, 96.92], 'train_loss': [4.326882811593308, 3.41402133885516, 2.9687363932091535, 2.6655769605935573, 2.4227681468864777, 2.2154858784269846, 2.024161513234588, 1.8418888376068778, 1.6747070065081615, 1.5106908986756074, 1.3592073464729202, 1.2235139486351916, 1.0940554909269853, 0.9853197186708603, 0.8875931880638833, 0.8001611204633176, 0.729407171389268, 0.6650418417048012, 0.6114725434791561, 0.5676916397998368, 0.5304291437660664, 0.49288038680626656, 0.47393776661813525, 0.4449635020201586, 0.4252973730775384, 0.40476634104093223, 0.38853616274116287, 0.37761704836777216, 0.3633217811608307, 0.3506638595377949, 0.3448702129877994, 0.33418252466393067, 0.3237327943464845, 0.3132646836349961, 0.3096007287778766, 0.2996591924119476, 0.2867653840558123, 0.28605452967630085, 0.2797408431592044, 0.26933986448604785, 0.271998459288537, 0.2671861749372409, 0.26509887524311426, 0.25860290453362295, 0.2457068985171964, 0.2458534689104126, 0.2358104334876027, 0.2430630773198639, 0.23162013363219458, 0.23796455979213757, 0.22443179101209493, 0.22548169398266804, 0.22635160809262433, 0.21571105226674145, 0.22067673821169084, 0.210264121082755, 0.21373883259476129, 0.20760493319283788, 0.20780598491868832, 0.20261555618021973, 0.20185979603243348, 0.20006012894160766, 0.20223684501920614, 0.19408926276176672, 0.19167438770453096, 0.19056623321686772, 0.19130621572464265, 0.18902889137100892, 0.19048779654022846, 0.18390310241322982, 0.18234085689834484, 0.17905141311737585, 0.17995572015249856, 0.17937263430668826, 0.17384076407489082, 0.17241230373070007, 0.17531309079322832, 0.16714003290063437, 0.1670931564132735, 0.17386883810360368, 0.1667420399820364, 0.17011328788697053, 0.16788462943338472, 0.16681649451938316, 0.16368692271948881, 0.16128336555656386, 0.16071663131569144, 0.1570325032827252, 0.16257320827210436, 0.1584893654670235, 0.15972731510440624, 0.15386617624320648, 0.15522790868943576, 0.15436412768065452, 0.14998118581830674, 0.1508767223101817, 0.1486721325848044, 0.15133520521483337, 0.15234192611735436, 0.1486719593176915, 0.14828198697085487, 0.14620380624127707, 0.1476476307622004, 0.1485031772950415, 0.15250260251415124, 0.14266073315901, 0.14208494934643084, 0.13876466856350597, 0.1414928890215632, 0.1403590291342266, 0.13948780219028903, 0.1396591451116769, 0.14109042304599573, 0.14180412831801448, 0.13859096963062253, 0.13580785361723408, 0.13462820525060068, 0.13599890559987401, 0.13532615473174556, 0.13854566638811383, 0.13562761654261613, 0.13005182248932295, 0.1360714628309922, 0.12871958513464718, 0.13597010207119364, 0.13797124525977827, 0.12841273541428191, 0.12980576168885186, 0.13193530501676837, 0.13178932541648852, 0.1277348888779961, 0.12254497602654077, 0.1326877613443469, 0.125345503231728, 0.13257167888118546, 0.1303083972447693, 0.12681769245493055, 0.12268265522390805, 0.12890356473498982, 0.12336081424230497, 0.12671162890090404, 0.12682030151147689, 0.11665658468409768, 0.12926635797627412, 0.11979436243058111, 0.11963255506347607, 0.12143889786200378, 0.1247057091439964, 0.11899554448672085, 0.12153448555620014, 0.12331041024347751, 0.12003602679048427, 0.1189423944913262, 0.11762556038915775, 0.12490904229323387, 0.11746659492428807, 0.1234938976387967, 0.1191691823157119, 0.11912423205354578, 0.11933765513450026, 0.11921779828431217, 0.11834398493521102, 0.11635154310841189, 0.11751231666781266, 0.11620950196709982, 0.11217231038275281, 0.11582257140522272, 0.11564026028156205, 0.1118914288460019, 0.11183909670123861, 0.11677598352773415, 0.1175772577934078, 0.10895394891806892, 0.11146137799730409, 0.11089538541580451, 0.1129449865495058, 0.1180709080133195, 0.10759035821937642, 0.1110995319702541, 0.11231629303720574, 0.11035202695229594, 0.11264969702417976, 0.10748595212514274, 0.10641207312260113, 0.11254577393834529, 0.11137799543478441, 0.10674609853467479, 0.11125861320815032, 0.10502925381740413, 0.11189519645487121, 0.10886975280222973, 0.10839117337075603, 0.10808478347069012, 0.1090059799813041, 0.1076509651357948, 0.10927124593606677, 0.1051100972083219, 0.1071614715663516, 0.10572827438732564, 0.10703953815558871], 'val_metric': [14.34, 26.33, 31.48, 34.15, 36.48, 38.61, 39.01, 37.38, 39.9, 39.25, 38.96, 38.37, 39.06, 38.61, 37.97, 38.74, 37.93, 37.78, 38.18, 37.35, 37.98, 37.27, 37.25, 36.92, 37.46, 36.75, 37.59, 37.92, 37.02, 37.51, 37.48, 36.81, 37.05, 37.27, 37.64, 37.37, 37.77, 36.98, 37.74, 37.86, 37.77, 37.54, 37.4, 37.89, 38.32, 37.69, 37.4, 36.9, 37.97, 37.84, 37.42, 37.39, 37.37, 38.46, 38.11, 37.88, 37.35, 37.13, 37.08, 37.5, 37.61, 38.72, 37.65, 38.29, 38.03, 38.56, 37.78, 38.04, 38.25, 37.7, 38.04, 38.2, 38.77, 37.47, 37.85, 37.84, 37.84, 38.54, 37.99, 38.55, 38.47, 38.47, 38.38, 38.4, 39.1, 38.14, 38.51, 37.86, 39.94, 39.13, 37.79, 37.49, 37.66, 37.18, 38.51, 39.42, 37.55, 38.47, 38.32, 38.0, 36.9, 39.03, 38.29, 39.01, 38.72, 38.65, 38.78, 38.21, 38.7, 38.67, 39.82, 38.72, 38.12, 39.18, 39.17, 39.05, 38.75, 38.58, 38.76, 38.86, 39.06, 38.37, 38.8, 37.96, 38.79, 37.86, 38.39, 38.97, 38.52, 38.47, 38.97, 39.1, 39.08, 38.85, 39.08, 38.73, 38.59, 38.51, 39.33, 38.97, 38.48, 38.37, 39.38, 38.63, 39.05, 38.89, 39.12, 38.9, 38.5, 39.27, 39.39, 38.54, 39.08, 38.61, 39.57, 39.32, 39.15, 38.82, 39.33, 39.86, 38.63, 39.24, 39.69, 38.99, 38.87, 39.27, 38.96, 38.4, 38.45, 39.8, 38.49, 38.51, 39.24, 38.6, 38.42, 39.24, 38.8, 38.69, 38.84, 38.68, 38.41, 38.63, 38.72, 38.96, 39.55, 38.96, 39.39, 38.96, 38.56, 39.16, 39.05, 39.76, 39.87, 39.3, 39.6, 39.08, 39.44, 39.37, 39.17, 39.56], 'val_loss': [4.0052605662376255, 3.236190203648464, 2.93998375060452, 2.8348603134701964, 2.6777512784216815, 2.637142188989433, 2.6790426339313482, 2.7651753224385014, 2.7077081161699477, 2.761010536722317, 2.854759848041899, 3.020784270991186, 3.1078313186669804, 3.169016124336583, 3.3675191174646852, 3.423951230231364, 3.548468227599077, 3.6454222794551, 3.762306866372467, 3.950756337612298, 4.155579735139373, 4.140346296273979, 4.2559913207011615, 4.2358381991173815, 4.314347032528774, 4.538625739941931, 4.495664621614347, 4.584887421814499, 4.623433217121537, 4.619552640398597, 4.799819613717924, 4.7319332862355905, 4.759549321642347, 4.930600290845154, 4.909240838069065, 5.0662127672487, 4.919452529044667, 5.029402391925739, 4.978135623749654, 5.0333900406102465, 5.0789418903885375, 5.132633347420176, 5.203602679975473, 5.01928198033837, 5.21767752869114, 5.219928466590347, 5.270218691248803, 5.336961427311988, 5.33831817557098, 5.466983297068602, 5.426459218286405, 5.44191320003218, 5.388733976206202, 5.506285501893159, 5.526237961592948, 5.5830894746598165, 5.487467049793073, 5.590216238027925, 5.731505927766205, 5.657868423279683, 5.770012237463787, 5.739123311012414, 5.606394668293607, 5.815934463671058, 5.727716948576035, 5.785697640127437, 5.610354157010461, 5.838355257252979, 5.813189927939397, 5.647189014276881, 5.851792737177223, 5.785350275647109, 5.970940426656395, 5.846436767061805, 5.959059236915248, 5.926688945217497, 6.038201094433001, 5.970390782234775, 5.921266413797998, 5.925537490541009, 5.828080719443643, 5.969986268669177, 6.006517993416756, 5.894422421789473, 5.900860487275822, 6.055238014573504, 6.024432729004295, 6.1682028732482035, 6.188776215929894, 6.182639768169184, 6.211510573223138, 6.197083251491474, 6.293417197124214, 6.2436929705795965, 6.132702109919991, 6.339204313648734, 6.168463962093281, 6.238988057822938, 6.214862105193411, 6.378292984263912, 6.310607563917804, 6.396243503898572, 6.0307300060418, 6.293446486163291, 6.235658442138866, 6.478146224264886, 6.40878984351067, 6.204767387384062, 6.386066953088068, 6.523018041234107, 6.5747811490563075, 6.576087556826841, 6.345384939460997, 6.414909668029494, 6.390523899133038, 6.358188443123155, 6.543659807010821, 6.458260528600899, 6.496733179517612, 6.438584652675945, 6.58573188979155, 6.654868505562947, 6.455034713836232, 6.8553157475343935, 6.626525436996654, 6.6472943163221805, 6.736762924558797, 6.53688818937654, 6.589419812913153, 6.6680794650582, 6.735148637158097, 6.6473769488608, 6.6055461342927, 6.772922963853095, 6.582504655145536, 6.561526956072279, 6.57368559594367, 6.662842086925628, 6.659582088707359, 6.80083942868907, 6.716843516963302, 6.668479164694525, 6.888175387291392, 6.818995917678639, 6.741883451012289, 6.649873105583677, 6.710977529264559, 6.7008847081737155, 6.817906700122129, 7.062949440281862, 6.866091749470705, 6.6989766913614455, 6.7768907114199015, 6.910494921313729, 6.6951954957026585, 6.897157430648804, 7.01425885091162, 6.778877317525779, 6.8657350889436755, 6.809197889012136, 6.839487346114626, 6.808435244165408, 7.072535912702038, 6.8973533530144175, 6.876772974706759, 7.028767338224277, 6.9260915373540985, 6.84298643185075, 7.063883169441466, 6.9813478144870444, 7.040113860634482, 6.956397402818036, 6.932601535396212, 6.992118981233828, 6.980916785586412, 7.064372985985629, 7.141700085560987, 7.188337196969682, 6.867620185681969, 7.155064347443307, 7.033143386719333, 7.13415583531568, 7.318857660718784, 7.058895641071781, 7.163161004424855, 7.032583210878311, 7.067459224895307, 7.033523647648514, 7.133476453981582, 7.33852102650199, 7.0873455409031765, 7.250932433043316, 7.177452646243345, 6.991590982789447, 7.121017337604693, 6.982139406690172, 7.4425463737196225, 7.159418033186797, 7.2200094438662195, 7.603033889630797]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="SGD")
data.update({'train_metric': [0.842, 1.499, 1.997, 2.301, 2.533, 2.78, 2.993, 3.187, 3.553, 3.768, 4.079, 4.338, 4.658, 5.062, 5.411, 5.911, 6.306, 6.635, 6.91, 7.367, 7.598, 7.936, 8.28, 8.589, 8.937, 9.278, 9.759, 10.109, 10.358, 10.708, 11.215, 11.432, 11.794, 11.976, 12.385, 12.715, 12.927, 13.251, 13.569, 13.925, 14.271, 14.64, 14.83, 15.114, 15.587, 15.783, 16.048, 16.417, 16.791, 17.102, 17.353, 17.661, 17.986, 18.32, 18.741, 18.869, 19.202, 19.577, 19.721, 20.009, 20.289, 20.619, 20.966, 21.085, 21.374, 21.666, 21.885, 22.194, 22.276, 22.753, 22.994, 23.256, 23.396, 23.58, 23.914, 24.201, 24.562, 24.703, 24.861, 25.318, 25.406, 25.684, 25.716, 26.008, 26.374, 26.342, 26.735, 27.062, 26.996, 27.362, 27.623, 27.817, 28.083, 28.448, 28.568, 28.683, 28.777, 29.094, 29.072, 29.421, 29.62, 29.849, 30.157, 30.255, 30.415, 30.552, 30.809, 31.02, 31.183, 31.375, 31.698, 31.662, 32.099, 32.31, 32.413, 32.374, 32.633, 33.147, 33.235, 33.541, 33.496, 33.962, 34.021, 34.084, 34.228, 34.392, 34.761, 34.874, 34.975, 35.19, 35.453, 35.648, 35.869, 35.962, 36.136, 36.386, 36.594, 36.798, 36.936, 37.046, 37.241, 37.489, 37.271, 37.83, 37.835, 38.119, 38.234, 38.506, 38.64, 38.935, 38.952, 39.268, 39.446, 39.294, 39.507, 39.812, 40.016, 39.914, 40.246, 40.586, 40.735, 40.86, 41.056, 40.904, 41.403, 41.491, 41.476, 41.696, 41.945, 41.893, 42.172, 42.689, 42.692, 42.6, 42.862, 42.789, 43.087, 43.214, 43.581, 43.442, 43.704, 43.812, 44.062, 44.226, 44.61, 44.558, 44.89, 44.73, 44.974, 45.289, 45.194, 45.453, 45.737, 45.782, 46.035, 46.087, 46.063, 46.19, 46.665, 46.485], 'train_loss': [5.287255688348186, 5.253395889633676, 5.211071103792197, 5.156126049185745, 5.093454079374776, 5.030943351606489, 4.974717427276299, 4.9233050578233915, 4.8796966611309385, 4.838277940329114, 4.800879429947163, 4.762393140823354, 4.726437115013333, 4.691043190519854, 4.652074731166593, 4.612574391538946, 4.575438088205329, 4.53957203360452, 4.50285663470502, 4.465560693429665, 4.428382636794507, 4.395507166497004, 4.359798590342204, 4.32856090649991, 4.2963719668482945, 4.262550029553287, 4.237221918118283, 4.205803521077601, 4.176421697370074, 4.149354350925331, 4.12518754984733, 4.0993653068310625, 4.073615904580456, 4.048940519453697, 4.026363644322293, 4.000988053421294, 3.98162174148584, 3.954638827403844, 3.9338340917917987, 3.9108859922210146, 3.8876730753531916, 3.8637497214575416, 3.842389486725332, 3.823315623778223, 3.7998349268468465, 3.7814107171557154, 3.7596027303489445, 3.737114941242484, 3.71525647483113, 3.695758946187513, 3.676416992912366, 3.6566031291861605, 3.6365591610233063, 3.6180248377953137, 3.59691335585967, 3.579986032734906, 3.5658273442159154, 3.5422800252320137, 3.527822574589852, 3.5058691312468975, 3.487596272964624, 3.474037072632607, 3.4537839773596666, 3.440614042873956, 3.424549174629109, 3.4058447514141665, 3.393155485563223, 3.3747255224031436, 3.362695960455496, 3.341911248495696, 3.327097461838335, 3.312247699754633, 3.2980417823913535, 3.2866486830354424, 3.2676358195504394, 3.253963761198467, 3.2378016570708117, 3.22349480292161, 3.2099583356409447, 3.1956902656201285, 3.184185208224823, 3.1679937853236573, 3.1583382596743848, 3.14341624776141, 3.1297720392316255, 3.1187360118919663, 3.1006722361783683, 3.090907503379436, 3.0784384757375687, 3.066868026059786, 3.04909135863633, 3.040730127827601, 3.0295412723482684, 3.015343729990534, 3.00007386033686, 2.991225538693097, 2.976674262179218, 2.9716280080230284, 2.953797737909904, 2.9470239086785686, 2.9320198241823845, 2.9164378864217553, 2.9086398912101545, 2.9005167458351804, 2.885657385229988, 2.875138156168444, 2.862343131008624, 2.8509287420786578, 2.8415305501592516, 2.829052553524669, 2.8198223470955113, 2.8107146902566376, 2.799353245962757, 2.783166735735141, 2.7748394473119187, 2.767181148760912, 2.7524643231109405, 2.7401365539772877, 2.734614098598312, 2.7192464288197797, 2.7075208642890036, 2.696476207661156, 2.686734682615186, 2.677607708532537, 2.668901190876732, 2.6594027032931495, 2.649576406713792, 2.6373091430444404, 2.6305983133676034, 2.619388469731434, 2.610540786082365, 2.6024923451192397, 2.59023812483765, 2.577950340696275, 2.573492913465811, 2.5614431272007607, 2.552621552902998, 2.5403093675429136, 2.5336606770239043, 2.522040790193598, 2.512862108871865, 2.504528951583882, 2.4983993260736885, 2.4844488162530665, 2.4780459195013163, 2.4687044671614506, 2.459000283605535, 2.4473992364191504, 2.4395789830103487, 2.433414090999181, 2.4244830490150133, 2.4106703185913125, 2.403642647478417, 2.401734446304697, 2.3919543087368047, 2.378511549415149, 2.371259182519968, 2.368436203503258, 2.3552049949088314, 2.344749579350306, 2.334931368714941, 2.3299022440107984, 2.323646453429092, 2.315071882838556, 2.305299015664475, 2.2921784699763994, 2.2903580905837426, 2.277495620423071, 2.274109397343314, 2.263519703617328, 2.2537452191476706, 2.2418139635258125, 2.2370667841979044, 2.23311618193555, 2.223931026016377, 2.2226796862565013, 2.2055760653295047, 2.199921142116847, 2.1899896163019092, 2.1848650710062576, 2.1771269328916065, 2.1709964736028122, 2.1585501021326006, 2.154078042743607, 2.1450324606910702, 2.139115989475165, 2.1243003493917345, 2.1257826812131544, 2.116198617910164, 2.1038946525759217, 2.0984910140034487, 2.092073858203754, 2.0818452581868137, 2.0770155471330716, 2.072999222798753, 2.0647911738678193, 2.057650127169877, 2.0481288718323025, 2.0397080542258705, 2.03721505430213], 'val_metric': [1.15, 1.88, 2.5, 2.7, 2.82, 3.19, 3.3, 3.63, 4.14, 4.24, 4.71, 5.09, 5.39, 5.99, 6.4, 6.64, 7.32, 7.63, 7.69, 8.22, 8.41, 8.61, 8.83, 9.46, 9.75, 9.59, 9.97, 10.75, 10.81, 11.41, 11.13, 11.31, 12.16, 12.19, 11.95, 12.91, 12.86, 12.39, 13.2, 13.54, 13.74, 14.21, 14.48, 14.81, 14.47, 14.88, 15.41, 15.4, 15.91, 16.24, 16.28, 16.38, 16.31, 16.6, 17.24, 17.13, 17.53, 18.13, 18.28, 18.32, 17.96, 18.19, 18.62, 18.99, 18.66, 19.31, 19.02, 19.0, 19.55, 19.85, 19.8, 20.15, 20.4, 19.61, 20.33, 20.63, 20.5, 20.65, 20.51, 20.47, 20.77, 20.3, 21.0, 20.86, 20.94, 20.89, 21.39, 21.3, 21.34, 20.63, 21.51, 21.14, 21.23, 21.3, 21.03, 21.2, 21.58, 21.47, 21.51, 21.7, 21.04, 21.21, 21.4, 20.99, 21.54, 21.31, 21.21, 21.3, 21.68, 20.97, 21.22, 21.8, 21.57, 21.53, 21.41, 21.12, 21.78, 21.99, 21.82, 21.24, 21.74, 21.92, 21.59, 21.59, 21.21, 21.51, 21.73, 22.02, 21.5, 22.06, 21.89, 21.5, 21.76, 22.01, 21.34, 21.34, 21.94, 21.91, 22.15, 21.71, 21.43, 22.02, 21.78, 22.06, 21.31, 21.42, 21.45, 21.2, 21.67, 21.44, 21.34, 21.39, 21.0, 20.81, 21.6, 21.65, 21.5, 21.11, 21.32, 20.93, 20.82, 20.97, 21.55, 21.97, 20.95, 20.92, 21.41, 21.44, 21.25, 21.14, 21.49, 21.04, 20.87, 21.22, 21.33, 20.41, 21.13, 20.97, 21.02, 20.58, 21.02, 21.28, 20.44, 21.21, 21.1, 21.3, 20.81, 20.12, 20.44, 20.86, 20.89, 20.68, 20.42, 20.61, 20.13, 20.45, 20.42, 20.9, 20.68, 19.83], 'val_loss': [5.268571039673629, 5.2308862224505965, 5.172367847649155, 5.111907990874758, 5.044646806777663, 4.982146320828966, 4.924470433763638, 4.872306398525359, 4.821578756259505, 4.792610100120496, 4.747668987626483, 4.71311996393143, 4.681930684739617, 4.634256024269542, 4.601641823531716, 4.569753229238425, 4.523928075839001, 4.494874510795447, 4.447584342045389, 4.413070774382087, 4.3785817942042256, 4.350767571455354, 4.311319437755901, 4.275715352623326, 4.2471385791802865, 4.23122946471925, 4.207775412091784, 4.173221594209124, 4.148265243335894, 4.117375193128161, 4.111393573177848, 4.112354547354825, 4.04437443557059, 4.037607686534809, 4.035796804792562, 4.0041398394639325, 3.9731616214582117, 4.006090147480084, 3.9499352069417384, 3.9242465769409374, 3.905800533902114, 3.8872750200283757, 3.876333213915491, 3.846792081359086, 3.8499357563674828, 3.856430731001933, 3.810707970029989, 3.8053102766632274, 3.792966233696907, 3.7600924300540024, 3.765155348808143, 3.741317633610622, 3.735991907727187, 3.7343953885849874, 3.705721823273191, 3.6962141519898823, 3.6683410808538937, 3.6537691210485566, 3.6495594234223576, 3.6635180278948156, 3.6537395799235934, 3.649305820465088, 3.612148360841593, 3.5996209162815362, 3.614016490377438, 3.5847870756866067, 3.6003935200393578, 3.592124651951395, 3.5649602443549284, 3.5679501036929477, 3.5713701354470224, 3.555748040509072, 3.562207683636125, 3.5816273043869407, 3.5604281653264525, 3.531212363273475, 3.539514539348092, 3.5351898427222186, 3.561048193342367, 3.545039804118454, 3.549237491218907, 3.557441879989235, 3.5271433242567025, 3.530307333940154, 3.5312024499200714, 3.528739206350533, 3.517264770094756, 3.5245912257273484, 3.5201790628919176, 3.551610910209121, 3.530651025711351, 3.518206828718732, 3.550856164306592, 3.5435579066063947, 3.529819979789151, 3.5414474633089297, 3.5351702999916808, 3.5445219726319523, 3.5359523433029274, 3.5497267200688647, 3.5739102029496697, 3.5756671049033, 3.5548360438863185, 3.5599329813270812, 3.5449601874989307, 3.560810666175405, 3.559171901387014, 3.5948898898568125, 3.569057655182614, 3.5711451905548195, 3.574474793330879, 3.5761583686634233, 3.5984284892962997, 3.6105608309909796, 3.59912938524963, 3.6091786205388936, 3.608621635254781, 3.5686150622216, 3.601694290805015, 3.610322501249374, 3.6311838330736586, 3.6151037907144827, 3.6172111231809967, 3.639782433297224, 3.6444950384698855, 3.6140383595873597, 3.650076112929423, 3.6205200784525293, 3.6670077181166145, 3.611794698010584, 3.656540312584798, 3.6614441552739234, 3.6858391807337476, 3.663778393131912, 3.6813748923076948, 3.6859638311301066, 3.6793982101853486, 3.683442563008351, 3.6929375639387, 3.7085436605344153, 3.6973610486194586, 3.7150325820704175, 3.7200541830366585, 3.7127622145756036, 3.7433843620263847, 3.76765361029631, 3.7400666832164595, 3.7616689053310712, 3.7445691634135643, 3.8079630622438563, 3.793000479412686, 3.7836768217147534, 3.8073368786246915, 3.820561273842101, 3.838126788473433, 3.807956530789661, 3.7736962904596023, 3.8348811857259957, 3.8530337385311246, 3.8587082616842476, 3.8434225404338473, 3.8847416343203016, 3.8932275734129984, 3.8693085696287217, 3.8725639763911057, 3.8611383134392416, 3.850061910167621, 3.883999322630038, 3.928656948599846, 3.9289010421485657, 3.8859801330384176, 3.9436447529276464, 3.938892171641064, 3.909521292728983, 3.92016212651684, 3.938581404412628, 3.942879830196405, 3.904239976482027, 3.9248804134927737, 3.9718637451244767, 3.999251815164165, 3.9656899036115902, 4.046651364132098, 4.004975235386259, 3.991751423307285, 4.025921321978235, 4.022649772607597, 4.10711534008099, 4.076918851038453, 4.04781090378002, 4.0179215434250555, 4.084921238528695, 4.086189995905396, 4.074999528326047, 4.186119662728279, 4.09879905554899, 4.153974745683609, 4.095916087460366, 4.079853637962584, 4.165662284110002]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="Adam")
data.update({'train_metric': [6.109, 12.299, 15.561, 17.84, 19.504, 21.17, 22.597, 23.791, 24.664, 25.721, 26.514, 27.185, 28.046, 28.46, 28.986, 29.427, 29.955, 30.254, 30.59, 30.829, 31.103, 31.523, 31.54, 32.043, 32.512, 32.802, 32.75, 33.159, 33.498, 33.671, 34.002, 34.059, 34.388, 34.622, 34.604, 35.066, 35.157, 35.039, 35.108, 35.406, 35.594, 35.851, 35.842, 36.005, 36.329, 36.264, 36.38, 36.399, 36.711, 36.429, 36.614, 36.759, 36.912, 37.142, 37.134, 37.052, 37.279, 37.522, 37.489, 37.517, 37.515, 37.678, 37.687, 37.888, 37.917, 37.729, 37.867, 37.918, 37.987, 38.0, 38.181, 38.075, 38.254, 37.981, 38.365, 38.448, 38.28, 38.462, 38.463, 38.603, 38.675, 38.559, 38.611, 38.729, 38.656, 38.88, 38.713, 38.728, 38.909, 39.08, 38.836, 39.176, 38.885, 39.005, 39.052, 39.226, 39.203, 39.266, 39.226, 39.26, 39.413, 39.319, 39.424, 39.347, 39.552, 39.522, 39.425, 39.363, 39.328, 39.691, 39.684, 39.618, 39.618, 39.712, 39.765, 39.773, 39.879, 40.027, 39.897, 39.902, 39.969, 39.835, 39.849, 39.911, 40.27, 40.091, 40.211, 40.193, 40.087, 40.006, 40.082, 40.14, 40.541, 40.262, 40.412, 40.321, 40.313, 40.192, 40.239, 40.228, 40.179, 40.485, 40.636, 40.41, 40.547, 40.312, 40.394, 40.507, 40.453, 40.66, 40.475, 40.582, 40.489, 40.678, 40.586, 40.656, 40.427, 40.626, 40.841, 40.655, 40.641, 40.718, 40.686, 40.5, 40.745, 40.72, 40.589, 40.919, 40.874, 40.736, 40.751, 40.535, 40.911, 40.763, 40.861, 40.934, 40.928, 41.0, 41.037, 40.927, 41.063, 40.83, 41.074, 40.872, 40.917, 40.951, 41.067, 40.961, 41.053, 40.979, 40.906, 40.853, 41.0, 41.283, 40.992, 41.017, 41.108, 41.198, 41.141, 41.183], 'train_loss': [4.547092943411185, 3.9498474144889846, 3.7224150234815485, 3.576308261867677, 3.474664823412514, 3.3807669424202222, 3.302754747356578, 3.236534497864492, 3.1874364348306004, 3.1334664408243853, 3.0886961211779904, 3.052324054794897, 3.018343962237992, 2.9852823741300245, 2.9590565021878548, 2.9367353797187734, 2.913041517205217, 2.897643221088197, 2.8776680349312147, 2.8582580370240995, 2.8508574273139335, 2.8295949737917363, 2.8171443214648364, 2.801171048360228, 2.7837691647230014, 2.7742412820963698, 2.759493836972169, 2.746394132088181, 2.7361203300136827, 2.7273386699300657, 2.7126250085507304, 2.704525286542706, 2.695429418228867, 2.685923452035632, 2.680144394427938, 2.6680312815646063, 2.6615547429119557, 2.6537470255459414, 2.6513217048468074, 2.643137863409954, 2.6378116670969733, 2.6285271840605002, 2.6215137616991617, 2.6169190442645998, 2.610232770252289, 2.608005292813746, 2.599323036269507, 2.6013439037978308, 2.589014101318267, 2.5925590026401513, 2.5868638465592135, 2.582739703943542, 2.574009083206655, 2.570227962842906, 2.5641183114860278, 2.566396806763291, 2.561232292797042, 2.5573085452682, 2.5587504388274707, 2.547184859569913, 2.5496585680289825, 2.544073498561759, 2.5417267685888367, 2.5355586629606406, 2.5357057024711076, 2.53591792284489, 2.528903470005809, 2.527186225792268, 2.520945113054545, 2.5217937457584374, 2.51797697731721, 2.5168509404017083, 2.5129309870536254, 2.5151363938417637, 2.508639220312781, 2.509217322292194, 2.503929737128284, 2.501892188917881, 2.5013500824999673, 2.4964967101945836, 2.494613813194646, 2.495404419880682, 2.488879924543531, 2.488761565277993, 2.492288806388108, 2.488963346487425, 2.4823309312778945, 2.484835055724063, 2.4866266159086905, 2.4840461911105685, 2.4815320417961857, 2.476686746968875, 2.4795600177382933, 2.4774530148246847, 2.473697826256755, 2.4675034980932566, 2.471104330842646, 2.4654967364636433, 2.468968557762322, 2.466026200213954, 2.465167007534762, 2.463547364230043, 2.46237657585742, 2.4623904974889235, 2.457991273602994, 2.457477140868999, 2.460121207258599, 2.4558331285313164, 2.458287922449777, 2.45249583007278, 2.4504847705783708, 2.449540450644661, 2.450101173541825, 2.447392084777012, 2.4432499327113515, 2.441390383190172, 2.4388057557879086, 2.438529137915247, 2.438623220281424, 2.4365341082949405, 2.4391687770570156, 2.4405362066060245, 2.436094679515177, 2.4330117837481215, 2.432802827252994, 2.4301298111581833, 2.43088782108219, 2.426155938220497, 2.4278044927905786, 2.426171798928762, 2.4245705756329605, 2.4282648213307825, 2.421816655633088, 2.425860264784849, 2.423594827462829, 2.4232313211957233, 2.4235400842384736, 2.420984760050734, 2.4215253103412424, 2.420611417911332, 2.419941521041758, 2.4173686836899204, 2.418039203605359, 2.415730630138785, 2.4138171196136424, 2.414324748996581, 2.415338822915168, 2.412883068992019, 2.4132787090650525, 2.412126997077, 2.4106938383629593, 2.4088884149692946, 2.4093512073206886, 2.4058821787837217, 2.4075762969289767, 2.4046688983628024, 2.4051110574395245, 2.411180007785692, 2.4016478446074503, 2.4036115384300167, 2.403051145474879, 2.404118889657946, 2.4036308117425373, 2.4062562032304204, 2.401133702187224, 2.400805342754186, 2.4025702342267077, 2.3988299680229037, 2.3993922390391713, 2.400599606587806, 2.399914305025541, 2.4008587662104377, 2.398709371542976, 2.398207972771223, 2.397349725834315, 2.3959025138475463, 2.398649549453745, 2.3936249184745746, 2.394099816174669, 2.392156280543815, 2.3913407121342263, 2.390792108009202, 2.3897670011679026, 2.3908742436673194, 2.3925691792694943, 2.3920682537502307, 2.3907924933991826, 2.386993350619623, 2.392998880403437, 2.391378058886879, 2.386697520983959, 2.391841988264561, 2.3871769980597177, 2.3829322275410685, 2.3887212380185314, 2.3905022970316736, 2.3846889436816987, 2.3862171105978507, 2.3830361248816883, 2.3811008189629073], 'val_metric': [5.43, 12.49, 14.91, 17.5, 18.34, 21.61, 22.01, 23.62, 23.51, 24.0, 25.5, 23.91, 26.16, 24.69, 22.92, 24.19, 27.77, 24.22, 24.46, 24.94, 28.1, 25.51, 20.96, 29.37, 28.84, 26.16, 27.45, 29.18, 28.72, 31.26, 28.61, 29.86, 30.78, 31.05, 32.05, 31.88, 30.05, 29.61, 29.94, 29.82, 31.81, 29.73, 31.35, 30.4, 27.68, 30.22, 29.84, 32.08, 35.07, 33.44, 32.7, 32.92, 29.06, 31.37, 34.24, 31.98, 28.66, 30.37, 30.41, 30.91, 33.2, 32.42, 33.7, 33.18, 32.46, 35.57, 33.77, 31.17, 31.79, 30.11, 33.04, 32.94, 32.55, 33.81, 35.12, 36.22, 31.87, 30.0, 34.98, 30.37, 29.51, 32.13, 33.77, 32.38, 35.82, 35.3, 33.53, 32.53, 34.78, 34.02, 31.96, 34.27, 32.03, 28.71, 31.77, 32.67, 34.03, 33.39, 34.6, 32.63, 31.92, 32.45, 32.13, 36.73, 34.48, 35.0, 31.25, 34.35, 35.03, 34.0, 34.35, 36.45, 35.24, 35.45, 34.14, 37.01, 34.57, 33.32, 35.18, 30.97, 33.9, 36.32, 31.81, 35.53, 33.37, 31.32, 35.21, 35.57, 33.96, 34.2, 34.94, 34.73, 36.23, 34.74, 31.2, 34.82, 35.97, 34.96, 34.68, 34.38, 32.94, 36.35, 33.57, 36.12, 33.91, 35.28, 36.17, 34.53, 36.83, 34.58, 35.19, 37.21, 36.85, 33.84, 35.89, 34.0, 35.31, 36.42, 33.93, 35.83, 32.47, 34.71, 36.47, 36.52, 37.17, 33.15, 36.62, 33.87, 34.89, 36.62, 37.05, 36.91, 34.29, 33.76, 36.77, 35.39, 37.59, 33.32, 35.18, 35.95, 36.92, 31.7, 36.32, 33.71, 33.21, 35.37, 34.37, 36.64, 30.52, 34.07, 34.37, 34.36, 33.03, 35.3, 35.95, 33.57, 35.07, 33.75, 37.65, 36.87], 'val_loss': [4.477708988128954, 3.9151732951972136, 3.723374676552548, 3.563293067512998, 3.5521003395129163, 3.3749738658309743, 3.3362405102723725, 3.2625462295143466, 3.248166716022856, 3.2006804624180885, 3.1532366647841825, 3.217179077446081, 3.1294721737029447, 3.1875975511635946, 3.30609763351975, 3.24658259312818, 3.068352902011507, 3.260453233293667, 3.2587974170211016, 3.1726875669637304, 3.011074886959829, 3.1931432705775946, 3.4829681094285028, 2.9607156367058964, 2.9439489689602216, 3.094856436085549, 3.0307993531986406, 2.961402616683085, 3.0211529238208845, 2.8437301603851806, 3.003815638031929, 2.982969946162716, 2.8998355998355114, 2.905694625939533, 2.802259048838524, 2.8666973379766865, 3.0275651649305018, 2.9930722554018545, 2.9392677727778245, 2.9460853247126195, 2.8302875164967434, 2.953000890221565, 2.8894365467842977, 2.9585188103329605, 3.0749659823004607, 2.9552271681226743, 3.0019694794515135, 2.84523664841986, 2.673831976902713, 2.766491360345464, 2.823960126585262, 2.7884809997431033, 3.008841144810816, 2.879800951404936, 2.733840752559103, 2.844209226073733, 3.027182386938933, 2.9476346529213484, 2.9705570685635707, 2.945260214957462, 2.775702235805001, 2.798930738382279, 2.745223193791262, 2.7862368055209994, 2.846846951041252, 2.6754312727861342, 2.7310752086578662, 2.92064455360364, 2.8487851255258936, 2.947772446711352, 2.8153804533041207, 2.799302607584911, 2.8154951607345775, 2.7235265917079463, 2.6855011426719133, 2.654568827076323, 2.850482060651111, 2.983762220971903, 2.728086593804086, 2.9766562653195328, 3.095561043472047, 2.852976366213173, 2.768249961980589, 2.85488261538706, 2.6806586745438303, 2.6611808697888804, 2.8349706207870677, 2.8216491451688634, 2.673043652704567, 2.74412336395045, 2.849161106310073, 2.73229910309907, 2.8499323258734055, 3.078824064534181, 2.866259629559365, 2.8606476586335785, 2.7652476532444075, 2.8103748081596036, 2.73165661817903, 2.8211405049463747, 2.8678047816464853, 2.8522559989030194, 2.8481319284742805, 2.630008847470496, 2.7273573708382384, 2.6978874889908324, 2.921932429265065, 2.742287199208691, 2.7408663695025597, 2.750709183656486, 2.743079273564041, 2.638754707233162, 2.7232504103593764, 2.665398220347751, 2.7574026728891265, 2.6196125381311792, 2.7325610674110945, 2.747099234040376, 2.701862100203326, 2.9561590753543148, 2.7810848222416675, 2.6579296490189375, 2.840360085675671, 2.67759077943814, 2.8114642565417443, 2.8858090806159242, 2.704695331822535, 2.70588609101666, 2.744523475883873, 2.7692349279762074, 2.725527431554855, 2.697759180311944, 2.6809342879398614, 2.715471185696353, 2.9289685988881784, 2.714920646825414, 2.6616287068196924, 2.7130485408625025, 2.7086678424458595, 2.7487233383640364, 2.819127606738145, 2.6410248644033056, 2.803731985152907, 2.70955395015182, 2.7447687372280534, 2.681713142212789, 2.6261107587510613, 2.730413363238049, 2.6121654700321755, 2.7055837420900914, 2.7116493328361755, 2.6117065677976914, 2.6039356418475985, 2.8015341432231247, 2.6604452831729963, 2.770288811367788, 2.7029744679001486, 2.6460782308487376, 2.789023564879302, 2.6540430670331236, 2.844131372536823, 2.7788086620865355, 2.6633193166392624, 2.6468425867663825, 2.6131042082598257, 2.791060811774746, 2.6443506585564585, 2.7627133548639384, 2.7084763763816495, 2.6292528339252352, 2.615844049651152, 2.6197014621868253, 2.7061670585802404, 2.7935963823537158, 2.6082835736548065, 2.722161151421298, 2.5861426720953293, 2.772895277305773, 2.697952611431195, 2.651327455879017, 2.6003101198536576, 2.945548117160797, 2.6675887981038184, 2.7912836431697676, 2.783457614054346, 2.68933164010382, 2.750550004327373, 2.6275986600074037, 2.9312336232252183, 2.761463848648557, 2.7336234909713646, 2.759115546372286, 2.819378708578219, 2.6925745686148383, 2.662608082886714, 2.7669170365971363, 2.710471033290693, 2.775780551752467, 2.588126868958686, 2.622096296328648]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="AdamW")
data.update({'train_metric': [7.763, 18.372, 25.933, 31.71, 36.682, 41.018, 44.781, 48.179, 51.043, 53.981, 56.807, 59.589, 62.018, 64.31, 66.704, 68.935, 71.157, 72.877, 74.836, 76.467, 77.645, 78.914, 80.511, 81.154, 82.221, 83.203, 84.024, 84.816, 85.185, 85.654, 86.38, 86.675, 87.417, 87.503, 88.153, 88.205, 88.498, 89.083, 89.429, 89.31, 89.843, 90.084, 90.107, 90.44, 90.643, 90.748, 90.91, 91.257, 91.263, 91.487, 91.574, 91.581, 91.982, 91.972, 91.875, 92.365, 92.155, 92.354, 92.741, 92.494, 92.71, 92.872, 92.886, 93.092, 93.078, 92.937, 93.252, 93.293, 93.549, 93.615, 93.415, 93.634, 93.483, 93.707, 93.709, 93.769, 93.763, 93.964, 94.103, 94.026, 94.283, 94.088, 94.241, 94.275, 94.347, 94.236, 94.38, 94.578, 94.409, 94.48, 94.537, 94.58, 94.77, 94.759, 94.648, 94.795, 94.763, 94.885, 94.823, 94.858, 94.946, 94.986, 94.921, 94.889, 95.108, 95.118, 95.059, 95.163, 95.175, 95.321, 95.306, 95.176, 95.259, 95.227, 95.411, 95.341, 95.28, 95.434, 95.368, 95.625, 95.456, 95.605, 95.4, 95.532, 95.57, 95.716, 95.605, 95.654, 95.669, 95.653, 95.854, 95.688, 95.793, 95.769, 95.788, 95.706, 95.729, 95.881, 95.88, 95.821, 95.949, 95.895, 95.837, 96.045, 95.978, 95.917, 95.922, 96.1, 96.067, 95.997, 96.233, 96.119, 95.985, 96.147, 96.045, 96.283, 96.05, 96.145, 96.146, 96.266, 96.108, 96.31, 96.198, 96.182, 96.322, 96.295, 96.263, 96.279, 96.19, 96.343, 96.372, 96.353, 96.276, 96.368, 96.339, 96.286, 96.522, 96.402, 96.31, 96.523, 96.417, 96.46, 96.482, 96.611, 96.495, 96.472, 96.475, 96.511, 96.545, 96.557, 96.536, 96.696, 96.375, 96.628, 96.514, 96.626, 96.653, 96.629, 96.623, 96.5], 'train_loss': [4.447664340337117, 3.6317639201402816, 3.194250587767237, 2.880182179135538, 2.624011810590118, 2.4145236536240775, 2.23120198597606, 2.0693714985539344, 1.9208149829127432, 1.7781723878815323, 1.6472009470351445, 1.5211760316685232, 1.4000003506415788, 1.2928901897053344, 1.1889999368140427, 1.0941365085308628, 1.0032256930704233, 0.9237294633916304, 0.8498904996397246, 0.7836987325779841, 0.733814231512719, 0.6905155856107491, 0.6329238991171445, 0.6034096199792696, 0.5640299791662036, 0.5338067831477521, 0.5083897768669379, 0.4819886699671632, 0.46794375645679614, 0.45015457517068813, 0.43023684982660865, 0.4127323732056529, 0.39542763252156865, 0.390945145956843, 0.3719638413198469, 0.370482058000351, 0.359210642389548, 0.34212734542134016, 0.3286872645166732, 0.33603676359907453, 0.3193191293083603, 0.30987769761435585, 0.3117665807148698, 0.3001257789441011, 0.29360850293115404, 0.2881759850800476, 0.28588063251501233, 0.2751632787642365, 0.27256890982675464, 0.26909699132731535, 0.26406005128231363, 0.2619403790901829, 0.2471807581847456, 0.25431847519891354, 0.25425919080039894, 0.24102487341427528, 0.24804484287201786, 0.24044134263700007, 0.22928361810853445, 0.2350036298340128, 0.22776092817471794, 0.2282285552897317, 0.2250402000549792, 0.21865009680337924, 0.22097400070113854, 0.21926675437427032, 0.21435528573766588, 0.21068609228282595, 0.2067596538538162, 0.20205897250959337, 0.21317407596076976, 0.20111232578053698, 0.2086420801095188, 0.19851824235807453, 0.19617119015707507, 0.19795210429146096, 0.19688443894130409, 0.19140335015168441, 0.18950066491824433, 0.18999588744499157, 0.18573332204223061, 0.1883785508636216, 0.18076946846461514, 0.18609297289962923, 0.1811577125001992, 0.18288519561007113, 0.18164488857389163, 0.17193473191361014, 0.17726614422082032, 0.17500176497747597, 0.1772114167702566, 0.17367674866113253, 0.1691800624910344, 0.1703949715022739, 0.1734190277394172, 0.16789461424489643, 0.16973683695318995, 0.16349848465259906, 0.16575500848795444, 0.16529913444008848, 0.16283642537239404, 0.16202977094120005, 0.16345679930989856, 0.15997675929253685, 0.15696830641995502, 0.15731530481090225, 0.1602612571619208, 0.15634943023192924, 0.1556121224012543, 0.1533309512090007, 0.15360862390696525, 0.15627109635515843, 0.15133630260419975, 0.15326329581385867, 0.15106151691482714, 0.150313741039045, 0.15296330274024922, 0.14636446009238113, 0.15040430526165094, 0.142093579866551, 0.14735188693855666, 0.14182403468796823, 0.15032896856072472, 0.1417508825729169, 0.14446908640359443, 0.13922323667046854, 0.14334526498428404, 0.13979894822622488, 0.1425277712698001, 0.14023039020368233, 0.13484168682343928, 0.14280522222212388, 0.14165062158575886, 0.13675166511883147, 0.13658812345649915, 0.14047641469307495, 0.1423074736422844, 0.13396306484026008, 0.1341549625640505, 0.1377856143020625, 0.13193490771354507, 0.133850046114949, 0.1369840754096242, 0.12988625071845683, 0.13288284137800946, 0.13363717578921136, 0.13315552591613744, 0.13220184707137628, 0.12800569098781836, 0.1316541456481113, 0.12324736873656202, 0.13020957575176062, 0.13051486209568808, 0.1266005720954534, 0.12931193991073228, 0.12413584959035726, 0.12794699657299352, 0.1262902056119659, 0.12667609682313796, 0.12299858816060394, 0.12809741330885413, 0.12238313308334536, 0.1255729090991732, 0.12841910511354773, 0.12368590171047876, 0.12246055906748439, 0.12121409217799314, 0.12241537846147868, 0.12541610407334663, 0.12113574715387804, 0.12080747098751338, 0.12010595518104304, 0.12395501809009372, 0.12200313794169806, 0.12181026271695744, 0.12134635115446915, 0.11410085088059471, 0.11686416793976906, 0.12052785206233227, 0.11685583644815598, 0.11997334240332141, 0.11959326753080884, 0.11732644704021561, 0.11261974575401118, 0.11827872354407352, 0.11809431088770879, 0.11777914093870798, 0.11603113341463106, 0.11423035555818432, 0.11514349561877327, 0.11593082709614365, 0.10932802582029742, 0.11986090886759347, 0.11316711685717969, 0.11590350366646504, 0.11306640859043807, 0.11210697658407821, 0.11422131179916235, 0.11367933755315678, 0.11602317356489306], 'val_metric': [11.91, 20.84, 26.88, 31.21, 34.84, 37.5, 39.54, 39.89, 42.31, 41.73, 43.28, 42.85, 42.78, 41.58, 41.26, 42.3, 40.7, 41.67, 41.74, 40.67, 41.21, 40.94, 41.05, 40.43, 40.89, 40.23, 40.67, 41.0, 40.35, 41.0, 41.27, 41.13, 40.22, 40.19, 39.37, 40.24, 40.77, 41.08, 40.55, 40.59, 40.77, 41.16, 39.91, 40.95, 40.9, 40.06, 40.25, 40.8, 40.29, 40.4, 40.71, 41.25, 40.16, 40.31, 41.28, 40.71, 41.51, 41.04, 42.29, 40.4, 40.59, 41.0, 41.72, 41.14, 41.47, 40.55, 41.51, 40.89, 40.8, 40.92, 40.4, 41.42, 40.68, 41.39, 41.47, 40.64, 41.58, 41.4, 41.6, 40.9, 40.97, 40.4, 41.84, 41.53, 40.99, 40.77, 41.67, 41.38, 41.36, 41.71, 41.06, 41.84, 42.02, 41.61, 40.99, 41.18, 41.59, 41.37, 41.89, 41.86, 41.79, 41.01, 41.36, 40.98, 42.2, 42.04, 41.33, 41.79, 40.92, 42.07, 41.96, 42.41, 42.0, 42.38, 42.12, 42.08, 41.64, 42.31, 42.33, 42.03, 42.29, 41.96, 42.06, 42.17, 41.95, 42.25, 42.53, 42.03, 42.26, 41.83, 42.08, 42.28, 42.74, 42.28, 42.61, 42.62, 42.84, 42.58, 42.63, 41.44, 41.72, 43.38, 42.31, 42.53, 42.26, 42.76, 42.16, 42.25, 42.19, 42.51, 42.04, 42.32, 42.06, 42.29, 42.05, 41.99, 42.56, 42.57, 42.4, 41.71, 42.11, 41.37, 42.06, 42.16, 42.02, 42.89, 42.67, 42.19, 42.34, 41.72, 42.13, 42.31, 42.29, 42.8, 42.81, 42.1, 42.6, 42.67, 42.82, 42.44, 41.75, 42.38, 42.6, 42.27, 42.38, 42.38, 42.43, 42.7, 42.0, 41.82, 42.08, 42.31, 42.89, 42.43, 41.9, 42.76, 42.22, 42.52, 42.97, 41.58], 'val_loss': [4.081316588790553, 3.494564171050005, 3.1629865123967456, 2.9045161137914963, 2.7340003130542243, 2.6574237513694032, 2.553185740853571, 2.520786211748791, 2.478235799795503, 2.4997497728675793, 2.4942167537986855, 2.5568200501666705, 2.6385075225951566, 2.732945732250335, 2.8440711521039344, 2.91100383336377, 3.126377098119942, 3.094715793421314, 3.194830163269286, 3.3589158490964564, 3.4029500389554697, 3.500302247940355, 3.663148780537259, 3.752562839514131, 3.8103897131172713, 3.984388649843301, 3.969243527218035, 4.075250168514859, 4.113209482970511, 4.129989899647463, 4.189833149029191, 4.349287518270456, 4.383008510443815, 4.401296141041312, 4.629332225793486, 4.556227730337981, 4.452934413199213, 4.507420470000832, 4.625810773509323, 4.760045592192632, 4.7582237857162575, 4.782450190015659, 4.8556457978145335, 4.7626443099064435, 4.952821146151063, 4.861317578394702, 4.977727723729079, 5.065088120995054, 5.000011688584735, 5.043420491704516, 5.036938949755043, 5.019722050162637, 5.222748959899708, 5.2401556049942215, 5.16237557769581, 5.319012886399676, 5.174654458738436, 5.3749478129064965, 5.194035732062759, 5.356212840718069, 5.3689961122099765, 5.263843419445548, 5.359390750812118, 5.3235114698956725, 5.491636087939997, 5.44476529291481, 5.468683475901367, 5.508890656149312, 5.527196163584472, 5.587861367851306, 5.606813120993839, 5.536719141492418, 5.597355194152541, 5.623005800186449, 5.569214559664392, 5.514391265097697, 5.660000899794754, 5.668321396894516, 5.5802621074542875, 5.688467817701352, 5.646159691415774, 5.687995766378512, 5.7089257316224895, 5.675972627226714, 5.6781419902850105, 5.752855287236013, 5.698499861796191, 5.705727671362032, 5.783358166931541, 5.790930485269826, 5.75585122472921, 5.811776376833581, 5.772018412875521, 5.886598265095121, 5.75285727051413, 5.988174569075275, 5.889933824539185, 5.952678110948794, 6.055519427463508, 5.912606631114984, 5.86423532522408, 6.0159730038065815, 6.004624558861848, 5.957803615339243, 5.893701998291502, 6.185741143621457, 6.055028793918099, 6.037495849998134, 6.112646358028339, 6.121169372728676, 6.128627854547683, 5.939774247491435, 5.9310453849233635, 6.014088595748707, 6.1085284971127844, 6.239833049713426, 6.260608320782898, 6.2033413352480355, 5.988991802665079, 6.121569220427495, 6.1820527869425, 6.1082143130575774, 6.233081990746176, 6.127834497743351, 6.297122660715869, 6.165869750794331, 6.020319142159383, 6.385797910629564, 6.015709614298146, 6.222456505344172, 6.236298310528895, 6.218533429370564, 6.1549769875350275, 6.347799228255156, 6.261490313110838, 6.199311690725339, 6.153694784565336, 6.331522106365034, 6.326245546340942, 6.499680555550156, 6.430127897080342, 6.432994947311984, 6.284173648828154, 6.244707500858671, 6.208778112557284, 6.347326466991643, 6.375015160080734, 6.303710122017344, 6.368382040862065, 6.340320300144755, 6.282721794334946, 6.430770110172831, 6.57570457914073, 6.401572001208166, 6.550075799796232, 6.551179251093774, 6.544001942986895, 6.3215734229725635, 6.479938620974304, 6.565104334217728, 6.651973095669109, 6.638477960209937, 6.478677026785103, 6.58255012020184, 6.599817567570194, 6.370528861975214, 6.512082277589543, 6.508445984239032, 6.637430353529134, 6.758741454713664, 6.52194708016268, 6.526894955118751, 6.527874270821832, 6.639525052088841, 6.516914633428975, 6.735088064412403, 6.62516839944633, 6.3001360878063615, 6.614978893547301, 6.643752860415513, 6.652223445807293, 6.636002601331966, 6.671139141556564, 6.6269577688472285, 6.602401288451662, 6.9160802516208335, 6.70233687929287, 6.636728230555346, 6.6269686480236665, 6.656186093190673, 6.605347965932955, 6.5875842419399575, 6.517934367914868, 6.5597494605240545, 6.764112459626167, 6.86955531995008, 6.6024185092585865, 6.658539706734335, 6.7495364945405605, 6.92832421041598]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="SGD")
data.update({'train_metric': [0.677, 0.955, 1.223, 1.422, 1.675, 2.007, 2.201, 2.498, 2.709, 2.96, 3.1, 3.231, 3.554, 3.753, 3.903, 4.129, 4.538, 4.72, 4.988, 5.23, 5.492, 5.528, 5.886, 6.057, 6.357, 6.574, 6.886, 7.203, 7.368, 7.59, 7.964, 8.209, 8.501, 8.847, 9.048, 9.382, 9.616, 9.985, 10.137, 10.539, 10.822, 11.105, 11.334, 11.574, 11.762, 12.351, 12.353, 12.762, 12.963, 13.23, 13.626, 13.835, 14.107, 14.484, 14.698, 14.959, 15.226, 15.653, 15.963, 16.028, 16.308, 16.607, 16.692, 17.141, 17.437, 17.853, 17.953, 18.192, 18.552, 18.887, 19.011, 19.277, 19.6, 19.83, 20.002, 20.454, 20.632, 20.911, 21.28, 21.483, 21.806, 21.959, 22.292, 22.524, 22.833, 23.049, 23.277, 23.488, 23.914, 24.04, 24.325, 24.555, 24.762, 25.016, 25.306, 25.462, 25.59, 26.102, 26.237, 26.394, 26.718, 26.854, 27.082, 27.462, 27.424, 27.811, 28.018, 28.304, 28.388, 28.842, 28.943, 29.131, 29.32, 29.759, 30.004, 30.106, 30.239, 30.602, 30.687, 31.022, 31.26, 31.304, 31.706, 32.026, 32.159, 32.438, 32.5, 32.784, 33.103, 33.319, 33.487, 33.727, 34.121, 34.317, 34.612, 34.812, 35.177, 35.155, 35.512, 35.696, 36.02, 36.172, 36.389, 36.659, 36.99, 37.113, 37.369, 37.567, 37.834, 38.178, 38.241, 38.749, 38.743, 39.007, 39.304, 39.488, 39.841, 39.793, 39.982, 40.394, 40.53, 40.92, 41.279, 41.354, 41.678, 41.632, 42.176, 42.338, 42.256, 42.811, 43.012, 43.34, 43.295, 43.526, 44.011, 44.198, 44.198, 44.615, 44.704, 45.052, 45.289, 45.407, 45.599, 45.619, 46.006, 46.409, 46.497, 46.38, 46.93, 47.118, 47.362, 47.755, 47.711, 47.908, 48.215, 48.324, 48.796, 48.747, 49.018, 49.171], 'train_loss': [5.29408479636858, 5.27268769446658, 5.250994792140148, 5.233163937344737, 5.215622082476576, 5.19378688742698, 5.164543188769926, 5.131049445403057, 5.09439328078345, 5.05430646500981, 5.016257850771445, 4.978373270040891, 4.940209739877868, 4.9047750468751365, 4.873523864392203, 4.8414083604086535, 4.811112724201693, 4.783418275268125, 4.755802070949601, 4.729852533309947, 4.700705773542115, 4.676023697136155, 4.649028077311647, 4.620043511735425, 4.592169719251851, 4.5632674262375685, 4.532139495329756, 4.50237970083704, 4.47296942432035, 4.44493721329243, 4.418792918181465, 4.389973711830183, 4.364708885045213, 4.340505398013846, 4.317551720699132, 4.291428313526829, 4.265939279313432, 4.243010934163421, 4.219578344091268, 4.196838273806825, 4.174632547150342, 4.155202368780809, 4.130046572200167, 4.108393047074057, 4.085824370307947, 4.063972818950622, 4.046074237719759, 4.024903620807162, 4.003262894930019, 3.984295847929981, 3.962188330355624, 3.942236080737123, 3.9216472115641743, 3.9022438745809835, 3.882666197482089, 3.8611030612171877, 3.842566640881949, 3.8199440950929393, 3.8036476874763356, 3.7885748333299456, 3.764679873210836, 3.745753046342065, 3.729977066976019, 3.7111235023383826, 3.692764607439877, 3.673713771334384, 3.655498071122612, 3.6338373633889303, 3.6189090468833176, 3.6026869773559667, 3.584839044132831, 3.566429723018419, 3.547814577875119, 3.534226741839584, 3.5184608277646077, 3.4994055588735042, 3.48208618911504, 3.469426742975939, 3.4491557219206945, 3.4328415724839143, 3.4197789207148537, 3.4018931034964317, 3.3872858103619734, 3.3705532023026556, 3.3580560691831054, 3.3441717935844024, 3.325867778890345, 3.308684643002862, 3.29549127226065, 3.2829027886735425, 3.265739720827177, 3.2532526081522075, 3.237431787636062, 3.2242039020291826, 3.2087805301656496, 3.1942775252531983, 3.1831081554207827, 3.1645344197559417, 3.1507588386230565, 3.1410411769430073, 3.122973172846164, 3.113455191836171, 3.0993490678258837, 3.0826916693115725, 3.072124101378867, 3.0577163719169924, 3.044003173318034, 3.0292145710150096, 3.0172969558188645, 3.0049935979715006, 2.9908165390187933, 2.9782986892619654, 2.9666016859956374, 2.946681551954644, 2.937127689787461, 2.924246223172696, 2.9118330262062724, 2.899494436255496, 2.8842000543346638, 2.868779645390184, 2.8589811726251018, 2.844860914343836, 2.8309148018434884, 2.819986048068134, 2.8043231691066377, 2.7912267572362683, 2.7770183493674603, 2.765659944262172, 2.7481483484183986, 2.738829416719218, 2.7256295443038794, 2.7111250713553403, 2.698226842068741, 2.6812393151257026, 2.6663980924846724, 2.6563324775744612, 2.6414088428745037, 2.6354652191688066, 2.6191646474641788, 2.6072430941666536, 2.595427832386849, 2.5779674743431467, 2.569443499973319, 2.5526960053965593, 2.5420492065311318, 2.530269757189662, 2.5166521052557616, 2.503802734769771, 2.4881555761043184, 2.4781025049401184, 2.4638769127814646, 2.4507731858233344, 2.4369775374882967, 2.4299159737939036, 2.418694184593718, 2.4037193435930093, 2.393482945930935, 2.3807243262813857, 2.367644930755337, 2.356599077534691, 2.3417016566753084, 2.3273251380816684, 2.317129722712365, 2.308771956676256, 2.2909133293959703, 2.284068310710763, 2.264982816887756, 2.255891018697869, 2.250069236648594, 2.233194655969367, 2.2235947759656365, 2.2075880466938322, 2.198221688841065, 2.190554274325941, 2.1764400774900223, 2.166222864057647, 2.1541631623711712, 2.1422176567774436, 2.131140992493486, 2.119230531685183, 2.110196044943841, 2.101302596825632, 2.091112015648523, 2.0764675013010727, 2.0674997049497628, 2.0551090806398498, 2.045464788349637, 2.0365448985737404, 2.0273644121808267, 2.01715904127232, 2.0037812741994094, 1.9913269760360035, 1.9841559284097936, 1.9733338437626473, 1.9637126976758794, 1.9532693145523754, 1.9426709723945466, 1.9349536073566322, 1.9217204377960868, 1.9173403247685594], 'val_metric': [0.84, 0.96, 1.45, 1.7, 2.02, 2.28, 2.48, 2.75, 2.99, 3.05, 3.34, 3.53, 3.84, 4.03, 4.34, 4.42, 4.75, 5.11, 5.34, 5.57, 5.77, 6.0, 6.21, 6.47, 6.83, 7.37, 7.26, 7.76, 8.0, 8.31, 8.55, 9.19, 9.28, 9.35, 9.96, 10.43, 10.44, 11.01, 10.62, 11.11, 11.43, 11.6, 11.95, 12.28, 12.64, 12.72, 13.01, 12.88, 13.44, 13.61, 13.61, 14.2, 14.28, 14.61, 14.72, 14.92, 15.43, 15.93, 15.8, 16.08, 16.34, 16.53, 17.06, 16.89, 17.02, 17.61, 17.77, 17.72, 18.36, 18.21, 18.76, 18.88, 19.08, 19.49, 19.46, 19.59, 20.08, 19.93, 20.31, 20.05, 20.5, 20.74, 21.09, 20.95, 21.43, 21.47, 21.51, 21.82, 21.82, 22.37, 22.13, 22.43, 22.39, 21.94, 22.78, 23.02, 23.15, 23.19, 23.03, 23.46, 23.9, 23.68, 23.85, 24.12, 23.76, 24.35, 24.34, 24.14, 24.14, 24.64, 24.3, 24.29, 24.36, 24.98, 25.09, 25.19, 24.79, 23.9, 24.94, 24.33, 24.69, 24.61, 24.94, 25.18, 24.85, 25.05, 25.68, 24.85, 25.09, 25.0, 24.96, 25.05, 24.99, 24.81, 25.2, 25.55, 24.76, 25.21, 24.88, 25.01, 25.09, 25.19, 24.89, 24.6, 25.15, 25.28, 25.47, 24.65, 25.14, 25.05, 25.12, 24.81, 25.03, 25.11, 25.41, 24.75, 25.0, 24.88, 24.47, 24.87, 24.43, 25.12, 24.74, 24.18, 24.83, 24.28, 24.46, 24.36, 24.67, 24.16, 24.48, 24.97, 23.8, 24.13, 24.2, 24.07, 23.45, 22.75, 23.96, 23.47, 24.28, 24.07, 24.03, 24.18, 24.0, 23.63, 23.93, 23.51, 23.98, 23.87, 23.78, 23.54, 23.3, 23.54, 24.63, 22.6, 23.44, 23.14, 23.24, 23.53], 'val_loss': [5.284535587213601, 5.2595801292711, 5.238822774522624, 5.220503488164039, 5.200458772622856, 5.17542449835759, 5.140107734947447, 5.103975690853823, 5.064375195533606, 5.02501785527369, 4.986521169638178, 4.941200229013042, 4.907838104636806, 4.865648075273842, 4.833480833442348, 4.803080575481342, 4.773066423501179, 4.740741129893406, 4.714275715457406, 4.686264238539775, 4.6682515341764805, 4.6361572605789085, 4.61122348050403, 4.5820764265242655, 4.550211144101088, 4.516687456969243, 4.49060625331417, 4.460579304178809, 4.435407266495334, 4.3977139618746035, 4.376835842800748, 4.345507796402949, 4.324627968915708, 4.307375974715895, 4.284124717590915, 4.259725889582543, 4.238991246861257, 4.205058766018813, 4.188712024385003, 4.166506723233849, 4.145835469482811, 4.122702644129467, 4.099738449048084, 4.079905755960258, 4.065614724614818, 4.042265843433939, 4.021607092231702, 4.007717814415124, 3.981502757710256, 3.96390358353876, 3.9489889919378194, 3.931745482098525, 3.9137056739466964, 3.888976399306279, 3.874418761320175, 3.8523240605736992, 3.8412385351338965, 3.827116120393109, 3.8079970961163756, 3.790444518350492, 3.7768784905694854, 3.766302901468459, 3.739243123182066, 3.7316453715038906, 3.718264786301145, 3.692827798758343, 3.6997932218442298, 3.6681067776528136, 3.6550815325633734, 3.6493255234068367, 3.6316547074894996, 3.6079859824696925, 3.603594024469898, 3.5840917521980917, 3.5719950032082335, 3.576423868252214, 3.5534922211033524, 3.541675493216059, 3.5270147065448154, 3.518892357303838, 3.5035293208565683, 3.493828379424514, 3.483823182476554, 3.4924423983142634, 3.461990619920621, 3.4555917317700233, 3.4437186983740253, 3.4452039595622166, 3.435030372279465, 3.4175937730035963, 3.418335545594525, 3.4105416103533117, 3.4056114732839498, 3.41982751211543, 3.379615358486297, 3.378743981100192, 3.375130490892252, 3.363331103021172, 3.3869655063957165, 3.354348526638784, 3.3416399287570053, 3.3518141006967825, 3.333900860160779, 3.322700080598236, 3.3439101216140066, 3.314734134704444, 3.3201491164553696, 3.3161393753282584, 3.3276139520535803, 3.2983356509239052, 3.3032693832543245, 3.3081948779950476, 3.3104306383497395, 3.2918903880817876, 3.2929554145047617, 3.2965961315069987, 3.3018567918971846, 3.3395110755969006, 3.290007627693711, 3.307078770011853, 3.2958382375680717, 3.292263687795894, 3.3084213832381426, 3.295371323634105, 3.292074637048563, 3.29498227766365, 3.287572844772582, 3.318930499872584, 3.3105407376198253, 3.3156017315615514, 3.330276490776402, 3.315830902688822, 3.322199643797176, 3.327604430496313, 3.33749888505146, 3.3318368758365606, 3.358209752732781, 3.3513004028113786, 3.374551811795326, 3.3379934973018184, 3.344877943111833, 3.364123742291882, 3.387573224723719, 3.3697871736660123, 3.3594896034070643, 3.3796697528498947, 3.3877140518965994, 3.4313890789724457, 3.400386461027109, 3.411478816323979, 3.4261399051945682, 3.409299621916121, 3.428662647107604, 3.4127052020115456, 3.439002885939969, 3.455832793454456, 3.455298723688551, 3.4687046139103592, 3.5012079925294133, 3.473707247691549, 3.538489298456034, 3.4878708093788973, 3.53436237080082, 3.5227113628083733, 3.5475804950021637, 3.567226917880356, 3.5809444568719075, 3.5658890418945606, 3.6132767367514833, 3.607678250142723, 3.599590019055992, 3.592378222258987, 3.683601914697392, 3.640459769091029, 3.64460629110883, 3.661966530380735, 3.7151470966399853, 3.7617199519637285, 3.712092838469584, 3.758180893910159, 3.675627583910705, 3.734167045848385, 3.7572854218209626, 3.718391414660557, 3.732923159933394, 3.7759478190901934, 3.7916327441573903, 3.8239206311049734, 3.799039929535738, 3.7612320108778157, 3.825347797126527, 3.862659853734788, 3.9069219728943647, 3.8295500491075454, 3.8214160165968973, 3.958721505608528, 3.9204521908122265, 3.9225803317537733, 3.927218625499944, 3.9152964391526144]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="Adam")
data.update({'train_metric': [7.452, 14.282, 17.629, 19.812, 21.729, 23.147, 23.946, 24.951, 25.95, 26.651, 27.337, 27.854, 28.651, 29.127, 29.689, 30.254, 30.449, 31.047, 31.408, 31.884, 32.103, 32.582, 32.917, 33.15, 33.493, 33.705, 33.989, 33.992, 34.318, 34.686, 34.69, 34.915, 35.341, 35.4, 35.488, 35.717, 36.004, 36.173, 36.259, 36.534, 36.648, 36.724, 36.908, 37.131, 37.306, 37.427, 37.394, 37.676, 37.75, 37.907, 38.002, 38.181, 38.313, 37.98, 38.264, 38.336, 38.56, 38.389, 38.416, 38.622, 38.812, 38.975, 38.845, 38.869, 38.938, 38.954, 39.157, 39.166, 39.237, 39.036, 39.182, 39.414, 39.476, 39.432, 39.553, 39.681, 39.496, 39.599, 39.535, 39.648, 39.771, 39.683, 39.862, 39.728, 39.753, 39.829, 39.935, 39.86, 40.019, 39.956, 40.036, 40.238, 40.2, 40.043, 40.298, 40.161, 40.303, 40.344, 40.351, 40.274, 40.153, 40.433, 40.419, 40.269, 40.145, 40.408, 40.519, 40.556, 40.735, 40.576, 40.621, 40.627, 40.614, 40.554, 40.821, 40.803, 40.677, 40.666, 40.627, 40.815, 40.738, 40.705, 40.894, 40.876, 40.745, 40.732, 40.841, 40.899, 40.954, 40.976, 41.055, 40.922, 40.751, 40.898, 40.969, 40.9, 41.002, 41.183, 41.054, 41.033, 40.989, 40.919, 41.118, 41.334, 41.166, 41.168, 41.16, 41.259, 41.059, 41.135, 41.09, 41.234, 41.255, 41.2, 41.206, 41.166, 41.286, 41.43, 41.497, 41.294, 41.349, 41.433, 41.447, 41.322, 41.536, 41.425, 41.279, 41.58, 41.452, 41.405, 41.553, 41.46, 41.541, 41.463, 41.519, 41.559, 41.536, 41.521, 41.643, 41.395, 41.646, 41.708, 41.75, 41.721, 41.694, 41.68, 41.7, 41.765, 41.594, 41.829, 41.806, 41.869, 41.976, 41.87, 41.675, 41.67, 41.822, 41.948, 41.791, 41.859], 'train_loss': [4.4021092695222785, 3.8225535756109315, 3.5990837001678506, 3.4622816464417423, 3.3578988971881047, 3.279124703684909, 3.2208238117830614, 3.169653097635954, 3.1204219858995708, 3.084015989639175, 3.0445053385223857, 3.0166329969905235, 2.9807141057512965, 2.954045820449761, 2.9250061838427035, 2.9037710770688143, 2.8810767230511627, 2.8590990894510435, 2.843260418506898, 2.8237037362765594, 2.802228968645317, 2.790928618082692, 2.7725143522615245, 2.7595838713325604, 2.7460275909798466, 2.7320848394492767, 2.7155157520613913, 2.708130117265063, 2.7005028521793437, 2.687133403367441, 2.682224978183373, 2.6718687893714343, 2.6607401242869373, 2.651332948578525, 2.637571091691577, 2.6293327653552963, 2.623137252268239, 2.6159852890349016, 2.6071073099808744, 2.6000977985689597, 2.591353024951327, 2.5867437555175217, 2.5745506827372124, 2.571178748679329, 2.566320293771862, 2.554285448785783, 2.5562198268093517, 2.548290695506491, 2.54628913233239, 2.5377867921986645, 2.5340134043458633, 2.5268304488327895, 2.525301488255814, 2.5231464398036914, 2.5185099110264697, 2.5133045631117037, 2.512298516790911, 2.509491698679372, 2.5079823698817503, 2.50359536567256, 2.5014181538262736, 2.492292059589027, 2.4907522743051813, 2.488142296433525, 2.4849274791514957, 2.4882867546777123, 2.4814062345050805, 2.4814827079888877, 2.4803051098904696, 2.474215111287069, 2.473343655610039, 2.4690822165361674, 2.466339608681789, 2.46638451244918, 2.461025469896508, 2.459301104548644, 2.4605506021703425, 2.4565976226627253, 2.458979135206397, 2.4543146904050275, 2.449303454492463, 2.4454964063751796, 2.44599385926606, 2.4458868174848845, 2.444196106605017, 2.4386552771313865, 2.4420426270173134, 2.4409876032205093, 2.4325213294264145, 2.43616687816759, 2.433960630431514, 2.4326882166353005, 2.4245682994448368, 2.4334844560556053, 2.426139822543163, 2.4238982300535654, 2.422456066195048, 2.423060787463905, 2.426127824734513, 2.4194750438038537, 2.4240794994094323, 2.4172878729099656, 2.4171236326354326, 2.4192174451136084, 2.4166114614014433, 2.4141505288528617, 2.4129376808802285, 2.4145385894879117, 2.411109773989145, 2.4110692488559913, 2.406848485700152, 2.4107976435699756, 2.405762538800084, 2.40729880127019, 2.403893471298047, 2.398395721529511, 2.403885537206707, 2.40309500679021, 2.4040010041025153, 2.405217700102203, 2.4010404875395928, 2.400066951643711, 2.399519762013558, 2.398565797644095, 2.3988613850477027, 2.3948345999647542, 2.39438590702916, 2.3929054252016795, 2.3946045453321148, 2.3949593612191316, 2.39389398055281, 2.3910386913949986, 2.3898025256887276, 2.388961335900344, 2.390942177098299, 2.393229494396876, 2.3864332827252603, 2.390056136549854, 2.3882818697205126, 2.3861355523001437, 2.3869138325320858, 2.3904476561610393, 2.387653056811005, 2.3857611543615125, 2.3825345998838476, 2.386482089586313, 2.3832814303713583, 2.376919431100651, 2.3838274954834278, 2.383266010729838, 2.381647240635072, 2.375085919771298, 2.3793545620455165, 2.3771656087325006, 2.3789297977625674, 2.3754663057077106, 2.375253279164901, 2.3697783560304404, 2.371817288609247, 2.3700118016067866, 2.3735367235127582, 2.371805656207958, 2.3722916706509873, 2.369340987977353, 2.3702364748132894, 2.371466646877833, 2.3710935703852356, 2.3686159290873845, 2.3689039700777195, 2.3726464854702307, 2.368193060483829, 2.3646272183684913, 2.365722898176978, 2.3622572822442667, 2.3629748690837635, 2.366445991219577, 2.3608221468525823, 2.3636684776191443, 2.3590934173616933, 2.3637592182354674, 2.3613096310096298, 2.3614550381994217, 2.3538805822950866, 2.357439245997677, 2.356423055294303, 2.358919562091449, 2.3586569521874092, 2.3559087148783533, 2.3570640292292744, 2.354368719700736, 2.3557412461905014, 2.3583756866778463, 2.353801899359002, 2.3551285120438705, 2.356632278427739, 2.3566479885799336, 2.352861316480167, 2.354131183865279, 2.3539065195060433, 2.349210777728129], 'val_metric': [9.89, 15.31, 16.15, 20.27, 18.83, 22.53, 21.87, 23.01, 24.17, 24.08, 25.33, 23.5, 22.75, 28.41, 26.99, 28.16, 27.53, 19.88, 26.9, 27.88, 27.52, 27.16, 30.31, 29.43, 29.98, 30.43, 29.04, 26.43, 30.44, 32.61, 29.08, 31.0, 32.4, 31.45, 32.54, 30.94, 27.75, 31.41, 34.6, 32.12, 32.32, 32.31, 31.63, 33.14, 32.14, 29.41, 32.55, 34.37, 33.51, 35.37, 33.12, 33.01, 31.74, 34.11, 34.32, 30.12, 31.83, 33.12, 34.68, 29.41, 33.68, 34.07, 32.49, 33.55, 34.71, 32.66, 33.85, 30.54, 35.41, 35.51, 34.1, 35.6, 30.73, 33.9, 34.69, 35.92, 34.9, 33.94, 37.05, 34.82, 35.09, 35.33, 31.62, 35.84, 34.87, 31.18, 33.56, 36.2, 33.39, 33.82, 36.6, 34.85, 32.16, 36.12, 34.15, 37.52, 34.87, 32.8, 35.14, 34.27, 31.76, 35.16, 33.03, 37.09, 30.62, 35.01, 36.45, 36.0, 33.31, 34.44, 33.13, 34.75, 34.84, 34.93, 35.32, 34.76, 35.75, 33.74, 30.15, 37.77, 35.27, 35.23, 35.29, 34.62, 36.51, 33.92, 34.18, 34.74, 34.07, 36.69, 36.35, 35.78, 32.26, 34.5, 34.46, 33.71, 33.02, 36.28, 35.27, 35.84, 34.67, 34.43, 37.21, 36.99, 37.38, 34.85, 36.84, 37.73, 33.46, 36.07, 33.85, 36.33, 35.29, 33.45, 35.99, 36.05, 35.01, 34.69, 37.04, 36.48, 36.46, 35.47, 35.4, 34.06, 35.09, 35.83, 38.0, 37.09, 32.51, 33.08, 35.56, 34.87, 38.28, 35.0, 34.31, 34.82, 35.66, 36.51, 33.38, 35.88, 35.46, 37.22, 37.55, 37.34, 34.24, 37.22, 35.72, 33.64, 34.32, 34.5, 35.54, 35.07, 36.17, 34.88, 34.71, 36.32, 35.39, 32.79, 35.35, 35.02], 'val_loss': [4.21493996632327, 3.7361603162850545, 3.67634993525827, 3.451582909389666, 3.5526790801127244, 3.335668204696315, 3.3959553340438067, 3.2579702867823803, 3.2072911444742966, 3.2441391565237834, 3.1663981304047213, 3.2542318561274532, 3.3162929920633886, 2.9843303747237866, 3.063648180597147, 3.0106408159444285, 3.0836265611041123, 3.6458479272332163, 3.142503472650127, 3.064513043992838, 3.0803232299294443, 3.109523064011981, 2.9368948822568175, 2.962504978392534, 2.917501822398726, 2.9029300182488313, 3.0176207723131605, 3.2016192416476597, 2.9397752824102996, 2.8015940872726928, 3.032528636561837, 2.882371805276081, 2.8105107864756493, 2.897277625503054, 2.826265572742292, 2.888859120144206, 3.1034330702891015, 2.885806273502909, 2.7216116243107304, 2.826070067988839, 2.8135719777672152, 2.811872174785395, 2.887501629294863, 2.7951689639668555, 2.81915641514359, 3.0442356509008226, 2.8487104830468537, 2.728726574569751, 2.7872312281541762, 2.675325168166191, 2.7940807183077383, 2.827245747207836, 2.890514527916149, 2.7414763900125103, 2.7668562125248513, 3.037744670916515, 2.8828270002535192, 2.8176672435869836, 2.7592789640851842, 3.0071785210803816, 2.8009346845043694, 2.7509665109549357, 2.881900874672422, 2.8009934850559115, 2.726135429683005, 2.8500243535467016, 2.7742649658470397, 2.9902911994867263, 2.7049662003851243, 2.6503027795226712, 2.7356588863263465, 2.6611815322736265, 2.9583746627637537, 2.769150637137662, 2.7224604563348613, 2.647404089854781, 2.6841118343316825, 2.7723798178563452, 2.5884976869176146, 2.6977959195519707, 2.696036429162238, 2.6902371599416064, 2.8103078672081043, 2.656547296578717, 2.6992479259041464, 2.93479562261302, 2.76233577652342, 2.6475712486133456, 2.748737976049921, 2.768580554397243, 2.6219183757046984, 2.714869616897243, 2.8721701859668562, 2.6488386392593384, 2.7355793350061792, 2.572933498461535, 2.702273027152772, 2.853953579428849, 2.6631946973739917, 2.7584095046778394, 2.867396881246263, 2.719375867372865, 2.7994895205376253, 2.589276849084599, 2.898875327626611, 2.70844063181786, 2.6460851484043584, 2.66010458454205, 2.802920493350667, 2.722999906843635, 2.827994367119613, 2.703287246120963, 2.72206659112007, 2.742914513038222, 2.71754914522171, 2.7128241912574524, 2.6346690024539923, 2.767629505722386, 2.956507766322725, 2.5813223017249136, 2.6877145979814467, 2.679127588393582, 2.7479918724412373, 2.68882516083444, 2.6189751488387962, 2.718054925560192, 2.7567135376535403, 2.7265944154399215, 2.7341962183357045, 2.6470855087231677, 2.6191405350235617, 2.6373050121744726, 2.8442504834976927, 2.712795059392407, 2.715636215012544, 2.7426528680096767, 2.831997481121379, 2.6341075631463604, 2.70470795585851, 2.6413827785261117, 2.7388755257721917, 2.73845847624882, 2.624824838273844, 2.615404811254732, 2.5793129258854375, 2.7669397220490084, 2.630988247075658, 2.584547725452739, 2.788886736153038, 2.6238848305052254, 2.7391887227441094, 2.654187222195279, 2.672225097941745, 2.8003374892435255, 2.6439171737166727, 2.6474921961498867, 2.6990667034865945, 2.7178234704740487, 2.5935548536337105, 2.6312395106455324, 2.6596927714955276, 2.6740880908480116, 2.7204097744765554, 2.7996868457004522, 2.6865931969539374, 2.650365537139261, 2.5283097712097655, 2.599497733222451, 2.84177994196582, 2.839701682898649, 2.6917741750455964, 2.686216454597036, 2.5455065320251853, 2.742566816366402, 2.7108448432509307, 2.7644395851025916, 2.651986159336795, 2.657835200334051, 2.8348225727202787, 2.683305918790732, 2.7013436836801517, 2.593154728412628, 2.5862592891523035, 2.6057729512263257, 2.74588475371622, 2.5872580306545183, 2.6718696727874174, 2.84198833081373, 2.736524236050381, 2.7380494949923957, 2.696079275410646, 2.700343878785516, 2.6685742079072696, 2.7238831182194363, 2.738087479475957, 2.668526289189697, 2.6974078227000633, 2.8813753424176745, 2.694571475694134, 2.7133632788232935]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="AdamW")
data.update({'train_metric': [10.466, 21.978, 29.171, 34.645, 39.214, 43.18, 46.637, 49.904, 52.851, 55.478, 58.317, 60.993, 63.799, 65.945, 68.51, 70.643, 72.592, 74.443, 76.361, 77.925, 79.393, 80.544, 81.938, 82.794, 83.734, 84.599, 85.338, 85.819, 86.672, 86.693, 87.473, 87.922, 88.222, 88.495, 89.047, 89.093, 89.61, 89.935, 90.284, 90.342, 90.69, 90.746, 91.078, 91.328, 91.435, 91.67, 91.776, 91.764, 91.862, 92.267, 92.331, 92.364, 92.482, 92.648, 92.892, 92.875, 93.015, 92.933, 93.218, 93.105, 93.434, 93.249, 93.621, 93.681, 93.473, 93.847, 93.897, 93.952, 93.9, 93.942, 94.12, 94.325, 94.172, 94.293, 94.28, 94.314, 94.431, 94.419, 94.451, 94.629, 94.562, 94.927, 94.726, 94.806, 94.823, 94.709, 95.01, 94.972, 94.948, 95.085, 95.049, 95.242, 95.192, 95.11, 95.173, 95.184, 95.481, 95.209, 95.258, 95.386, 95.366, 95.48, 95.453, 95.399, 95.586, 95.463, 95.495, 95.67, 95.525, 95.803, 95.644, 95.769, 95.624, 95.752, 95.831, 95.613, 95.765, 95.904, 95.639, 96.06, 96.002, 95.953, 96.084, 95.865, 95.954, 95.951, 96.188, 95.981, 96.13, 95.962, 96.217, 96.146, 96.147, 96.127, 96.115, 96.221, 96.304, 96.178, 96.269, 96.319, 96.35, 96.221, 96.259, 96.306, 96.365, 96.34, 96.443, 96.407, 96.356, 96.257, 96.541, 96.338, 96.48, 96.475, 96.504, 96.476, 96.428, 96.401, 96.598, 96.629, 96.615, 96.617, 96.488, 96.514, 96.581, 96.713, 96.592, 96.675, 96.612, 96.681, 96.73, 96.606, 96.818, 96.488, 96.763, 96.759, 96.819, 96.512, 96.804, 96.719, 96.879, 96.86, 96.673, 96.769, 96.895, 96.794, 96.874, 96.867, 96.714, 96.884, 97.01, 96.836, 96.97, 96.832, 96.943, 96.894, 96.9, 96.997, 96.933, 96.833], 'train_loss': [4.200469688963448, 3.3915977571076183, 2.9982095222326706, 2.711272943195287, 2.4856344055687094, 2.294940430539888, 2.1320386851207616, 1.976713026019906, 1.8333174785740927, 1.701278705285744, 1.5705903882562389, 1.4474630715827186, 1.3272788725979268, 1.2208141276108784, 1.1168656880399468, 1.0263875604476673, 0.940374129907641, 0.8634680065869217, 0.7886846712096257, 0.7310988277826108, 0.6747081520575708, 0.6248504949050764, 0.5800867308925072, 0.5505722764776978, 0.515154934318418, 0.4890284827568938, 0.46089736027589456, 0.4465642980158672, 0.42110480506375897, 0.4183217344933111, 0.39066656143121514, 0.377675406203408, 0.36706112809026387, 0.357284284837339, 0.34365894790364626, 0.3413811247779136, 0.32101972724572636, 0.3139411421970572, 0.30713203743634054, 0.30155148488241207, 0.29054598446627955, 0.2944789351007626, 0.2767491090687627, 0.2740705291158445, 0.26638323611800896, 0.26519191643355444, 0.25934396496236856, 0.2583648903868611, 0.25249390435653546, 0.2467836603720125, 0.24250954412691347, 0.23884400377623255, 0.235571963737599, 0.2330684279671424, 0.2242223663673863, 0.22585858609968282, 0.2201157537821063, 0.22228352917252217, 0.2147079104699447, 0.21399652766646557, 0.20577291906790443, 0.21222533806544738, 0.2059424933259419, 0.19845596974642896, 0.2047105489700308, 0.19419770633655237, 0.19386393261771895, 0.19297918270688244, 0.1937510273273336, 0.1940541541399297, 0.18775627702195732, 0.1828597097777388, 0.18565472962021332, 0.18402963767474087, 0.18362246896036002, 0.18227508458262995, 0.1796440016990736, 0.17851208152531892, 0.17720656961298553, 0.17213544512724102, 0.17322432877369565, 0.16177864164031772, 0.16706853833076946, 0.16616782499030375, 0.16399697991122592, 0.17062810558935368, 0.1606808792569115, 0.1595297939256096, 0.16261981179614768, 0.16000372201865104, 0.15920878845366507, 0.15495160469989117, 0.15775449288995866, 0.15562625951774214, 0.1566440664608358, 0.15335943563680401, 0.14787614297852283, 0.1524480585403912, 0.153614081616346, 0.14850315878984666, 0.15159237455747668, 0.14574188590216583, 0.14817097380022046, 0.14732530473904853, 0.14366837855135295, 0.1464624484921719, 0.14484105327524727, 0.13964578694075833, 0.1451442343980117, 0.13610187876029314, 0.14417474945591224, 0.14041869095039525, 0.14132112590610976, 0.14091897467854395, 0.1373339328236052, 0.1415767606713655, 0.13561816366719498, 0.12967108601915173, 0.14300024390441257, 0.12939953363440354, 0.1288891116554759, 0.13484068061400895, 0.13068585382847167, 0.13581178371999497, 0.13287804664293767, 0.1319630807845109, 0.12353003063096599, 0.1325339306243031, 0.12836269708678022, 0.13252670791995802, 0.1218910881449836, 0.12624348816283104, 0.1286225652491098, 0.1257025398619673, 0.12729534551278424, 0.12259751270982179, 0.12122946864335725, 0.127557743120413, 0.12244640821391407, 0.1265579872950554, 0.12106232273273804, 0.12696947798091782, 0.12236598380844294, 0.12116855231296361, 0.12177035602489493, 0.1201918637768055, 0.11802597517591275, 0.11861052456006803, 0.12105123888163738, 0.12297393427118992, 0.1146993749008275, 0.12041527025612525, 0.11906823294553953, 0.1176306896165826, 0.11703228851264433, 0.11605759366860545, 0.11825891653280937, 0.11922412808030851, 0.11274719952794253, 0.11178008893480011, 0.11233427205885106, 0.11305133169093119, 0.11943547038441275, 0.11567438441283717, 0.11540399878475009, 0.10974806326608902, 0.11344511244436059, 0.11183073650918941, 0.11267042421987274, 0.11308927692804975, 0.10783521084063367, 0.11345219121911036, 0.10694969126982085, 0.11700341502952784, 0.10917789243918118, 0.10968986242534075, 0.10824663903623354, 0.11773786047725, 0.10701042586364483, 0.10915226798271611, 0.10513870775443696, 0.10494833169718018, 0.11285778121920782, 0.10718637312578767, 0.10307604753947384, 0.1056827185373105, 0.10517973469341471, 0.10626769344009857, 0.10898770207858878, 0.10628892055923275, 0.10228935212842222, 0.10553622138951634, 0.10170307490523006, 0.10568007427254025, 0.10211699185964443, 0.10370847759730381, 0.10476118687931174, 0.10078130669833589, 0.10328750230156103, 0.10685210774270716], 'val_metric': [14.62, 22.82, 29.41, 34.94, 37.2, 38.8, 39.72, 41.84, 42.3, 43.92, 44.09, 43.46, 42.91, 44.11, 43.03, 42.36, 43.22, 42.9, 42.94, 42.43, 42.1, 42.51, 41.68, 41.3, 41.91, 42.41, 41.43, 41.54, 42.17, 41.64, 42.2, 41.66, 42.04, 41.17, 41.84, 41.51, 41.73, 41.9, 42.04, 42.42, 41.07, 42.04, 42.67, 42.0, 41.03, 42.02, 41.94, 41.52, 41.16, 42.05, 42.32, 41.94, 41.8, 42.29, 42.34, 42.83, 41.92, 42.36, 42.45, 42.56, 42.16, 42.59, 42.77, 42.51, 42.54, 42.84, 41.91, 42.86, 42.42, 42.7, 41.85, 42.79, 41.99, 42.49, 43.15, 42.47, 43.19, 42.79, 43.19, 43.06, 43.48, 42.36, 42.9, 42.92, 42.83, 43.07, 43.41, 42.07, 42.6, 43.19, 43.0, 42.89, 42.97, 43.13, 42.65, 42.63, 42.87, 43.27, 43.24, 43.51, 43.44, 43.43, 43.14, 43.24, 43.43, 43.35, 43.68, 44.02, 43.88, 43.29, 43.47, 42.67, 42.89, 43.48, 43.42, 43.51, 42.68, 43.85, 43.66, 43.3, 43.03, 43.73, 43.44, 43.68, 43.32, 43.71, 42.55, 43.3, 43.22, 43.44, 43.36, 43.78, 43.2, 42.77, 43.96, 43.44, 43.78, 43.16, 43.64, 42.78, 43.46, 43.28, 43.71, 43.29, 43.96, 44.33, 43.65, 43.55, 43.19, 43.86, 43.4, 44.46, 43.23, 43.54, 43.81, 43.17, 43.71, 43.31, 43.73, 44.22, 44.43, 44.04, 43.49, 43.67, 43.64, 44.08, 43.63, 43.41, 44.03, 43.89, 43.31, 43.65, 44.27, 43.2, 43.68, 43.95, 43.34, 43.97, 43.36, 43.72, 43.65, 43.8, 43.87, 43.78, 43.57, 43.77, 43.23, 44.21, 43.78, 43.92, 44.14, 43.66, 43.92, 43.26, 44.36, 43.82, 43.86, 43.74, 44.17, 44.49], 'val_loss': [3.9186087656932274, 3.513424745790518, 2.9796441512502683, 2.724233981910025, 2.62912406435438, 2.5771703078488635, 2.5373265655936708, 2.4413681307416053, 2.443903533516416, 2.4235406909019326, 2.4346940080831003, 2.543195919626078, 2.6376485520867026, 2.6569630720053508, 2.8160579394383034, 2.8935644003995664, 2.984369391848327, 3.0445858547641973, 3.1770762235495695, 3.273677545747939, 3.501354785861483, 3.5818912557735563, 3.682291018355424, 3.676826844549483, 3.8002431673608768, 3.852518958650577, 3.9835060356528897, 4.079463517589933, 4.0099860422170845, 4.236096034384078, 4.194419049912957, 4.32788399526268, 4.273156658099715, 4.449742203305481, 4.410860143649351, 4.423484563827515, 4.543489680168735, 4.627258361524837, 4.527327004511645, 4.603686312960971, 4.797502148682904, 4.815343673821467, 4.717995801549049, 4.689204257005339, 4.753081039258629, 4.761957939263362, 4.7797181803709385, 4.919740308621886, 5.139098917602733, 4.953263423245424, 4.937705941261, 4.9876703836356, 4.958426574992526, 4.938723570222308, 5.023690261658589, 5.02920218561865, 4.997494784130413, 5.151510217387205, 5.228292992919873, 5.209286124083646, 5.144963809639026, 5.198052723696278, 5.341754797157968, 5.329247294717534, 5.2921376304262004, 5.228594978144215, 5.3415207498392485, 5.212881953852951, 5.283155992532232, 5.390058693612457, 5.382194292773107, 5.312043440569738, 5.50394470980213, 5.495391535151536, 5.361056618629747, 5.532354620611592, 5.521879691986522, 5.537599490706328, 5.648473454129165, 5.551503583883783, 5.504562181272324, 5.830979477827716, 5.691421186088756, 5.60013658377775, 5.64096385658167, 5.5465287554795575, 5.618061660201686, 5.654262764438702, 5.64744585334875, 5.545516870583699, 5.558599787912551, 5.715796231464216, 5.819648757861678, 5.782192328932938, 5.680710570068117, 5.883551596076625, 5.816065482273223, 5.865293590885819, 5.7989848738263365, 5.829586508927072, 5.744146233910968, 5.78111406800094, 5.905400435635998, 5.855140280571713, 5.659903577178906, 5.90849584227155, 5.70231505564064, 5.887303449545696, 5.8843065674897215, 5.951946922927905, 5.9044822651869175, 6.016934662867504, 6.0762706691292445, 5.959890793842875, 5.8518420822301485, 5.82567563178433, 5.871414450323506, 5.9209378798296495, 5.962476200358883, 6.120877754916052, 6.010848686193964, 5.983598734163175, 6.070173609788251, 5.954495012380515, 6.077900194058753, 5.937066066037318, 6.059047780978452, 5.962009253775238, 6.1587115351561526, 5.994645637311753, 6.1620252823374075, 6.1147498583338065, 6.011958315873602, 6.121909774033127, 6.348380764578558, 6.037921278340042, 6.263562946562555, 6.06160688779916, 5.902350551763158, 6.112166846633717, 6.058405794155825, 6.07802030539057, 6.010367132296228, 6.271649550480448, 6.1921957495865545, 6.326008622813377, 6.344966810979661, 6.1938472371192494, 6.211562123268274, 6.161034729830019, 6.2299745386573155, 6.2646626317577, 6.186012754015103, 6.176384099729502, 6.303914098223304, 6.382039566708219, 6.3747344138515984, 6.37070158788353, 6.3149748987453, 6.365107862053404, 6.298024633887467, 6.340683028956128, 6.238733387297126, 6.360627465946659, 6.222529752998595, 6.321337321002012, 6.462425439221085, 6.217124831144977, 6.459883358827822, 6.224202403597012, 6.396961910709454, 6.276295339985258, 6.4662484257084545, 6.355184401676154, 6.525802584970073, 6.369187927549811, 6.371730992748479, 6.553697007476904, 6.431719843749028, 6.611282752577666, 6.318000667414088, 6.419539904138844, 6.511608333344672, 6.226924988874204, 6.612165414603653, 6.496335293836655, 6.551179840306568, 6.431514099145391, 6.189641402785186, 6.478301471965328, 6.630596516238656, 6.475468894478622, 6.531790379506008, 6.529053464816634, 6.559392391496403, 6.573034272831716, 6.52897797116808, 6.561337290296129, 6.454311486262425, 6.320803660496026]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="SGD")
data.update({'train_metric': [1.137, 1.61, 1.806, 1.948, 2.086, 2.21, 2.365, 2.564, 2.696, 2.882, 3.16, 3.398, 3.65, 3.936, 4.166, 4.404, 4.58, 4.772, 4.996, 5.111, 5.284, 5.596, 5.717, 5.977, 6.069, 6.336, 6.473, 6.698, 6.895, 7.062, 7.227, 7.551, 7.822, 8.006, 8.212, 8.461, 8.689, 8.885, 9.263, 9.305, 9.509, 9.703, 9.854, 10.122, 10.184, 10.494, 10.661, 10.836, 11.095, 11.177, 11.479, 11.5, 11.785, 12.049, 12.17, 12.3, 12.459, 12.69, 12.805, 12.99, 13.07, 13.308, 13.476, 13.61, 13.617, 13.923, 14.026, 14.127, 14.251, 14.443, 14.52, 14.546, 14.786, 14.998, 15.221, 15.165, 15.468, 15.494, 15.692, 15.767, 15.853, 16.16, 16.143, 16.384, 16.431, 16.556, 16.744, 16.822, 16.918, 17.154, 17.261, 17.439, 17.5, 17.612, 17.693, 17.93, 18.059, 18.316, 18.261, 18.455, 18.588, 18.642, 18.875, 18.906, 19.017, 19.282, 19.38, 19.483, 19.543, 19.676, 19.672, 19.933, 20.064, 20.156, 20.318, 20.421, 20.51, 20.672, 20.886, 20.892, 21.025, 21.261, 21.266, 21.448, 21.666, 21.728, 21.794, 22.078, 22.018, 22.156, 22.209, 22.331, 22.484, 22.578, 22.805, 22.866, 22.983, 23.196, 23.18, 23.284, 23.44, 23.575, 23.611, 23.74, 23.77, 23.934, 24.032, 24.07, 24.303, 24.366, 24.538, 24.682, 24.72, 24.9, 24.911, 25.212, 25.197, 25.277, 25.392, 25.543, 25.544, 25.786, 25.772, 25.966, 25.929, 26.179, 26.109, 26.336, 26.362, 26.596, 26.613, 26.819, 26.717, 26.961, 27.013, 27.133, 27.303, 27.416, 27.451, 27.591, 27.667, 27.796, 27.972, 28.008, 28.027, 28.281, 28.279, 28.394, 28.452, 28.449, 28.635, 28.91, 28.935, 28.929, 29.151, 29.129, 29.252, 29.312, 29.546, 29.616], 'train_loss': [5.26962593405657, 5.251573400930647, 5.237072612105923, 5.221015725468338, 5.205972271856404, 5.189114962948184, 5.1724788067208145, 5.1530743981506, 5.131846192702222, 5.112482625249862, 5.0917231800155, 5.066156379702147, 5.039484634204164, 5.007837557594363, 4.973365816334769, 4.936300770106105, 4.901989147896502, 4.866255140274058, 4.831144330445117, 4.80088789067967, 4.773698523650166, 4.7464758426961575, 4.72032731538847, 4.696726454577992, 4.672058890129768, 4.652212298534196, 4.628597078305059, 4.606018834593048, 4.583527508562982, 4.560868208940717, 4.540522522599897, 4.515508106253655, 4.495540939228548, 4.4733521212237966, 4.452357178228602, 4.429911434841095, 4.405546246014263, 4.386015776251648, 4.363999025225258, 4.34549453620032, 4.326197828761447, 4.306046263193825, 4.2887586603085355, 4.269376139723179, 4.252778112773932, 4.233822347716651, 4.220928863615693, 4.2038989700114815, 4.186717648423794, 4.1732991228329395, 4.156740395594162, 4.145872279305681, 4.129250484479976, 4.113259465741715, 4.101758600577893, 4.088179827194067, 4.077307227019385, 4.06169077744487, 4.050729552370878, 4.038149213455307, 4.028300190765127, 4.014459209265193, 4.0049825703113875, 3.9932392949258677, 3.982618243283975, 3.9691617781736115, 3.959224563642564, 3.9479990556159237, 3.940478915673986, 3.9277106891895666, 3.919091200416697, 3.909863795062631, 3.8969320684995545, 3.8888772255475903, 3.8801041514310635, 3.8690638772509303, 3.8593223898821125, 3.8497292493599313, 3.8406910032930086, 3.832065616291605, 3.822445102479316, 3.812298492979561, 3.804907332302589, 3.794074052583691, 3.787480904822615, 3.778308211880011, 3.7702071207574903, 3.7598592057414186, 3.752985292265069, 3.7449068850763165, 3.735186815719458, 3.726593679597724, 3.7184503627601373, 3.7093646504981197, 3.7011163222507566, 3.6919802432630737, 3.684058484402667, 3.6754944127718, 3.6719113236730556, 3.660714479417124, 3.6512740671215194, 3.6441884437434124, 3.635624355790864, 3.626533053124172, 3.6215073696253626, 3.6127017700588255, 3.603897446939294, 3.593607046019932, 3.5890745779526823, 3.5823902449238703, 3.5740187575400677, 3.564680791252024, 3.5558495925773967, 3.549458216949678, 3.5429671698476897, 3.5340653075976625, 3.5249060060607267, 3.521238580088698, 3.511021036256679, 3.504843658006733, 3.4979246396402175, 3.4903282599043406, 3.482459442934316, 3.473507535663539, 3.4653644166691366, 3.459630780393926, 3.4534812786605062, 3.444313714234248, 3.4393675894746396, 3.433126361417374, 3.4257580269931296, 3.418031230769093, 3.41280480218254, 3.4044266360277406, 3.398570817781425, 3.387917153589709, 3.3861121380855392, 3.3724532051110834, 3.3680903516514364, 3.3621272665525352, 3.3541781830619866, 3.347277666617874, 3.3411048985564853, 3.338007923127441, 3.329196807747839, 3.3217978631542495, 3.315682781101112, 3.309382647638205, 3.300025919958787, 3.297804211471909, 3.2903339037587074, 3.2818392522046755, 3.27650806786384, 3.2700626993514907, 3.263672761404583, 3.256968380012195, 3.254172703126113, 3.24394756513609, 3.2378411233539546, 3.2331197547058377, 3.226794682629049, 3.220519502271236, 3.214723306059151, 3.206520103447268, 3.2044628427643387, 3.194117259460615, 3.19371383295407, 3.185743859846929, 3.179458880836355, 3.171263242484817, 3.1658256125617927, 3.1573988334231093, 3.156767476619396, 3.151025048601879, 3.1414343872973345, 3.1381158772295894, 3.130458545928877, 3.127245409246141, 3.1191813695606174, 3.1139854474168365, 3.104073421663759, 3.102099433436427, 3.094522763274834, 3.089727555492789, 3.085367436448657, 3.0773072430359885, 3.07198026206199, 3.065500704355905, 3.0643426911310745, 3.0572787913007, 3.050574890864025, 3.043230050356054, 3.035665241175558, 3.031823608864597, 3.0289363450753863, 3.0208028512968137, 3.017184887493717, 3.009235530042984, 3.006699169284628, 3.0004638916242605], 'val_metric': [1.5, 1.69, 1.75, 1.81, 2.15, 2.17, 2.35, 2.55, 2.77, 2.98, 3.11, 3.43, 3.81, 4.02, 4.39, 4.65, 4.86, 4.99, 5.11, 5.21, 5.34, 5.54, 5.53, 5.76, 6.04, 6.27, 6.3, 6.51, 6.79, 7.01, 7.22, 7.36, 7.79, 8.09, 8.43, 8.49, 8.8, 8.88, 9.12, 9.32, 9.46, 9.68, 9.94, 10.03, 10.43, 10.42, 10.59, 10.87, 11.08, 11.25, 11.49, 11.67, 11.64, 12.06, 12.08, 12.15, 12.42, 12.65, 12.82, 12.85, 12.94, 13.0, 13.42, 13.35, 13.59, 13.86, 13.98, 14.1, 14.27, 14.35, 14.4, 14.63, 14.73, 14.88, 14.92, 14.92, 15.03, 15.32, 15.22, 15.58, 15.56, 15.8, 15.87, 15.99, 15.88, 16.24, 16.46, 16.38, 16.62, 16.95, 16.98, 16.93, 17.21, 17.11, 17.24, 17.53, 17.59, 17.73, 17.94, 18.08, 18.18, 18.4, 18.42, 18.39, 18.82, 18.89, 19.09, 19.04, 19.22, 19.01, 19.45, 19.68, 19.48, 19.75, 19.68, 20.06, 19.91, 19.91, 20.01, 19.95, 20.27, 20.36, 20.28, 20.23, 20.78, 20.83, 20.78, 21.12, 21.26, 21.35, 21.12, 21.64, 21.7, 21.71, 21.91, 21.75, 21.91, 21.83, 21.98, 22.2, 22.1, 22.22, 22.33, 22.34, 22.55, 22.68, 22.71, 22.62, 22.96, 23.11, 23.3, 23.25, 23.28, 23.31, 23.47, 23.5, 23.64, 23.89, 23.99, 24.06, 23.92, 24.04, 24.33, 24.4, 24.35, 24.83, 24.8, 24.81, 24.92, 24.91, 25.0, 24.9, 25.24, 25.24, 25.36, 25.21, 25.35, 25.39, 25.45, 25.37, 25.66, 25.88, 25.97, 25.74, 25.9, 26.1, 26.15, 26.09, 26.18, 26.28, 26.57, 26.65, 26.96, 26.63, 26.65, 26.8, 26.91, 26.93, 26.88, 27.03], 'val_loss': [5.266246464601748, 5.256924721845396, 5.2418682651155315, 5.236653376536764, 5.217892242844697, 5.2019770221345745, 5.184903743160758, 5.163302497499308, 5.140153117999909, 5.121000566300313, 5.095616899478208, 5.063484734031046, 5.034630617518334, 4.998538720379969, 4.96023042034951, 4.9212428369339865, 4.885020933333476, 4.849822606250739, 4.820454469911612, 4.787022798683993, 4.758361591654978, 4.732444430612455, 4.707162834276819, 4.685256670994364, 4.6601668011610675, 4.6374947721031825, 4.614693473099144, 4.593615739968173, 4.569475611304022, 4.548591165785577, 4.525259431000728, 4.502548967956737, 4.476768776110023, 4.453109114033402, 4.4303148187649475, 4.405282930204064, 4.383259571281967, 4.361913996896926, 4.338811430961463, 4.32076138599663, 4.303541916950493, 4.28352094911466, 4.265443268854907, 4.248110862294579, 4.232084874134914, 4.21534739330316, 4.199603103528357, 4.181584756085827, 4.169508741160107, 4.155518635063414, 4.140183556611371, 4.123498599240734, 4.111299286982057, 4.098344859044263, 4.087945107441799, 4.074778716275646, 4.06384394275155, 4.053331525462448, 4.041043907214123, 4.032303211795297, 4.019445745808304, 4.0113848182046485, 3.998444522262379, 3.992029543894871, 3.978196938326404, 3.969001434411213, 3.960101842880249, 3.9495240001921443, 3.942933061320311, 3.931061635351485, 3.923209992184001, 3.9138046070268957, 3.905320976949801, 3.8951057142512813, 3.8868297528309426, 3.879250708658984, 3.871642621459475, 3.8628226966614934, 3.855161082213092, 3.845412343930287, 3.836149952214235, 3.829848805810236, 3.8240215990953383, 3.8179149749172723, 3.8093653454142773, 3.7985203569861734, 3.7925240401249782, 3.783422433646621, 3.7757211748961432, 3.7691266764501097, 3.760035303747578, 3.752037262460988, 3.746573382881796, 3.7386988484935397, 3.733310476230208, 3.7255342963394846, 3.7180305225833967, 3.7113040070624868, 3.6996810284389814, 3.6956856463365493, 3.689162993886668, 3.683970214454991, 3.6754170101918993, 3.671606290112635, 3.6621806181160506, 3.65313950313884, 3.649430025914672, 3.641815091394315, 3.6339332768871526, 3.6266960308050655, 3.6175398522881186, 3.612917822637376, 3.61158196333867, 3.601026276873935, 3.595719375428121, 3.5851795141863976, 3.5791384065227145, 3.575451814444961, 3.568361688571371, 3.564149996277633, 3.5565228932982036, 3.5511628404544417, 3.542252640056003, 3.5361516460491593, 3.5314945696265836, 3.526911364239492, 3.519281568041273, 3.5120285506461077, 3.505007032376186, 3.502922257040716, 3.494302843786349, 3.4934169418492895, 3.481883263132375, 3.4755637258481067, 3.47320793312826, 3.468566037287378, 3.4649342085905137, 3.4599137693453748, 3.4513625134328367, 3.4464644603668506, 3.441393583443514, 3.4373708211692278, 3.4288246198824255, 3.428145363831976, 3.418043918670363, 3.415807436985575, 3.4096820749294987, 3.4019598361033543, 3.4029844156496085, 3.39231480230951, 3.3844781430663575, 3.380316372130327, 3.377916898697045, 3.3733310297036625, 3.367483091961806, 3.362832840081233, 3.358128401883848, 3.3510593942775846, 3.347259565523476, 3.342712774398221, 3.334794095367383, 3.3325119831000163, 3.333220947320294, 3.3258286805669215, 3.317990090436996, 3.312324182243104, 3.3076888816371843, 3.3018708768164275, 3.300659090090709, 3.2959861937601853, 3.2915139190710274, 3.284159699822687, 3.2821139985588705, 3.2820702031919153, 3.274202445510087, 3.275136329565838, 3.265280710663765, 3.2609351372263236, 3.2543836862418303, 3.2508432341229385, 3.2473439988057327, 3.243887358410343, 3.2387972121026105, 3.235590555105999, 3.2328396033329567, 3.224061898365142, 3.220155771370906, 3.2232913507777416, 3.2135501864609446, 3.2101028664096907, 3.2094249512739244, 3.205388157990328, 3.1978652932841305, 3.1923578825725873, 3.1936903022656775, 3.1875904555533343, 3.1886504227948036, 3.180864807906424, 3.1755333561806163, 3.1745338629765114]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="Adam")
data.update({'train_metric': [8.296, 15.876, 19.505, 21.898, 23.592, 25.332, 26.836, 27.957, 28.944, 30.184, 30.594, 31.268, 32.089, 32.852, 33.23, 33.945, 34.297, 34.916, 34.989, 35.287, 35.828, 36.113, 36.722, 36.91, 37.25, 37.403, 37.819, 37.764, 38.137, 38.457, 38.413, 38.572, 39.099, 39.106, 39.181, 39.249, 39.527, 39.602, 39.578, 39.826, 39.797, 39.812, 40.114, 40.361, 40.234, 40.159, 40.512, 40.519, 40.629, 40.571, 40.864, 40.893, 40.881, 41.026, 41.328, 41.108, 41.266, 41.283, 41.423, 41.429, 41.422, 41.483, 41.508, 41.739, 41.6, 41.54, 41.698, 41.89, 41.924, 41.893, 42.118, 42.027, 42.175, 42.142, 42.138, 42.271, 42.463, 42.295, 42.51, 42.438, 42.334, 42.758, 42.573, 42.615, 42.698, 42.65, 42.824, 42.55, 42.761, 42.931, 42.749, 42.696, 42.911, 43.089, 42.848, 43.014, 43.037, 42.992, 43.2, 42.923, 43.163, 43.23, 43.283, 43.287, 43.263, 43.357, 43.168, 43.32, 43.375, 43.15, 43.393, 43.538, 43.455, 43.543, 43.386, 43.501, 43.419, 43.524, 43.679, 43.423, 43.458, 43.733, 43.722, 43.513, 43.534, 43.632, 43.568, 43.706, 43.716, 43.482, 43.724, 43.791, 43.762, 43.637, 43.855, 43.739, 44.068, 43.969, 43.96, 43.877, 43.903, 44.038, 43.766, 43.779, 44.03, 44.123, 43.991, 44.087, 44.02, 44.011, 43.799, 44.002, 44.068, 43.969, 44.117, 44.134, 44.14, 43.986, 44.213, 44.193, 44.138, 44.281, 44.177, 44.183, 44.135, 44.118, 44.223, 44.362, 44.175, 44.167, 44.213, 44.222, 44.318, 44.217, 44.091, 44.386, 44.307, 44.407, 44.334, 44.371, 44.415, 44.468, 44.18, 44.41, 44.373, 44.402, 44.245, 44.271, 44.273, 44.38, 44.419, 44.437, 44.412, 44.554, 44.528, 44.205, 44.285, 44.296, 44.421, 44.558], 'train_loss': [4.3968476747291945, 3.7697458486868185, 3.526282853982575, 3.374921022282147, 3.2671619010749566, 3.1776255626214747, 3.1010417979417975, 3.0330916402893653, 2.9756810799365003, 2.927243262014554, 2.8894128019963787, 2.8524883161045693, 2.816769435782503, 2.7881480450059692, 2.76297600766595, 2.734526931286163, 2.7100376272842204, 2.6887498897996682, 2.6684502785738204, 2.6557056084704262, 2.633765731571732, 2.6197342047383216, 2.601565390005374, 2.587229569066585, 2.575412739261327, 2.5623851226677288, 2.5484906580687636, 2.5449428379116195, 2.5316959251902915, 2.520854251322194, 2.514218672833531, 2.5058703336361807, 2.4914518322459567, 2.4876016321200556, 2.480795222143294, 2.47533822448606, 2.4697575826943874, 2.464265535446748, 2.4563582811459317, 2.4526509128162974, 2.446158768271912, 2.4408394872417682, 2.4348232890273698, 2.4267285300307906, 2.427497451837751, 2.423014869616723, 2.4163648700073446, 2.4130954965138693, 2.406840706817324, 2.407633462168815, 2.3985269730928533, 2.3983708382873146, 2.3971503654200528, 2.392238054760587, 2.386564463968088, 2.3871936151330013, 2.378138575047464, 2.3749252904018223, 2.3712639981023944, 2.3739443462168035, 2.367865069661473, 2.369837820034186, 2.365550189009097, 2.3606277451786717, 2.361767014554122, 2.356148376574672, 2.354176440455558, 2.3462616775711607, 2.348807613436259, 2.3458070615431628, 2.3410734725318805, 2.3428551047868784, 2.3416077854232156, 2.3396962285423157, 2.3410107822503634, 2.3305280115538807, 2.3278079007729002, 2.330381575488007, 2.3279113457588836, 2.3260467919644374, 2.326532338188767, 2.321134298715695, 2.321036896183944, 2.322234758793812, 2.3152142458822356, 2.313591878839738, 2.3109050015951684, 2.3139392870477735, 2.311384609854534, 2.3031851871610067, 2.305103522954808, 2.3062529468566884, 2.3019568913271238, 2.3008413173720688, 2.30418632736743, 2.300040079627522, 2.3005356828295414, 2.2971692710676335, 2.295734678028641, 2.2952230570793763, 2.2905391011570635, 2.289298689373014, 2.2898335163210417, 2.2897085695791457, 2.2859763872600563, 2.2915993188332076, 2.287555646072651, 2.2854564776423643, 2.287253774669181, 2.2849530641344673, 2.2850157414501626, 2.282181630055262, 2.2816002825781543, 2.2809105267985386, 2.280424908194417, 2.2767141133794704, 2.2764461871834802, 2.2755722615174276, 2.2750931574759847, 2.2768816188292402, 2.278510411961751, 2.27106199192833, 2.2722995201899163, 2.2735574881540837, 2.2711659907074364, 2.2736830527555156, 2.271032001830337, 2.2700933614603924, 2.267769044054218, 2.2640861452808, 2.2684377871944745, 2.26471800904814, 2.267784567193503, 2.263499570136946, 2.261921137781076, 2.2598948049301226, 2.2558510748911424, 2.26052220319222, 2.259968449652996, 2.2594213876217815, 2.2591964097947397, 2.254083463418049, 2.257902297009586, 2.257778522377966, 2.2503862159037085, 2.2554674620668016, 2.2580532084957423, 2.252239965736599, 2.2514849852386836, 2.256004189194126, 2.25561825243693, 2.252304784472143, 2.2466103833071025, 2.2468825481064565, 2.2521176038456514, 2.2491020421835373, 2.2492104405710047, 2.254618706721491, 2.247964169913504, 2.248399753030568, 2.246620844742158, 2.2431604506797083, 2.24557969254404, 2.244301608946563, 2.2438418165812184, 2.2456953074790236, 2.2450612278527657, 2.241579851658773, 2.2416198393204847, 2.2440126512726377, 2.2472443846045436, 2.2435124400176076, 2.2407233405403044, 2.239806790345766, 2.243618981851955, 2.238717597871733, 2.2407477031666274, 2.241319370971455, 2.2407159130312433, 2.2413075146428, 2.238921630481688, 2.2362788260478816, 2.241375512063923, 2.239428547964749, 2.2381794798168246, 2.2355961133025812, 2.240565173456628, 2.2377046175820663, 2.2339464824167643, 2.235079374209628, 2.23744458665622, 2.2351365736182194, 2.2344296137949478, 2.2364963211467153, 2.229032653230776, 2.235150033819012, 2.2333367375784476, 2.2329608778578307, 2.2289133717139715, 2.2282723485088773], 'val_metric': [5.39, 13.08, 18.17, 21.82, 20.76, 22.52, 24.29, 27.13, 26.53, 25.5, 28.15, 28.68, 30.32, 29.32, 32.2, 29.67, 32.24, 30.07, 30.8, 31.65, 32.74, 31.23, 32.33, 30.18, 35.35, 32.36, 33.64, 34.97, 33.51, 35.22, 33.8, 34.39, 34.37, 36.12, 35.92, 32.82, 34.37, 36.22, 35.52, 34.77, 34.56, 36.15, 35.47, 34.6, 32.88, 35.94, 35.16, 37.05, 36.26, 37.88, 36.01, 36.56, 36.67, 35.48, 36.53, 36.74, 35.19, 36.46, 34.41, 37.4, 35.02, 36.4, 37.74, 35.92, 35.57, 36.46, 38.02, 36.94, 37.11, 36.6, 35.72, 37.78, 37.36, 35.86, 37.3, 36.96, 37.48, 36.01, 37.21, 34.79, 36.81, 38.11, 38.27, 36.27, 37.63, 37.71, 37.38, 36.73, 37.58, 39.4, 37.99, 36.46, 37.26, 36.44, 35.97, 37.95, 37.23, 41.39, 38.42, 37.89, 35.57, 39.16, 38.14, 39.14, 38.03, 39.49, 38.2, 35.33, 37.82, 38.35, 37.88, 35.89, 36.59, 35.61, 37.34, 33.52, 34.91, 39.39, 36.4, 38.44, 38.45, 39.2, 38.16, 35.67, 35.36, 38.27, 39.72, 36.63, 37.25, 39.42, 39.0, 38.84, 38.44, 38.55, 37.94, 37.49, 39.18, 39.4, 39.76, 35.06, 35.16, 38.4, 40.0, 39.76, 38.29, 38.85, 39.67, 37.19, 36.03, 39.71, 39.19, 39.13, 36.72, 38.79, 35.39, 39.74, 37.94, 38.18, 39.07, 37.28, 38.48, 39.38, 39.83, 36.79, 37.52, 38.34, 37.51, 36.99, 39.41, 38.04, 39.51, 36.14, 37.84, 38.43, 39.87, 36.16, 40.71, 37.4, 38.39, 36.44, 38.78, 38.23, 37.35, 35.25, 40.06, 35.88, 39.03, 38.45, 34.36, 37.96, 38.94, 39.81, 37.43, 37.31, 36.91, 38.42, 35.15, 38.98, 37.27, 38.95], 'val_loss': [4.560833341756444, 3.93192214570987, 3.5686476587489913, 3.366836695914056, 3.4362648383826966, 3.3149309955584774, 3.2441076321207034, 3.0907867410380367, 3.1322357419190134, 3.1846233401328896, 3.002827440857128, 3.0199801109399007, 2.9164126967168915, 2.965548783350902, 2.849307373830467, 2.9454089357594775, 2.816874011307006, 2.9201523452807385, 2.912767402685372, 2.88074486954197, 2.8422557639468247, 2.9245001829353865, 2.8483388955426063, 2.996740559863437, 2.6632818105114495, 2.8622117885358773, 2.770139075008927, 2.7201893071460117, 2.7637380801948015, 2.6746685755480626, 2.7794379671667793, 2.7357591838593693, 2.7416527590174584, 2.648254202429656, 2.6377274018184393, 2.7472361212323424, 2.708198004467472, 2.6609863460443584, 2.6977990918858037, 2.7256665131089033, 2.740834585420645, 2.676571962180411, 2.6831909046051607, 2.715973245110481, 2.8227617896286543, 2.709483204373888, 2.7039607049553256, 2.6146973751153157, 2.6660845351826614, 2.569376917022049, 2.6787066436876916, 2.611609047385538, 2.610024905508491, 2.708995864649487, 2.650817550291681, 2.6372563535240805, 2.707721022283955, 2.6243525090490936, 2.7703706102006755, 2.5989125955636334, 2.7408895234393467, 2.693710495332244, 2.561958102663611, 2.6578776228959393, 2.6958712939244167, 2.677285904337646, 2.578603061141482, 2.6301178317161122, 2.6232148158322475, 2.6602163747617396, 2.687769073589592, 2.5917150207385897, 2.6235161537577394, 2.727523346615445, 2.631831361989307, 2.6412332133882366, 2.6073916224157734, 2.674728700309802, 2.647006508651053, 2.7637234709824727, 2.6233465853770066, 2.5828508654977105, 2.588662159291043, 2.663830125407808, 2.6162138190239097, 2.599344751257805, 2.63353208827365, 2.6549236098672173, 2.6168100864264616, 2.525117199891692, 2.5871061132212354, 2.658523463139868, 2.6128902925047903, 2.671902783357414, 2.7128849811614697, 2.5836147988677785, 2.6285303705816814, 2.4174436501636625, 2.536380120143769, 2.585389471357795, 2.7194413690809993, 2.5590783220947166, 2.5768142915835046, 2.506630191757421, 2.5624173879623413, 2.5293135301322693, 2.5942303526933026, 2.69743404646588, 2.5904952272488053, 2.5915489128440807, 2.5874905981075993, 2.738754200707575, 2.627983852556557, 2.71906259940688, 2.5907727692537246, 2.8747197921109047, 2.767122674140201, 2.520976231356335, 2.6647685523245745, 2.5653920450787635, 2.5672833175416203, 2.495469171530122, 2.5982420209107127, 2.722597224697186, 2.737819842851845, 2.5310511778874, 2.4965689034219, 2.657567648750961, 2.6590309063340447, 2.525048640882893, 2.5302719272625676, 2.558305085844295, 2.5265682123269246, 2.555108599602037, 2.6166379387211647, 2.610041912953565, 2.5279520750045776, 2.508177243979873, 2.4909937617125784, 2.765974895210023, 2.7177087679790084, 2.5742265126507755, 2.4653185962871382, 2.4905822975620344, 2.547215172439624, 2.5483158917943385, 2.5203134277064327, 2.6448771224659717, 2.649848227288313, 2.5080827467001168, 2.5280450024422567, 2.5141907034406237, 2.634466563060785, 2.5513285189677197, 2.7282686537238443, 2.501994726004874, 2.5615345042222626, 2.579869987858329, 2.5322980888330253, 2.62652405052428, 2.5750612005306657, 2.5277284467296237, 2.4935986236402186, 2.6314893824279686, 2.635258733466932, 2.5659346838665615, 2.630073444858478, 2.632687890605562, 2.5061583799921023, 2.59734608489237, 2.511058341545664, 2.687181806868049, 2.6000723876770895, 2.5359405339903134, 2.5036681040077453, 2.7016971870592443, 2.4526967288582187, 2.6073237972654355, 2.562898430475004, 2.5934455702259283, 2.524212204726638, 2.654609291037177, 2.607799847414539, 2.742449149963962, 2.4669686297702182, 2.706389294688109, 2.5687664807981747, 2.556999579356734, 2.763663376972174, 2.5720026979021204, 2.5465662016230786, 2.4921621217089855, 2.624933245835031, 2.6298760802123198, 2.636078237727949, 2.5704109304270166, 2.7340126914583194, 2.5227611334460556, 2.592695667485523, 2.534307005299125]})
data = database.get_data(model="MobileNetV3", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="AdamW")
data.update({'train_metric': [9.678, 22.389, 30.491, 36.2, 41.045, 45.123, 49.07, 52.546, 56.142, 59.746, 63.051, 66.037, 69.075, 71.566, 73.85, 76.134, 77.858, 79.801, 81.208, 82.34, 83.45, 84.372, 85.009, 85.787, 86.395, 87.135, 87.596, 87.983, 88.416, 88.88, 89.123, 89.323, 89.761, 90.123, 90.255, 90.543, 90.844, 90.975, 91.116, 91.49, 91.382, 91.521, 91.608, 91.867, 92.24, 92.243, 92.544, 92.323, 92.639, 92.501, 92.931, 92.875, 92.925, 93.224, 93.105, 93.33, 93.273, 93.44, 93.48, 93.502, 93.633, 93.765, 93.688, 93.956, 93.962, 94.048, 94.06, 94.105, 93.998, 94.251, 94.295, 94.406, 94.371, 94.44, 94.588, 94.595, 94.488, 94.774, 94.763, 94.634, 94.8, 94.656, 94.75, 94.799, 94.882, 95.073, 94.998, 95.169, 94.946, 95.045, 95.025, 95.195, 95.2, 95.232, 95.337, 95.284, 95.385, 95.323, 95.292, 95.342, 95.381, 95.46, 95.46, 95.431, 95.338, 95.577, 95.613, 95.737, 95.639, 95.693, 95.671, 95.713, 95.777, 95.569, 95.794, 95.825, 95.855, 95.873, 95.75, 95.732, 95.867, 96.047, 95.975, 96.023, 95.906, 95.824, 96.115, 95.983, 95.948, 95.934, 96.072, 96.219, 95.937, 96.168, 95.985, 96.072, 96.225, 96.32, 96.092, 96.311, 96.195, 96.238, 96.43, 96.138, 96.388, 96.33, 96.289, 96.291, 96.32, 96.356, 96.304, 96.316, 96.46, 96.437, 96.329, 96.5, 96.331, 96.478, 96.475, 96.429, 96.426, 96.427, 96.506, 96.475, 96.53, 96.595, 96.562, 96.473, 96.634, 96.613, 96.512, 96.507, 96.73, 96.613, 96.682, 96.642, 96.463, 96.772, 96.698, 96.673, 96.715, 96.611, 96.797, 96.72, 96.749, 96.739, 96.783, 96.753, 96.855, 96.688, 96.723, 96.719, 96.703, 96.758, 96.812, 96.751, 96.812, 96.803, 96.782, 96.92], 'train_loss': [4.326882811593308, 3.41402133885516, 2.9687363932091535, 2.6655769605935573, 2.4227681468864777, 2.2154858784269846, 2.024161513234588, 1.8418888376068778, 1.6747070065081615, 1.5106908986756074, 1.3592073464729202, 1.2235139486351916, 1.0940554909269853, 0.9853197186708603, 0.8875931880638833, 0.8001611204633176, 0.729407171389268, 0.6650418417048012, 0.6114725434791561, 0.5676916397998368, 0.5304291437660664, 0.49288038680626656, 0.47393776661813525, 0.4449635020201586, 0.4252973730775384, 0.40476634104093223, 0.38853616274116287, 0.37761704836777216, 0.3633217811608307, 0.3506638595377949, 0.3448702129877994, 0.33418252466393067, 0.3237327943464845, 0.3132646836349961, 0.3096007287778766, 0.2996591924119476, 0.2867653840558123, 0.28605452967630085, 0.2797408431592044, 0.26933986448604785, 0.271998459288537, 0.2671861749372409, 0.26509887524311426, 0.25860290453362295, 0.2457068985171964, 0.2458534689104126, 0.2358104334876027, 0.2430630773198639, 0.23162013363219458, 0.23796455979213757, 0.22443179101209493, 0.22548169398266804, 0.22635160809262433, 0.21571105226674145, 0.22067673821169084, 0.210264121082755, 0.21373883259476129, 0.20760493319283788, 0.20780598491868832, 0.20261555618021973, 0.20185979603243348, 0.20006012894160766, 0.20223684501920614, 0.19408926276176672, 0.19167438770453096, 0.19056623321686772, 0.19130621572464265, 0.18902889137100892, 0.19048779654022846, 0.18390310241322982, 0.18234085689834484, 0.17905141311737585, 0.17995572015249856, 0.17937263430668826, 0.17384076407489082, 0.17241230373070007, 0.17531309079322832, 0.16714003290063437, 0.1670931564132735, 0.17386883810360368, 0.1667420399820364, 0.17011328788697053, 0.16788462943338472, 0.16681649451938316, 0.16368692271948881, 0.16128336555656386, 0.16071663131569144, 0.1570325032827252, 0.16257320827210436, 0.1584893654670235, 0.15972731510440624, 0.15386617624320648, 0.15522790868943576, 0.15436412768065452, 0.14998118581830674, 0.1508767223101817, 0.1486721325848044, 0.15133520521483337, 0.15234192611735436, 0.1486719593176915, 0.14828198697085487, 0.14620380624127707, 0.1476476307622004, 0.1485031772950415, 0.15250260251415124, 0.14266073315901, 0.14208494934643084, 0.13876466856350597, 0.1414928890215632, 0.1403590291342266, 0.13948780219028903, 0.1396591451116769, 0.14109042304599573, 0.14180412831801448, 0.13859096963062253, 0.13580785361723408, 0.13462820525060068, 0.13599890559987401, 0.13532615473174556, 0.13854566638811383, 0.13562761654261613, 0.13005182248932295, 0.1360714628309922, 0.12871958513464718, 0.13597010207119364, 0.13797124525977827, 0.12841273541428191, 0.12980576168885186, 0.13193530501676837, 0.13178932541648852, 0.1277348888779961, 0.12254497602654077, 0.1326877613443469, 0.125345503231728, 0.13257167888118546, 0.1303083972447693, 0.12681769245493055, 0.12268265522390805, 0.12890356473498982, 0.12336081424230497, 0.12671162890090404, 0.12682030151147689, 0.11665658468409768, 0.12926635797627412, 0.11979436243058111, 0.11963255506347607, 0.12143889786200378, 0.1247057091439964, 0.11899554448672085, 0.12153448555620014, 0.12331041024347751, 0.12003602679048427, 0.1189423944913262, 0.11762556038915775, 0.12490904229323387, 0.11746659492428807, 0.1234938976387967, 0.1191691823157119, 0.11912423205354578, 0.11933765513450026, 0.11921779828431217, 0.11834398493521102, 0.11635154310841189, 0.11751231666781266, 0.11620950196709982, 0.11217231038275281, 0.11582257140522272, 0.11564026028156205, 0.1118914288460019, 0.11183909670123861, 0.11677598352773415, 0.1175772577934078, 0.10895394891806892, 0.11146137799730409, 0.11089538541580451, 0.1129449865495058, 0.1180709080133195, 0.10759035821937642, 0.1110995319702541, 0.11231629303720574, 0.11035202695229594, 0.11264969702417976, 0.10748595212514274, 0.10641207312260113, 0.11254577393834529, 0.11137799543478441, 0.10674609853467479, 0.11125861320815032, 0.10502925381740413, 0.11189519645487121, 0.10886975280222973, 0.10839117337075603, 0.10808478347069012, 0.1090059799813041, 0.1076509651357948, 0.10927124593606677, 0.1051100972083219, 0.1071614715663516, 0.10572827438732564, 0.10703953815558871], 'val_metric': [14.34, 26.33, 31.48, 34.15, 36.48, 38.61, 39.01, 37.38, 39.9, 39.25, 38.96, 38.37, 39.06, 38.61, 37.97, 38.74, 37.93, 37.78, 38.18, 37.35, 37.98, 37.27, 37.25, 36.92, 37.46, 36.75, 37.59, 37.92, 37.02, 37.51, 37.48, 36.81, 37.05, 37.27, 37.64, 37.37, 37.77, 36.98, 37.74, 37.86, 37.77, 37.54, 37.4, 37.89, 38.32, 37.69, 37.4, 36.9, 37.97, 37.84, 37.42, 37.39, 37.37, 38.46, 38.11, 37.88, 37.35, 37.13, 37.08, 37.5, 37.61, 38.72, 37.65, 38.29, 38.03, 38.56, 37.78, 38.04, 38.25, 37.7, 38.04, 38.2, 38.77, 37.47, 37.85, 37.84, 37.84, 38.54, 37.99, 38.55, 38.47, 38.47, 38.38, 38.4, 39.1, 38.14, 38.51, 37.86, 39.94, 39.13, 37.79, 37.49, 37.66, 37.18, 38.51, 39.42, 37.55, 38.47, 38.32, 38.0, 36.9, 39.03, 38.29, 39.01, 38.72, 38.65, 38.78, 38.21, 38.7, 38.67, 39.82, 38.72, 38.12, 39.18, 39.17, 39.05, 38.75, 38.58, 38.76, 38.86, 39.06, 38.37, 38.8, 37.96, 38.79, 37.86, 38.39, 38.97, 38.52, 38.47, 38.97, 39.1, 39.08, 38.85, 39.08, 38.73, 38.59, 38.51, 39.33, 38.97, 38.48, 38.37, 39.38, 38.63, 39.05, 38.89, 39.12, 38.9, 38.5, 39.27, 39.39, 38.54, 39.08, 38.61, 39.57, 39.32, 39.15, 38.82, 39.33, 39.86, 38.63, 39.24, 39.69, 38.99, 38.87, 39.27, 38.96, 38.4, 38.45, 39.8, 38.49, 38.51, 39.24, 38.6, 38.42, 39.24, 38.8, 38.69, 38.84, 38.68, 38.41, 38.63, 38.72, 38.96, 39.55, 38.96, 39.39, 38.96, 38.56, 39.16, 39.05, 39.76, 39.87, 39.3, 39.6, 39.08, 39.44, 39.37, 39.17, 39.56], 'val_loss': [4.0052605662376255, 3.236190203648464, 2.93998375060452, 2.8348603134701964, 2.6777512784216815, 2.637142188989433, 2.6790426339313482, 2.7651753224385014, 2.7077081161699477, 2.761010536722317, 2.854759848041899, 3.020784270991186, 3.1078313186669804, 3.169016124336583, 3.3675191174646852, 3.423951230231364, 3.548468227599077, 3.6454222794551, 3.762306866372467, 3.950756337612298, 4.155579735139373, 4.140346296273979, 4.2559913207011615, 4.2358381991173815, 4.314347032528774, 4.538625739941931, 4.495664621614347, 4.584887421814499, 4.623433217121537, 4.619552640398597, 4.799819613717924, 4.7319332862355905, 4.759549321642347, 4.930600290845154, 4.909240838069065, 5.0662127672487, 4.919452529044667, 5.029402391925739, 4.978135623749654, 5.0333900406102465, 5.0789418903885375, 5.132633347420176, 5.203602679975473, 5.01928198033837, 5.21767752869114, 5.219928466590347, 5.270218691248803, 5.336961427311988, 5.33831817557098, 5.466983297068602, 5.426459218286405, 5.44191320003218, 5.388733976206202, 5.506285501893159, 5.526237961592948, 5.5830894746598165, 5.487467049793073, 5.590216238027925, 5.731505927766205, 5.657868423279683, 5.770012237463787, 5.739123311012414, 5.606394668293607, 5.815934463671058, 5.727716948576035, 5.785697640127437, 5.610354157010461, 5.838355257252979, 5.813189927939397, 5.647189014276881, 5.851792737177223, 5.785350275647109, 5.970940426656395, 5.846436767061805, 5.959059236915248, 5.926688945217497, 6.038201094433001, 5.970390782234775, 5.921266413797998, 5.925537490541009, 5.828080719443643, 5.969986268669177, 6.006517993416756, 5.894422421789473, 5.900860487275822, 6.055238014573504, 6.024432729004295, 6.1682028732482035, 6.188776215929894, 6.182639768169184, 6.211510573223138, 6.197083251491474, 6.293417197124214, 6.2436929705795965, 6.132702109919991, 6.339204313648734, 6.168463962093281, 6.238988057822938, 6.214862105193411, 6.378292984263912, 6.310607563917804, 6.396243503898572, 6.0307300060418, 6.293446486163291, 6.235658442138866, 6.478146224264886, 6.40878984351067, 6.204767387384062, 6.386066953088068, 6.523018041234107, 6.5747811490563075, 6.576087556826841, 6.345384939460997, 6.414909668029494, 6.390523899133038, 6.358188443123155, 6.543659807010821, 6.458260528600899, 6.496733179517612, 6.438584652675945, 6.58573188979155, 6.654868505562947, 6.455034713836232, 6.8553157475343935, 6.626525436996654, 6.6472943163221805, 6.736762924558797, 6.53688818937654, 6.589419812913153, 6.6680794650582, 6.735148637158097, 6.6473769488608, 6.6055461342927, 6.772922963853095, 6.582504655145536, 6.561526956072279, 6.57368559594367, 6.662842086925628, 6.659582088707359, 6.80083942868907, 6.716843516963302, 6.668479164694525, 6.888175387291392, 6.818995917678639, 6.741883451012289, 6.649873105583677, 6.710977529264559, 6.7008847081737155, 6.817906700122129, 7.062949440281862, 6.866091749470705, 6.6989766913614455, 6.7768907114199015, 6.910494921313729, 6.6951954957026585, 6.897157430648804, 7.01425885091162, 6.778877317525779, 6.8657350889436755, 6.809197889012136, 6.839487346114626, 6.808435244165408, 7.072535912702038, 6.8973533530144175, 6.876772974706759, 7.028767338224277, 6.9260915373540985, 6.84298643185075, 7.063883169441466, 6.9813478144870444, 7.040113860634482, 6.956397402818036, 6.932601535396212, 6.992118981233828, 6.980916785586412, 7.064372985985629, 7.141700085560987, 7.188337196969682, 6.867620185681969, 7.155064347443307, 7.033143386719333, 7.13415583531568, 7.318857660718784, 7.058895641071781, 7.163161004424855, 7.032583210878311, 7.067459224895307, 7.033523647648514, 7.133476453981582, 7.33852102650199, 7.0873455409031765, 7.250932433043316, 7.177452646243345, 6.991590982789447, 7.121017337604693, 6.982139406690172, 7.4425463737196225, 7.159418033186797, 7.2200094438662195, 7.603033889630797]})


# EfficientNetV2 Tiny-Imagenet
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="SGD")
data.update({'train_metric': [0.815, 0.895, 1.14, 1.39, 1.719, 2.073, 2.531, 2.862, 3.452, 3.996, 4.506, 5.095, 5.71, 6.392, 6.874, 7.486, 8.14, 8.832, 9.408, 10.048, 10.562, 11.159, 11.741, 12.532, 12.904, 13.495, 13.959, 14.572, 15.119, 15.625, 16.276, 16.677, 17.148, 17.706, 18.108, 18.512, 19.351, 19.57, 20.065, 20.542, 20.997, 21.483, 21.716, 22.202, 22.697, 23.144, 23.55, 24.098, 24.396, 25.003, 25.294, 25.693, 26.354, 26.763, 26.962, 27.546, 27.758, 28.095, 28.602, 29.129, 29.568, 29.934, 30.422, 30.769, 31.26, 31.65, 32.053, 32.352, 32.853, 33.359, 33.816, 34.222, 34.69, 35.019, 35.538, 35.9, 36.428, 36.892, 37.35, 37.931, 38.366, 38.723, 39.275, 39.761, 40.301, 40.583, 41.276, 41.769, 42.144, 42.678, 42.793, 43.288, 44.203, 44.571, 44.908, 45.285, 45.723, 46.304, 46.975, 47.4, 47.856, 48.361, 48.896, 49.34, 50.024, 50.315, 50.994, 51.328, 51.838, 52.324, 52.918, 53.367, 53.637, 54.239, 54.655, 54.986, 55.486, 56.067, 56.556, 57.317, 57.208, 57.999, 58.029, 58.911, 59.286, 59.497, 59.727, 60.606, 60.813, 61.25, 61.596, 62.132, 62.465, 62.659, 63.214, 63.647, 64.049, 64.301, 64.72, 65.199, 65.241, 65.821, 66.214, 66.408, 67.221, 67.165, 67.426, 67.779, 67.956, 68.496, 68.868, 69.048, 69.42, 69.827, 70.039, 70.046, 70.677, 70.927, 71.229, 71.255, 71.949, 72.013, 72.152, 72.479, 72.504, 73.221, 73.38, 73.454, 73.464, 73.544, 73.943, 74.581, 74.544, 74.328, 74.903, 75.031, 75.534, 75.49, 75.984, 76.136, 76.262, 76.193, 76.653, 76.496, 76.692, 77.253, 77.366, 77.663, 77.623, 77.75, 77.95, 77.854, 78.173, 78.276, 78.643, 78.796, 78.906, 78.908, 79.088, 79.24], 'train_loss': [5.2839279049722645, 5.253478591745661, 5.231341381219435, 5.206127076749991, 5.176951250355746, 5.140342578473033, 5.085887968730866, 5.017226361953823, 4.947477862686967, 4.884046538701366, 4.822956691700453, 4.763544991774507, 4.706876371887656, 4.6463756161626915, 4.581633368899101, 4.5210764537769785, 4.460768983368681, 4.400041869719366, 4.344984045718163, 4.293625892009479, 4.245892126897322, 4.1981946734076345, 4.154245862576417, 4.109155172273583, 4.072816188565753, 4.030606226972945, 3.9913928488928465, 3.952322736430153, 3.9183211829978077, 3.881727927934643, 3.842250828398243, 3.80754046293687, 3.7754238013342567, 3.737457759854737, 3.7061418076013037, 3.6730749027437684, 3.6394134833884406, 3.6102908420318682, 3.577199273893487, 3.548973163274032, 3.5180023436506667, 3.4901385025114715, 3.459031285495233, 3.433339583133934, 3.4049522097264813, 3.37668901929776, 3.349122769086695, 3.322689026346286, 3.299705374950182, 3.2735533252101026, 3.2475983153835597, 3.222010840838793, 3.1971008713551994, 3.1677977015250627, 3.14664755017042, 3.1218242126630806, 3.0976926441461097, 3.080789175692538, 3.0523987682828215, 3.0278393268890285, 3.0067501435734414, 2.982020795917328, 2.958465089877294, 2.9355728482864487, 2.9091266673570706, 2.88525350919078, 2.861660327044955, 2.8388454303936705, 2.8166993394999342, 2.795983472392106, 2.7684451044940523, 2.7471799882497074, 2.7224541722546003, 2.699267054092251, 2.680872376729339, 2.6526577612260023, 2.631108710038227, 2.6069257484364035, 2.5816884782935134, 2.556218873668922, 2.535095216674219, 2.5135639825083853, 2.4885480372629636, 2.4622182367859327, 2.4430468800887035, 2.4210853013028264, 2.3974373731869423, 2.3710793384129776, 2.3505218739091625, 2.322586317285085, 2.3097781403584885, 2.2879305463987363, 2.257087453541966, 2.2361718542821425, 2.2116270590042046, 2.1936075393008028, 2.1697849350256257, 2.1474533332744166, 2.121264327586803, 2.100684984631822, 2.084273097611206, 2.0611730835335575, 2.037377987240494, 2.017435605222418, 1.9959100161465178, 1.972974548489332, 1.9466586291446795, 1.9349336389235328, 1.9104016987238643, 1.8884155960778586, 1.8671247580077355, 1.8478207796411643, 1.834438422972471, 1.8129777529265587, 1.7935466219504828, 1.7725208985523315, 1.7599761402156975, 1.7334294281780758, 1.7175796892272306, 1.692598467291126, 1.6770864266732985, 1.6551465614438439, 1.6508821780606866, 1.623034101911485, 1.6039020941185784, 1.5930370453528235, 1.5803309446409277, 1.5560877044004122, 1.5465732896777962, 1.5257187664546954, 1.5137548156449678, 1.495379768879232, 1.4720697943323786, 1.472313167609546, 1.4535748755710673, 1.4370706777731272, 1.4216964095354232, 1.4031056877671337, 1.3968526033812125, 1.380498369420406, 1.3685424719875772, 1.35300109021113, 1.336204651678821, 1.3286359112230692, 1.2993666614695993, 1.3004749300612597, 1.2858132841643506, 1.2743211711208102, 1.2664291486096397, 1.2465751443394315, 1.2320283225005206, 1.2232296208655002, 1.2159849234063582, 1.1994022002452014, 1.1886656605045687, 1.1838924713952375, 1.1669992192464231, 1.1619122927568695, 1.1439572701222303, 1.1437674929374162, 1.122112876127259, 1.1203295905393282, 1.109657042238549, 1.0992564105179092, 1.097761847274241, 1.0714288301675348, 1.0659545796922743, 1.0612259104857442, 1.0585507864152783, 1.0542390416695075, 1.0440000873762145, 1.0208760101445882, 1.0208879034258354, 1.024623819787153, 1.0040845567418915, 1.001208416483605, 0.982615458225487, 0.9844142265298469, 0.972011561585937, 0.9623927844691872, 0.9582254273420103, 0.9590260334985041, 0.9431199685396937, 0.943761619385892, 0.9353693900242572, 0.9224106326594387, 0.9164571736534666, 0.9048053916646209, 0.9033617061334623, 0.8972855124882376, 0.8924020012257882, 0.8901563236824763, 0.8824108067187298, 0.8735380757335509, 0.8675840766629728, 0.8591782602071, 0.8553656998614203, 0.8528851534186916, 0.8430017589645666, 0.8424400135407597], 'val_metric': [1.0, 1.18, 1.5, 1.82, 2.02, 2.57, 3.05, 3.52, 4.14, 4.59, 5.2, 5.98, 6.76, 7.48, 7.88, 8.73, 9.38, 10.33, 10.57, 11.64, 12.1, 13.0, 13.2, 13.5, 14.56, 14.33, 15.28, 15.43, 15.79, 16.49, 17.44, 18.15, 17.99, 18.86, 19.15, 19.75, 19.69, 20.33, 20.39, 21.21, 20.75, 21.41, 22.19, 22.43, 23.13, 23.11, 23.72, 23.42, 23.87, 24.39, 24.2, 24.06, 24.51, 24.75, 25.17, 25.14, 25.41, 25.61, 25.16, 25.59, 25.64, 25.88, 26.14, 26.19, 26.13, 26.74, 26.46, 27.0, 26.7, 26.79, 26.87, 27.23, 26.77, 27.33, 26.13, 27.71, 27.22, 27.27, 27.35, 27.35, 27.47, 27.07, 27.81, 27.49, 27.87, 27.15, 28.31, 27.02, 27.71, 27.26, 27.63, 27.36, 27.44, 26.94, 27.18, 27.42, 25.75, 28.08, 27.55, 27.44, 27.35, 27.38, 27.1, 27.87, 26.9, 27.21, 27.8, 27.81, 27.05, 27.35, 27.85, 27.16, 27.72, 27.87, 27.27, 27.09, 27.3, 27.77, 26.34, 27.23, 27.22, 26.87, 27.44, 27.48, 27.96, 25.75, 26.82, 26.7, 26.72, 27.34, 27.7, 25.91, 25.71, 26.59, 27.6, 27.14, 26.68, 27.72, 26.89, 26.72, 26.0, 27.23, 26.52, 26.49, 27.02, 26.15, 28.02, 27.04, 26.68, 27.14, 27.03, 26.3, 27.23, 26.43, 26.71, 27.5, 27.28, 26.11, 26.54, 27.29, 26.24, 27.15, 26.63, 25.61, 26.91, 27.14, 26.38, 26.15, 26.49, 27.54, 26.69, 26.22, 26.05, 26.09, 26.18, 26.44, 26.31, 25.91, 25.79, 26.38, 26.36, 26.84, 27.47, 27.6, 26.98, 26.67, 26.19, 26.99, 27.21, 26.5, 27.48, 27.6, 27.46, 27.57, 26.55, 26.86, 27.54, 26.78, 27.35, 26.37], 'val_loss': [5.341625137693563, 5.267210158572834, 5.262394513294196, 5.220795891087526, 5.192527877297371, 5.119404707744622, 5.028401702832264, 4.953458717674207, 4.873235933340279, 4.7991685639521116, 4.733114942623551, 4.6720317336404396, 4.606562379059518, 4.533734374744877, 4.451471181432153, 4.39189821291881, 4.333076642577056, 4.26958989793328, 4.240008509083158, 4.173178274920033, 4.1200143820161275, 4.066305707214744, 4.039221162249328, 3.982790487587072, 3.941747600865212, 3.935446862202541, 3.8721243150674614, 3.861566558764998, 3.8243461824526452, 3.777451269945521, 3.741592761057957, 3.6897019527520345, 3.675967807982378, 3.646477292297752, 3.602756568580676, 3.576248643504586, 3.5686503595607295, 3.5353410486962384, 3.5120566642967757, 3.4660406515097164, 3.499366808089481, 3.452655442201408, 3.423147126367897, 3.3971508635077505, 3.3598029742574997, 3.3573478194558697, 3.3289837746103856, 3.3032893048729868, 3.2966450961532106, 3.278285771418529, 3.2850741998405213, 3.2810493297637646, 3.258192322056764, 3.2264268565329774, 3.219296071180113, 3.22419615687838, 3.2039020266502525, 3.195252304623841, 3.2130697608753374, 3.190562312010747, 3.194195515031268, 3.180275756082717, 3.1620149445381895, 3.1655900668186745, 3.1896660730337643, 3.151287191233058, 3.1759399152865075, 3.1437609658879078, 3.1471006376728132, 3.134388671559133, 3.136753357899417, 3.151323770261874, 3.215198910919724, 3.1518057918852302, 3.2101534658176885, 3.1305599865640046, 3.1866303203971524, 3.180095545045889, 3.139517968627298, 3.151983426634673, 3.1997744482793626, 3.170192358600106, 3.155999788812771, 3.165696514639885, 3.1508599899377034, 3.2041965920454376, 3.159042590742658, 3.2258134017324753, 3.187121984305655, 3.2236573308896106, 3.206805644521288, 3.2108155823057625, 3.2341696897130103, 3.250087323462128, 3.2279546139346564, 3.25656846981899, 3.398564572546892, 3.2053794185067437, 3.27458667451409, 3.2688814750902213, 3.3200969961798115, 3.2655433659340924, 3.3198607582954844, 3.2584907651706865, 3.3126851046920582, 3.344876378205172, 3.3347081583776292, 3.292129946362441, 3.3643844476930655, 3.327677891512585, 3.3097294379191795, 3.332975321514591, 3.3785202320973586, 3.3550409389908906, 3.408214569091797, 3.4237384310193884, 3.386841542401891, 3.4143465316978987, 3.511767414724751, 3.4751740200504377, 3.437221151248665, 3.4696458327542445, 3.4907282788282745, 3.4641178276888125, 3.4241183662110832, 3.6740957825047196, 3.478928620648232, 3.5511930634261697, 3.521401993787972, 3.502837550867895, 3.516892649565533, 3.6696170067331595, 3.6934343280306288, 3.6076061672465816, 3.515708211121286, 3.580822192179929, 3.6312368781703293, 3.5201872647947567, 3.6083512328992224, 3.66237054281174, 3.750935773940603, 3.5667721200141176, 3.7302547738810254, 3.6754094567268516, 3.7000558983748126, 3.6839467362993084, 3.574733985457451, 3.7130900560670597, 3.7538083085588587, 3.6769437835474683, 3.7102357581922205, 3.7281644427852267, 3.6808439515958167, 3.7320205208602224, 3.801183031622771, 3.7048494679153343, 3.718558513434829, 3.827298773322136, 3.8343382565079223, 3.7283885273963784, 3.824327999619162, 3.7190966439095274, 3.81108234870206, 3.98942161973115, 3.8003341589763666, 3.727992418465341, 3.799623171235346, 3.9759042976768155, 3.8671741242621356, 3.7481918638678873, 3.865156838848333, 3.873296336763224, 3.938713939326584, 3.9499161220659875, 3.9231213703277006, 3.9466541648670366, 3.981745486046858, 3.953868191712981, 3.9888224100611014, 3.9123397351829867, 3.915505169303554, 3.902535347422217, 3.885417147806496, 3.8542240402501102, 3.9034099844610615, 3.9478433625713274, 3.980163957662643, 3.9264049765410696, 3.947962066170516, 3.9366592533269507, 3.9413463925100434, 3.9547225973408695, 3.942954531900442, 3.9150207354004976, 3.986699516606179, 3.9645301255450884, 3.9436987889040807, 3.990820612117743, 3.964751058323368, 4.0071140299936765]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="Adam")
data.update({'train_metric': [1.969, 4.438, 7.492, 9.717, 12.097, 13.795, 14.823, 16.088, 16.785, 17.803, 18.281, 18.808, 19.152, 19.367, 19.763, 20.061, 20.338, 20.589, 20.83, 21.338, 21.551, 21.653, 22.024, 22.153, 22.276, 22.522, 22.654, 22.786, 22.9, 22.883, 23.139, 23.308, 23.267, 23.557, 23.57, 23.761, 23.887, 23.661, 23.835, 24.083, 24.117, 24.168, 24.359, 24.358, 24.181, 24.342, 24.289, 24.562, 24.499, 24.499, 24.608, 24.545, 24.767, 24.772, 24.794, 24.798, 24.818, 24.936, 24.949, 24.87, 24.934, 25.083, 25.227, 25.024, 24.931, 25.058, 25.132, 25.149, 25.273, 25.318, 25.398, 25.356, 25.56, 25.429, 25.381, 25.502, 25.566, 25.555, 25.466, 25.368, 25.609, 25.542, 25.544, 25.826, 25.564, 25.541, 25.588, 25.597, 25.692, 25.658, 25.62, 25.662, 25.638, 25.65, 25.745, 25.885, 25.747, 25.793, 25.798, 25.928, 26.006, 25.783, 25.843, 25.885, 25.858, 25.891, 25.973, 26.089, 26.011, 26.133, 25.98, 25.967, 25.876, 26.033, 25.966, 25.919, 26.207, 26.078, 26.08, 26.159, 26.037, 26.06, 25.927, 26.111, 26.114, 26.057, 25.944, 26.024, 26.043, 26.134, 26.057, 26.291, 26.145, 26.2, 26.24, 26.045, 26.198, 26.182, 26.088, 26.539, 26.254, 26.223, 26.298, 26.314, 26.352, 26.236, 26.314, 26.359, 26.433, 26.305, 26.406, 26.444, 26.362, 26.538, 26.38, 26.393, 26.399, 26.317, 26.454, 26.424, 26.609, 26.55, 26.616, 26.492, 26.525, 26.627, 26.493, 26.527, 26.46, 26.534, 26.594, 26.495, 26.47, 26.477, 26.485, 26.533, 26.621, 26.573, 26.549, 26.718, 26.626, 26.669, 26.591, 26.593, 26.696, 26.63, 26.643, 26.577, 26.628, 26.822, 26.744, 26.776, 26.634, 26.532, 26.662, 26.713, 26.739, 26.502, 26.672, 26.576], 'train_loss': [5.032089919824289, 4.684702707572542, 4.381697434762771, 4.1883437128610055, 4.023232192620969, 3.916670980624335, 3.8334484602195364, 3.7672406870511885, 3.7145912145393747, 3.6725856745311716, 3.643208504790918, 3.613895891876611, 3.592208919659381, 3.5731049622775495, 3.5531592361452637, 3.536639478064773, 3.519291779969033, 3.50670853259086, 3.4858870986784716, 3.469912422519423, 3.4565381663622037, 3.4435217327135006, 3.4292799287016242, 3.4227468952183835, 3.4114453661388415, 3.401361196985324, 3.3959208462227632, 3.3882703149616145, 3.3783232833205776, 3.375865609998209, 3.3670467459385165, 3.36050705183643, 3.3576693536376463, 3.3485936674641557, 3.3451080577005885, 3.3364192318321417, 3.329269865118992, 3.328162909011694, 3.326324234997242, 3.3189194997151694, 3.315811097736322, 3.3102014035806393, 3.305483717836025, 3.3023103458028684, 3.3023937073412877, 3.2973183382037963, 3.2926412312098217, 3.292097885640706, 3.291323971611067, 3.2863577415908063, 3.285164551786788, 3.2838602349762724, 3.2804981444939085, 3.2769267057197946, 3.275453038041743, 3.274438518785317, 3.2733225618046364, 3.27158374673498, 3.267388372488382, 3.267604624088651, 3.2645406294387653, 3.25752184212551, 3.258263663458504, 3.2592206205684104, 3.2578634584247492, 3.2547058839486795, 3.2517665292846036, 3.2515996364317714, 3.2496736566759576, 3.249234774291172, 3.2458067852491306, 3.245184941392485, 3.243244173583203, 3.242413055873878, 3.2414041470047454, 3.2399701512325136, 3.2366134450745294, 3.2332394211550057, 3.238799898424594, 3.2372442959061205, 3.2369797324341074, 3.2338171685573314, 3.235368091787044, 3.2297115778968797, 3.2321972866814943, 3.232958901332726, 3.229150644266979, 3.2316522392338847, 3.227067849610146, 3.2253229062830266, 3.2247096956805854, 3.224907571660809, 3.2243255243344104, 3.224978649379806, 3.2236396165055794, 3.220309696514791, 3.2225575463861817, 3.2144626401131227, 3.217267187756754, 3.217926557904547, 3.216167239142166, 3.2168147666135507, 3.2140143244066683, 3.211609577720774, 3.212846374481211, 3.2143134259292885, 3.2099659973737604, 3.2122895986089626, 3.20894284989692, 3.2086082319380456, 3.2110868267576738, 3.212182914303116, 3.212203325862238, 3.204584364851392, 3.2069287985010324, 3.207868681606847, 3.2051080762005277, 3.2071777820892238, 3.206220967145738, 3.206357285218291, 3.2052286482742027, 3.2034619903991564, 3.205966626385123, 3.200585454225693, 3.1991183275758495, 3.200575319224264, 3.2020002065830635, 3.204308433557121, 3.199320091319557, 3.2031783424580014, 3.197488185006384, 3.196609689574629, 3.197273486406469, 3.196195405031425, 3.1961825671900717, 3.194359088355886, 3.1964130223140605, 3.19354976756559, 3.1959031525134125, 3.1925791244970556, 3.191890392102115, 3.1921207398385936, 3.1905172655236163, 3.19314072762097, 3.191925588053766, 3.187774442817947, 3.1878697124720383, 3.1900474152653473, 3.1872638782017977, 3.187706458743993, 3.1851311263867244, 3.188106458002531, 3.1852986158198906, 3.1877996897133056, 3.1834811403136642, 3.1835923863006417, 3.182468596895917, 3.186748801479718, 3.183143968240466, 3.1838020299080467, 3.1847340761661833, 3.1762703955364167, 3.178807641326504, 3.1819842029517837, 3.179200321912613, 3.1802509502196115, 3.1739658385000395, 3.1793163082039824, 3.177021266555298, 3.1838417582533256, 3.1815344948686244, 3.1767539846843738, 3.1792223432166256, 3.1795501802033215, 3.1794815266353535, 3.1796523208276475, 3.1742679511440617, 3.1747461878483065, 3.174452054676, 3.172406188357128, 3.1719016328654224, 3.175579548034619, 3.1715539461210303, 3.177100387049728, 3.172522746059884, 3.170603472074483, 3.173131336711266, 3.171302811121681, 3.174154090027129, 3.1714961972666793, 3.171497943686585, 3.1693220019569326, 3.169545259982138, 3.170073390388367, 3.16998265206013, 3.17095660019287, 3.168151207177668, 3.170489821614017, 3.1655095723372426, 3.166834387959232], 'val_metric': [2.85, 6.2, 8.46, 10.91, 13.25, 14.54, 16.65, 16.86, 18.22, 18.63, 19.46, 20.17, 19.67, 20.03, 20.85, 20.59, 21.2, 22.26, 21.98, 21.9, 22.97, 23.44, 22.83, 23.35, 22.52, 23.33, 23.38, 23.63, 24.33, 24.32, 23.93, 24.39, 23.75, 25.07, 24.28, 24.51, 24.15, 25.29, 24.42, 25.3, 25.05, 25.13, 25.95, 25.26, 26.19, 25.24, 25.53, 25.49, 26.55, 26.3, 26.14, 25.87, 25.43, 26.46, 26.33, 26.49, 27.17, 25.88, 25.64, 26.78, 26.09, 26.76, 26.01, 26.36, 26.9, 26.29, 26.22, 26.9, 27.7, 26.75, 26.01, 26.16, 26.42, 26.75, 26.32, 26.06, 26.47, 26.74, 26.82, 26.7, 26.7, 26.43, 26.21, 26.38, 26.21, 26.72, 26.96, 26.86, 27.23, 27.13, 26.8, 27.36, 25.83, 26.49, 26.96, 27.49, 27.47, 26.27, 27.05, 26.95, 27.16, 26.05, 27.08, 26.66, 26.51, 27.18, 27.21, 26.54, 26.2, 27.64, 27.21, 27.76, 27.46, 27.16, 26.79, 25.83, 27.22, 26.7, 26.67, 27.39, 27.54, 27.56, 27.01, 27.67, 26.46, 27.1, 26.96, 26.57, 26.7, 27.66, 27.6, 26.49, 27.33, 26.16, 26.55, 26.78, 27.34, 26.94, 27.12, 26.94, 27.27, 27.7, 26.76, 26.76, 26.83, 26.87, 27.21, 26.73, 26.72, 27.33, 27.5, 27.66, 27.84, 27.39, 27.83, 28.04, 27.03, 27.19, 27.42, 28.24, 27.86, 27.15, 27.84, 26.95, 27.36, 28.83, 28.11, 28.89, 27.42, 28.64, 27.5, 27.87, 27.49, 26.35, 27.27, 26.45, 28.11, 28.35, 28.3, 27.02, 27.58, 26.66, 27.07, 28.03, 27.42, 26.68, 27.47, 27.97, 27.37, 27.67, 27.03, 27.24, 26.14, 27.92, 27.65, 26.89, 27.81, 28.23, 28.55, 27.45], 'val_loss': [4.851453618638835, 4.509948256668771, 4.258511307892526, 4.092399876588469, 3.9177475081887216, 3.8643862426660625, 3.730711106282131, 3.757502022822192, 3.621153784405654, 3.613153033955082, 3.574275415414458, 3.513535583854481, 3.543000007131297, 3.5336466632830867, 3.4766989285778847, 3.4893904901613855, 3.448038711669339, 3.4062590887592097, 3.410138833295008, 3.3982459891373944, 3.347594208018795, 3.3657537850604693, 3.369234955994187, 3.363090491598579, 3.3853696053195153, 3.321354275296448, 3.3323504119921643, 3.3361603067179395, 3.280534339558547, 3.2965305496932595, 3.320923994301231, 3.2922467950043406, 3.3126619727748214, 3.266533032344405, 3.3094136608634024, 3.276015898983949, 3.3057926604702215, 3.253918793550722, 3.284342992077967, 3.2553880093203986, 3.2500534118360775, 3.2367310721403473, 3.2138686886258947, 3.247033691709968, 3.217348191388853, 3.25872247613919, 3.2565582474325874, 3.2175032887489174, 3.205249355097485, 3.195588240957564, 3.203833382600432, 3.2135709136914294, 3.2237609381888324, 3.1959954727986815, 3.1885945121194146, 3.1781480904597386, 3.165950777424369, 3.2122661011993507, 3.206527872450033, 3.200123835521139, 3.19567421439347, 3.183946250350612, 3.212869633534911, 3.1899812996007832, 3.169098346096695, 3.1774172615853087, 3.1885208506493052, 3.1822094158002527, 3.145245669753688, 3.2128781710460688, 3.199717485980623, 3.1819672903437524, 3.184004555082625, 3.190533227981276, 3.18032621350258, 3.201294807871436, 3.1906014749198963, 3.171723058269282, 3.1773924721274405, 3.1542492262117423, 3.171785091138949, 3.2006629424489987, 3.1908997760456836, 3.1574715398679114, 3.1819341068814513, 3.172759204153802, 3.1574160399710296, 3.1609656104616297, 3.1564607195034147, 3.1659022653178805, 3.1731908085999216, 3.1466275362452123, 3.1907849645918342, 3.1872104937863197, 3.1680824475683225, 3.1327289980687913, 3.1430672285663093, 3.1938076080030697, 3.1416878882487107, 3.1514096518231045, 3.147653326866733, 3.183479483719844, 3.1471038661944637, 3.1779445212358124, 3.158895024068796, 3.130782563215608, 3.146022525562602, 3.1657870058800763, 3.175400410488153, 3.1051265715034146, 3.1356796003451013, 3.1106940409180464, 3.122606920588548, 3.1460383743237537, 3.147757710924574, 3.1935897032926035, 3.1350585115942984, 3.168482495720979, 3.1487797673340814, 3.15006401812195, 3.1285374096244762, 3.1452831083042607, 3.155880476259122, 3.091774273829855, 3.148293181589455, 3.1248751582613417, 3.135206369837378, 3.1256943834815054, 3.139139400166311, 3.1094113717413254, 3.1098923576865225, 3.1824166121756194, 3.117479592371898, 3.1926690848769654, 3.151008808688753, 3.1471533509576397, 3.1134235957625567, 3.121940683407389, 3.125563995853351, 3.1372838035510604, 3.1220500271791107, 3.115258803033525, 3.1638324511278966, 3.1541798653875945, 3.144038195822649, 3.1268717847811947, 3.1251651845919857, 3.1556920094095218, 3.1535824415790046, 3.1247200411596117, 3.12233274377835, 3.1151162097408513, 3.137187750476181, 3.116145917564441, 3.106228700868643, 3.1096803564934214, 3.138983958845685, 3.1178411404797983, 3.1229836424444892, 3.085314650444468, 3.115945286811537, 3.140312459818117, 3.1073796483361797, 3.1371790975522083, 3.128389452673068, 3.084152157139626, 3.089463401751913, 3.0629272445751603, 3.1274249060138777, 3.0927133005895433, 3.1080496075806345, 3.127717858666827, 3.138992172137947, 3.1518241144289636, 3.1297775074175207, 3.156331837556924, 3.1104291571173697, 3.1101604441928257, 3.09540535082483, 3.1505505977922184, 3.118506319203954, 3.156256631681114, 3.1219528905904976, 3.106303623527478, 3.109429210614247, 3.1276224919944813, 3.143290653350247, 3.128498367443206, 3.1299168235936743, 3.092040247977919, 3.156511364469103, 3.13903679741416, 3.188400608718775, 3.095474584087445, 3.1404665738913664, 3.141001037731292, 3.116793362198362, 3.0895099237466312, 3.0792321035057117, 3.1188781967588293]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Default initialization", optimizer="AdamW")
data.update({'train_metric': [4.406, 13.706, 23.167, 31.012, 37.0, 42.762, 47.378, 52.225, 56.734, 60.952, 65.44, 69.311, 73.584, 77.166, 80.125, 82.838, 84.556, 86.448, 87.524, 88.885, 89.267, 90.501, 90.988, 91.635, 91.912, 92.36, 92.558, 93.128, 93.371, 93.518, 93.881, 94.275, 94.352, 94.445, 94.667, 94.845, 94.968, 95.305, 95.199, 95.495, 95.591, 95.569, 95.856, 95.761, 96.095, 95.876, 96.125, 96.206, 96.222, 96.377, 96.287, 96.546, 96.522, 96.504, 96.74, 96.847, 96.712, 96.898, 96.885, 96.802, 97.042, 96.977, 97.1, 97.057, 97.21, 97.127, 97.274, 97.313, 97.263, 97.296, 97.41, 97.336, 97.355, 97.528, 97.487, 97.548, 97.587, 97.567, 97.545, 97.707, 97.633, 97.663, 97.719, 97.644, 97.718, 97.694, 97.75, 97.783, 97.831, 97.797, 97.925, 97.757, 97.863, 98.03, 97.794, 97.925, 97.906, 97.964, 98.144, 97.912, 98.024, 97.967, 98.059, 98.001, 98.076, 98.177, 98.122, 98.068, 98.201, 98.139, 98.108, 98.184, 98.292, 98.27, 98.28, 98.212, 98.22, 98.16, 98.19, 98.267, 98.304, 98.254, 98.406, 98.304, 98.271, 98.309, 98.379, 98.284, 98.409, 98.257, 98.503, 98.311, 98.309, 98.422, 98.328, 98.468, 98.294, 98.461, 98.374, 98.429, 98.416, 98.469, 98.456, 98.513, 98.362, 98.452, 98.515, 98.402, 98.594, 98.413, 98.545, 98.354, 98.571, 98.549, 98.488, 98.603, 98.53, 98.507, 98.588, 98.466, 98.643, 98.533, 98.56, 98.52, 98.629, 98.6, 98.457, 98.701, 98.593, 98.663, 98.645, 98.602, 98.672, 98.604, 98.554, 98.664, 98.649, 98.731, 98.577, 98.657, 98.669, 98.641, 98.71, 98.78, 98.708, 98.562, 98.703, 98.746, 98.743, 98.75, 98.68, 98.682, 98.748, 98.713, 98.692, 98.749, 98.706, 98.654, 98.846, 98.732], 'train_loss': [4.777203631416316, 3.9394953327154547, 3.365172599876682, 2.9314391478772204, 2.615085764947185, 2.3378347995871547, 2.106983216618851, 1.8831834911308605, 1.6700169760221406, 1.473415023877845, 1.2775948098357184, 1.097886900164268, 0.9345604854749703, 0.7884539409432744, 0.6729082087160911, 0.5706162628022357, 0.5117901681290173, 0.44199653760180285, 0.40490064623641114, 0.3580509286781419, 0.34458455625475787, 0.3087272039583037, 0.290371108530846, 0.26725362238646394, 0.2602279878108836, 0.24414689017081978, 0.2387417566021684, 0.21967595718891592, 0.21417480047317894, 0.20483760390447373, 0.1965202953325619, 0.1850217127122619, 0.1813743624833823, 0.17669625414782086, 0.16945439497853843, 0.16187638839467244, 0.1614599175231623, 0.14828447249651872, 0.15407873730646518, 0.14372205242671915, 0.14054485185158822, 0.1412234093614938, 0.13240034979668588, 0.1359330490103405, 0.12385075707679766, 0.13110719345136404, 0.12225571353887033, 0.11963943755808296, 0.11916685893998188, 0.11349919995010585, 0.1162443627297678, 0.11003542076218313, 0.109179706093739, 0.11153020293905291, 0.10178249599102239, 0.10238109016880365, 0.10412361307048208, 0.09932959219977087, 0.09790710447015276, 0.10255183536000431, 0.0933574166438487, 0.09648189616019547, 0.09085409851775755, 0.09324098495565121, 0.0882517878418718, 0.09279979044772323, 0.08524747068802478, 0.08496031199608138, 0.08729743189267786, 0.08494952578581416, 0.08360854262216177, 0.08535087341204362, 0.08415407349411529, 0.07958954518141913, 0.08002822917140663, 0.07719103202923312, 0.07632085242426656, 0.0767027726827424, 0.07684799050867572, 0.07351456601062452, 0.07648072111785593, 0.07483227243015372, 0.07322723475810419, 0.07184716702724306, 0.07130558873700742, 0.07288513094695048, 0.07105155433280845, 0.06966614705843818, 0.06931681413741536, 0.07046254872535405, 0.06649413273420414, 0.07051165756871966, 0.06809108932563643, 0.06311069227759837, 0.06913808255631569, 0.06586900644864761, 0.06557047659780303, 0.06607645984574609, 0.05992731145226771, 0.06665508977765539, 0.06085633861514878, 0.06414007440649429, 0.06119687287564439, 0.06426475424246811, 0.06176961219681264, 0.05750294268122021, 0.060497437816231514, 0.060810008655529844, 0.05753437772268693, 0.058573199143801276, 0.06001551760953773, 0.05793064251571586, 0.054464169777631855, 0.05640591028094606, 0.05581830788650434, 0.056138863854610165, 0.056852098576531174, 0.05685712159858565, 0.056446733390697904, 0.05446253500188214, 0.05414014145769465, 0.05545163001541599, 0.05060497197645501, 0.054037512671952455, 0.05502510231612944, 0.05448298177980387, 0.050843954472790664, 0.054362979115961774, 0.04994848805545398, 0.0553893220460612, 0.04855748312108359, 0.0526774986380424, 0.05399793261314741, 0.05033399467855532, 0.0524813992530897, 0.04875178751818745, 0.05391823104206593, 0.04904066543692904, 0.051788171060381534, 0.049989763100191945, 0.04889755126036571, 0.04822658702529435, 0.050189290976758706, 0.04628090401033657, 0.04997046859328643, 0.04850843248712924, 0.047402857332901525, 0.05059301868415219, 0.04520728266964799, 0.05011514866988574, 0.045494463617772804, 0.052418888554942894, 0.04501121240331042, 0.045517623800924674, 0.04830721539171277, 0.04479858789146685, 0.04649814788328608, 0.04626970338144757, 0.04336775113380918, 0.04776806338254204, 0.0428810031550623, 0.046717982542186696, 0.04543553272674697, 0.04623030500020385, 0.044608174247851316, 0.044062268984706936, 0.049047443646888005, 0.041280698472157575, 0.045015980297388535, 0.04278035635855263, 0.04256858860608935, 0.044728749188057804, 0.04206000331065141, 0.04435679848366658, 0.04557701231506894, 0.042773648949908434, 0.04196267994653263, 0.04133679164113699, 0.04502868363096737, 0.04156867719694236, 0.042336076533149865, 0.043627203379070084, 0.04150783568074729, 0.039914540669708606, 0.039702645220809574, 0.04396268317889439, 0.04086026387836311, 0.040010175806369966, 0.04040453087912976, 0.040569902967330325, 0.04199755833710955, 0.041582059674549664, 0.039641968086698544, 0.04085392336038901, 0.04078689337527176, 0.04005369512020608, 0.04098951584337084, 0.04338461370009992, 0.036694457639455356, 0.040611814854749864], 'val_metric': [9.39, 19.42, 27.55, 33.41, 38.0, 42.34, 44.08, 45.69, 47.25, 48.68, 48.26, 48.85, 49.02, 48.45, 48.68, 48.7, 48.44, 49.22, 48.66, 48.83, 48.27, 48.54, 49.61, 49.27, 49.53, 48.62, 49.29, 49.26, 49.68, 49.59, 49.37, 49.26, 49.7, 49.49, 50.06, 49.59, 49.35, 48.99, 49.99, 50.36, 49.82, 50.13, 49.89, 49.68, 49.65, 50.33, 50.72, 50.09, 50.05, 50.56, 50.28, 50.58, 50.73, 50.6, 50.5, 50.87, 50.58, 50.19, 50.62, 50.38, 50.4, 50.94, 50.61, 50.13, 50.83, 51.21, 51.44, 50.7, 50.98, 50.61, 50.27, 50.52, 50.86, 50.93, 50.58, 50.88, 50.75, 51.25, 51.38, 51.6, 50.82, 51.14, 51.17, 52.03, 50.87, 51.23, 50.89, 51.02, 50.93, 51.59, 51.14, 50.9, 51.52, 50.68, 50.99, 50.96, 50.26, 51.33, 50.84, 51.15, 51.24, 51.03, 51.0, 51.39, 51.84, 51.89, 51.28, 51.38, 51.85, 51.66, 51.88, 51.7, 51.85, 51.55, 51.49, 51.48, 51.06, 51.13, 51.36, 51.07, 51.74, 51.91, 51.7, 51.25, 51.01, 51.66, 51.63, 51.89, 52.03, 52.03, 51.65, 51.26, 51.86, 51.57, 51.3, 51.66, 51.66, 50.95, 52.26, 51.2, 52.28, 51.39, 51.07, 51.13, 51.55, 51.24, 51.42, 51.7, 51.84, 51.36, 51.44, 52.13, 51.64, 52.01, 51.82, 52.0, 51.68, 52.33, 51.79, 51.53, 51.87, 51.86, 51.79, 51.54, 51.9, 51.9, 51.21, 52.03, 51.8, 52.23, 51.67, 51.96, 52.14, 51.97, 51.91, 52.66, 51.92, 51.61, 51.93, 51.75, 51.61, 51.46, 51.67, 51.6, 51.96, 51.63, 51.78, 52.25, 51.27, 52.1, 51.93, 52.39, 51.97, 52.06, 51.74, 51.56, 51.98, 51.79, 52.13, 51.74], 'val_loss': [4.2428064862633965, 3.5382892159140034, 3.0906604133593807, 2.800846275250623, 2.5720758259676066, 2.3835282196664505, 2.3162674300230233, 2.25793550880092, 2.2185127211224502, 2.1873631538099545, 2.2338011390084676, 2.2942368505866666, 2.3661949148603307, 2.4719252783781402, 2.5420741378122074, 2.6641444522104445, 2.737205449183276, 2.7919714029427545, 2.86707445808277, 2.9012993388115222, 2.9602117208158893, 3.0644624772345184, 2.955943606461689, 3.070603032780301, 3.0543645043281993, 3.194118951536288, 3.1694853856305407, 3.1369949138847883, 3.1784826790451244, 3.220091429485637, 3.213945419165739, 3.222585050163755, 3.2710431305466185, 3.254467618313565, 3.2976721802334876, 3.3247898657610464, 3.367608009629948, 3.3759586006213147, 3.3532130722027675, 3.308515087054793, 3.380800136714984, 3.3598320772693415, 3.4267539180767765, 3.3671736132567096, 3.4068209259373368, 3.4370761380833423, 3.4485133529468706, 3.4917399594738225, 3.4689502852737526, 3.478754390197195, 3.4499996369052086, 3.4536803573559802, 3.4788076368866454, 3.458530744549575, 3.4642137440906207, 3.485456899472862, 3.53637290076845, 3.4934563591222094, 3.525001522082432, 3.5012631701056365, 3.549754457868588, 3.503668700054193, 3.5999616619887624, 3.6029452837196883, 3.5771637858858534, 3.540204654833314, 3.5440635711524138, 3.5543801192265407, 3.5599024303399833, 3.5346419689761603, 3.535439855733495, 3.5472803358819074, 3.5573019609329806, 3.586146503497081, 3.630938744848701, 3.576605145338994, 3.65806290222581, 3.6466349469628305, 3.5790195100626367, 3.577951343196213, 3.6456108002146337, 3.6086286500001408, 3.60590480239528, 3.6072541080462703, 3.64646992485994, 3.641023572842786, 3.6260131384916368, 3.647782167431655, 3.6590892570033953, 3.65197674332151, 3.635422926799507, 3.6607655355125477, 3.64041108500426, 3.725375958309052, 3.6674176363428685, 3.7164380390932608, 3.718032647090353, 3.6588863787377717, 3.695881212592884, 3.7238325996763386, 3.749168233127351, 3.768003927674263, 3.7103036042231663, 3.6701335664008075, 3.6667598060741544, 3.704093425896517, 3.7539973501946515, 3.7136390930528096, 3.7031168770638243, 3.7717213926801256, 3.6839151822837297, 3.713160238448222, 3.725391129399561, 3.7259628339937536, 3.7913950271667187, 3.740028986505642, 3.802543962837025, 3.7679743630111595, 3.7534794731504597, 3.727821510308867, 3.74391978789287, 3.7664494119632015, 3.7949536393402488, 3.7962799019114986, 3.8181069876737657, 3.7811692133071317, 3.7767905406891162, 3.763814869200348, 3.774593941345336, 3.759258803288648, 3.8027170706706444, 3.8302652759916462, 3.7775252040024774, 3.7951935241176824, 3.76011532355266, 3.8035012369702574, 3.813657319849464, 3.8206714323371838, 3.7592458337735217, 3.811338168040962, 3.7770427450252946, 3.7899465940560506, 3.811335540121528, 3.8012820216500836, 3.7920835823010486, 3.78378566321294, 3.8591905817104752, 3.784995057020977, 3.8096747109844427, 3.823672479125345, 3.8416669831913746, 3.7591646211162493, 3.826045258030011, 3.800781193812182, 3.8147157104152023, 3.8397568889484286, 3.847297837020485, 3.7790393267467524, 3.8726382878175967, 3.8895263755397433, 3.86219685700289, 3.8763320058774036, 3.894263937974432, 3.86697781237827, 3.8209966736234677, 3.956622333283637, 3.882496971233635, 3.8351088443379493, 3.843364541697654, 3.8364285792514776, 3.8756592106667296, 3.8277694062822185, 3.8532960995747025, 3.876778274964375, 3.8169853140594094, 3.764783943914304, 3.815204790823019, 3.8969863425394533, 3.841319338531251, 3.851679483037086, 3.849544799251921, 3.904761877409212, 3.868285644585919, 3.912619252493427, 3.8778434092072165, 3.8771095754234652, 3.908232316849338, 3.9156553563039016, 3.9428986128728103, 3.88096686419408, 3.9295006246323796, 3.8490948229079036, 3.86376121773082, 3.872872562924768, 3.919087293421387, 3.9680580777727115, 3.860281519829088, 3.9239489250122364, 3.8939672245341503, 3.953768244214878]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="SGD")
data.update({'train_metric': [0.769, 1.16, 1.68, 2.427, 3.02, 3.602, 4.344, 5.051, 5.754, 6.575, 7.461, 8.155, 9.153, 10.02, 10.905, 12.047, 12.749, 13.501, 14.497, 15.444, 16.159, 17.173, 18.028, 18.807, 19.667, 20.392, 21.237, 22.337, 22.864, 23.757, 24.453, 25.488, 26.46, 27.03, 27.672, 28.392, 29.172, 30.1, 30.723, 31.29, 32.225, 32.928, 33.537, 34.48, 35.012, 35.685, 36.764, 37.236, 37.992, 38.711, 39.333, 40.235, 40.903, 41.645, 42.265, 43.142, 43.668, 44.418, 44.897, 45.656, 46.401, 47.088, 47.792, 48.401, 49.308, 49.785, 50.528, 51.134, 51.777, 52.591, 53.071, 53.601, 54.448, 55.025, 55.583, 56.456, 56.937, 57.708, 58.196, 58.879, 59.421, 60.026, 61.068, 61.372, 61.941, 62.482, 63.069, 63.753, 64.346, 64.786, 65.488, 66.193, 66.495, 67.114, 67.634, 68.047, 68.624, 69.05, 69.957, 70.343, 70.763, 71.198, 71.471, 72.47, 72.756, 73.174, 73.408, 74.009, 74.546, 74.811, 75.236, 75.577, 76.083, 76.228, 76.815, 77.063, 77.322, 77.896, 78.22, 78.63, 78.866, 79.318, 79.724, 79.891, 79.998, 80.85, 80.816, 81.007, 81.356, 81.736, 81.997, 82.211, 82.359, 82.515, 82.954, 83.172, 83.375, 83.411, 83.698, 84.228, 84.142, 84.347, 84.663, 84.932, 85.194, 85.453, 85.718, 85.81, 85.852, 86.102, 86.187, 86.377, 86.296, 86.688, 86.956, 86.996, 87.209, 87.271, 87.497, 87.448, 87.73, 87.947, 88.006, 88.095, 88.376, 88.217, 88.658, 88.626, 88.54, 88.857, 88.845, 88.977, 88.956, 89.089, 89.297, 89.538, 89.515, 89.56, 89.675, 90.084, 89.857, 89.99, 90.105, 90.233, 90.511, 90.346, 90.567, 90.485, 90.839, 90.699, 90.799, 91.023, 90.949, 91.023, 91.148, 91.06, 91.105, 91.401, 91.553, 91.457], 'train_loss': [5.323357233387952, 5.264121549524562, 5.186702221536667, 5.101893215094021, 5.022678149791383, 4.940817299975238, 4.861451935783381, 4.777202943007456, 4.6945593633181915, 4.608191674440546, 4.525434669133416, 4.455542751405, 4.3817971458972, 4.309285680224174, 4.235888443768062, 4.1662702976856485, 4.096607244403715, 4.030134389130488, 3.965604740918941, 3.903713076830063, 3.8434396182125528, 3.786247985910622, 3.72561955345188, 3.6785860816713947, 3.62262141757948, 3.5701565544191873, 3.5190309630093175, 3.4658461295864327, 3.419960123106065, 3.3776019990863664, 3.327911585779123, 3.2813088669657935, 3.232976109876285, 3.1926989514173334, 3.1531962608574142, 3.1134434109380895, 3.0706888791924474, 3.028681229721333, 2.9896111369361806, 2.9515825715190083, 2.911724928015711, 2.874508537852604, 2.8355406395380722, 2.7986729900118488, 2.7571386811981884, 2.7220421647384847, 2.681165170303462, 2.648160435950535, 2.611691999038823, 2.5737543653694392, 2.540828686529295, 2.503466262240785, 2.469930947932843, 2.4324236016813487, 2.3997736997659285, 2.367456436386035, 2.3368713974723887, 2.2998867477275438, 2.26899858925942, 2.2403435037445734, 2.199426296118811, 2.1659718831990364, 2.1382366164860938, 2.1106721615837083, 2.071081798120866, 2.0442953523884806, 2.008953163353816, 1.9783446920734145, 1.951020116082995, 1.916148269855282, 1.890624554738431, 1.8595204219860826, 1.8293933456552691, 1.8079872305242206, 1.7772472463047664, 1.7456297854620604, 1.7223456076758075, 1.6849306341325023, 1.6640378553289217, 1.6334812145391795, 1.6143933379795028, 1.5869298191766135, 1.5481259988350358, 1.5282829572814287, 1.5044555863507345, 1.4809550141495005, 1.4581271160205662, 1.4263410479001943, 1.401550958954365, 1.3868271858739456, 1.3612501111003121, 1.3355908274879382, 1.3147851938020703, 1.2934088536126447, 1.275792603529346, 1.2528773164108482, 1.234349005891967, 1.213350977221896, 1.183371795459352, 1.167613820814583, 1.1422626836438707, 1.1302895388844223, 1.1144250931453035, 1.0832149618265343, 1.071616324261832, 1.0544399403259683, 1.0416446351806703, 1.0202794589450248, 0.998512908463591, 0.9856248486903868, 0.969936713909996, 0.9566748221181405, 0.9412643499124226, 0.9298205111016086, 0.9072216741373657, 0.9009771470221204, 0.8869429308294259, 0.869321837382521, 0.8561692482488551, 0.839271262831514, 0.827393511759495, 0.8137998829571315, 0.7979182673049751, 0.7898685249928397, 0.785877270494145, 0.7582533277919181, 0.7559743748402183, 0.7434466458899961, 0.735674910738311, 0.7274310178887897, 0.7087982797462514, 0.7035542812510629, 0.6966621171413746, 0.6862814899483936, 0.6728206838618771, 0.6711742181237966, 0.6570969137219534, 0.653657872975826, 0.6448110069362155, 0.6298926896913191, 0.6270843982048242, 0.6219519677828766, 0.6081531967631686, 0.5976162356630168, 0.5913510064398411, 0.5787244285632613, 0.5727446446396645, 0.5655764638996247, 0.5609555136143055, 0.5520868958072332, 0.549290508966147, 0.5416716795965257, 0.5414525782898002, 0.5315767858113071, 0.5249426564152853, 0.5167485307842512, 0.5102280866085377, 0.5080193394529309, 0.5004896574575628, 0.5005181507181832, 0.4932332301055935, 0.4848702754432089, 0.47963554526054164, 0.47474432029711916, 0.4678150911649221, 0.46777054483472574, 0.458268333810381, 0.4562470468472611, 0.46078597585970366, 0.44761321654094005, 0.44813837923266836, 0.44250819620915277, 0.4407873601198044, 0.43557913459346453, 0.42914726682870113, 0.4217085628481302, 0.4171309395575859, 0.418264537611148, 0.4141225530238618, 0.4022505783540884, 0.4070881602288208, 0.3997386643902583, 0.39675188587021537, 0.39224408171303515, 0.3864854257398893, 0.3855556670912397, 0.3804088904738655, 0.38208623230457306, 0.3713511960176955, 0.37370342926687716, 0.3676957530866276, 0.3640197152730676, 0.36731429760340156, 0.3626856354857132, 0.3579582927828406, 0.36090552211951843, 0.3560514122495572, 0.34573440002521794, 0.34678647207645596, 0.34296246192829777], 'val_metric': [0.96, 1.74, 2.89, 3.19, 3.94, 4.71, 5.92, 6.26, 7.24, 8.42, 8.94, 9.38, 10.71, 11.7, 12.37, 13.86, 14.54, 15.7, 16.12, 16.5, 18.08, 18.32, 19.32, 20.28, 20.67, 21.58, 22.15, 22.88, 23.06, 24.05, 24.71, 24.86, 26.15, 25.55, 26.07, 26.95, 27.19, 27.74, 27.63, 28.73, 28.21, 29.21, 29.58, 29.59, 29.81, 29.32, 30.36, 30.3, 30.52, 31.23, 30.76, 31.21, 30.99, 31.68, 32.02, 31.97, 31.39, 30.47, 31.66, 31.22, 31.75, 31.86, 32.03, 32.19, 31.35, 32.24, 32.21, 32.01, 32.61, 32.15, 32.07, 32.1, 32.08, 32.0, 31.89, 31.91, 31.83, 32.04, 31.81, 32.22, 31.8, 32.51, 32.2, 31.44, 32.13, 32.02, 31.75, 31.56, 30.95, 31.08, 32.2, 31.98, 31.29, 31.73, 31.76, 31.67, 31.19, 31.59, 31.82, 31.22, 31.54, 31.9, 31.31, 30.79, 30.97, 30.97, 31.51, 30.93, 30.89, 31.18, 30.64, 31.36, 30.39, 31.45, 30.84, 31.24, 31.25, 31.84, 29.98, 30.15, 31.05, 30.37, 30.73, 31.03, 30.96, 30.23, 31.03, 31.02, 30.17, 31.05, 31.16, 31.01, 30.72, 30.87, 30.92, 30.52, 30.64, 31.01, 31.14, 30.83, 31.16, 30.51, 30.33, 30.24, 30.03, 31.01, 30.56, 30.48, 29.09, 31.01, 30.71, 31.4, 30.05, 30.8, 30.76, 30.83, 30.37, 30.94, 30.31, 30.28, 29.81, 30.52, 30.24, 31.51, 30.25, 30.95, 31.1, 30.25, 30.81, 30.57, 31.04, 30.7, 30.84, 30.38, 31.11, 31.03, 30.13, 30.95, 30.05, 31.23, 31.5, 30.66, 30.29, 30.85, 30.7, 31.32, 31.04, 31.16, 30.94, 30.68, 30.81, 31.25, 30.24, 30.37, 30.85, 30.8, 31.0, 31.17, 30.83, 31.09], 'val_loss': [5.356332807783868, 5.192061740122024, 5.095441081721312, 4.995663282977548, 4.925497290435111, 4.83962340719381, 4.738721885498921, 4.666373624923123, 4.570971915676336, 4.472404721436227, 4.407846400692205, 4.3457416774360995, 4.24192982114804, 4.179638859572684, 4.1058580845024935, 4.0344001001613155, 3.9824038250431135, 3.89905290998471, 3.8584505266444697, 3.7989859823968, 3.7129664785543066, 3.6782135887510456, 3.6385968870418086, 3.586983067214869, 3.5322251403407687, 3.4885838783470686, 3.4681355710242205, 3.4299591043192867, 3.3922155510847736, 3.361798269733502, 3.3182390419540893, 3.311278425963821, 3.2490181429370955, 3.241410784660631, 3.205325010475839, 3.1916086240938513, 3.178281368723341, 3.1497041533707053, 3.141490145094076, 3.105941986582082, 3.109143219935666, 3.068635011174876, 3.044136199222249, 3.058889572787437, 3.029751467097337, 3.03971440017603, 3.0127516263609477, 3.004628985550753, 3.0014664344726856, 2.983823487713079, 2.9859450941632506, 2.9793249604049, 2.9700619680866316, 2.968323395510388, 2.95447327604719, 2.9627871353914785, 2.9814149088161006, 3.015204917853046, 2.978768895386131, 2.999742537547069, 2.986796666862099, 2.992452433154841, 2.998023493274762, 2.977419671738983, 3.0258700832439835, 3.0014296071544573, 2.9893543469678066, 3.018177989182199, 2.9899057992704354, 3.023008909954387, 3.034564989387609, 3.024556669459981, 3.047499829796469, 3.0684568175844325, 3.105117332403827, 3.0808402945281594, 3.1009451242009547, 3.1084401394910874, 3.1086212890163347, 3.1172480492075536, 3.1381071228889903, 3.0966363287275764, 3.1602278552996883, 3.1870155023161773, 3.1533801039313056, 3.159237883652851, 3.235585074515859, 3.2177573503202694, 3.2926414726646085, 3.2754442380492095, 3.185308856569278, 3.2263001112421605, 3.2865473715363036, 3.2826079699643858, 3.299290372307893, 3.2815848862289623, 3.334486390375028, 3.312978515199795, 3.318736846279946, 3.345581517098056, 3.3679902386513487, 3.344672886429319, 3.4233975516762705, 3.427643305177142, 3.4393208421719303, 3.4098313743141806, 3.4269369703948875, 3.448258672550226, 3.4268941362951972, 3.4518516496488245, 3.498538885146949, 3.5035145077735756, 3.5524192289182337, 3.509699853362551, 3.5418704193868455, 3.547555463329242, 3.535308032278802, 3.487035111257225, 3.610695025723451, 3.616329067831586, 3.5582735174021143, 3.656961914080723, 3.5952144147484164, 3.6173651408238015, 3.6470517056762795, 3.6555336933986395, 3.594847227357755, 3.6360104349768085, 3.6747344284300594, 3.618419238716174, 3.6595259821339017, 3.68261245709316, 3.6807307027707434, 3.6721350615191612, 3.728123631446984, 3.730845285069411, 3.7296361399304336, 3.7029978036880493, 3.701300987012827, 3.71934292726456, 3.731800220574543, 3.7157146581418954, 3.809556951948032, 3.7814541356578752, 3.8028911625503734, 3.737995588096084, 3.8160402364791577, 3.808307494327521, 3.877736038463131, 3.7981230886119186, 3.7839312204130136, 3.741679662352155, 3.9072394758273084, 3.8121060041864965, 3.808768790238982, 3.8112837956969146, 3.8404337737210996, 3.8123537727222323, 3.863842109206376, 3.8735750807318716, 3.8874418317891988, 3.88457230853427, 3.9143231454168914, 3.8383326279889247, 3.8955975391302897, 3.858070042482607, 3.8421464543433705, 3.894189299291866, 3.865830120766998, 3.889834788954182, 3.8645497864219034, 3.8971725808587045, 3.8845513109948224, 3.939491778422313, 3.8772708382576133, 3.907405954257698, 3.968602629983501, 3.8747084254671815, 3.994249031042597, 3.897728339122359, 3.909669030244183, 3.9494795875184856, 3.95303279199418, 3.987035609354639, 3.95780128734127, 3.9113303810168225, 3.922233686325656, 3.930612736446842, 3.9299469844550843, 3.9695771780742963, 3.9696301487600727, 3.984997694659385, 3.994368090751065, 4.002311940405779, 3.952192589735529, 3.9531284221418344, 3.939806037647709, 3.906975040010586, 4.007247940749879, 4.011596041879836]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="Adam")
data.update({'train_metric': [4.015, 10.203, 13.892, 17.088, 19.089, 20.881, 22.179, 23.234, 24.435, 25.215, 26.209, 26.715, 27.444, 28.033, 28.594, 29.114, 29.618, 30.035, 30.531, 30.746, 30.962, 31.25, 31.504, 31.853, 32.073, 32.353, 32.432, 32.72, 32.751, 32.874, 33.342, 33.348, 33.304, 33.591, 33.697, 33.773, 34.008, 34.258, 34.13, 34.281, 34.284, 34.315, 34.385, 34.448, 34.69, 34.83, 34.71, 34.886, 35.138, 35.12, 35.181, 35.183, 35.29, 35.313, 35.365, 35.322, 35.256, 35.632, 35.566, 35.514, 35.733, 35.716, 35.696, 35.856, 35.979, 35.729, 35.864, 35.784, 35.904, 35.99, 35.977, 36.251, 36.149, 36.119, 36.31, 36.224, 36.084, 36.277, 36.173, 36.234, 36.393, 36.299, 36.31, 36.366, 36.424, 36.457, 36.688, 36.575, 36.49, 36.626, 36.941, 36.709, 36.702, 36.809, 36.623, 36.584, 36.839, 36.851, 36.9, 36.733, 36.928, 37.04, 36.839, 36.938, 36.899, 36.86, 36.999, 36.99, 37.046, 36.847, 37.034, 37.126, 37.076, 37.21, 37.052, 37.1, 36.963, 37.197, 37.042, 37.224, 37.152, 37.171, 37.195, 37.033, 37.166, 37.308, 37.125, 37.214, 37.145, 37.378, 37.162, 37.278, 37.22, 37.296, 37.222, 37.232, 37.202, 37.317, 37.145, 37.307, 37.361, 37.416, 37.164, 37.361, 37.481, 37.244, 37.307, 37.281, 37.393, 37.342, 37.686, 37.471, 37.388, 37.501, 37.412, 37.491, 37.517, 37.529, 37.36, 37.487, 37.368, 37.682, 37.635, 37.636, 37.639, 37.509, 37.607, 37.653, 37.369, 37.538, 37.387, 37.721, 37.684, 37.536, 37.468, 37.638, 37.631, 37.616, 37.604, 37.764, 37.637, 37.826, 37.75, 37.636, 37.636, 37.657, 37.737, 37.655, 37.868, 37.698, 37.582, 37.651, 37.902, 37.753, 37.816, 37.743, 38.033, 37.907, 37.963, 38.008], 'train_loss': [4.770341612937278, 4.146053669930114, 3.8845577447443382, 3.679941456667216, 3.5456306220473803, 3.450421330567285, 3.3809392231058326, 3.318613016826559, 3.26425439199422, 3.215467156466962, 3.1752818965484755, 3.1357179087701703, 3.10054482104911, 3.0693675159874134, 3.04082202407998, 3.020194046785644, 2.999194038120204, 2.9734055902892322, 2.956679310771188, 2.938492654304968, 2.925274225510776, 2.9100374720909623, 2.895532482187487, 2.8849783524899473, 2.871148495512442, 2.8629775572036675, 2.850055436483958, 2.84235366216014, 2.8322553619389645, 2.8238243754674284, 2.8170130374869395, 2.8096717366482764, 2.7991074062049655, 2.796767863568326, 2.789405793466403, 2.7893373577242393, 2.7754838094448937, 2.7737425499365105, 2.768924793675399, 2.762869701580748, 2.7643370161587355, 2.756490176714008, 2.7551779211597114, 2.7488407934619614, 2.742096523253184, 2.737525107535657, 2.734949065566597, 2.7302155966035695, 2.730640512510362, 2.728695407252394, 2.7207966670727624, 2.7178459568658244, 2.7124406969554897, 2.7114478892724785, 2.7072107184795104, 2.7089309094429628, 2.7063403788546454, 2.70181806134781, 2.7038025116966233, 2.702566598671335, 2.6963145875503676, 2.695887765362716, 2.6941380442828606, 2.6872613082234094, 2.6866664895626955, 2.6870514895774122, 2.684896346474792, 2.6820866564337598, 2.678823057078278, 2.6780930334226642, 2.67701595644118, 2.672552466926404, 2.6701340562475084, 2.672859949708671, 2.667418393277237, 2.6695121548073613, 2.6621680384786632, 2.6663157799574937, 2.6666617867890183, 2.661337191297393, 2.6616219041138525, 2.661127735541863, 2.6609742631533932, 2.653895298334855, 2.6549019269735785, 2.653333113045549, 2.653323719917927, 2.650459251339742, 2.6536515016854763, 2.649849980745419, 2.6441410873764535, 2.6475659213002034, 2.6446159996440297, 2.645845325650577, 2.6454010343826204, 2.6392925922030144, 2.6424727035651814, 2.6391951879170636, 2.6406206794831513, 2.6418038901806793, 2.6337625629537316, 2.6350150756628485, 2.6396659569182, 2.6371250970960043, 2.634499517374899, 2.6384628870055526, 2.6320617475039825, 2.6326719218160735, 2.62964575708637, 2.630310139286922, 2.631108188888467, 2.6336007747448793, 2.628214591295385, 2.624916133099615, 2.6256820603356634, 2.625336087825431, 2.628093331117929, 2.624540623532452, 2.62431252506095, 2.6246196038434695, 2.626681756912251, 2.624544072669817, 2.6233319408681557, 2.628111790558198, 2.623638668963334, 2.6185076946031565, 2.623271260060184, 2.625018814017356, 2.624493499253701, 2.618555591416069, 2.6245509534788223, 2.619818795436632, 2.6175183073038943, 2.6205406310996104, 2.617279055022461, 2.6127251959426694, 2.617719607710152, 2.6171792692964226, 2.617120126539976, 2.6138354327841897, 2.615055631996345, 2.6113610426279803, 2.6189695399919537, 2.609943491331065, 2.6116650409600863, 2.614430318409559, 2.6135153399776816, 2.6137055879362254, 2.6139145355840867, 2.6120588649028553, 2.6049184268358343, 2.6075056402330894, 2.61095982748045, 2.612757875838496, 2.6069870293331086, 2.607918389165394, 2.605520152084658, 2.6066600947675993, 2.6057099630950127, 2.6063887486454775, 2.6073000590464126, 2.604440964114872, 2.604265875261103, 2.6034542336650026, 2.6076693894080605, 2.6037735705641087, 2.6006595031313613, 2.6015569364422038, 2.6039705486993188, 2.599931737130373, 2.603326214786073, 2.602565885352844, 2.603546397242116, 2.596020086255504, 2.6010026407790963, 2.6002725684635166, 2.595885612990562, 2.595441283740375, 2.603022871685577, 2.5993925152264263, 2.59678522623737, 2.593753455086389, 2.5944445043973867, 2.597506968470163, 2.5958584006291816, 2.5987579075861498, 2.5952008613316737, 2.595679114815217, 2.5909681360613286, 2.5973494533385058, 2.595350896938444, 2.5967546946256497, 2.5895825555823357, 2.5965477078676376, 2.5929246277513216, 2.5933169590992113, 2.5872068860480515, 2.5897749507572128, 2.594257759269963, 2.593967669679809], 'val_metric': [7.98, 11.22, 16.17, 17.59, 20.32, 22.61, 23.16, 23.87, 25.83, 26.85, 27.45, 27.75, 27.78, 27.7, 29.21, 29.02, 30.57, 31.74, 31.94, 31.63, 32.14, 31.79, 32.47, 32.95, 33.1, 32.74, 32.49, 32.98, 33.9, 33.48, 34.49, 34.0, 34.79, 35.75, 33.97, 33.24, 34.91, 33.99, 35.33, 35.24, 35.21, 35.27, 35.23, 35.77, 36.14, 33.94, 34.92, 35.49, 35.2, 36.23, 35.5, 35.43, 36.19, 36.02, 34.81, 36.23, 35.29, 35.87, 36.26, 36.86, 36.62, 37.13, 35.62, 36.05, 35.84, 36.77, 37.51, 37.23, 35.77, 37.15, 36.82, 37.43, 36.79, 36.83, 36.9, 37.35, 36.38, 37.31, 36.18, 37.02, 36.67, 36.28, 36.31, 37.5, 37.65, 37.71, 38.11, 37.46, 36.61, 38.37, 38.11, 37.25, 37.51, 37.04, 36.38, 38.22, 36.64, 36.92, 37.27, 37.72, 37.4, 36.63, 37.74, 37.58, 37.77, 37.4, 38.04, 37.91, 37.76, 37.74, 38.82, 37.32, 38.27, 38.61, 38.01, 37.17, 38.29, 37.17, 37.42, 38.35, 38.02, 38.19, 36.81, 37.78, 38.32, 37.69, 38.08, 37.84, 37.52, 37.55, 37.16, 37.26, 37.19, 37.73, 37.44, 36.26, 37.56, 38.27, 38.61, 38.1, 38.8, 37.91, 38.51, 37.94, 38.23, 37.83, 36.97, 37.96, 38.91, 38.3, 37.74, 37.72, 38.14, 37.84, 38.22, 37.93, 38.34, 38.63, 37.84, 38.08, 37.8, 37.92, 38.02, 36.8, 38.81, 38.42, 37.63, 38.14, 38.71, 37.26, 37.26, 38.98, 37.94, 38.49, 38.38, 38.11, 38.61, 38.48, 38.29, 37.06, 38.73, 38.19, 38.47, 37.58, 38.39, 37.52, 38.42, 38.15, 37.89, 37.5, 38.4, 38.72, 38.56, 38.03, 38.35, 37.98, 38.95, 37.41, 38.11, 38.78], 'val_loss': [4.356769118339392, 4.010743856430054, 3.714329402158215, 3.6363737112397603, 3.4527897758848347, 3.3647852057863954, 3.3124321030962998, 3.2647619429667283, 3.1822503964612436, 3.126559389624626, 3.1115799086868385, 3.052140586695094, 3.069932465340681, 3.063101453386295, 3.013431284837662, 3.0280492351313306, 2.955326229144054, 2.891694717346483, 2.891661003896385, 2.88209129822482, 2.8588559255478487, 2.873177132788737, 2.8392532539975113, 2.853076499738511, 2.8288423453166986, 2.840620499507637, 2.8148629893163206, 2.8283146338857663, 2.7652062253587566, 2.775871102976951, 2.77819498557194, 2.7877969058455934, 2.7456167222587924, 2.727663460051178, 2.7721979686409046, 2.81237083255865, 2.750592767053349, 2.760067817511832, 2.699792404083689, 2.7207981606197964, 2.704165252910298, 2.730619389919718, 2.749965726190312, 2.699832266683032, 2.696218939343835, 2.7495973208907305, 2.7246475493072704, 2.693536054556537, 2.737812203966129, 2.6769964527931944, 2.673890631669646, 2.7200475824866324, 2.662043904802602, 2.680614743263099, 2.744821188556161, 2.6689688654462245, 2.714502218802264, 2.677761992831139, 2.679373432116903, 2.662483123457356, 2.6432734917683205, 2.63408824137062, 2.6939819284305453, 2.6773159147068193, 2.6856803149934025, 2.659240656597599, 2.6083989037070303, 2.625386695193637, 2.6828059967915725, 2.63073187117364, 2.6335315749903394, 2.641507321102604, 2.644073602500235, 2.6565959408025073, 2.6313482029422834, 2.622447754926742, 2.6525088298092983, 2.612504014543667, 2.6531017358136024, 2.63884343387215, 2.656976219954764, 2.662891518917813, 2.647417454203223, 2.621438876838441, 2.6088751994880144, 2.611579094722772, 2.603185689373381, 2.6145319247701364, 2.613546681252255, 2.5961021693648805, 2.589490125893028, 2.6383767583567623, 2.621063775317684, 2.635402554159711, 2.644845551745907, 2.576887734376701, 2.6628418018104165, 2.645344267984864, 2.6359639342423455, 2.625218241837374, 2.610733274441616, 2.6762676106137073, 2.596734684743699, 2.608525305037286, 2.6237569189375374, 2.6110861886079144, 2.585168019981141, 2.596189255167724, 2.5797451696578104, 2.6240559375969466, 2.5892167007847195, 2.6388642620888487, 2.5951821842011373, 2.5711059919588126, 2.6266619141694085, 2.63425757626819, 2.621398163449233, 2.645984972358509, 2.620841267761911, 2.5921434429800434, 2.590543087121028, 2.5962679818936976, 2.6612830218995454, 2.603641948882182, 2.574469391707402, 2.608227126157967, 2.5901196021942576, 2.618454849644072, 2.598100619710934, 2.6289475078036073, 2.625516808716355, 2.6200138102671144, 2.632409608288176, 2.6208250902260946, 2.6274222416482913, 2.6880059583931213, 2.6250979915546004, 2.6141540058858834, 2.5892406033862168, 2.60568879136614, 2.579941955341655, 2.603104842316573, 2.5887272502206695, 2.598685379240923, 2.5867324718244515, 2.6088063257515053, 2.6400828763937496, 2.625843443688314, 2.5709114211380104, 2.596242858346101, 2.604689336126777, 2.6190843043053986, 2.5843788392984184, 2.602577622529048, 2.588864909615486, 2.6025752504919746, 2.597327756274278, 2.584630692081087, 2.5955881601685933, 2.599039818071256, 2.620215692337911, 2.5891569154277727, 2.603764871882785, 2.6739492970667067, 2.5772985204769547, 2.5763613722126957, 2.628732221901037, 2.584994095905571, 2.5663372255434656, 2.5995273893805826, 2.604531719426441, 2.5558241567793925, 2.576587100697171, 2.58597881626931, 2.572832182714134, 2.594279200028462, 2.5682226518157183, 2.586294759610656, 2.5950750932571993, 2.6190200322752544, 2.572790606386343, 2.5938337145337633, 2.581459187398291, 2.6001693503871843, 2.582497324153876, 2.6076830959623787, 2.598154550145386, 2.5787840800680173, 2.598633731246754, 2.6070037571487914, 2.5868282606647273, 2.5679176924334968, 2.586469752773358, 2.5864057715531366, 2.6035605304560083, 2.5793870565997565, 2.5437454683765486, 2.6344802208766818, 2.5871952521573207, 2.5829659897810333]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Sinusoidal", optimizer="AdamW")
data.update({'train_metric': [4.167, 14.557, 23.765, 31.479, 38.252, 44.212, 49.549, 54.609, 59.217, 63.796, 68.899, 73.2, 77.227, 80.675, 83.62, 85.876, 87.42, 88.872, 89.583, 90.702, 91.365, 91.928, 92.128, 92.777, 93.236, 93.45, 93.817, 94.313, 94.424, 94.563, 94.8, 94.958, 95.278, 95.314, 95.499, 95.445, 95.715, 95.896, 96.057, 96.005, 96.194, 96.343, 96.373, 96.409, 96.545, 96.57, 96.608, 96.662, 96.86, 96.89, 96.95, 96.995, 97.065, 97.095, 97.007, 97.201, 97.206, 97.291, 97.4, 97.317, 97.361, 97.493, 97.367, 97.449, 97.54, 97.638, 97.541, 97.718, 97.673, 97.686, 97.719, 97.769, 97.816, 97.757, 97.782, 97.923, 97.883, 97.988, 97.942, 97.935, 98.072, 98.017, 98.077, 98.078, 97.981, 98.017, 98.133, 98.088, 98.006, 98.201, 98.16, 98.132, 98.108, 98.273, 98.248, 98.184, 98.25, 98.143, 98.276, 98.199, 98.355, 98.282, 98.254, 98.284, 98.422, 98.262, 98.378, 98.362, 98.35, 98.362, 98.391, 98.4, 98.373, 98.322, 98.436, 98.462, 98.454, 98.505, 98.51, 98.397, 98.499, 98.543, 98.521, 98.598, 98.551, 98.553, 98.558, 98.575, 98.563, 98.564, 98.594, 98.571, 98.66, 98.502, 98.555, 98.703, 98.627, 98.623, 98.625, 98.681, 98.63, 98.653, 98.673, 98.663, 98.576, 98.728, 98.675, 98.7, 98.716, 98.647, 98.673, 98.769, 98.746, 98.742, 98.693, 98.677, 98.784, 98.785, 98.746, 98.736, 98.771, 98.841, 98.721, 98.754, 98.865, 98.759, 98.758, 98.774, 98.927, 98.819, 98.825, 98.815, 98.746, 98.831, 98.732, 98.862, 98.816, 98.833, 98.82, 98.841, 98.831, 98.783, 98.894, 98.769, 98.718, 98.954, 98.849, 98.848, 98.836, 98.942, 98.858, 98.937, 98.866, 98.901, 98.955, 98.802, 98.876, 98.945, 98.863, 98.94], 'train_loss': [4.785273050850046, 3.855740832580791, 3.3209457864230516, 2.9050391130697553, 2.56538260600846, 2.2728820626581623, 2.011774248369062, 1.7759173022427013, 1.563171933762019, 1.3474754908873496, 1.1363832658479707, 0.951920682393963, 0.7882671427703865, 0.6503214136919606, 0.5432986285697171, 0.4645777647112397, 0.40864649448860785, 0.3571348163994626, 0.33178048020780504, 0.2986858984072927, 0.2775610134587102, 0.2579885684907131, 0.2493252305818039, 0.22866766207201392, 0.2153065710883261, 0.20836346130877714, 0.19792840811313725, 0.18374797691110228, 0.1788246451197186, 0.17271741495187073, 0.16593831115100183, 0.16162147292222587, 0.15058025419926574, 0.1500381839080873, 0.1426403331162166, 0.1438122339325022, 0.13670861642564613, 0.13295966182416513, 0.12621441571803446, 0.12654801015101577, 0.12126055217073349, 0.11732230628664848, 0.11724991801177567, 0.1149320901882187, 0.11083331406033066, 0.10880045399906367, 0.10773784088975466, 0.10495436995010764, 0.09944705516505112, 0.09938586727160095, 0.09615284822981325, 0.09661388418786776, 0.09338030761604746, 0.09206770566501848, 0.09363652630104317, 0.08965615969287113, 0.08824844606371474, 0.0865173405070592, 0.0821199995249281, 0.08479793890488412, 0.08328657739683128, 0.0796661731332388, 0.08345366661035845, 0.07937984047406928, 0.07871911030602942, 0.07522895432661883, 0.07829125130341119, 0.07506517223731965, 0.07566410584508734, 0.07340628980435204, 0.0713869697834119, 0.07097659651273462, 0.06935199150857019, 0.07140699219523487, 0.07043930251861584, 0.06758557276094782, 0.06835997650091151, 0.06417082114084383, 0.06644105309790617, 0.06729185449925353, 0.06186277726901992, 0.06355756330819264, 0.06189249614716918, 0.06165550332394281, 0.06437037446944582, 0.06436027558529374, 0.05986783989731661, 0.0605155619999998, 0.06322851368312211, 0.057535098211899977, 0.056481368986470236, 0.05868588612789988, 0.060488551456757696, 0.05575004788105193, 0.055982719192052555, 0.056519612136653845, 0.05589924661445766, 0.058833723150876545, 0.05481524885742629, 0.056603934067067284, 0.05174775709913059, 0.05410035734269852, 0.05605256909596742, 0.054385464345534006, 0.04958006593803038, 0.055200483494078335, 0.05107456552784149, 0.052151716187562355, 0.053136170229102025, 0.05237745344619459, 0.051685108568482706, 0.04940154208275546, 0.05099573689719968, 0.05223220051124737, 0.04826416255464844, 0.0500889382578277, 0.04954991851061415, 0.046584671098600074, 0.048441229032242926, 0.050380972163255745, 0.047966347507063196, 0.04707456913591087, 0.0462903077586781, 0.04594986375539861, 0.04613949301518096, 0.04704784597651539, 0.04568873434254551, 0.04540481776231006, 0.0454699030384792, 0.04555127585425339, 0.044926461111329634, 0.04541011243810286, 0.0419055810507288, 0.04617573444686323, 0.045027031830947854, 0.041818560907619894, 0.043804727894349665, 0.04335541056484128, 0.0442989811842194, 0.04297010680620087, 0.04388234381652893, 0.042136039454122004, 0.04267429196384812, 0.041315897964532486, 0.044216260376776034, 0.04001629865549264, 0.04199596622515463, 0.04119173656173809, 0.040544601890545394, 0.04201373993039253, 0.04099550952924706, 0.0397966529910388, 0.03966553149280771, 0.03891101271284328, 0.041572995307110935, 0.04079463745724551, 0.038708424381014246, 0.039230506336970415, 0.03886844595473469, 0.0394787061608695, 0.03938773571113178, 0.03673624400516375, 0.04081434346311675, 0.03940653332469029, 0.03641914253300967, 0.039087151822483436, 0.040452945354836, 0.03909933592596297, 0.034335283417487365, 0.03805105614401647, 0.0369693136140013, 0.037577679535250104, 0.039646902580561395, 0.036433648828551726, 0.03996367752248518, 0.036029339795076006, 0.037426457987702465, 0.03636673362066499, 0.03749152747691494, 0.03729552181607207, 0.03682636060810145, 0.038332584493121574, 0.03477724392356003, 0.03775550258823778, 0.03975104955145678, 0.03366365969587434, 0.036108214472418576, 0.036376300157865325, 0.03700796336566306, 0.03397508473682601, 0.036057122876403516, 0.033932250013382195, 0.036973277903596176, 0.03412540076146026, 0.03486757370121869, 0.03904160264984224, 0.03561852554095312, 0.033194367394410416, 0.035191138910857504, 0.03305457003536372], 'val_metric': [8.97, 20.22, 27.56, 34.88, 40.87, 44.01, 46.53, 49.36, 50.3, 51.09, 51.78, 52.12, 51.68, 51.87, 52.0, 52.26, 52.24, 52.56, 52.46, 52.53, 51.18, 51.73, 51.65, 52.38, 52.79, 51.81, 52.23, 52.01, 52.0, 52.75, 52.53, 53.06, 52.54, 52.81, 52.86, 53.02, 52.56, 52.17, 53.07, 52.76, 52.69, 52.84, 53.04, 52.91, 53.37, 53.23, 53.04, 53.15, 54.01, 53.0, 53.51, 53.38, 53.19, 53.25, 53.88, 53.69, 53.55, 54.08, 54.08, 53.21, 53.58, 53.73, 53.62, 53.43, 53.63, 53.87, 53.72, 53.01, 53.37, 53.53, 53.33, 53.9, 54.36, 53.79, 54.17, 53.66, 53.74, 54.16, 53.87, 54.12, 53.96, 53.99, 53.98, 54.04, 54.02, 53.47, 53.98, 54.11, 53.7, 54.38, 54.07, 54.2, 53.6, 53.35, 54.32, 54.72, 54.66, 54.14, 54.12, 54.45, 54.08, 53.72, 54.53, 53.35, 54.47, 54.22, 54.91, 53.85, 54.76, 54.34, 54.61, 54.93, 54.19, 54.56, 54.05, 54.71, 53.62, 54.55, 54.16, 53.99, 54.7, 53.95, 54.14, 54.57, 53.95, 53.81, 54.32, 54.27, 53.64, 54.36, 54.94, 54.04, 54.37, 54.39, 54.18, 53.98, 54.3, 54.3, 54.17, 54.24, 53.94, 54.14, 54.92, 55.24, 54.9, 54.77, 54.25, 54.89, 54.97, 54.95, 54.78, 54.91, 54.98, 55.2, 54.68, 54.56, 54.69, 54.44, 54.35, 54.56, 54.87, 54.05, 54.39, 54.43, 54.48, 55.11, 54.9, 54.64, 54.7, 54.92, 54.2, 54.25, 54.49, 54.6, 54.8, 55.33, 55.05, 54.98, 54.93, 54.63, 54.67, 55.07, 54.07, 54.65, 54.45, 55.09, 55.24, 55.45, 55.45, 55.04, 54.96, 54.85, 54.98, 55.24, 54.96, 54.48, 55.14, 55.39, 54.97, 55.15], 'val_loss': [4.211801671678094, 3.509587667550251, 3.0887891327499584, 2.7503626354181083, 2.4608882809900177, 2.343359818883762, 2.195896929996029, 2.079314438780402, 2.0658667406458764, 2.0801390827081767, 2.077275511945129, 2.1492879910833516, 2.261517984282439, 2.3207158920871223, 2.430334742281847, 2.4778229666363663, 2.5954912190984007, 2.6319622059536587, 2.6512574128284574, 2.7028606179033874, 2.815114105962644, 2.8124602205434424, 2.891006300783461, 2.9456462962612227, 2.8138002115450087, 2.9218466741264244, 2.9472246925542307, 2.992782420793157, 3.0034947516811883, 3.030753428009665, 3.0568481987449014, 2.9957333104625627, 3.0714604133253642, 3.0825272650475712, 3.0833793430571346, 3.0492588691650684, 3.0514675002948493, 3.127967357635498, 3.1281289868294055, 3.122219122708983, 3.186896431218287, 3.1607604470981916, 3.169911692096929, 3.1974977968604703, 3.1723698487707006, 3.236308690089329, 3.181080656446469, 3.244599884862353, 3.2084962850922993, 3.2461626248754514, 3.1958764366283536, 3.2381192817809477, 3.237413501283925, 3.276127566197875, 3.238897124673151, 3.278609439825556, 3.299228685486848, 3.245044076518648, 3.288767625951463, 3.28301178687697, 3.265779063200495, 3.3172059484348178, 3.3000518979540296, 3.3042071399035726, 3.342567266931959, 3.3320355050882715, 3.3119179090117195, 3.3490197806601314, 3.3892890461690866, 3.358611353643381, 3.332951480415976, 3.3370014300012283, 3.3596112853402547, 3.3448031669969014, 3.370338799467512, 3.37169763692625, 3.3655073080852533, 3.374615008284332, 3.3932092603604507, 3.3537833580545557, 3.440870516619105, 3.4313837304996078, 3.4576215189733324, 3.4168853968571704, 3.3782027792778746, 3.41966386965126, 3.459656407119362, 3.4440510546325878, 3.505295054168458, 3.4648574689391314, 3.467712452457209, 3.5425995607284984, 3.43922699560785, 3.4956885944506166, 3.5157765727134267, 3.493424817634996, 3.4773450288802956, 3.426710427946346, 3.4547386617417546, 3.4486714836898122, 3.5083392072635093, 3.515473201396359, 3.439913261088596, 3.4667357604974396, 3.447716803307746, 3.523412430362337, 3.4873260115362275, 3.530783771709272, 3.4994157416046043, 3.5262683060518496, 3.505588002645286, 3.5067013494528023, 3.548945673332093, 3.554642265010032, 3.530797609857693, 3.5134611266433815, 3.5583831386960996, 3.4780796538492678, 3.5659432935107285, 3.5806483409966634, 3.4961968759062945, 3.5807286333885924, 3.5262252468212396, 3.5531887541151352, 3.569448721256985, 3.5910073256796333, 3.5105386738944206, 3.5948616938226543, 3.619864007849602, 3.5826923247355564, 3.5232232871708598, 3.5437265292854065, 3.5600555880813842, 3.5118651166083708, 3.619697431849826, 3.6167127865894586, 3.607229839844309, 3.6119948815388283, 3.603032796246231, 3.606164231042194, 3.6087105331147553, 3.5780204215626807, 3.5819144461564956, 3.558823097283673, 3.577212518187845, 3.556891109913018, 3.616792123408834, 3.565280850525874, 3.6239586476307766, 3.562388151314608, 3.6353978480503057, 3.626067175227366, 3.563656004371157, 3.6070729957264698, 3.5928639112764102, 3.6076722786684705, 3.628207964122675, 3.6158861984872512, 3.6362223993441103, 3.623279651259161, 3.6421560926042544, 3.6899262801477106, 3.6508314165340106, 3.588821941879904, 3.645878640329762, 3.6250665024587305, 3.570497700363208, 3.636806553716113, 3.6887772884338523, 3.6646429816628716, 3.6064059453405393, 3.694181950608636, 3.703108782980852, 3.6773799430033205, 3.6024326689683708, 3.6310524082487556, 3.6204523702335965, 3.7075685695478113, 3.7263351299200846, 3.678264105395906, 3.669631314885085, 3.6376846778165004, 3.643203693210699, 3.6216631558290713, 3.6140763900082584, 3.6279366726328615, 3.6550454744108163, 3.590861914643816, 3.5991659984467135, 3.6181236163825745, 3.6275871304949376, 3.6594748956382652, 3.6267844074091333, 3.620902118409515, 3.631892796154994, 3.662475748046948, 3.665931495891255, 3.6516461034489285, 3.6603956127622324, 3.6936881063850064]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="SGD")
data.update({'train_metric': [0.784, 1.081, 1.413, 1.64, 2.057, 2.226, 2.792, 3.217, 3.702, 4.207, 4.771, 5.145, 5.629, 6.153, 6.737, 7.426, 8.078, 8.777, 9.524, 10.254, 10.8, 11.338, 12.097, 12.66, 13.091, 13.866, 14.36, 14.921, 15.35, 15.948, 16.465, 17.147, 17.534, 17.942, 18.547, 19.089, 19.696, 20.115, 20.454, 20.979, 21.56, 22.116, 22.582, 23.138, 23.556, 24.13, 24.578, 24.984, 25.763, 26.051, 26.813, 27.18, 27.552, 28.234, 28.709, 29.23, 29.742, 30.352, 30.818, 31.41, 32.118, 32.494, 33.054, 33.598, 34.135, 34.781, 35.303, 35.807, 36.468, 37.004, 37.768, 38.186, 39.019, 39.701, 40.302, 40.821, 41.52, 42.059, 42.908, 43.285, 44.001, 44.595, 45.246, 46.053, 46.739, 47.368, 47.973, 48.787, 49.329, 50.301, 50.594, 51.417, 52.118, 52.795, 53.484, 53.904, 54.936, 55.468, 56.398, 57.054, 57.206, 58.164, 58.602, 59.331, 59.969, 60.412, 61.424, 61.559, 62.313, 63.143, 63.683, 64.238, 64.74, 65.275, 66.097, 66.381, 67.025, 67.386, 67.685, 68.573, 68.774, 69.384, 69.679, 70.139, 70.735, 71.248, 71.623, 72.157, 72.22, 72.596, 72.886, 73.367, 73.88, 74.127, 74.718, 75.061, 75.361, 75.572, 76.04, 76.27, 76.852, 76.623, 77.091, 77.295, 77.738, 78.145, 78.267, 78.449, 78.861, 79.167, 79.291, 79.431, 79.444, 79.499, 80.186, 80.298, 80.648, 80.874, 80.93, 81.127, 81.304, 81.828, 81.503, 81.995, 82.126, 82.555, 82.494, 82.811, 82.821, 83.042, 83.08, 83.377, 83.488, 83.74, 83.928, 84.059, 83.672, 84.293, 84.383, 84.553, 84.768, 84.571, 84.846, 84.933, 85.138, 85.465, 85.362, 85.426, 85.698, 85.563, 85.95, 86.083, 86.09, 86.054, 86.351, 86.727, 86.647, 86.585, 86.55, 87.086], 'train_loss': [5.299674318451494, 5.246723755917637, 5.2153887626687, 5.186455781232525, 5.14583006930214, 5.092305974173225, 5.027917039752846, 4.9679733591818005, 4.9089983417220555, 4.851864562458665, 4.791507822538292, 4.7348556854140655, 4.67690114675999, 4.61881270442189, 4.554624088589991, 4.4997813327908895, 4.4358149639551865, 4.377412438773987, 4.325325595821544, 4.273803091323765, 4.222904504909015, 4.176645114035921, 4.132863334563018, 4.086470533317278, 4.046665610827778, 4.0013327659587405, 3.9603541487695617, 3.926343826933389, 3.8882212895120634, 3.849613963680548, 3.8158930460001823, 3.7748877022865868, 3.7432068204239095, 3.7101955287210924, 3.673009252822788, 3.642112226419089, 3.608059409331299, 3.574609532847438, 3.5501156725795013, 3.514043464007777, 3.485617864917504, 3.4497135733459823, 3.421673982782541, 3.393747339626954, 3.3613968894638773, 3.331236041972672, 3.3004709783457673, 3.274698417612321, 3.243849079348075, 3.2127527687233837, 3.1816214360415898, 3.1588816555813954, 3.1248661936359077, 3.0976445507103256, 3.070231856555414, 3.0389384279171776, 3.0106512171598254, 2.979780857637763, 2.9553953710459626, 2.925972585257093, 2.894109612157996, 2.8668954604265404, 2.8409667989800393, 2.8080021993364954, 2.7786995769691103, 2.7455043858316412, 2.7216114877357898, 2.6910659627737483, 2.6574331419024038, 2.628502053323649, 2.593069937163565, 2.5659040355255263, 2.5333501455193517, 2.4996146338914955, 2.471656395469197, 2.4416659701274743, 2.4089527715877015, 2.3828583824581164, 2.3452072837607494, 2.3240634421088036, 2.28869676719624, 2.2614630859628826, 2.2296888679551987, 2.1928732151109593, 2.162492946867598, 2.140060691137918, 2.1032422654383165, 2.072117013879411, 2.042920819857299, 2.008917793965233, 1.9858676338531387, 1.9501228224216787, 1.9239980131864396, 1.8924978325478328, 1.8617380076467571, 1.8388708119810353, 1.8050967362624135, 1.7777515070147036, 1.7477495948702422, 1.7235403455226603, 1.6988788300878483, 1.6675710355480435, 1.6464846906796222, 1.616243210650375, 1.5953218751585185, 1.5659441509387162, 1.5388932321899913, 1.524122654095111, 1.493647421313949, 1.4689239879792422, 1.4462949177506483, 1.4250862383720437, 1.395659703439577, 1.3773707351620503, 1.352035432996768, 1.334703728623369, 1.313234230416445, 1.2956892118878038, 1.2781618920489144, 1.2547449877989727, 1.2372121488674284, 1.2194187550383047, 1.2014446418329605, 1.1857942162762067, 1.1675078445188678, 1.1472148342690862, 1.1321276068992516, 1.1158452104164558, 1.1045793263254757, 1.0901150505510722, 1.0773976193889927, 1.057985539399731, 1.0451805448577867, 1.032993924175404, 1.0127521670558708, 1.0000303659161465, 0.9847180974918226, 0.9776906242602464, 0.958475054778583, 0.9481535619142646, 0.9294680476150525, 0.9349971630027183, 0.9164849709259418, 0.8999282915242879, 0.8926239876089688, 0.8781196038576554, 0.8678339008558887, 0.86297726474812, 0.8484465336464035, 0.8373628514017422, 0.8307683023976273, 0.8239458337740797, 0.8169975210021722, 0.8145753559933514, 0.7933825964891064, 0.7903647070234583, 0.7748054431862199, 0.7701173974052119, 0.7654030574908717, 0.7534051949559918, 0.7485341076773097, 0.7276198361557565, 0.7386081530661898, 0.7196792648987211, 0.7142249408701788, 0.7011618715825938, 0.7022471905555469, 0.6895607140723208, 0.6855669066796147, 0.6809135901378808, 0.6769164497663177, 0.6665920037454469, 0.659693298169, 0.6514440634486314, 0.6436228603315293, 0.6397003525545104, 0.6439612858278662, 0.6293865202217627, 0.6235808911646931, 0.618285558807949, 0.6097369270586311, 0.6165728802510888, 0.6063983229609231, 0.5995135781899219, 0.5955877076889259, 0.5849810275830143, 0.5842435135722389, 0.5832848202339442, 0.5698553789561938, 0.5789648095766703, 0.5583892025515885, 0.5573801386074156, 0.5546907434098169, 0.5574134268100187, 0.5478614301004245, 0.5331957558562034, 0.5328796706993612, 0.5377060948124469, 0.5366620624858602, 0.52333641873097], 'val_metric': [1.11, 1.64, 1.82, 2.14, 2.56, 3.02, 3.67, 4.04, 4.48, 5.04, 5.74, 6.19, 6.72, 7.39, 7.74, 8.94, 9.85, 9.97, 11.57, 11.74, 12.62, 12.53, 13.6, 14.82, 14.78, 14.94, 15.68, 16.42, 16.91, 18.02, 17.88, 18.28, 19.08, 19.34, 19.93, 20.1, 20.09, 20.73, 21.05, 22.07, 21.9, 22.69, 22.97, 22.77, 23.45, 23.48, 24.57, 25.06, 25.41, 24.91, 26.03, 25.78, 26.93, 27.12, 26.74, 26.7, 27.47, 27.37, 27.51, 28.03, 27.8, 28.08, 29.15, 29.01, 29.35, 29.71, 29.73, 29.71, 29.9, 29.65, 29.91, 30.05, 30.55, 29.99, 29.86, 30.17, 29.97, 29.87, 30.41, 30.85, 30.68, 30.8, 30.48, 30.99, 29.76, 29.78, 30.9, 30.08, 29.96, 30.44, 30.33, 30.44, 30.54, 30.02, 30.46, 29.79, 29.75, 30.57, 30.85, 30.26, 30.3, 30.15, 29.78, 30.01, 30.13, 30.23, 29.73, 30.42, 30.11, 29.33, 29.61, 29.83, 30.07, 30.04, 29.44, 29.81, 27.44, 29.54, 29.72, 29.38, 29.18, 29.05, 29.92, 29.7, 29.5, 29.92, 30.0, 29.63, 29.57, 29.18, 30.57, 29.42, 29.65, 29.04, 30.53, 29.13, 29.97, 30.17, 29.6, 29.81, 30.32, 30.42, 29.36, 30.25, 29.62, 29.4, 26.8, 29.34, 29.77, 29.43, 29.69, 29.77, 29.77, 30.4, 29.37, 29.35, 30.15, 29.74, 30.36, 29.9, 29.03, 29.4, 29.28, 30.64, 29.67, 30.44, 30.94, 30.51, 30.47, 30.32, 30.26, 30.29, 29.73, 30.14, 30.84, 30.24, 29.8, 29.53, 30.51, 29.82, 29.3, 29.98, 30.46, 29.81, 30.89, 29.47, 29.7, 30.68, 30.37, 29.92, 29.92, 30.57, 30.04, 30.43, 30.9, 30.1, 30.42, 29.6, 30.6, 29.92], 'val_loss': [5.242962132593629, 5.211516245155577, 5.18895200711147, 5.158084532257858, 5.09813061793139, 5.032100053349877, 4.950751394223255, 4.867291524911383, 4.811401386929166, 4.762750888326366, 4.681279259882156, 4.63556854132634, 4.574880167177528, 4.492929340168169, 4.432671030615546, 4.366041479596666, 4.290415309796668, 4.261530908050051, 4.180412482304178, 4.147683848241332, 4.083894513974524, 4.082922920299943, 4.014461524926933, 3.9404723127936103, 3.934915993623673, 3.9191924963787104, 3.862673278067522, 3.806672908697918, 3.779537464403043, 3.71796471677768, 3.6964310111513563, 3.67546965514019, 3.6434708575534214, 3.6011190961120993, 3.578589771203934, 3.548635766764355, 3.535281393178709, 3.5207812224224115, 3.491547254240437, 3.4401801900499187, 3.4399171680401843, 3.396967663127146, 3.3631341282729132, 3.3720360037627493, 3.3532384770691017, 3.366325505220207, 3.3099553979885807, 3.2633606886408133, 3.2571733149753253, 3.279025369389042, 3.2191202663312293, 3.21444821661445, 3.1639203858223692, 3.1554127164707064, 3.160278230715709, 3.172578210283996, 3.129536616574427, 3.140130572258287, 3.1455441591845954, 3.112434755465028, 3.107589634360781, 3.1100704282712024, 3.0639486024334173, 3.0457099835584116, 3.035588915940303, 3.0182298133327703, 3.027783300466598, 3.011355110794116, 3.031523390180746, 3.0427154249446406, 3.0185720419428153, 3.0195534829121486, 2.9895193280687757, 3.0258882114082386, 3.0158854996322826, 3.0096733904188606, 3.010453180902323, 3.0513416126275517, 3.029384001804765, 3.0277327466162904, 3.034157686172777, 3.0424903418607774, 3.0501491818458413, 3.0334604781144745, 3.1017813067527333, 3.13608513022684, 3.0717594661530416, 3.0860081113827458, 3.1175049619310222, 3.1316052880256797, 3.117693638345998, 3.0974465965465376, 3.0956773507367275, 3.1430367011173517, 3.1291367772278513, 3.167028735397728, 3.2208921294303456, 3.1613501811483102, 3.159415334652943, 3.189346216286823, 3.2058545070089353, 3.2306116538442624, 3.2233940271814916, 3.2271366339580267, 3.229851339273392, 3.2254244377658625, 3.301316706238279, 3.2598433760321064, 3.291443754912941, 3.336199951779311, 3.3462949884924917, 3.321195346534632, 3.3568950314430674, 3.3519685678421314, 3.4018245145773434, 3.3534502117497147, 3.6320072351747257, 3.429528260686595, 3.4049963784065977, 3.44261767757926, 3.5281660123995153, 3.5025575495069954, 3.4432728768913607, 3.4615960083190043, 3.464149133414979, 3.4665620440890077, 3.529595154865532, 3.5440579607228564, 3.528975515608575, 3.5534700314710093, 3.4837239851617507, 3.634518378099818, 3.518041707907513, 3.691923802825296, 3.5819817128454803, 3.646782599437009, 3.591616503751961, 3.57310271490911, 3.618894770646551, 3.647652730820285, 3.5959182301903985, 3.608444839526134, 3.7601291973879385, 3.683157210137434, 3.692694001896366, 3.7419341665923973, 4.04062765646892, 3.7696074535892268, 3.7117795519008756, 3.7030743573121963, 3.6912832487920286, 3.736492380974399, 3.7292794376421887, 3.711775598252655, 3.788266720285841, 3.773835312029359, 3.7157320414379145, 3.816036239551131, 3.77769206739535, 3.728730265501958, 3.868407918389436, 3.8526493364078984, 3.8612681141324865, 3.718317868603263, 3.811299304293979, 3.8083236885678238, 3.771570157093607, 3.8100204262763833, 3.8511733439317934, 3.7925486868354166, 3.865870799228644, 3.8017261066254537, 3.869328704609233, 3.8375331049512145, 3.7675113541305443, 3.820743131789432, 3.869162396260887, 3.916696955443947, 3.875980369604317, 3.8877699891473076, 3.9807634748471012, 3.886209711907016, 3.882728667775537, 3.9342134689829154, 3.852312812379971, 3.947421051894024, 3.9500645885042323, 3.856550477112934, 3.88084525287531, 3.9919006087977413, 3.9809563000490713, 3.867949683954761, 3.949421214449937, 3.941449957288754, 3.925425480885111, 3.9730169317524906, 3.953438152173522, 4.009018191866055, 3.9125872113902096, 3.986843560911288]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="Adam")
data.update({'train_metric': [2.666, 6.417, 9.145, 11.489, 13.538, 15.139, 16.502, 17.912, 18.747, 19.624, 20.105, 20.609, 21.363, 21.724, 21.958, 22.46, 22.903, 23.446, 23.556, 23.942, 24.241, 24.513, 24.646, 25.031, 25.081, 25.5, 25.341, 25.794, 25.694, 25.937, 26.072, 26.185, 26.238, 26.373, 26.296, 26.538, 26.647, 26.711, 26.691, 26.702, 26.773, 26.986, 27.143, 26.956, 27.154, 27.071, 27.274, 27.106, 27.34, 27.25, 27.31, 27.186, 27.558, 27.454, 27.38, 27.46, 27.559, 27.515, 27.671, 27.743, 27.987, 27.675, 27.775, 27.742, 27.931, 27.826, 27.871, 28.051, 27.942, 27.947, 27.816, 28.133, 27.897, 28.151, 28.057, 28.092, 28.014, 28.222, 28.141, 28.148, 28.219, 28.188, 28.324, 28.157, 28.261, 28.155, 28.31, 28.185, 28.316, 28.229, 28.4, 28.43, 28.311, 28.279, 28.513, 28.489, 28.317, 28.425, 28.402, 28.378, 28.324, 28.524, 28.357, 28.46, 28.449, 28.633, 28.533, 28.589, 28.753, 28.56, 28.621, 28.803, 28.558, 28.539, 28.445, 28.62, 28.77, 28.645, 28.618, 28.757, 28.685, 28.703, 28.651, 28.659, 28.745, 28.755, 28.644, 28.742, 28.788, 28.801, 28.764, 28.664, 28.916, 28.817, 28.658, 28.769, 28.762, 28.814, 28.798, 29.013, 28.631, 28.886, 28.822, 28.722, 29.002, 28.903, 28.848, 28.945, 28.806, 28.891, 29.06, 29.122, 28.981, 28.878, 28.881, 28.933, 28.952, 29.02, 29.043, 28.81, 28.918, 28.866, 28.789, 28.907, 29.002, 28.968, 29.098, 28.901, 28.87, 29.083, 29.19, 29.025, 28.975, 28.943, 28.93, 29.156, 29.05, 29.136, 29.127, 29.123, 29.157, 29.181, 29.205, 28.983, 29.155, 29.064, 29.038, 29.146, 29.154, 29.047, 29.108, 29.167, 29.092, 29.131, 29.217, 29.13, 29.253, 29.159, 29.159, 29.269], 'train_loss': [4.944412340053136, 4.496548394705345, 4.253229158319728, 4.074336231326874, 3.938227303731312, 3.8266704141979253, 3.739391725336369, 3.6700185133872396, 3.6135508887522203, 3.5696307577083144, 3.534429857842219, 3.503034986598478, 3.473933924640209, 3.44381993913681, 3.4216671752685626, 3.397301972713214, 3.3799136870195676, 3.3556608153091205, 3.3378997643788657, 3.3199781431880275, 3.305414357096891, 3.2915088710919145, 3.280329073383041, 3.263348356394606, 3.255556128380471, 3.2470185553806377, 3.2379807150676627, 3.225908737798875, 3.2244314523515074, 3.217283143039247, 3.208527396294221, 3.2013148056415894, 3.1979175750368767, 3.1967075697825034, 3.1878717691564287, 3.180946301285151, 3.1796451502706633, 3.173467156220459, 3.1754506401579423, 3.1700883375400926, 3.1650551576608277, 3.1568570771586493, 3.1584465143700404, 3.160768325254083, 3.1517356935404695, 3.1476255881809228, 3.144865068654105, 3.1416664030486317, 3.1409417733273592, 3.13743935436754, 3.131830335197278, 3.1304637070885852, 3.1327648745541077, 3.1281348942032396, 3.123866962608586, 3.126008538961868, 3.1227937322813046, 3.1209405966470123, 3.1192627020242196, 3.1184689252710616, 3.1130402088165283, 3.11488521198241, 3.1121302885042117, 3.1109491334233006, 3.105444982733699, 3.1081603967601947, 3.1038792554339154, 3.1015388565954303, 3.09962006570129, 3.10236893444586, 3.104365411509479, 3.0997384070434864, 3.096793369078438, 3.095122177785433, 3.093170406417212, 3.095727415932956, 3.09159652803925, 3.088125365099233, 3.0881908989379747, 3.0892071373098107, 3.085732155706512, 3.086083042720763, 3.0878261853240656, 3.0819642781753687, 3.084417361291799, 3.085525212803485, 3.0801206475561123, 3.082356385519622, 3.0787706422409182, 3.079487939408706, 3.0773030143476645, 3.0757751763820345, 3.0778234909531097, 3.0769712511576373, 3.0733743557469326, 3.0721511342017527, 3.0762778046950268, 3.070208919101698, 3.0694991962618348, 3.073580872303236, 3.073846212809313, 3.066760893899206, 3.0705740212326393, 3.0680494059832983, 3.0681288468097923, 3.0660083681364054, 3.067511558532715, 3.0657875071102736, 3.0646633863906714, 3.0648684823505405, 3.0660100283717315, 3.06540437958901, 3.061589106030748, 3.0659141290363254, 3.061993328486203, 3.0598561254282908, 3.061074705514401, 3.0579014793086037, 3.058722592132334, 3.0613103158490746, 3.062805705091851, 3.0609411015696657, 3.058773948073921, 3.057044150962024, 3.0519177410286815, 3.0565817937283506, 3.0562917507388847, 3.0599938111662177, 3.056777753970292, 3.0576105474739292, 3.0582189135877886, 3.056151838082956, 3.0507458487307497, 3.0521438356705834, 3.054204877492181, 3.052114129600354, 3.052237820182942, 3.0518507271032647, 3.0546524899934853, 3.0502611202684182, 3.0541555446763873, 3.0534130422564094, 3.0509555949969545, 3.0504222881771095, 3.049768679277758, 3.04913513414843, 3.048816344559536, 3.046761242457101, 3.0466026001379265, 3.0481067205649954, 3.0450288447064615, 3.04299919725761, 3.047486657754626, 3.047228465992445, 3.045522823291029, 3.044113798470964, 3.043749849077836, 3.0434233876885473, 3.042125796173447, 3.0460973565424396, 3.0436903473969386, 3.043530270142656, 3.0458647132453747, 3.043421239633249, 3.0417420278355163, 3.0439874320478677, 3.0404079419561327, 3.0425319805865247, 3.044258227808996, 3.037201273235387, 3.0363360041620178, 3.0398656752959172, 3.0424266039981953, 3.0413158309055457, 3.041004971518245, 3.039422585730818, 3.0396182999272265, 3.0361920733216934, 3.035529189588775, 3.035288712189736, 3.0401009854336847, 3.035698812433488, 3.036646349645164, 3.0364192046191705, 3.0360280999142772, 3.0375717438266436, 3.035857806812855, 3.0351998278214545, 3.0346385460973777, 3.036995974398544, 3.032631539261196, 3.0306875858258073, 3.036991746472916, 3.033084398191553, 3.035290823025499, 3.0324396004069714, 3.031473633576416, 3.034525676789531, 3.032995301641414, 3.0275197759775954], 'val_metric': [4.7, 7.33, 9.95, 12.55, 14.83, 16.63, 18.08, 18.97, 18.92, 21.24, 20.57, 22.27, 22.16, 23.19, 22.91, 24.7, 24.16, 24.01, 26.24, 25.25, 26.43, 26.08, 26.02, 26.59, 26.67, 26.59, 27.1, 27.4, 27.38, 27.57, 27.24, 27.86, 28.31, 27.53, 27.43, 27.8, 27.45, 28.12, 28.74, 28.56, 28.51, 28.98, 27.99, 26.82, 28.7, 28.29, 29.0, 28.07, 28.94, 29.1, 28.15, 28.54, 29.08, 29.25, 29.05, 28.82, 28.45, 29.07, 28.33, 28.84, 28.99, 29.37, 29.38, 30.28, 29.49, 28.88, 29.94, 29.69, 29.08, 28.96, 29.72, 29.44, 29.3, 29.12, 29.06, 30.41, 29.72, 29.27, 29.82, 28.69, 29.14, 29.46, 29.9, 29.33, 29.88, 30.06, 29.81, 29.65, 29.29, 30.24, 29.3, 29.17, 29.56, 29.47, 29.9, 29.35, 30.35, 29.88, 30.41, 29.72, 30.42, 29.7, 30.16, 29.8, 29.99, 30.36, 29.92, 29.82, 30.7, 29.97, 29.79, 30.56, 30.43, 30.17, 28.75, 30.17, 29.57, 29.9, 29.78, 29.96, 30.41, 29.9, 29.45, 30.82, 29.55, 30.76, 30.52, 30.13, 29.65, 30.15, 29.88, 30.39, 29.32, 30.04, 30.03, 30.16, 29.91, 30.8, 30.48, 29.07, 30.88, 29.54, 30.12, 29.95, 30.73, 30.2, 31.04, 30.08, 30.17, 30.75, 29.91, 30.5, 29.44, 30.1, 29.66, 29.65, 30.73, 30.7, 29.81, 29.13, 29.81, 30.4, 29.41, 30.65, 29.62, 29.83, 29.94, 30.42, 29.81, 29.56, 30.33, 30.09, 29.45, 30.43, 30.45, 29.4, 30.57, 29.92, 30.01, 30.75, 30.18, 29.64, 31.19, 30.38, 30.47, 30.58, 30.09, 29.84, 30.31, 30.98, 29.99, 29.96, 29.67, 30.71, 30.03, 30.04, 29.63, 30.8, 31.23, 30.47], 'val_loss': [4.695163256043841, 4.364769464845111, 4.162840747529534, 3.975215028046043, 3.805939404068479, 3.7409060912527097, 3.649926191682269, 3.5902402932476845, 3.5792327238495942, 3.4806870366357696, 3.4691699302879866, 3.413773372674444, 3.4022693808671014, 3.37929475838971, 3.3690457997048737, 3.2982648390873224, 3.3163107466545836, 3.3045814606794126, 3.2307602463254503, 3.258575689261127, 3.195487919886401, 3.2265540566414024, 3.216464136816134, 3.200721813614961, 3.1804028893731964, 3.2056923474475836, 3.1559410201516123, 3.1368785435986366, 3.1602132806352747, 3.127233447542616, 3.148207104130156, 3.1257039027608884, 3.0938260494523746, 3.098113109351723, 3.1217654951059135, 3.1149893330920273, 3.1041892865660845, 3.1022114525934694, 3.083998258706111, 3.0802146765836484, 3.0775843122202877, 3.084616612476908, 3.0988444439165153, 3.1483754003123874, 3.068971052291287, 3.0740582768324836, 3.045422470493681, 3.070640019550445, 3.05545702102078, 3.056778173537771, 3.0935750501171038, 3.077880602733345, 3.0742331530637803, 3.0530198197455922, 3.033344909643671, 3.0525210207434976, 3.0534657763827378, 3.0358011297359586, 3.0668147817538802, 3.0491468913995536, 3.059312718689062, 3.036627373877604, 3.017787447400913, 2.9876252670956265, 3.0143618105323453, 3.0483273343675457, 3.0151310468175607, 3.016614778026654, 3.052365648518702, 3.045010172637405, 3.021589198689552, 3.0272885158563114, 3.0267259246984106, 3.02842111648268, 3.044643626850881, 2.999566712197225, 3.0227627981999876, 3.024552005871086, 2.988982844504581, 3.066626558637923, 3.0444800975216424, 3.023905063131053, 2.9889112915962364, 3.0445975938420387, 3.016773898130769, 3.0002894431922087, 3.001875568347372, 3.0122545646254424, 3.0673139725521112, 2.976482557642991, 3.0342779318997812, 3.012757391686652, 3.007496553621474, 3.035403224313335, 2.993400334552595, 3.007222127003275, 2.9773278600850683, 3.0139106223537664, 2.998179241350502, 3.0322358289342017, 3.0021535887080395, 3.0049791601812763, 3.000682185409935, 3.0019162285859418, 3.020111126504886, 2.9806834664314414, 2.990386135259252, 3.010792505209613, 2.9771147822118866, 3.018958513144475, 3.017591662467665, 2.9671111577635356, 2.9862826523507477, 2.976801009694482, 3.0591924866293647, 2.999418480381085, 2.998944307588468, 2.9996326683433194, 3.0087055484200738, 2.9877752969219427, 2.9948696065100893, 3.029090677097345, 3.029904419449484, 2.974705269382258, 2.9998826008693427, 2.9795844152474857, 2.955765673309375, 2.9766637231134307, 2.990097625240399, 3.000182643058194, 3.0125391050508825, 2.977152248856368, 3.0196549019236474, 3.012545717749626, 2.9774319943349075, 2.9751807200680873, 2.999337388451692, 2.978583751970036, 2.9769848349747385, 3.0534532723153474, 2.9537694446600167, 2.9990501494923976, 2.987046095975645, 2.990719615274174, 2.979962584319388, 2.9892810583114624, 2.967922391405531, 2.997186080665345, 2.9872231802363305, 2.9733245274063886, 3.0025539868956157, 2.9864909869090766, 2.990360445277706, 2.9728315392876885, 3.003594854834733, 3.01969107397043, 2.978045668571618, 2.9787793151891915, 2.9901485936656877, 3.0402476240874856, 3.017776899277025, 3.003781707423508, 3.021321222281, 2.9714685579773725, 2.9990472672091926, 2.9888572176550605, 3.0146131766070225, 3.00743345546115, 3.0231252870741923, 3.0176079425082842, 3.001613357264525, 2.995031592192923, 3.013466688478069, 2.9741155683614644, 2.9672941910992763, 3.0180707691581388, 2.962028370541372, 3.0180825405060108, 2.9976005683279343, 2.971222234379714, 2.9834876964046697, 2.99336246244467, 2.9692677752986834, 2.9738216453297124, 2.980660375516126, 2.9750795250485655, 2.9913660401751283, 2.99466897955366, 2.9881600490800895, 2.955865973879577, 2.9935491449514013, 2.9976460424957763, 3.011523985559014, 2.975283830788485, 3.001904905981319, 2.984679768039922, 2.992083024826779, 2.970677305179037, 2.96146662675651, 2.9927943815850906]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="Orthogonal", optimizer="AdamW")
data.update({'train_metric': [3.921, 13.477, 22.371, 29.858, 35.667, 41.048, 46.132, 50.466, 54.896, 59.271, 63.596, 67.866, 72.063, 75.785, 79.06, 81.998, 84.359, 86.046, 87.534, 88.702, 89.592, 90.24, 90.98, 91.608, 92.088, 92.556, 92.882, 93.205, 93.189, 93.967, 93.918, 94.264, 94.369, 94.691, 94.706, 94.873, 95.089, 95.24, 95.444, 95.394, 95.769, 95.726, 95.908, 95.929, 96.18, 96.208, 96.269, 96.233, 96.466, 96.331, 96.511, 96.547, 96.776, 96.702, 96.79, 96.722, 96.878, 96.905, 97.02, 97.096, 96.987, 97.082, 97.194, 97.247, 97.213, 97.201, 97.391, 97.218, 97.465, 97.346, 97.402, 97.507, 97.415, 97.597, 97.557, 97.58, 97.668, 97.756, 97.52, 97.763, 97.688, 97.776, 97.72, 97.809, 97.908, 97.823, 97.827, 98.016, 97.846, 97.89, 97.845, 97.98, 98.07, 97.976, 97.936, 98.099, 98.019, 98.09, 97.963, 98.131, 98.121, 98.092, 98.017, 98.044, 98.221, 98.202, 98.191, 98.176, 98.274, 98.233, 98.192, 98.275, 98.229, 98.187, 98.266, 98.351, 98.252, 98.246, 98.294, 98.357, 98.33, 98.332, 98.365, 98.356, 98.292, 98.316, 98.396, 98.474, 98.365, 98.365, 98.427, 98.446, 98.439, 98.374, 98.464, 98.461, 98.368, 98.466, 98.592, 98.493, 98.504, 98.533, 98.575, 98.442, 98.582, 98.647, 98.491, 98.494, 98.686, 98.58, 98.504, 98.606, 98.492, 98.589, 98.627, 98.621, 98.584, 98.607, 98.661, 98.523, 98.619, 98.611, 98.54, 98.722, 98.62, 98.611, 98.776, 98.757, 98.623, 98.723, 98.7, 98.716, 98.738, 98.66, 98.616, 98.824, 98.606, 98.708, 98.786, 98.668, 98.75, 98.748, 98.793, 98.716, 98.761, 98.748, 98.693, 98.801, 98.676, 98.748, 98.764, 98.794, 98.779, 98.738, 98.79, 98.787, 98.806, 98.712, 98.787, 98.841], 'train_loss': [4.797712533350412, 3.939004255424153, 3.388441761868624, 2.99625051669867, 2.6928910399886634, 2.4165795872932967, 2.1719696145750205, 1.9688432249440189, 1.7530561016678277, 1.5544011198170125, 1.3573584858988312, 1.1703954923633422, 0.9946817932835162, 0.839215400618616, 0.7101235631865259, 0.6034830763163813, 0.5141028377560568, 0.455527239725213, 0.40484617957493774, 0.3656799882383432, 0.33575429126706097, 0.31249230025635266, 0.2919699743590291, 0.26964739209573696, 0.25464042089522, 0.2400026641862368, 0.22819328929225527, 0.2205566799438541, 0.2152802468178006, 0.1938183848708582, 0.19351445253097088, 0.18309107374087671, 0.18016430961655755, 0.17162957319528607, 0.16628562107858602, 0.16157082009521895, 0.1568969679687718, 0.15152271430026584, 0.14683598116211835, 0.14658621701239702, 0.13452105900130915, 0.13670409232871375, 0.1301483327894688, 0.1302621415521299, 0.12266187136724677, 0.12245927169360102, 0.12137600310773648, 0.11819452633461554, 0.11248686087454215, 0.11738918134013115, 0.11149658801293887, 0.10978009797285906, 0.10422591819241404, 0.10822486996314633, 0.10129865312380376, 0.10354353018433943, 0.1000454234877233, 0.09759667020807102, 0.09482346657039127, 0.09460364702864957, 0.09340354909087153, 0.09259205298636511, 0.08932655942593526, 0.08848285363848254, 0.08773244002217848, 0.0886963037946765, 0.08226771490759814, 0.08687696087079376, 0.07964574331054208, 0.085928723157514, 0.08289259268979107, 0.07913642511577944, 0.08169322377648765, 0.0761972056071364, 0.07759716062336118, 0.07655056007645884, 0.07523229709992951, 0.07222838718140223, 0.07733717987996717, 0.07284629241261281, 0.07450037391964162, 0.07182106351085431, 0.07305566315363851, 0.0694261154356387, 0.06748743788431555, 0.06933627401714547, 0.06939816839459113, 0.06349906987700997, 0.0701203391038451, 0.06696139226065173, 0.06827594599923714, 0.06383904006255592, 0.06252355766844633, 0.06516693183593333, 0.06587879201194709, 0.06160461674562059, 0.06402015088771673, 0.06018368158264828, 0.0647189512407027, 0.060347028993968675, 0.05859986491617821, 0.06144822185979894, 0.06190626414001741, 0.061459305554282626, 0.055892381362911366, 0.057174895706086616, 0.059435168144225205, 0.05884685110211699, 0.05537981368961517, 0.056598866312555736, 0.05673527951091314, 0.05501459007716296, 0.05560495602029542, 0.05747199018625274, 0.05619385055181709, 0.05301547104535089, 0.0563959716723145, 0.053995206966842464, 0.05371147425998996, 0.05303865419438565, 0.05295378352862187, 0.053290581583560544, 0.05253228645024539, 0.052590801090802594, 0.05335065110503015, 0.052974419586908045, 0.050746566562669654, 0.049552549147418024, 0.05163332899414647, 0.05114042772020685, 0.04874425259491331, 0.05028089317420149, 0.04946957981009549, 0.05100607627529653, 0.049223082958793914, 0.048854347175293765, 0.051770382769711513, 0.048875376775985024, 0.04554235311893905, 0.046667536386209456, 0.046975892432615275, 0.04735121333549158, 0.045877976087599776, 0.049442062649605656, 0.045218236989524006, 0.044679146010508514, 0.048769974027873836, 0.04769505381943022, 0.04252821603372529, 0.04566888281374202, 0.04686546734168796, 0.044230962930097775, 0.048220517406463104, 0.04631569123110702, 0.04428764269339032, 0.044555976724410605, 0.04563739764844071, 0.04442757187722226, 0.04183980111271514, 0.045749531699366415, 0.0439388302964607, 0.04422665815168077, 0.04577741747781913, 0.04013785386080564, 0.044956656826897784, 0.04526980407603064, 0.0406834981046359, 0.041602112539276426, 0.043317710251035854, 0.04041063112571123, 0.04345425052733071, 0.03986022359328386, 0.04109657461163144, 0.04215869348579916, 0.04405569195116811, 0.03883082682570122, 0.044582721730429764, 0.04113465549440453, 0.03922977720851899, 0.042907668960360267, 0.03869242226120346, 0.039925348347467644, 0.039781426237659234, 0.04179026451383694, 0.039868641683409645, 0.039626298251855874, 0.04078506959746881, 0.03745711329505929, 0.04228216860645498, 0.03881819709373584, 0.039111514251727, 0.03838286391584237, 0.03797568067092263, 0.04021464117616996, 0.03694576976322691, 0.03829243297208664, 0.03830571653628549, 0.03990844513476357, 0.03819288122007398, 0.0367174455476314], 'val_metric': [8.51, 17.96, 27.59, 33.13, 37.78, 41.88, 43.44, 45.33, 47.47, 48.22, 47.89, 49.52, 48.37, 48.87, 49.02, 49.06, 48.8, 48.62, 49.33, 48.89, 48.85, 49.58, 49.19, 49.14, 49.87, 50.42, 49.48, 48.69, 49.7, 49.06, 49.24, 49.65, 49.62, 49.62, 49.93, 49.62, 49.47, 49.88, 50.18, 50.54, 49.75, 50.19, 49.83, 50.68, 49.86, 50.43, 50.43, 50.12, 50.3, 50.74, 50.27, 50.46, 50.24, 50.9, 50.3, 50.8, 50.21, 50.66, 51.19, 51.06, 51.12, 50.8, 50.58, 51.01, 50.84, 50.23, 50.86, 50.86, 51.08, 50.88, 51.12, 50.67, 51.12, 50.96, 51.35, 50.73, 51.31, 50.97, 51.02, 51.0, 50.89, 50.59, 50.36, 50.82, 50.7, 51.2, 50.24, 50.99, 50.9, 51.11, 51.01, 51.53, 50.61, 51.34, 51.27, 50.93, 51.7, 51.5, 51.46, 52.06, 51.46, 50.74, 52.02, 51.71, 51.36, 51.12, 51.06, 51.29, 51.15, 51.84, 51.79, 52.03, 51.41, 51.16, 51.19, 51.77, 51.85, 51.52, 51.23, 51.53, 51.38, 51.35, 51.43, 51.52, 51.55, 51.43, 51.3, 51.64, 51.38, 52.02, 51.41, 51.64, 51.9, 51.8, 51.78, 52.06, 51.74, 51.43, 52.03, 52.09, 51.89, 51.89, 52.21, 51.4, 52.55, 52.27, 52.13, 51.75, 52.23, 51.83, 52.59, 51.66, 52.35, 52.59, 51.57, 52.2, 52.02, 52.44, 51.55, 51.6, 52.03, 51.54, 50.97, 51.6, 51.66, 51.37, 52.14, 51.6, 51.8, 52.17, 51.67, 52.35, 52.14, 51.92, 52.09, 51.94, 51.69, 51.91, 51.69, 52.14, 51.9, 51.72, 52.14, 51.72, 52.68, 52.02, 52.14, 51.9, 52.04, 52.1, 51.86, 51.95, 52.88, 52.21, 52.17, 52.5, 52.86, 52.52, 52.48, 52.33], 'val_loss': [4.333563948892484, 3.6233798698255213, 3.127647281452349, 2.8213136477075564, 2.614410976695407, 2.422514553282671, 2.323094879745678, 2.268810206158146, 2.1857437586328787, 2.182019844176663, 2.2345880106755884, 2.2162747238851654, 2.3680733673891443, 2.4404280766560014, 2.5428044606166282, 2.5850492784172108, 2.6819076181217363, 2.7651786500481284, 2.796306021274275, 2.895801807664762, 2.99536900041969, 3.0061994677136656, 3.0233581504624363, 3.0899739075618187, 3.0268039972918808, 3.076149701692496, 3.1815964492263307, 3.2169866512535483, 3.109670336459093, 3.251620721665158, 3.219016408085064, 3.248338551278327, 3.236575057552119, 3.278001178981392, 3.288747842144814, 3.434473659961846, 3.452581200250395, 3.4236477419828915, 3.3069357116510916, 3.297383090493026, 3.3767739185102426, 3.4072834762038697, 3.3941386061109555, 3.41059706962792, 3.443285700242231, 3.448250050377694, 3.4143493786738937, 3.446604206303882, 3.4952517299895076, 3.4772980087881633, 3.5051181760563215, 3.422978293364215, 3.440995852279056, 3.4754335811943005, 3.5088089947487897, 3.482709972721756, 3.5091400438812888, 3.4627320204570795, 3.4823524352091892, 3.551696093978396, 3.6288839214167017, 3.791665708942778, 3.5481825850571798, 3.731542160936222, 3.601088296076295, 4.0196497129027255, 3.626984559806289, 3.5972613350600953, 3.593907837275487, 3.6620219008178467, 3.989510939759054, 3.754692057895053, 3.5987421350114666, 3.6668636491344233, 3.6151399692152717, 3.6592892632362948, 3.692887316843507, 3.689766899035994, 3.6572771569725813, 3.747594052059635, 3.7478391099127992, 3.741942642600673, 3.866706498109611, 3.745098034287714, 3.7176805791581513, 3.6523627448993126, 5.248097814951732, 3.8859052790957653, 3.955440136657399, 3.723176141073749, 3.887658685255962, 3.897067724139827, 4.34354806401927, 3.781744103522817, 3.6738646854260923, 3.664919308416403, 3.699553607755406, 3.7201063222945874, 3.7133600954796857, 3.687255737887826, 3.7655408807620883, 3.770497359667614, 3.672713277825884, 3.9571177849344386, 3.6796536346909345, 3.856794490556049, 3.7798114460744676, 3.836190465908901, 3.9899876839036397, 3.7753930103247333, 3.9938063276041844, 3.9421739714920143, 4.193546025995996, 4.14303846495926, 4.2926044889316435, 3.828262644968215, 4.251634287226731, 4.437361620034382, 3.8351345738028266, 3.7857229254048343, 3.860050634214073, 4.042665786803908, 3.7965698105514427, 4.16407451280363, 3.799822234044409, 3.8377567430970014, 3.9493609860444523, 3.687355409761903, 3.935447238433133, 3.899978915217576, 3.984926694517682, 3.810547469527858, 3.8516110003374187, 3.8373057443624847, 3.8866383163792313, 3.7796821233573232, 4.019287254400314, 4.033212793860466, 3.784653970770016, 3.8990766861636166, 3.8247634935530885, 3.9267653788730597, 3.8198793116648484, 3.9506693463416616, 3.852252414651737, 3.7549321347740805, 3.9118078377596133, 3.8984590449910255, 3.9185355153812726, 3.9834578602936617, 3.9669913064901996, 4.932000953300744, 3.8116986060598093, 3.7478847051881683, 3.8616983860161653, 3.9630305114065765, 3.7954262886077736, 3.8842880900498407, 4.042749985767777, 3.9037656009576884, 4.238361211719027, 4.286870355818682, 4.59035039555495, 3.997247353860527, 4.0703544969771315, 3.9352752934595583, 3.8890563750722604, 3.8487986569192, 4.009463027024725, 3.8528081645631485, 3.9183643281839458, 3.8944630239419875, 3.8813727410735597, 3.9279592978726527, 3.8469859224975487, 3.9153548543620262, 3.8834930183781182, 3.8621639594150956, 3.934417983149267, 3.8726919809724114, 3.851612120676952, 3.850367713126407, 3.8044355747046743, 3.9424901278155624, 3.8439924253779614, 3.852930066691842, 3.8215372839551063, 3.8231738054068987, 3.8367029048834635, 3.924863066642907, 3.8782262847681714, 3.8678788083374123, 3.9253217665253173, 3.845032440628975, 3.8923502659342093, 3.8469626402399344, 3.8411558623526507, 3.7999059136506097, 3.8337079909197085, 3.8273286136092652]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="SGD")
data.update({'train_metric': [1.289, 3.346, 5.581, 8.186, 10.635, 13.162, 15.569, 17.733, 19.956, 22.158, 23.936, 25.664, 27.681, 29.212, 30.98, 32.723, 34.088, 35.768, 37.418, 39.067, 40.533, 41.8, 43.381, 44.617, 46.171, 47.467, 48.933, 50.241, 51.597, 52.973, 54.398, 55.448, 57.048, 58.286, 59.343, 60.68, 61.687, 62.991, 64.207, 64.928, 66.282, 67.247, 67.999, 69.086, 70.087, 71.056, 71.81, 72.985, 73.642, 74.723, 75.432, 75.895, 76.858, 77.407, 77.993, 78.733, 79.218, 80.161, 80.543, 80.897, 81.544, 81.983, 82.416, 82.945, 83.339, 83.775, 84.144, 84.734, 84.82, 85.218, 85.718, 85.971, 86.125, 86.614, 86.745, 86.99, 87.549, 87.744, 88.094, 87.923, 88.393, 88.774, 88.896, 88.883, 89.178, 89.645, 89.645, 89.857, 89.982, 90.162, 90.272, 90.531, 90.53, 90.724, 90.889, 90.956, 91.324, 91.373, 91.6, 91.486, 91.77, 92.052, 92.156, 92.048, 92.107, 92.426, 92.45, 92.564, 92.658, 92.84, 92.866, 92.95, 93.015, 93.018, 93.229, 93.291, 93.395, 93.471, 93.526, 93.518, 93.574, 93.716, 93.805, 93.883, 93.919, 94.104, 94.066, 94.229, 94.264, 94.32, 94.214, 94.385, 94.424, 94.56, 94.477, 94.823, 94.782, 94.82, 94.709, 94.748, 94.932, 94.856, 94.905, 94.989, 94.905, 95.205, 95.147, 95.022, 95.188, 95.314, 95.186, 95.418, 95.399, 95.379, 95.41, 95.414, 95.587, 95.712, 95.623, 95.685, 95.688, 95.622, 95.871, 95.707, 95.857, 95.797, 95.866, 95.837, 95.903, 95.833, 95.949, 95.899, 96.133, 96.102, 95.994, 96.073, 96.24, 96.213, 96.232, 96.197, 96.31, 96.125, 96.278, 96.277, 96.28, 96.345, 96.274, 96.218, 96.518, 96.506, 96.574, 96.637, 96.491, 96.405, 96.523, 96.55, 96.552, 96.512, 96.694, 96.845], 'train_loss': [5.279095245185603, 4.957291979249746, 4.683774637550554, 4.441661140663991, 4.234366729636262, 4.049207813680287, 3.8794807292525766, 3.7300866188639947, 3.598900171860776, 3.4738871784295626, 3.3603562070098505, 3.2613636837200866, 3.153280535342216, 3.059247010805175, 2.9666691355726615, 2.8797718613710606, 2.800321082617332, 2.7157206796943876, 2.633994318320823, 2.557532905464514, 2.482418361186066, 2.4157151770149374, 2.345354066624217, 2.280083066289881, 2.210039045745108, 2.146469192785555, 2.080062294723281, 2.0211129432600123, 1.9591590095923943, 1.8936638408796342, 1.8393322547429354, 1.7840784630253768, 1.7177431469762927, 1.6660355409215217, 1.6182561303588419, 1.5690111060288954, 1.5135560091000981, 1.4674713727151134, 1.420275861532049, 1.3806779555075457, 1.3298771268117908, 1.2891880396994275, 1.2586684725411184, 1.2122029102695193, 1.1735442304794252, 1.1367908456122655, 1.1032836031090045, 1.063772457841872, 1.0342024051449044, 0.9995600244820461, 0.9696862984374786, 0.9435190303311924, 0.9129947903098315, 0.889549807982039, 0.8656595449415598, 0.8397243099569588, 0.8155478015589699, 0.7881915449562243, 0.7704126627034891, 0.7543353181539708, 0.7327844944430404, 0.7162737793176508, 0.6961965879338412, 0.6809535815749348, 0.662505719708237, 0.6461651458697524, 0.6317480961176652, 0.6134795434034107, 0.6051225749750742, 0.5857574160539105, 0.5691410416948132, 0.5593552409134381, 0.5552516465033008, 0.5350753972527315, 0.5316169278520998, 0.5195064056552684, 0.5021688788652573, 0.4938092937815746, 0.48054815287286473, 0.4790958756353332, 0.46646564726027173, 0.4551956254323934, 0.4444004466956194, 0.4484924067977294, 0.4361349792813767, 0.42135236296452394, 0.42045114593614924, 0.4119745858456947, 0.40728783585555417, 0.40093204379081726, 0.3960577679877852, 0.38493565405627816, 0.3833823068805101, 0.3792687857658224, 0.371057255359239, 0.3669361697029625, 0.35568339296605295, 0.35223055561803046, 0.35017060633851255, 0.3455099448923949, 0.3399752202090436, 0.33091177525843707, 0.32400089653920305, 0.3267744974360127, 0.3237824803236121, 0.3151595461315172, 0.30991924993136105, 0.30768515013744646, 0.30259241850633156, 0.2985154542540482, 0.2970329029122111, 0.2916385290461401, 0.2884467567006747, 0.2906081363206023, 0.2820376570521832, 0.2790893694422829, 0.2746261567477988, 0.2736452825114808, 0.27120109028299116, 0.2698667690358098, 0.2683367621456288, 0.26491786009004614, 0.25813367642505153, 0.25694640520161643, 0.2545023069658, 0.24988474696397933, 0.24994599746727286, 0.24294570922584618, 0.2421075672826474, 0.23889701861127857, 0.2425526477496592, 0.23552456337265, 0.23448393783953353, 0.23181583101951153, 0.2338756897790037, 0.22275931835984925, 0.2234459920039714, 0.22248857759144242, 0.22552215103929957, 0.22193899602460618, 0.21683706873247277, 0.2186638334919761, 0.21574503211965945, 0.2145290650041227, 0.21675948314821575, 0.20758941438542447, 0.2091296677592658, 0.21293050429938087, 0.20552960929108674, 0.2008248700123335, 0.20374658167533818, 0.19982141346664134, 0.20073166063206743, 0.19836197523203555, 0.19771627501196687, 0.19735993311447894, 0.19344812214508014, 0.18754847986970433, 0.1885312898936595, 0.18744777348095112, 0.18692121729111716, 0.18813356915184953, 0.18247064321956724, 0.1861985636826173, 0.18132662448033415, 0.18397017076411312, 0.17995565832233246, 0.1827539761031734, 0.1803511350069455, 0.18022411774607972, 0.17768420426679093, 0.1769457061823293, 0.1725453932722524, 0.17322000012669286, 0.17564797728910556, 0.17236826155318027, 0.16822623484604113, 0.1684179913397981, 0.16758244630409333, 0.16807693073088667, 0.16566036124394937, 0.16685481597588983, 0.16493833319582546, 0.16527076624214687, 0.16432829358408219, 0.1646543982369028, 0.1641221569466118, 0.1637780448053597, 0.15749571516142202, 0.15888112216296482, 0.15375162015406277, 0.15422837501504982, 0.15657556892842797, 0.15825207767172722, 0.15526994301205213, 0.15503144351096962, 0.15533791740930813, 0.15694734193408483, 0.15018563891951084, 0.14582090214924484], 'val_metric': [2.98, 5.61, 8.85, 11.94, 13.96, 16.63, 18.56, 20.44, 22.93, 23.49, 23.99, 26.85, 27.75, 28.83, 29.73, 29.7, 31.23, 31.52, 32.35, 33.07, 33.32, 34.13, 34.43, 34.2, 34.87, 35.27, 35.51, 34.99, 34.91, 34.98, 35.52, 34.75, 35.84, 34.65, 35.13, 35.52, 35.04, 35.31, 36.24, 35.21, 35.89, 34.95, 35.82, 35.31, 35.6, 35.79, 35.17, 34.57, 35.98, 35.12, 35.67, 35.35, 34.36, 35.41, 35.78, 35.22, 36.23, 35.53, 35.12, 35.35, 35.12, 35.16, 35.46, 36.33, 35.33, 35.03, 35.35, 35.76, 35.12, 35.34, 35.13, 35.29, 35.73, 35.96, 35.4, 35.65, 35.32, 35.78, 36.01, 35.68, 35.6, 34.81, 35.66, 35.71, 36.02, 36.42, 35.94, 35.77, 35.96, 35.64, 35.49, 36.25, 35.96, 35.9, 36.06, 36.02, 36.42, 36.23, 35.93, 35.86, 35.77, 36.14, 36.63, 35.99, 35.56, 36.36, 35.85, 36.04, 35.9, 36.03, 35.8, 35.8, 36.24, 35.7, 36.1, 35.97, 36.18, 35.97, 35.74, 35.54, 35.64, 36.34, 36.41, 36.23, 35.85, 36.75, 36.22, 36.03, 36.24, 35.8, 35.95, 35.93, 36.41, 36.2, 36.65, 36.36, 35.66, 36.22, 36.57, 37.0, 36.59, 36.32, 36.66, 35.64, 36.41, 36.21, 36.42, 36.66, 36.5, 36.71, 36.83, 36.67, 36.75, 36.32, 36.95, 36.31, 36.55, 36.62, 36.67, 35.59, 36.59, 36.58, 35.92, 36.64, 36.22, 36.29, 36.27, 36.5, 37.25, 36.52, 36.48, 36.62, 36.55, 36.84, 36.66, 36.53, 36.53, 36.37, 36.28, 36.05, 36.85, 36.92, 36.78, 36.87, 36.37, 35.71, 36.22, 36.16, 36.85, 36.73, 36.46, 36.48, 36.72, 36.74, 36.98, 36.51, 37.06, 36.81, 36.84, 36.11], 'val_loss': [5.031114058889401, 4.723984766917623, 4.407468841334057, 4.1786123157306845, 3.982205845747784, 3.812016046730576, 3.680547699047502, 3.560612074129141, 3.4071138808681707, 3.3677970049487556, 3.3122770588868744, 3.165509980954942, 3.1223568232955445, 3.0637699312465205, 3.025783380125738, 3.0165382456627623, 2.9482795000076294, 2.93800425377621, 2.9001124246864562, 2.8802061027781978, 2.872512524295005, 2.840294454507767, 2.831083321267632, 2.8336075734181008, 2.8279960573099223, 2.8149979797897826, 2.8076909385669, 2.827084971081679, 2.8555352718207487, 2.873233155839762, 2.8534360471045135, 2.8956647883554933, 2.8753112076194425, 2.935336152459406, 2.9128680548090844, 2.9443605234668513, 2.9658797706008717, 2.9776663438529725, 2.9542003444805265, 3.0212180196859273, 3.01398646603724, 3.0541120775186332, 3.0556408819878937, 3.0612818678473213, 3.1051661861929922, 3.0534035028166073, 3.1258167013241227, 3.156875008230756, 3.1344709009121936, 3.214866505306997, 3.200535629964938, 3.211924366890245, 3.2827663004018697, 3.2081001123804955, 3.2209085339953187, 3.2819439909260746, 3.290293663170687, 3.3114392142386953, 3.3338892459869385, 3.3707906182404535, 3.3617016502246737, 3.357929697461948, 3.374733156459347, 3.3328642245310887, 3.381520401140687, 3.4242625722459925, 3.4166146099187764, 3.426625633695323, 3.4320377119027885, 3.463564795293626, 3.4373516277143152, 3.4886223601687485, 3.460392277711516, 3.4838467464325533, 3.53139511642942, 3.484754054409683, 3.5290489234742086, 3.5139196807411825, 3.547337419667821, 3.587450546064195, 3.5382517454730475, 3.6103827148486096, 3.613181944106035, 3.621997629001642, 3.605036785648127, 3.6287617402471555, 3.5807394935826586, 3.6090772273434197, 3.56664736787225, 3.6108561792191427, 3.614785954451105, 3.6041745258744355, 3.624319332420446, 3.6353515925680755, 3.6621775794181093, 3.6290336543587363, 3.6414736805448107, 3.6258007849857306, 3.6615490192061015, 3.664426968355847, 3.6468211936343247, 3.6830762374173305, 3.670527294183233, 3.698402306836122, 3.683635072343668, 3.6770007519205663, 3.7173392514514316, 3.6742318451024922, 3.689953463092731, 3.7123150051019755, 3.6996510712204467, 3.6961348147908595, 3.697453958973004, 3.734647891324037, 3.7111429826469178, 3.697032609562965, 3.725451060161469, 3.7089501421922333, 3.713094620947625, 3.751493943724663, 3.754136278371143, 3.725757383237219, 3.751928493475458, 3.746687628660992, 3.751526835617746, 3.7344890195093337, 3.7542557116526707, 3.7520928101934445, 3.7346557872310564, 3.8230149890207183, 3.7532410074950784, 3.7688595436181234, 3.7055935760971845, 3.8035140447555835, 3.76280475953582, 3.752009798766701, 3.8045365643349425, 3.782706258403268, 3.7899321165813764, 3.7698332435765844, 3.768321951483465, 3.7976394534870317, 3.776479499355243, 3.8029130925038817, 3.752863527103594, 3.799883428652575, 3.742682649831104, 3.7612819648852014, 3.765886242222634, 3.7646897294718746, 3.7918081579694323, 3.764123769322778, 3.7502842579677607, 3.7363721732121364, 3.7697811453205765, 3.7738318200323993, 3.7928379280551985, 3.7704285406003333, 3.7604075351338477, 3.8523859825863203, 3.763675393572279, 3.8184995241225903, 3.802782849901041, 3.780628022874237, 3.826687421768334, 3.80091734828463, 3.8088413355456794, 3.7899523836791893, 3.742763319592567, 3.750888996063524, 3.769961207535616, 3.8000254942353364, 3.757302076953232, 3.7528370740307366, 3.800692376057813, 3.7682695328050357, 3.769621730610064, 3.8034073486449613, 3.7952482996472887, 3.8261438099442016, 3.771178966115235, 3.757684059963105, 3.789662207767462, 3.7708285895122846, 3.791486254922903, 3.821824354730594, 3.7957808751209527, 3.817796994166769, 3.791321067293738, 3.7662423628910333, 3.776994623955648, 3.777376111905286, 3.792487716219228, 3.7527173681623616, 3.794934401086941, 3.7963075721339816, 3.775362842401881, 3.7609411903247714, 3.7629218268546327, 3.801960431086789]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="Adam")
data.update({'train_metric': [2.945, 7.507, 10.523, 12.817, 14.968, 16.728, 18.38, 19.682, 20.7, 21.378, 22.24, 23.046, 23.205, 23.753, 24.163, 24.45, 24.834, 25.378, 25.844, 26.043, 26.293, 26.628, 26.794, 27.201, 27.415, 27.578, 27.938, 28.018, 28.135, 28.319, 28.34, 28.684, 28.632, 28.854, 28.979, 29.037, 28.934, 29.249, 29.337, 29.409, 29.384, 29.367, 29.518, 29.716, 29.818, 29.827, 29.917, 29.928, 29.968, 30.046, 30.268, 30.025, 30.164, 30.218, 30.442, 30.328, 30.196, 30.372, 30.485, 30.546, 30.559, 30.708, 30.672, 30.777, 30.739, 30.625, 30.973, 30.844, 31.126, 30.98, 31.043, 30.986, 31.002, 31.112, 31.024, 31.148, 31.225, 31.364, 31.258, 31.272, 31.376, 31.39, 31.397, 31.497, 31.303, 31.511, 31.677, 31.471, 31.563, 31.349, 31.588, 31.627, 31.648, 31.667, 31.718, 31.722, 31.651, 31.644, 31.805, 31.598, 31.828, 32.002, 31.905, 31.75, 31.947, 31.855, 31.845, 31.931, 31.99, 31.792, 31.775, 31.983, 32.026, 31.959, 31.984, 32.081, 31.907, 32.037, 32.082, 32.169, 32.193, 31.977, 32.253, 31.977, 32.121, 32.149, 32.004, 32.139, 32.163, 32.348, 32.221, 32.208, 32.062, 32.117, 32.243, 32.18, 32.051, 32.3, 32.201, 32.288, 32.183, 32.312, 32.329, 32.337, 32.416, 32.357, 32.252, 32.431, 32.334, 32.452, 32.493, 32.429, 32.253, 32.56, 32.465, 32.508, 32.403, 32.359, 32.439, 32.418, 32.408, 32.476, 32.574, 32.587, 32.66, 32.724, 32.355, 32.528, 32.494, 32.515, 32.527, 32.511, 32.344, 32.536, 32.469, 32.737, 32.506, 32.474, 32.686, 32.777, 32.537, 32.694, 32.564, 32.621, 32.52, 32.52, 32.658, 32.828, 32.527, 32.652, 32.759, 32.759, 32.761, 32.852, 32.824, 32.777, 32.712, 32.782, 32.695, 32.68], 'train_loss': [4.913463756356267, 4.342634208981837, 4.107395107900189, 3.9504690130627926, 3.8121738793830113, 3.7007613490196807, 3.605631453061973, 3.5320180910028713, 3.475677318780451, 3.433195224879113, 3.3960621723057898, 3.3622378988748016, 3.3348527888190036, 3.311154506485659, 3.2868373145373755, 3.268124651008894, 3.2450292407894317, 3.225862956550437, 3.2056046310175863, 3.189576417608133, 3.1791590733628814, 3.160643018367423, 3.1475095783985356, 3.131509574040799, 3.121301871267405, 3.113681678350965, 3.0959766490750793, 3.092106679732115, 3.079210903273892, 3.075439218977515, 3.063738683363755, 3.0567056130539205, 3.0536958275890775, 3.0443211488058686, 3.0422242755548203, 3.0351401339413187, 3.030337681163219, 3.026454549375743, 3.023690538381966, 3.0158084698846075, 3.0176738695082417, 3.0117581513014957, 3.005012936723286, 3.0040236472778417, 2.9984760661195353, 2.997175585056678, 2.9882202044710926, 2.9871277456777796, 2.987372902670657, 2.977953944538773, 2.9784646194406754, 2.977572163632491, 2.9727308359652547, 2.971149163145479, 2.971361390612328, 2.9639904476173093, 2.969656015616995, 2.9609141195728013, 2.9620577765822946, 2.9537832567650617, 2.954498494197677, 2.953883729374569, 2.952071641090964, 2.9447352370617867, 2.945034026489453, 2.9472860919308066, 2.934017803755923, 2.9417389720506724, 2.9328449658682465, 2.9351175292058396, 2.9320878274762623, 2.9332720197627578, 2.9319771827983305, 2.9291355170581252, 2.9270669143320425, 2.9248299202092243, 2.924573639761692, 2.923768793857792, 2.919687097681232, 2.917342576779239, 2.918187115639353, 2.9166511534424218, 2.913665753942381, 2.911864261328221, 2.9123572581712818, 2.9137894553552433, 2.90882079508239, 2.9070973789852093, 2.9083391672818997, 2.905737021300401, 2.904698826911277, 2.904045070354098, 2.9039491876454515, 2.9014838364211246, 2.9006795533864222, 2.8989480891749406, 2.89723151826889, 2.899103688949663, 2.8959984455975065, 2.896868985201103, 2.8929967518731408, 2.8921306839526197, 2.8898662417955454, 2.893755850849896, 2.8937172621240697, 2.8937737063116855, 2.8881599196850756, 2.889825478167543, 2.8895040473645115, 2.884394350527802, 2.8832638721319626, 2.883985623745909, 2.881745964612857, 2.883338849970109, 2.882180392398944, 2.8806632050168264, 2.8817838150495456, 2.881306020556088, 2.8796966182674573, 2.877220521275233, 2.873961279656135, 2.8780013610366364, 2.8732194807463856, 2.877033784003572, 2.872235556710476, 2.8738952877425414, 2.8736214672840337, 2.873031667616607, 2.8725369827761074, 2.867840588588556, 2.867931258731825, 2.868613768295073, 2.8717466738463515, 2.871094226227002, 2.867114250582148, 2.866781525633233, 2.8677055654202372, 2.8641127935984314, 2.869890462566627, 2.8635870539371737, 2.863375813703238, 2.862220571960918, 2.8624657570209857, 2.858451101464182, 2.858599213858255, 2.8586498052739824, 2.8601502960336873, 2.8587352759702345, 2.8613803789391397, 2.855607690524384, 2.8541837553145104, 2.8609307791587257, 2.85714263589582, 2.8548002383377944, 2.855899635218537, 2.856279790668402, 2.856427495020441, 2.8574167214672457, 2.856079070597067, 2.855934695562947, 2.855674090174933, 2.852888312922482, 2.8494263268859434, 2.851559467828205, 2.8490782415569402, 2.8460689875382457, 2.8534553398783973, 2.8538747834457623, 2.851333999664297, 2.8490348865950788, 2.8481820297790352, 2.8490709061052124, 2.848584371885274, 2.847145310595336, 2.848350900606093, 2.844932130492046, 2.8510359364751814, 2.8449713056543584, 2.846125233119982, 2.8474715148037393, 2.8470105153204, 2.843914905573722, 2.845752934195335, 2.8424098560311286, 2.8445868672122576, 2.845419879304394, 2.8441298737864575, 2.8394353882441212, 2.8428944742077067, 2.8439024985027253, 2.8421717249881895, 2.8422293040665463, 2.8387852316091857, 2.8417400697523862, 2.8368157782161076, 2.8380749829671204, 2.839046724317017, 2.8402731212681864, 2.842623106119958, 2.8399311172146104], 'val_metric': [5.48, 9.99, 11.75, 14.15, 15.29, 18.34, 19.96, 20.53, 22.85, 22.23, 22.54, 22.41, 25.02, 23.76, 24.53, 25.89, 25.77, 26.95, 27.34, 27.76, 26.86, 27.97, 27.74, 28.38, 28.65, 28.7, 28.42, 28.8, 29.28, 29.5, 29.69, 29.32, 29.02, 30.29, 29.99, 29.09, 29.2, 30.38, 30.4, 30.53, 29.94, 31.49, 30.49, 31.4, 30.92, 31.83, 30.54, 31.75, 30.7, 30.26, 30.25, 31.26, 29.76, 31.05, 31.11, 31.4, 31.29, 31.86, 31.67, 31.8, 31.68, 31.16, 30.81, 31.85, 32.09, 31.38, 32.54, 31.64, 32.45, 32.36, 31.77, 31.26, 31.25, 31.84, 32.25, 31.9, 31.87, 31.5, 32.39, 31.36, 31.04, 32.58, 32.08, 32.76, 32.38, 32.39, 31.81, 31.7, 31.74, 32.04, 32.64, 31.45, 32.64, 32.79, 32.24, 32.23, 32.73, 32.5, 32.57, 32.26, 32.38, 32.7, 31.6, 31.44, 32.97, 32.86, 32.58, 31.56, 32.33, 32.69, 32.92, 32.11, 32.85, 33.06, 31.94, 32.37, 33.16, 33.87, 32.37, 33.14, 31.77, 33.03, 32.04, 32.32, 32.28, 32.71, 32.76, 32.09, 32.24, 33.58, 32.48, 32.81, 32.46, 31.31, 32.67, 32.77, 33.31, 32.94, 32.39, 32.11, 32.67, 33.69, 33.23, 33.29, 32.45, 32.54, 33.08, 34.44, 32.67, 34.29, 32.57, 33.19, 33.03, 32.44, 33.74, 32.58, 32.63, 33.5, 34.02, 32.83, 33.62, 33.88, 32.89, 33.06, 33.2, 32.26, 32.71, 33.08, 32.74, 33.24, 32.23, 31.86, 32.85, 33.23, 31.84, 32.86, 32.83, 33.59, 31.85, 33.03, 33.95, 32.56, 32.78, 33.15, 33.32, 33.5, 32.96, 33.58, 32.78, 32.46, 32.28, 30.53, 33.63, 33.89, 31.95, 33.1, 32.84, 32.39, 32.57, 32.76], 'val_loss': [4.595712953312382, 4.158721591256986, 4.015858861291484, 3.8619957304304573, 3.7435600544996324, 3.581736267751949, 3.5116436504254676, 3.4512881025387223, 3.3606339108412433, 3.3699298960388084, 3.3419621765233907, 3.393586344779677, 3.250015538209563, 3.2764442589632266, 3.2452207910027475, 3.2048412174176257, 3.2059288488072193, 3.141644878751913, 3.1368391005096923, 3.09859196699349, 3.1294244527816772, 3.0966988718433743, 3.088313037422812, 3.0721816963450923, 3.0668600653387177, 3.0528125140317686, 3.0606615702817392, 3.0665539867559057, 3.01449472661231, 3.039436836910855, 3.0001075472801353, 3.0108835416234982, 3.0317217627908013, 2.9884413761697757, 2.9815664071186334, 3.010049004463633, 2.9829961091849455, 2.960479098520461, 2.9565707202170306, 2.9529924817905306, 2.968355454457034, 2.94318093084226, 2.973353114097741, 2.9316505193710327, 2.9404062222523293, 2.910820188795685, 2.948816969136524, 2.9193665510529927, 2.9310005774163894, 2.9660092895957315, 2.971728905750688, 2.901754585800657, 2.9924169027121965, 2.933862477351146, 2.9180458032401506, 2.9046379062020855, 2.9264555903756695, 2.8956005717538726, 2.9223660701399394, 2.893345331690114, 2.9059834252497194, 2.937293019264367, 2.9186590249371376, 2.900800227739249, 2.8771436358713043, 2.9290336192033855, 2.8612486025330366, 2.8888795186000267, 2.8747018006197207, 2.8902991751956333, 2.888215161432886, 2.9180867846604364, 2.9324237761224152, 2.906077611977887, 2.873042158260467, 2.8871345823737466, 2.884312783836559, 2.9000815206272588, 2.886203984546054, 2.9237953158700543, 2.945258893784444, 2.8752177475364347, 2.8720237859495126, 2.861004858259942, 2.8709924296968303, 2.8646890054083176, 2.873344818497919, 2.896242146279402, 2.9027754379685518, 2.8843049471545372, 2.858886045255479, 2.8992004812143413, 2.8947868096600673, 2.8504993103112386, 2.8583812721215995, 2.894216276278162, 2.8537221767340495, 2.839009756495239, 2.8495862931962224, 2.869765881520168, 2.855503098979877, 2.8561360395638045, 2.88548553977043, 2.8887529555399705, 2.830895336570254, 2.838930434482113, 2.844249860496278, 2.8889827675120845, 2.8557947866476265, 2.830208642467572, 2.8524169382775666, 2.8486050997570063, 2.8550164790669825, 2.8390608775387904, 2.8658005424365878, 2.8585828178247827, 2.8174531816676924, 2.795511024013446, 2.828404144117027, 2.8304140324805194, 2.8823201534854377, 2.8314621266286086, 2.88182921698139, 2.849196088541845, 2.849576936405935, 2.841734850482576, 2.8254770647947955, 2.839536340373337, 2.866978318068632, 2.792810471194565, 2.8380275979922835, 2.8430583241638865, 2.851606334850287, 2.9085254327506775, 2.847390834692937, 2.8246456938944045, 2.8058273336689945, 2.8297831905875235, 2.85787818765944, 2.863485264929996, 2.83837626010749, 2.78448868177499, 2.8259901282893622, 2.8024565512966957, 2.8319646750286127, 2.8245701911343133, 2.819559505031367, 2.783452330121569, 2.820730232129431, 2.7903899037913913, 2.8324338820329897, 2.8164978270318097, 2.822373849571131, 2.8214130257345307, 2.7889312217190008, 2.822621590392605, 2.8323291578110616, 2.8032987861876277, 2.789862792962676, 2.8460380765283184, 2.8088402003998967, 2.7949985136651687, 2.8418817360689688, 2.810878232026556, 2.819286908313727, 2.8406009727222905, 2.863070511514214, 2.814617863885916, 2.847089339213766, 2.8058397610476065, 2.866350451852106, 2.8937517184360773, 2.8311970006128786, 2.810893332122997, 2.8622372044119864, 2.8215219184851192, 2.8271044287711953, 2.796231307041873, 2.8864679184688886, 2.830112605337884, 2.7822490292749587, 2.8269606107359477, 2.851303692835911, 2.825764072928459, 2.788658357729578, 2.791342726938284, 2.8262590466031603, 2.806887702577433, 2.800299006662551, 2.8419410339586295, 2.8471675594900825, 2.940648752412978, 2.7834007371762755, 2.779388320673803, 2.874601031564603, 2.828546339539206, 2.8164814148738886, 2.8523562258216226, 2.8250359866269834, 2.8320225629077593]})
data = database.get_data(model="EfficientNetV2", dataset="Tiny-Imagenet", initialization="LSUV", optimizer="AdamW")
data.update({'train_metric': [4.904, 16.541, 26.748, 34.911, 41.508, 47.331, 52.735, 58.123, 63.404, 69.098, 74.14, 78.627, 82.404, 85.008, 87.167, 88.619, 89.709, 90.763, 91.308, 92.037, 92.614, 92.87, 93.407, 93.849, 94.106, 94.268, 94.514, 94.739, 94.856, 95.071, 95.399, 95.556, 95.506, 96.004, 95.834, 96.087, 96.145, 96.318, 96.266, 96.454, 96.64, 96.404, 96.871, 96.686, 96.941, 96.737, 97.017, 96.974, 97.035, 97.106, 97.177, 97.239, 97.308, 97.333, 97.332, 97.383, 97.586, 97.401, 97.466, 97.668, 97.531, 97.599, 97.706, 97.591, 97.724, 97.844, 97.776, 97.662, 97.935, 97.845, 97.854, 97.909, 97.987, 97.918, 98.014, 97.879, 98.077, 98.051, 98.072, 98.111, 98.218, 98.122, 98.039, 98.299, 98.115, 98.228, 98.121, 98.289, 98.185, 98.259, 98.198, 98.319, 98.305, 98.213, 98.427, 98.411, 98.296, 98.366, 98.359, 98.437, 98.395, 98.437, 98.373, 98.513, 98.375, 98.342, 98.56, 98.409, 98.46, 98.52, 98.385, 98.578, 98.521, 98.451, 98.553, 98.642, 98.583, 98.518, 98.596, 98.641, 98.518, 98.604, 98.637, 98.67, 98.634, 98.541, 98.638, 98.629, 98.715, 98.7, 98.66, 98.567, 98.709, 98.72, 98.681, 98.671, 98.699, 98.697, 98.698, 98.706, 98.606, 98.799, 98.676, 98.781, 98.791, 98.633, 98.796, 98.732, 98.782, 98.748, 98.886, 98.742, 98.809, 98.789, 98.864, 98.737, 98.791, 98.801, 98.856, 98.852, 98.747, 98.836, 98.861, 98.85, 98.771, 98.841, 98.897, 98.83, 98.893, 98.889, 98.801, 98.848, 98.925, 98.804, 98.899, 98.939, 98.866, 98.933, 98.811, 98.918, 98.972, 98.873, 98.897, 98.98, 98.907, 98.965, 98.938, 99.041, 98.931, 98.896, 98.985, 98.991, 98.951, 98.91, 98.947, 98.988, 99.005, 98.881, 98.893, 99.013], 'train_loss': [4.714088362909171, 3.744072317085583, 3.1488488843177116, 2.730119340708068, 2.400947015329728, 2.116568048184908, 1.8585459471359058, 1.6042394638442872, 1.364163203645195, 1.1199734573248328, 0.9042271462222055, 0.730407240695093, 0.5849636467751675, 0.4893513039393221, 0.41598144548772925, 0.36682125742970867, 0.32729373756445757, 0.29391539667423766, 0.2774439220932227, 0.2550870130006617, 0.2359684070551998, 0.22714556631127458, 0.21318822315943523, 0.19609545359312916, 0.19012640544731152, 0.18244977373813331, 0.1758444443492842, 0.16657123695260542, 0.1626577322321848, 0.15489927303076972, 0.14782400275756363, 0.1427148066214402, 0.14270937956585975, 0.12806666950008194, 0.13136629193159619, 0.12427565817717016, 0.12039191224829783, 0.11723212919049608, 0.1191728559596153, 0.11267865108539393, 0.10673677446100187, 0.1127814272625976, 0.09974502791905024, 0.10437397148563829, 0.09644702001714928, 0.10430316812649575, 0.09575558458743896, 0.09668603840552781, 0.0942240798359073, 0.09231383290013116, 0.0898946804183363, 0.08820113774016022, 0.08746602222405646, 0.08490511909173712, 0.08529242396000737, 0.08214759462100697, 0.07817285626448015, 0.07987658105929449, 0.08020016919658832, 0.0745949352945732, 0.07822544498861275, 0.07692580303637238, 0.07291865724368916, 0.07665739117010889, 0.07217379015845367, 0.06844356958784154, 0.07178862210421544, 0.07264633437362991, 0.06666483711479596, 0.06988406622305963, 0.06855270210142597, 0.06653157032514379, 0.06443088048699915, 0.06533302892725236, 0.06307547092878678, 0.06762241491373397, 0.060995932640889164, 0.06252431451726284, 0.06174333367028386, 0.05989146735417875, 0.057996882019226065, 0.059337829782193145, 0.06235445158687788, 0.05398121242575593, 0.05990316091142285, 0.05626667488504306, 0.06005691920903315, 0.05562542261265609, 0.05762014924134447, 0.05529959713226817, 0.057329696498621165, 0.05474081575144762, 0.05393440803240029, 0.05666246208000597, 0.04995315726672914, 0.05104301647741648, 0.0536836782900181, 0.05201552187782901, 0.05243638521491598, 0.049956125780831015, 0.05131014958460489, 0.050808899430304845, 0.05155109065134669, 0.04669014969159136, 0.05210508559412643, 0.05122655899691743, 0.04414452796593971, 0.050645071247502296, 0.04823661632473883, 0.04653826109711641, 0.0508579718966382, 0.04445719347781992, 0.04776249792191759, 0.04843196553252316, 0.046477268435177466, 0.043174233997609494, 0.045819899005239195, 0.0486881826825944, 0.04421431209352189, 0.04264064481510139, 0.04843279617934346, 0.04356626781087693, 0.043149417745205976, 0.04163110652684948, 0.04271034693384307, 0.04634215323421604, 0.04377578036527889, 0.04320082826956526, 0.04106008307796039, 0.04197124227704835, 0.04168447420802158, 0.04577027487380714, 0.040253678235965436, 0.0407494565832723, 0.041699482245184244, 0.042303718475673185, 0.041519000333354296, 0.04228273631497039, 0.042134715535770924, 0.04084998735025813, 0.04368256698693229, 0.03755264269180673, 0.04155211036921549, 0.03810395451981493, 0.038669178997787934, 0.04351784211824161, 0.03906430774694577, 0.040156491088535834, 0.03927992662580705, 0.039180068902776206, 0.035781103847206494, 0.03927154099581833, 0.03822600361865767, 0.03815100947331657, 0.03664530480055732, 0.038894485088851204, 0.038438740294336256, 0.036797642416898126, 0.03667648915600351, 0.03653179637244464, 0.038919816964024544, 0.03728196442276788, 0.03601179117475704, 0.03826091695591505, 0.038017417554533105, 0.036248224243658436, 0.03543795723839656, 0.03650005051951605, 0.03479665408696504, 0.03490058064671315, 0.03844528783793008, 0.03721727851550503, 0.03439698213198498, 0.03720487200934201, 0.03562306480708869, 0.03415339018822417, 0.03551943472243806, 0.033753397316124166, 0.03759971002545034, 0.03532200895407894, 0.03389986653377478, 0.03462889319740016, 0.03416155646063172, 0.03265986766077617, 0.03563859899556926, 0.03308878646962334, 0.03379752593067974, 0.032121769489277446, 0.033338146730723404, 0.03580707061800957, 0.03339906762139679, 0.03201000325695549, 0.03392243450730595, 0.03450175896695498, 0.034278749982388904, 0.031870575126363154, 0.03084676412337666, 0.035211892800116405, 0.0339473048007744, 0.030824747572859162], 'val_metric': [11.83, 21.32, 31.09, 37.86, 41.52, 44.53, 46.7, 47.45, 48.59, 48.55, 48.67, 48.69, 48.6, 47.45, 48.51, 48.4, 48.55, 48.64, 48.55, 48.51, 48.63, 49.2, 49.19, 49.32, 48.98, 48.9, 48.29, 49.14, 49.32, 49.29, 49.43, 49.66, 49.45, 48.98, 48.94, 49.1, 49.41, 49.0, 49.62, 49.97, 50.21, 48.97, 49.27, 49.57, 49.35, 49.96, 49.45, 50.22, 50.04, 50.29, 50.18, 49.73, 50.23, 50.27, 49.95, 50.0, 50.48, 49.81, 50.1, 50.04, 49.66, 50.21, 50.28, 50.13, 49.46, 50.01, 50.04, 49.9, 49.05, 49.69, 50.06, 50.29, 50.94, 50.66, 50.76, 50.82, 50.48, 50.53, 50.46, 50.75, 50.46, 50.16, 50.14, 50.08, 50.07, 50.03, 50.7, 49.86, 49.73, 50.38, 49.59, 50.48, 50.51, 50.85, 50.89, 50.91, 50.57, 50.69, 50.96, 50.88, 50.24, 50.74, 50.41, 50.37, 50.33, 50.71, 50.68, 50.39, 50.51, 51.18, 51.13, 51.21, 50.55, 51.22, 50.76, 50.56, 50.95, 51.47, 50.7, 51.07, 51.08, 51.03, 50.62, 51.35, 51.1, 51.75, 50.99, 51.4, 50.19, 50.85, 51.17, 51.41, 51.19, 50.79, 51.06, 50.82, 51.53, 51.12, 51.29, 51.79, 50.79, 51.36, 51.09, 51.17, 50.85, 50.88, 50.83, 50.68, 50.8, 51.12, 51.87, 52.03, 51.54, 51.72, 51.36, 51.16, 51.9, 50.84, 51.52, 50.83, 51.23, 51.55, 50.92, 51.76, 50.56, 51.41, 51.47, 50.23, 50.71, 51.76, 50.88, 51.13, 51.24, 50.98, 50.86, 51.06, 51.27, 52.04, 51.55, 51.17, 51.44, 50.99, 51.35, 50.8, 51.97, 51.1, 51.2, 51.5, 50.91, 51.19, 51.09, 51.05, 51.33, 51.09, 52.16, 51.43, 50.98, 51.26, 51.25, 50.98], 'val_loss': [4.07066428281699, 3.393641241796457, 2.9302952016235158, 2.602834528418863, 2.419753912907497, 2.269894658000606, 2.1776832964769595, 2.187560978968432, 2.2161111072370203, 2.2878127853581858, 2.3926511722005857, 2.464887950071104, 2.612202266598963, 2.7791816884545004, 2.85426661998603, 2.879700576803487, 2.9483678804081714, 3.043529838513417, 3.058532160178871, 3.125542433018897, 3.1677610540086296, 3.1749392117664312, 3.1883830348397515, 3.200181622034425, 3.289606213569641, 3.283512570675771, 3.3695087079789228, 3.310613401376518, 3.346414401652707, 3.3371724323102625, 3.4490207927242205, 3.39949918666463, 3.422504880625731, 3.4356061220169067, 3.479165309553693, 3.550128837299954, 3.542191767388848, 3.5615200707866888, 3.5300727015847615, 3.504945379913233, 3.5350835171474775, 3.58255159399312, 3.619448696731762, 3.5402346295156297, 3.6477938222277695, 3.6300234111251344, 3.612979479656098, 3.562405254810479, 3.5873537082581004, 3.5684903017274894, 3.5927016435155443, 3.624377287876834, 3.640371246322705, 3.588052106131414, 3.636967947148973, 3.66912218719531, 3.672409647968924, 3.6748996342823004, 3.664622337195524, 3.7316968706762714, 3.740521513352728, 3.7156083690133066, 3.7145283628421226, 3.7020404437545, 3.7591830659064516, 3.742560174814455, 3.738977308485918, 3.750050153701928, 3.8417837205965806, 3.8358630393720734, 3.7816719063528024, 3.7812636695849666, 3.778155464275627, 3.7272702987027015, 3.7657975896148925, 3.6911115278104307, 3.796041555845054, 3.8129519262131613, 3.8147171974941423, 3.7762377991038525, 3.838133082268344, 3.8355895941424523, 3.842730371056089, 3.8865058019662357, 3.9201601565263835, 3.8175709566492944, 3.83759939632598, 3.8354424439418087, 3.8579108058267337, 3.874426673931681, 3.9060529652674485, 3.8400808045059254, 3.814884333853509, 3.7657972676738813, 3.871534013444451, 3.838266701455329, 3.8044997525822586, 3.841252179282486, 3.8870249918312023, 3.876934866237033, 3.913491657205448, 3.8532253511392387, 3.9070599398035912, 3.95447984812366, 3.979996232849777, 3.9249105350986406, 3.99046148691967, 3.9899306198593916, 3.9038570117039284, 3.8657519183341105, 3.911593210165668, 3.8718955949613245, 3.9493407937371807, 3.860247788915209, 3.887582026089832, 3.981953872996531, 3.966924056505701, 3.925940938815949, 3.922235408406349, 3.9603250429129147, 3.8831632771309774, 3.9169947372120655, 3.9774305721756757, 3.8929432387564593, 3.9394373415382047, 3.880216738980287, 3.945320715570146, 3.9695557371066634, 4.057939902992006, 3.9864065066264693, 3.938668109049463, 3.9135741674975986, 3.903136638699064, 3.920646769225977, 3.9948079536674888, 3.959209210933394, 3.9230589946364143, 3.9622169672303897, 3.947954381347462, 3.9223847146246844, 3.922974105853184, 3.9062273737731252, 3.9576065859217553, 3.967489401246332, 3.992500488925132, 3.940348870435338, 3.9698589675745386, 3.936703113993262, 4.001036608295077, 3.9500500487673813, 3.9743538039505104, 3.984960657775782, 3.9998215455917796, 4.0189952349207205, 4.0094236150668685, 4.036701635190636, 3.9626774112130425, 4.010142071611562, 3.9535315583465964, 4.041683357232695, 3.969869880919244, 3.94275267564567, 3.976390191703845, 3.9966807502090553, 3.9313790452708104, 3.9445467671011665, 4.001527761957448, 4.0426728482459, 4.057015086055562, 4.08219522199813, 4.049480113254231, 4.063948953986928, 4.040881410525863, 4.016378058369752, 4.079352528805945, 4.011495146781775, 3.9912189822288076, 3.96868903545817, 4.01372977123139, 4.044016925772285, 4.050932121124997, 4.1224496060875575, 4.104792058847512, 4.098390661227476, 4.009796583348779, 4.079873465428687, 4.070661319289238, 4.0547703413447, 4.104301147400194, 4.07633715602243, 4.028834563152046, 4.112189918187014, 3.9685042041122536, 4.032737258133615, 4.011775520197146, 4.021172933897395, 4.094906097005127, 4.085720055422206, 4.0131073882625365, 4.070653658763618]})


# BERT-mini WikiText
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="Default initialization", optimizer="AdamW")
data.update({'train_loss': [7.5363, 6.9873, 6.8727, 6.7951, 6.7285, 6.6668, 6.6134, 6.5562, 6.5086, 6.4646, 6.4227, 6.3988, 6.3762, 6.3367, 6.3097, 6.2876, 6.2684, 6.24, 6.2192, 6.2098, 6.1878, 6.1758, 6.1553, 6.1359, 6.1201, 6.093, 6.0846, 6.0693, 6.0599, 6.0408, 6.031, 6.0105, 6.0012, 5.9906, 5.9738, 5.9558, 5.952, 5.9366, 5.9262, 5.9135, 5.906, 5.8951, 5.8821, 5.873, 5.8632, 5.8466, 5.8468, 5.8294, 5.831, 5.8157, 5.7969, 5.7941, 5.7684, 5.7429, 5.7357, 5.7193, 5.6937, 5.685, 5.6617, 5.6236, 5.5669, 5.5447, 5.4959, 5.4521, 5.3817, 5.3238, 5.2526, 5.1982, 5.1537, 5.0987, 5.0394, 5.003, 4.9645, 4.9193, 4.8825, 4.8498, 4.8099, 4.7775, 4.7263, 4.6906, 4.6294, 4.5718, 4.5183, 4.4704, 4.4194, 4.3755, 4.3228, 4.2926, 4.2513, 4.1977, 4.172, 4.129, 4.0871, 4.0564, 4.0173, 3.9821, 3.9657, 3.9368, 3.9011, 3.8722, 3.8366, 3.8154, 3.7915, 3.7684, 3.7488, 3.7261, 3.7035, 3.6769, 3.6569, 3.6393, 3.617, 3.5935, 3.5752, 3.5572, 3.5288, 3.5234, 3.4883, 3.4903, 3.4662, 3.4463, 3.4409, 3.4159, 3.4057, 3.3955, 3.3838, 3.3642, 3.3424, 3.3425, 3.3298, 3.301, 3.2921, 3.2705, 3.2588, 3.2524, 3.2412, 3.234, 3.2185, 3.2054, 3.1893, 3.1813, 3.167, 3.1574, 3.1426, 3.1383, 3.124, 3.1211, 3.1171, 3.0983, 3.09, 3.0765, 3.066, 3.0559, 3.0465, 3.0373, 3.0207, 3.0234, 3.0106, 3.0086, 2.992, 2.9861, 2.9635, 2.9671, 2.9583, 2.9523, 2.9454, 2.9296, 2.9256, 2.9158, 2.9169, 2.8987, 2.8966, 2.8866, 2.8746, 2.8732, 2.8673, 2.8582, 2.8452, 2.8459, 2.8351, 2.8326, 2.8185, 2.8076, 2.8102, 2.8039, 2.7934, 2.7939, 2.7845, 2.7781, 2.7692, 2.7673, 2.7697, 2.7535, 2.7527, 2.7427, 2.7282, 2.7285, 2.7237, 2.7135, 2.7128, 2.703], 'val_metric': [0.09461323916912079, 0.09938714653253555, 0.10629073530435562, 0.11283937096595764, 0.11649756878614426, 0.12067950516939163, 0.1271970570087433, 0.1291804015636444, 0.134076789021492, 0.1351054608821869, 0.1382254958152771, 0.13696740567684174, 0.14228902757167816, 0.1445142924785614, 0.14175322651863098, 0.1463770568370819, 0.14677613973617554, 0.14635515213012695, 0.14806580543518066, 0.14585663378238678, 0.14969277381896973, 0.14767742156982422, 0.14855103194713593, 0.14799100160598755, 0.14885123074054718, 0.14989401400089264, 0.1498226523399353, 0.1482110470533371, 0.1470341831445694, 0.15056148171424866, 0.15108512341976166, 0.15197813510894775, 0.15137794613838196, 0.15202994644641876, 0.15295659005641937, 0.1532858908176422, 0.1513301581144333, 0.1533786654472351, 0.1535063087940216, 0.15298964083194733, 0.1528065949678421, 0.15251348912715912, 0.148220956325531, 0.1488771140575409, 0.1532556414604187, 0.15024830400943756, 0.15261416137218475, 0.1544036716222763, 0.15757900476455688, 0.15591353178024292, 0.1529630720615387, 0.1583569496870041, 0.1618228256702423, 0.1693432778120041, 0.17112477123737335, 0.17020313441753387, 0.16996783018112183, 0.1742565929889679, 0.17889535427093506, 0.18662384152412415, 0.19634361565113068, 0.20140372216701508, 0.20492447912693024, 0.21144944429397583, 0.2212371975183487, 0.2283855825662613, 0.2345389574766159, 0.24135705828666687, 0.242228165268898, 0.2495587170124054, 0.2510492205619812, 0.257677286863327, 0.26208430528640747, 0.2606334090232849, 0.26377183198928833, 0.26898741722106934, 0.26495224237442017, 0.2803221642971039, 0.27959102392196655, 0.28492528200149536, 0.28803330659866333, 0.29809850454330444, 0.29810479283332825, 0.3047957718372345, 0.30923378467559814, 0.3105546534061432, 0.3153798580169678, 0.31657344102859497, 0.3201165795326233, 0.31889012455940247, 0.324662983417511, 0.32537421584129333, 0.327322393655777, 0.3318333029747009, 0.33379727602005005, 0.33365124464035034, 0.33577650785446167, 0.3409990668296814, 0.33981868624687195, 0.3395598530769348, 0.34554314613342285, 0.3450138568878174, 0.34887829422950745, 0.33965861797332764, 0.34402307868003845, 0.3498897850513458, 0.35111597180366516, 0.35121411085128784, 0.35262906551361084, 0.35484951734542847, 0.35434916615486145, 0.3548918664455414, 0.3544834852218628, 0.35663217306137085, 0.36112967133522034, 0.36000001430511475, 0.36063551902770996, 0.3620699346065521, 0.36279237270355225, 0.3654271364212036, 0.3668321967124939, 0.3658132255077362, 0.36588647961616516, 0.3710380494594574, 0.3690881133079529, 0.3752708435058594, 0.3720799386501312, 0.3745921850204468, 0.37424030900001526, 0.3737131953239441, 0.37628576159477234, 0.3718092441558838, 0.3750527799129486, 0.37265491485595703, 0.37901145219802856, 0.37585997581481934, 0.3776155412197113, 0.37772321701049805, 0.3749162256717682, 0.3803620934486389, 0.3794197738170624, 0.38629552721977234, 0.38119032979011536, 0.37857022881507874, 0.3874675929546356, 0.38133886456489563, 0.38662344217300415, 0.3836319148540497, 0.39069560170173645, 0.3864259719848633, 0.3858073353767395, 0.3902616500854492, 0.38442280888557434, 0.38928931951522827, 0.3890470564365387, 0.38772958517074585, 0.38626202940940857, 0.38946011662483215, 0.39135879278182983, 0.3924829959869385, 0.3860021233558655, 0.3923130929470062, 0.39113864302635193, 0.39152175188064575, 0.39370715618133545, 0.3938366770744324, 0.39490053057670593, 0.39165571331977844, 0.3963509798049927, 0.39562684297561646, 0.398031085729599, 0.39166271686553955, 0.3997582197189331, 0.397821307182312, 0.3941517472267151, 0.39161860942840576, 0.39745426177978516, 0.39162588119506836, 0.3968382775783539, 0.40122491121292114, 0.40164873003959656, 0.4005623459815979, 0.4004359841346741, 0.40146520733833313, 0.3990442454814911, 0.39882391691207886, 0.40253713726997375, 0.39715972542762756, 0.3984789252281189, 0.4038547873497009, 0.3994593322277069, 0.4026995301246643, 0.39944547414779663, 0.39984312653541565, 0.3975917100906372, 0.404975026845932, 0.4037022590637207, 0.3990505337715149, 0.40143778920173645, 0.40175503492355347], 'val_perplexity': [1167.647705078125, 1011.3155517578125, 947.2588500976562, 873.9374389648438, 847.5292358398438, 811.3309936523438, 758.7297973632812, 714.9442749023438, 700.560546875, 683.1981201171875, 673.5028076171875, 658.0140991210938, 631.3950805664062, 607.822021484375, 610.7193603515625, 593.398193359375, 599.0452270507812, 594.7000732421875, 580.88037109375, 576.6796875, 565.3199462890625, 556.1347045898438, 559.7343139648438, 547.1480712890625, 575.0965576171875, 536.170654296875, 548.2192993164062, 535.345703125, 544.8920288085938, 521.2632446289062, 527.3933715820312, 518.1002807617188, 523.8331909179688, 522.4658813476562, 505.1377258300781, 528.6051635742188, 528.7881469726562, 496.41143798828125, 519.733642578125, 503.1959533691406, 517.2945556640625, 513.4033203125, 511.78497314453125, 509.9226989746094, 497.3838195800781, 496.8478698730469, 503.8060302734375, 496.2100524902344, 476.74951171875, 485.4791259765625, 499.53607177734375, 480.4117431640625, 484.7463073730469, 456.60333251953125, 463.8334045410156, 470.48260498046875, 464.13604736328125, 453.55364990234375, 444.0513000488281, 418.33837890625, 392.7976989746094, 376.2638854980469, 354.52423095703125, 339.3710632324219, 304.5731201171875, 283.3908996582031, 262.3001403808594, 247.45306396484375, 239.12852478027344, 228.7698211669922, 223.0745391845703, 209.74578857421875, 200.99209594726562, 198.34739685058594, 192.34519958496094, 184.5504150390625, 186.62579345703125, 168.17454528808594, 165.82725524902344, 161.531494140625, 153.4987335205078, 140.9803924560547, 138.74505615234375, 129.98731994628906, 121.51513671875, 117.18522644042969, 113.63550567626953, 111.35828399658203, 105.87004852294922, 106.96876525878906, 102.31258392333984, 99.04954528808594, 98.94630432128906, 93.62322998046875, 93.79901885986328, 89.122314453125, 89.69021606445312, 87.36837005615234, 84.89387512207031, 85.83033752441406, 82.6728515625, 80.40082550048828, 79.7298812866211, 83.19505310058594, 82.66259765625, 77.23526000976562, 75.79537200927734, 76.05364990234375, 73.60420989990234, 76.06431579589844, 74.50746154785156, 73.82935333251953, 74.73479461669922, 72.18106079101562, 70.51315307617188, 70.65636444091797, 70.8205337524414, 69.49039459228516, 66.67118072509766, 67.93592834472656, 67.6891098022461, 66.97021484375, 66.20343017578125, 66.81908416748047, 65.63910675048828, 63.27857208251953, 65.43791198730469, 63.677574157714844, 63.28623580932617, 63.23978042602539, 62.425662994384766, 61.98160934448242, 62.93975830078125, 61.69076156616211, 60.15317916870117, 60.16270065307617, 60.15991973876953, 62.35694122314453, 62.01933288574219, 59.43533706665039, 60.14549255371094, 57.311866760253906, 58.5590934753418, 58.655479431152344, 59.15435028076172, 58.46752166748047, 58.17105484008789, 56.983177185058594, 56.23399353027344, 58.170997619628906, 57.3217887878418, 56.54134750366211, 56.02143859863281, 56.41681671142578, 56.418216705322266, 57.758338928222656, 56.43008041381836, 55.702354431152344, 55.093631744384766, 53.93959045410156, 57.169307708740234, 55.18681335449219, 55.20063400268555, 55.41490936279297, 55.15269470214844, 54.10774230957031, 54.840335845947266, 55.27166748046875, 53.94260025024414, 53.58513641357422, 52.8623161315918, 53.78089904785156, 51.57100296020508, 52.610294342041016, 53.849388122558594, 54.889705657958984, 53.52262496948242, 54.71433639526367, 52.778907775878906, 52.77994155883789, 52.514488220214844, 52.362613677978516, 52.06482696533203, 51.34021759033203, 52.68231964111328, 52.849525451660156, 51.29686737060547, 53.436622619628906, 51.5932731628418, 50.94160842895508, 52.81557846069336, 51.79966354370117, 52.353675842285156, 51.585391998291016, 52.78749084472656, 50.687034606933594, 50.44987106323242, 53.46218490600586, 51.416961669921875, 51.814151763916016]})
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="Sinusoidal", optimizer="AdamW")
data.update({'train_loss': [7.6584, 7.1844, 7.1321, 7.0407, 6.925, 6.7976, 6.6709, 6.5321, 6.4002, 6.2814, 6.1444, 6.021, 5.894, 5.7743, 5.6737, 5.5839, 5.4976, 5.4244, 5.3474, 5.2849, 5.2155, 5.158, 5.0975, 5.039, 4.9859, 4.9208, 4.8778, 4.835, 4.7931, 4.7463, 4.7113, 4.6637, 4.6276, 4.5886, 4.546, 4.5117, 4.4782, 4.4407, 4.4112, 4.3763, 4.3561, 4.3133, 4.2855, 4.2585, 4.2293, 4.1937, 4.1757, 4.1477, 4.1228, 4.0997, 4.069, 4.0468, 4.0266, 3.9953, 3.9786, 3.9654, 3.9314, 3.9222, 3.8887, 3.8741, 3.8473, 3.8367, 3.8169, 3.8054, 3.7818, 3.7616, 3.7345, 3.7271, 3.7225, 3.7051, 3.675, 3.6709, 3.658, 3.6397, 3.6204, 3.6138, 3.5924, 3.5861, 3.5601, 3.5629, 3.5495, 3.527, 3.5135, 3.4963, 3.482, 3.4685, 3.4588, 3.4572, 3.4443, 3.4224, 3.4205, 3.4038, 3.3937, 3.3864, 3.3651, 3.3546, 3.3563, 3.3442, 3.3307, 3.3181, 3.3073, 3.3029, 3.2916, 3.2867, 3.2798, 3.2691, 3.261, 3.2481, 3.2371, 3.2361, 3.2213, 3.2119, 3.2044, 3.198, 3.1765, 3.1832, 3.1603, 3.1637, 3.1496, 3.1399, 3.1477, 3.1273, 3.1245, 3.1221, 3.1201, 3.1064, 3.0869, 3.0961, 3.0892, 3.0704, 3.0671, 3.0534, 3.0469, 3.0414, 3.0389, 3.0353, 3.032, 3.0168, 3.0125, 3.0075, 2.9961, 2.9903, 2.9877, 2.9844, 2.9712, 2.9737, 2.9764, 2.9611, 2.9595, 2.9478, 2.9381, 2.9328, 2.9283, 2.9237, 2.9093, 2.914, 2.9086, 2.9141, 2.8943, 2.8954, 2.8714, 2.8829, 2.8764, 2.8766, 2.868, 2.8537, 2.8578, 2.8477, 2.8538, 2.8382, 2.8415, 2.8258, 2.8231, 2.8203, 2.8194, 2.8123, 2.8037, 2.8058, 2.7995, 2.7942, 2.7833, 2.7768, 2.7814, 2.779, 2.7673, 2.7711, 2.7619, 2.7632, 2.7479, 2.7507, 2.7573, 2.7435, 2.7458, 2.733, 2.7298, 2.7253, 2.7263, 2.7141, 2.7117, 2.7064], 'val_metric': [0.06344155967235565, 0.0694378986954689, 0.07359234988689423, 0.09214700758457184, 0.10888116806745529, 0.11442828178405762, 0.12989678978919983, 0.15391767024993896, 0.16880293190479279, 0.1780756413936615, 0.19402524828910828, 0.2047334909439087, 0.2144065946340561, 0.22304439544677734, 0.23064647614955902, 0.23558960855007172, 0.2362765222787857, 0.24718818068504333, 0.25386282801628113, 0.2567937970161438, 0.26065561175346375, 0.2664211094379425, 0.2702423930168152, 0.27226749062538147, 0.26799923181533813, 0.2838335335254669, 0.28216317296028137, 0.28937768936157227, 0.2948782444000244, 0.29778215289115906, 0.2980404198169708, 0.29994145035743713, 0.3050757348537445, 0.3062393367290497, 0.31073400378227234, 0.30785053968429565, 0.3145897388458252, 0.32041069865226746, 0.3211391270160675, 0.32162341475486755, 0.32179105281829834, 0.3247268497943878, 0.3284562826156616, 0.3315109312534332, 0.3373849093914032, 0.33814987540245056, 0.33692359924316406, 0.33968809247016907, 0.34554722905158997, 0.34748440980911255, 0.34287649393081665, 0.346819669008255, 0.34696337580680847, 0.3522992432117462, 0.3527717888355255, 0.35452383756637573, 0.35503074526786804, 0.35931873321533203, 0.3576783835887909, 0.3557944595813751, 0.3631209135055542, 0.36211058497428894, 0.36181142926216125, 0.3635603189468384, 0.3668396770954132, 0.36958765983581543, 0.3700323700904846, 0.3674654960632324, 0.36998170614242554, 0.3704855442047119, 0.36934906244277954, 0.3704913556575775, 0.37540003657341003, 0.3723294734954834, 0.37838366627693176, 0.37720680236816406, 0.3751855790615082, 0.3827246129512787, 0.38121452927589417, 0.3765174150466919, 0.3798643946647644, 0.3852647542953491, 0.3819119334220886, 0.3862026035785675, 0.38765084743499756, 0.3853681683540344, 0.3871375620365143, 0.3861611485481262, 0.3910667896270752, 0.3855661451816559, 0.3903619050979614, 0.3868575990200043, 0.39011678099632263, 0.3953377902507782, 0.3905019164085388, 0.39132142066955566, 0.3927212357521057, 0.39145341515541077, 0.394723504781723, 0.392438679933548, 0.39564254879951477, 0.3939419686794281, 0.3971957266330719, 0.39055973291397095, 0.39195725321769714, 0.4001171886920929, 0.3950544595718384, 0.39564311504364014, 0.3978551924228668, 0.3957711458206177, 0.39481717348098755, 0.3942839503288269, 0.396361380815506, 0.397229939699173, 0.3965776264667511, 0.40084150433540344, 0.4002017676830292, 0.4003978371620178, 0.40190693736076355, 0.4036766588687897, 0.40403446555137634, 0.4016750454902649, 0.40162578225135803, 0.40239614248275757, 0.4007306396961212, 0.4049947261810303, 0.40360260009765625, 0.40671443939208984, 0.4051017463207245, 0.4062100052833557, 0.4062447249889374, 0.4037587642669678, 0.40499845147132874, 0.40243834257125854, 0.41135546565055847, 0.40695738792419434, 0.4066801071166992, 0.40556222200393677, 0.40407171845436096, 0.40812042355537415, 0.4095262289047241, 0.4085295796394348, 0.409636527299881, 0.4089900851249695, 0.40828919410705566, 0.40786969661712646, 0.4130355715751648, 0.4113650321960449, 0.4152674674987793, 0.40890008211135864, 0.4114361107349396, 0.41326430439949036, 0.41048678755760193, 0.4130629003047943, 0.41140511631965637, 0.4088749289512634, 0.4104907214641571, 0.41234976053237915, 0.4116959571838379, 0.4158029556274414, 0.40718311071395874, 0.4106326401233673, 0.4129137098789215, 0.4113548994064331, 0.4147762060165405, 0.4137554168701172, 0.4138414263725281, 0.41422221064567566, 0.4148189425468445, 0.4131530225276947, 0.41755229234695435, 0.4114059805870056, 0.41769519448280334, 0.4130777418613434, 0.40825170278549194, 0.41051891446113586, 0.4139200448989868, 0.4116436839103699, 0.4180470108985901, 0.41657811403274536, 0.4202319383621216, 0.4148179292678833, 0.41362735629081726, 0.4195829927921295, 0.414051353931427, 0.4153677225112915, 0.4195263385772705, 0.41425758600234985, 0.414559543132782, 0.4198462665081024, 0.41687801480293274, 0.4168665409088135, 0.41487663984298706, 0.41365304589271545, 0.41390952467918396, 0.41894641518592834, 0.41526132822036743, 0.41510751843452454, 0.4167109429836273, 0.4152809977531433], 'val_perplexity': [1378.584716796875, 1292.4534912109375, 1265.1123046875, 1113.2325439453125, 998.0388793945312, 881.8883056640625, 759.23828125, 643.3222045898438, 576.9722900390625, 505.7889709472656, 440.71563720703125, 385.50238037109375, 341.71246337890625, 305.0086975097656, 282.87030029296875, 258.1652526855469, 252.7965850830078, 233.20538330078125, 216.84292602539062, 206.00332641601562, 193.0596160888672, 182.2854766845703, 177.28695678710938, 170.7037811279297, 171.5435028076172, 153.477294921875, 152.2681884765625, 145.24551391601562, 140.572998046875, 134.76956176757812, 133.00035095214844, 128.30633544921875, 123.76022338867188, 123.16520690917969, 114.01984405517578, 115.9287109375, 112.14924621582031, 105.30785369873047, 107.5376205444336, 102.6552505493164, 103.04086303710938, 100.52887725830078, 96.00430297851562, 97.19991302490234, 92.01676177978516, 89.9478759765625, 90.40116882324219, 88.43791198730469, 83.73544311523438, 83.11016082763672, 84.62934112548828, 82.2504653930664, 82.84269714355469, 79.3887939453125, 78.72187042236328, 78.5095443725586, 77.8769302368164, 75.1526870727539, 74.8751220703125, 74.99653625488281, 72.326171875, 73.0226058959961, 71.2400131225586, 70.72344207763672, 68.02464294433594, 67.73127746582031, 67.58277893066406, 67.39413452148438, 68.0823745727539, 66.28072357177734, 66.33365631103516, 65.81248474121094, 63.7170295715332, 64.40741729736328, 64.013916015625, 63.428955078125, 64.02893829345703, 60.84590530395508, 61.18336868286133, 64.2189712524414, 62.45814895629883, 60.15687942504883, 61.04319763183594, 59.446956634521484, 58.63361358642578, 58.196189880371094, 56.72368240356445, 57.94343566894531, 57.11955261230469, 58.47407531738281, 57.12497329711914, 56.44773483276367, 57.83409881591797, 55.091609954833984, 56.29283142089844, 54.6162223815918, 54.647220611572266, 54.799556732177734, 54.069828033447266, 55.385719299316406, 54.085262298583984, 53.48946762084961, 53.05030059814453, 55.76041793823242, 55.33746337890625, 52.20058059692383, 52.49613952636719, 53.10181427001953, 51.64945983886719, 54.796630859375, 54.51889419555664, 52.57561111450195, 53.6360969543457, 52.470985412597656, 52.55076217651367, 52.182376861572266, 53.14463806152344, 52.345375061035156, 50.46416473388672, 51.071022033691406, 50.385982513427734, 50.69039535522461, 50.22806930541992, 52.415348052978516, 50.5526008605957, 50.05010986328125, 50.36711502075195, 49.59417724609375, 49.6743049621582, 49.82585525512695, 50.001373291015625, 49.19886016845703, 50.14670181274414, 49.822566986083984, 48.29145812988281, 48.69745635986328, 48.62004089355469, 50.417694091796875, 49.972389221191406, 48.61454772949219, 48.93633270263672, 47.869625091552734, 47.96970748901367, 48.59101104736328, 49.57235336303711, 49.09974670410156, 48.60310745239258, 47.340213775634766, 47.24211883544922, 49.14789581298828, 48.1724853515625, 47.52820587158203, 47.71055221557617, 47.18952560424805, 48.217002868652344, 49.987178802490234, 48.058563232421875, 47.71414566040039, 47.637115478515625, 46.00143814086914, 49.52260971069336, 48.21696853637695, 47.963565826416016, 47.723167419433594, 47.883060455322266, 47.36676788330078, 48.18191909790039, 48.15209197998047, 47.43049240112305, 47.622840881347656, 45.86115646362305, 47.12118148803711, 45.46967697143555, 46.43389129638672, 48.10138702392578, 48.70637512207031, 47.273216247558594, 48.53135299682617, 46.375396728515625, 47.71205520629883, 45.801910400390625, 47.24893569946289, 47.21046829223633, 45.48845672607422, 47.926780700683594, 47.11882400512695, 46.3369255065918, 47.760345458984375, 46.75334548950195, 46.68667984008789, 47.1961669921875, 46.4122428894043, 47.44076156616211, 46.78754425048828, 47.75526428222656, 46.05363082885742, 46.2259407043457, 48.59139633178711, 46.528377532958984, 47.71236038208008]})
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="Orthogonal", optimizer="AdamW")
data.update({'train_loss': [7.6756, 7.1962, 7.1528, 7.1074, 7.0509, 6.9843, 6.9202, 6.8416, 6.7555, 6.6528, 6.5251, 6.403, 6.2626, 6.0912, 5.9327, 5.7865, 5.651, 5.5381, 5.4341, 5.3487, 5.2595, 5.1876, 5.1142, 5.0417, 4.9792, 4.9093, 4.8599, 4.8111, 4.7613, 4.7116, 4.6761, 4.6226, 4.5832, 4.5423, 4.5001, 4.461, 4.4257, 4.3862, 4.3546, 4.3224, 4.2961, 4.253, 4.2245, 4.1924, 4.1661, 4.1266, 4.1072, 4.0777, 4.0549, 4.0249, 3.9978, 3.9756, 3.9501, 3.9216, 3.9053, 3.8869, 3.8531, 3.8421, 3.8089, 3.7926, 3.7658, 3.7566, 3.73, 3.7156, 3.6904, 3.6726, 3.6486, 3.6356, 3.6358, 3.6132, 3.5808, 3.5783, 3.5612, 3.5408, 3.5272, 3.5222, 3.4944, 3.4873, 3.4645, 3.4638, 3.4516, 3.4263, 3.4119, 3.3955, 3.3822, 3.3695, 3.3532, 3.3546, 3.3411, 3.3212, 3.3198, 3.3002, 3.2901, 3.2818, 3.2633, 3.2501, 3.2544, 3.2411, 3.2272, 3.2171, 3.1986, 3.1956, 3.185, 3.1782, 3.1724, 3.1632, 3.1514, 3.1448, 3.1325, 3.1249, 3.1163, 3.1035, 3.0952, 3.0947, 3.0682, 3.0718, 3.0518, 3.0559, 3.0446, 3.0271, 3.0326, 3.0163, 3.0112, 3.0118, 3.0094, 2.994, 2.977, 2.9811, 2.9784, 2.9587, 2.9523, 2.9425, 2.9345, 2.929, 2.9286, 2.9231, 2.9154, 2.9026, 2.8993, 2.8966, 2.8838, 2.8779, 2.8716, 2.8705, 2.8594, 2.8558, 2.8638, 2.8449, 2.8442, 2.8347, 2.8211, 2.8176, 2.8123, 2.809, 2.792, 2.799, 2.7908, 2.7967, 2.7776, 2.7805, 2.7582, 2.7668, 2.7599, 2.7604, 2.7536, 2.7366, 2.7393, 2.7316, 2.7361, 2.7166, 2.7216, 2.713, 2.7045, 2.7035, 2.7035, 2.6947, 2.6847, 2.6848, 2.6782, 2.6737, 2.6633, 2.6556, 2.6632, 2.6609, 2.6513, 2.6498, 2.6425, 2.6417, 2.6332, 2.6356, 2.6388, 2.6238, 2.6251, 2.6171, 2.6094, 2.6112, 2.6097, 2.5985, 2.5979, 2.5913], 'val_metric': [0.06344155967235565, 0.06408585608005524, 0.0646415501832962, 0.06432321667671204, 0.0687977746129036, 0.0752670094370842, 0.08878202736377716, 0.10182289779186249, 0.11355166882276535, 0.12161743640899658, 0.14158524572849274, 0.16282996535301208, 0.18312838673591614, 0.19942694902420044, 0.21037600934505463, 0.22456081211566925, 0.2263849526643753, 0.24410288035869598, 0.2515840232372284, 0.2571013271808624, 0.2628520727157593, 0.271510511636734, 0.27169275283813477, 0.27878618240356445, 0.27419671416282654, 0.2894120216369629, 0.28984853625297546, 0.2929695248603821, 0.3006311357021332, 0.300814151763916, 0.30247512459754944, 0.3059636950492859, 0.3101067841053009, 0.31193941831588745, 0.3142729699611664, 0.313160240650177, 0.31758612394332886, 0.3262661397457123, 0.32353106141090393, 0.3274969458580017, 0.32737061381340027, 0.3296448290348053, 0.33342695236206055, 0.33616507053375244, 0.3410285711288452, 0.3441261351108551, 0.3439638614654541, 0.34414392709732056, 0.34915226697921753, 0.35304975509643555, 0.34978705644607544, 0.3552088737487793, 0.3556794822216034, 0.3566305637359619, 0.36022040247917175, 0.36216261982917786, 0.3618588149547577, 0.3657405972480774, 0.3623115122318268, 0.3622172474861145, 0.36790573596954346, 0.3691291809082031, 0.36697736382484436, 0.369583398103714, 0.3724786043167114, 0.3748359978199005, 0.37469300627708435, 0.3744247257709503, 0.3756646513938904, 0.3783867061138153, 0.3764188885688782, 0.3791736364364624, 0.37971335649490356, 0.380490779876709, 0.38092315196990967, 0.3803011476993561, 0.380508154630661, 0.3906124234199524, 0.38753440976142883, 0.38248899579048157, 0.3826479911804199, 0.39196768403053284, 0.3893533945083618, 0.3907753825187683, 0.39160820841789246, 0.3912007212638855, 0.3941335082054138, 0.39183834195137024, 0.39479362964630127, 0.3936939537525177, 0.3969905376434326, 0.39458179473876953, 0.3954402208328247, 0.39980992674827576, 0.3958745002746582, 0.3978850543498993, 0.3996560871601105, 0.3983665108680725, 0.3982430398464203, 0.39688488841056824, 0.4050239324569702, 0.3987216353416443, 0.404767245054245, 0.39577317237854004, 0.3993665277957916, 0.40229371190071106, 0.40289562940597534, 0.40222007036209106, 0.4045927822589874, 0.40525537729263306, 0.4006669223308563, 0.40124043822288513, 0.4005012810230255, 0.40233269333839417, 0.4035615026950836, 0.40555399656295776, 0.40255555510520935, 0.4073461890220642, 0.4078173339366913, 0.408713161945343, 0.40886008739471436, 0.4067339301109314, 0.40754029154777527, 0.4097646176815033, 0.40831810235977173, 0.4117451012134552, 0.40965381264686584, 0.41153836250305176, 0.4088096022605896, 0.4142785668373108, 0.4131864309310913, 0.41158485412597656, 0.4096422791481018, 0.4077548384666443, 0.41446924209594727, 0.4103972613811493, 0.4102886915206909, 0.4106948673725128, 0.40943363308906555, 0.41433247923851013, 0.4117729663848877, 0.41849851608276367, 0.41586175560951233, 0.40984970331192017, 0.4149104058742523, 0.4110601246356964, 0.41583052277565, 0.41674524545669556, 0.417275607585907, 0.4173523783683777, 0.4167976677417755, 0.4200083911418915, 0.4164395034313202, 0.4209129512310028, 0.4140010178089142, 0.4174608886241913, 0.413592666387558, 0.4162440896034241, 0.4161473512649536, 0.4192299246788025, 0.4134341776371002, 0.42011502385139465, 0.41924726963043213, 0.41820940375328064, 0.4178975522518158, 0.4192744195461273, 0.41885682940483093, 0.41791337728500366, 0.4174930453300476, 0.4183884859085083, 0.4214957058429718, 0.4174765348434448, 0.4218280017375946, 0.4194955825805664, 0.4156534969806671, 0.4150189757347107, 0.4192587733268738, 0.41611987352371216, 0.42213207483291626, 0.4185636639595032, 0.4232499599456787, 0.4204695522785187, 0.41871383786201477, 0.4234713912010193, 0.4182153642177582, 0.42051658034324646, 0.42602384090423584, 0.4178009033203125, 0.4180675148963928, 0.42307260632514954, 0.4182436466217041, 0.42316916584968567, 0.41608089208602905, 0.41763073205947876, 0.418410986661911, 0.4227948486804962, 0.41978445649147034, 0.4170343577861786, 0.4204593002796173, 0.4195008873939514], 'val_perplexity': [1387.418212890625, 1319.5791015625, 1303.8822021484375, 1242.369140625, 1189.413818359375, 1130.209228515625, 1037.975830078125, 947.8525390625, 871.9302978515625, 767.5014038085938, 661.6976318359375, 568.3268432617188, 478.8316345214844, 401.3408508300781, 345.0378723144531, 298.9374694824219, 280.41522216796875, 249.945068359375, 226.43548583984375, 210.46441650390625, 192.28182983398438, 181.05650329589844, 173.38717651367188, 164.91632080078125, 165.8617401123047, 147.215087890625, 145.87240600585938, 138.30264282226562, 133.9222869873047, 127.60188293457031, 126.23487854003906, 121.41725158691406, 118.05072021484375, 116.59563446044922, 108.26353454589844, 109.40127563476562, 106.53141021728516, 99.62980651855469, 101.94097137451172, 97.28137969970703, 97.21826934814453, 95.50265502929688, 91.69408416748047, 91.8481674194336, 86.66482543945312, 84.32642364501953, 84.98590087890625, 82.9999008178711, 79.30933380126953, 78.31489562988281, 79.03740692138672, 77.31898498535156, 77.31997680664062, 74.61796569824219, 73.27059936523438, 74.14036560058594, 73.17105865478516, 70.29269409179688, 70.54103088378906, 70.4586410522461, 67.85198211669922, 68.57085418701172, 66.02503204345703, 65.46681213378906, 64.04475402832031, 63.92692184448242, 63.52360153198242, 64.37042236328125, 63.51893997192383, 62.59314727783203, 62.19782638549805, 61.45001983642578, 59.872623443603516, 60.591007232666016, 59.91072082519531, 60.17621612548828, 59.50021743774414, 56.50364303588867, 57.26622009277344, 60.30326461791992, 59.27553939819336, 56.27846908569336, 57.140010833740234, 55.89658737182617, 55.71632766723633, 54.911746978759766, 53.70097351074219, 54.740692138671875, 53.921051025390625, 55.61670684814453, 53.031089782714844, 53.06102752685547, 54.246273040771484, 51.748558044433594, 52.74428939819336, 51.41138458251953, 51.79766082763672, 52.24032211303711, 51.1381950378418, 52.17473602294922, 50.89286804199219, 50.53102111816406, 49.72957229614258, 52.69950485229492, 52.04035568237305, 49.63850402832031, 49.354881286621094, 50.23545837402344, 49.189640045166016, 51.540802001953125, 50.84037780761719, 49.65644836425781, 50.891448974609375, 49.370460510253906, 49.77394485473633, 49.27475357055664, 50.2654914855957, 49.89501953125, 47.67938232421875, 48.28877639770508, 48.07412338256836, 47.48981857299805, 47.861568450927734, 49.2553825378418, 47.74372100830078, 46.84900665283203, 48.43429946899414, 47.507564544677734, 47.32917785644531, 47.29380416870117, 47.32170867919922, 46.93976974487305, 47.54829025268555, 47.21903610229492, 46.26200485229492, 46.60710525512695, 46.08461380004883, 47.85588455200195, 48.0985107421875, 46.329986572265625, 46.299312591552734, 44.95168685913086, 45.67686080932617, 46.74675750732422, 47.00458908081055, 46.5553092956543, 46.38447570800781, 45.658233642578125, 45.32277297973633, 46.62046432495117, 46.174503326416016, 44.9158821105957, 45.898380279541016, 45.57687759399414, 46.28891372680664, 47.46272277832031, 46.53364562988281, 45.27859878540039, 45.87563705444336, 44.4642448425293, 46.894901275634766, 45.430213928222656, 45.32988357543945, 45.59993362426758, 45.74625015258789, 45.4893913269043, 46.04889678955078, 45.93185806274414, 45.09843063354492, 45.85087966918945, 43.920223236083984, 44.78440856933594, 43.28975296020508, 44.486087799072266, 45.923580169677734, 46.60901641845703, 45.178184509277344, 46.23617172241211, 45.43391799926758, 45.97861862182617, 44.36384963989258, 45.14816665649414, 45.20991897583008, 44.27210235595703, 46.10850524902344, 45.789955139160156, 44.43080139160156, 46.74751663208008, 45.405372619628906, 45.63970947265625, 46.18629455566406, 45.0794563293457, 45.87016677856445, 45.17264938354492, 46.105560302734375, 44.62294006347656, 45.0794792175293, 46.527313232421875, 45.07206344604492, 45.69915771484375]})
data = database.get_data(model="BERT-mini", dataset="WikiText", initialization="LSUV", optimizer="AdamW")
data.update({'train_loss': [7.4276, 6.8734, 6.7751, 6.7152, 6.6647, 6.628, 6.5947, 6.5587, 6.5283, 6.4927, 6.4633, 6.4438, 6.4315, 6.3975, 6.3765, 6.3572, 6.3434, 6.3187, 6.2994, 6.2922, 6.2709, 6.2624, 6.2416, 6.2221, 6.2076, 6.1814, 6.1749, 6.1621, 6.1555, 6.1367, 6.1282, 6.1099, 6.0999, 6.0908, 6.072, 6.0572, 6.0527, 6.0386, 6.0262, 6.0142, 6.0057, 5.9946, 5.984, 5.9756, 5.9642, 5.948, 5.9485, 5.9307, 5.934, 5.919, 5.8995, 5.9022, 5.8855, 5.8742, 5.8762, 5.8687, 5.853, 5.8514, 5.8367, 5.8288, 5.8161, 5.8219, 5.8103, 5.8067, 5.7843, 5.7828, 5.7704, 5.7738, 5.7758, 5.7699, 5.7551, 5.7544, 5.7494, 5.7324, 5.731, 5.7263, 5.7195, 5.7164, 5.7045, 5.7123, 5.7011, 5.6911, 5.6854, 5.6803, 5.6695, 5.6709, 5.6588, 5.6602, 5.656, 5.6419, 5.6497, 5.639, 5.6275, 5.6331, 5.6181, 5.6169, 5.6205, 5.6194, 5.6079, 5.6016, 5.5929, 5.5903, 5.5889, 5.5878, 5.5848, 5.5811, 5.5738, 5.5668, 5.5692, 5.5625, 5.5583, 5.5506, 5.5527, 5.544, 5.5318, 5.5331, 5.5206, 5.5309, 5.5214, 5.5134, 5.5213, 5.5134, 5.5095, 5.5087, 5.5031, 5.4971, 5.4906, 5.4923, 5.4906, 5.4745, 5.4741, 5.4715, 5.4656, 5.4717, 5.4668, 5.4576, 5.4522, 5.4526, 5.4508, 5.446, 5.4345, 5.4422, 5.4285, 5.433, 5.4299, 5.4339, 5.4343, 5.4168, 5.4218, 5.418, 5.4053, 5.4084, 5.4031, 5.3995, 5.3881, 5.4028, 5.3928, 5.3974, 5.3891, 5.3907, 5.3734, 5.3862, 5.3787, 5.3724, 5.3755, 5.3652, 5.363, 5.3559, 5.3666, 5.3548, 5.3549, 5.3463, 5.3383, 5.3403, 5.3369, 5.3267, 5.319, 5.3164, 5.3053, 5.303, 5.2994, 5.2914, 5.2971, 5.2808, 5.2752, 5.2695, 5.2681, 5.2685, 5.2512, 5.2522, 5.2548, 5.2409, 5.2417, 5.2278, 5.2208, 5.2155, 5.2126, 5.1962, 5.2, 5.1961], 'val_metric': [0.1105792224407196, 0.12359613180160522, 0.1365276426076889, 0.14212314784526825, 0.14279361069202423, 0.14111512899398804, 0.14544841647148132, 0.14800101518630981, 0.14517222344875336, 0.1465378850698471, 0.1476047784090042, 0.14988461136817932, 0.15077842772006989, 0.15130187571048737, 0.15146848559379578, 0.15339282155036926, 0.15266063809394836, 0.15246963500976562, 0.15326255559921265, 0.1525106281042099, 0.1538076549768448, 0.15242941677570343, 0.15273477137088776, 0.15321719646453857, 0.15167081356048584, 0.15474729239940643, 0.15308822691440582, 0.1536962240934372, 0.15169794857501984, 0.15176866948604584, 0.15377366542816162, 0.15518443286418915, 0.15411704778671265, 0.15418145060539246, 0.15329097211360931, 0.15523184835910797, 0.15301036834716797, 0.15523795783519745, 0.1551947295665741, 0.15710945427417755, 0.15336735546588898, 0.1541062444448471, 0.1498497575521469, 0.15229764580726624, 0.15587016940116882, 0.1535591036081314, 0.15460562705993652, 0.15708282589912415, 0.1577761471271515, 0.15725591778755188, 0.15533241629600525, 0.1549900323152542, 0.15532076358795166, 0.15812121331691742, 0.15478761494159698, 0.15515193343162537, 0.1537441462278366, 0.1545441895723343, 0.15631932020187378, 0.1539793312549591, 0.1546354740858078, 0.15546369552612305, 0.15436016023159027, 0.1559031903743744, 0.15429933369159698, 0.15504871308803558, 0.15477785468101501, 0.15369290113449097, 0.1525108963251114, 0.15435263514518738, 0.1499873250722885, 0.15424343943595886, 0.15555864572525024, 0.15286482870578766, 0.15429480373859406, 0.15620942413806915, 0.14931227266788483, 0.15903346240520477, 0.15333408117294312, 0.1560738980770111, 0.14948737621307373, 0.1532420963048935, 0.15281493961811066, 0.1547689586877823, 0.1523435264825821, 0.15386125445365906, 0.15832209587097168, 0.15471740067005157, 0.15291282534599304, 0.15325112640857697, 0.1555909812450409, 0.1528768390417099, 0.15033863484859467, 0.1507672518491745, 0.15363416075706482, 0.15508681535720825, 0.15112902224063873, 0.15186485648155212, 0.15302962064743042, 0.1498839557170868, 0.15483491122722626, 0.15314780175685883, 0.15471115708351135, 0.14802253246307373, 0.14976951479911804, 0.15221698582172394, 0.1508583277463913, 0.1524628847837448, 0.15002386271953583, 0.1485397070646286, 0.14977674186229706, 0.14910319447517395, 0.15128985047340393, 0.14932571351528168, 0.1535336673259735, 0.15279102325439453, 0.14798106253147125, 0.1499495655298233, 0.14859628677368164, 0.15350177884101868, 0.14928597211837769, 0.14963042736053467, 0.1500210165977478, 0.1489439457654953, 0.15042854845523834, 0.14981387555599213, 0.1468336582183838, 0.14892786741256714, 0.1462782323360443, 0.1497134268283844, 0.15080659091472626, 0.14998596906661987, 0.14936253428459167, 0.14652341604232788, 0.1489003598690033, 0.1461384892463684, 0.14895938336849213, 0.1493486613035202, 0.1444370001554489, 0.1473263055086136, 0.14825735986232758, 0.15023382008075714, 0.14616715908050537, 0.14658088982105255, 0.14625830948352814, 0.14065822958946228, 0.1493334025144577, 0.14668035507202148, 0.14991353452205658, 0.14791491627693176, 0.1418987214565277, 0.1476983278989792, 0.14642558991909027, 0.14635713398456573, 0.14587171375751495, 0.1435716450214386, 0.1470489650964737, 0.14677947759628296, 0.14703427255153656, 0.14713585376739502, 0.146480992436409, 0.15157806873321533, 0.1446346938610077, 0.1468663066625595, 0.1445850282907486, 0.14186860620975494, 0.14693191647529602, 0.14499035477638245, 0.14437325298786163, 0.14905649423599243, 0.14870230853557587, 0.1493971198797226, 0.1509179323911667, 0.14789168536663055, 0.1500900536775589, 0.1502179652452469, 0.15268762409687042, 0.15363655984401703, 0.1559317260980606, 0.15560154616832733, 0.15987145900726318, 0.16133838891983032, 0.15829630196094513, 0.1571146845817566, 0.160467267036438, 0.1609644889831543, 0.16291628777980804, 0.15866141021251678, 0.16147953271865845, 0.16165412962436676, 0.16156183183193207, 0.16250209510326385, 0.16352535784244537, 0.1622174233198166, 0.16424712538719177, 0.17142139375209808, 0.1676066517829895, 0.16347388923168182, 0.16917955875396729, 0.17097504436969757], 'val_perplexity': [987.544189453125, 877.4257202148438, 837.2774047851562, 780.6781005859375, 781.5015869140625, 762.5421142578125, 728.1361083984375, 694.70849609375, 694.8101806640625, 679.1522827148438, 681.10986328125, 664.985595703125, 646.121337890625, 624.01708984375, 626.510498046875, 619.063232421875, 620.7401123046875, 616.8567504882812, 600.5736694335938, 594.4021606445312, 590.22802734375, 582.7410888671875, 584.1004028320312, 576.7805786132812, 595.7703857421875, 561.97998046875, 569.4813842773438, 561.261962890625, 563.972412109375, 547.7750854492188, 548.83056640625, 542.2169799804688, 544.4235229492188, 547.3505859375, 529.472412109375, 552.3899536132812, 549.9308471679688, 516.0757446289062, 536.4797973632812, 525.071044921875, 534.3070068359375, 529.9134521484375, 531.1097412109375, 528.0484008789062, 515.9610595703125, 514.5411987304688, 523.146240234375, 521.6227416992188, 494.57257080078125, 499.5484619140625, 516.9998779296875, 502.8709411621094, 513.1815185546875, 488.53472900390625, 503.0244445800781, 516.9359741210938, 516.1261596679688, 507.568603515625, 506.0367126464844, 509.0332946777344, 501.8883056640625, 505.8408203125, 495.0524597167969, 501.655029296875, 491.1011047363281, 491.5159912109375, 490.49542236328125, 497.1981506347656, 503.1750793457031, 509.3185729980469, 506.87689208984375, 496.9891052246094, 493.42633056640625, 495.89166259765625, 494.66290283203125, 490.0884094238281, 508.2794494628906, 482.5574645996094, 494.8502197265625, 509.0709228515625, 512.9024047851562, 490.21929931640625, 507.98724365234375, 500.823974609375, 497.94293212890625, 487.8349609375, 479.0965576171875, 504.8044738769531, 488.9507141113281, 502.66455078125, 495.48345947265625, 500.27264404296875, 515.8099975585938, 505.02838134765625, 499.0801086425781, 492.8105163574219, 507.3382568359375, 498.3873596191406, 497.1884460449219, 516.2322387695312, 500.7609558105469, 498.9052429199219, 491.4874267578125, 520.0968017578125, 514.59619140625, 498.99517822265625, 504.3283386230469, 508.74017333984375, 497.0858154296875, 516.6644287109375, 521.6695556640625, 515.4650268554688, 525.98046875, 510.61614990234375, 503.0040588378906, 513.0594482421875, 515.9059448242188, 521.4317626953125, 512.7781982421875, 517.655517578125, 509.093505859375, 506.2582702636719, 512.7901611328125, 520.6079711914062, 516.4740600585938, 513.4728393554688, 524.9108276367188, 518.2033081054688, 523.5050659179688, 520.213623046875, 527.5307006835938, 526.0589599609375, 524.8267211914062, 538.7454833984375, 511.0163269042969, 533.5569458007812, 523.272216796875, 529.0706787109375, 554.1546630859375, 541.4686889648438, 526.344482421875, 516.5572509765625, 521.5439453125, 542.0939331054688, 536.4434814453125, 545.6390380859375, 528.8257446289062, 541.5774536132812, 523.52001953125, 540.67919921875, 553.9221801757812, 548.9208984375, 534.7285766601562, 542.8406982421875, 554.0236206054688, 568.5125122070312, 542.4315795898438, 544.325927734375, 545.8018798828125, 535.468994140625, 559.6683959960938, 539.35205078125, 557.8379516601562, 568.9431762695312, 573.397216796875, 559.0045166015625, 563.3832397460938, 561.6734619140625, 560.7101440429688, 542.349853515625, 535.4721069335938, 545.0333862304688, 538.6072998046875, 565.2442016601562, 562.4729614257812, 559.5982055664062, 542.4398803710938, 558.2693481445312, 536.7418212890625, 529.8275146484375, 525.8565673828125, 528.2216796875, 523.3593139648438, 521.8626098632812, 533.8630981445312, 532.432861328125, 517.895263671875, 516.4176635742188, 513.2637939453125, 519.7705688476562, 526.4501342773438, 506.6903381347656, 510.5796203613281, 511.0799255371094, 517.5089111328125, 500.14599609375, 491.3091125488281, 497.06707763671875, 493.9067687988281, 484.0146179199219]})



In [5]:
# Merge
database = DataBase()
database.database = merge_dicts([database1.database, database2.database])

In [6]:
model="ResNet-50"
dataset="CIFAR-100"
plot_loss(model=model, dataset=dataset, show=False)
plot_acc(model=model, dataset=dataset, show=False)

In [7]:
model="MobileNetV3"
dataset="Tiny-Imagenet"
plot_loss(model=model, dataset=dataset, show=False)
plot_acc(model=model, dataset=dataset, show=False)

In [8]:
model="EfficientNetV2"
dataset="Tiny-Imagenet"
plot_loss(model=model, dataset=dataset, show=False)
plot_acc(model=model, dataset=dataset, show=False)

In [7]:
model="ViT B16"
dataset="ImageNet-1K"
plot_loss(model=model, dataset=dataset, show=False)
plot_acc(model=model, dataset=dataset, show=False)

Adam fail
AdamW fail
Adam fail
AdamW fail


In [10]:
model="BERT-mini"
dataset="WikiText"
plot_loss(model=model, dataset=dataset, show=False)
plot_acc(model=model, dataset=dataset, show=False)

SGD fail
Adam fail
SGD fail
Adam fail


In [11]:
plot_legend()

<Figure size 400x200 with 0 Axes>

In [6]:
for model, dataset, optimizer in [["ResNet-50", "CIFAR-100", "AdamW"], 
                                  ["MobileNetV3", "Tiny-Imagenet", "AdamW"], 
                                  ["EfficientNetV2", "Tiny-Imagenet", "AdamW"], 
                                  ["ViT B16","ImageNet-1K", "SGD"], 
                                  ["BERT-mini", "WikiText", "AdamW"]]:
    plot_acc_4inits(model, dataset, optimizer)
plot_4inits_legend()
plot_4inits_legend_horizontal()

<Figure size 400x200 with 0 Axes>

<Figure size 400x200 with 0 Axes>

In [17]:
# Acc at 1, 10 and final
for model, dataset in [["ResNet-50", "CIFAR-100"], ["MobileNetV3", "Tiny-Imagenet"], ["EfficientNetV2", "Tiny-Imagenet"], ["ViT B16","ImageNet-1K"], ["BERT-mini", "WikiText"]]:
    print(f"{model}; {dataset};")
    early_stopping_patience = 10
    for i, optimizer in enumerate(["SGD", "Adam", "AdamW"]):    
        #print(f"{model}; {dataset}; {optimizer};")
        one_array = []
        ten_array = []
        best_array = []
        epochs_array = []
        default_epochs = []
        auc_array = []
        for j, initialization in enumerate(["Default initialization", "Orthogonal", "LSUV", "Sinusoidal"]):
            try:
                data = np.array(database.get_data(model, dataset, initialization, optimizer)["val_metric"])
                if len(data > 200):
                    data = data[:200]
                best = np.max(data)
                if best < 1:
                    data = data * 100
                    best = best * 100
                if initialization == "Default initialization":
                    default_best = best - 1
                one = data[0]
                ten = np.max(data[:10])

                best = 0
                epoch = 0
                epochs_without_improvement = 0
                for i, acc in enumerate(data):
                    if acc > best:
                        best = acc
                        epoch = i + 1
                        epochs_without_improvement = 0
                    else:
                        epochs_without_improvement += 1
                        if epochs_without_improvement >= early_stopping_patience:
                            break

                if initialization == "Default initialization":
                    default_best = best 
                    
                mask = data >= default_best
                if not any(mask):
                    default_epoch = 300
                else:
                    default_epoch = int(mask.argmax()) + 1
                
                one_array.append(one)
                ten_array.append(ten)
                best_array.append(best)
                epochs_array.append(epoch)
                default_epochs.append(default_epoch) 
                auc_array.append(np.trapezoid(data) / 100)
                
                #print(f"{initialization: <23}; {one:6.1f}; {ten:6.1f}; {best:6.1f}; {epochs:>3}")
            except:
                one_array.append(0)
                ten_array.append(0)
                best_array.append(0)
                epochs_array.append(0)
                auc_array.append(0)
                default_epochs.append(300)
        print(f"& {optimizer:<6}", end=" & ")
        for one in one_array:
            if one == 0:
                print(f"\\NA", end=" & ")
            elif one != max(one_array):
                print(f"{one:4.1f}", end=" & ")
            else:
                print(f"\\bfseries {one:4.1f}", end=" & ")
        for ten in ten_array:
            if ten == 0:
                print(f"\\NA", end=" & ")
            elif ten != max(ten_array):
                print(f"{ten:4.1f}", end=" & ")
            else:
                print(f"\\bfseries {ten:4.1f}", end=" & ")
        for best in best_array:
            if best == 0:
                print(f"\\NA", end=" & ")
            elif best != max(best_array):
                print(f"{best:4.1f}", end=" & ")
            else:
                print(f"\\bfseries {best:4.1f}", end=" & ")
        for i, auc in enumerate(auc_array):
            if i == 3:
                end = " \\\\ "
            else:
                end = " & "
            if auc == 0:
                print(f"\\NA", end=end)
            elif auc != max(auc_array):
                print(f"{auc:4.1f}", end=end)
            else:
                print(f"\\bfseries {auc:4.1f}", end=end)
        print()

ResNet-50; CIFAR-100;
& SGD    &  2.6 &  4.0 &  3.6 & \bfseries  5.0 &  9.7 & 18.2 & 14.8 & \bfseries 24.3 & 37.3 & 46.5 & 44.8 & \bfseries 51.9 & 24.7 & 35.1 & 30.9 & \bfseries 41.8 \\ 
& Adam   & 10.1 & 12.5 & 11.8 & \bfseries 19.9 & 39.9 & 44.1 & 43.1 & \bfseries 48.7 & 53.1 & 56.6 & 56.3 & \bfseries 61.5 & 47.9 & 51.2 & 50.9 & \bfseries 56.5 \\ 
& AdamW  & 12.9 & 13.9 & 12.6 & \bfseries 23.1 & 58.5 & 60.9 & 62.5 & \bfseries 63.7 & 67.5 & 67.7 & 69.7 & \bfseries 71.0 & 63.6 & 64.6 & 65.5 & \bfseries 67.9 \\ 
MobileNetV3; Tiny-Imagenet;
& SGD    &  0.7 &  0.8 & \bfseries  1.4 &  1.2 &  1.5 &  3.0 &  2.9 & \bfseries  4.5 & 18.4 & 25.8 & \bfseries 28.0 & 21.6 & 26.0 & \bfseries 38.1 & 34.8 & 36.0 \\ 
& Adam   &  5.0 & \bfseries  8.4 &  5.4 &  5.5 & 22.0 & 24.8 & \bfseries 26.7 & 24.0 & 32.8 & 34.4 & \bfseries 35.2 & 34.8 & 61.9 & 66.1 & \bfseries 71.3 & 64.7 \\ 
& AdamW  & 13.4 & 14.5 & \bfseries 14.5 & 12.3 & 40.9 & \bfseries 43.2 & 40.1 & 42.1 & 40.9 & \bfseries 43.6 & 40.1 & 42.6 & 

In [ ]:
def auc(training):
    auc = np.trapezoid(training)
    return auc

# Acc at 1, 10 and final
all_best = [[],[],[],[]]
all_epochs = [[],[],[],[]]
all_auc = [[],[],[],[]]
for model, dataset in [["ResNet-50", "CIFAR-100"], ["MobileNetV3", "Tiny-Imagenet"], ["EfficientNetV2", "Tiny-Imagenet"], ["BERT-mini", "WikiText"], ["ViT B16","ImageNet-1K"]]:
# for model, dataset in [["ResNet-50", "CIFAR-100"], ["EfficientNetV2", "Tiny-Imagenet"], ["BERT-mini", "WikiText"], ["ViT B16","ImageNet-1K"]]:

    print(f"{model}; {dataset};")
    early_stopping_patience = 10
    for i, optimizer in enumerate(["SGD", "Adam", "AdamW"]):    
        #print(f"{model}; {dataset}; {optimizer};")
        one_array = []
        ten_array = []
        best_array = []
        epochs_array = []
        default_epochs = []
        for j, initialization in enumerate(["Default initialization", "Orthogonal", "LSUV", "Sinusoidal"]):
            try:
                data = np.array(database.get_data(model, dataset, initialization, optimizer)["val_metric"])
                if len(data > 200):
                    data = data[:200]
                best = np.max(data)
                if best < 1:
                    data = data * 100
                    best = best * 100
                if initialization == "Default initialization":
                    default_best = best - 1
                one = data[0]
                ten = np.max(data[:10])

                best = 0
                epoch = 0
                epochs_without_improvement = 0
                for i, acc in enumerate(data):
                    if acc > best:
                        best = acc
                        epoch = i + 1
                        epochs_without_improvement = 0
                    else:
                        epochs_without_improvement += 1
                        if epochs_without_improvement >= early_stopping_patience:
                            break

                if initialization == "Default initialization":
                    default_best = best 
                    
                mask = data >= default_best - 0.5
                if not any(mask):
                    default_epoch = len(data)
                else:
                    default_epoch = int(mask.argmax()) + 1
                
                all_best[j].append(best)
                all_epochs[j].append(default_epoch)
                all_auc[j].append(auc(data))
                
                #print(f"{initialization: <23}; {one:6.1f}; {ten:6.1f}; {best:6.1f}; {epochs:>3}")
            except:
                pass

default = np.array(all_best[0])
orthogonal = np.array(all_best[1])
lsuv = np.array(all_best[2])
sinusoidal = np.array(all_best[3])

diff = sinusoidal - default
print(f"Default    improvement: {np.mean(diff):4.1f}% +- {np.std(diff):4.1f}%")
diff = sinusoidal - orthogonal
print(f"Orthogonal improvement: {np.mean(diff):4.1f}% +- {np.std(diff):4.1f}%")
diff = sinusoidal - lsuv
print(f"LSUV       improvement: {np.mean(diff):4.1f}% +- {np.std(diff):4.1f}%")

default = np.array(all_auc[0])
orthogonal = np.array(all_auc[1])
lsuv = np.array(all_auc[2])
sinusoidal = np.array(all_auc[3])

diff = (sinusoidal - default) / default
print(f"Default    improvement: {np.mean(diff)*100:4.1f}% +- {np.std(diff)*100:4.1f}%")
diff = (sinusoidal - orthogonal) / orthogonal
print(f"Orthogonal improvement: {np.mean(diff)*100:4.1f}% +- {np.std(diff)*100:4.1f}%")
diff = (sinusoidal - lsuv) / lsuv
print(f"LSUV       improvement: {np.mean(diff)*100:4.1f}% +- {np.std(diff)*100:4.1f}%")




ResNet-50; CIFAR-100;
MobileNetV3; Tiny-Imagenet;
EfficientNetV2; Tiny-Imagenet;
BERT-mini; WikiText;
ViT B16; ImageNet-1K;
Default    improvement:  4.9% +-  3.8%
Orthogonal improvement:  1.7% +-  3.1%
LSUV       improvement:  3.4% +-  6.1%
Default    improvement: 20.9% +- 18.8%
Orthogonal improvement:  5.7% +-  8.8%
LSUV       improvement: 18.0% +- 35.0%
